In [ ]:
################################
#
# アルゴリズム入門のまとめ課題
#
# BoardFaerie
#
################################
#
# 概要
#
# opencvなどの画像処理系のライブラリを使わずに、numpyと画像表示用のita(もしくはmatplotlib、画像保存用のPIL.Image)(とデバッグ用のwarningsとtime)ライブラリのみでレンダリングパイプラインとシェーダーを作りました。なおOpenGLやDirect3Dは触ったことがなかったので、かなり標準から外れた部分もあるかと思いますがご了承ください。
# また、フォントは M+ FONTS PROJECT による M+ BITMAP FONTS を使用させていただきます。このフォントは自由なライセンスで公開されてます。詳細はこちら: https://mplus-fonts.osdn.jp/mplus-bitmap-fonts/index.html 。このフォントを編集してノートブックに埋め込みました。このため、このノートブックを開くか走らせる際は、かなり重くなると思われるので注意してください。
# 最後に、作ったライブラリを使い、デフォルメした地球をアニメーションしてみました。最終アニメーションはtest.mp4およびtest2.gifです。
# 標準設定の場合、かなり高性能なデスクトップパソコンで開くのに3秒ほど、実行ボタンを押してから実行されるまで5秒ほど、そしてレンダリング(実行そのもの)に3時間程度かかります。
#
# 参考にしたウェブサイト(主なものだけ)
#
# [Gabriel Gambetta Computer Graphics from Scratch](https://www.gabrielgambetta.com/computer-graphics-from-scratch/introduction.html)
# [Scratchapixel](https://www.scratchapixel.com/index.php)
#
# (これら以外にもQiitaの記事やStack Overflowの質問と答えなどを多数参考にしています。一応参考にしたウェブサイトはコメントに載せてありますが、多分入れ忘れたものもあります。)
#
################################
#
# 目次
#
# (準備部)
# Imports
# Settings
# Util
# Quaternion
# (ラスタライズ部)
# Layer
# Line2D
# Rectangle2D
# Circle2D
# Triangle2D
# Curve2D
# Polygon2D
# Bezier2D
# Text2D
# Color
# Canvas
# (ピクセルシェーダー部)
# AntiAliasing
# (ジオメトリーシェーダー部)
# ShaderGeometry
# (バーテックスシェーダー部)
# ShaderVertex
# (レンダリングパイプライン部)
# RayTracing
# GraphicsPipeline
# (3D部)
# Material
# WorldObject
# Light
# World
# (メイン部)
# Main
# Animation


# プログラム開始

In [ ]:
#!pip install ita

# Imports

In [ ]:
# https://qiita.com/sotasato/items/cc36a532ba6487dd3dba

import numpy as np
import ita
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from PIL import Image
import warnings
import time

%matplotlib inline
plt.rcParams["animation.html"] = "jshtml"


# Settings

In [ ]:
#left-handed system!!!(Same as Unity)
class Settings:
    USE_ITA = False

    SHOW_HEIGHT = 360 #360
    SHOW_WIDTH = 640 #640
    SSAA_RATIO = 4 # 4
    FPS = 15 # 15

    CAMERA_NEARCLIP = 1
    CAMERA_FARCLIP = 100000
    LIGHT_NEARCLIP = 1
    LIGHT_FARCLIP = 100000

    EPSILON = 0.0005

    FOCAL = SHOW_HEIGHT * SSAA_RATIO
    HEIGHT = SHOW_HEIGHT * SSAA_RATIO
    WIDTH = SHOW_WIDTH * SSAA_RATIO
    C_U = WIDTH // 2
    C_V = HEIGHT // 2


# Util

In [ ]:
class Util:
    """Utility class for application"""

    @staticmethod
    def ipart(x):
        """integer part of x"""
        return np.floor(x)

    @staticmethod
    def fpart(x):
        """fraction part of x"""
        return x - np.floor(x)

    @staticmethod
    def rfpart(x):
        """1 minus fraction part of x"""
        return 1 - Util.fpart(x)

    @staticmethod
    def interpolate(x0, y0, x1, y1):
        if x0 == x1:
            return np.array([y0])
        else:
            return np.linspace(y0, y1, x1 - x0, dtype=int)


# Quaternion

In [ ]:
class Quaternion:
    """Quaternion class for rotation"""
    # https://ja.wikipedia.org/wiki/四元数
    # https://en.wikipedia.org/wiki/Quaternion
    # https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation
    # https://qiita.com/mebiusbox2/items/2fa0f0a9ca1cf2044e82
    # https://qiita.com/drken/items/0639cf34cce14e8d58a5
    # https://kamino.hatenablog.com/entry/rotation_expressions

    def __init__(self, w, x, y, z):
        self._w = w
        self._v = np.array([x, y, z])

    @property
    def abs(self):
        return np.sqrt(self._w ** 2 + np.sum(self._v ** 2))

    @property
    def abssqrd(self):
        return self._w ** 2 + np.sum(self._v ** 2)

    @property
    def con(self):
        return self.conjucate

    @property
    def conjugate(self):
        v = -self._v
        return Quaternion(self._w, v[0], v[1], v[2])

    @property
    def inv(self):
        return self.inverse

    @property
    def inverse(self):
        return self.conjugate / self.abssqrd

    @property
    def matrix(self):
        x = self._v[0]
        y = self._v[1]
        z = self._v[2]
        w = self._w
        return np.array([
            [x ** 2 - y ** 2 - z ** 2 + w ** 2, 2 * (x * y - z * w), 2 * (x * z + y * w)],
            [2 * (x * y + z * w), - x ** 2 + y ** 2 - z ** 2 + w ** 2, 2 * (y * z - x * w)],
            [2 * (x * z - y * w), 2 * (y * z + x * w), - x ** 2 - y ** 2 + z ** 2 + w ** 2]
        ])

    def __add__(self, other):
        v = self._v + other._v
        return Quaternion(self._w + other._w, v[0], v[1], v[2])

    def __sub__(self, other):
        v = self._v - other._v
        return Quaternion(self._w - other._w, v[0], v[1], v[2])

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            v = self._v * other
            return Quaternion(self._w * other, v[0], v[1], v[2])
        elif isinstance(other, Quaternion):
            return Quaternion.mul(self, other)
        else:
            return 0

    def __truediv__(self, other):
        if isinstance(other, (int, float)):
            v = self._v / other
            return Quaternion(self._w / other, v[0], v[1], v[2])
        else:
            return 0

    def __eq__(self, other):
        if isinstance(other, Quaternion):
            return np.all(self._v == other._v) and self._w == other._w
        else:
            return False

    def __ne__(self, other):
        if isinstance(other, Quaternion):
            return np.any(self._v != other._v) or self._w != other._w
        else:
            return True

    def __iadd__(self, other):
        self._v += other._v
        self._w += other._w
        return self

    def __isub__(self, other):
        self._v -= other._v
        self._w -= other._w
        return self

    def __imul__(self, other):
        if isinstance(other, (int, float)):
            self._v *= other
            self._w *= other
            return self
        elif isinstance(other, Quaternion):
            self = Quaternion.mul(self, other)
            return self
        else:
            return 0

    def __itruediv__(self, other):
        if isinstance(other, (int, float)):
            v = self._v / other
            return Quaternion(self._w / other, v[0], v[1], v[2])
        else:
            return 0

    def __str__(self):
        return str(np.array([self._w, self._v[0], self._v[1], self._v[2]]))

    def normalize(self):
        self /= self.abs

    @staticmethod
    def mul(q1, q2):
        v = q1._w * q2._v + q2._w * q1._v + np.cross(q1._v, q2._v)
        return Quaternion(q1._w * q2._w - np.inner(q1._v, q2._v), v[0], v[1], v[2])

    @staticmethod
    def vector(v):
        return Quaternion(0, v[0], v[1], v[2])

    @staticmethod
    def rotate_quaternion_with_quaternion(p, q):
        return Quaternion.mul(Quaternion.mul(q, p), q.inv)

    @staticmethod
    def rotation_quaternion(n, theta):
        v = np.sin(theta / 2) * n
        return Quaternion(np.cos(theta / 2), v[0], v[1], v[2])

    @staticmethod
    def rotate_vector_with_vector_theta(r, n, theta):
        return Quaternion.rotate_quaternion_with_quaternion(Quaternion.vector(r), Quaternion.rotation_quaternion(n, theta))

    @staticmethod
    def euler_to_quaternion(roll, pitch, yaw):
        cos_roll = np.cos(roll / 2)
        sin_roll = np.sin(roll / 2)
        cos_pitch = np.cos(pitch / 2)
        sin_pitch = np.sin(pitch / 2)
        cos_yaw = np.cos(yaw / 2)
        sin_yaw = np.sin(yaw / 2)

        return Quaternion(
            cos_roll * cos_pitch * cos_yaw + sin_roll * sin_pitch * sin_yaw,
            sin_roll * cos_pitch * cos_yaw - cos_roll * sin_pitch * sin_yaw,
            cos_roll * sin_pitch * cos_yaw + sin_roll * cos_pitch * sin_yaw,
            cos_roll * cos_pitch * sin_yaw - sin_roll * sin_pitch * cos_yaw)

    @staticmethod
    def interpolate(t, q1, q2):
        q1 = q1 / q1.abs
        q2 = q2 / q2.abs
        if q1 == q2:
            return q1
        inner = q1._w * q2._w + np.inner(q1._v, q2._v)
        if inner < 0:
            q2 = -q2
            inner = -inner
        theta = np.arccos(inner)
        return (q1 * np.sin((1 - t) * theta) + q2 * np.sin(t * theta)) / np.sin(theta)


# Layer

In [ ]:
class Layer:
    """Layer class for canvas class"""

    def __init__(self, W: int, H: int, f=None, c_u:int=None, c_v:int=None, name="Default Layer", opaque=False, layer_type=0):
        self.W = W
        self.H = H
        self.f = f
        self.c_u = c_u
        self.c_v = c_v
        self.name = name
        self.layer_type = layer_type

        if layer_type == 0:
            if opaque:
                self.img = np.tile(np.array([0, 0, 0, 255], dtype='u1').reshape(1, 1, 4), (W, H, 1))
            else:
                self.img = np.full((W, H, 4), 0, dtype='u1')
        elif layer_type == 1:
            if opaque:
                self.img = np.full((W, H, 4), 255, dtype='u1')
            else:
                self.img = np.full((W, H, 4), 0, dtype='u1')
            self.z_buffer = np.zeros((W, H), dtype='f8')
        elif layer_type == 2:
            self.z_buffer = np.zeros((W, H), dtype='f8')

    def save_pos_rot(self, pos, rot):
        self.pos = pos
        self.rot = rot

        self.R = self.rot.matrix
        self.Rinv = self.rot.inv.matrix
        self.T = -np.matmul(self.rot.matrix, self.pos.reshape(3, 1))
        self.Tinv = self.pos.reshape(3, 1)

    def draw_point(self, x, y, c=None, z=None, n=None, lights=None, shadows=None, wire_frame=False):
        if c is not None:
            c = c.reshape(1, -1)
        if z is not None:
            z = np.array([z])
        if n is not None:
            n = n.reshape(1, -1)
        self.draw(np.array([x]), np.array([y]), c, z, n, lights, shadows, wire_frame)

    def draw(self, x, y, c=None, z=None, n=None, lights=None, shadows=None, wire_frame=False):
        xyfilter = np.logical_and(np.logical_and(0 <= x, x < self.W), np.logical_and(0 <= y, y < self.H))
        x = x[xyfilter]
        if x.size > 0:
            y = y[xyfilter]
            z = z[xyfilter]

            if self.layer_type == 0:
                self.img[x, y] = np.clip(c, 0, 255).astype('u1')
            elif self.layer_type == 1:
                zfilter = self.z_buffer[x, y] < z
                x = x[zfilter]
                if x.size > 0:
                    y = y[zfilter]
                    c = c[xyfilter][zfilter]
                    z = z[zfilter]
                    n = n[xyfilter][zfilter]

                    self.z_buffer[x, y] = z
                    light_col = np.tile(Color.clear_color.reshape(1, -1), (x.size, 1)) # row vector
                    P = Raytracing.transform_canvas_to_world(self.f, self.c_u, self.c_v, self.Rinv, self.Tinv, np.concatenate([x.reshape(1, -1), y.reshape(1, -1)], axis=0), z) # column vector

                    for lightid in range(len(lights)):
                        light = lights[lightid]
                        shadow = shadows[lightid]

                        if light.light_type == 0:
                            light_col = Color.add(light_col, light.calc(P, n, self.pos, c=c))
                        elif light.light_type == 1:
                            v = ShaderVertex.transform_world_to_camera(light.R, light.T, P.T)
                            vfilter = np.logical_and(Settings.LIGHT_NEARCLIP <= v[2], v[2] <= Settings.LIGHT_FARCLIP)
                            v = v[:, vfilter]
                            if v.size > 0:
                                newv, dist = ShaderVertex.transform_camera_to_canvas(shadow.f, shadow.c_u, shadow.c_v, v, False)
                                newvfilter = np.logical_and(np.logical_and(0 <= newv[0], newv[0] < shadow.W), np.logical_and(0 <= newv[1], newv[1] < shadow.H))
                                newv = newv[:, newvfilter]
                                if newv.size > 0:
                                    newv = newv.astype('i8')
                                    vfilter[vfilter] = newvfilter
                                    light_col[vfilter] = Color.add(light_col[vfilter], light.calc(P[:, vfilter], n[vfilter], self.pos, dist[newvfilter], shadow.z_buffer[newv[0], newv[1]], c[vfilter], wire_frame))
                        else:
                            v = ShaderVertex.transform_world_to_camera(light.R, light.T, P.T)
                            vfilter = np.logical_and(Settings.LIGHT_NEARCLIP <= v[2], v[2] <= Settings.LIGHT_FARCLIP)
                            v = v[:, vfilter]
                            if v.size > 0:
                                newv, dist = ShaderVertex.transform_camera_to_canvas(shadow.f, shadow.c_u, shadow.c_v, v, True)
                                newvfilter = np.logical_and(np.logical_and(0 <= newv[0], newv[0] < shadow.W), np.logical_and(0 <= newv[1], newv[1] < shadow.H))
                                newv = newv[:, newvfilter]
                                if newv.size > 0:
                                    newv = newv.astype('i8')
                                    vfilter[vfilter] = newvfilter
                                    light_col[vfilter] = Color.add(light_col[vfilter], light.calc(P[:, vfilter], n[vfilter], self.pos, dist[newvfilter], shadow.z_buffer[newv[0], newv[1]], c[vfilter], wire_frame))
                    self.img[x, y] = Color.over(light_col, self.img[x, y])
            else:
                zfilter = self.z_buffer[x, y] < z
                x = x[zfilter]
                if x.size > 0:
                    y = y[zfilter]
                    self.z_buffer[x, y] = z[zfilter]


# Line2D

In [ ]:
class Line2D:
    """Line class for a layer"""

    @staticmethod
    def vline(layer:Layer, x: int, y0: int, y1: int, c0=None, c1=None, z0=0, z1=0, n0=None, n1=None, lights=None, shadows=None):
        """Draw a vertical line from (x, y0) to (x, y1) with color c"""
        #print("vline: x: ", x, ", y0: ", y0, ", y1: ", y1, ", c0: ", c0, ", c1: ", c1, ", z0: ", z0, ", z1: ", z1, ", n0: ", n0, ", n1: ", n1, ", lights: ", lights, ", shadows: ", shadows)
        if y0 == y1:
            layer.draw_point(x, y0, c0, max([z0, z1]), n0, lights, shadows)
        else:
            if y0 > y1:
                y0, y1 = y1, y0
                z0, z1 = z1, z0
                if n0 is not None:
                    c0, c1 = c1.copy(), c0.copy()
                    n0, n1 = n1.copy(), n0.copy()

            if 0 <= x < layer.W:
                if 0 <= y1 and y0 < layer.H:
                    if y0 < 0:
                        z0 = z1 + (z0 - z1) * (0 - y1) / (y0 - y1)
                        if n0 is not None:
                            c0 = c1 + (c0 - c1) * (0 - y1) / (y0 - y1)
                            n0 = n1 + (n0 - n1) * (0 - y1) / (y0 - y1)
                        y0 = 0
                    if y0 == y1:
                        layer.draw_point(x, y0, c0, max([z0, z1]), n0, lights, shadows)
                    else:
                        if y1 >= layer.H:
                            z1 = z0 + (z1 - z0) * (layer.H - 1 - y0) / (y1 - y0)
                            if n0 is not None:
                                c1 = c0 + (c1 - c0) * (layer.H - 1 - y0) / (y1 - y0)
                                n1 = n0 + (n1 - n0) * (layer.H - 1 - y0) / (y1 - y0)
                            y1 = layer.H - 1
                        if y0 == y1:
                            layer.draw_point(x, y0, c0, max([z0, z1]), n0, lights, shadows)
                        else:
                            y = np.arange(y0, y1 + 1)
                            if n0 is not None:
                                layer.draw(np.full(y.size, x, dtype='i8'), y,
                                np.tile(c0.reshape(1, -1), (y.size, 1)) + np.tile((c1 - c0).reshape(1, -1), (y.size, 1)) * (np.tile(y.reshape(-1, 1), (1, c0.size)) - y0) / (y1 - y0),
                                np.full(y.size, z0) + np.full(y.size, z1 - z0) * (y - y0) / (y1 - y0),
                                np.tile(n0.reshape(1, -1), (y.size, 1)) + np.tile((n1 - n0).reshape(1, -1), (y.size, 1)) * (np.tile(y.reshape(-1, 1), (1, n0.size)) - y0) / (y1 - y0), lights, shadows)
                            else:
                                layer.draw(np.full(y.size, x, dtype='i8'), y, z=np.full(y.size, z0) + np.full(y.size, z1 - z0) * (y - y0) / (y1 - y0))

    @staticmethod
    def hline(layer:Layer, x0: int, x1: int, y: int, c0=None, c1=None, z0=0, z1=0, n0=None, n1=None, lights=None, shadows=None):
        """Draw a horizontal line from (x0, y) to (x1, y) with color c"""
        #print("hline: x0: ", x0, ", x1: ", x1, ", y: ", y, ", c0: ", c0, ", c1: ", c1, ", z0: ", z0, ", z1: ", z1, ", n0: ", n0, ", n1: ", n1, ", lights: ", lights, ", shadows: ", shadows)
        #print("hline: x0: ", x0, ", x1: ", x1, ", y: ", y, ", c0: ", c0, ", c1: ", c1, ", z0: ", z0, ", z1: ", z1, ", n0: ", n0, ", n1: ", n1)
        if x0 == x1:
            layer.draw_point(x0, y, c0, max([z0, z1]), n0, lights, shadows)
        else:
            if x0 > x1:
                x0, x1 = x1, x0
                z0, z1 = z1, z0
                if n0 is not None:
                    c0, c1 = c1.copy(), c0.copy()
                    n0, n1 = n1.copy(), n0.copy()

            if 0 <= y < layer.H:
                if 0 <= x1 and x0 < layer.W:
                    if x0 < 0:
                        z0 = z1 + (z0 - z1) * (0 - x1) / (x0 - x1)
                        if n0 is not None:
                            c0 = c1 + (c0 - c1) * (0 - x1) / (x0 - x1)
                            n0 = n1 + (n0 - n1) * (0 - x1) / (x0 - x1)
                        x0 = 0
                    if x0 == x1:
                        layer.draw_point(x0, y, c0, max([z0, z1]), n0, lights, shadows)
                    else:
                        if x1 >= layer.W:
                            z1 = z0 + (z1 - z0) * (layer.W - 1 - x0) / (x1 - x0)
                            if n0 is not None:
                                c1 = c0 + (c1 - c0) * (layer.W - 1 - x0) / (x1 - x0)
                                n1 = n0 + (n1 - n0) * (layer.W - 1 - x0) / (x1 - x0)
                            x1 = layer.W - 1
                        if x0 == x1:
                            layer.draw_point(x0, y, c0, max([z0, z1]), n0, lights, shadows)
                        else:
                            x = np.arange(x0, x1 + 1)
                            if n0 is not None:
                                layer.draw(x, np.full(x.size, y, dtype='i8'),
                                np.tile(c0.reshape(1, -1), (x.size, 1)) + np.tile((c1 - c0).reshape(1, -1), (x.size, 1)) * (np.tile(x.reshape(-1, 1), (1, c0.size)) - x0) / (x1 - x0),
                                np.full(x.size, z0) + np.full(x.size, z1 - z0) * (x - x0) / (x1 - x0),
                                np.tile(n0.reshape(1, -1), (x.size, 1)) + np.tile((n1 - n0).reshape(1, -1), (x.size, 1)) * (np.tile(x.reshape(-1, 1), (1, n0.size)) - x0) / (x1 - x0), lights, shadows)
                            else:
                                layer.draw(x, np.full(x.size, y, dtype='i8'), z=np.full(x.size, z0) + np.full(x.size, z1 - z0) * (x - x0) / (x1 - x0))

    @staticmethod
    def hline_triangle(layer:Layer, x0: int, x1: int, y: int, c0, c1, z0, z1, n0, n1):
        if x0 == x1:
            return np.array([x0], dtype='i8'), np.array([y], dtype='i8'), c0.reshape(1, -1), np.array([max([z0, z1])]), n0.reshape(1, -1)
        else:
            if x0 > x1:
                x0, x1 = x1, x0
                z0, z1 = z1, z0
                c0, c1 = c1.copy(), c0.copy()
                n0, n1 = n1.copy(), n0.copy()

            if 0 <= y < layer.H:
                if 0 <= x1 and x0 < layer.W:
                    if x0 < 0:
                        z0 = z1 + (z0 - z1) * (0 - x1) / (x0 - x1)
                        c0 = c1 + (c0 - c1) * (0 - x1) / (x0 - x1)
                        n0 = n1 + (n0 - n1) * (0 - x1) / (x0 - x1)
                        x0 = 0
                    if x0 == x1:
                        return np.array([x0], dtype='i8'), np.array([y], dtype='i8'), c0.reshape(1, -1), np.array([max([z0, z1])]), n0.reshape(1, -1)
                    else:
                        if x1 >= layer.W:
                            z1 = z0 + (z1 - z0) * (layer.W - 1 - x0) / (x1 - x0)
                            c1 = c0 + (c1 - c0) * (layer.W - 1 - x0) / (x1 - x0)
                            n1 = n0 + (n1 - n0) * (layer.W - 1 - x0) / (x1 - x0)
                            x1 = layer.W - 1
                        if x0 == x1:
                            return np.array([x0], dtype='i8'), np.array([y], dtype='i8'), c0.reshape(1, -1), np.array([max([z0, z1])]), n0.reshape(1, -1)
                        else:
                            x = np.arange(x0, x1 + 1, dtype='i8')
                            return x, np.full(x.size, y, dtype='i8'), np.tile(c0.reshape(1, -1), (x.size, 1)) + np.tile((c1 - c0).reshape(1, -1), (x.size, 1)) * (np.tile(x.reshape(-1, 1), (1, c0.size)) - x0) / (x1 - x0), np.full(x.size, z0) + np.full(x.size, z1 - z0) * (x - x0) / (x1 - x0), np.tile(n0.reshape(1, -1), (x.size, 1)) + np.tile((n1 - n0).reshape(1, -1), (x.size, 1)) * (np.tile(x.reshape(-1, 1), (1, n0.size)) - x0) / (x1 - x0)
                else:
                    return None, None, None, None, None
            else:
                return None, None, None, None, None

    @staticmethod
    def hline_triangle_shadow(layer:Layer, x0: int, x1: int, y: int, z0, z1):
        if x0 == x1:
            return np.array([x0], dtype='i8'), np.array([y], dtype='i8'), np.array([max([z0, z1])])
        else:
            if x0 > x1:
                x0, x1 = x1, x0
                z0, z1 = z1, z0

            if 0 <= y < layer.H:
                if 0 <= x1 and x0 < layer.W:
                    if x0 < 0:
                        z0 = z1 + (z0 - z1) * (0 - x1) / (x0 - x1)
                        x0 = 0
                    if x0 == x1:
                        return np.array([x0], dtype='i8'), np.array([y], dtype='i8'), np.array([max([z0, z1])])
                    else:
                        if x1 >= layer.W:
                            z1 = z0 + (z1 - z0) * (layer.W - 1 - x0) / (x1 - x0)
                            x1 = layer.W - 1
                        if x0 == x1:
                            return np.array([x0], dtype='i8'), np.array([y], dtype='i8'), np.array([max([z0, z1])])
                        else:
                            x = np.arange(x0, x1 + 1, dtype='i8')
                            return x, np.full(x.size, y, dtype='i8'), np.full(x.size, z0) + np.full(x.size, z1 - z0) * (x - x0) / (x1 - x0)
                else:
                    return None, None, None
            else:
                return None, None, None

    @staticmethod
    def line(layer:Layer, x0: int, y0: int, x1: int, y1: int, c0=None, c1=None, z0=0, z1=0, n0=None, n1=None, lights=None, shadows=None, antialias=False, wire_frame=False):
        """Draw a line from (x0, y0) to (x1, y1) with color c"""
        #print("line: x0: ", x0, ", y0: ", y0, ", x1: ", x1, ", y1: ", y1, ", c0: ", c0, ", c1: ", c1, ", z0: ", z0, ", z1: ", z1, ", n0: ", n0, ", n1: ", n1, ", lights: ", lights, ", shadows: ", shadows)
        if antialias:
            # Xiaolin Wu
            # https://en.wikipedia.org/wiki/Xiaolin_Wu%27s_line_algorithm

            steep = (abs(y1 - y0) > abs(x1 - x0))

            if steep:
                x0, y0 = y0, x0
                x1, y1 = y1, x1

            if x0 > x1:
                x0, x1 = x1, x0
                y0, y1 = y1, y0
                z0, z1 = z1, z0
                c0, c1 = c1.copy(), c0.copy()
                n0, n1 = n1.copy(), n0.copy()

            dx = x1 - x0
            dy = y1 - y0

            grad = 1 if dx == 0 else dy / dx

            xend = x0
            yend = y0 + grad * (xend - x0)
            xgap = 0.5
            xpxl1 = xend
            ypxl1 = Util.ipart(yend)

            if steep:
                layer.draw_point(ypxl1, xpxl1, np.clip(np.floor(c * (Util.rfpart(yend) * xgap)), 0, 255))
                layer.draw_point(ypxl1 + 1, xpxl1, np.clip(np.floor(c * (Util.fpart(yend) * xgap)), 0, 255))
            else:
                layer.draw_point(xpxl1, ypxl1, np.clip(np.floor(c * (Util.rfpart(yend) * xgap)), 0, 255))
                layer.draw_point(xpxl1, ypxl1 + 1, np.clip(np.floor(c * (Util.fpart(yend) * xgap)), 0, 255))

            intery = yend + grad

            xend = x1
            yend = y1 + grad * (xend - x1)
            xgap = 0.5
            xpxl2 = xend
            ypxl2 = Util.ipart(yend)

            if steep:
                layer.draw_point(ypxl2, xpxl2, np.clip(np.floor(c * (Util.rfpart(yend) * xgap)), 0, 255))
                layer.draw_point(ypxl2 + 1, xpxl2, np.clip(np.floor(c * (Util.fpart(yend) * xgap)), 0, 255))
            else:
                layer.draw_point(xpxl2, ypxl2, np.clip(np.floor(c * (Util.rfpart(yend) * xgap)), 0, 255))
                layer.draw_point(xpxl2, ypxl2 + 1, np.clip(np.floor(c * (Util.fpart(yend) * xgap)), 0, 255))

            if steep:
                for x in range(xpxl1 + 1, xpxl2):
                    layer.draw_point(Util.ipart(intery), x, np.clip(np.floor(c * Util.rfpart(intery)), 0, 255))
                    layer.draw_point(Util.ipart(intery) + 1, x, np.clip(np.floor(c * Util.fpart(intery)), 0, 255))
                    intery += grad
            else:
                for x in range(xpxl1 + 1, xpxl2):
                    layer.draw_point(x, Util.ipart(intery), np.clip(np.floor(c * Util.rfpart(intery)), 0, 255))
                    layer.draw_point(x, Util.ipart(intery) + 1, np.clip(np.floor(c * Util.fpart(intery)), 0, 255))
                    intery += grad
        else:
            # Bresenham
            # https://en.wikipedia.org/wiki/Bresenham%27s_line_algorithm

            dx = abs(x1 - x0)
            dy = abs(y1 - y0)
            sx = 1 if x0 < x1 else -1
            sy = 1 if y0 < y1 else -1
            err = dx - dy
            x = x0
            y = y0

            if n0 is not None:
                while True:
                    layer.draw_point(x, y, c0 + (c1 - c0) * (x - x0) / (x1 - x0), z0 + (z1 - z0) * (x - x0) / (x1 - x0), n0 + (n1 - n0) * (x - x0) / (x1 - x0), lights, shadows, wire_frame)

                    if x == x1 and y == y1:
                        break

                    e2 = 2 * err
                    if e2 > -dy:
                        err -= dy
                        x += sx
                    if e2 < dx:
                        err += dx
                        y += sy
            else:
                while True:
                    layer.draw_point(x, y, z=z0 + (z1 - z0) * (x - x0) / (x1 - x0))

                    if x == x1 and y == y1:
                        break

                    e2 = 2 * err
                    if e2 > -dy:
                        err -= dy
                        x += sx
                    if e2 < dx:
                        err += dx
                        y += sy


# Rectangle2D

In [ ]:
class Rectangle2D:
    """Rectangle class for a layer"""

    @staticmethod
    def rectangle(layer:Layer, x0: int, y0: int, x1: int, y1: int, t: int, c, antialias=False):
        """Draw a rectangle of (x0, y0), (x0, y1), (x1, y0), (x1, y1) with line thickness t and color c"""
        if x0 > x1:
            x0, x1 = x1, x0
        if y0 > y1:
            y0, y1 = y1, y0

        if x0 == x1:
            Line2D.vline(layer, x0, y0, y1, c, c)
            return
        if y0 == y1:
            Line2D.hline(layer, x0, x1, y0, c, c)
            return

        if 2 * t > x1 - x0 + 1:
            t = round((x1 - x0 - 1) / 2)
        if 2 * t > y1 - y0 + 1:
            t = round((y1 - y0 - 1) / 2)

        for i in range(t):
            Line2D.vline(layer, x0 + i, y0, y1, c, c)
            Line2D.vline(layer, x1 - i, y0, y1, c, c)
            Line2D.hline(layer, x0, x1, y0 + i, c, c)
            Line2D.hline(layer, x0, x1, y1 - i, c, c)

    @staticmethod
    def filled_rectangle(layer:Layer, x0: int, y0: int, x1: int, y1: int, c, antialias=False):
        """Draw a filled rectangle of (x0, y0), (x0, y1), (x1, y0), (x1, y1) with color c"""
        if x0 > x1:
            x0, x1 = x1, x0
        if y0 > y1:
            y0, y1 = y1, y0

        for i in range(x0, x1 + 1):
            Line2D.vline(layer, i, y0, y1, c, c)


# Circle2D

In [ ]:
class Circle2D:
    """Circle class for a layer"""

    RADIUS_MAX = 500
    D = np.zeros((RADIUS_MAX, np.ceil(np.sqrt(2) * RADIUS_MAX * RADIUS_MAX / 4).astype('i8') + 1))
    Dinit = False

    @classmethod
    def circle(cls, layer:Layer, cx: int, cy: int, r: int, c, antialias=False):
        """Draw a circle with center (cx, cy), radius r, and color c"""
        if antialias:
            # Xiaolin Wu
            # https://pieceofnostalgia-bd472.firebaseapp.com/java/anti_aliased_circle_algorithm.html
            if not cls.Dinit:
                for rdash in range(1, cls.RADIUS_MAX):
                    for y in range(1, np.ceil(rdash / np.sqrt(2)) + 1):
                        xt = np.sqrt(rdash * rdash - y * y)
                        cls.D[rdash, y] = np.ceil(xt) - xt
                cls.Dinit = True

            if r < cls.RADIUS_MAX:
                x = r
                y = 0
                d = 0
                d_old = 0
                layer.draw_point(cx + x, cy + y, c, c)
                layer.draw_point(cx - x, cy + y, c, c)
                layer.draw_point(cx + x, cy - y, c, c)
                layer.draw_point(cx - x, cy - y, c, c)

                while y < x - 1:
                    y += 1
                    d = cls.D[r, y]
                    if d < d_old:
                        x -= 1

                    ctmp1 = np.clip(np.floor(c * (1 - d)), 0, 255)
                    ctmp2 = np.clip(np.floor(c * d), 0, 255)
                    layer.draw_point(cx + x, cy + y, ctmp1, ctmp1)
                    layer.draw_point(cx - x, cy + y, ctmp1, ctmp1)
                    layer.draw_point(cx + x, cy - y, ctmp1, ctmp1)
                    layer.draw_point(cx - x, cy - y, ctmp1, ctmp1)
                    layer.draw_point(cx + y, cy + x, ctmp1, ctmp1)
                    layer.draw_point(cx - y, cy + x, ctmp1, ctmp1)
                    layer.draw_point(cx + y, cy - x, ctmp1, ctmp1)
                    layer.draw_point(cx - y, cy - x, ctmp1, ctmp1)
                    layer.draw_point(cx + x - 1, cy + y, ctmp2, ctmp2)
                    layer.draw_point(cx - x - 1, cy + y, ctmp2, ctmp2)
                    layer.draw_point(cx + x - 1, cy - y, ctmp2, ctmp2)
                    layer.draw_point(cx - x - 1, cy - y, ctmp2, ctmp2)
                    layer.draw_point(cx + y - 1, cy + x, ctmp2, ctmp2)
                    layer.draw_point(cx - y - 1, cy + x, ctmp2, ctmp2)
                    layer.draw_point(cx + y - 1, cy - x, ctmp2, ctmp2)
                    layer.draw_point(cx - y - 1, cy - x, ctmp2, ctmp2)
                    d_old = d
            else:
                darr = np.zeros(np.ceil(np.sqrt(2) * r * r / 4) + 1)
                for y in range(1, np.ceil(r / np.sqrt(2)) + 0.1):
                    xt = np.sqrt(r * r - y * y)
                    darr[y] = np.ceil(xt) - xt

                x = r
                y = 0
                d = 0
                d_old = 0
                layer.draw_point(cx + x, cy + y, c, c)
                layer.draw_point(cx - x, cy + y, c, c)
                layer.draw_point(cx + x, cy - y, c, c)
                layer.draw_point(cx - x, cy - y, c, c)

                while y < x - 1:
                    y += 1
                    d = darr[y]
                    if d < d_old:
                        x -= 1

                    ctmp1 = np.clip(np.floor(c * (1 - d)), 0, 255)
                    ctmp2 = np.clip(np.floor(c * d), 0, 255)
                    layer.draw_point(cx + x, cy + y, ctmp1, ctmp1)
                    layer.draw_point(cx - x, cy + y, ctmp1, ctmp1)
                    layer.draw_point(cx + x, cy - y, ctmp1, ctmp1)
                    layer.draw_point(cx - x, cy - y, ctmp1, ctmp1)
                    layer.draw_point(cx + y, cy + x, ctmp1, ctmp1)
                    layer.draw_point(cx - y, cy + x, ctmp1, ctmp1)
                    layer.draw_point(cx + y, cy - x, ctmp1, ctmp1)
                    layer.draw_point(cx - y, cy - x, ctmp1, ctmp1)
                    layer.draw_point(cx + x - 1, cy + y, ctmp2, ctmp2)
                    layer.draw_point(cx - x - 1, cy + y, ctmp2, ctmp2)
                    layer.draw_point(cx + x - 1, cy - y, ctmp2, ctmp2)
                    layer.draw_point(cx - x - 1, cy - y, ctmp2, ctmp2)
                    layer.draw_point(cx + y - 1, cy + x, ctmp2, ctmp2)
                    layer.draw_point(cx - y - 1, cy + x, ctmp2, ctmp2)
                    layer.draw_point(cx + y - 1, cy - x, ctmp2, ctmp2)
                    layer.draw_point(cx - y - 1, cy - x, ctmp2, ctmp2)
                    d_old = d
        else:
            # mid-point circle drawing algorithm
            # https://www.geeksforgeeks.org/mid-point-circle-drawing-algorithm/?ref=lbp
            if r == 0:
                layer.draw_point(cx, cy, c, c)
                return
            if r < 0:
                r = -r

            layer.draw_point(cx + r, cy, c, c)
            layer.draw_point(cx - r, cy, c, c)
            layer.draw_point(cx, cy + r, c, c)
            layer.draw_point(cx, cy - r, c, c)

            x = r
            y = 0
            P = 1 - r

            while x > y:
                y += 1
                if P <= 0:
                    P += 2 * y + 1
                else:
                    x -= 1
                    P += 2 * y - 2 * x + 1

                if x < y:
                    break

                layer.draw_point(cx + x, cy + y, c, c)
                layer.draw_point(cx - x, cy + y, c, c)
                layer.draw_point(cx + x, cy - y, c, c)
                layer.draw_point(cx - x, cy - y, c, c)

                if x != y:
                    layer.draw_point(cx + y, cy + x, c, c)
                    layer.draw_point(cx - y, cy + x, c, c)
                    layer.draw_point(cx + y, cy - x, c, c)
                    layer.draw_point(cx - y, cy - x, c, c)

    @staticmethod
    def ellipse(layer:Layer, cx: int, cy: int, rx: int, ry: int, c, antialias=False):
        """Draw an ellipse with center (cx, cy), radius (rx, ry), and color c"""
        # https://www.geeksforgeeks.org/midpoint-ellipse-drawing-algorithm/?ref=lbp
        if rx == 0 and ry == 0:
            layer.draw_point(cx, cy, c, c)
            return
        if rx == 0:
            Line2D.vline(layer, cx, cy - ry, cy + ry, c, c)
            return
        if ry == 0:
            Line2D.hline(layer, cx - ry, cx + ry, cy, c, c)
            return
        if rx < 0:
            rx = -rx
        if ry < 0:
            ry = -ry

        x = 0
        y = ry
        d1 = (ry * ry) - (rx * rx * ry) + (rx * rx / 4)
        dx = 2 * ry * ry * x
        dy = 2 * rx * rx * y

        while dx < dy:
            layer.draw_point(cx + x, cy + y, c, c)
            layer.draw_point(cx - x, cy + y, c, c)
            layer.draw_point(cx + x, cy - y, c, c)
            layer.draw_point(cx - x, cy - y, c, c)

            if d1 < 0:
                x += 1
                dx += 2 * ry * ry
                d1 += dx + ry * ry
            else:
                x += 1
                y -= 1
                dx += 2 * ry * ry
                dy -= 2 * rx * rx
                d1 += dx - dy + ry * ry

        d2 = ry * ry * (x + 0.5) * (x + 0.5) + rx * rx * (y - 1) * (y - 1) - rx * rx * ry * ry

        while y >= 0:
            layer.draw_point(cx + x, cy + y, c, c)
            layer.draw_point(cx - x, cy + y, c, c)
            layer.draw_point(cx + x, cy - y, c, c)
            layer.draw_point(cx - x, cy - y, c, c)

            if d2 > 0:
                y -= 1
                dy -= 2 * rx * rx
                d2 += rx * rx - dy
            else:
                y -= 1
                x += 1
                dx += 2 * ry * ry
                dy -= 2 * rx * rx
                d2 += dx - dy + rx * rx

    @staticmethod
    def filled_circle(layer:Layer, cx: int, cy: int, r: int, c, antialias=False):
        """Draw a filled circle with center (cx, cy), radius r, and color c"""
        if r == 0:
            layer.draw_point(cx, cy, c, c)
            return
        if r < 0:
            r = -r

        Line2D.hline(layer, cx - r, cx + r, cy, c, c)

        x = r
        y = 0
        P = 1 - r

        while x > y:
            y += 1
            if P <= 0:
                P += 2 * y + 1
            else:
                x -= 1
                P += 2 * y - 2 * x + 1

            if x < y:
                break

            Line2D.hline(layer, cx - x, cx + x, cy + y, c, c)
            Line2D.hline(layer, cx - x, cx + x, cy - y, c, c)

            if x != y:
                Line2D.hline(layer, cx - y, cx + y, cy + x, c, c)
                Line2D.hline(layer, cx - y, cx + y, cy - x, c, c)

    @staticmethod
    def filled_ellipse(layer:Layer, cx: int, cy: int, rx: int, ry: int, c, antialias=False):
        """Draw a filled ellipse with center (cx, cy), radius (rx, ry), and color c"""
        if rx == 0 and ry == 0:
            layer.draw_point(cx, cy, c, c)
            return
        if rx == 0:
            Line2D.vline(layer, cx, cy - ry, cy + ry, c, c)
            return
        if ry == 0:
            Line2D.hline(layer, cx - ry, cx + ry, cy, c, c)
            return
        if rx < 0:
            rx = -rx
        if ry < 0:
            ry = -ry

        x = 0
        y = ry
        d1 = (ry * ry) - (rx * rx * ry) + (rx * rx / 4)
        dx = 2 * ry * ry * x
        dy = 2 * rx * rx * y

        while dx < dy:
            Line2D.hline(layer, cx - x, cx + x, cy + y, c, c)
            Line2D.hline(layer, cx - x, cx + x, cy - y, c, c)

            if d1 < 0:
                x += 1
                dx += 2 * ry * ry
                d1 += dx + ry * ry
            else:
                x += 1
                y -= 1
                dx += 2 * ry * ry
                dy -= 2 * rx * rx
                d1 += dx - dy + ry * ry

        d2 = ry * ry * (x + 0.5) * (x + 0.5) + rx * rx * (y - 1) * (y - 1) - rx * rx * ry * ry

        while y >= 0:
            Line2D.hline(layer, cx - x, cx + x, cy + y, c, c)
            Line2D.hline(layer, cx - x, cx + x, cy - y, c, c)

            if d2 > 0:
                y -= 1
                dy -= 2 * rx * rx
                d2 += rx * rx - dy
            else:
                y -= 1
                x += 1
                dx += 2 * ry * ry
                dy -= 2 * rx * rx
                d2 += dx - dy + rx * rx


# Triangle2D

In [ ]:
class Triangle2D:
    """Triangle class for a layer"""

    @staticmethod
    def triangle(layer:Layer, x0:int, y0:int, x1:int, y1:int, x2:int, y2:int, c0=None, c1=None, c2=None, z0=0, z1=0, z2=0, n0=None, n1=None, n2=None, lights=None, shadows=None, antialias=False):
        """Draw a triangle with vertex (x0, y0), (x1, y1), (x2, y2) and color c"""
        Line2D.line(layer, x0, y0, x1, y1, c0, c1, z0, z1, n0, n1, lights, shadows, antialias, True)
        Line2D.line(layer, x1, y1, x2, y2, c1, c2, z1, z2, n1, n2, lights, shadows, antialias, True)
        Line2D.line(layer, x2, y2, x0, y0, c2, c0, z2, z0, n2, n0, lights, shadows, antialias, True)

    @staticmethod
    def filled_flat_triangle(layer:Layer, x0:int, y0:int, x1:int, y1:int, x2:int, y2:int, c0=None, c1=None, c2=None, z0=0, z1=0, z2=0, n0=None, n1=None, n2=None, lights=None, shadows=None, antialias=False):
        #print("filled_flat_triangle: x0: ", x0, ", y0: ", y0, ", x1: ", x1, ", y1: ", y1, ", x2: ", x2, ", y2: ", y2, ", c0: ", c0, ", c1: ", c1, ", c2: ", c2, ", z0: ", z0, ", z1: ", z1, ", z2: ", z2, ", n0: ", n0, ", n1: ", n1, ", n2: ", n2, ", lights: ", lights, ", shadows: ", shadows)

        dx1 = abs(x1 - x0)
        dx2 = abs(x2 - x0)
        dy = abs(y1 - y0)

        sx1 = 1 if x0 < x1 else -1
        sx2 = 1 if x0 < x2 else -1
        sy = 1 if y0 < y1 else -1

        err1 = dx1 - dy
        err2 = dx2 - dy

        xnow1 = x0
        xnow2 = x0
        ynow = y0
        finish = False

        if n0 is not None:
            xarr = []
            yarr = []
            carr = []
            zarr = []
            narr = []
            while True:
                while True:
                    #layer.draw_point(xnow1, ynow, c0 + (c1 - c0) * (ynow - y0) / (y1 - y0), z0 + (z1 - z0) * (ynow - y0) / (y1 - y0), n0 + (n1 - n0) * (ynow - y0) / (y1 - y0), lights, shadows)
                    if xnow1 == x1 and ynow == y1:
                        finish = True
                        break
                    e21 = 2 * err1
                    if e21 > -dy:
                        err1 -= dy
                        xnow1 += sx1
                    if e21 < dx1:
                        break

                while True:
                    #layer.draw_point(xnow2, ynow, c0 + (c2 - c0) * (ynow - y0) / (y2 - y0), z0 + (z2 - z0) * (ynow - y0) / (y2 - y0), n0 + (n2 - n0) * (ynow - y0) / (y2 - y0), lights, shadows)
                    if xnow2 == x2 and ynow == y2:
                        finish = True
                        break
                    e22 = 2 * err2
                    if e22 > -dy:
                        err2 -= dy
                        xnow2 += sx2
                    if e22 < dx2:
                        break
                xret, yret, cret, zret, nret = Line2D.hline_triangle(layer, xnow1, xnow2, ynow, c0 + (c1 - c0) * (ynow - y0) / (y1 - y0), c0 + (c2 - c0) * (ynow - y0) / (y2 - y0), z0 + (z1 - z0) * (ynow - y0) / (y1 - y0), z0 + (z2 - z0) * (ynow - y0) / (y2 - y0), n0 + (n1 - n0) * (ynow - y0) / (y1 - y0), n0 + (n2 - n0) * (ynow - y0) / (y2 - y0))
                if xret is not None:
                    xarr.append(xret)
                    yarr.append(yret)
                    carr.append(cret)
                    zarr.append(zret)
                    narr.append(nret)
                if ((xnow1 == x1 or xnow2 == x2) and ynow == y2) or finish:
                    break
                err1 += dx1
                err2 += dx2
                ynow += sy
            if len(xarr) > 0:
                layer.draw(np.concatenate(xarr, axis=0), np.concatenate(yarr, axis=0), np.concatenate(carr, axis=0), np.concatenate(zarr, axis=0), np.concatenate(narr, axis=0), lights, shadows)
        else:
            xarr = []
            yarr = []
            zarr = []
            while True:
                while True:
                    #layer.draw_point(xnow1, ynow, z=z0 + (z1 - z0) * (ynow - y0) / (y1 - y0))
                    if xnow1 == x1 and ynow == y1:
                        finish = True
                        break
                    e21 = 2 * err1
                    if e21 > -dy:
                        err1 -= dy
                        xnow1 += sx1
                    if e21 < dx1:
                        break

                while True:
                    #layer.draw_point(xnow2, ynow, z=z0 + (z2 - z0) * (ynow - y0) / (y2 - y0))
                    if xnow2 == x2 and ynow == y2:
                        finish = True
                        break
                    e22 = 2 * err2
                    if e22 > -dy:
                        err2 -= dy
                        xnow2 += sx2
                    if e22 < dx2:
                        break
                xret, yret, zret = Line2D.hline_triangle_shadow(layer, xnow1, xnow2, ynow, z0 + (z1 - z0) * (ynow - y0) / (y1 - y0), z1=z0 + (z2 - z0) * (ynow - y0) / (y2 - y0))
                if xret is not None:
                    xarr.append(xret)
                    yarr.append(yret)
                    zarr.append(zret)
                if ((xnow1 == x1 or xnow2 == x2) and ynow == y2) or finish:
                    break
                err1 += dx1
                err2 += dx2
                ynow += sy
            if len(xarr) > 0:
                layer.draw(np.concatenate(xarr, axis=0), np.concatenate(yarr, axis=0), z=np.concatenate(zarr, axis=0))

    @staticmethod
    def filled_triangle(layer:Layer, x0:int, y0:int, x1:int, y1:int, x2:int, y2:int, c0=None, c1=None, c2=None, z0=0, z1=0, z2=0, n0=None, n1=None, n2=None, lights=None, shadows=None, antialias=False):
        """Draw a filled triangle with vertex (x0, y0), (x1, y1), (x2, y2) and color c"""
        # Bresenham
        # http://www.sunshine2k.de/coding/java/TriangleRasterization/TriangleRasterization.html

        #print("filled_triangle: x0: ", x0, ", y0: ", y0, ", x1: ", x1, ", y1: ", y1, ", x2: ", x2, ", y2: ", y2, ", c0: ", c0, ", c1: ", c1, ", c2: ", c2, ", z0: ", z0, ", z1: ", z1, ", z2: ", z2, ", n0: ", n0, ", n1: ", n1, ", n2: ", n2, ", lights: ", lights, ", shadows: ", shadows)

        if y0 == y1 == y2:
            Line2D.hline(layer, x0, x1, y0, c0, c1, z0, z1, n0, n1, lights, shadows)
            Line2D.hline(layer, x1, x2, y0, c1, c2, z1, z2, n1, n2, lights, shadows)
            return
    
        if y1 < y0:
            x0, x1 = x1, x0
            y0, y1 = y1, y0
            z0, z1 = z1, z0
            if n0 is not None:
                c0, c1 = c1.copy(), c0.copy()
                n0, n1 = n1.copy(), n0.copy()
        if y2 < y0:
            x0, x2 = x2, x0
            y0, y2 = y2, y0
            z0, z2 = z2, z0
            if n0 is not None:
                c0, c2 = c2.copy(), c0.copy()
                n0, n2 = n2.copy(), n0.copy()
        if y2 < y1:
            x1, x2 = x2, x1
            y1, y2 = y2, y1
            z1, z2 = z2, z1
            if n0 is not None:
                c1, c2 = c2.copy(), c1.copy()
                n1, n2 = n2.copy(), n1.copy()
        
        if y1 == y2:
            Line2D.hline(layer, x1, x2, y1, c1, c2, z1, z2, n1, n2, lights, shadows)
            Triangle2D.filled_flat_triangle(layer, x0, y0, x1, y1, x2, y2, c0, c1, c2, z0, z1, z2, n0, n1, n2, lights, shadows, antialias)
        elif y0 == y1:
            Line2D.hline(layer, x0, x1, y0, c0, c1, z0, z1, n0, n1, lights, shadows)
            Triangle2D.filled_flat_triangle(layer, x2, y2, x0, y0, x1, y1, c2, c0, c1, z2, z0, z1, n2, n0, n1, lights, shadows, antialias)
        else:
            x3 = x0 + round((x2 - x0) * (y1 - y0) / (y2 - y0))
            y3 = y1
            z3 = z0 + (z2 - z0) * (y1 - y0) / (y2 - y0)
            if n0 is not None:
                c3 = c0 + (c2 - c0) * (y1 - y0) / (y2 - y0)
                n3 = n0 + (n2 - n0) * (y1 - y0) / (y2 - y0)
                Line2D.hline(layer, x1, x3, y1, c1, c3, z1, z3, n1, n3, lights, shadows)
                Triangle2D.filled_flat_triangle(layer, x0, y0, x1, y1, x3, y3, c0, c1, c3, z0, z1, z3, n0, n1, n3, lights, shadows, antialias)
                Triangle2D.filled_flat_triangle(layer, x2, y2, x1, y1, x3, y3, c2, c1, c3, z2, z1, z3, n2, n1, n3, lights, shadows, antialias)
            else:
                Line2D.hline(layer, x1, x3, y1, z0=z1, z1=z3)
                Triangle2D.filled_flat_triangle(layer, x0, y0, x1, y1, x3, y3, z0=z0, z1=z1, z2=z3, antialias=antialias)
                Triangle2D.filled_flat_triangle(layer, x2, y2, x1, y1, x3, y3, z0=z2, z1=z1, z2=z3, antialias=antialias)


# Curve2D

In [ ]:
class Curve2D:
    """Curve class for a layer"""

    @staticmethod
    def thick_line_with_circles(layer:Layer, x0: int, y0: int, x1: int, y1: int, t: int, c, antialias=False):
        """Draw a line from (x0, y0) to (x1, y1) with width t and color c, using circles"""
        if x0 == x1:
            for y in range(y0, y1 + 1):
                Circle2D.filled_circle(layer, x0, y, t, c, antialias)
            return
        if y0 == y1:
            for x in range(x0, x1 + 1):
                Circle2D.filled_circle(layer, x, y0, t, c, antialias)
            return

        slope = (y1 - y0) / (x1 - x0)

        if x0 < x1:
            for x in range(x0, x1 + 1):
                y = y0 + slope * (x - x0)
                Circle2D.filled_circle(layer, x, round(y), t, c, antialias)
        else:
            for x in range(x1, x0 + 1):
                y = y0 + slope * (x - x0)
                Circle2D.filled_circle(layer, x, round(y), t, c, antialias)

        slope = (x1 - x0) / (y0 - y1)

        if y0 < y1:
            for y in range(y0, y1 + 1):
                x = x0 + slope * (y - y0)
                Circle2D.filled_circle(layer, round(x), y, t, c, antialias)
        else:
            for y in range(y1, y0 + 1):
                x = x0 + slope * (y - y0)
                Circle2D.filled_circle(layer, round(x), y, t, c, antialias)

    @staticmethod
    def circles_along_points(layer:Layer, points, r: int, c, antialias=False):
        """Draw a curve along points with circle of radius r and color c"""
        for point in points:
            Circle2D.filled_circle(layer, point[0], point[1], r, c, antialias)


# Polygon2D

In [ ]:
class Polygon2D:
    """(Regular) Polygon class for a layer"""

    @staticmethod
    def regular_polygon(layer:Layer, x, y, r, n, c, antialias=False):
        """Draw a regular polygon with center (x, y), radius r, division n, color c"""
        for i in range(n):
            Line2D.line(layer, round(x + r * np.cos(2 * np.pi * i / n)), round(y + r * np.sin(2 * np.pi * i / n)), round(x + r * np.cos(2 * np.pi * (i + 1) / n)), round(y + r * np.sin(2 * np.pi * (i + 1) / n)), c, c)

    @staticmethod
    def filled_regular_polygon(layer:Layer, x, y, r, n, c, antialias=False):
        """Draw a filled regular polygon with center (x, y), radius r, division n, color c"""
        for i in range(n):
            Triangle2D.filled_triangle(layer, round(x), round(y), round(x + r * np.cos(2 * np.pi * i / n)), round(y + r * np.sin(2 * np.pi * i / n)), round(x + r * np.cos(2 * np.pi * (i + 1) / n)), round(y + r * np.sin(2 * np.pi * (i + 1) / n)), c, c, c)


# Bezier2D

In [ ]:
class Bezier2D:
    """Bezier curve class for a layer"""
    # https://ja.wikipedia.org/wiki/ベジェ曲線
    # https://en.wikipedia.org/wiki/Bézier_curve

    @staticmethod
    def quadratic_curve(layer:Layer, x0:int, y0:int, x1:int, y1:int, x2:int, y2:int, c, div=50):
        t = np.linspace(0, 1, div)
        x = ((1 - t) ** 2) * x0 + 2 * (1 - t) * t * x1 + (t ** 2) * x2
        y = ((1 - t) ** 2) * y0 + 2 * (1 - t) * t * y1 + (t ** 2) * y2
        layer.draw(x, y, c)

    @staticmethod
    def cubic_curve(layer:Layer, x0:int, y0:int, x1:int, y1:int, x2:int, y2:int, x3:int, y3:int, c, div=50):
        t = np.linspace(0, 1, div)
        x = ((1 - t) ** 3) * x0 + 3 * ((1 - t) ** 2) * t * x1 + (1 - t) * (t ** 2) * x2 + (t ** 3) * x3
        y = ((1 - t) ** 3) * y0 + 3 * ((1 - t) ** 2) * t * y1 + (1 - t) * (t ** 2) * y2 + (t ** 3) * y3
        layer.draw(x, y, c)


# Text2D

In [ ]:
class Text2D:
    """Text rendering class for a layer"""
    # https://ja.wikipedia.org/wiki/フォント
    # https://mplus-fonts.osdn.jp/mplus-bitmap-fonts/index.html
    # https://software.fujitsu.com/jp/manual/manualfiles/m200002/b1wd0741/14z200/b0741-d-06-00.html
    # https://en.wikipedia.org/wiki/Glyph_Bitmap_Distribution_Format
    # https://www.adobe.com/content/dam/acom/en/devnet/font/pdfs/5005.BDF_Spec.pdf
    # https://ja.wikipedia.org/wiki/JIS_X_0208
    # http://www.tohoho-web.com/ex/charset.html

    # char : [BBw, BBh, BBxoff, BByoff, Dwx, BITMAP]
    font_dic = {
        ' ' : [0, 0, 0, 0, 6, []],
        '!' : [1, 9, 1, 0, 4, [128, 128, 128, 128, 128, 128, 128, 0, 128]],
        '"' : [3, 4, 1, 6, 6, [160, 160, 160, 160]],
        '#' : [6, 9, 0, 0, 7, [72, 72, 252, 72, 72, 72, 252, 72, 72]],
        '$' : [6, 10, 0, -1, 7, [16, 124, 144, 144, 120, 36, 36, 36, 248, 32]],
        '%' : [7, 9, 0, 0, 8, [96, 146, 148, 104, 16, 44, 82, 146, 12]],
        '&' : [6, 9, 0, 0, 7, [112, 136, 136, 80, 100, 148, 136, 136, 116]],
        '\'' : [1, 4, 1, 6, 4, [128, 128, 128, 128]],
        '(' : [3, 11, 1, -1, 6, [32, 64, 64, 128, 128, 128, 128, 128, 64, 64, 32]],
        ')' : [3, 11, 1, -1, 6, [128, 64, 64, 32, 32, 32, 32, 32, 64, 64, 128]],
        '*' : [5, 7, 0, 2, 6, [32, 168, 112, 32, 112, 168, 32]],
        '+' : [7, 7, 0, 1, 8, [16, 16, 16, 254, 16, 16, 16]],
        ',' : [2, 2, 1, -1, 6, [64, 128]],
        '-' : [6, 1, 0, 4, 7, [252]],
        '.' : [1, 1, 2, 0, 6, [128]],
        '/' : [5, 10, 1, -1, 8, [8, 8, 16, 16, 32, 32, 64, 64, 128, 128]],
        '0' : [5, 9, 1, 0, 7, [112, 136, 136, 136, 136, 136, 136, 136, 112]],
        '1' : [3, 9, 1, 0, 7, [32, 96, 160, 32, 32, 32, 32, 32, 32]],
        '2' : [5, 9, 1, 0, 7, [112, 136, 136, 8, 16, 32, 64, 128, 248]],
        '3' : [5, 9, 1, 0, 7, [248, 8, 16, 32, 112, 8, 8, 136, 112]],
        '4' : [6, 9, 0, 0, 7, [24, 24, 40, 40, 72, 72, 252, 8, 8]],
        '5' : [5, 9, 1, 0, 7, [248, 128, 128, 240, 8, 8, 136, 136, 112]],
        '6' : [5, 9, 1, 0, 7, [48, 64, 128, 240, 136, 136, 136, 136, 112]],
        '7' : [5, 9, 1, 0, 7, [248, 8, 8, 16, 16, 32, 32, 64, 64]],
        '8' : [5, 9, 1, 0, 7, [112, 136, 136, 136, 112, 136, 136, 136, 112]],
        '9' : [5, 9, 1, 0, 7, [112, 136, 136, 136, 136, 120, 8, 16, 96]],
        ':' : [1, 5, 2, 1, 6, [128, 0, 0, 0, 128]],
        ';' : [2, 6, 1, 0, 6, [64, 0, 0, 0, 64, 128]],
        '<' : [5, 9, 1, 0, 8, [8, 16, 32, 64, 128, 64, 32, 16, 8]],
        '=' : [6, 3, 0, 3, 7, [252, 0, 252]],
        '>' : [5, 9, 1, 0, 8, [128, 64, 32, 16, 8, 16, 32, 64, 128]],
        '?' : [6, 9, 0, 0, 7, [120, 132, 132, 8, 16, 32, 32, 0, 32]],
        '@' : [7, 9, 0, 0, 8, [60, 66, 158, 162, 162, 162, 156, 64, 60]],
        'A' : [7, 9, 0, 0, 8, [16, 16, 40, 40, 68, 68, 124, 130, 130]],
        'B' : [6, 9, 0, 0, 7, [248, 132, 132, 132, 248, 132, 132, 132, 248]],
        'C' : [6, 9, 0, 0, 7, [56, 68, 128, 128, 128, 128, 128, 68, 56]],
        'D' : [7, 9, 0, 0, 8, [248, 132, 130, 130, 130, 130, 130, 132, 248]],
        'E' : [6, 9, 0, 0, 7, [252, 128, 128, 128, 248, 128, 128, 128, 252]],
        'F' : [6, 9, 0, 0, 7, [252, 128, 128, 128, 248, 128, 128, 128, 128]],
        'G' : [7, 9, 0, 0, 8, [60, 66, 128, 128, 142, 130, 130, 70, 58]],
        'H' : [7, 9, 0, 0, 8, [130, 130, 130, 130, 254, 130, 130, 130, 130]],
        'I' : [1, 9, 1, 0, 4, [128, 128, 128, 128, 128, 128, 128, 128, 128]],
        'J' : [6, 9, 0, 0, 7, [4, 4, 4, 4, 4, 4, 132, 136, 112]],
        'K' : [6, 9, 0, 0, 7, [132, 136, 144, 160, 192, 160, 144, 136, 132]],
        'L' : [6, 9, 0, 0, 7, [128, 128, 128, 128, 128, 128, 128, 128, 252]],
        'M' : [9, 9, 0, 0, 10, [32896, 49536, 41600, 38016, 34944, 32896, 32896, 32896, 32896]],
        'N' : [7, 9, 0, 0, 8, [130, 194, 162, 146, 138, 134, 130, 130, 130]],
        'O' : [7, 9, 0, 0, 8, [56, 68, 130, 130, 130, 130, 130, 68, 56]],
        'P' : [6, 9, 0, 0, 7, [248, 132, 132, 132, 248, 128, 128, 128, 128]],
        'Q' : [7, 11, 0, -2, 8, [56, 68, 130, 130, 130, 130, 130, 68, 56, 8, 6]],
        'R' : [6, 9, 0, 0, 7, [248, 132, 132, 132, 248, 160, 144, 136, 132]],
        'S' : [6, 9, 0, 0, 7, [120, 132, 128, 128, 120, 4, 4, 132, 120]],
        'T' : [7, 9, 0, 0, 8, [254, 16, 16, 16, 16, 16, 16, 16, 16]],
        'U' : [7, 9, 0, 0, 8, [130, 130, 130, 130, 130, 130, 130, 68, 56]],
        'V' : [7, 9, 0, 0, 8, [130, 130, 68, 68, 68, 40, 40, 16, 16]],
        'W' : [9, 9, 0, 0, 10, [34944, 34944, 34944, 18688, 21760, 21760, 8704, 8704, 8704]],
        'X' : [6, 9, 0, 0, 7, [132, 132, 72, 72, 48, 72, 72, 132, 132]],
        'Y' : [7, 9, 0, 0, 8, [130, 130, 68, 68, 40, 16, 16, 16, 16]],
        'Z' : [6, 9, 0, 0, 7, [252, 4, 8, 16, 32, 64, 128, 128, 252]],
        '[' : [3, 11, 1, -1, 6, [224, 128, 128, 128, 128, 128, 128, 128, 128, 128, 224]],
        '\\' : [5, 10, 1, -1, 8, [128, 128, 64, 64, 32, 32, 16, 16, 8, 8]],
        ']' : [3, 11, 1, -1, 6, [224, 32, 32, 32, 32, 32, 32, 32, 32, 32, 224]],
        '^' : [5, 3, 0, 7, 6, [32, 80, 136]],
        '_' : [6, 1, 0, -1, 7, [252]],
        '`' : [2, 4, 1, 6, 5, [128, 128, 64, 64]],
        'a' : [5, 6, 0, 0, 6, [112, 8, 120, 136, 136, 120]],
        'b' : [5, 9, 0, 0, 6, [128, 128, 128, 240, 136, 136, 136, 136, 240]],
        'c' : [5, 6, 0, 0, 6, [112, 136, 128, 128, 136, 112]],
        'd' : [5, 9, 0, 0, 6, [8, 8, 8, 120, 136, 136, 136, 136, 120]],
        'e' : [5, 6, 0, 0, 6, [112, 136, 248, 128, 136, 112]],
        'f' : [5, 9, 0, 0, 6, [24, 32, 32, 32, 248, 32, 32, 32, 32]],
        'g' : [5, 9, 0, -3, 6, [120, 136, 136, 136, 136, 120, 8, 136, 112]],
        'h' : [5, 9, 0, 0, 6, [128, 128, 128, 176, 200, 136, 136, 136, 136]],
        'i' : [1, 10, 1, 0, 4, [128, 0, 0, 0, 128, 128, 128, 128, 128, 128]],
        'j' : [4, 13, -1, -3, 5, [16, 0, 0, 0, 16, 16, 16, 16, 16, 16, 16, 32, 192]],
        'k' : [5, 9, 0, 0, 6, [128, 128, 128, 144, 160, 192, 160, 144, 136]],
        'l' : [1, 9, 1, 0, 4, [128, 128, 128, 128, 128, 128, 128, 128, 128]],
        'm' : [7, 6, 0, 0, 8, [252, 146, 146, 146, 146, 146]],
        'n' : [5, 6, 0, 0, 6, [176, 200, 136, 136, 136, 136]],
        'o' : [5, 6, 0, 0, 6, [112, 136, 136, 136, 136, 112]],
        'p' : [5, 9, 0, -3, 6, [240, 136, 136, 136, 136, 240, 128, 128, 128]],
        'q' : [5, 9, 0, -3, 6, [120, 136, 136, 136, 136, 120, 8, 8, 8]],
        'r' : [4, 6, 0, 0, 5, [176, 192, 128, 128, 128, 128]],
        's' : [5, 6, 0, 0, 6, [112, 136, 112, 8, 136, 112]],
        't' : [4, 9, 0, 0, 5, [64, 64, 64, 240, 64, 64, 64, 64, 48]],
        'u' : [5, 6, 0, 0, 6, [136, 136, 136, 136, 152, 104]],
        'v' : [5, 6, 0, 0, 6, [136, 136, 80, 80, 32, 32]],
        'w' : [9, 6, 0, 0, 10, [34944, 34944, 21760, 21760, 8704, 8704]],
        'x' : [5, 6, 0, 0, 6, [136, 80, 32, 32, 80, 136]],
        'y' : [5, 9, 0, -3, 6, [136, 136, 136, 136, 136, 120, 8, 136, 112]],
        'z' : [5, 6, 0, 0, 6, [248, 16, 32, 64, 128, 248]],
        '{' : [5, 11, 1, -1, 8, [24, 32, 32, 32, 32, 192, 32, 32, 32, 32, 24]],
        '|' : [1, 11, 2, -1, 6, [128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128]],
        '}' : [5, 11, 1, -1, 8, [192, 32, 32, 32, 32, 24, 32, 32, 32, 32, 192]],
        '~' : [6, 2, 0, 7, 7, [100, 152]],
        '' : [0, 0, 0, 0, 0, []],
        ' ' : [0, 0, 0, 0, 0, []],
        '¡' : [1, 9, 1, 0, 4, [128, 0, 128, 128, 128, 128, 128, 128, 128]],
        '¢' : [5, 9, 0, 0, 6, [32, 32, 112, 168, 160, 160, 160, 120, 32]],
        '£' : [6, 9, 0, 0, 7, [56, 68, 64, 64, 248, 64, 64, 96, 156]],
        '¤' : [6, 6, 0, 1, 7, [132, 120, 72, 72, 120, 132]],
        '¥' : [7, 9, 0, 0, 8, [130, 68, 40, 16, 254, 16, 254, 16, 16]],
        '¦' : [1, 11, 2, -1, 6, [128, 128, 128, 128, 128, 0, 128, 128, 128, 128, 128]],
        '§' : [5, 11, 0, -1, 6, [112, 136, 128, 96, 144, 136, 72, 48, 8, 136, 112]],
        '¨' : [6, 1, 0, 8, 7, [204]],
        '©' : [9, 9, 0, 0, 10, [15872, 16640, 40064, 41088, 41088, 41088, 40064, 16640, 15872]],
        'ª' : [5, 9, 0, 0, 6, [112, 8, 120, 136, 136, 120, 0, 0, 248]],
        '«' : [8, 9, 0, 0, 9, [9, 18, 36, 72, 144, 72, 36, 18, 9]],
        '¬' : [7, 3, 0, 2, 8, [254, 2, 2]],
        '­' : [6, 1, 0, 4, 7, [252]],
        '®' : [9, 9, 0, 0, 10, [15872, 16640, 48768, 41600, 48256, 41600, 41600, 16640, 15872]],
        '¯' : [6, 1, 0, 8, 7, [252]],
        '°' : [4, 4, 0, 6, 5, [96, 144, 144, 96]],
        '±' : [7, 9, 0, 0, 8, [16, 16, 16, 254, 16, 16, 16, 0, 254]],
        '²' : [4, 6, 0, 4, 5, [96, 144, 16, 32, 64, 240]],
        '³' : [4, 6, 0, 4, 5, [240, 32, 96, 16, 144, 96]],
        '´' : [3, 3, 1, 8, 6, [32, 64, 128]],
        'µ' : [5, 9, 0, -3, 6, [136, 136, 136, 136, 152, 232, 128, 128, 128]],
        '¶' : [6, 9, 0, 0, 7, [60, 116, 244, 244, 116, 52, 20, 20, 20]],
        '·' : [1, 1, 2, 4, 6, [128]],
        '¸' : [3, 3, 1, -2, 6, [32, 64, 128]],
        '¹' : [3, 6, 0, 4, 5, [32, 96, 160, 32, 32, 32]],
        'º' : [5, 8, 0, 2, 5, [112, 136, 136, 136, 112, 0, 0, 248]],
        '»' : [8, 9, 0, 0, 9, [144, 72, 36, 18, 9, 18, 36, 72, 144]],
        '¼' : [7, 9, 0, 0, 8, [64, 194, 68, 72, 16, 44, 84, 158, 4]],
        '½' : [7, 9, 0, 0, 8, [64, 194, 68, 72, 16, 44, 66, 132, 14]],
        '¾' : [7, 9, 0, 0, 8, [224, 66, 36, 200, 16, 44, 84, 158, 4]],
        '¿' : [5, 9, 0, 0, 6, [32, 0, 32, 32, 64, 128, 136, 136, 112]],
        'À' : [7, 11, 0, 0, 8, [16, 8, 0, 16, 16, 40, 40, 68, 124, 130, 130]],
        'Á' : [7, 11, 0, 0, 8, [8, 16, 0, 16, 16, 40, 40, 68, 124, 130, 130]],
        'Â' : [7, 11, 0, 0, 8, [24, 36, 0, 16, 16, 40, 40, 68, 124, 130, 130]],
        'Ã' : [7, 11, 0, 0, 8, [50, 76, 0, 16, 16, 40, 40, 68, 124, 130, 130]],
        'Ä' : [7, 11, 0, 0, 8, [36, 36, 0, 16, 16, 40, 40, 68, 124, 130, 130]],
        'Å' : [7, 11, 0, 0, 8, [24, 36, 36, 24, 16, 40, 40, 68, 124, 130, 130]],
        'Æ' : [8, 9, 0, 0, 9, [63, 40, 40, 72, 79, 120, 136, 136, 143]],
        'Ç' : [6, 11, 0, -2, 7, [56, 68, 128, 128, 128, 128, 128, 68, 56, 16, 32]],
        'È' : [6, 11, 0, 0, 7, [32, 16, 0, 252, 128, 128, 248, 128, 128, 128, 252]],
        'É' : [6, 11, 0, 0, 7, [16, 32, 0, 252, 128, 128, 248, 128, 128, 128, 252]],
        'Ê' : [6, 11, 0, 0, 7, [48, 72, 0, 252, 128, 128, 248, 128, 128, 128, 252]],
        'Ë' : [6, 11, 0, 0, 7, [72, 72, 0, 252, 128, 128, 248, 128, 128, 128, 252]],
        'Ì' : [2, 11, 1, 0, 4, [128, 64, 0, 128, 128, 128, 128, 128, 128, 128, 128]],
        'Í' : [2, 11, 1, 0, 4, [64, 128, 0, 128, 128, 128, 128, 128, 128, 128, 128]],
        'Î' : [4, 11, 0, 0, 4, [96, 144, 0, 64, 64, 64, 64, 64, 64, 64, 64]],
        'Ï' : [4, 11, 0, 0, 4, [144, 144, 0, 64, 64, 64, 64, 64, 64, 64, 64]],
        'Ð' : [8, 9, 0, 0, 9, [124, 66, 65, 65, 249, 65, 65, 66, 124]],
        'Ñ' : [7, 11, 0, 0, 8, [50, 76, 0, 130, 194, 162, 146, 138, 134, 130, 130]],
        'Ò' : [7, 11, 0, 0, 8, [16, 8, 0, 56, 68, 130, 130, 130, 130, 68, 56]],
        'Ó' : [7, 11, 0, 0, 8, [8, 16, 0, 56, 68, 130, 130, 130, 130, 68, 56]],
        'Ô' : [7, 11, 0, 0, 8, [24, 36, 0, 56, 68, 130, 130, 130, 130, 68, 56]],
        'Õ' : [7, 11, 0, 0, 8, [50, 76, 0, 56, 68, 130, 130, 130, 130, 68, 56]],
        'Ö' : [7, 11, 0, 0, 8, [36, 36, 0, 56, 68, 130, 130, 130, 130, 68, 56]],
        '×' : [7, 7, 0, 1, 8, [130, 68, 40, 16, 40, 68, 130]],
        'Ø' : [9, 11, -1, -1, 9, [128, 7424, 8704, 17152, 17664, 18688, 20736, 24832, 8704, 23552, 32768]],
        'Ù' : [7, 11, 0, 0, 8, [16, 8, 0, 130, 130, 130, 130, 130, 130, 68, 56]],
        'Ú' : [7, 11, 0, 0, 8, [8, 16, 0, 130, 130, 130, 130, 130, 130, 68, 56]],
        'Û' : [7, 11, 0, 0, 8, [24, 36, 0, 130, 130, 130, 130, 130, 130, 68, 56]],
        'Ü' : [7, 11, 0, 0, 8, [36, 36, 0, 130, 130, 130, 130, 130, 130, 68, 56]],
        'Ý' : [7, 11, 0, 0, 8, [8, 16, 0, 130, 130, 68, 40, 16, 16, 16, 16]],
        'Þ' : [6, 10, 0, 0, 7, [128, 128, 248, 132, 132, 132, 248, 128, 128, 128]],
        'ß' : [5, 11, 0, -3, 6, [112, 136, 136, 240, 136, 136, 136, 240, 128, 128, 128]],
        'à' : [5, 11, 0, 0, 6, [32, 16, 0, 0, 0, 112, 8, 120, 136, 136, 120]],
        'á' : [5, 11, 0, 0, 6, [16, 32, 0, 0, 0, 112, 8, 120, 136, 136, 120]],
        'â' : [5, 11, 0, 0, 6, [48, 72, 0, 0, 0, 112, 8, 120, 136, 136, 120]],
        'ã' : [6, 11, 0, 0, 6, [100, 152, 0, 0, 0, 112, 8, 120, 136, 136, 120]],
        'ä' : [5, 11, 0, 0, 6, [72, 72, 0, 0, 0, 112, 8, 120, 136, 136, 120]],
        'å' : [5, 11, 0, 0, 6, [48, 72, 72, 48, 0, 112, 8, 120, 136, 136, 120]],
        'æ' : [9, 6, 0, 0, 10, [32512, 2176, 32640, 34816, 34944, 30464]],
        'ç' : [5, 8, 0, -2, 6, [112, 136, 128, 128, 136, 112, 32, 64]],
        'è' : [5, 11, 0, 0, 6, [32, 16, 0, 0, 0, 112, 136, 248, 128, 136, 112]],
        'é' : [5, 11, 0, 0, 6, [16, 32, 0, 0, 0, 112, 136, 248, 128, 136, 112]],
        'ê' : [5, 11, 0, 0, 6, [48, 72, 0, 0, 0, 112, 136, 248, 128, 136, 112]],
        'ë' : [5, 11, 0, 0, 6, [72, 72, 0, 0, 0, 112, 136, 248, 128, 136, 112]],
        'ì' : [2, 11, 1, 0, 4, [128, 64, 0, 0, 0, 128, 128, 128, 128, 128, 128]],
        'í' : [2, 11, 1, 0, 4, [64, 128, 0, 0, 0, 128, 128, 128, 128, 128, 128]],
        'î' : [4, 11, 0, 0, 4, [96, 144, 0, 0, 0, 64, 64, 64, 64, 64, 64]],
        'ï' : [4, 11, 0, 0, 4, [144, 144, 0, 0, 0, 64, 64, 64, 64, 64, 64]],
        'ð' : [5, 10, 0, 0, 6, [192, 56, 224, 16, 112, 136, 136, 136, 136, 112]],
        'ñ' : [6, 11, 0, 0, 6, [100, 152, 0, 0, 0, 176, 200, 136, 136, 136, 136]],
        'ò' : [5, 11, 0, 0, 6, [32, 16, 0, 0, 0, 112, 136, 136, 136, 136, 112]],
        'ó' : [5, 11, 0, 0, 6, [16, 32, 0, 0, 0, 112, 136, 136, 136, 136, 112]],
        'ô' : [5, 11, 0, 0, 6, [48, 72, 0, 0, 0, 112, 136, 136, 136, 136, 112]],
        'õ' : [6, 11, 0, 0, 6, [100, 152, 0, 0, 0, 112, 136, 136, 136, 136, 112]],
        'ö' : [5, 11, 0, 0, 6, [72, 72, 0, 0, 0, 112, 136, 136, 136, 136, 112]],
        '÷' : [7, 9, 0, 0, 8, [16, 16, 0, 0, 254, 0, 0, 16, 16]],
        'ø' : [5, 8, 0, -1, 6, [8, 112, 152, 168, 168, 200, 112, 128]],
        'ù' : [5, 11, 0, 0, 6, [32, 16, 0, 0, 0, 136, 136, 136, 136, 152, 104]],
        'ú' : [5, 11, 0, 0, 6, [16, 32, 0, 0, 0, 136, 136, 136, 136, 152, 104]],
        'û' : [5, 11, 0, 0, 6, [48, 72, 0, 0, 0, 136, 136, 136, 136, 152, 104]],
        'ü' : [5, 11, 0, 0, 6, [72, 72, 0, 0, 0, 136, 136, 136, 136, 152, 104]],
        'ý' : [5, 14, 0, -3, 6, [16, 32, 0, 0, 0, 136, 136, 136, 136, 136, 120, 8, 136, 112]],
        'þ' : [5, 11, 0, -3, 6, [128, 128, 240, 136, 136, 136, 136, 240, 128, 128, 128]],
        'ÿ' : [5, 14, 0, -3, 6, [72, 72, 0, 0, 0, 136, 136, 136, 136, 136, 120, 8, 136, 112]],
        '　' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '、' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16384, 8192, 0]],
        '。' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 0, 0, 12288, 18432, 18432, 12288, 0]],
        '，' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 0, 0, 0, 12288, 12288, 24576, 0]],
        '．' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 0, 0, 0, 12288, 12288, 0, 0]],
        '・' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 3072, 3072, 0, 0, 0, 0, 0]],
        '：' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 3072, 3072, 0, 0, 3072, 3072, 0, 0, 0]],
        '；' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 3072, 3072, 0, 0, 3072, 3072, 6144, 0, 0]],
        '？' : [12, 13, 0, -2, 12, [0, 3584, 4352, 8320, 128, 256, 512, 1024, 1024, 0, 1024, 0, 0]],
        '！' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 0, 0, 1024, 0, 0]],
        '゛' : [12, 13, 0, -2, 12, [4096, 18432, 8192, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '゜' : [12, 13, 0, -2, 12, [12288, 18432, 18432, 12288, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '´' : [12, 13, 0, -2, 12, [1024, 2048, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '｀' : [12, 13, 0, -2, 12, [2048, 1024, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '¨' : [12, 13, 0, -2, 12, [6528, 6528, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '＾' : [12, 13, 0, -2, 12, [1024, 2560, 4352, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '￣' : [12, 13, 0, -2, 12, [65520, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '＿' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 65520, 0]],
        'ヽ' : [12, 13, 0, -2, 12, [0, 0, 0, 4096, 2048, 1024, 1024, 512, 512, 0, 0, 0, 0]],
        'ヾ' : [12, 13, 0, -2, 12, [0, 128, 576, 8448, 4096, 2048, 2048, 1024, 1024, 0, 0, 0, 0]],
        'ゝ' : [12, 13, 0, -2, 12, [0, 0, 0, 2048, 1024, 512, 512, 256, 3840, 0, 0, 0, 0]],
        'ゞ' : [12, 13, 0, -2, 12, [0, 128, 576, 4352, 2048, 1024, 1024, 512, 7680, 0, 0, 0, 0]],
        '〃' : [12, 13, 0, -2, 12, [0, 0, 0, 2304, 2304, 2304, 4608, 4608, 4608, 0, 0, 0, 0]],
        '仝' : [12, 13, 0, -2, 12, [0, 3584, 4352, 8320, 16448, 32800, 16256, 1024, 1024, 1024, 1024, 32704, 0]],
        '々' : [12, 13, 0, -2, 12, [0, 0, 4096, 4096, 8064, 8320, 8320, 256, 2560, 1024, 512, 0, 0]],
        '〆' : [12, 13, 0, -2, 12, [0, 0, 64, 128, 7424, 8704, 17664, 18560, 20544, 24576, 16384, 0, 0]],
        '〇' : [12, 13, 0, -2, 12, [0, 0, 3584, 12672, 16448, 32800, 32800, 32800, 16448, 12672, 3584, 0, 0]],
        'ー' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 16384, 16320, 0, 0, 0, 0, 0, 0]],
        '―' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 65520, 0, 0, 0, 0, 0, 0]],
        '‐' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 3584, 0, 0, 0, 0, 0, 0]],
        '／' : [12, 13, 0, -2, 12, [0, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768, 0]],
        '＼' : [12, 13, 0, -2, 12, [0, 32768, 16384, 8192, 4096, 2048, 1024, 512, 256, 128, 64, 32, 0]],
        '〜' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 14336, 17472, 896, 0, 0, 0, 0, 0]],
        '‖' : [12, 13, 0, -2, 12, [2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560]],
        '｜' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024]],
        '…' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 26208, 26208, 0, 0, 0, 0, 0]],
        '‥' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 12672, 12672, 0, 0, 0, 0, 0]],
        '‘' : [12, 13, 0, -2, 12, [192, 384, 384, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '’' : [12, 13, 0, -2, 12, [12288, 12288, 24576, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '“' : [12, 13, 0, -2, 12, [1728, 3456, 3456, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '”' : [12, 13, 0, -2, 12, [13824, 13824, 27648, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '（' : [12, 13, 0, -2, 12, [64, 128, 256, 256, 512, 512, 512, 512, 512, 256, 256, 128, 64]],
        '）' : [12, 13, 0, -2, 12, [16384, 8192, 4096, 4096, 2048, 2048, 2048, 2048, 2048, 4096, 4096, 8192, 16384]],
        '〔' : [12, 13, 0, -2, 12, [64, 128, 256, 256, 256, 256, 256, 256, 256, 256, 256, 128, 64]],
        '〕' : [12, 13, 0, -2, 12, [16384, 8192, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 8192, 16384]],
        '［' : [12, 13, 0, -2, 12, [960, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 960]],
        '］' : [12, 13, 0, -2, 12, [30720, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 30720]],
        '｛' : [12, 13, 0, -2, 12, [192, 256, 256, 256, 256, 256, 1536, 256, 256, 256, 256, 256, 192]],
        '｝' : [12, 13, 0, -2, 12, [24576, 4096, 4096, 4096, 4096, 4096, 3072, 4096, 4096, 4096, 4096, 4096, 24576]],
        '〈' : [12, 13, 0, -2, 12, [0, 64, 64, 128, 128, 256, 256, 256, 128, 128, 64, 64, 0]],
        '〉' : [12, 13, 0, -2, 12, [0, 16384, 16384, 8192, 8192, 4096, 4096, 4096, 8192, 8192, 16384, 16384, 0]],
        '《' : [12, 13, 0, -2, 12, [0, 288, 288, 576, 576, 1152, 1152, 1152, 576, 576, 288, 288, 0]],
        '》' : [12, 13, 0, -2, 12, [0, 36864, 36864, 18432, 18432, 9216, 9216, 9216, 18432, 18432, 36864, 36864, 0]],
        '「' : [12, 13, 0, -2, 12, [960, 512, 512, 512, 512, 512, 512, 512, 0, 0, 0, 0, 0]],
        '」' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 30720]],
        '『' : [12, 13, 0, -2, 12, [4032, 2112, 3008, 2560, 2560, 2560, 2560, 2560, 3584, 0, 0, 0, 0]],
        '』' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 3584, 2560, 2560, 2560, 2560, 2560, 31232, 16896, 32256]],
        '【' : [12, 13, 0, -2, 12, [960, 896, 768, 768, 512, 512, 512, 512, 512, 768, 768, 896, 960]],
        '】' : [12, 13, 0, -2, 12, [30720, 14336, 6144, 6144, 2048, 2048, 2048, 2048, 2048, 6144, 6144, 14336, 30720]],
        '＋' : [12, 13, 0, -2, 12, [0, 0, 1024, 1024, 1024, 1024, 32704, 1024, 1024, 1024, 1024, 0, 0]],
        '−' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 32704, 0, 0, 0, 0, 0, 0]],
        '±' : [12, 13, 0, -2, 12, [0, 0, 1024, 1024, 1024, 16256, 1024, 1024, 1024, 0, 16256, 0, 0]],
        '×' : [12, 13, 0, -2, 12, [0, 0, 0, 8320, 4352, 2560, 1024, 2560, 4352, 8320, 0, 0, 0]],
        '÷' : [12, 13, 0, -2, 12, [0, 0, 1024, 3584, 1024, 0, 32704, 0, 1024, 3584, 1024, 0, 0]],
        '＝' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 32704, 0, 0, 32704, 0, 0, 0, 0]],
        '≠' : [12, 13, 0, -2, 12, [0, 0, 0, 128, 256, 32704, 1024, 2048, 32704, 8192, 16384, 0, 0]],
        '＜' : [12, 13, 0, -2, 12, [0, 0, 192, 768, 3072, 12288, 49152, 12288, 3072, 768, 192, 0, 0]],
        '＞' : [12, 13, 0, -2, 12, [0, 0, 24576, 6144, 1536, 384, 96, 384, 1536, 6144, 24576, 0, 0]],
        '≦' : [12, 13, 0, -2, 12, [0, 0, 448, 3584, 28672, 3584, 448, 0, 32704, 0, 32704, 0, 0]],
        '≧' : [12, 13, 0, -2, 12, [0, 0, 28672, 3584, 448, 3584, 28672, 0, 32704, 0, 32704, 0, 0]],
        '∞' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 12672, 19008, 33824, 33824, 19008, 12672, 0, 0, 0]],
        '∴' : [12, 13, 0, -2, 12, [0, 0, 1024, 3584, 1024, 0, 0, 8320, 29120, 8320, 0, 0, 0]],
        '♂' : [12, 13, 0, -2, 12, [0, 0, 1984, 192, 320, 576, 15424, 17408, 17408, 17408, 14336, 0, 0]],
        '♀' : [12, 13, 0, -2, 12, [0, 3584, 4352, 4352, 4352, 3584, 1024, 1024, 16256, 1024, 1024, 0, 0]],
        '°' : [12, 13, 0, -2, 12, [0, 12288, 18432, 18432, 12288, 0, 0, 0, 0, 0, 0, 0, 0]],
        '′' : [12, 13, 0, -2, 12, [0, 4096, 8192, 16384, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '″' : [12, 13, 0, -2, 12, [0, 4608, 9216, 18432, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '℃' : [12, 13, 0, -2, 12, [12288, 18432, 18432, 13184, 1088, 2048, 2048, 2048, 2048, 1088, 896, 0, 0]],
        '￥' : [12, 13, 0, -2, 12, [0, 8320, 8320, 4352, 4352, 16256, 1024, 1024, 16256, 1024, 1024, 0, 0]],
        '＄' : [12, 13, 0, -2, 12, [0, 1024, 7936, 9344, 9216, 7936, 1152, 1152, 9344, 7936, 1024, 0, 0]],
        '¢' : [12, 13, 0, -2, 12, [0, 128, 128, 1792, 2432, 4608, 4608, 4608, 3200, 1792, 2048, 2048, 0]],
        '£' : [12, 13, 0, -2, 12, [0, 1792, 2176, 2048, 2048, 32512, 2048, 14336, 18432, 19520, 13184, 0, 0]],
        '％' : [12, 13, 0, -2, 12, [0, 12544, 18688, 18944, 18944, 13312, 1408, 2624, 2624, 4672, 4480, 0, 0]],
        '＃' : [12, 13, 0, -2, 12, [0, 2304, 2304, 2304, 16320, 2304, 2304, 16320, 2304, 2304, 2304, 0, 0]],
        '＆' : [12, 13, 0, -2, 12, [0, 3072, 4608, 4608, 4608, 3520, 5248, 8832, 8448, 8832, 7232, 0, 0]],
        '＊' : [12, 13, 0, -2, 12, [0, 1024, 9344, 5376, 3584, 5376, 9344, 1024, 0, 0, 0, 0, 0]],
        '＠' : [12, 13, 0, -2, 12, [0, 3840, 12416, 9792, 18752, 18752, 18752, 18752, 10112, 12288, 3584, 0, 0]],
        '§' : [12, 13, 0, -2, 12, [3584, 4352, 4352, 2048, 3584, 4352, 4352, 3584, 512, 4352, 4352, 3584, 0]],
        '☆' : [12, 13, 0, -2, 12, [1024, 1024, 2560, 2560, 64480, 8320, 4352, 5376, 10880, 12672, 16448, 0, 0]],
        '★' : [12, 13, 0, -2, 12, [1024, 1024, 3584, 3584, 65504, 16256, 7936, 7936, 15232, 12672, 16448, 0, 0]],
        '○' : [12, 13, 0, -2, 12, [0, 1536, 6528, 8256, 8256, 16416, 16416, 8256, 8256, 6528, 1536, 0, 0]],
        '●' : [12, 13, 0, -2, 12, [0, 1536, 8064, 16320, 16320, 32736, 32736, 16320, 16320, 8064, 1536, 0, 0]],
        '◎' : [12, 13, 0, -2, 12, [0, 1536, 6528, 8256, 9792, 18720, 18720, 9792, 8256, 6528, 1536, 0, 0]],
        '◇' : [12, 13, 0, -2, 12, [0, 1024, 2560, 4352, 8320, 16448, 32800, 16448, 8320, 4352, 2560, 1024, 0]],
        '◆' : [12, 13, 0, -2, 12, [0, 1024, 3584, 7936, 16256, 32704, 65504, 32704, 16256, 7936, 3584, 1024, 0]],
        '□' : [12, 13, 0, -2, 12, [0, 0, 32704, 16448, 16448, 16448, 16448, 16448, 16448, 16448, 32704, 0, 0]],
        '■' : [12, 13, 0, -2, 12, [0, 0, 32704, 32704, 32704, 32704, 32704, 32704, 32704, 32704, 32704, 0, 0]],
        '△' : [12, 13, 0, -2, 12, [0, 0, 1024, 2560, 2560, 4352, 4352, 8320, 8320, 16448, 32704, 0, 0]],
        '▲' : [12, 13, 0, -2, 12, [0, 0, 1024, 3584, 3584, 7936, 7936, 16256, 16256, 32704, 32704, 0, 0]],
        '▽' : [12, 13, 0, -2, 12, [0, 0, 32704, 16448, 8320, 8320, 4352, 4352, 2560, 2560, 1024, 0, 0]],
        '▼' : [12, 13, 0, -2, 12, [0, 0, 32704, 32704, 16256, 16256, 7936, 7936, 3584, 3584, 1024, 0, 0]],
        '※' : [12, 13, 0, -2, 12, [0, 1024, 20032, 9344, 4352, 19008, 58592, 19008, 4352, 9344, 20032, 1024, 0]],
        '〒' : [12, 13, 0, -2, 12, [0, 32704, 0, 0, 0, 32704, 1024, 1024, 1024, 1024, 1024, 0, 0]],
        '→' : [12, 13, 0, -2, 12, [0, 0, 512, 256, 128, 64, 65504, 64, 128, 256, 512, 0, 0]],
        '←' : [12, 13, 0, -2, 12, [0, 0, 2048, 4096, 8192, 16384, 65504, 16384, 8192, 4096, 2048, 0, 0]],
        '↑' : [12, 13, 0, -2, 12, [0, 1024, 3584, 5376, 9344, 17472, 1024, 1024, 1024, 1024, 1024, 1024, 0]],
        '↓' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1024, 1024, 1024, 1024, 17472, 9344, 5376, 3584, 1024, 0]],
        '〓' : [12, 13, 0, -2, 12, [0, 0, 0, 32704, 32704, 32704, 0, 32704, 32704, 32704, 0, 0, 0]],
        '∈' : [12, 13, 0, -2, 12, [0, 0, 0, 8128, 8192, 16384, 32704, 16384, 8192, 8128, 0, 0, 0]],
        '∋' : [12, 13, 0, -2, 12, [0, 0, 0, 32512, 128, 64, 32704, 64, 128, 32512, 0, 0, 0]],
        '⊆' : [12, 13, 0, -2, 12, [0, 0, 8128, 8192, 16384, 16384, 8192, 8128, 0, 0, 32704, 0, 0]],
        '⊇' : [12, 13, 0, -2, 12, [0, 0, 32512, 128, 64, 64, 128, 32512, 0, 0, 32704, 0, 0]],
        '⊂' : [12, 13, 0, -2, 12, [0, 0, 0, 8128, 8192, 16384, 16384, 16384, 8192, 8128, 0, 0, 0]],
        '⊃' : [12, 13, 0, -2, 12, [0, 0, 0, 32512, 128, 64, 64, 64, 128, 32512, 0, 0, 0]],
        '∪' : [12, 13, 0, -2, 12, [0, 0, 8320, 8320, 8320, 8320, 8320, 8320, 8320, 4352, 3584, 0, 0]],
        '∩' : [12, 13, 0, -2, 12, [0, 0, 3584, 4352, 8320, 8320, 8320, 8320, 8320, 8320, 8320, 0, 0]],
        '∧' : [12, 13, 0, -2, 12, [0, 1024, 1024, 2560, 2560, 4352, 4352, 8320, 8320, 16448, 16448, 0, 0]],
        '∨' : [12, 13, 0, -2, 12, [0, 16448, 16448, 8320, 8320, 4352, 4352, 2560, 2560, 1024, 1024, 0, 0]],
        '¬' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 32704, 64, 64, 0, 0, 0, 0]],
        '⇒' : [12, 13, 0, -2, 12, [0, 0, 0, 256, 128, 65472, 32, 65472, 128, 256, 0, 0, 0]],
        '⇔' : [12, 13, 0, -2, 12, [0, 0, 0, 4352, 8320, 32704, 32800, 32704, 8320, 4352, 0, 0, 0]],
        '∀' : [12, 13, 0, -2, 12, [0, 16448, 16448, 8320, 16256, 4352, 4352, 2560, 2560, 1024, 1024, 0, 0]],
        '∃' : [12, 13, 0, -2, 12, [0, 16256, 128, 128, 128, 16256, 128, 128, 128, 128, 16256, 0, 0]],
        '∠' : [12, 13, 0, -2, 12, [0, 0, 128, 256, 512, 1024, 2048, 4096, 8192, 32704, 0, 0, 0]],
        '⊥' : [12, 13, 0, -2, 12, [0, 0, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 32704, 0, 0, 0]],
        '⌒' : [12, 13, 0, -2, 12, [7936, 24768, 32800, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '∂' : [12, 13, 0, -2, 12, [0, 0, 1792, 2176, 128, 128, 3712, 4352, 8448, 8704, 7168, 0, 0]],
        '∇' : [12, 13, 0, -2, 12, [0, 0, 32704, 32704, 12416, 12416, 6400, 6400, 2560, 3584, 1024, 0, 0]],
        '≡' : [12, 13, 0, -2, 12, [0, 0, 0, 32704, 0, 0, 32704, 0, 0, 32704, 0, 0, 0]],
        '≒' : [12, 13, 0, -2, 12, [0, 0, 12288, 12288, 0, 32704, 0, 32704, 0, 384, 384, 0, 0]],
        '≪' : [12, 13, 0, -2, 12, [0, 0, 1088, 2176, 4352, 8704, 17408, 8704, 4352, 2176, 1088, 0, 0]],
        '≫' : [12, 13, 0, -2, 12, [0, 0, 34816, 17408, 8704, 4352, 2176, 4352, 8704, 17408, 34816, 0, 0]],
        '√' : [12, 13, 0, -2, 12, [992, 512, 512, 1024, 1024, 17408, 18432, 43008, 43008, 4096, 4096, 0, 0]],
        '∽' : [12, 13, 0, -2, 12, [0, 0, 0, 8576, 16960, 33824, 33824, 18496, 12416, 0, 0, 0, 0]],
        '∝' : [12, 13, 0, -2, 12, [0, 0, 0, 12672, 19008, 33792, 33792, 19008, 12672, 0, 0, 0, 0]],
        '∵' : [12, 13, 0, -2, 12, [0, 0, 8320, 29120, 8320, 0, 0, 1024, 3584, 1024, 0, 0, 0]],
        '∫' : [12, 13, 0, -2, 12, [1536, 2560, 2048, 2048, 2048, 1024, 1024, 512, 512, 512, 2560, 3072, 0]],
        '∬' : [12, 13, 0, -2, 12, [6528, 10880, 8704, 8704, 8704, 4352, 4352, 2176, 2176, 2176, 10880, 13056, 0]],
        'Å' : [12, 13, 0, -2, 12, [1536, 2304, 2304, 1536, 1024, 2560, 2560, 4352, 7936, 8320, 8320, 0, 0]],
        '‰' : [12, 13, 0, -2, 12, [0, 25088, 37376, 37888, 37888, 26624, 3776, 6432, 6432, 10528, 9920, 0, 0]],
        '♯' : [12, 13, 0, -2, 12, [0, 2304, 2304, 2496, 3840, 14592, 2496, 3840, 14592, 2304, 2304, 0, 0]],
        '♭' : [12, 13, 0, -2, 12, [0, 4096, 4096, 4096, 5632, 6400, 4352, 4352, 4608, 5120, 6144, 0, 0]],
        '♪' : [12, 13, 0, -2, 12, [0, 1024, 1536, 1280, 1152, 1152, 1280, 1024, 7168, 15360, 14336, 0, 0]],
        '†' : [12, 13, 0, -2, 12, [1024, 1024, 7936, 1024, 1024, 3584, 3584, 3584, 1024, 1024, 1024, 1024, 0]],
        '‡' : [12, 13, 0, -2, 12, [1024, 1024, 7936, 1024, 3584, 1024, 1024, 3584, 1024, 7936, 1024, 1024, 0]],
        '¶' : [12, 13, 0, -2, 12, [0, 3840, 7424, 15616, 15616, 7424, 3328, 1280, 1280, 1280, 1280, 0, 0]],
        '◯' : [12, 13, 0, -2, 12, [0, 3584, 12672, 16448, 16448, 32800, 32800, 32800, 16448, 16448, 12672, 3584, 0]],
        '０' : [12, 13, 0, -2, 12, [0, 3584, 4352, 8320, 8320, 8320, 8320, 8320, 8320, 4352, 3584, 0, 0]],
        '１' : [12, 13, 0, -2, 12, [0, 1024, 3072, 5120, 9216, 1024, 1024, 1024, 1024, 1024, 1024, 0, 0]],
        '２' : [12, 13, 0, -2, 12, [0, 3584, 4352, 8320, 128, 256, 512, 1024, 2048, 4096, 16256, 0, 0]],
        '３' : [12, 13, 0, -2, 12, [0, 16256, 256, 512, 1024, 3584, 256, 128, 8320, 4352, 3584, 0, 0]],
        '４' : [12, 13, 0, -2, 12, [0, 1536, 2560, 2560, 4608, 4608, 8704, 8704, 32640, 512, 512, 0, 0]],
        '５' : [12, 13, 0, -2, 12, [0, 16256, 8192, 8192, 11776, 12544, 128, 128, 128, 8448, 7680, 0, 0]],
        '６' : [12, 13, 0, -2, 12, [0, 3840, 4096, 8192, 11776, 12544, 8320, 8320, 8320, 4352, 3584, 0, 0]],
        '７' : [12, 13, 0, -2, 12, [0, 16256, 128, 256, 256, 512, 512, 1024, 1024, 2048, 2048, 0, 0]],
        '８' : [12, 13, 0, -2, 12, [0, 3584, 4352, 8320, 4352, 3584, 4352, 8320, 8320, 4352, 3584, 0, 0]],
        '９' : [12, 13, 0, -2, 12, [0, 3584, 4352, 8320, 8320, 8320, 4480, 3712, 128, 256, 7680, 0, 0]],
        'Ａ' : [12, 13, 0, -2, 12, [0, 1024, 1024, 2560, 2560, 4352, 4352, 16256, 8320, 16448, 16448, 0, 0]],
        'Ｂ' : [12, 13, 0, -2, 12, [0, 16128, 8320, 8320, 8320, 16128, 8320, 8256, 8256, 8320, 16128, 0, 0]],
        'Ｃ' : [12, 13, 0, -2, 12, [0, 3968, 4160, 8192, 8192, 8192, 8192, 8192, 8192, 4160, 3968, 0, 0]],
        'Ｄ' : [12, 13, 0, -2, 12, [0, 15872, 8576, 8320, 8256, 8256, 8256, 8256, 8320, 8576, 15872, 0, 0]],
        'Ｅ' : [12, 13, 0, -2, 12, [0, 16256, 8192, 8192, 8192, 16128, 8192, 8192, 8192, 8192, 16256, 0, 0]],
        'Ｆ' : [12, 13, 0, -2, 12, [0, 16256, 8192, 8192, 8192, 16128, 8192, 8192, 8192, 8192, 8192, 0, 0]],
        'Ｇ' : [12, 13, 0, -2, 12, [0, 3968, 4160, 8192, 8192, 9152, 8256, 8256, 8256, 4288, 3904, 0, 0]],
        'Ｈ' : [12, 13, 0, -2, 12, [0, 8256, 8256, 8256, 8256, 16320, 8256, 8256, 8256, 8256, 8256, 0, 0]],
        'Ｉ' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 0, 0]],
        'Ｊ' : [12, 13, 0, -2, 12, [0, 256, 256, 256, 256, 256, 16640, 16640, 16640, 8704, 7168, 0, 0]],
        'Ｋ' : [12, 13, 0, -2, 12, [0, 8320, 8448, 8704, 9216, 10240, 13312, 8704, 8448, 8320, 8256, 0, 0]],
        'Ｌ' : [12, 13, 0, -2, 12, [0, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 16256, 0, 0]],
        'Ｍ' : [12, 13, 0, -2, 12, [0, 16448, 24768, 20800, 19008, 17472, 16448, 16448, 16448, 16448, 16448, 0, 0]],
        'Ｎ' : [12, 13, 0, -2, 12, [0, 8320, 12416, 10368, 9344, 8832, 8576, 8320, 8320, 8320, 8320, 0, 0]],
        'Ｏ' : [12, 13, 0, -2, 12, [0, 3840, 4224, 8256, 8256, 8256, 8256, 8256, 8256, 4224, 3840, 0, 0]],
        'Ｐ' : [12, 13, 0, -2, 12, [0, 15872, 8448, 8320, 8320, 8448, 15872, 8192, 8192, 8192, 8192, 0, 0]],
        'Ｑ' : [12, 13, 0, -2, 12, [0, 3840, 4224, 8256, 8256, 8256, 8256, 8256, 8256, 4224, 3840, 512, 448]],
        'Ｒ' : [12, 13, 0, -2, 12, [0, 15872, 8448, 8320, 8320, 8448, 15872, 8448, 8448, 8320, 8320, 0, 0]],
        'Ｓ' : [12, 13, 0, -2, 12, [0, 3840, 4224, 8256, 4096, 3840, 128, 64, 8256, 4224, 3840, 0, 0]],
        'Ｔ' : [12, 13, 0, -2, 12, [0, 32704, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 0, 0]],
        'Ｕ' : [12, 13, 0, -2, 12, [0, 8320, 8320, 8320, 8320, 8320, 8320, 8320, 8320, 4352, 3584, 0, 0]],
        'Ｖ' : [12, 13, 0, -2, 12, [0, 8320, 8320, 8320, 4352, 4352, 4352, 2560, 2560, 1024, 1024, 0, 0]],
        'Ｗ' : [12, 13, 0, -2, 12, [0, 17472, 17472, 17472, 17472, 10880, 10880, 10880, 4352, 4352, 4352, 0, 0]],
        'Ｘ' : [12, 13, 0, -2, 12, [0, 8320, 8320, 4352, 2560, 1024, 1024, 2560, 4352, 8320, 8320, 0, 0]],
        'Ｙ' : [12, 13, 0, -2, 12, [0, 8320, 8320, 4352, 4352, 2560, 2560, 1024, 1024, 1024, 1024, 0, 0]],
        'Ｚ' : [12, 13, 0, -2, 12, [0, 16256, 128, 256, 512, 1024, 2048, 4096, 8192, 8192, 16256, 0, 0]],
        'ａ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 3840, 128, 1920, 6272, 4224, 4480, 3712, 0, 0]],
        'ｂ' : [12, 13, 0, -2, 12, [0, 4096, 4096, 4096, 5632, 6400, 4224, 4224, 4224, 6400, 5632, 0, 0]],
        'ｃ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 1792, 2176, 4096, 4096, 4096, 2176, 1792, 0, 0]],
        'ｄ' : [12, 13, 0, -2, 12, [0, 128, 128, 128, 1664, 2432, 4224, 4224, 4224, 2432, 1664, 0, 0]],
        'ｅ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 1536, 2304, 4224, 8064, 4096, 2176, 1792, 0, 0]],
        'ｆ' : [12, 13, 0, -2, 12, [0, 896, 1024, 1024, 1024, 8064, 1024, 1024, 1024, 1024, 1024, 0, 0]],
        'ｇ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 1664, 2432, 4224, 4224, 2432, 1664, 128, 256, 3584]],
        'ｈ' : [12, 13, 0, -2, 12, [0, 4096, 4096, 4096, 5632, 6400, 4224, 4224, 4224, 4224, 4224, 0, 0]],
        'ｉ' : [12, 13, 0, -2, 12, [0, 1024, 1024, 0, 0, 1024, 1024, 1024, 1024, 1024, 1024, 0, 0]],
        'ｊ' : [12, 13, 0, -2, 12, [0, 512, 512, 0, 0, 512, 512, 512, 512, 512, 512, 1024, 14336]],
        'ｋ' : [12, 13, 0, -2, 12, [0, 4096, 4096, 4096, 4224, 4352, 4608, 5120, 4608, 4352, 4224, 0, 0]],
        'ｌ' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 0, 0]],
        'ｍ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 22912, 26176, 17472, 17472, 17472, 17472, 17472, 0, 0]],
        'ｎ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 5888, 6272, 4224, 4224, 4224, 4224, 4224, 0, 0]],
        'ｏ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 3584, 4352, 8320, 8320, 8320, 4352, 3584, 0, 0]],
        'ｐ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 5632, 6400, 4224, 4224, 4224, 6400, 5632, 4096, 4096]],
        'ｑ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 1664, 2432, 4224, 4224, 4224, 2432, 1664, 128, 128]],
        'ｒ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 5632, 6400, 4096, 4096, 4096, 4096, 4096, 0, 0]],
        'ｓ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 3840, 4224, 4096, 3840, 128, 4224, 3840, 0, 0]],
        'ｔ' : [12, 13, 0, -2, 12, [0, 2048, 2048, 2048, 16128, 2048, 2048, 2048, 2048, 2176, 1792, 0, 0]],
        'ｕ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 4224, 4224, 4224, 4224, 4224, 4480, 3712, 0, 0]],
        'ｖ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 8320, 8320, 4352, 4352, 2560, 2560, 1024, 0, 0]],
        'ｗ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 17472, 17472, 17472, 10880, 10880, 4352, 4352, 0, 0]],
        'ｘ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 8320, 4352, 2560, 1024, 2560, 4352, 8320, 0, 0]],
        'ｙ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 4224, 4224, 2176, 2304, 1280, 512, 512, 1024, 6144]],
        'ｚ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 8064, 256, 512, 1024, 2048, 4096, 8064, 0, 0]],
        'ぁ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 1024, 16256, 2048, 3840, 6784, 10880, 11392, 6400, 0]],
        'あ' : [12, 13, 0, -2, 12, [0, 1024, 1216, 32512, 2048, 3968, 6464, 10528, 18976, 19488, 13376, 384, 0]],
        'ぃ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 8448, 8448, 8320, 8320, 9344, 6144, 0, 0]],
        'い' : [12, 13, 0, -2, 12, [0, 0, 0, 8192, 8320, 16448, 16448, 16416, 16928, 9216, 6144, 0, 0]],
        'ぅ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 3072, 768, 0, 1792, 6272, 128, 256, 1536, 0]],
        'う' : [12, 13, 0, -2, 12, [0, 7168, 768, 0, 3584, 12544, 128, 128, 128, 256, 1536, 6144, 0]],
        'ぇ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 3072, 768, 0, 7936, 512, 1024, 2560, 4480, 0]],
        'え' : [12, 13, 0, -2, 12, [0, 3072, 768, 0, 16128, 256, 512, 1024, 3072, 4608, 8704, 16832, 0]],
        'ぉ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 2048, 2176, 15936, 2048, 7936, 10368, 10368, 13056, 0]],
        'お' : [12, 13, 0, -2, 12, [0, 2048, 2240, 32288, 2048, 2048, 3968, 6208, 10272, 18464, 18496, 12672, 0]],
        'か' : [12, 13, 0, -2, 12, [0, 4096, 4224, 4224, 31808, 4672, 4640, 8736, 8704, 16896, 16896, 7168, 0]],
        'が' : [12, 13, 0, -2, 12, [0, 4160, 4384, 4224, 31744, 4672, 4672, 8736, 8736, 16896, 16896, 7168, 0]],
        'き' : [12, 13, 0, -2, 12, [0, 1024, 1408, 32256, 960, 32256, 256, 7808, 8576, 8192, 4096, 3840, 0]],
        'ぎ' : [12, 13, 0, -2, 12, [64, 1312, 1152, 32256, 960, 32256, 256, 7808, 8576, 8192, 4096, 3840, 0]],
        'く' : [12, 13, 0, -2, 12, [0, 0, 256, 512, 1024, 2048, 4096, 4096, 3072, 512, 256, 128, 0]],
        'ぐ' : [12, 13, 0, -2, 12, [0, 0, 512, 1088, 2336, 4224, 8192, 8192, 6144, 1024, 512, 256, 0]],
        'け' : [12, 13, 0, -2, 12, [0, 128, 8320, 8320, 20448, 16512, 16512, 16512, 16512, 16512, 8448, 9728, 0]],
        'げ' : [12, 13, 0, -2, 12, [0, 320, 16672, 16768, 40768, 33024, 33024, 33024, 33024, 33024, 16896, 19456, 0]],
        'こ' : [12, 13, 0, -2, 12, [0, 0, 8064, 0, 0, 0, 0, 4096, 8192, 8256, 8064, 0, 0]],
        'ご' : [12, 13, 0, -2, 12, [0, 0, 16192, 32, 128, 64, 0, 8192, 16384, 16512, 16128, 0, 0]],
        'さ' : [12, 13, 0, -2, 12, [0, 512, 512, 704, 32512, 256, 256, 7808, 8576, 8192, 4096, 3840, 0]],
        'ざ' : [12, 13, 0, -2, 12, [0, 2112, 2336, 3712, 63488, 1024, 512, 15616, 17152, 16384, 8192, 7680, 0]],
        'し' : [12, 13, 0, -2, 12, [0, 0, 2048, 2048, 2048, 4096, 4096, 4096, 4160, 4160, 2176, 1792, 0]],
        'じ' : [12, 13, 0, -2, 12, [0, 0, 4224, 4672, 4352, 8192, 8192, 8192, 8320, 8320, 4352, 3584, 0]],
        'す' : [12, 13, 0, -2, 12, [0, 512, 512, 65504, 512, 3584, 4608, 4608, 3584, 512, 1024, 6144, 0]],
        'ず' : [12, 13, 0, -2, 12, [0, 512, 512, 65504, 512, 3648, 4640, 4736, 3648, 512, 1024, 6144, 0]],
        'せ' : [12, 13, 0, -2, 12, [0, 128, 4224, 4224, 5088, 31872, 4224, 4224, 4992, 4096, 4096, 4032, 0]],
        'ぜ' : [12, 13, 0, -2, 12, [0, 256, 8512, 8480, 10112, 63808, 8448, 8448, 9984, 8192, 8192, 8064, 0]],
        'そ' : [12, 13, 0, -2, 12, [0, 0, 8064, 256, 512, 1024, 32736, 1024, 2048, 2048, 2048, 1920, 0]],
        'ぞ' : [12, 13, 0, -2, 12, [0, 0, 15936, 800, 1152, 2048, 65472, 2048, 4096, 4096, 4096, 3840, 0]],
        'た' : [12, 13, 0, -2, 12, [0, 4096, 4096, 32640, 4096, 5056, 4096, 8704, 9216, 9216, 17376, 16384, 0]],
        'だ' : [12, 13, 0, -2, 12, [64, 4384, 4224, 32512, 4096, 5056, 4096, 8704, 9216, 9216, 17376, 16384, 0]],
        'ち' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1504, 32256, 2048, 5888, 6272, 64, 64, 128, 3840, 0]],
        'ぢ' : [12, 13, 0, -2, 12, [64, 2336, 2176, 3840, 63488, 4096, 11776, 12544, 128, 128, 256, 7680, 0]],
        'っ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 3584, 12544, 128, 128, 256, 3584, 0]],
        'つ' : [12, 13, 0, -2, 12, [0, 0, 0, 3840, 28800, 64, 64, 64, 128, 768, 7168, 0, 0]],
        'づ' : [12, 13, 0, -2, 12, [64, 288, 128, 3840, 28800, 64, 64, 64, 128, 768, 7168, 0, 0]],
        'て' : [12, 13, 0, -2, 12, [0, 0, 0, 4032, 61952, 1024, 2048, 2048, 2048, 2048, 1024, 896, 0]],
        'で' : [12, 13, 0, -2, 12, [0, 0, 0, 4032, 61952, 1088, 2336, 2176, 2048, 2048, 1024, 896, 0]],
        'と' : [12, 13, 0, -2, 12, [0, 2048, 2048, 2048, 1024, 1920, 3072, 4096, 8192, 8192, 4096, 3968, 0]],
        'ど' : [12, 13, 0, -2, 12, [128, 4672, 4352, 4096, 2048, 3840, 6144, 8192, 16384, 16384, 8192, 7936, 0]],
        'な' : [12, 13, 0, -2, 12, [0, 2048, 2240, 65056, 4096, 4224, 8320, 8320, 18304, 18624, 2208, 1792, 0]],
        'に' : [12, 13, 0, -2, 12, [0, 0, 8192, 9152, 16384, 16384, 16384, 16896, 17408, 17408, 9184, 8192, 0]],
        'ぬ' : [12, 13, 0, -2, 12, [0, 512, 512, 8704, 12160, 12864, 8736, 21536, 21920, 19040, 12864, 416, 0]],
        'ね' : [12, 13, 0, -2, 12, [0, 4096, 4096, 4096, 29568, 5184, 6176, 4128, 12704, 21088, 4672, 4512, 0]],
        'の' : [12, 13, 0, -2, 12, [0, 0, 0, 3968, 5184, 9248, 17440, 18464, 18464, 12352, 384, 0, 0]],
        'は' : [12, 13, 0, -2, 12, [0, 128, 8320, 8320, 20448, 16512, 16512, 16512, 18304, 18624, 10400, 9984, 0]],
        'ば' : [12, 13, 0, -2, 12, [0, 320, 16672, 16768, 40768, 33024, 33024, 33024, 36608, 37248, 20800, 19968, 0]],
        'ぱ' : [12, 13, 0, -2, 12, [0, 192, 16672, 16672, 40896, 33024, 33024, 33024, 36608, 37248, 20800, 19968, 0]],
        'ひ' : [12, 13, 0, -2, 12, [0, 0, 30976, 2304, 4224, 8384, 8352, 16512, 16640, 16640, 8704, 7168, 0]],
        'び' : [12, 13, 0, -2, 12, [0, 64, 31008, 2176, 4096, 8320, 8384, 16544, 16640, 16640, 8704, 7168, 0]],
        'ぴ' : [12, 13, 0, -2, 12, [0, 192, 31008, 2336, 4288, 8352, 8320, 16512, 16640, 16640, 8704, 7168, 0]],
        'ふ' : [12, 13, 0, -2, 12, [0, 0, 7936, 256, 512, 1024, 9280, 9280, 8736, 16928, 16928, 3072, 0]],
        'ぶ' : [12, 13, 0, -2, 12, [0, 64, 7968, 384, 576, 1024, 9216, 9280, 8768, 16928, 16928, 3072, 0]],
        'ぷ' : [12, 13, 0, -2, 12, [0, 192, 7968, 288, 704, 1024, 9216, 9280, 8768, 16928, 16928, 3072, 0]],
        'へ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 14336, 17408, 33280, 384, 96, 0, 0, 0]],
        'べ' : [12, 13, 0, -2, 12, [0, 0, 128, 576, 256, 14336, 17408, 33280, 384, 96, 0, 0, 0]],
        'ぺ' : [12, 13, 0, -2, 12, [0, 0, 384, 576, 576, 14720, 17408, 33280, 384, 96, 0, 0, 0]],
        'ほ' : [12, 13, 0, -2, 12, [0, 0, 12256, 8320, 16512, 20448, 16512, 16512, 18304, 18624, 10400, 9984, 0]],
        'ぼ' : [12, 13, 0, -2, 12, [0, 64, 24352, 16768, 33024, 40896, 33024, 33024, 36608, 37248, 20800, 19968, 0]],
        'ぽ' : [12, 13, 0, -2, 12, [0, 192, 24352, 16672, 33216, 40704, 33024, 33024, 36608, 37248, 20800, 19968, 0]],
        'ま' : [12, 13, 0, -2, 12, [0, 512, 512, 32704, 512, 16320, 512, 512, 7680, 9088, 8768, 7168, 0]],
        'み' : [12, 13, 0, -2, 12, [0, 0, 31744, 1152, 1152, 1152, 16256, 18624, 34976, 36992, 24832, 1536, 0]],
        'む' : [12, 13, 0, -2, 12, [0, 2048, 2048, 32448, 2080, 2048, 14336, 18432, 18496, 14400, 2112, 1920, 0]],
        'め' : [12, 13, 0, -2, 12, [0, 512, 512, 8704, 12160, 4672, 12832, 21536, 19488, 18464, 12352, 384, 0]],
        'も' : [12, 13, 0, -2, 12, [0, 2048, 2048, 32640, 2048, 2048, 32640, 4096, 4160, 4160, 2176, 1792, 0]],
        'ゃ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 512, 4608, 5888, 14976, 2176, 2304, 1024, 1024, 0]],
        'や' : [12, 13, 0, -2, 12, [0, 256, 4352, 4544, 5920, 6432, 26656, 1216, 1024, 1024, 512, 512, 0]],
        'ゅ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 512, 8704, 10112, 10560, 12608, 6016, 4352, 512, 0]],
        'ゆ' : [12, 13, 0, -2, 12, [0, 256, 256, 18368, 18720, 20768, 20768, 8480, 10176, 8448, 256, 1536, 0]],
        'ょ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 512, 512, 896, 512, 3584, 4864, 4736, 3072, 0]],
        'よ' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1024, 1920, 1024, 1024, 1024, 15360, 18176, 17536, 14336, 0]],
        'ら' : [12, 13, 0, -2, 12, [0, 3072, 768, 0, 8192, 8192, 9984, 14464, 64, 64, 384, 3584, 0]],
        'り' : [12, 13, 0, -2, 12, [0, 0, 4224, 4224, 4224, 4224, 4224, 5248, 2176, 128, 256, 1536, 0]],
        'る' : [12, 13, 0, -2, 12, [0, 0, 8064, 256, 512, 1792, 6272, 8256, 3136, 4672, 4736, 3840, 0]],
        'れ' : [12, 13, 0, -2, 12, [0, 4096, 4096, 4096, 29568, 5184, 6208, 4160, 12416, 20608, 4224, 4192, 0]],
        'ろ' : [12, 13, 0, -2, 12, [0, 0, 8064, 256, 512, 1792, 6272, 8256, 64, 64, 384, 3584, 0]],
        'ゎ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 2048, 2048, 15104, 3200, 2176, 6272, 10368, 2816, 0]],
        'わ' : [12, 13, 0, -2, 12, [0, 4096, 4096, 4096, 29568, 5184, 6176, 4128, 12320, 20512, 4160, 4480, 0]],
        'ゐ' : [12, 13, 0, -2, 12, [0, 0, 15872, 512, 896, 7232, 9248, 17440, 18656, 18720, 12576, 192, 0]],
        'ゑ' : [12, 13, 0, -2, 12, [0, 16320, 256, 512, 3968, 12352, 3648, 4480, 7936, 2048, 14784, 17952, 0]],
        'を' : [12, 13, 0, -2, 12, [0, 2048, 2048, 32640, 4096, 7168, 8896, 18176, 18944, 4608, 4096, 3968, 0]],
        'ん' : [12, 13, 0, -2, 12, [0, 0, 1024, 1024, 2048, 2048, 5632, 6400, 8448, 8480, 16672, 16576, 0]],
        'ゔ' : [12, 13, 0, -2, 12, [128, 12352, 3328, 128, 7168, 25088, 256, 256, 256, 512, 3072, 12288, 0]],
        'ァ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 16256, 128, 1152, 1280, 1024, 2048, 4096, 0]],
        'ア' : [12, 13, 0, -2, 12, [0, 0, 32736, 32, 64, 1152, 1024, 1024, 1024, 1024, 2048, 12288, 0]],
        'ィ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 128, 256, 1536, 6656, 512, 512, 512, 0]],
        'イ' : [12, 13, 0, -2, 12, [0, 0, 64, 128, 256, 1536, 6656, 25088, 512, 512, 512, 512, 0]],
        'ゥ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 1024, 1024, 16256, 8320, 8320, 128, 256, 3584, 0]],
        'ウ' : [12, 13, 0, -2, 12, [0, 1024, 1024, 32704, 16448, 16448, 16448, 128, 128, 256, 1536, 6144, 0]],
        'ェ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 16256, 1024, 1024, 1024, 1024, 16256, 0, 0]],
        'エ' : [12, 13, 0, -2, 12, [0, 0, 0, 32704, 1024, 1024, 1024, 1024, 1024, 1024, 65504, 0, 0]],
        'ォ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 512, 512, 16256, 512, 1536, 2560, 12800, 1536, 0]],
        'オ' : [12, 13, 0, -2, 12, [0, 256, 256, 256, 32736, 256, 768, 1280, 6400, 24832, 256, 1792, 0]],
        'カ' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1024, 32736, 1056, 1056, 1056, 2080, 2080, 4128, 25024, 0]],
        'ガ' : [12, 13, 0, -2, 12, [64, 2336, 2176, 2048, 65472, 2112, 2112, 2112, 4160, 4160, 8256, 50048, 0]],
        'キ' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1024, 32704, 1024, 1024, 1024, 32736, 512, 512, 512, 0]],
        'ギ' : [12, 13, 0, -2, 12, [0, 2112, 2336, 2176, 65280, 2048, 2048, 2048, 65472, 1024, 1024, 1024, 0]],
        'ク' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1984, 2112, 4160, 8256, 128, 128, 256, 1536, 14336, 0]],
        'グ' : [12, 13, 0, -2, 12, [64, 2336, 2176, 3968, 4224, 8320, 16512, 256, 256, 512, 3072, 28672, 0]],
        'ケ' : [12, 13, 0, -2, 12, [0, 4096, 4096, 4096, 8160, 8448, 16640, 256, 512, 512, 3072, 12288, 0]],
        'ゲ' : [12, 13, 0, -2, 12, [64, 8480, 8320, 8192, 16320, 16896, 33280, 512, 1024, 1024, 6144, 24576, 0]],
        'コ' : [12, 13, 0, -2, 12, [0, 0, 0, 32704, 64, 64, 64, 64, 64, 64, 32704, 64, 0]],
        'ゴ' : [12, 13, 0, -2, 12, [64, 544, 256, 32704, 64, 64, 64, 64, 64, 64, 32704, 64, 0]],
        'サ' : [12, 13, 0, -2, 12, [0, 4224, 4224, 4224, 32736, 4224, 4224, 4224, 256, 256, 1536, 6144, 0]],
        'ザ' : [12, 13, 0, -2, 12, [0, 8512, 8480, 8576, 65344, 8448, 8448, 8448, 512, 512, 3072, 12288, 0]],
        'シ' : [12, 13, 0, -2, 12, [0, 0, 12288, 3072, 32, 24608, 6208, 64, 128, 256, 1536, 14336, 0]],
        'ジ' : [12, 13, 0, -2, 12, [64, 288, 12416, 3072, 32, 24608, 6208, 64, 128, 256, 1536, 14336, 0]],
        'ス' : [12, 13, 0, -2, 12, [0, 0, 16256, 128, 128, 128, 256, 256, 768, 1152, 6208, 24608, 0]],
        'ズ' : [12, 13, 0, -2, 12, [576, 288, 32512, 256, 256, 256, 512, 512, 1536, 2304, 12416, 49216, 0]],
        'セ' : [12, 13, 0, -2, 12, [0, 2048, 2048, 2272, 2848, 7232, 59456, 2176, 2048, 2048, 2048, 2016, 0]],
        'ゼ' : [12, 13, 0, -2, 12, [64, 2336, 2176, 2272, 2848, 7232, 59456, 2176, 2048, 2048, 2048, 2016, 0]],
        'ソ' : [12, 13, 0, -2, 12, [0, 0, 16448, 16448, 8256, 8256, 64, 128, 128, 256, 1536, 6144, 0]],
        'ゾ' : [12, 13, 0, -2, 12, [64, 288, 32896, 32768, 16512, 16512, 128, 256, 256, 512, 3072, 12288, 0]],
        'タ' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1984, 2112, 4160, 9792, 384, 128, 256, 1536, 14336, 0]],
        'ダ' : [12, 13, 0, -2, 12, [64, 2336, 2176, 3968, 4224, 8320, 19584, 768, 256, 512, 3072, 28672, 0]],
        'チ' : [12, 13, 0, -2, 12, [0, 0, 960, 31744, 1024, 1024, 65504, 1024, 1024, 1024, 2048, 12288, 0]],
        'ヂ' : [12, 13, 0, -2, 12, [0, 64, 800, 31872, 1024, 1024, 65504, 1024, 1024, 1024, 2048, 12288, 0]],
        'ッ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 9344, 9344, 9344, 256, 256, 1536, 6144, 0]],
        'ツ' : [12, 13, 0, -2, 12, [0, 0, 18496, 18496, 9280, 9280, 64, 128, 128, 256, 1536, 6144, 0]],
        'ヅ' : [12, 13, 0, -2, 12, [64, 288, 36992, 36864, 18560, 18560, 128, 256, 256, 512, 3072, 12288, 0]],
        'テ' : [12, 13, 0, -2, 12, [0, 0, 16320, 0, 0, 32736, 512, 512, 512, 512, 1024, 6144, 0]],
        'デ' : [12, 13, 0, -2, 12, [64, 288, 32384, 0, 0, 65472, 1024, 1024, 1024, 1024, 2048, 12288, 0]],
        'ト' : [12, 13, 0, -2, 12, [0, 4096, 4096, 4096, 4096, 7168, 4864, 4288, 4096, 4096, 4096, 4096, 0]],
        'ド' : [12, 13, 0, -2, 12, [0, 8320, 8768, 8448, 8192, 14336, 9728, 8576, 8192, 8192, 8192, 8192, 0]],
        'ナ' : [12, 13, 0, -2, 12, [0, 512, 512, 512, 32736, 512, 512, 512, 1024, 1024, 2048, 12288, 0]],
        'ニ' : [12, 13, 0, -2, 12, [0, 0, 0, 16320, 0, 0, 0, 0, 0, 0, 32736, 0, 0]],
        'ヌ' : [12, 13, 0, -2, 12, [0, 0, 16320, 64, 64, 2176, 1152, 768, 256, 640, 3136, 28672, 0]],
        'ネ' : [12, 13, 0, -2, 12, [0, 1024, 1024, 32640, 128, 256, 512, 1536, 7552, 58464, 1024, 1024, 0]],
        'ノ' : [12, 13, 0, -2, 12, [0, 0, 128, 128, 128, 128, 128, 256, 256, 512, 3072, 12288, 0]],
        'ハ' : [12, 13, 0, -2, 12, [0, 0, 4096, 4224, 4224, 4160, 8256, 8256, 8224, 16416, 16416, 16416, 0]],
        'バ' : [12, 13, 0, -2, 12, [0, 64, 288, 4224, 4096, 4096, 4224, 8320, 8256, 8256, 16416, 16416, 0]],
        'パ' : [12, 13, 0, -2, 12, [0, 192, 288, 4384, 4288, 4096, 4224, 8320, 8256, 8256, 16416, 16416, 0]],
        'ヒ' : [12, 13, 0, -2, 12, [0, 8192, 8192, 8192, 8640, 15872, 8192, 8192, 8192, 8192, 8192, 8128, 0]],
        'ビ' : [12, 13, 0, -2, 12, [0, 64, 8480, 8320, 8192, 8640, 15872, 8192, 8192, 8192, 8192, 8128, 0]],
        'ピ' : [12, 13, 0, -2, 12, [0, 192, 16672, 16672, 16576, 17920, 30720, 16384, 16384, 16384, 16384, 16256, 0]],
        'フ' : [12, 13, 0, -2, 12, [0, 0, 32704, 64, 64, 64, 64, 128, 128, 256, 1536, 14336, 0]],
        'ブ' : [12, 13, 0, -2, 12, [64, 544, 256, 65408, 128, 128, 128, 256, 256, 512, 3072, 12288, 0]],
        'プ' : [12, 13, 0, -2, 12, [192, 288, 65312, 192, 128, 128, 128, 256, 256, 512, 3072, 12288, 0]],
        'ヘ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 7168, 8704, 16640, 32896, 64, 32, 0, 0, 0]],
        'ベ' : [12, 13, 0, -2, 12, [0, 64, 288, 128, 0, 7168, 8704, 16640, 32896, 64, 32, 0, 0]],
        'ペ' : [12, 13, 0, -2, 12, [0, 192, 288, 288, 192, 7168, 8704, 16640, 32896, 64, 32, 0, 0]],
        'ホ' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1024, 65504, 1024, 1024, 9344, 17472, 33824, 1024, 3072, 0]],
        'ボ' : [12, 13, 0, -2, 12, [64, 1312, 1152, 1024, 65504, 1024, 1024, 9344, 17472, 33824, 1024, 3072, 0]],
        'ポ' : [12, 13, 0, -2, 12, [0, 1216, 1312, 1312, 65472, 1024, 1024, 9344, 17472, 33824, 1024, 3072, 0]],
        'マ' : [12, 13, 0, -2, 12, [0, 0, 0, 32736, 32, 64, 128, 4352, 2560, 1024, 512, 256, 0]],
        'ミ' : [12, 13, 0, -2, 12, [0, 0, 7168, 896, 0, 0, 7168, 768, 0, 0, 15360, 896, 0]],
        'ム' : [12, 13, 0, -2, 12, [0, 2048, 2048, 2048, 2048, 4096, 4224, 4224, 8320, 10048, 63552, 64, 0]],
        'メ' : [12, 13, 0, -2, 12, [0, 256, 256, 256, 256, 12544, 3584, 896, 608, 1024, 6144, 24576, 0]],
        'モ' : [12, 13, 0, -2, 12, [0, 0, 32640, 2048, 2048, 2048, 65504, 2048, 2048, 2048, 2048, 1920, 0]],
        'ャ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 2048, 2048, 2944, 15488, 2304, 1280, 1024, 1024, 0]],
        'ヤ' : [12, 13, 0, -2, 12, [0, 4096, 4096, 4544, 7744, 59456, 2176, 2176, 2048, 1024, 1024, 1024, 0]],
        'ュ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 15872, 512, 512, 512, 512, 32640, 0, 0]],
        'ユ' : [12, 13, 0, -2, 12, [0, 0, 0, 16128, 256, 256, 256, 256, 256, 256, 65504, 0, 0]],
        'ョ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 8064, 128, 128, 8064, 128, 128, 8064, 0]],
        'ヨ' : [12, 13, 0, -2, 12, [0, 0, 16320, 64, 64, 64, 16320, 64, 64, 64, 16320, 64, 0]],
        'ラ' : [12, 13, 0, -2, 12, [0, 0, 16256, 0, 0, 32704, 64, 64, 128, 256, 1536, 6144, 0]],
        'リ' : [12, 13, 0, -2, 12, [0, 8320, 8320, 8320, 8320, 8320, 8320, 128, 256, 256, 512, 3072, 0]],
        'ル' : [12, 13, 0, -2, 12, [0, 512, 4608, 4608, 4608, 4608, 4640, 4640, 8768, 8768, 17024, 17152, 0]],
        'レ' : [12, 13, 0, -2, 12, [0, 0, 8192, 8192, 8192, 8192, 8192, 8256, 8320, 8448, 9728, 14336, 0]],
        'ロ' : [12, 13, 0, -2, 12, [0, 0, 0, 32704, 16448, 16448, 16448, 16448, 16448, 16448, 32704, 16448, 0]],
        'ヮ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 16256, 8320, 8320, 128, 128, 256, 3584, 0]],
        'ワ' : [12, 13, 0, -2, 12, [0, 0, 32704, 16448, 16448, 16448, 64, 128, 128, 256, 1536, 6144, 0]],
        'ヰ' : [12, 13, 0, -2, 12, [0, 256, 256, 256, 16320, 4352, 4352, 4352, 32736, 256, 256, 256, 0]],
        'ヱ' : [12, 13, 0, -2, 12, [0, 0, 0, 16320, 128, 256, 1536, 1024, 1024, 1024, 32736, 0, 0]],
        'ヲ' : [12, 13, 0, -2, 12, [0, 0, 16320, 64, 64, 16320, 64, 128, 128, 256, 1536, 6144, 0]],
        'ン' : [12, 13, 0, -2, 12, [0, 0, 12288, 3072, 32, 32, 64, 64, 128, 256, 1536, 14336, 0]],
        'ヴ' : [12, 13, 0, -2, 12, [64, 1312, 1152, 32704, 16448, 16448, 16448, 128, 128, 256, 1536, 6144, 0]],
        'ヵ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 2048, 2048, 16256, 2176, 2176, 2176, 4224, 4864, 0]],
        'ヶ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 4096, 4096, 8064, 8704, 8704, 512, 1024, 6144, 0]],
        'Α' : [12, 13, 0, -2, 12, [0, 7168, 1024, 2560, 2560, 4352, 4352, 7936, 8320, 8320, 61920, 0, 0]],
        'Β' : [12, 13, 0, -2, 12, [0, 32512, 4224, 4224, 4224, 7936, 4224, 4160, 4160, 4224, 32512, 0, 0]],
        'Γ' : [12, 13, 0, -2, 12, [0, 16320, 2112, 2112, 2048, 2048, 2048, 2048, 2048, 2048, 15872, 0, 0]],
        'Δ' : [12, 13, 0, -2, 12, [0, 1024, 1024, 2560, 2560, 4352, 4352, 8320, 8320, 16448, 32704, 0, 0]],
        'Ε' : [12, 13, 0, -2, 12, [0, 16320, 2112, 2048, 2304, 3840, 2304, 2048, 2112, 2112, 16320, 0, 0]],
        'Ζ' : [12, 13, 0, -2, 12, [0, 16256, 8320, 8448, 512, 1024, 2048, 4096, 8320, 8320, 16256, 0, 0]],
        'Η' : [12, 13, 0, -2, 12, [0, 31200, 4224, 4224, 4224, 8064, 4224, 4224, 4224, 4224, 31200, 0, 0]],
        'Θ' : [12, 13, 0, -2, 12, [0, 8064, 8256, 16416, 20640, 24480, 20640, 16416, 16416, 8256, 8064, 0, 0]],
        'Ι' : [12, 13, 0, -2, 12, [0, 7936, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 7936, 0, 0]],
        'Κ' : [12, 13, 0, -2, 12, [0, 31200, 4224, 4352, 4608, 5120, 7168, 4608, 4352, 4224, 31200, 0, 0]],
        'Λ' : [12, 13, 0, -2, 12, [0, 7168, 1024, 2560, 2560, 4352, 4352, 4352, 8320, 8320, 61920, 0, 0]],
        'Μ' : [12, 13, 0, -2, 12, [0, 61920, 12672, 12672, 10880, 10880, 10880, 9344, 9344, 9344, 61920, 0, 0]],
        'Ν' : [12, 13, 0, -2, 12, [0, 31200, 6272, 5248, 5248, 4736, 4736, 4480, 4480, 4224, 30848, 0, 0]],
        'Ξ' : [12, 13, 0, -2, 12, [0, 8160, 8256, 0, 4224, 7936, 8448, 0, 16512, 33024, 65280, 0, 0]],
        'Ο' : [12, 13, 0, -2, 12, [0, 7936, 8320, 16448, 16448, 16448, 16448, 16448, 16448, 8320, 7936, 0, 0]],
        'Π' : [12, 13, 0, -2, 12, [0, 16320, 4224, 4224, 4224, 4224, 4224, 4224, 4224, 4224, 31200, 0, 0]],
        'Ρ' : [12, 13, 0, -2, 12, [0, 32512, 4224, 4160, 4160, 4224, 7936, 4096, 4096, 4096, 30720, 0, 0]],
        'Σ' : [12, 13, 0, -2, 12, [128, 16256, 4224, 2176, 1024, 512, 1024, 2048, 4224, 8320, 16256, 128, 0]],
        'Τ' : [12, 13, 0, -2, 12, [0, 32704, 17472, 17472, 1024, 1024, 1024, 1024, 1024, 1024, 7936, 0, 0]],
        'Υ' : [12, 13, 0, -2, 12, [0, 12672, 19008, 33824, 1024, 1024, 1024, 1024, 1024, 1024, 7936, 0, 0]],
        'Φ' : [12, 13, 0, -2, 12, [0, 7936, 1024, 7936, 9344, 17472, 17472, 9344, 7936, 1024, 7936, 0, 0]],
        'Χ' : [12, 13, 0, -2, 12, [0, 31680, 4352, 4352, 2560, 1024, 1024, 2560, 4352, 4352, 31680, 0, 0]],
        'Ψ' : [12, 13, 0, -2, 12, [0, 7936, 1024, 17472, 42144, 9344, 9344, 7936, 1024, 1024, 7936, 0, 0]],
        'Ω' : [12, 13, 0, -2, 12, [0, 3840, 4224, 8256, 8256, 8256, 8256, 4224, 2304, 10560, 14784, 0, 0]],
        'α' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 3136, 12928, 8448, 16640, 16896, 17920, 14720, 0, 0]],
        'β' : [12, 13, 0, -2, 12, [0, 768, 1152, 2176, 2688, 4352, 4224, 4224, 10368, 9984, 8192, 8192, 0]],
        'γ' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 12672, 2624, 1024, 1024, 2048, 2048, 4096, 4096, 0]],
        'δ' : [12, 13, 0, -2, 12, [0, 1536, 2432, 2048, 1024, 2560, 4352, 8448, 8448, 8704, 7168, 0, 0]],
        'ε' : [12, 13, 0, -2, 12, [0, 0, 0, 1920, 2112, 2048, 1792, 3072, 4096, 4224, 3840, 0, 0]],
        'ζ' : [12, 13, 0, -2, 12, [0, 2048, 4480, 5632, 2048, 4096, 8192, 8192, 6144, 1536, 256, 3584, 0]],
        'η' : [12, 13, 0, -2, 12, [0, 0, 0, 13056, 3200, 2176, 4224, 4352, 8448, 8704, 512, 0, 0]],
        'θ' : [12, 13, 0, -2, 12, [0, 768, 1152, 2176, 2176, 4224, 8064, 4224, 4352, 4352, 4608, 3072, 0]],
        'ι' : [12, 13, 0, -2, 12, [0, 0, 0, 2048, 2048, 2048, 4096, 4224, 8320, 8448, 7680, 0, 0]],
        'κ' : [12, 13, 0, -2, 12, [0, 0, 0, 4288, 2304, 2560, 3072, 5120, 4608, 8832, 8448, 0, 0]],
        'λ' : [12, 13, 0, -2, 12, [0, 7168, 512, 512, 512, 3584, 4608, 8704, 8448, 16640, 16576, 0, 0]],
        'μ' : [12, 13, 0, -2, 12, [0, 0, 0, 64, 2112, 2176, 2176, 4352, 4864, 7424, 8384, 8192, 0]],
        'ν' : [12, 13, 0, -2, 12, [0, 0, 0, 64, 12352, 2176, 2176, 2304, 4352, 4608, 7168, 0, 0]],
        'ξ' : [12, 13, 0, -2, 12, [0, 1920, 2048, 3968, 4096, 8064, 8192, 8192, 7680, 256, 256, 3584, 0]],
        'ο' : [12, 13, 0, -2, 12, [0, 0, 0, 1792, 6272, 4224, 8320, 8320, 8448, 8960, 7168, 0, 0]],
        'π' : [12, 13, 0, -2, 12, [0, 0, 0, 64, 16256, 2304, 2304, 4608, 4672, 9280, 17280, 0, 0]],
        'ρ' : [12, 13, 0, -2, 12, [0, 0, 0, 896, 3136, 2112, 4160, 4224, 12672, 11776, 8192, 8192, 0]],
        'σ' : [12, 13, 0, -2, 12, [0, 0, 32, 4032, 12544, 8448, 16640, 16640, 16896, 17920, 14336, 0, 0]],
        'τ' : [12, 13, 0, -2, 12, [0, 0, 0, 64, 16256, 1024, 1024, 2048, 2112, 2176, 1792, 0, 0]],
        'υ' : [12, 13, 0, -2, 12, [0, 0, 0, 2176, 2112, 2112, 4224, 4224, 8448, 8704, 7168, 0, 0]],
        'φ' : [12, 13, 0, -2, 12, [0, 512, 512, 3840, 5248, 9344, 9344, 9344, 9472, 7680, 2048, 2048, 0]],
        'χ' : [12, 13, 0, -2, 12, [0, 0, 0, 3136, 4736, 768, 512, 512, 1536, 2592, 4672, 8576, 0]],
        'ψ' : [12, 13, 0, -2, 12, [0, 256, 256, 512, 25120, 4672, 5184, 9344, 9344, 7936, 2048, 2048, 0]],
        'ω' : [12, 13, 0, -2, 12, [0, 0, 0, 2176, 4160, 4672, 8768, 9280, 9280, 9344, 6912, 0, 0]],
        'А' : [12, 13, 0, -2, 12, [0, 7168, 1024, 2560, 2560, 4352, 4352, 7936, 8320, 8320, 61920, 0, 0]],
        'Б' : [12, 13, 0, -2, 12, [0, 32512, 4224, 4160, 4096, 7936, 4224, 4160, 4160, 4224, 32512, 0, 0]],
        'В' : [12, 13, 0, -2, 12, [0, 32512, 4224, 4224, 4224, 7936, 4224, 4160, 4160, 4224, 32512, 0, 0]],
        'Г' : [12, 13, 0, -2, 12, [0, 16320, 2112, 2112, 2048, 2048, 2048, 2048, 2048, 2048, 15360, 0, 0]],
        'Д' : [12, 13, 0, -2, 12, [0, 16320, 2176, 2176, 2176, 2176, 2176, 4224, 4224, 16256, 8256, 16416, 0]],
        'Е' : [12, 13, 0, -2, 12, [0, 32512, 4352, 4224, 4608, 7680, 4608, 4096, 4160, 4224, 32640, 0, 0]],
        'Ё' : [12, 13, 0, -2, 12, [12672, 0, 32640, 4224, 4160, 4608, 7680, 4608, 4128, 4160, 32704, 0, 0]],
        'Ж' : [12, 13, 0, -2, 12, [0, 7936, 17472, 42144, 9344, 7936, 5376, 9344, 9344, 17472, 57184, 0, 0]],
        'З' : [12, 13, 0, -2, 12, [0, 12032, 12416, 8256, 128, 768, 128, 64, 8256, 12416, 12032, 0, 0]],
        'И' : [12, 13, 0, -2, 12, [0, 31200, 4480, 4736, 4736, 5248, 5248, 6272, 6272, 4224, 29152, 0, 0]],
        'Й' : [12, 13, 0, -2, 12, [2304, 1536, 28896, 4480, 4480, 4736, 4736, 5248, 5248, 6272, 31200, 0, 0]],
        'К' : [12, 13, 0, -2, 12, [0, 30912, 4384, 4352, 4608, 5120, 7168, 4608, 4352, 4224, 31200, 0, 0]],
        'Л' : [12, 13, 0, -2, 12, [0, 8160, 1152, 1152, 1152, 1152, 1152, 17536, 17536, 18560, 12768, 0, 0]],
        'М' : [12, 13, 0, -2, 12, [0, 29120, 4352, 4352, 10880, 10880, 10880, 9344, 9344, 9344, 61920, 0, 0]],
        'Н' : [12, 13, 0, -2, 12, [0, 31200, 4224, 4224, 4224, 8064, 4224, 4224, 4224, 4224, 31200, 0, 0]],
        'О' : [12, 13, 0, -2, 12, [0, 7936, 8320, 16448, 16448, 16448, 16448, 16448, 16448, 8320, 7936, 0, 0]],
        'П' : [12, 13, 0, -2, 12, [0, 16320, 4224, 4224, 4224, 4224, 4224, 4224, 4224, 4224, 31200, 0, 0]],
        'Р' : [12, 13, 0, -2, 12, [0, 32512, 4224, 4160, 4160, 4224, 7936, 4096, 4096, 4096, 30720, 0, 0]],
        'С' : [12, 13, 0, -2, 12, [0, 3904, 4288, 8256, 8192, 8192, 8192, 8192, 8256, 4288, 3840, 0, 0]],
        'Т' : [12, 13, 0, -2, 12, [0, 32704, 17472, 17472, 33824, 1024, 1024, 1024, 1024, 1024, 7936, 0, 0]],
        'У' : [12, 13, 0, -2, 12, [0, 31200, 4224, 4224, 2304, 2304, 2560, 2560, 1024, 17408, 18432, 12288, 0]],
        'Ф' : [12, 13, 0, -2, 12, [0, 7936, 1024, 7936, 9344, 17472, 17472, 9344, 7936, 1024, 7936, 0, 0]],
        'Х' : [12, 13, 0, -2, 12, [0, 31680, 4352, 4352, 2560, 1024, 1024, 2560, 4352, 4352, 31680, 0, 0]],
        'Ц' : [12, 13, 0, -2, 12, [0, 31680, 4352, 4352, 4352, 4352, 4352, 4352, 4352, 4352, 32704, 32, 0]],
        'Ч' : [12, 13, 0, -2, 12, [0, 31200, 4224, 4224, 4224, 4224, 4480, 3712, 128, 128, 480, 0, 0]],
        'Ш' : [12, 13, 0, -2, 12, [0, 61152, 9344, 9344, 9344, 9344, 9344, 9344, 9344, 9344, 65504, 0, 0]],
        'Щ' : [12, 13, 0, -2, 12, [0, 61152, 9344, 9344, 9344, 9344, 9344, 9344, 9344, 9344, 65504, 16, 0]],
        'Ъ' : [12, 13, 0, -2, 12, [0, 32256, 18432, 18432, 2048, 3968, 2112, 2080, 2080, 2112, 16256, 0, 0]],
        'Ы' : [12, 13, 0, -2, 12, [0, 64480, 8320, 8320, 8320, 15488, 8832, 8832, 8832, 8832, 64992, 0, 0]],
        'Ь' : [12, 13, 0, -2, 12, [0, 31744, 4096, 4096, 4096, 7936, 4224, 4160, 4160, 4224, 32512, 0, 0]],
        'Э' : [12, 13, 0, -2, 12, [0, 12032, 12416, 8256, 64, 1600, 2496, 64, 8256, 12416, 12032, 0, 0]],
        'Ю' : [12, 13, 0, -2, 12, [0, 61824, 8768, 9248, 9248, 15392, 9248, 9248, 9248, 8768, 61824, 0, 0]],
        'Я' : [12, 13, 0, -2, 12, [0, 4064, 4224, 8320, 8320, 4224, 3968, 1152, 2176, 4224, 25056, 0, 0]],
        'а' : [12, 13, 0, -2, 12, [0, 0, 0, 3840, 4224, 128, 1920, 6272, 4224, 4480, 3776, 0, 0]],
        'б' : [12, 13, 0, -2, 12, [256, 1536, 2048, 4096, 5632, 6400, 4224, 4224, 4224, 2304, 1536, 0, 0]],
        'в' : [12, 13, 0, -2, 12, [0, 0, 0, 15872, 4352, 4352, 7680, 4352, 4224, 4352, 15872, 0, 0]],
        'г' : [12, 13, 0, -2, 12, [0, 0, 0, 7936, 2176, 2112, 2048, 2048, 2048, 2048, 7680, 0, 0]],
        'д' : [12, 13, 0, -2, 12, [0, 0, 0, 16256, 2304, 2304, 2304, 2304, 4352, 8064, 8256, 16448, 0]],
        'е' : [12, 13, 0, -2, 12, [0, 0, 0, 3584, 4352, 8320, 16256, 8192, 8192, 4224, 3840, 0, 0]],
        'ё' : [12, 13, 0, -2, 12, [12672, 0, 0, 3584, 4352, 8320, 16256, 8192, 8192, 4224, 3840, 0, 0]],
        'ж' : [12, 13, 0, -2, 12, [0, 0, 0, 20032, 17472, 13696, 7936, 9344, 9344, 9344, 28352, 0, 0]],
        'з' : [12, 13, 0, -2, 12, [0, 0, 0, 7936, 8320, 128, 1792, 128, 8320, 12416, 3840, 0, 0]],
        'и' : [12, 13, 0, -2, 12, [0, 0, 0, 14784, 4480, 4736, 4736, 5248, 5248, 6272, 14784, 0, 0]],
        'й' : [12, 13, 0, -2, 12, [2304, 1536, 0, 14784, 4480, 4736, 4736, 5248, 5248, 6272, 14784, 0, 0]],
        'к' : [12, 13, 0, -2, 12, [0, 0, 0, 14720, 4672, 4608, 5120, 6656, 4352, 4224, 14784, 0, 0]],
        'л' : [12, 13, 0, -2, 12, [0, 0, 0, 8128, 1152, 1152, 1152, 1152, 9344, 9344, 6592, 0, 0]],
        'м' : [12, 13, 0, -2, 12, [0, 0, 0, 29120, 12672, 10880, 10880, 10880, 9344, 9344, 29120, 0, 0]],
        'н' : [12, 13, 0, -2, 12, [0, 0, 0, 14784, 4224, 4224, 8064, 4224, 4224, 4224, 14784, 0, 0]],
        'о' : [12, 13, 0, -2, 12, [0, 0, 0, 3840, 4224, 8256, 8256, 8256, 8256, 4224, 3840, 0, 0]],
        'п' : [12, 13, 0, -2, 12, [0, 0, 0, 16320, 4224, 4224, 4224, 4224, 4224, 4224, 14784, 0, 0]],
        'р' : [12, 13, 0, -2, 12, [0, 0, 0, 14080, 6272, 4160, 4160, 4160, 6272, 5888, 4096, 14336, 0]],
        'с' : [12, 13, 0, -2, 12, [0, 0, 0, 3840, 4224, 8320, 8192, 8192, 8192, 4224, 3840, 0, 0]],
        'т' : [12, 13, 0, -2, 12, [0, 0, 0, 16256, 9344, 17472, 1024, 1024, 1024, 1024, 3584, 0, 0]],
        'у' : [12, 13, 0, -2, 12, [0, 0, 0, 7616, 2176, 2176, 1280, 1280, 512, 4608, 4608, 3072, 0]],
        'ф' : [12, 13, 0, -2, 12, [0, 3072, 1024, 13696, 20032, 17472, 17472, 20032, 13696, 1024, 3584, 0, 0]],
        'х' : [12, 13, 0, -2, 12, [0, 0, 0, 15232, 4352, 2560, 1024, 1024, 2560, 4352, 15232, 0, 0]],
        'ц' : [12, 13, 0, -2, 12, [0, 0, 0, 15232, 4352, 4352, 4352, 4352, 4352, 4352, 16256, 64, 0]],
        'ч' : [12, 13, 0, -2, 12, [0, 0, 0, 15232, 4352, 4352, 4352, 4864, 3328, 256, 896, 0, 0]],
        'ш' : [12, 13, 0, -2, 12, [0, 0, 0, 28352, 9344, 9344, 9344, 9344, 9344, 9344, 32704, 0, 0]],
        'щ' : [12, 13, 0, -2, 12, [0, 0, 0, 28352, 9344, 9344, 9344, 9344, 9344, 9344, 32704, 32, 0]],
        'ъ' : [12, 13, 0, -2, 12, [0, 0, 0, 7680, 9216, 9216, 1920, 1088, 1088, 1088, 8064, 0, 0]],
        'ы' : [12, 13, 0, -2, 12, [0, 0, 0, 29120, 8320, 8320, 15488, 8832, 8832, 8832, 32192, 0, 0]],
        'ь' : [12, 13, 0, -2, 12, [0, 0, 0, 7168, 2048, 2048, 3840, 2176, 2176, 2176, 7936, 0, 0]],
        'э' : [12, 13, 0, -2, 12, [0, 0, 0, 7680, 8448, 128, 3200, 4992, 128, 8448, 7680, 0, 0]],
        'ю' : [12, 13, 0, -2, 12, [0, 0, 0, 15232, 5184, 5184, 7232, 5184, 5184, 5184, 15232, 0, 0]],
        'я' : [12, 13, 0, -2, 12, [0, 0, 0, 8064, 8448, 8448, 7936, 1280, 2304, 2304, 13184, 0, 0]],
        '─' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 65520, 0, 0, 0, 0, 0, 0]],
        '│' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024]],
        '┌' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 2032, 1024, 1024, 1024, 1024, 1024, 1024]],
        '┐' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 64512, 1024, 1024, 1024, 1024, 1024, 1024]],
        '┘' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 1024, 64512, 0, 0, 0, 0, 0, 0]],
        '└' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 1024, 2032, 0, 0, 0, 0, 0, 0]],
        '├' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 1024, 2032, 1024, 1024, 1024, 1024, 1024, 1024]],
        '┬' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 65520, 1024, 1024, 1024, 1024, 1024, 1024]],
        '┤' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 1024, 64512, 1024, 1024, 1024, 1024, 1024, 1024]],
        '┴' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 1024, 65520, 0, 0, 0, 0, 0, 0]],
        '┼' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 1024, 65520, 1024, 1024, 1024, 1024, 1024, 1024]],
        '━' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 65520, 65520, 0, 0, 0, 0, 0, 0]],
        '┃' : [12, 13, 0, -2, 12, [1536, 1536, 1536, 1536, 1536, 1536, 1536, 1536, 1536, 1536, 1536, 1536, 1536]],
        '┏' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 2032, 2032, 1536, 1536, 1536, 1536, 1536, 1536]],
        '┓' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 65024, 65024, 1536, 1536, 1536, 1536, 1536, 1536]],
        '┛' : [12, 13, 0, -2, 12, [1536, 1536, 1536, 1536, 1536, 65024, 65024, 0, 0, 0, 0, 0, 0]],
        '┗' : [12, 13, 0, -2, 12, [1536, 1536, 1536, 1536, 1536, 2032, 2032, 0, 0, 0, 0, 0, 0]],
        '┣' : [12, 13, 0, -2, 12, [1536, 1536, 1536, 1536, 1536, 2032, 2032, 1536, 1536, 1536, 1536, 1536, 1536]],
        '┳' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 65520, 65520, 1536, 1536, 1536, 1536, 1536, 1536]],
        '┫' : [12, 13, 0, -2, 12, [1536, 1536, 1536, 1536, 1536, 65024, 65024, 1536, 1536, 1536, 1536, 1536, 1536]],
        '┻' : [12, 13, 0, -2, 12, [1536, 1536, 1536, 1536, 1536, 65520, 65520, 0, 0, 0, 0, 0, 0]],
        '╋' : [12, 13, 0, -2, 12, [1536, 1536, 1536, 1536, 1536, 65520, 65520, 1536, 1536, 1536, 1536, 1536, 1536]],
        '┠' : [12, 13, 0, -2, 12, [1536, 1536, 1536, 1536, 1536, 1536, 2032, 1536, 1536, 1536, 1536, 1536, 1536]],
        '┯' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 65520, 65520, 1024, 1024, 1024, 1024, 1024, 1024]],
        '┨' : [12, 13, 0, -2, 12, [1536, 1536, 1536, 1536, 1536, 1536, 65024, 1536, 1536, 1536, 1536, 1536, 1536]],
        '┷' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 65520, 65520, 0, 0, 0, 0, 0, 0]],
        '┿' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 65520, 65520, 1024, 1024, 1024, 1024, 1024, 1024]],
        '┝' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 2032, 2032, 1024, 1024, 1024, 1024, 1024, 1024]],
        '┰' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 65520, 1536, 1536, 1536, 1536, 1536, 1536]],
        '┥' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 64512, 64512, 1024, 1024, 1024, 1024, 1024, 1024]],
        '┸' : [12, 13, 0, -2, 12, [1536, 1536, 1536, 1536, 1536, 65520, 0, 0, 0, 0, 0, 0, 0]],
        '╂' : [12, 13, 0, -2, 12, [1536, 1536, 1536, 1536, 1536, 1536, 65520, 1536, 1536, 1536, 1536, 1536, 1536]],
        '①' : [12, 13, 0, -2, 12, [0, 3584, 12672, 17472, 19520, 37920, 33824, 33824, 17472, 17472, 12672, 3584, 0]],
        '②' : [12, 13, 0, -2, 12, [0, 3584, 12672, 20032, 20800, 33056, 34336, 34848, 20544, 24384, 12672, 3584, 0]],
        '③' : [12, 13, 0, -2, 12, [0, 3584, 12672, 20032, 20800, 33056, 34336, 33056, 20800, 20032, 12672, 3584, 0]],
        '④' : [12, 13, 0, -2, 12, [0, 3584, 12672, 17984, 17984, 35360, 35360, 37408, 24384, 16960, 12672, 3584, 0]],
        '⑤' : [12, 13, 0, -2, 12, [0, 3584, 12672, 24384, 20544, 40480, 33056, 33056, 20800, 20032, 12672, 3584, 0]],
        '⑥' : [12, 13, 0, -2, 12, [0, 3584, 12672, 20032, 20544, 40480, 37152, 37152, 20800, 20032, 12672, 3584, 0]],
        '⑦' : [12, 13, 0, -2, 12, [0, 3584, 12672, 24384, 16704, 33312, 33824, 33824, 18496, 18496, 12672, 3584, 0]],
        '⑧' : [12, 13, 0, -2, 12, [0, 3584, 12672, 20032, 20800, 37152, 36384, 37152, 20800, 20032, 12672, 3584, 0]],
        '⑨' : [12, 13, 0, -2, 12, [0, 3584, 12672, 20032, 20800, 37152, 37152, 36640, 16704, 20032, 12672, 3584, 0]],
        '⑩' : [12, 13, 0, -2, 12, [0, 3584, 12672, 21312, 29888, 38048, 38048, 38048, 21696, 21312, 12672, 3584, 0]],
        '⑪' : [12, 13, 0, -2, 12, [0, 3584, 12672, 20800, 29504, 37152, 37152, 37152, 20800, 20800, 12672, 3584, 0]],
        '⑫' : [12, 13, 0, -2, 12, [0, 3584, 12672, 21312, 29888, 37024, 37152, 37408, 21568, 22464, 12672, 3584, 0]],
        '⑬' : [12, 13, 0, -2, 12, [0, 3584, 12672, 21312, 29888, 37024, 37664, 37024, 21696, 21312, 12672, 3584, 0]],
        '⑭' : [12, 13, 0, -2, 12, [0, 3584, 12672, 21312, 29504, 38176, 38176, 38176, 22464, 20800, 12672, 3584, 0]],
        '⑮' : [12, 13, 0, -2, 12, [0, 3584, 12672, 22464, 29760, 38688, 37024, 37024, 21696, 21312, 12672, 3584, 0]],
        '⑯' : [12, 13, 0, -2, 12, [0, 3584, 12672, 21312, 29760, 38688, 38048, 38048, 21696, 21312, 12672, 3584, 0]],
        '⑰' : [12, 13, 0, -2, 12, [0, 3584, 12672, 22464, 28864, 37152, 37152, 37408, 21056, 21056, 12672, 3584, 0]],
        '⑱' : [12, 13, 0, -2, 12, [0, 3584, 12672, 21312, 29888, 38048, 37664, 38048, 21696, 21312, 12672, 3584, 0]],
        '⑲' : [12, 13, 0, -2, 12, [0, 3584, 12672, 21312, 29888, 38048, 38048, 37792, 20672, 22336, 12672, 3584, 0]],
        '⑳' : [12, 13, 0, -2, 12, [0, 3584, 12672, 23872, 22208, 34464, 35488, 37536, 21184, 23872, 12672, 3584, 0]],
        'Ⅰ' : [12, 13, 0, -2, 12, [0, 7936, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 7936, 0, 0]],
        'Ⅱ' : [12, 13, 0, -2, 12, [0, 8064, 2304, 2304, 2304, 2304, 2304, 2304, 2304, 2304, 8064, 0, 0]],
        'Ⅲ' : [12, 13, 0, -2, 12, [0, 32704, 9344, 9344, 9344, 9344, 9344, 9344, 9344, 9344, 32704, 0, 0]],
        'Ⅳ' : [12, 13, 0, -2, 12, [0, 32192, 10368, 10368, 10368, 9472, 9472, 9472, 8704, 8704, 29184, 0, 0]],
        'Ⅴ' : [12, 13, 0, -2, 12, [0, 29120, 8320, 8320, 4352, 4352, 4352, 2560, 2560, 1024, 1024, 0, 0]],
        'Ⅵ' : [12, 13, 0, -2, 12, [0, 30656, 8832, 8832, 8832, 5248, 5248, 5248, 2176, 2176, 2496, 0, 0]],
        'Ⅶ' : [12, 13, 0, -2, 12, [0, 61408, 17728, 17728, 17728, 10560, 10560, 10560, 4416, 4416, 5088, 0, 0]],
        'Ⅷ' : [12, 13, 0, -2, 12, [0, 57312, 21824, 21824, 21824, 21824, 21824, 9536, 9536, 9536, 12256, 0, 0]],
        'Ⅸ' : [12, 13, 0, -2, 12, [0, 32192, 10368, 9472, 9472, 8704, 8704, 9472, 9472, 10368, 32192, 0, 0]],
        'Ⅹ' : [12, 13, 0, -2, 12, [0, 29120, 8320, 4352, 2560, 1024, 1024, 2560, 4352, 8320, 29120, 0, 0]],
        '㍉' : [12, 13, 0, -2, 12, [0, 57344, 6144, 57344, 6144, 57344, 6688, 544, 544, 544, 64, 384, 0]],
        '㌔' : [12, 13, 0, -2, 12, [0, 8192, 63488, 8192, 8192, 64512, 4096, 5088, 544, 544, 544, 992, 0]],
        '㌢' : [12, 13, 0, -2, 12, [0, 16896, 63776, 18464, 16448, 15232, 0, 30720, 4096, 64512, 4096, 24576, 0]],
        '㍍' : [12, 13, 0, -2, 12, [0, 2048, 18432, 13280, 6144, 58368, 0, 17024, 17024, 31360, 17056, 17600, 0]],
        '㌘' : [12, 13, 0, -2, 12, [0, 21952, 30720, 35808, 2080, 4128, 57792, 8192, 8192, 18432, 17408, 64512, 0]],
        '㌧' : [12, 13, 0, -2, 12, [0, 16384, 16384, 16384, 30720, 16384, 16384, 17952, 288, 32, 64, 1920, 0]],
        '㌃' : [12, 13, 0, -2, 12, [0, 64512, 1024, 10240, 9184, 8192, 16384, 4096, 20480, 20992, 20992, 39936, 0]],
        '㌶' : [12, 13, 0, -2, 12, [0, 256, 25056, 37664, 34848, 1088, 16768, 28736, 37184, 20800, 12096, 49504, 0]],
        '㍑' : [12, 13, 0, -2, 12, [0, 36864, 37536, 37536, 4128, 25024, 0, 33920, 33920, 62592, 33952, 33984, 0]],
        '㍗' : [12, 13, 0, -2, 12, [0, 63488, 35488, 35488, 2080, 29120, 0, 16384, 16384, 30720, 16384, 16384, 0]],
        '㌍' : [12, 13, 0, -2, 12, [0, 16384, 64480, 18976, 18976, 39904, 0, 18432, 18432, 19424, 2048, 12288, 0]],
        '㌦' : [12, 13, 0, -2, 12, [0, 21504, 21504, 16384, 30720, 16384, 17024, 17024, 640, 672, 672, 1216, 0]],
        '㌣' : [12, 13, 0, -2, 12, [0, 16896, 63776, 18464, 16448, 15232, 0, 16384, 16384, 30720, 16384, 16384, 0]],
        '㌫' : [12, 13, 0, -2, 12, [0, 20992, 21760, 20992, 18912, 34816, 0, 18752, 62784, 20832, 16704, 30272, 0]],
        '㍊' : [12, 13, 0, -2, 12, [0, 62016, 576, 62016, 64, 61824, 0, 43328, 33088, 44864, 37184, 37216, 0]],
        '㌻' : [12, 13, 0, -2, 12, [0, 1024, 27136, 37888, 35296, 1024, 0, 49792, 2688, 51200, 4096, 57344, 0]],
        '㎜' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 62400, 43680, 43680, 43680, 43680, 43680, 0, 0]],
        '㎝' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 29632, 35488, 33440, 33440, 35488, 29344, 0, 0]],
        '㎞' : [12, 13, 0, -2, 12, [0, 0, 32768, 32768, 32768, 37824, 41632, 49824, 41632, 37536, 35488, 0, 0]],
        '㎎' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 96, 61824, 43584, 43392, 43520, 43456, 43552, 448, 0]],
        '㎏' : [12, 13, 0, -2, 12, [0, 0, 32768, 32768, 32864, 37248, 41536, 49536, 41472, 37312, 35360, 448, 0]],
        '㏄' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 29120, 35360, 33280, 33280, 35360, 29120, 0, 0]],
        '㎡' : [12, 13, 0, -2, 12, [0, 192, 288, 64, 128, 64992, 37376, 37376, 37376, 37376, 37376, 0, 0]],
        '㍻' : [12, 13, 0, -2, 12, [0, 160, 64480, 8832, 43680, 43680, 43840, 9024, 64832, 9536, 11168, 10400, 0]],
        '〝' : [12, 13, 0, -2, 12, [320, 320, 640, 640, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        '〟' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 0, 10240, 10240, 20480, 20480, 0, 0]],
        '№' : [12, 13, 0, -2, 12, [0, 0, 34816, 51200, 51200, 43392, 43584, 39488, 39488, 35392, 35232, 0, 0]],
        '㏍' : [12, 13, 0, -2, 12, [0, 0, 19008, 19008, 21120, 21120, 25344, 21120, 21120, 19008, 19008, 0, 0]],
        '℡' : [12, 13, 0, -2, 12, [0, 0, 63616, 8320, 11904, 10368, 10368, 11904, 10368, 10368, 12000, 0, 0]],
        '㊤' : [12, 13, 0, -2, 12, [0, 3584, 12672, 17472, 17472, 34592, 33824, 33824, 32704, 16448, 12672, 3584, 0]],
        '㊥' : [12, 13, 0, -2, 12, [0, 3584, 12672, 17472, 32704, 42144, 42144, 49056, 17472, 17472, 12672, 3584, 0]],
        '㊦' : [12, 13, 0, -2, 12, [0, 3584, 12672, 16448, 32704, 33824, 34336, 34080, 17472, 17472, 12672, 3584, 0]],
        '㊧' : [12, 13, 0, -2, 12, [0, 3584, 12672, 18496, 32704, 34848, 36640, 37408, 25152, 20416, 12672, 3584, 0]],
        '㊨' : [12, 13, 0, -2, 12, [0, 3584, 12672, 18496, 32704, 34848, 40736, 43296, 18752, 20288, 12672, 3584, 0]],
        '㈱' : [12, 13, 0, -2, 12, [0, 8320, 21824, 38176, 65504, 38176, 47072, 55584, 37792, 38240, 20800, 8320, 0]],
        '㈲' : [12, 13, 0, -2, 12, [0, 8320, 18496, 49056, 34848, 36640, 39200, 44832, 35104, 36640, 18752, 8320, 0]],
        '㈹' : [12, 13, 0, -2, 12, [0, 8320, 19008, 35488, 37408, 40864, 45600, 53792, 37408, 37408, 20800, 8320, 0]],
        '㍾' : [12, 13, 0, -2, 12, [0, 14464, 60544, 43648, 47392, 60704, 44000, 47104, 59872, 11040, 11552, 18912, 0]],
        '㍽' : [12, 13, 0, -2, 12, [0, 8192, 10208, 8320, 63616, 8832, 8928, 21120, 21120, 21120, 35456, 36832, 0]],
        '㍼' : [12, 13, 0, -2, 12, [0, 30464, 10752, 60128, 44960, 47776, 58016, 48800, 43936, 43680, 60064, 15072, 0]],
        '≒' : [12, 13, 0, -2, 12, [0, 4096, 14336, 4096, 0, 32704, 0, 32704, 0, 256, 896, 256, 0]],
        '≡' : [12, 13, 0, -2, 12, [0, 0, 0, 32704, 0, 0, 32704, 0, 0, 32704, 0, 0, 0]],
        '∫' : [12, 13, 0, -2, 12, [1024, 2560, 2048, 2048, 2048, 1024, 1024, 512, 512, 512, 2560, 1024, 0]],
        '∮' : [12, 13, 0, -2, 12, [1024, 2560, 2048, 3584, 5376, 5376, 5376, 5376, 3584, 512, 2560, 1024, 0]],
        '∑' : [12, 13, 0, -2, 12, [0, 0, 32704, 8192, 4096, 2048, 1024, 2048, 4096, 8192, 32704, 0, 0]],
        '√' : [12, 13, 0, -2, 12, [992, 512, 512, 1024, 1024, 17408, 18432, 43008, 43008, 4096, 4096, 0, 0]],
        '⊥' : [12, 13, 0, -2, 12, [0, 0, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 32704, 0, 0, 0]],
        '∠' : [12, 13, 0, -2, 12, [0, 0, 128, 256, 512, 1024, 2048, 4096, 8192, 32704, 0, 0, 0]],
        '∟' : [12, 13, 0, -2, 12, [0, 0, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 16320, 0, 0, 0]],
        '⊿' : [12, 13, 0, -2, 12, [0, 0, 128, 384, 640, 1152, 2176, 4224, 8320, 32640, 0, 0, 0]],
        '∵' : [12, 13, 0, -2, 12, [0, 0, 8320, 29120, 8320, 0, 0, 1024, 3584, 1024, 0, 0, 0]],
        '∩' : [12, 13, 0, -2, 12, [0, 0, 3584, 4352, 8320, 8320, 8320, 8320, 8320, 8320, 8320, 0, 0]],
        '∪' : [12, 13, 0, -2, 12, [0, 0, 8320, 8320, 8320, 8320, 8320, 8320, 8320, 4352, 3584, 0, 0]],
        '亜' : [12, 13, 0, -2, 12, [0, 65504, 2560, 2560, 32704, 19008, 19008, 19008, 32704, 2560, 2560, 65504, 0]],
        '唖' : [12, 13, 0, -2, 12, [0, 8160, 57984, 41600, 45024, 43680, 43680, 43680, 45024, 41600, 57984, 8160, 0]],
        '娃' : [12, 13, 0, -2, 12, [256, 16640, 18400, 61696, 20736, 24544, 20736, 37120, 59360, 12544, 16640, 20448, 0]],
        '阿' : [12, 13, 0, -2, 12, [0, 61408, 41024, 41024, 52800, 43584, 43584, 43584, 44608, 49216, 32832, 33216, 0]],
        '哀' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 0, 16256, 8320, 8320, 16256, 5152, 57920, 8576, 31840, 0]],
        '愛' : [12, 13, 0, -2, 12, [0, 32576, 18496, 9344, 65504, 37408, 20800, 36768, 12416, 2304, 3584, 61920, 0]],
        '挨' : [12, 13, 0, -2, 12, [512, 16960, 17472, 65504, 18432, 20448, 18688, 28928, 57312, 17024, 17472, 55328, 0]],
        '姶' : [12, 13, 0, -2, 12, [0, 17344, 17440, 63488, 20480, 22496, 20480, 36864, 59360, 13344, 17440, 18400, 0]],
        '逢' : [12, 13, 0, -2, 12, [0, 1984, 38976, 17536, 1888, 6400, 59328, 8448, 12256, 8448, 20480, 36832, 0]],
        '葵' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 64800, 19008, 12672, 57184, 1024, 32704, 4352, 57568, 0]],
        '茜' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 4608, 32704, 19008, 21056, 25024, 16448, 32704, 0]],
        '穐' : [12, 13, 0, -2, 12, [0, 7104, 58496, 12256, 9504, 63456, 9504, 28640, 26912, 45024, 43264, 8416, 0]],
        '悪' : [12, 13, 0, -2, 12, [0, 65504, 2560, 32704, 19008, 32704, 2560, 65504, 1024, 4672, 20512, 36736, 0]],
        '握' : [12, 13, 0, -2, 12, [0, 20448, 18464, 61408, 18432, 20448, 19008, 27616, 51328, 21472, 20608, 51168, 0]],
        '渥' : [12, 13, 0, -2, 12, [0, 4064, 34848, 20448, 2048, 36832, 19008, 3040, 18560, 21472, 36992, 34784, 0]],
        '旭' : [12, 13, 0, -2, 12, [0, 18368, 17472, 17472, 63424, 21568, 21568, 21568, 22464, 20480, 36896, 36832, 0]],
        '葦' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 16256, 2176, 65504, 4352, 16320, 9216, 65504, 1024, 0]],
        '芦' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 0, 16320, 8256, 16320, 8192, 16384, 16384, 0]],
        '鯵' : [12, 13, 0, -2, 12, [512, 29248, 37952, 10208, 63744, 45024, 63808, 43552, 64704, 768, 43104, 43904, 0]],
        '梓' : [12, 13, 0, -2, 12, [256, 8448, 12256, 62528, 9280, 9344, 28640, 24832, 45312, 45024, 8448, 8448, 0]],
        '圧' : [12, 13, 0, -2, 12, [0, 32736, 16384, 16640, 16640, 16640, 24544, 16640, 16640, 33024, 33024, 8160, 0]],
        '斡' : [12, 13, 0, -2, 12, [8192, 8576, 64064, 9248, 63808, 35008, 64064, 35136, 63584, 8384, 64320, 8256, 0]],
        '扱' : [12, 13, 0, -2, 12, [0, 24512, 17472, 58496, 17664, 17856, 18496, 27200, 53568, 20608, 17216, 52256, 0]],
        '宛' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 16416, 31680, 19008, 27200, 39616, 2560, 12832, 49632, 0]],
        '姐' : [12, 13, 0, -2, 12, [0, 18400, 17440, 62496, 22496, 21536, 21536, 38880, 58400, 13344, 17440, 20448, 0]],
        '虻' : [12, 13, 0, -2, 12, [256, 8448, 8448, 65504, 43520, 43520, 43520, 64000, 8704, 10752, 10752, 63968, 0]],
        '飴' : [12, 13, 0, -2, 12, [0, 28928, 35072, 8736, 64032, 35808, 63488, 34816, 64480, 37408, 35360, 63456, 0]],
        '絢' : [12, 13, 0, -2, 12, [1024, 9216, 18400, 34848, 59424, 10144, 17568, 63392, 9376, 47008, 40992, 8384, 0]],
        '綾' : [12, 13, 0, -2, 12, [256, 8448, 20448, 33024, 61408, 8832, 19552, 62400, 11328, 41536, 41856, 11360, 0]],
        '鮎' : [12, 13, 0, -2, 12, [256, 28928, 37120, 8672, 63744, 43264, 63744, 44000, 64032, 544, 43552, 44000, 0]],
        '或' : [12, 13, 0, -2, 12, [256, 288, 65504, 256, 30976, 18720, 18592, 30880, 64, 6336, 58144, 3104, 0]],
        '粟' : [12, 13, 0, -2, 12, [0, 65504, 2560, 32704, 19008, 19008, 32704, 9344, 65504, 3328, 13440, 50272, 0]],
        '袷' : [12, 13, 0, -2, 12, [0, 9152, 9248, 63488, 4096, 6112, 8192, 24576, 47072, 9248, 9248, 10208, 0]],
        '安' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 34848, 2048, 65504, 4224, 4224, 12544, 3840, 28864, 0]],
        '庵' : [12, 13, 0, -2, 12, [512, 512, 32736, 17408, 32736, 19072, 21056, 32736, 21056, 24512, 37408, 33760, 0]],
        '按' : [12, 13, 0, -2, 12, [512, 16896, 24544, 61472, 21024, 16896, 24544, 25664, 50240, 19584, 17280, 56416, 0]],
        '暗' : [12, 13, 0, -2, 12, [256, 256, 65504, 37440, 37440, 40928, 61440, 38880, 37920, 38880, 62496, 2016, 0]],
        '案' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 34848, 65504, 4224, 7936, 9344, 65504, 3328, 13440, 50272, 0]],
        '闇' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 35360, 49056, 37152, 40736, 37152, 40800, 0]],
        '鞍' : [12, 13, 0, -2, 12, [256, 20736, 65504, 21536, 29984, 8448, 65504, 43584, 64064, 8832, 63872, 9824, 0]],
        '杏' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 3328, 13440, 50240, 1056, 16256, 8320, 8320, 16256, 0]],
        '以' : [12, 13, 0, -2, 12, [0, 8256, 9280, 9280, 8768, 8768, 8256, 8320, 14464, 49472, 1568, 6176, 0]],
        '伊' : [12, 13, 0, -2, 12, [0, 12256, 8480, 16672, 24544, 49440, 16672, 20448, 16640, 16640, 16896, 19456, 0]],
        '位' : [12, 13, 0, -2, 12, [256, 8448, 8448, 24544, 16384, 50240, 17472, 17472, 17472, 17536, 16512, 24544, 0]],
        '依' : [12, 13, 0, -2, 12, [256, 8448, 16352, 16896, 16896, 50464, 18720, 22848, 26752, 18560, 19264, 23584, 0]],
        '偉' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16928, 24544, 51264, 20416, 16640, 20448, 18688, 24544, 16640, 0]],
        '囲' : [12, 13, 0, -2, 12, [0, 65504, 32800, 35104, 49056, 35104, 35104, 49056, 37152, 37152, 32800, 65504, 0]],
        '夷' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 1088, 32704, 17408, 65504, 2592, 12672, 49248, 0]],
        '委' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 3328, 13440, 50272, 2048, 65504, 4224, 7936, 24768, 0]],
        '威' : [12, 13, 0, -2, 12, [128, 160, 32736, 16512, 32384, 18592, 32672, 21152, 21056, 21568, 36000, 45856, 0]],
        '尉' : [12, 13, 0, -2, 12, [32, 32288, 16928, 32736, 16416, 32288, 16672, 32416, 18464, 43552, 43552, 6240, 0]],
        '惟' : [12, 13, 0, -2, 12, [512, 8768, 8768, 10208, 42112, 45024, 46208, 42112, 42976, 9344, 9344, 10208, 0]],
        '意' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 65504, 8320, 16256, 8320, 16256, 4672, 20512, 36736, 0]],
        '慰' : [12, 13, 0, -2, 12, [64, 63552, 36832, 63552, 33344, 63808, 8256, 43072, 42176, 4096, 20544, 36640, 0]],
        '易' : [12, 13, 0, -2, 12, [0, 16320, 8256, 16320, 8256, 16320, 4096, 16352, 18720, 35104, 4640, 9408, 0]],
        '椅' : [12, 13, 0, -2, 12, [256, 8448, 12256, 62080, 9280, 10272, 28640, 24608, 42784, 42272, 10016, 8288, 0]],
        '為' : [12, 13, 0, -2, 12, [8704, 4608, 1024, 32640, 2176, 8128, 8256, 32736, 32800, 21792, 21792, 192, 0]],
        '畏' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 32704, 0, 65504, 9280, 8768, 8576, 63584, 0]],
        '異' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 32704, 4352, 65504, 4352, 65504, 8320, 16448, 0]],
        '移' : [12, 13, 0, -2, 12, [256, 6400, 58336, 11296, 8768, 61824, 9728, 25568, 27680, 41536, 41344, 11776, 0]],
        '維' : [12, 13, 0, -2, 12, [512, 8768, 16960, 34784, 62592, 12256, 17536, 62592, 10208, 46208, 42112, 10208, 0]],
        '緯' : [12, 13, 0, -2, 12, [128, 8320, 20448, 33056, 61408, 9248, 18400, 61568, 10208, 46208, 45024, 8320, 0]],
        '胃' : [12, 13, 0, -2, 12, [0, 32736, 17440, 32736, 17440, 32736, 8256, 16320, 8256, 16320, 8256, 8384, 0]],
        '萎' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 65504, 5376, 25792, 2048, 65504, 4224, 7936, 24768, 0]],
        '衣' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 2048, 4640, 8768, 24960, 41216, 8320, 9792, 30752, 0]],
        '謂' : [12, 13, 0, -2, 12, [0, 61408, 2336, 61408, 2336, 61408, 1088, 1984, 58432, 42944, 42048, 58560, 0]],
        '違' : [12, 13, 0, -2, 12, [256, 256, 20448, 8480, 4064, 1088, 59360, 9344, 12256, 8320, 20480, 36832, 0]],
        '遺' : [12, 13, 0, -2, 12, [256, 256, 36832, 18720, 8160, 1088, 59328, 9280, 10176, 10272, 20480, 36832, 0]],
        '医' : [12, 13, 0, -2, 12, [0, 65504, 36864, 40896, 37376, 41472, 49088, 34048, 34944, 45120, 32768, 65504, 0]],
        '井' : [12, 13, 0, -2, 12, [4352, 4352, 4352, 65504, 4352, 4352, 4352, 65504, 8448, 8448, 16640, 16640, 0]],
        '亥' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4224, 8448, 4608, 3136, 28800, 256, 1536, 6528, 57440, 0]],
        '域' : [12, 13, 0, -2, 12, [128, 16544, 24544, 16512, 61056, 19104, 19104, 20032, 16448, 24224, 57632, 1568, 0]],
        '育' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2304, 4224, 65472, 8352, 16256, 8320, 16256, 8320, 8576, 0]],
        '郁' : [12, 13, 0, -2, 12, [4096, 4576, 65312, 8480, 8512, 32128, 50496, 32032, 17696, 32032, 17856, 19712, 0]],
        '磯' : [12, 13, 0, -2, 12, [256, 1344, 63904, 9536, 10912, 12256, 28928, 55264, 21632, 22176, 22720, 29472, 0]],
        '一' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 0, 65504, 0, 0, 0, 0, 0, 0]],
        '壱' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 0, 65504, 36896, 37280, 7680, 4160, 4032, 0]],
        '溢' : [12, 13, 0, -2, 12, [4224, 2176, 33024, 32736, 2176, 34944, 20544, 8160, 21824, 21824, 38208, 49120, 0]],
        '逸' : [12, 13, 0, -2, 12, [0, 1984, 34944, 24544, 2336, 2336, 61408, 8832, 9344, 10464, 20480, 36832, 0]],
        '稲' : [12, 13, 0, -2, 12, [0, 8096, 57888, 10560, 9280, 61440, 11232, 27168, 27616, 43552, 43552, 11232, 0]],
        '茨' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 18400, 10528, 4416, 256, 8832, 17472, 38944, 0]],
        '芋' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 32704, 1024, 1024, 65504, 1024, 1024, 7168, 0]],
        '鰯' : [12, 13, 0, -2, 12, [0, 32736, 37408, 12000, 63616, 44768, 62240, 43680, 63072, 2720, 45856, 44224, 0]],
        '允' : [12, 13, 0, -2, 12, [2048, 2048, 4224, 4224, 8256, 65472, 4640, 4640, 4608, 4608, 8736, 49632, 0]],
        '印' : [12, 13, 0, -2, 12, [0, 6144, 58336, 33312, 33312, 64032, 33312, 33312, 33312, 33312, 64224, 512, 0]],
        '咽' : [12, 13, 0, -2, 12, [0, 4064, 59680, 43296, 45024, 43296, 43296, 43296, 43680, 44128, 59424, 4064, 0]],
        '員' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 0, 32704, 16448, 32704, 16448, 32704, 8320, 49248, 0]],
        '因' : [12, 13, 0, -2, 12, [0, 65504, 33824, 33824, 49056, 33824, 33824, 35360, 37152, 41120, 32800, 65504, 0]],
        '姻' : [12, 13, 0, -2, 12, [0, 20448, 18720, 63776, 20448, 18720, 18720, 35104, 60064, 15456, 18464, 20448, 0]],
        '引' : [12, 13, 0, -2, 12, [0, 63552, 2112, 2112, 63552, 32832, 32832, 63552, 2112, 2112, 4160, 24640, 0]],
        '飲' : [12, 13, 0, -2, 12, [512, 29184, 35808, 8864, 64160, 35488, 63616, 34944, 63808, 37184, 35360, 62496, 0]],
        '淫' : [12, 13, 0, -2, 12, [0, 8096, 33312, 18752, 1088, 32768, 20448, 256, 24544, 16640, 33024, 40928, 0]],
        '胤' : [12, 13, 0, -2, 12, [1024, 18496, 20800, 24384, 17472, 18752, 24384, 20800, 24384, 20768, 40736, 37152, 0]],
        '蔭' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 62400, 37920, 41920, 41088, 38880, 37376, 58400, 34784, 0]],
        '院' : [12, 13, 0, -2, 12, [256, 57600, 45024, 43040, 51168, 40960, 40960, 45024, 41600, 49792, 33952, 39136, 0]],
        '陰' : [12, 13, 0, -2, 12, [0, 58240, 42048, 43040, 55232, 40960, 42944, 41088, 45024, 50176, 34848, 36832, 0]],
        '隠' : [12, 13, 0, -2, 12, [0, 61408, 42272, 42304, 53216, 40992, 45024, 40992, 45024, 50432, 37952, 41888, 0]],
        '韻' : [12, 13, 0, -2, 12, [8192, 10208, 62496, 22496, 20480, 65504, 1056, 63456, 37920, 63456, 37440, 64544, 0]],
        '吋' : [12, 13, 0, -2, 12, [64, 64, 61504, 40928, 36928, 36928, 37952, 37440, 36928, 36928, 61504, 448, 0]],
        '右' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 1024, 2048, 8128, 12352, 53312, 4160, 4160, 8128, 0]],
        '宇' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 32800, 16256, 1024, 1024, 65504, 1024, 1024, 7168, 0]],
        '烏' : [12, 13, 0, -2, 12, [1024, 2048, 16256, 8320, 16256, 8192, 16352, 8192, 16352, 32, 21792, 38240, 0]],
        '羽' : [12, 13, 0, -2, 12, [0, 64480, 2080, 2080, 35360, 18720, 6240, 10400, 18720, 35360, 2080, 6240, 0]],
        '迂' : [12, 13, 0, -2, 12, [0, 4064, 16512, 8320, 128, 4064, 57472, 8320, 8320, 8576, 20480, 36832, 0]],
        '雨' : [12, 13, 0, -2, 12, [0, 65504, 1024, 1024, 65504, 33824, 42272, 38048, 33824, 42272, 38048, 33888, 0]],
        '卯' : [12, 13, 0, -2, 12, [0, 3552, 28960, 16672, 17696, 17696, 17696, 23840, 58656, 1376, 2304, 12544, 0]],
        '鵜' : [12, 13, 0, -2, 12, [34944, 18688, 5056, 64064, 11200, 64064, 41952, 64000, 11232, 26656, 42272, 10592, 0]],
        '窺' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 30656, 9280, 65472, 9280, 65472, 8832, 21152, 36064, 0]],
        '丑' : [12, 13, 0, -2, 12, [0, 32640, 2176, 2176, 2176, 2176, 65504, 4352, 4352, 4352, 4352, 65504, 0]],
        '碓' : [12, 13, 0, -2, 12, [512, 576, 64064, 10208, 9344, 12256, 29824, 54400, 22496, 21632, 21632, 30688, 0]],
        '臼' : [12, 13, 0, -2, 12, [0, 1536, 6144, 25056, 16416, 16416, 16416, 31200, 16416, 16416, 16416, 32736, 0]],
        '渦' : [12, 13, 0, -2, 12, [0, 4032, 34880, 19392, 2624, 40928, 20512, 6048, 21664, 21664, 38816, 36960, 0]],
        '嘘' : [12, 13, 0, -2, 12, [512, 960, 57856, 49120, 42016, 49088, 41984, 42976, 45728, 43712, 57984, 8160, 0]],
        '唄' : [12, 13, 0, -2, 12, [0, 2016, 62496, 38880, 37920, 38880, 37920, 37920, 38880, 61440, 1088, 2080, 0]],
        '欝' : [12, 13, 0, -2, 12, [8320, 10880, 62944, 27328, 42144, 65504, 37152, 65504, 34080, 64672, 34848, 63072, 0]],
        '蔚' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 63552, 36832, 63552, 33344, 63808, 8256, 43072, 42176, 0]],
        '鰻' : [12, 13, 0, -2, 12, [0, 29632, 37440, 9152, 64064, 45024, 64160, 45024, 64064, 576, 43392, 44640, 0]],
        '姥' : [12, 13, 0, -2, 12, [512, 16896, 20448, 62016, 21120, 24544, 20992, 38080, 59136, 13312, 17440, 18400, 0]],
        '厩' : [12, 13, 0, -2, 12, [0, 65504, 32768, 48608, 42304, 48448, 42304, 49120, 41024, 43200, 42304, 47712, 0]],
        '浦' : [12, 13, 0, -2, 12, [256, 288, 40928, 16640, 4064, 35104, 20448, 2336, 4064, 10528, 18720, 35168, 0]],
        '瓜' : [12, 13, 0, -2, 12, [0, 32736, 18560, 18560, 18560, 18560, 18560, 18496, 18752, 35136, 48928, 32800, 0]],
        '閏' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 32800, 49056, 33824, 40736, 33824, 49056, 0]],
        '噂' : [12, 13, 0, -2, 12, [1088, 1152, 65504, 41600, 45024, 43680, 44256, 43040, 49120, 41024, 58432, 704, 0]],
        '云' : [12, 13, 0, -2, 12, [0, 32704, 0, 0, 0, 65504, 4096, 4224, 8320, 8320, 17216, 31808, 0]],
        '運' : [12, 13, 0, -2, 12, [0, 8160, 37152, 20448, 2336, 4064, 59680, 12256, 8448, 12256, 20736, 36832, 0]],
        '雲' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 1024, 32704, 0, 65504, 8320, 32576, 0]],
        '荏' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 8192, 12256, 16640, 49408, 24544, 16640, 16640, 20448, 0]],
        '餌' : [12, 13, 0, -2, 12, [0, 30688, 35392, 8768, 64448, 35392, 64064, 35776, 64064, 41568, 38848, 59456, 0]],
        '叡' : [12, 13, 0, -2, 12, [4096, 7648, 4128, 65056, 37536, 21664, 43680, 31904, 50752, 31808, 17568, 32544, 0]],
        '営' : [12, 13, 0, -2, 12, [1024, 9280, 9344, 65504, 37152, 37152, 40736, 1024, 16256, 8320, 8320, 16256, 0]],
        '嬰' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 9344, 18496, 65504, 4224, 7936, 24768, 0]],
        '影' : [12, 13, 0, -2, 12, [0, 64544, 33856, 64640, 33792, 64544, 4160, 65152, 18432, 30752, 37952, 45696, 0]],
        '映' : [12, 13, 0, -2, 12, [256, 256, 61408, 43296, 43296, 43296, 59680, 49120, 41216, 41600, 58432, 6176, 0]],
        '曳' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 17472, 32704, 17472, 32704, 576, 640, 768, 3360, 61664, 0]],
        '栄' : [12, 13, 0, -2, 12, [2112, 17536, 8448, 65504, 33824, 1024, 32704, 1024, 3328, 13440, 50272, 1024, 0]],
        '永' : [12, 13, 0, -2, 12, [6144, 1536, 0, 15424, 1152, 62720, 5632, 5376, 9344, 17472, 33824, 7168, 0]],
        '泳' : [12, 13, 0, -2, 12, [512, 256, 32768, 20224, 288, 40256, 17792, 1280, 18816, 18752, 37152, 37632, 0]],
        '洩' : [12, 13, 0, -2, 12, [256, 256, 40928, 20768, 8160, 37152, 24544, 288, 320, 8576, 18080, 39008, 0]],
        '瑛' : [12, 13, 0, -2, 12, [1088, 1088, 61408, 17728, 16640, 61408, 18720, 18720, 24544, 17024, 58432, 6176, 0]],
        '盈' : [12, 13, 0, -2, 12, [0, 65408, 8448, 15840, 17696, 22560, 36032, 36864, 32704, 19008, 19008, 65504, 0]],
        '穎' : [12, 13, 0, -2, 12, [0, 36832, 61696, 34784, 62496, 2016, 62496, 10208, 64544, 10208, 29248, 44064, 0]],
        '頴' : [12, 13, 0, -2, 12, [0, 36832, 61696, 34784, 62496, 2016, 62496, 2016, 64544, 10208, 41536, 44064, 0]],
        '英' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 5376, 1024, 16256, 9344, 9344, 65504, 2560, 12672, 49248, 0]],
        '衛' : [12, 13, 0, -2, 12, [1024, 9440, 24320, 34048, 8160, 10528, 20256, 49696, 24480, 21024, 32672, 16992, 0]],
        '詠' : [12, 13, 0, -2, 12, [512, 57600, 0, 61184, 288, 57664, 7552, 1280, 58752, 43328, 45344, 59136, 0]],
        '鋭' : [12, 13, 0, -2, 12, [1088, 12864, 18560, 34784, 62496, 9248, 63456, 8832, 45696, 41600, 9376, 63712, 0]],
        '液' : [12, 13, 0, -2, 12, [256, 256, 49120, 18688, 2496, 37440, 21056, 13632, 21824, 20608, 37248, 38496, 0]],
        '疫' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 53120, 18592, 22752, 49152, 24512, 18496, 34688, 47200, 0]],
        '益' : [12, 13, 0, -2, 12, [8448, 4352, 512, 65504, 4352, 8320, 16448, 65504, 19008, 19008, 19008, 65504, 0]],
        '駅' : [12, 13, 0, -2, 12, [0, 64480, 41504, 64032, 41504, 64480, 41600, 64128, 2624, 43584, 3104, 13344, 0]],
        '悦' : [12, 13, 0, -2, 12, [1088, 8768, 8320, 10208, 46112, 44064, 42976, 41600, 41600, 8832, 9376, 14560, 0]],
        '謁' : [12, 13, 0, -2, 12, [0, 61376, 2112, 61376, 2112, 61376, 1024, 4064, 63520, 43296, 44832, 57536, 0]],
        '越' : [12, 13, 0, -2, 12, [8320, 8352, 64480, 8832, 8864, 65184, 8768, 14912, 41632, 42784, 61440, 36832, 0]],
        '閲' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 35360, 40736, 37152, 40736, 35488, 45984, 0]],
        '榎' : [12, 13, 0, -2, 12, [0, 12256, 8448, 63456, 9248, 10208, 25632, 26592, 42048, 43584, 8576, 11872, 0]],
        '厭' : [12, 13, 0, -2, 12, [0, 65504, 42112, 48288, 33760, 48256, 42112, 48320, 42304, 48448, 42528, 44064, 0]],
        '円' : [12, 13, 0, -2, 12, [0, 65504, 33824, 33824, 33824, 33824, 65504, 32800, 32800, 32800, 32800, 32992, 0]],
        '園' : [12, 13, 0, -2, 12, [0, 65504, 33824, 49056, 33824, 65504, 37152, 40736, 35488, 47392, 34976, 65504, 0]],
        '堰' : [12, 13, 0, -2, 12, [0, 20448, 18976, 19424, 59936, 19424, 18688, 20448, 18752, 18880, 59936, 4064, 0]],
        '奄' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 9344, 32736, 42112, 16256, 9344, 16256, 1056, 2016, 0]],
        '宴' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 41120, 49056, 8320, 16256, 2048, 65504, 4224, 7936, 24768, 0]],
        '延' : [12, 13, 0, -2, 12, [0, 192, 61184, 8448, 16640, 34272, 58624, 9472, 42240, 20448, 45056, 36832, 0]],
        '怨' : [12, 13, 0, -2, 12, [0, 31712, 18976, 27168, 39648, 2560, 12832, 49632, 1024, 4672, 20512, 36736, 0]],
        '掩' : [12, 13, 0, -2, 12, [256, 16640, 24544, 58432, 18720, 24544, 18720, 28640, 51488, 20448, 16640, 49632, 0]],
        '援' : [12, 13, 0, -2, 12, [0, 24544, 17696, 58688, 20448, 16640, 24544, 25088, 51136, 23104, 16768, 52832, 0]],
        '沿' : [12, 13, 0, -2, 12, [0, 960, 32832, 17472, 1056, 34848, 18432, 2016, 1056, 9248, 17440, 34784, 0]],
        '演' : [12, 13, 0, -2, 12, [256, 256, 40928, 20512, 4064, 33024, 20448, 2336, 20448, 18720, 36832, 45088, 0]],
        '炎' : [12, 13, 0, -2, 12, [1024, 1024, 9280, 17536, 2560, 12672, 50272, 9344, 9344, 2560, 12672, 49248, 0]],
        '焔' : [12, 13, 0, -2, 12, [0, 9152, 9280, 43136, 41952, 43552, 10784, 11232, 23072, 18976, 35360, 35808, 0]],
        '煙' : [12, 13, 0, -2, 12, [0, 12256, 8832, 45024, 43680, 43680, 12256, 8448, 22496, 20736, 33024, 36832, 0]],
        '燕' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 2560, 11936, 57536, 11904, 27296, 44768, 0, 21056, 35104, 0]],
        '猿' : [12, 13, 0, -2, 12, [256, 8448, 45024, 16640, 24544, 42048, 9280, 26560, 41632, 15520, 9280, 53024, 0]],
        '縁' : [12, 13, 0, -2, 12, [0, 10176, 16448, 34752, 57408, 12256, 16640, 63136, 8640, 42656, 47264, 9088, 0]],
        '艶' : [12, 13, 0, -2, 12, [9216, 9664, 65088, 42112, 65504, 42144, 64672, 18400, 31744, 9216, 9248, 65504, 0]],
        '苑' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16384, 31680, 19008, 27200, 39616, 2560, 12832, 49632, 0]],
        '薗' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 33824, 49056, 37152, 40736, 43680, 35104, 65504, 0]],
        '遠' : [12, 13, 0, -2, 12, [256, 256, 36832, 16640, 8160, 1088, 59328, 8480, 9536, 10528, 20480, 36832, 0]],
        '鉛' : [12, 13, 0, -2, 12, [0, 12736, 18496, 33344, 61984, 9248, 62464, 9184, 43552, 43552, 8736, 64480, 0]],
        '鴛' : [12, 13, 0, -2, 12, [0, 31712, 18976, 43744, 4608, 16288, 57568, 16256, 8320, 16352, 16416, 38240, 0]],
        '塩' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 18432, 63424, 17472, 18368, 16384, 20448, 19104, 60064, 8160, 0]],
        '於' : [12, 13, 0, -2, 12, [8192, 8640, 8736, 64512, 8192, 8576, 15456, 17408, 17408, 33792, 35200, 12384, 0]],
        '汚' : [12, 13, 0, -2, 12, [0, 8160, 33280, 16896, 8160, 33792, 17408, 2016, 2080, 10272, 16448, 33152, 0]],
        '甥' : [12, 13, 0, -2, 12, [0, 12256, 43296, 65504, 43296, 12256, 61696, 12256, 8736, 8736, 62496, 1216, 0]],
        '凹' : [12, 13, 0, -2, 12, [0, 61920, 37152, 37152, 37152, 37152, 40736, 32800, 32800, 32800, 32800, 65504, 0]],
        '央' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 17472, 17472, 17472, 17472, 65504, 1024, 2560, 12672, 49248, 0]],
        '奥' : [12, 13, 0, -2, 12, [1024, 2048, 32704, 21824, 17472, 32704, 17472, 21824, 65504, 2560, 12672, 49248, 0]],
        '往' : [12, 13, 0, -2, 12, [512, 8448, 16384, 36832, 256, 8448, 16640, 53216, 16640, 16640, 16640, 20448, 0]],
        '応' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 16896, 16640, 16384, 17472, 17440, 21504, 42016, 33760, 0]],
        '押' : [12, 13, 0, -2, 12, [0, 20448, 18720, 59680, 20448, 18720, 18720, 26912, 53216, 16640, 16640, 49408, 0]],
        '旺' : [12, 13, 0, -2, 12, [0, 4064, 61696, 37120, 37120, 37120, 63456, 37120, 37120, 37120, 61696, 4064, 0]],
        '横' : [12, 13, 0, -2, 12, [1152, 9344, 12256, 58496, 16352, 10528, 28640, 26912, 45024, 40960, 9280, 10272, 0]],
        '欧' : [12, 13, 0, -2, 12, [512, 64000, 33760, 37536, 53920, 41632, 41088, 53376, 35136, 33088, 61984, 3104, 0]],
        '殴' : [12, 13, 0, -2, 12, [0, 64448, 33344, 37440, 53824, 42080, 40960, 55264, 37408, 33344, 61824, 3680, 0]],
        '王' : [12, 13, 0, -2, 12, [0, 65504, 1024, 1024, 1024, 1024, 32704, 1024, 1024, 1024, 1024, 65504, 0]],
        '翁' : [12, 13, 0, -2, 12, [0, 3840, 8320, 51296, 4224, 32640, 0, 64480, 18720, 10400, 18720, 39520, 0]],
        '襖' : [12, 13, 0, -2, 12, [256, 16896, 20448, 60768, 11168, 11616, 19360, 56672, 24832, 24544, 17472, 22560, 0]],
        '鴬' : [12, 13, 0, -2, 12, [1024, 9344, 65504, 34848, 49056, 8320, 16256, 8320, 16352, 32, 21792, 38240, 0]],
        '鴎' : [12, 13, 0, -2, 12, [256, 61952, 34752, 37952, 55232, 42048, 42976, 54272, 38880, 32800, 62752, 5472, 0]],
        '黄' : [12, 13, 0, -2, 12, [4352, 4352, 32704, 4352, 65504, 1024, 16256, 9344, 16256, 9344, 16256, 49248, 0]],
        '岡' : [12, 13, 0, -2, 12, [0, 65504, 32800, 37152, 37152, 37408, 49056, 33824, 42144, 42144, 49056, 32864, 0]],
        '沖' : [12, 13, 0, -2, 12, [256, 256, 33024, 20448, 2336, 35104, 18720, 4064, 256, 8448, 16640, 33024, 0]],
        '荻' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4096, 20736, 8448, 25888, 38208, 12544, 20864, 37440, 27680, 0]],
        '億' : [12, 13, 0, -2, 12, [256, 8448, 16352, 17472, 24544, 51232, 20448, 18464, 20448, 17664, 21568, 26528, 0]],
        '屋' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 16384, 32736, 18560, 24512, 16928, 40896, 33280, 16352, 0]],
        '憶' : [12, 13, 0, -2, 12, [256, 8448, 12256, 9280, 49120, 43040, 45024, 43040, 45024, 9472, 13376, 10144, 0]],
        '臆' : [12, 13, 0, -2, 12, [256, 57600, 45024, 42112, 61408, 43040, 45024, 59424, 45024, 42240, 46144, 42912, 0]],
        '桶' : [12, 13, 0, -2, 12, [0, 12224, 8320, 61696, 12256, 10528, 28640, 26912, 45024, 43296, 10528, 10592, 0]],
        '牡' : [12, 13, 0, -2, 12, [128, 41088, 41088, 63616, 42976, 41088, 8320, 14464, 57472, 8320, 8320, 10208, 0]],
        '乙' : [12, 13, 0, -2, 12, [0, 32640, 128, 256, 512, 1024, 2048, 4096, 8256, 16448, 16448, 16256, 0]],
        '俺' : [12, 13, 0, -2, 12, [256, 8448, 16352, 17472, 18720, 57312, 26912, 20448, 18720, 20448, 16640, 16864, 0]],
        '卸' : [12, 13, 0, -2, 12, [16384, 16384, 32224, 37152, 37152, 64800, 4384, 7456, 37152, 37152, 40288, 57600, 0]],
        '恩' : [12, 13, 0, -2, 12, [0, 32704, 17472, 24384, 17472, 19008, 20800, 32704, 1024, 4672, 20512, 36736, 0]],
        '温' : [12, 13, 0, -2, 12, [0, 4032, 34880, 20416, 2112, 36800, 16384, 4064, 2720, 10912, 19104, 40928, 0]],
        '穏' : [12, 13, 0, -2, 12, [0, 8160, 58656, 9536, 12256, 61472, 12256, 24608, 28640, 42240, 46144, 10144, 0]],
        '音' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 4352, 4352, 65504, 0, 16256, 8320, 16256, 8320, 16256, 0]],
        '下' : [12, 13, 0, -2, 12, [0, 65504, 1024, 1024, 1024, 1792, 1216, 1024, 1024, 1024, 1024, 1024, 0]],
        '化' : [12, 13, 0, -2, 12, [512, 4608, 4608, 8736, 8768, 25216, 41728, 8704, 8704, 8736, 8736, 9184, 0]],
        '仮' : [12, 13, 0, -2, 12, [0, 12256, 10240, 18432, 20416, 51264, 19008, 18752, 20608, 20608, 17216, 19488, 0]],
        '何' : [12, 13, 0, -2, 12, [0, 12256, 8256, 16448, 20032, 51776, 19008, 19008, 20032, 16448, 16448, 16832, 0]],
        '伽' : [12, 13, 0, -2, 12, [2048, 10240, 10464, 24224, 19104, 51872, 19104, 19104, 21152, 21152, 17056, 19680, 0]],
        '価' : [12, 13, 0, -2, 12, [0, 12256, 8832, 17024, 20448, 51872, 19104, 19104, 19104, 19104, 19104, 20448, 0]],
        '佳' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16640, 16640, 57312, 16640, 16640, 20448, 16640, 16640, 24544, 0]],
        '加' : [12, 13, 0, -2, 12, [8192, 8192, 8672, 64800, 9504, 9504, 9504, 17696, 17696, 34080, 35104, 12768, 0]],
        '可' : [12, 13, 0, -2, 12, [0, 65504, 128, 128, 30848, 18560, 18560, 18560, 30848, 128, 128, 896, 0]],
        '嘉' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 8320, 16256, 4352, 65504, 10816, 19008, 39872, 0]],
        '夏' : [12, 13, 0, -2, 12, [0, 65504, 1024, 16320, 8256, 16320, 8256, 16320, 4224, 26752, 3840, 61664, 0]],
        '嫁' : [12, 13, 0, -2, 12, [256, 16640, 24544, 61472, 22464, 20768, 21312, 40064, 57792, 9888, 22688, 17152, 0]],
        '家' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 49056, 9216, 3136, 29312, 1792, 6784, 57952, 3072, 0]],
        '寡' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 40736, 4352, 65504, 8320, 32704, 34976, 4224, 25344, 0]],
        '科' : [12, 13, 0, -2, 12, [0, 6208, 57920, 8512, 8256, 62528, 8768, 24672, 29120, 44608, 41024, 8256, 0]],
        '暇' : [12, 13, 0, -2, 12, [0, 7904, 61984, 37408, 40672, 36864, 61920, 40480, 37024, 40608, 61632, 4896, 0]],
        '果' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 17472, 32704, 1024, 65504, 3328, 13440, 50272, 0]],
        '架' : [12, 13, 0, -2, 12, [8192, 8672, 64800, 9504, 9504, 17696, 39392, 1024, 65504, 3328, 13440, 50272, 0]],
        '歌' : [12, 13, 0, -2, 12, [512, 65024, 44000, 43680, 60064, 2720, 64640, 2176, 59712, 43328, 59936, 7200, 0]],
        '河' : [12, 13, 0, -2, 12, [0, 16352, 32832, 16448, 3648, 35392, 19008, 2624, 3648, 8256, 16448, 33216, 0]],
        '火' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 9280, 9280, 17536, 17536, 2560, 2560, 4352, 8320, 49248, 0]],
        '珂' : [12, 13, 0, -2, 12, [0, 8160, 57408, 16448, 20032, 19008, 59968, 19008, 20032, 16448, 57408, 448, 0]],
        '禍' : [12, 13, 0, -2, 12, [0, 18368, 17472, 62912, 5440, 12256, 18464, 52128, 27296, 19104, 19360, 18528, 0]],
        '禾' : [12, 13, 0, -2, 12, [0, 960, 31744, 1024, 1024, 65504, 1536, 3328, 5248, 9280, 50208, 1024, 0]],
        '稼' : [12, 13, 0, -2, 12, [256, 6400, 61408, 10272, 10176, 61728, 9024, 27776, 25024, 42656, 47264, 8960, 0]],
        '箇' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 65504, 33824, 49056, 33824, 40736, 37152, 65504, 0]],
        '花' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 4096, 4608, 8800, 25472, 41472, 8704, 8736, 9184, 0]],
        '苛' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 65504, 64, 15424, 9280, 9280, 15424, 448, 0]],
        '茄' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 8192, 8672, 64800, 9504, 9504, 17696, 35104, 12768, 0]],
        '荷' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 8192, 32736, 49216, 20032, 19008, 19008, 20032, 16576, 0]],
        '華' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 9344, 65504, 9344, 32704, 1024, 65504, 1024, 0]],
        '菓' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 9344, 16256, 9344, 65504, 3328, 13440, 50272, 0]],
        '蝦' : [12, 13, 0, -2, 12, [0, 12256, 10528, 63776, 44896, 43008, 43488, 65056, 10400, 11936, 10432, 64288, 0]],
        '課' : [12, 13, 0, -2, 12, [0, 61408, 2336, 61408, 2336, 59680, 4064, 256, 61408, 41728, 42304, 63776, 0]],
        '嘩' : [12, 13, 0, -2, 12, [576, 576, 61408, 41536, 45024, 42144, 49120, 42144, 45024, 41088, 61408, 128, 0]],
        '貨' : [12, 13, 0, -2, 12, [512, 4704, 9088, 57888, 16352, 8320, 16256, 8320, 16256, 8320, 16256, 49248, 0]],
        '迦' : [12, 13, 0, -2, 12, [2048, 2048, 35040, 24224, 2720, 2720, 60064, 10912, 12960, 9952, 20480, 36832, 0]],
        '過' : [12, 13, 0, -2, 12, [0, 1984, 17472, 9664, 1344, 4064, 59424, 11168, 10912, 11168, 20480, 36832, 0]],
        '霞' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 5248, 64480, 34848, 65504, 33344, 63872, 34400, 0]],
        '蚊' : [12, 13, 0, -2, 12, [128, 8320, 8320, 65504, 42048, 42048, 42304, 64832, 8512, 10368, 10624, 65120, 0]],
        '俄' : [12, 13, 0, -2, 12, [128, 9888, 14496, 18560, 19424, 64640, 18592, 20128, 30784, 18496, 18848, 24096, 0]],
        '峨' : [12, 13, 0, -2, 12, [128, 11936, 9376, 9344, 42976, 48256, 42144, 42656, 48192, 42048, 62880, 3616, 0]],
        '我' : [12, 13, 0, -2, 12, [256, 6464, 57632, 8448, 12256, 61568, 8352, 11424, 61504, 8384, 8480, 26144, 0]],
        '牙' : [12, 13, 0, -2, 12, [0, 32704, 8448, 8448, 8448, 65504, 768, 1280, 2304, 12544, 49408, 1792, 0]],
        '画' : [12, 13, 0, -2, 12, [0, 65504, 1024, 16256, 9344, 42144, 49056, 42144, 42144, 49056, 32800, 65504, 0]],
        '臥' : [12, 13, 0, -2, 12, [128, 64640, 36992, 36992, 63616, 34944, 35136, 63808, 37440, 37408, 64544, 1056, 0]],
        '芽' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 4352, 4352, 65504, 1280, 6400, 57600, 1792, 0]],
        '蛾' : [12, 13, 0, -2, 12, [128, 11936, 9376, 62592, 45024, 42112, 42144, 65184, 9280, 9280, 9632, 65056, 0]],
        '賀' : [12, 13, 0, -2, 12, [8192, 9184, 64032, 10784, 19424, 34944, 16256, 8320, 16256, 8320, 16256, 49248, 0]],
        '雅' : [12, 13, 0, -2, 12, [512, 64064, 21056, 22496, 21632, 24544, 62592, 5248, 14304, 21632, 38016, 14304, 0]],
        '餓' : [12, 13, 0, -2, 12, [128, 28320, 38048, 9344, 65504, 38016, 62624, 38560, 64576, 42048, 38304, 60960, 0]],
        '駕' : [12, 13, 0, -2, 12, [8192, 8672, 64800, 9504, 32736, 41984, 16320, 9216, 16352, 32, 21792, 38240, 0]],
        '介' : [12, 13, 0, -2, 12, [0, 3584, 4352, 8320, 20800, 37152, 4352, 4352, 8448, 8448, 16640, 16640, 0]],
        '会' : [12, 13, 0, -2, 12, [0, 3584, 4352, 8320, 24384, 32800, 0, 65504, 4224, 4224, 9024, 15424, 0]],
        '解' : [12, 13, 0, -2, 12, [0, 30688, 37152, 8736, 64704, 43648, 64480, 43648, 44160, 64480, 34944, 39040, 0]],
        '回' : [12, 13, 0, -2, 12, [0, 65504, 32800, 32800, 40736, 37152, 37152, 37152, 40736, 32800, 32800, 65504, 0]],
        '塊' : [12, 13, 0, -2, 12, [256, 16896, 20448, 18720, 61408, 18720, 18720, 20448, 17728, 17728, 59680, 2528, 0]],
        '壊' : [12, 13, 0, -2, 12, [256, 16640, 24544, 19104, 60064, 20448, 16896, 24544, 17696, 22848, 59520, 3680, 0]],
        '廻' : [12, 13, 0, -2, 12, [0, 4064, 59424, 11168, 19104, 35488, 60320, 10272, 45024, 16384, 45056, 36832, 0]],
        '快' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8480, 41248, 41248, 49120, 41216, 41216, 8832, 9280, 14368, 0]],
        '怪' : [12, 13, 0, -2, 12, [0, 12256, 8224, 8768, 41344, 44640, 41216, 41216, 42976, 8448, 8448, 12256, 0]],
        '悔' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 10240, 44992, 42304, 42304, 49120, 43584, 10816, 12256, 8320, 0]],
        '恢' : [12, 13, 0, -2, 12, [0, 12256, 10368, 10368, 43680, 43680, 43680, 43136, 43328, 8512, 8736, 11296, 0]],
        '懐' : [12, 13, 0, -2, 12, [256, 8448, 16352, 10912, 43680, 45024, 41472, 49120, 42272, 14656, 10368, 11872, 0]],
        '戒' : [12, 13, 0, -2, 12, [256, 288, 65504, 256, 10528, 10528, 64832, 10560, 10368, 16768, 17984, 6176, 0]],
        '拐' : [12, 13, 0, -2, 12, [0, 18368, 17472, 62528, 18368, 16384, 16384, 28640, 49696, 16928, 17440, 55488, 0]],
        '改' : [12, 13, 0, -2, 12, [512, 512, 62432, 4672, 5184, 62528, 33088, 33088, 36992, 36992, 62272, 3104, 0]],
        '魁' : [12, 13, 0, -2, 12, [4096, 8480, 64672, 42016, 64800, 42144, 42016, 64992, 21536, 22272, 36896, 40928, 0]],
        '晦' : [12, 13, 0, -2, 12, [1024, 1024, 59360, 43008, 44992, 42304, 58688, 49120, 43584, 43584, 61408, 128, 0]],
        '械' : [12, 13, 0, -2, 12, [128, 8352, 12256, 61568, 10880, 10912, 32672, 27296, 43584, 43584, 8352, 8992, 0]],
        '海' : [12, 13, 0, -2, 12, [2048, 2048, 36832, 20480, 8128, 35136, 18752, 8160, 21056, 21056, 40928, 32896, 0]],
        '灰' : [12, 13, 0, -2, 12, [0, 32736, 16384, 16640, 18720, 18720, 18720, 16640, 17024, 33408, 33856, 6176, 0]],
        '界' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 17472, 32704, 4352, 12672, 53568, 4384, 24832, 0]],
        '皆' : [12, 13, 0, -2, 12, [512, 16896, 31712, 16896, 16928, 64480, 1024, 16320, 8256, 16320, 8256, 16320, 0]],
        '絵' : [12, 13, 0, -2, 12, [0, 9152, 17440, 36864, 63456, 8192, 16384, 63456, 8448, 45600, 42016, 10208, 0]],
        '芥' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 3584, 12672, 53600, 4352, 4352, 8448, 8448, 0]],
        '蟹' : [12, 13, 0, -2, 12, [0, 31712, 37152, 64480, 43648, 65504, 34944, 16256, 9344, 16256, 1056, 65504, 0]],
        '開' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 32800, 49056, 35360, 49056, 35360, 37472, 0]],
        '階' : [12, 13, 0, -2, 12, [128, 59520, 44768, 43136, 51328, 44768, 41216, 45024, 43040, 53216, 34848, 36832, 0]],
        '貝' : [12, 13, 0, -2, 12, [0, 16320, 8256, 16320, 8256, 16320, 8256, 8256, 16320, 0, 4224, 24672, 0]],
        '凱' : [12, 13, 0, -2, 12, [8192, 43968, 64064, 576, 64064, 576, 64064, 35392, 64064, 21056, 21568, 63584, 0]],
        '劾' : [12, 13, 0, -2, 12, [8448, 8448, 63744, 18400, 35104, 20768, 8736, 18976, 37920, 9248, 20544, 35200, 0]],
        '外' : [12, 13, 0, -2, 12, [256, 16640, 30976, 18688, 19712, 43776, 39296, 2368, 4384, 4352, 8448, 49408, 0]],
        '咳' : [12, 13, 0, -2, 12, [256, 256, 65504, 37120, 37408, 37952, 37504, 37152, 38464, 36992, 62272, 3104, 0]],
        '害' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33824, 49056, 1024, 16256, 1024, 65504, 8320, 8320, 16256, 0]],
        '崖' : [12, 13, 0, -2, 12, [1024, 17440, 32736, 0, 32736, 16896, 24512, 16896, 24512, 33280, 33280, 16352, 0]],
        '慨' : [12, 13, 0, -2, 12, [0, 12256, 10560, 12096, 43328, 47584, 44864, 43072, 43840, 11456, 8512, 9824, 0]],
        '概' : [12, 13, 0, -2, 12, [0, 12256, 10560, 65344, 10560, 10720, 28480, 26688, 43840, 44224, 8512, 9824, 0]],
        '涯' : [12, 13, 0, -2, 12, [0, 4064, 34944, 19424, 2176, 34944, 20448, 2176, 19424, 20608, 36992, 34784, 0]],
        '碍' : [12, 13, 0, -2, 12, [0, 2016, 62496, 10208, 9248, 10208, 28736, 57312, 20544, 21568, 21056, 28864, 0]],
        '蓋' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 65504, 4224, 32704, 32, 32704, 19008, 65504, 0]],
        '街' : [12, 13, 0, -2, 12, [1024, 9440, 24320, 33792, 1248, 16160, 17440, 50208, 24352, 17440, 17440, 24416, 0]],
        '該' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57600, 544, 58432, 640, 288, 58944, 41088, 41792, 60448, 0]],
        '鎧' : [12, 13, 0, -2, 12, [256, 13600, 18400, 32768, 63456, 8192, 65504, 9248, 47072, 41536, 8768, 65504, 0]],
        '骸' : [12, 13, 0, -2, 12, [256, 30976, 20448, 22784, 18976, 64832, 33920, 31008, 20032, 30848, 18752, 24096, 0]],
        '浬' : [12, 13, 0, -2, 12, [0, 4064, 35104, 20448, 2336, 35104, 20448, 256, 4064, 8448, 16640, 36832, 0]],
        '馨' : [12, 13, 0, -2, 12, [4096, 65472, 4672, 64608, 21888, 32352, 33792, 65504, 9344, 32704, 41120, 16256, 0]],
        '蛙' : [12, 13, 0, -2, 12, [128, 8320, 9184, 63616, 43136, 45024, 43136, 63616, 9184, 10368, 10368, 65504, 0]],
        '垣' : [12, 13, 0, -2, 12, [0, 24544, 16384, 20416, 59456, 18496, 20416, 18496, 18496, 20416, 57344, 8160, 0]],
        '柿' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61696, 8448, 12256, 26912, 26912, 43296, 43296, 10592, 8448, 0]],
        '蛎' : [12, 13, 0, -2, 12, [0, 10208, 9216, 64512, 42976, 42240, 42240, 64992, 8736, 8736, 9248, 64704, 0]],
        '鈎' : [12, 13, 0, -2, 12, [1024, 13312, 18400, 34848, 64032, 8736, 64032, 9376, 46240, 42912, 8224, 63680, 0]],
        '劃' : [12, 13, 0, -2, 12, [0, 64544, 9504, 64800, 9504, 64800, 8480, 65312, 42272, 64544, 42016, 65120, 0]],
        '嚇' : [12, 13, 0, -2, 12, [1152, 1152, 61408, 42112, 42112, 49120, 42304, 44352, 46432, 42304, 59712, 2880, 0]],
        '各' : [12, 13, 0, -2, 12, [4096, 4096, 8064, 8320, 18688, 1536, 6528, 57440, 16256, 8320, 8320, 16256, 0]],
        '廓' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 36864, 64992, 42272, 48448, 35136, 37152, 65312, 35264, 47360, 0]],
        '拡' : [12, 13, 0, -2, 12, [256, 16640, 20448, 63488, 18688, 18688, 18688, 27200, 51776, 21056, 21664, 50976, 0]],
        '撹' : [12, 13, 0, -2, 12, [512, 18752, 17536, 65504, 18464, 20416, 18496, 26688, 53184, 17024, 17568, 55520, 0]],
        '格' : [12, 13, 0, -2, 12, [512, 8704, 9184, 62496, 10816, 8576, 25152, 29728, 45024, 42016, 9248, 10208, 0]],
        '核' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61696, 8736, 9280, 25216, 24864, 46656, 41088, 9024, 11296, 0]],
        '殻' : [12, 13, 0, -2, 12, [8192, 9152, 64064, 8768, 64064, 1120, 63488, 36832, 29216, 21056, 37248, 40544, 0]],
        '獲' : [12, 13, 0, -2, 12, [576, 8768, 45024, 16960, 18400, 44160, 10208, 25728, 45024, 9280, 9088, 52320, 0]],
        '確' : [12, 13, 0, -2, 12, [256, 256, 65504, 10784, 8832, 10208, 31872, 54400, 22496, 21632, 21632, 30688, 0]],
        '穫' : [12, 13, 0, -2, 12, [576, 6720, 61408, 8768, 10208, 64640, 10208, 25728, 28640, 42048, 41856, 11360, 0]],
        '覚' : [12, 13, 0, -2, 12, [1152, 9472, 65504, 41120, 49056, 8320, 16256, 8320, 16288, 4640, 8736, 50144, 0]],
        '角' : [12, 13, 0, -2, 12, [0, 8064, 8448, 32704, 50240, 32704, 17472, 17472, 32704, 16448, 32832, 33216, 0]],
        '赫' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 4352, 65504, 10880, 43680, 43680, 43680, 18560, 22912, 0]],
        '較' : [12, 13, 0, -2, 12, [256, 8448, 65504, 8192, 64064, 44064, 63488, 43584, 63808, 8320, 63808, 9760, 0]],
        '郭' : [12, 13, 0, -2, 12, [4096, 4576, 65312, 288, 32064, 17792, 32064, 2336, 4384, 65312, 4544, 28928, 0]],
        '閣' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 37152, 44576, 37152, 49056, 37152, 40800, 0]],
        '隔' : [12, 13, 0, -2, 12, [0, 61408, 40960, 42944, 50240, 42944, 40960, 45024, 43680, 52320, 35744, 35104, 0]],
        '革' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 7936, 1024, 32704, 17472, 32704, 1024, 65504, 1024, 0]],
        '学' : [12, 13, 0, -2, 12, [2112, 17536, 8448, 65504, 32800, 48928, 512, 1024, 65504, 1024, 1024, 7168, 0]],
        '岳' : [12, 13, 0, -2, 12, [0, 896, 15360, 8192, 16320, 8448, 8448, 65504, 1024, 17472, 17472, 32704, 0]],
        '楽' : [12, 13, 0, -2, 12, [1024, 2048, 40736, 20800, 7936, 20800, 40736, 1024, 65504, 3328, 13440, 50272, 0]],
        '額' : [12, 13, 0, -2, 12, [4096, 5088, 64640, 44000, 43552, 21472, 10784, 18400, 64032, 19424, 19008, 31776, 0]],
        '顎' : [12, 13, 0, -2, 12, [0, 64480, 43136, 64480, 544, 64480, 544, 64480, 16928, 64480, 35136, 13856, 0]],
        '掛' : [12, 13, 0, -2, 12, [1024, 17472, 20288, 58432, 17472, 24384, 17504, 25680, 53056, 17472, 17472, 57152, 0]],
        '笠' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 9344, 1024, 32704, 4352, 4352, 4608, 512, 65504, 0]],
        '樫' : [12, 13, 0, -2, 12, [0, 12256, 10784, 65440, 10560, 12096, 27296, 28576, 41216, 42976, 8448, 12256, 0]],
        '橿' : [12, 13, 0, -2, 12, [0, 12256, 9504, 63456, 9504, 10208, 24576, 28640, 42272, 42976, 9504, 12256, 0]],
        '梶' : [12, 13, 0, -2, 12, [0, 10208, 9248, 63456, 9216, 10208, 25728, 30688, 42112, 44000, 10368, 8416, 0]],
        '鰍' : [12, 13, 0, -2, 12, [128, 29568, 40064, 9376, 62624, 44960, 62592, 42176, 64832, 1344, 43552, 43552, 0]],
        '潟' : [12, 13, 0, -2, 12, [512, 3264, 34880, 19904, 2112, 36800, 17408, 4064, 20512, 19104, 35360, 32960, 0]],
        '割' : [12, 13, 0, -2, 12, [4096, 4128, 65312, 37152, 64800, 4384, 32032, 4384, 64800, 17440, 17440, 31840, 0]],
        '喝' : [12, 13, 0, -2, 12, [0, 4032, 59456, 44992, 43072, 44992, 41984, 45024, 47136, 43296, 61216, 192, 0]],
        '恰' : [12, 13, 0, -2, 12, [0, 9152, 9248, 10240, 40960, 42976, 40960, 40960, 42976, 9248, 9248, 10208, 0]],
        '括' : [12, 13, 0, -2, 12, [0, 8288, 10112, 61568, 8320, 10208, 8320, 12416, 59360, 9248, 9248, 26592, 0]],
        '活' : [12, 13, 0, -2, 12, [0, 224, 36608, 16640, 256, 36832, 16640, 256, 4064, 10272, 18464, 36832, 0]],
        '渇' : [12, 13, 0, -2, 12, [0, 4032, 34880, 20416, 2112, 36800, 17408, 4064, 22560, 18720, 36640, 32960, 0]],
        '滑' : [12, 13, 0, -2, 12, [0, 4032, 34880, 19392, 2624, 40928, 20512, 4032, 2112, 12224, 18496, 35008, 0]],
        '葛' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8320, 16256, 8320, 16256, 4096, 16352, 57376, 8736, 16064, 0]],
        '褐' : [12, 13, 0, -2, 12, [0, 20416, 18496, 61376, 10304, 12224, 17408, 61408, 22560, 18720, 20256, 16576, 0]],
        '轄' : [12, 13, 0, -2, 12, [128, 8320, 65504, 9376, 64480, 43136, 64480, 43136, 65504, 8736, 64032, 9184, 0]],
        '且' : [12, 13, 0, -2, 12, [0, 16256, 8320, 8320, 16256, 8320, 8320, 16256, 8320, 8320, 8320, 65504, 0]],
        '鰹' : [12, 13, 0, -2, 12, [0, 28640, 43552, 20384, 63808, 44864, 64160, 44960, 63616, 992, 43136, 45024, 0]],
        '叶' : [12, 13, 0, -2, 12, [128, 128, 61568, 36992, 40928, 36992, 36992, 36992, 36992, 36992, 61568, 128, 0]],
        '椛' : [12, 13, 0, -2, 12, [1152, 9344, 16352, 58496, 9472, 9504, 26944, 27008, 47360, 43264, 10528, 10720, 0]],
        '樺' : [12, 13, 0, -2, 12, [576, 8768, 12256, 62016, 12256, 9376, 32736, 25760, 45024, 41088, 12256, 8320, 0]],
        '鞄' : [12, 13, 0, -2, 12, [512, 20992, 64480, 21536, 30624, 9376, 63648, 43936, 64096, 8704, 64032, 9184, 0]],
        '株' : [12, 13, 0, -2, 12, [256, 10496, 10496, 61408, 12544, 8448, 28640, 28928, 41856, 42304, 14624, 8448, 0]],
        '兜' : [12, 13, 0, -2, 12, [1024, 2048, 57184, 37152, 40736, 37152, 37152, 57184, 512, 4608, 4640, 58336, 0]],
        '竃' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 32704, 9344, 32704, 17472, 32704, 17440, 2016, 0]],
        '蒲' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4384, 20448, 8448, 36832, 18720, 4064, 10528, 20448, 35104, 0]],
        '釜' : [12, 13, 0, -2, 12, [0, 12672, 51808, 3584, 12672, 49248, 16256, 1024, 32704, 9344, 5376, 65504, 0]],
        '鎌' : [12, 13, 0, -2, 12, [576, 12928, 20448, 33408, 63456, 8864, 65504, 8864, 47072, 41600, 9920, 64160, 0]],
        '噛' : [12, 13, 0, -2, 12, [512, 3008, 59904, 49120, 41536, 43680, 41504, 49120, 43680, 45664, 57888, 16352, 0]],
        '鴨' : [12, 13, 0, -2, 12, [128, 256, 64448, 43584, 43968, 64064, 44000, 43520, 64480, 8224, 10912, 10976, 0]],
        '栢' : [12, 13, 0, -2, 12, [0, 12256, 8448, 61952, 10208, 9248, 25632, 26592, 46112, 42016, 9248, 10208, 0]],
        '茅' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 256, 65504, 1568, 2624, 12800, 49664, 3584, 0]],
        '萱' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 65504, 32800, 49056, 4352, 7936, 4352, 65504, 0]],
        '粥' : [12, 13, 0, -2, 12, [0, 58592, 13600, 11808, 58592, 40832, 33920, 58592, 11808, 13600, 9248, 50368, 0]],
        '刈' : [12, 13, 0, -2, 12, [0, 2080, 2336, 35104, 18720, 12576, 4384, 4384, 10528, 9248, 16416, 16480, 0]],
        '苅' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 4128, 20768, 12576, 4384, 10528, 9504, 16416, 16480, 0]],
        '瓦' : [12, 13, 0, -2, 12, [0, 65504, 4096, 4096, 7936, 4352, 4352, 10496, 9472, 8480, 16160, 49632, 0]],
        '乾' : [12, 13, 0, -2, 12, [512, 8704, 64480, 8704, 64512, 35808, 63552, 34944, 63744, 8736, 64544, 9184, 0]],
        '侃' : [12, 13, 0, -2, 12, [0, 12256, 10272, 18464, 18464, 53216, 16384, 17728, 17728, 17728, 18752, 20832, 0]],
        '冠' : [12, 13, 0, -2, 12, [0, 65504, 32800, 64, 61504, 2016, 64, 64064, 20800, 20544, 36896, 36832, 0]],
        '寒' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 49056, 4352, 65504, 4352, 11392, 50016, 6144, 1792, 0]],
        '刊' : [12, 13, 0, -2, 12, [0, 64544, 8480, 8480, 8480, 8480, 64800, 8480, 8480, 8224, 8224, 8288, 0]],
        '勘' : [12, 13, 0, -2, 12, [256, 18688, 63744, 20448, 31008, 18720, 18720, 64800, 43552, 52768, 32832, 64896, 0]],
        '勧' : [12, 13, 0, -2, 12, [16640, 16640, 32000, 41952, 64800, 20768, 32032, 53536, 32288, 21024, 20544, 32128, 0]],
        '巻' : [12, 13, 0, -2, 12, [2112, 17536, 8448, 32704, 2304, 65504, 4224, 16320, 49312, 16256, 8256, 16320, 0]],
        '喚' : [12, 13, 0, -2, 12, [0, 1984, 59520, 49120, 43680, 43680, 44256, 43296, 49120, 41600, 58432, 6176, 0]],
        '堪' : [12, 13, 0, -2, 12, [1088, 17472, 24544, 17472, 59328, 17472, 17472, 20448, 19072, 19680, 63488, 4064, 0]],
        '姦' : [12, 13, 0, -2, 12, [2048, 2048, 32704, 4352, 12544, 7936, 8320, 65504, 19008, 51776, 14784, 50720, 0]],
        '完' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 49056, 0, 0, 32704, 4608, 4608, 8736, 50144, 0]],
        '官' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 48928, 41248, 16128, 8192, 16256, 8320, 8320, 16256, 0]],
        '寛' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 32704, 8320, 16256, 8320, 16256, 2560, 12832, 50144, 0]],
        '干' : [12, 13, 0, -2, 12, [0, 32704, 1024, 1024, 1024, 1024, 65504, 1024, 1024, 1024, 1024, 1024, 0]],
        '幹' : [12, 13, 0, -2, 12, [0, 8640, 64032, 9216, 64480, 34944, 63616, 34944, 64480, 8320, 63616, 8320, 0]],
        '患' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 17472, 32704, 1024, 32704, 17472, 32704, 4608, 20544, 36768, 0]],
        '感' : [12, 13, 0, -2, 12, [256, 288, 32736, 16640, 23872, 21632, 40256, 33312, 1024, 4672, 20512, 36736, 0]],
        '慣' : [12, 13, 0, -2, 12, [0, 12256, 10528, 16352, 43296, 45024, 43040, 45024, 43040, 12256, 8768, 11296, 0]],
        '憾' : [12, 13, 0, -2, 12, [128, 8352, 12256, 10368, 44704, 43584, 44704, 43296, 8704, 2368, 10272, 18304, 0]],
        '換' : [12, 13, 0, -2, 12, [0, 18368, 18560, 65504, 19104, 19104, 19680, 26912, 57312, 17024, 17472, 55328, 0]],
        '敢' : [12, 13, 0, -2, 12, [256, 64768, 4576, 64800, 17952, 31776, 17568, 31904, 17472, 31808, 50336, 1824, 0]],
        '柑' : [12, 13, 0, -2, 12, [1024, 9248, 9248, 65504, 9248, 9248, 26592, 25632, 46112, 42016, 9248, 10208, 0]],
        '桓' : [12, 13, 0, -2, 12, [0, 12256, 8192, 63456, 9248, 9248, 26592, 25632, 46112, 42976, 8192, 12256, 0]],
        '棺' : [12, 13, 0, -2, 12, [256, 8448, 12256, 63520, 10176, 9280, 26560, 25600, 47072, 42016, 9248, 10208, 0]],
        '款' : [12, 13, 0, -2, 12, [512, 8704, 65504, 8864, 64160, 672, 63616, 128, 64832, 8512, 43552, 44064, 0]],
        '歓' : [12, 13, 0, -2, 12, [16896, 16896, 32736, 41632, 65184, 21152, 31872, 53376, 32064, 20800, 21024, 31776, 0]],
        '汗' : [12, 13, 0, -2, 12, [0, 4064, 33024, 16640, 256, 33024, 20448, 256, 256, 8448, 16640, 33024, 0]],
        '漢' : [12, 13, 0, -2, 12, [1152, 1152, 40928, 17536, 4064, 35104, 20448, 256, 24544, 17024, 33856, 38944, 0]],
        '澗' : [12, 13, 0, -2, 12, [0, 7904, 37536, 24288, 4768, 40672, 20512, 6048, 21664, 22432, 38048, 38816, 0]],
        '潅' : [12, 13, 0, -2, 12, [1024, 1024, 34784, 18688, 4064, 33344, 18400, 3200, 2016, 9344, 17536, 34784, 0]],
        '環' : [12, 13, 0, -2, 12, [0, 4064, 64160, 12256, 8192, 12256, 62528, 10176, 8544, 13184, 52544, 288, 0]],
        '甘' : [12, 13, 0, -2, 12, [8320, 8320, 8320, 65504, 8320, 8320, 16256, 8320, 8320, 8320, 8320, 16256, 0]],
        '監' : [12, 13, 0, -2, 12, [512, 64000, 41952, 64512, 34816, 64448, 40960, 40960, 65472, 19008, 19008, 65504, 0]],
        '看' : [12, 13, 0, -2, 12, [0, 32736, 1024, 16320, 1024, 65504, 2048, 8128, 12352, 57280, 4160, 8128, 0]],
        '竿' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 32704, 1024, 1024, 65504, 1024, 1024, 1024, 0]],
        '管' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 1024, 65504, 37152, 40736, 4096, 8064, 4224, 8064, 0]],
        '簡' : [12, 13, 0, -2, 12, [16896, 16896, 31712, 42112, 64480, 35360, 64480, 36384, 61920, 40736, 37152, 40800, 0]],
        '緩' : [12, 13, 0, -2, 12, [0, 12256, 17696, 34112, 61408, 8448, 20448, 61952, 10176, 43584, 41344, 11872, 0]],
        '缶' : [12, 13, 0, -2, 12, [8192, 8192, 16320, 17408, 17408, 1024, 65504, 1024, 1024, 17472, 17472, 32704, 0]],
        '翰' : [12, 13, 0, -2, 12, [0, 8640, 64032, 9216, 65504, 35104, 64928, 35104, 64352, 9632, 63776, 9056, 0]],
        '肝' : [12, 13, 0, -2, 12, [0, 63456, 36992, 36992, 61568, 36992, 38880, 61568, 36992, 36992, 36992, 45184, 0]],
        '艦' : [12, 13, 0, -2, 12, [8320, 20352, 60032, 45024, 59648, 44800, 64224, 41472, 61408, 43680, 43680, 49120, 0]],
        '莞' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 32800, 16256, 0, 32704, 2560, 12832, 50144, 0]],
        '観' : [12, 13, 0, -2, 12, [16384, 17376, 32288, 41952, 65056, 21472, 32288, 54240, 31040, 20800, 21056, 31840, 0]],
        '諌' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57600, 4064, 59680, 4064, 2336, 61408, 41728, 42304, 63776, 0]],
        '貫' : [12, 13, 0, -2, 12, [0, 32704, 17472, 65504, 17472, 32704, 8320, 16256, 8320, 16256, 8320, 49248, 0]],
        '還' : [12, 13, 0, -2, 12, [0, 4064, 19104, 12256, 0, 8160, 58432, 10176, 8992, 11584, 20768, 36832, 0]],
        '鑑' : [12, 13, 0, -2, 12, [128, 16256, 19072, 36832, 63744, 12032, 64224, 8704, 45024, 43680, 10912, 65504, 0]],
        '間' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 32800, 40736, 37152, 40736, 37152, 40800, 0]],
        '閑' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 33824, 49056, 33824, 36384, 46496, 33888, 0]],
        '関' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 35360, 49056, 33824, 49056, 35360, 37216, 0]],
        '陥' : [12, 13, 0, -2, 12, [0, 58304, 42048, 43136, 50144, 43552, 43552, 44000, 43552, 51744, 35360, 35808, 0]],
        '韓' : [12, 13, 0, -2, 12, [8320, 8320, 63456, 8480, 65504, 35360, 64480, 34944, 65504, 9344, 65504, 8320, 0]],
        '館' : [12, 13, 0, -2, 12, [256, 28928, 34784, 9248, 64480, 35392, 64448, 35328, 64480, 41504, 37408, 60384, 0]],
        '舘' : [12, 13, 0, -2, 12, [256, 12544, 18400, 33824, 64480, 8768, 64448, 8704, 64480, 35360, 35360, 64480, 0]],
        '丸' : [12, 13, 0, -2, 12, [4096, 4096, 4096, 65280, 4352, 4352, 20736, 12544, 4352, 10528, 17696, 32992, 0]],
        '含' : [12, 13, 0, -2, 12, [0, 7936, 8320, 24384, 32800, 16256, 256, 512, 16256, 8320, 8320, 16256, 0]],
        '岸' : [12, 13, 0, -2, 12, [512, 16928, 32736, 0, 32736, 16384, 24512, 16896, 16896, 49120, 33280, 512, 0]],
        '巌' : [12, 13, 0, -2, 12, [1024, 17440, 32736, 8768, 32736, 18688, 32736, 21568, 23872, 21824, 48512, 34400, 0]],
        '玩' : [12, 13, 0, -2, 12, [0, 4064, 61440, 8192, 8192, 65504, 9344, 9344, 13440, 50304, 2208, 12512, 0]],
        '癌' : [12, 13, 0, -2, 12, [512, 512, 32736, 18496, 53184, 16384, 24288, 53920, 24288, 16640, 37152, 40928, 0]],
        '眼' : [12, 13, 0, -2, 12, [0, 2016, 62496, 38880, 62496, 37920, 38880, 62720, 38176, 38048, 62528, 1824, 0]],
        '岩' : [12, 13, 0, -2, 12, [1024, 17472, 17472, 32704, 0, 65504, 2048, 4096, 16320, 57408, 8256, 16320, 0]],
        '翫' : [12, 13, 0, -2, 12, [0, 64992, 9216, 46080, 27648, 47072, 9536, 63808, 35136, 63808, 35392, 64096, 0]],
        '贋' : [12, 13, 0, -2, 12, [0, 32736, 19008, 21472, 30336, 21120, 24544, 20544, 40896, 36928, 8128, 24672, 0]],
        '雁' : [12, 13, 0, -2, 12, [0, 32736, 18752, 18752, 21472, 29248, 22496, 23104, 21472, 37440, 37440, 5088, 0]],
        '頑' : [12, 13, 0, -2, 12, [0, 4064, 61696, 2016, 1056, 63456, 21536, 22496, 21536, 40928, 33344, 3104, 0]],
        '顔' : [12, 13, 0, -2, 12, [8192, 10208, 63616, 19424, 18976, 65504, 37408, 58336, 39456, 58336, 6464, 58912, 0]],
        '願' : [12, 13, 0, -2, 12, [0, 64480, 36992, 64480, 35360, 64480, 35360, 64480, 37408, 23520, 37184, 13856, 0]],
        '企' : [12, 13, 0, -2, 12, [0, 3584, 4352, 8320, 17472, 33824, 1024, 10112, 9216, 9216, 9216, 65504, 0]],
        '伎' : [12, 13, 0, -2, 12, [256, 8448, 16352, 16640, 16640, 53184, 16448, 17472, 17472, 17024, 17280, 23648, 0]],
        '危' : [12, 13, 0, -2, 12, [0, 7936, 8448, 16896, 65504, 16384, 20352, 18560, 18560, 18816, 34848, 36832, 0]],
        '喜' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 8320, 16256, 4352, 65504, 8320, 8320, 16256, 0]],
        '器' : [12, 13, 0, -2, 12, [0, 31680, 19008, 19008, 31680, 1024, 65504, 4352, 64480, 19008, 19008, 31680, 0]],
        '基' : [12, 13, 0, -2, 12, [4352, 4352, 32704, 4352, 7936, 4352, 65504, 4352, 9344, 57184, 1024, 32704, 0]],
        '奇' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 2560, 4352, 8320, 65504, 128, 15488, 9344, 15488, 384, 0]],
        '嬉' : [12, 13, 0, -2, 12, [128, 16512, 20448, 61568, 22496, 21536, 22496, 37440, 59360, 13344, 17440, 18400, 0]],
        '寄' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33824, 49056, 2560, 4352, 65504, 128, 15488, 9344, 15744, 0]],
        '岐' : [12, 13, 0, -2, 12, [256, 8448, 16352, 8448, 41216, 44992, 41024, 42048, 42048, 62080, 896, 7264, 0]],
        '希' : [12, 13, 0, -2, 12, [0, 12736, 3584, 29056, 1088, 65504, 4608, 16320, 53824, 4672, 4800, 512, 0]],
        '幾' : [12, 13, 0, -2, 12, [1024, 9344, 21824, 9344, 17696, 32224, 1024, 65504, 8736, 8768, 20864, 20064, 0]],
        '忌' : [12, 13, 0, -2, 12, [0, 16256, 128, 128, 16256, 8192, 8256, 16320, 1024, 4672, 20512, 36736, 0]],
        '揮' : [12, 13, 0, -2, 12, [0, 24544, 20768, 65504, 18720, 20448, 18720, 26912, 53216, 16640, 24544, 49408, 0]],
        '机' : [12, 13, 0, -2, 12, [0, 10176, 9280, 62528, 9280, 9280, 25664, 29760, 42048, 42048, 10304, 12384, 0]],
        '旗' : [12, 13, 0, -2, 12, [1024, 17408, 20448, 62528, 18400, 17472, 30656, 21568, 40928, 36864, 4672, 25632, 0]],
        '既' : [12, 13, 0, -2, 12, [0, 63456, 37440, 62016, 37440, 40928, 61568, 32896, 37248, 35488, 42144, 51424, 0]],
        '期' : [12, 13, 0, -2, 12, [0, 18912, 64800, 18720, 31200, 18720, 31008, 18912, 64800, 288, 21024, 35424, 0]],
        '棋' : [12, 13, 0, -2, 12, [1088, 9280, 12256, 62528, 10176, 9280, 26560, 25664, 45024, 40960, 9280, 10272, 0]],
        '棄' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4224, 32704, 9344, 65504, 9344, 32704, 3328, 13440, 50272, 0]],
        '機' : [12, 13, 0, -2, 12, [256, 9504, 10560, 62880, 10560, 12256, 24832, 28640, 42112, 42656, 10304, 10656, 0]],
        '帰' : [12, 13, 0, -2, 12, [0, 12224, 41024, 44992, 41024, 49120, 41248, 45024, 10560, 10560, 18880, 16640, 0]],
        '毅' : [12, 13, 0, -2, 12, [8192, 9152, 64064, 21056, 21056, 64608, 43008, 22496, 45600, 23104, 37248, 26208, 0]],
        '気' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16384, 24448, 0, 32704, 64, 12864, 3136, 4640, 24864, 0]],
        '汽' : [12, 13, 0, -2, 12, [2048, 2048, 36800, 20480, 6016, 32768, 24512, 64, 64, 8256, 16416, 32800, 0]],
        '畿' : [12, 13, 0, -2, 12, [1024, 5248, 11584, 22144, 9504, 65504, 1024, 65504, 21792, 32064, 21632, 32096, 0]],
        '祈' : [12, 13, 0, -2, 12, [0, 8384, 9984, 62464, 5120, 6112, 9280, 25664, 46144, 10304, 10304, 8256, 0]],
        '季' : [12, 13, 0, -2, 12, [0, 16256, 1024, 65504, 5376, 9344, 57152, 544, 1024, 65504, 1024, 7168, 0]],
        '稀' : [12, 13, 0, -2, 12, [0, 6752, 57728, 9792, 8480, 63456, 8704, 26592, 27936, 46368, 42336, 8448, 0]],
        '紀' : [12, 13, 0, -2, 12, [0, 10208, 16416, 36896, 61472, 10208, 17408, 64512, 9216, 46112, 42016, 10208, 0]],
        '徽' : [12, 13, 0, -2, 12, [1280, 13568, 21984, 40736, 1056, 10912, 21664, 51872, 24384, 17472, 21664, 26400, 0]],
        '規' : [12, 13, 0, -2, 12, [0, 10208, 9248, 63456, 9248, 10208, 64544, 10208, 21120, 21120, 33952, 39136, 0]],
        '記' : [12, 13, 0, -2, 12, [0, 63456, 32, 61472, 32, 63456, 1024, 1024, 62464, 37920, 37920, 63456, 0]],
        '貴' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 17472, 32704, 1024, 65504, 8320, 16256, 8320, 16256, 49248, 0]],
        '起' : [12, 13, 0, -2, 12, [8192, 9184, 63520, 8224, 9184, 64000, 8704, 14880, 41952, 40960, 61440, 36832, 0]],
        '軌' : [12, 13, 0, -2, 12, [256, 8448, 63744, 10176, 63808, 43328, 63808, 43328, 64064, 8768, 64576, 9312, 0]],
        '輝' : [12, 13, 0, -2, 12, [0, 12256, 10528, 45024, 42272, 10208, 64800, 21792, 22496, 20736, 40928, 33024, 0]],
        '飢' : [12, 13, 0, -2, 12, [0, 29632, 35392, 8768, 64064, 35392, 64064, 35392, 64064, 37440, 35392, 62560, 0]],
        '騎' : [12, 13, 0, -2, 12, [256, 61696, 45024, 62080, 42048, 63520, 45024, 61472, 5920, 54560, 5920, 24672, 0]],
        '鬼' : [12, 13, 0, -2, 12, [1024, 2048, 32640, 18560, 32640, 18560, 18560, 32640, 5440, 5568, 9248, 50144, 0]],
        '亀' : [12, 13, 0, -2, 12, [0, 16128, 16896, 65408, 18560, 32704, 18496, 65472, 34880, 65472, 34848, 2016, 0]],
        '偽' : [12, 13, 0, -2, 12, [2176, 9344, 8448, 24448, 17536, 53184, 20544, 32736, 16416, 21792, 21792, 16576, 0]],
        '儀' : [12, 13, 0, -2, 12, [1088, 9344, 16352, 16640, 20448, 49408, 24544, 18688, 32736, 18720, 23744, 19232, 0]],
        '妓' : [12, 13, 0, -2, 12, [256, 16640, 24544, 61696, 20736, 24512, 20544, 37952, 58432, 12928, 17280, 23648, 0]],
        '宜' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 49056, 8320, 16256, 8320, 16256, 8320, 8320, 65504, 0]],
        '戯' : [12, 13, 0, -2, 12, [4224, 7840, 4256, 32384, 21472, 32384, 20640, 24224, 38208, 46656, 5280, 32544, 0]],
        '技' : [12, 13, 0, -2, 12, [256, 16640, 24544, 57600, 16640, 20416, 16448, 25664, 50240, 17024, 17280, 56416, 0]],
        '擬' : [12, 13, 0, -2, 12, [0, 21472, 23616, 61568, 20448, 20640, 24192, 26752, 57056, 19072, 22144, 58336, 0]],
        '欺' : [12, 13, 0, -2, 12, [512, 18944, 64480, 19104, 31392, 19104, 30848, 18560, 64832, 320, 21024, 35872, 0]],
        '犠' : [12, 13, 0, -2, 12, [1088, 42112, 45024, 61696, 42976, 41216, 12256, 13568, 61408, 9376, 11840, 9632, 0]],
        '疑' : [12, 13, 0, -2, 12, [0, 39904, 57408, 34944, 64480, 16544, 31360, 37504, 65248, 4736, 10880, 51168, 0]],
        '祇' : [12, 13, 0, -2, 12, [0, 16480, 18304, 62592, 5248, 5248, 10208, 17472, 58432, 21568, 17824, 20000, 0]],
        '義' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 32704, 1024, 65504, 9248, 65504, 8768, 63872, 9824, 0]],
        '蟻' : [12, 13, 0, -2, 12, [576, 8832, 12256, 63616, 44000, 43136, 45024, 62592, 12256, 9376, 11840, 62880, 0]],
        '誼' : [12, 13, 0, -2, 12, [256, 57600, 4064, 59424, 1984, 58432, 1984, 1088, 59328, 42048, 42048, 61408, 0]],
        '議' : [12, 13, 0, -2, 12, [1088, 58496, 8160, 57600, 4064, 57600, 8160, 2304, 65504, 43296, 48320, 60192, 0]],
        '掬' : [12, 13, 0, -2, 12, [2048, 18432, 20448, 61472, 21152, 19232, 16928, 24544, 58144, 19104, 21024, 49344, 0]],
        '菊' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16352, 18720, 43552, 2080, 32672, 6688, 10528, 18624, 0]],
        '鞠' : [12, 13, 0, -2, 12, [1024, 21504, 65504, 21536, 31392, 8864, 64032, 45024, 64288, 9888, 64032, 8384, 0]],
        '吉' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 1024, 32704, 0, 0, 16256, 8320, 8320, 16256, 0]],
        '吃' : [12, 13, 0, -2, 12, [1024, 1024, 63456, 38912, 36864, 38848, 36992, 37120, 37376, 37920, 62496, 992, 0]],
        '喫' : [12, 13, 0, -2, 12, [1024, 1024, 61408, 42144, 44704, 42272, 44640, 41216, 49120, 41216, 59008, 6240, 0]],
        '桔' : [12, 13, 0, -2, 12, [128, 8320, 12256, 61568, 8320, 10208, 24576, 24576, 47072, 42016, 9248, 10208, 0]],
        '橘' : [12, 13, 0, -2, 12, [0, 10176, 8320, 65504, 9504, 10560, 28640, 27296, 44256, 43936, 10912, 11168, 0]],
        '詰' : [12, 13, 0, -2, 12, [128, 61568, 2016, 61568, 128, 63456, 0, 0, 62432, 37408, 37408, 62432, 0]],
        '砧' : [12, 13, 0, -2, 12, [256, 63744, 8448, 8672, 8448, 8448, 28928, 55264, 21536, 21536, 21536, 30688, 0]],
        '杵' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 63616, 10368, 8320, 24704, 28640, 41088, 41088, 8320, 8320, 0]],
        '黍' : [12, 13, 0, -2, 12, [0, 960, 31744, 1024, 65504, 9344, 19008, 12672, 50272, 5376, 9344, 52320, 0]],
        '却' : [12, 13, 0, -2, 12, [8192, 9184, 64032, 8736, 8736, 8736, 65056, 16928, 16928, 35360, 64224, 2560, 0]],
        '客' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37024, 43296, 17920, 6528, 57440, 16256, 8320, 8320, 16256, 0]],
        '脚' : [12, 13, 0, -2, 12, [1024, 58592, 42144, 49056, 58528, 42144, 49056, 58528, 43168, 43744, 44672, 41088, 0]],
        '虐' : [12, 13, 0, -2, 12, [1024, 1984, 1024, 32736, 17440, 24448, 17440, 24544, 20480, 40928, 36864, 8160, 0]],
        '逆' : [12, 13, 0, -2, 12, [1088, 576, 32896, 20448, 256, 2336, 59680, 12256, 8448, 8448, 20480, 36832, 0]],
        '丘' : [12, 13, 0, -2, 12, [0, 384, 15872, 8192, 8192, 16320, 8448, 8448, 8448, 8448, 8448, 65504, 0]],
        '久' : [12, 13, 0, -2, 12, [4096, 4096, 8064, 4224, 8320, 8448, 256, 512, 1280, 2176, 12352, 49184, 0]],
        '仇' : [12, 13, 0, -2, 12, [512, 8704, 8704, 24512, 16960, 49728, 16960, 16960, 17472, 17472, 18464, 28704, 0]],
        '休' : [12, 13, 0, -2, 12, [256, 8448, 8448, 24544, 16640, 49408, 17152, 17792, 18752, 20768, 16640, 16640, 0]],
        '及' : [12, 13, 0, -2, 12, [0, 32640, 4224, 4352, 4608, 5056, 8256, 9280, 17024, 16768, 1600, 14368, 0]],
        '吸' : [12, 13, 0, -2, 12, [0, 8128, 58432, 42112, 42240, 42432, 43072, 43584, 45376, 41088, 58176, 3104, 0]],
        '宮' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 40736, 37152, 7936, 2048, 16256, 8320, 8320, 16256, 0]],
        '弓' : [12, 13, 0, -2, 12, [0, 16320, 64, 64, 16320, 8192, 8192, 16352, 16416, 16416, 64, 896, 0]],
        '急' : [12, 13, 0, -2, 12, [0, 7936, 8704, 32704, 64, 16320, 64, 16320, 1024, 4672, 20512, 36736, 0]],
        '救' : [12, 13, 0, -2, 12, [8704, 8704, 11232, 64064, 8256, 42048, 26944, 8512, 26752, 42112, 8512, 26144, 0]],
        '朽' : [12, 13, 0, -2, 12, [0, 12256, 8704, 61952, 8704, 9184, 25120, 25632, 46112, 40992, 8256, 9088, 0]],
        '求' : [12, 13, 0, -2, 12, [1024, 1088, 1056, 65472, 1024, 17472, 9856, 1280, 3200, 13376, 50208, 3072, 0]],
        '汲' : [12, 13, 0, -2, 12, [0, 8128, 33856, 17536, 1280, 34240, 18496, 2624, 20800, 20608, 33600, 35872, 0]],
        '泣' : [12, 13, 0, -2, 12, [512, 512, 33280, 24544, 0, 33856, 17472, 1088, 1152, 9344, 16512, 40928, 0]],
        '灸' : [12, 13, 0, -2, 12, [0, 8064, 24832, 1536, 6528, 57440, 1024, 9280, 17536, 2560, 12672, 49248, 0]],
        '球' : [12, 13, 0, -2, 12, [256, 320, 57632, 24512, 16640, 18720, 58688, 16640, 17792, 18752, 61728, 768, 0]],
        '究' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 25024, 2048, 32512, 2304, 2336, 12576, 49376, 0]],
        '窮' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 31712, 18464, 31712, 18944, 65504, 10784, 18464, 39360, 0]],
        '笈' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 65472, 8448, 9152, 18496, 17536, 33664, 40032, 0]],
        '級' : [12, 13, 0, -2, 12, [0, 12224, 17472, 38016, 62720, 9664, 18496, 59968, 12608, 41088, 41792, 11296, 0]],
        '糾' : [12, 13, 0, -2, 12, [0, 8256, 16960, 37440, 62016, 8768, 19008, 64096, 9152, 44096, 41024, 8256, 0]],
        '給' : [12, 13, 0, -2, 12, [0, 8640, 16928, 37888, 61440, 9184, 18432, 63488, 9184, 43552, 43552, 9184, 0]],
        '旧' : [12, 13, 0, -2, 12, [0, 20448, 18464, 18464, 18464, 18464, 20448, 18464, 18464, 18464, 18464, 20448, 0]],
        '牛' : [12, 13, 0, -2, 12, [1024, 17408, 17408, 32704, 33792, 33792, 1024, 65504, 1024, 1024, 1024, 1024, 0]],
        '去' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 1024, 1024, 65504, 2048, 2176, 4224, 9024, 15424, 0]],
        '居' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16896, 32736, 16896, 16896, 24512, 36928, 36928, 8128, 0]],
        '巨' : [12, 13, 0, -2, 12, [0, 32736, 16384, 16384, 32640, 16512, 16512, 32640, 16384, 16384, 16384, 32736, 0]],
        '拒' : [12, 13, 0, -2, 12, [0, 20448, 18432, 59392, 20416, 18496, 18496, 28608, 51200, 18432, 18432, 53216, 0]],
        '拠' : [12, 13, 0, -2, 12, [4096, 20928, 23872, 62784, 21824, 21824, 21824, 27200, 51776, 18528, 21504, 58336, 0]],
        '挙' : [12, 13, 0, -2, 12, [2112, 9344, 4352, 65504, 8320, 24384, 33824, 16256, 1024, 65504, 1024, 7168, 0]],
        '渠' : [12, 13, 0, -2, 12, [0, 18400, 9216, 34752, 17472, 6080, 9216, 65504, 3328, 13440, 50272, 1024, 0]],
        '虚' : [12, 13, 0, -2, 12, [1024, 1984, 1024, 32736, 18464, 24512, 18432, 20416, 25888, 38208, 34048, 16352, 0]],
        '許' : [12, 13, 0, -2, 12, [512, 61952, 992, 62592, 1152, 61568, 128, 2016, 61568, 36992, 36992, 61568, 0]],
        '距' : [12, 13, 0, -2, 12, [0, 64480, 35328, 35328, 64480, 8736, 8736, 48096, 41472, 41472, 41472, 64480, 0]],
        '鋸' : [12, 13, 0, -2, 12, [0, 14304, 19488, 34784, 62592, 10208, 64640, 9344, 44000, 43552, 8736, 64480, 0]],
        '漁' : [12, 13, 0, -2, 12, [0, 1984, 34944, 24544, 2336, 36832, 18720, 2336, 20448, 16384, 35488, 37536, 0]],
        '禦' : [12, 13, 0, -2, 12, [4096, 24544, 38048, 16288, 17568, 54496, 24448, 0, 65504, 1024, 17472, 35872, 0]],
        '魚' : [12, 13, 0, -2, 12, [0, 8064, 8448, 32704, 50240, 32704, 17472, 17472, 32704, 0, 21056, 35104, 0]],
        '亨' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 4352, 7936, 0, 65504, 512, 1024, 1024, 7168, 0]],
        '享' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 4352, 16256, 256, 512, 65504, 1024, 1024, 7168, 0]],
        '京' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 0, 16256, 8320, 8320, 16256, 1024, 17472, 33824, 7168, 0]],
        '供' : [12, 13, 0, -2, 12, [1152, 9344, 9344, 24544, 17536, 50304, 17536, 24544, 16384, 17536, 18496, 20512, 0]],
        '侠' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16640, 18720, 50496, 16640, 24544, 16640, 17024, 17472, 22560, 0]],
        '僑' : [12, 13, 0, -2, 12, [0, 12256, 8704, 24544, 17536, 52416, 22432, 24576, 24544, 21664, 22432, 20576, 0]],
        '兇' : [12, 13, 0, -2, 12, [0, 20800, 19008, 17472, 19008, 20800, 16448, 32704, 4608, 4640, 8736, 49632, 0]],
        '競' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 20800, 65504, 0, 64480, 35360, 64480, 20800, 37440, 40032, 0]],
        '共' : [12, 13, 0, -2, 12, [4352, 4352, 4352, 32704, 4352, 4352, 4352, 65504, 0, 4352, 8320, 16448, 0]],
        '凶' : [12, 13, 0, -2, 12, [0, 32800, 41120, 37152, 35360, 33824, 35360, 37152, 41120, 32800, 32800, 65504, 0]],
        '協' : [12, 13, 0, -2, 12, [512, 16896, 20416, 16960, 62016, 17472, 24544, 19104, 19104, 19104, 21280, 22112, 0]],
        '匡' : [12, 13, 0, -2, 12, [0, 65504, 32768, 49120, 33280, 33280, 40896, 33280, 33280, 49120, 32768, 65504, 0]],
        '卿' : [12, 13, 0, -2, 12, [0, 16096, 51872, 36512, 43680, 43680, 44704, 43168, 60064, 10976, 20096, 17024, 0]],
        '叫' : [12, 13, 0, -2, 12, [0, 64, 62016, 37440, 37440, 37440, 37440, 37472, 37824, 40000, 61504, 64, 0]],
        '喬' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 4480, 12608, 57120, 0, 32704, 20800, 24384, 16576, 0]],
        '境' : [12, 13, 0, -2, 12, [512, 16896, 24544, 17536, 58496, 24544, 18496, 20416, 18496, 20416, 58496, 6368, 0]],
        '峡' : [12, 13, 0, -2, 12, [256, 8448, 16352, 8448, 43296, 42304, 41216, 49120, 41216, 41600, 58432, 6176, 0]],
        '強' : [12, 13, 0, -2, 12, [512, 58368, 10272, 12256, 57600, 36832, 35104, 59680, 12256, 8448, 8480, 53216, 0]],
        '彊' : [12, 13, 0, -2, 12, [0, 61408, 9504, 10208, 58656, 34784, 32768, 61408, 9504, 10208, 9504, 57312, 0]],
        '怯' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8448, 41216, 41216, 45024, 41472, 41536, 9280, 10400, 12064, 0]],
        '恐' : [12, 13, 0, -2, 12, [0, 63424, 9280, 9792, 9536, 13376, 49248, 1024, 4608, 4160, 20512, 36736, 0]],
        '恭' : [12, 13, 0, -2, 12, [4352, 4352, 32704, 4352, 4352, 65504, 8320, 17472, 37920, 9536, 17728, 3072, 0]],
        '挟' : [12, 13, 0, -2, 12, [256, 16640, 20448, 57600, 18720, 17728, 16640, 28640, 49408, 17024, 17472, 55328, 0]],
        '教' : [12, 13, 0, -2, 12, [8448, 9472, 64992, 10816, 10816, 64576, 16704, 47424, 2176, 64640, 4416, 13856, 0]],
        '橋' : [12, 13, 0, -2, 12, [0, 12256, 8448, 65504, 9344, 11456, 30624, 24576, 45024, 43680, 11168, 10336, 0]],
        '況' : [12, 13, 0, -2, 12, [0, 4064, 34848, 18464, 2080, 36832, 17024, 640, 17024, 17056, 33952, 39136, 0]],
        '狂' : [12, 13, 0, -2, 12, [0, 12256, 41216, 16640, 16640, 41216, 12256, 24832, 41216, 8448, 8448, 53216, 0]],
        '狭' : [12, 13, 0, -2, 12, [256, 8448, 45024, 16640, 18720, 42304, 8448, 28640, 41216, 8832, 9280, 55328, 0]],
        '矯' : [12, 13, 0, -2, 12, [0, 36832, 61696, 45024, 9344, 60608, 14240, 8192, 24544, 21664, 38816, 36960, 0]],
        '胸' : [12, 13, 0, -2, 12, [2048, 59392, 45024, 45344, 59936, 42144, 43680, 61856, 41120, 49056, 40992, 41152, 0]],
        '脅' : [12, 13, 0, -2, 12, [1024, 16256, 2176, 4224, 64480, 10528, 19040, 49088, 8256, 16320, 8256, 8384, 0]],
        '興' : [12, 13, 0, -2, 12, [0, 57184, 37152, 65504, 37152, 63456, 38176, 38176, 65504, 0, 16448, 32800, 0]],
        '蕎' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 65504, 4352, 16192, 49184, 32704, 20800, 24384, 16576, 0]],
        '郷' : [12, 13, 0, -2, 12, [0, 12000, 19104, 36512, 60064, 10944, 20128, 34976, 60064, 10944, 20096, 17024, 0]],
        '鏡' : [12, 13, 0, -2, 12, [256, 12544, 20448, 33344, 62016, 10208, 64576, 10176, 46144, 42944, 8832, 64736, 0]],
        '響' : [12, 13, 0, -2, 12, [0, 20192, 43680, 20160, 43680, 60896, 9344, 32704, 4352, 65504, 8320, 16256, 0]],
        '饗' : [12, 13, 0, -2, 12, [0, 20192, 43680, 20160, 43680, 60896, 9344, 32704, 41120, 16256, 8768, 63968, 0]],
        '驚' : [12, 13, 0, -2, 12, [512, 20992, 65504, 21824, 30080, 2656, 32704, 17408, 32736, 32, 21792, 38240, 0]],
        '仰' : [12, 13, 0, -2, 12, [0, 8960, 11488, 18720, 18720, 51488, 18720, 18720, 18720, 19232, 23904, 16640, 0]],
        '凝' : [12, 13, 0, -2, 12, [0, 5088, 40000, 20608, 4064, 4256, 7808, 2176, 24288, 19072, 38528, 41952, 0]],
        '尭' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 5376, 65504, 4352, 4352, 32704, 2560, 12832, 49632, 0]],
        '暁' : [12, 13, 0, -2, 12, [256, 256, 61408, 41216, 42304, 49120, 58432, 42048, 45024, 41600, 58528, 6368, 0]],
        '業' : [12, 13, 0, -2, 12, [2560, 10816, 10880, 65504, 4352, 4352, 32704, 1024, 65504, 3328, 13440, 50272, 0]],
        '局' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16384, 32736, 16416, 24352, 20768, 20768, 40736, 32960, 0]],
        '曲' : [12, 13, 0, -2, 12, [2304, 2304, 2304, 32736, 18720, 18720, 18720, 32736, 18720, 18720, 18720, 32736, 0]],
        '極' : [12, 13, 0, -2, 12, [0, 12256, 9216, 63200, 8736, 11936, 27296, 27200, 44736, 41760, 9216, 12256, 0]],
        '玉' : [12, 13, 0, -2, 12, [0, 65504, 1024, 1024, 1024, 1024, 32704, 1024, 1088, 1056, 1024, 65504, 0]],
        '桐' : [12, 13, 0, -2, 12, [0, 12256, 10272, 63520, 11168, 10272, 27552, 27296, 43680, 43936, 10272, 10336, 0]],
        '粁' : [12, 13, 0, -2, 12, [0, 8384, 44800, 41216, 8448, 61696, 12256, 24832, 28928, 41216, 41216, 8448, 0]],
        '僅' : [12, 13, 0, -2, 12, [1152, 9344, 16352, 17536, 20448, 51488, 18720, 20448, 16640, 20448, 16640, 24544, 0]],
        '勤' : [12, 13, 0, -2, 12, [256, 20736, 63744, 22496, 63776, 43296, 43296, 63776, 8736, 64032, 8256, 63872, 0]],
        '均' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 18464, 59424, 18208, 16416, 16800, 20000, 16416, 57408, 384, 0]],
        '巾' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 33824, 33824, 33824, 33824, 33824, 34016, 1024, 1024, 0]],
        '錦' : [12, 13, 0, -2, 12, [128, 12544, 18400, 33824, 63456, 9248, 63456, 8448, 47072, 42272, 9504, 64864, 0]],
        '斤' : [12, 13, 0, -2, 12, [0, 448, 32256, 16384, 16384, 32736, 16640, 16640, 16640, 33024, 33024, 256, 0]],
        '欣' : [12, 13, 0, -2, 12, [512, 6656, 25568, 17056, 17056, 32416, 18560, 18560, 18752, 35136, 35360, 3104, 0]],
        '欽' : [12, 13, 0, -2, 12, [512, 12800, 19424, 33440, 64160, 8864, 63616, 8320, 43328, 43328, 8736, 64544, 0]],
        '琴' : [12, 13, 0, -2, 12, [0, 64480, 8320, 31680, 8320, 65504, 4352, 11904, 16480, 49024, 128, 256, 0]],
        '禁' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8576, 29376, 42144, 16256, 0, 65504, 1024, 17472, 35872, 0]],
        '禽' : [12, 13, 0, -2, 12, [0, 7936, 9344, 32704, 35360, 9344, 10880, 10368, 65504, 35104, 40736, 32864, 0]],
        '筋' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42240, 30976, 20448, 31008, 18720, 31008, 18976, 35360, 39104, 0]],
        '緊' : [12, 13, 0, -2, 12, [0, 64480, 40992, 63808, 35200, 65120, 35072, 4672, 65504, 1024, 9344, 50272, 0]],
        '芹' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 960, 31744, 16384, 32736, 16640, 33024, 33024, 256, 0]],
        '菌' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 33824, 49056, 33824, 36128, 46240, 33824, 65504, 0]],
        '衿' : [12, 13, 0, -2, 12, [0, 9152, 9248, 63488, 6112, 4096, 8192, 26592, 45088, 10272, 8256, 8576, 0]],
        '襟' : [12, 13, 0, -2, 12, [1088, 17472, 24544, 58560, 11616, 9280, 20416, 24576, 57312, 16640, 18752, 21280, 0]],
        '謹' : [12, 13, 0, -2, 12, [576, 57920, 4064, 57920, 4064, 59680, 2336, 4064, 57600, 45024, 41216, 61408, 0]],
        '近' : [12, 13, 0, -2, 12, [0, 192, 18176, 9216, 1024, 2016, 58496, 10368, 10368, 8320, 20480, 36832, 0]],
        '金' : [12, 13, 0, -2, 12, [0, 3584, 4352, 8320, 24384, 33824, 1024, 32704, 1024, 17472, 9344, 65504, 0]],
        '吟' : [12, 13, 0, -2, 12, [0, 960, 62496, 38912, 38880, 36864, 36864, 38880, 36896, 36896, 61504, 384, 0]],
        '銀' : [12, 13, 0, -2, 12, [0, 13280, 18976, 33760, 64032, 8736, 64480, 8832, 43680, 43680, 8768, 64288, 0]],
        '九' : [12, 13, 0, -2, 12, [2048, 2048, 2048, 65280, 2304, 2304, 2304, 2304, 4352, 4384, 8480, 49376, 0]],
        '倶' : [12, 13, 0, -2, 12, [0, 12224, 10304, 20416, 18496, 53184, 18496, 18496, 24544, 16384, 18496, 20512, 0]],
        '句' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16416, 16416, 40480, 37408, 4640, 4640, 7712, 64, 384, 0]],
        '区' : [12, 13, 0, -2, 12, [0, 65504, 32768, 36928, 34944, 34048, 33280, 34048, 34944, 36928, 32768, 65504, 0]],
        '狗' : [12, 13, 0, -2, 12, [1024, 9216, 42976, 18464, 18464, 42784, 9504, 25888, 42784, 8224, 8256, 49536, 0]],
        '玖' : [12, 13, 0, -2, 12, [1024, 1024, 63424, 17472, 17472, 59456, 18560, 16512, 16640, 57984, 3136, 12320, 0]],
        '矩' : [12, 13, 0, -2, 12, [0, 34784, 64512, 41984, 10208, 64544, 9248, 10208, 21504, 19456, 33792, 34784, 0]],
        '苦' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 65504, 1024, 1024, 32704, 16448, 16448, 32704, 0]],
        '躯' : [12, 13, 0, -2, 12, [8192, 17376, 64000, 35488, 64160, 35392, 64064, 36512, 64160, 11040, 18944, 39904, 0]],
        '駆' : [12, 13, 0, -2, 12, [0, 64480, 41472, 64160, 41632, 64064, 41536, 64160, 2720, 43808, 2560, 13280, 0]],
        '駈' : [12, 13, 0, -2, 12, [0, 63680, 41728, 64000, 41472, 64480, 41536, 64064, 2624, 43584, 2624, 14304, 0]],
        '駒' : [12, 13, 0, -2, 12, [512, 64000, 41952, 62496, 42016, 62368, 41632, 64160, 2976, 43040, 2112, 12672, 0]],
        '具' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 16256, 8320, 8320, 65504, 0, 12672, 49248, 0]],
        '愚' : [12, 13, 0, -2, 12, [0, 16256, 9344, 16256, 9344, 65504, 33824, 48992, 1024, 4672, 20512, 36736, 0]],
        '虞' : [12, 13, 0, -2, 12, [1920, 1024, 32736, 17440, 24512, 17536, 21632, 24544, 16416, 49120, 34944, 12384, 0]],
        '喰' : [12, 13, 0, -2, 12, [0, 1920, 59456, 45344, 44992, 43072, 44992, 43072, 44992, 43360, 59520, 3680, 0]],
        '空' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 35360, 4608, 25536, 0, 16256, 1024, 1024, 65504, 0]],
        '偶' : [12, 13, 0, -2, 12, [0, 12256, 10528, 20448, 18720, 53216, 16640, 20448, 18976, 19104, 20384, 18528, 0]],
        '寓' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 42144, 16256, 9344, 16256, 1024, 65504, 33952, 49056, 32864, 0]],
        '遇' : [12, 13, 0, -2, 12, [0, 4064, 18720, 12256, 2336, 4064, 57600, 12256, 10528, 11168, 20480, 36832, 0]],
        '隅' : [12, 13, 0, -2, 12, [0, 61408, 43296, 45024, 51488, 45024, 41216, 45024, 43552, 51872, 36768, 34912, 0]],
        '串' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 17472, 17472, 32704, 1024, 65504, 33824, 65504, 1024, 1024, 0]],
        '櫛' : [12, 13, 0, -2, 12, [256, 10496, 12256, 62784, 12000, 10912, 28320, 27296, 44704, 43168, 10976, 11904, 0]],
        '釧' : [12, 13, 0, -2, 12, [0, 12832, 19104, 33440, 64160, 8864, 64160, 8864, 45728, 41632, 9248, 62496, 0]],
        '屑' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 21056, 19072, 24544, 20512, 24544, 36896, 40928, 4128, 0]],
        '屈' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 16640, 18720, 18720, 20448, 16640, 37152, 37152, 8160, 0]],
        '掘' : [12, 13, 0, -2, 12, [0, 20448, 18464, 61408, 18560, 19104, 19104, 27616, 51328, 21664, 21664, 51168, 0]],
        '窟' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 32736, 16416, 32736, 21056, 40896, 41504, 16352, 0]],
        '沓' : [12, 13, 0, -2, 12, [1024, 1024, 63008, 5440, 9344, 52320, 0, 32704, 16448, 32704, 16448, 32704, 0]],
        '靴' : [12, 13, 0, -2, 12, [128, 21120, 64128, 21120, 30368, 8896, 64128, 43648, 64128, 8864, 64160, 8928, 0]],
        '轡' : [12, 13, 0, -2, 12, [1024, 17472, 49056, 21824, 65504, 21824, 65504, 17472, 16256, 8320, 8320, 16256, 0]],
        '窪' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 8640, 37120, 20448, 256, 12224, 16640, 40928, 0]],
        '熊' : [12, 13, 0, -2, 12, [8704, 16992, 35712, 64032, 35808, 64000, 35808, 64000, 35808, 0, 21056, 35104, 0]],
        '隈' : [12, 13, 0, -2, 12, [0, 61408, 43296, 45024, 51488, 45024, 40960, 49120, 43296, 51520, 34944, 40544, 0]],
        '粂' : [12, 13, 0, -2, 12, [0, 8064, 24832, 1536, 6400, 58592, 9344, 1280, 65504, 3328, 13440, 50272, 0]],
        '栗' : [12, 13, 0, -2, 12, [0, 65504, 2560, 32704, 19008, 19008, 32704, 1024, 65504, 3328, 13440, 50272, 0]],
        '繰' : [12, 13, 0, -2, 12, [0, 10176, 17472, 36832, 60064, 10912, 20192, 61696, 12256, 41728, 42304, 10528, 0]],
        '桑' : [12, 13, 0, -2, 12, [0, 32640, 4224, 4352, 3584, 64480, 18720, 20800, 65504, 3328, 13440, 50272, 0]],
        '鍬' : [12, 13, 0, -2, 12, [128, 13184, 19584, 33952, 62624, 12192, 62592, 9408, 44352, 46400, 9760, 62496, 0]],
        '勲' : [12, 13, 0, -2, 12, [256, 63744, 10208, 63776, 43296, 63776, 43552, 64032, 8384, 63488, 576, 37152, 0]],
        '君' : [12, 13, 0, -2, 12, [0, 16320, 1088, 65504, 1088, 1088, 32704, 2048, 16320, 53312, 4160, 8128, 0]],
        '薫' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 32704, 9344, 16256, 9344, 65504, 0, 21056, 35104, 0]],
        '訓' : [12, 13, 0, -2, 12, [0, 61984, 672, 62112, 672, 62112, 672, 672, 62112, 37536, 37920, 62496, 0]],
        '群' : [12, 13, 0, -2, 12, [576, 64064, 10368, 65504, 10368, 10368, 64480, 8320, 30848, 53216, 18560, 30848, 0]],
        '軍' : [12, 13, 0, -2, 12, [0, 65504, 33824, 65504, 9344, 16256, 9344, 9344, 16256, 1024, 65504, 1024, 0]],
        '郡' : [12, 13, 0, -2, 12, [0, 64992, 9504, 65344, 9600, 9536, 64800, 8480, 32032, 50624, 17664, 32000, 0]],
        '卦' : [12, 13, 0, -2, 12, [4096, 4224, 64640, 4224, 4224, 65216, 4256, 4240, 64640, 4224, 4224, 65152, 0]],
        '袈' : [12, 13, 0, -2, 12, [8192, 8672, 64800, 9504, 17696, 39392, 1024, 65504, 5184, 57920, 8576, 31840, 0]],
        '祁' : [12, 13, 0, -2, 12, [8192, 9184, 8736, 64032, 2624, 4736, 8768, 29216, 43552, 8736, 8896, 8704, 0]],
        '係' : [12, 13, 0, -2, 12, [0, 12256, 8704, 17472, 18560, 50432, 16928, 24544, 16640, 18752, 20768, 16640, 0]],
        '傾' : [12, 13, 0, -2, 12, [0, 10208, 8320, 21472, 21024, 55264, 23072, 21472, 21024, 23520, 16960, 19488, 0]],
        '刑' : [12, 13, 0, -2, 12, [0, 64544, 18720, 18720, 18720, 64800, 18720, 18720, 18720, 18464, 34848, 34912, 0]],
        '兄' : [12, 13, 0, -2, 12, [0, 16256, 8320, 8320, 8320, 8320, 16256, 4608, 4608, 4640, 8736, 50144, 0]],
        '啓' : [12, 13, 0, -2, 12, [512, 64000, 992, 32320, 19008, 31104, 34400, 32768, 16256, 8320, 8320, 16256, 0]],
        '圭' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 1024, 65504, 1024, 1024, 32704, 1024, 1024, 65504, 0]],
        '珪' : [12, 13, 0, -2, 12, [256, 256, 63456, 16640, 16640, 20448, 57600, 16640, 18400, 16640, 61696, 4064, 0]],
        '型' : [12, 13, 0, -2, 12, [0, 64800, 18720, 18720, 64800, 18720, 34912, 1024, 16256, 1024, 1024, 65504, 0]],
        '契' : [12, 13, 0, -2, 12, [8192, 10208, 63776, 8480, 31264, 8736, 63680, 1024, 65504, 2560, 12672, 49248, 0]],
        '形' : [12, 13, 0, -2, 12, [0, 65056, 18496, 18560, 18432, 65056, 18496, 18560, 18432, 18464, 34880, 34944, 0]],
        '径' : [12, 13, 0, -2, 12, [0, 12256, 16416, 33344, 384, 11872, 16640, 49408, 18400, 16640, 16640, 20448, 0]],
        '恵' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 17472, 32704, 17472, 32704, 4672, 20512, 36736, 0]],
        '慶' : [12, 13, 0, -2, 12, [512, 512, 32736, 18688, 32736, 18720, 32736, 18592, 28608, 37952, 34688, 14432, 0]],
        '慧' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8320, 64480, 8320, 32704, 64, 32704, 4608, 20544, 36768, 0]],
        '憩' : [12, 13, 0, -2, 12, [128, 63744, 9184, 65056, 9184, 64032, 35360, 64480, 1024, 4672, 20512, 36736, 0]],
        '掲' : [12, 13, 0, -2, 12, [0, 20416, 18496, 61376, 18496, 20416, 17408, 28640, 55328, 18720, 20256, 49344, 0]],
        '携' : [12, 13, 0, -2, 12, [1152, 17536, 20448, 59648, 24544, 18688, 20448, 26880, 53184, 16992, 17440, 55488, 0]],
        '敬' : [12, 13, 0, -2, 12, [512, 20992, 64480, 21056, 16960, 31808, 34112, 58688, 42112, 58496, 2368, 12832, 0]],
        '景' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 16256, 1024, 65504, 4352, 7936, 17472, 35872, 0]],
        '桂' : [12, 13, 0, -2, 12, [256, 8448, 10208, 61696, 8448, 12256, 24832, 24832, 47072, 41216, 8448, 12256, 0]],
        '渓' : [12, 13, 0, -2, 12, [0, 8160, 32768, 21024, 2368, 33024, 20448, 256, 24544, 17024, 33856, 38944, 0]],
        '畦' : [12, 13, 0, -2, 12, [128, 128, 64480, 43136, 43136, 45024, 63616, 43136, 44000, 43136, 63616, 2016, 0]],
        '稽' : [12, 13, 0, -2, 12, [256, 6464, 61408, 8832, 11488, 63360, 9248, 26592, 28672, 42976, 42016, 10208, 0]],
        '系' : [12, 13, 0, -2, 12, [0, 32704, 2048, 4352, 8704, 5248, 2112, 65504, 1024, 9344, 17472, 33824, 0]],
        '経' : [12, 13, 0, -2, 12, [0, 10208, 16416, 37440, 61824, 9824, 16640, 61696, 10208, 45312, 41216, 12256, 0]],
        '継' : [12, 13, 0, -2, 12, [0, 9344, 17568, 38592, 62592, 10208, 17536, 64704, 9632, 46720, 42112, 10208, 0]],
        '繋' : [12, 13, 0, -2, 12, [4096, 5056, 65088, 21600, 32704, 21568, 64896, 4704, 9344, 65504, 9344, 50272, 0]],
        '罫' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 4224, 31872, 4224, 65216, 4256, 31872, 4224, 65152, 0]],
        '茎' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 4224, 7936, 58592, 1024, 16256, 1024, 65504, 0]],
        '荊' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 64800, 10528, 10528, 64800, 10528, 18464, 18528, 0]],
        '蛍' : [12, 13, 0, -2, 12, [2112, 17536, 8448, 65504, 33824, 49056, 9344, 9344, 16256, 1056, 1056, 65504, 0]],
        '計' : [12, 13, 0, -2, 12, [128, 61568, 128, 61568, 2016, 61568, 128, 128, 61568, 36992, 36992, 61568, 0]],
        '詣' : [12, 13, 0, -2, 12, [0, 62560, 1920, 62464, 1056, 63456, 0, 2016, 62496, 38880, 37920, 63456, 0]],
        '警' : [12, 13, 0, -2, 12, [512, 20992, 65504, 21824, 30080, 2624, 65504, 0, 32704, 8320, 8320, 16256, 0]],
        '軽' : [12, 13, 0, -2, 12, [0, 10208, 63520, 8768, 63872, 44640, 63616, 43136, 64480, 8320, 63616, 10208, 0]],
        '頚' : [12, 13, 0, -2, 12, [0, 64480, 2176, 21472, 12832, 53216, 8736, 9184, 64032, 9184, 8768, 64544, 0]],
        '鶏' : [12, 13, 0, -2, 12, [256, 64000, 1984, 37952, 22464, 1088, 63456, 9216, 65504, 8224, 21792, 38240, 0]],
        '芸' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 32704, 0, 65504, 4224, 4224, 9024, 15424, 0]],
        '迎' : [12, 13, 0, -2, 12, [0, 768, 19680, 10528, 2336, 2336, 59680, 11040, 11616, 8448, 20480, 36832, 0]],
        '鯨' : [12, 13, 0, -2, 12, [128, 28800, 38896, 8192, 64480, 43552, 64032, 44000, 63616, 672, 42128, 43392, 0]],
        '劇' : [12, 13, 0, -2, 12, [4096, 7200, 4384, 65312, 37152, 64800, 37152, 48928, 51488, 14880, 52256, 12896, 0]],
        '戟' : [12, 13, 0, -2, 12, [8448, 8512, 63776, 8448, 65504, 35072, 63808, 35136, 63616, 8320, 62272, 11296, 0]],
        '撃' : [12, 13, 0, -2, 12, [4096, 5056, 65088, 21600, 32704, 21056, 65408, 1120, 32640, 1024, 65504, 1024, 0]],
        '激' : [12, 13, 0, -2, 12, [1152, 2176, 40672, 21152, 7968, 37408, 24224, 2208, 20032, 21056, 33440, 36128, 0]],
        '隙' : [12, 13, 0, -2, 12, [256, 58688, 43296, 41216, 53216, 43040, 45024, 43040, 45024, 49408, 35136, 37664, 0]],
        '桁' : [12, 13, 0, -2, 12, [0, 9696, 10240, 61440, 8704, 9696, 27712, 29760, 42048, 42048, 9280, 9408, 0]],
        '傑' : [12, 13, 0, -2, 12, [64, 11840, 11232, 23104, 18400, 50240, 22848, 16640, 24544, 17152, 17728, 22816, 0]],
        '欠' : [12, 13, 0, -2, 12, [8192, 8192, 16320, 9280, 17472, 17536, 1024, 2560, 2560, 4352, 8320, 49248, 0]],
        '決' : [12, 13, 0, -2, 12, [256, 256, 36832, 16672, 288, 33056, 20448, 256, 256, 8832, 17472, 38944, 0]],
        '潔' : [12, 13, 0, -2, 12, [2048, 2528, 40608, 18592, 7456, 35424, 23808, 576, 24544, 16896, 37440, 41504, 0]],
        '穴' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 32800, 40736, 256, 256, 4352, 4224, 8320, 49248, 0]],
        '結' : [12, 13, 0, -2, 12, [128, 8320, 18400, 36992, 61568, 10208, 16384, 63488, 9184, 43552, 43552, 9184, 0]],
        '血' : [12, 13, 0, -2, 12, [512, 1024, 2048, 32704, 19008, 19008, 19008, 19008, 19008, 19008, 19008, 65504, 0]],
        '訣' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57632, 288, 57632, 4064, 256, 57600, 41600, 42048, 63520, 0]],
        '月' : [12, 13, 0, -2, 12, [0, 16320, 8256, 8256, 16320, 8256, 8256, 16320, 8256, 8256, 16448, 16832, 0]],
        '件' : [12, 13, 0, -2, 12, [128, 9344, 9344, 18400, 18560, 51328, 16512, 24544, 16512, 16512, 16512, 16512, 0]],
        '倹' : [12, 13, 0, -2, 12, [0, 9088, 9280, 20448, 20736, 53216, 18720, 18720, 20448, 16640, 18048, 22624, 0]],
        '倦' : [12, 13, 0, -2, 12, [512, 10528, 9280, 24544, 17536, 57312, 17472, 20448, 28736, 20416, 18464, 20448, 0]],
        '健' : [12, 13, 0, -2, 12, [128, 8320, 15328, 18592, 20448, 53408, 23520, 18560, 28640, 20608, 22528, 26592, 0]],
        '兼' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 4608, 32640, 4736, 65504, 4736, 32640, 4608, 21056, 37408, 0]],
        '券' : [12, 13, 0, -2, 12, [1024, 9280, 9344, 32704, 2304, 65504, 4224, 16320, 51360, 2176, 4224, 25344, 0]],
        '剣' : [12, 13, 0, -2, 12, [0, 30752, 34080, 64800, 8480, 64800, 42272, 42272, 64800, 8224, 22560, 34400, 0]],
        '喧' : [12, 13, 0, -2, 12, [512, 512, 65504, 40992, 44992, 43072, 44992, 43072, 43072, 44992, 57344, 8160, 0]],
        '圏' : [12, 13, 0, -2, 12, [0, 65504, 38048, 38176, 49056, 35360, 65504, 37664, 44704, 34848, 35104, 65504, 0]],
        '堅' : [12, 13, 0, -2, 12, [0, 64480, 40992, 63776, 35136, 63616, 41344, 65120, 1024, 16256, 1024, 65504, 0]],
        '嫌' : [12, 13, 0, -2, 12, [576, 17024, 20448, 62080, 22496, 21152, 24544, 37536, 59360, 12928, 18112, 19104, 0]],
        '建' : [12, 13, 0, -2, 12, [256, 256, 61408, 8480, 24544, 33056, 59360, 8448, 45024, 16640, 24576, 40928, 0]],
        '憲' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33824, 16256, 1024, 65504, 19008, 32704, 4608, 20544, 36768, 0]],
        '懸' : [12, 13, 0, -2, 12, [0, 15328, 43328, 47744, 43296, 44000, 64832, 20768, 37888, 4672, 20512, 36736, 0]],
        '拳' : [12, 13, 0, -2, 12, [2112, 17536, 8448, 32704, 4352, 65504, 9344, 57152, 1056, 65504, 1024, 7168, 0]],
        '捲' : [12, 13, 0, -2, 12, [512, 18720, 17472, 65504, 17536, 24544, 17472, 32736, 51264, 24512, 18464, 53216, 0]],
        '検' : [12, 13, 0, -2, 12, [0, 10112, 10304, 65504, 8448, 12256, 26912, 26912, 45024, 41216, 9856, 14432, 0]],
        '権' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 63744, 12256, 8832, 26592, 27776, 47072, 42112, 9344, 10208, 0]],
        '牽' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2304, 4608, 65504, 41248, 16320, 17408, 65504, 1024, 1024, 0]],
        '犬' : [12, 13, 0, -2, 12, [1024, 1152, 1088, 1024, 65504, 1024, 1024, 2560, 2560, 4352, 8320, 49248, 0]],
        '献' : [12, 13, 0, -2, 12, [8320, 8352, 64672, 8320, 65504, 38016, 42112, 64640, 42304, 64832, 42528, 35872, 0]],
        '研' : [12, 13, 0, -2, 12, [0, 63456, 8768, 8768, 8768, 32736, 53824, 21056, 21056, 21056, 21568, 30784, 0]],
        '硯' : [12, 13, 0, -2, 12, [0, 2016, 62496, 10208, 9248, 10208, 29728, 54304, 22496, 21120, 21664, 30944, 0]],
        '絹' : [12, 13, 0, -2, 12, [0, 10208, 17440, 38880, 61440, 10208, 17440, 63456, 9248, 47072, 42016, 9312, 0]],
        '県' : [12, 13, 0, -2, 12, [0, 8064, 36992, 40832, 36992, 40832, 36992, 36992, 65504, 1024, 17472, 33824, 0]],
        '肩' : [12, 13, 0, -2, 12, [0, 65504, 0, 32704, 16448, 32704, 20544, 24512, 20544, 40896, 36928, 4288, 0]],
        '見' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 16256, 8320, 8320, 16256, 4640, 8736, 50144, 0]],
        '謙' : [12, 13, 0, -2, 12, [576, 57984, 8160, 57984, 4064, 58016, 8160, 672, 61408, 41600, 42688, 60064, 0]],
        '賢' : [12, 13, 0, -2, 12, [0, 64480, 40992, 63808, 35200, 41536, 65440, 8320, 16256, 8320, 16256, 49248, 0]],
        '軒' : [12, 13, 0, -2, 12, [8192, 10208, 63616, 8320, 63616, 43136, 65504, 43136, 63616, 8320, 63616, 8320, 0]],
        '遣' : [12, 13, 0, -2, 12, [256, 256, 36832, 18720, 2336, 8160, 58432, 10208, 9248, 10208, 20480, 36832, 0]],
        '鍵' : [12, 13, 0, -2, 12, [128, 12416, 19936, 33952, 63456, 10400, 64992, 9344, 47072, 42112, 11264, 62432, 0]],
        '険' : [12, 13, 0, -2, 12, [0, 59264, 43072, 49120, 49408, 45024, 43296, 43296, 45024, 49408, 34432, 39008, 0]],
        '顕' : [12, 13, 0, -2, 12, [0, 64480, 34944, 64480, 35360, 64480, 21024, 21472, 55840, 21472, 21056, 64544, 0]],
        '験' : [12, 13, 0, -2, 12, [0, 62400, 42016, 65504, 41216, 63456, 42272, 62752, 6112, 53504, 4736, 27744, 0]],
        '鹸' : [12, 13, 0, -2, 12, [0, 17344, 29728, 20448, 63616, 44000, 55968, 43680, 56288, 43136, 43328, 65056, 0]],
        '元' : [12, 13, 0, -2, 12, [0, 32704, 0, 0, 0, 65504, 4608, 4608, 4608, 4640, 8736, 50144, 0]],
        '原' : [12, 13, 0, -2, 12, [0, 32736, 16896, 24512, 20544, 24512, 20544, 20544, 24512, 33280, 37440, 9760, 0]],
        '厳' : [12, 13, 0, -2, 12, [512, 4672, 32736, 18688, 32736, 21024, 24224, 21152, 24128, 37440, 40608, 13088, 0]],
        '幻' : [12, 13, 0, -2, 12, [0, 20448, 16416, 32800, 36896, 20512, 8224, 8224, 16416, 18464, 34880, 64384, 0]],
        '弦' : [12, 13, 0, -2, 12, [256, 61696, 6112, 4224, 61696, 33312, 33856, 62080, 4352, 4640, 9248, 51168, 0]],
        '減' : [12, 13, 0, -2, 12, [128, 160, 40928, 20608, 8064, 37024, 22432, 5440, 22336, 20544, 41376, 42528, 0]],
        '源' : [12, 13, 0, -2, 12, [0, 4064, 34944, 19424, 2592, 35808, 18976, 2592, 21472, 20608, 33952, 35232, 0]],
        '玄' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 2176, 4352, 8704, 5248, 2176, 4224, 8512, 32320, 0]],
        '現' : [12, 13, 0, -2, 12, [0, 2016, 62496, 10208, 9248, 10208, 62496, 9248, 10208, 12928, 50336, 6368, 0]],
        '絃' : [12, 13, 0, -2, 12, [256, 8448, 18400, 36992, 61696, 8736, 17472, 62080, 8448, 45600, 42016, 10208, 0]],
        '舷' : [12, 13, 0, -2, 12, [8448, 16640, 63456, 36992, 53504, 37408, 62528, 37504, 53504, 37408, 37920, 47072, 0]],
        '言' : [12, 13, 0, -2, 12, [0, 16256, 0, 65504, 0, 16256, 0, 0, 16256, 8320, 8320, 16256, 0]],
        '諺' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57920, 576, 61408, 2176, 2848, 59584, 45824, 41056, 59264, 0]],
        '限' : [12, 13, 0, -2, 12, [0, 63456, 37920, 42976, 50208, 42016, 38880, 38144, 38176, 58528, 33856, 34592, 0]],
        '乎' : [12, 13, 0, -2, 12, [0, 960, 31744, 1024, 17472, 9344, 1024, 65504, 1024, 1024, 1024, 7168, 0]],
        '個' : [12, 13, 0, -2, 12, [0, 12256, 10528, 18720, 20448, 51488, 19360, 19104, 19104, 19360, 18464, 20448, 0]],
        '古' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 1024, 1024, 1024, 16256, 8320, 8320, 8320, 16256, 0]],
        '呼' : [12, 13, 0, -2, 12, [0, 224, 61184, 41216, 43296, 42304, 41216, 49120, 41216, 41216, 57600, 1792, 0]],
        '固' : [12, 13, 0, -2, 12, [0, 65504, 33824, 33824, 49056, 33824, 40736, 37152, 37152, 40736, 32800, 65504, 0]],
        '姑' : [12, 13, 0, -2, 12, [128, 16512, 16512, 63456, 20608, 20608, 20608, 38880, 58400, 13344, 17440, 18400, 0]],
        '孤' : [12, 13, 0, -2, 12, [0, 61408, 10880, 10880, 19072, 19072, 27200, 51776, 19008, 19104, 22432, 53280, 0]],
        '己' : [12, 13, 0, -2, 12, [0, 32640, 128, 128, 128, 32640, 16384, 16384, 16448, 16448, 16448, 32704, 0]],
        '庫' : [12, 13, 0, -2, 12, [512, 512, 32736, 16640, 24544, 20768, 24544, 20768, 24544, 16640, 49120, 33024, 0]],
        '弧' : [12, 13, 0, -2, 12, [0, 61408, 10880, 10880, 60032, 35456, 35392, 59968, 10816, 10912, 14240, 53280, 0]],
        '戸' : [12, 13, 0, -2, 12, [0, 65504, 0, 0, 32704, 16448, 16448, 32704, 16384, 16384, 32768, 32768, 0]],
        '故' : [12, 13, 0, -2, 12, [8704, 8704, 9184, 64064, 9280, 9280, 8512, 63808, 34944, 34944, 35136, 65056, 0]],
        '枯' : [12, 13, 0, -2, 12, [128, 8320, 8320, 63456, 8320, 8320, 24704, 26592, 46112, 42016, 9248, 10208, 0]],
        '湖' : [12, 13, 0, -2, 12, [2048, 2528, 35104, 23840, 2528, 35104, 23840, 5600, 21792, 21792, 40480, 33376, 0]],
        '狐' : [12, 13, 0, -2, 12, [0, 12256, 43648, 19072, 19072, 43648, 10816, 27200, 43584, 10912, 14240, 53280, 0]],
        '糊' : [12, 13, 0, -2, 12, [1024, 9696, 42272, 48928, 9696, 62752, 12064, 27104, 26912, 43296, 44832, 8800, 0]],
        '袴' : [12, 13, 0, -2, 12, [256, 16640, 20448, 62016, 5152, 12256, 16384, 53216, 25088, 22496, 17440, 16576, 0]],
        '股' : [12, 13, 0, -2, 12, [0, 62400, 37440, 37440, 62016, 37984, 36864, 63456, 37408, 37440, 37248, 46688, 0]],
        '胡' : [12, 13, 0, -2, 12, [8192, 9184, 8736, 64032, 9184, 8736, 8736, 64480, 35360, 35360, 35872, 63712, 0]],
        '菰' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 61408, 10816, 10816, 19008, 59968, 19104, 22432, 53280, 0]],
        '虎' : [12, 13, 0, -2, 12, [1024, 1984, 1024, 32736, 17440, 24448, 17440, 18400, 18688, 35072, 35104, 12512, 0]],
        '誇' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57920, 1056, 61408, 0, 4064, 57856, 42976, 42016, 57536, 0]],
        '跨' : [12, 13, 0, -2, 12, [256, 61696, 40928, 37440, 62496, 12256, 8192, 16352, 41472, 42976, 42016, 61632, 0]],
        '鈷' : [12, 13, 0, -2, 12, [128, 12416, 18560, 34784, 63616, 8320, 63616, 10208, 46112, 42016, 9248, 63456, 0]],
        '雇' : [12, 13, 0, -2, 12, [0, 65504, 0, 32704, 16448, 32704, 18688, 24544, 28928, 24512, 37120, 40928, 0]],
        '顧' : [12, 13, 0, -2, 12, [0, 64480, 128, 64480, 35360, 64480, 37408, 64480, 37408, 64480, 37440, 64544, 0]],
        '鼓' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8320, 63616, 992, 63520, 35360, 64064, 20800, 20864, 65120, 0]],
        '五' : [12, 13, 0, -2, 12, [0, 32704, 1024, 1024, 1024, 16256, 2176, 2176, 4224, 4224, 4224, 65504, 0]],
        '互' : [12, 13, 0, -2, 12, [0, 65504, 1024, 2048, 3968, 4224, 4352, 8448, 15872, 512, 1024, 65504, 0]],
        '伍' : [12, 13, 0, -2, 12, [0, 12256, 8448, 16640, 16640, 53184, 16960, 16960, 17472, 17472, 17472, 24544, 0]],
        '午' : [12, 13, 0, -2, 12, [16384, 16384, 32704, 33792, 33792, 1024, 1024, 65504, 1024, 1024, 1024, 1024, 0]],
        '呉' : [12, 13, 0, -2, 12, [0, 8064, 20608, 20608, 24448, 16384, 32640, 128, 65504, 0, 12672, 49248, 0]],
        '吾' : [12, 13, 0, -2, 12, [0, 65504, 1024, 16256, 2176, 2176, 65504, 0, 16256, 8320, 8320, 16256, 0]],
        '娯' : [12, 13, 0, -2, 12, [0, 17344, 19008, 64064, 19392, 18432, 20448, 36896, 61408, 8192, 17472, 18464, 0]],
        '後' : [12, 13, 0, -2, 12, [0, 8832, 17536, 35072, 1344, 8768, 20448, 50176, 20416, 21568, 17280, 23648, 0]],
        '御' : [12, 13, 0, -2, 12, [4096, 4320, 24480, 38048, 1184, 16288, 17568, 50848, 21664, 21728, 22400, 22656, 0]],
        '悟' : [12, 13, 0, -2, 12, [0, 12256, 8448, 10208, 41504, 41504, 45024, 40960, 42976, 9248, 9248, 10208, 0]],
        '梧' : [12, 13, 0, -2, 12, [0, 12256, 8448, 63456, 8736, 8736, 28640, 24576, 47072, 42016, 9248, 10208, 0]],
        '檎' : [12, 13, 0, -2, 12, [0, 10176, 10528, 65504, 10528, 10912, 28640, 24832, 45024, 43552, 11168, 10336, 0]],
        '瑚' : [12, 13, 0, -2, 12, [1024, 1504, 58656, 24352, 17888, 17696, 61216, 18912, 18720, 18720, 61216, 608, 0]],
        '碁' : [12, 13, 0, -2, 12, [4352, 4352, 32704, 4352, 7936, 4352, 65504, 10368, 57280, 12448, 20608, 8064, 0]],
        '語' : [12, 13, 0, -2, 12, [0, 63456, 256, 63456, 544, 61984, 2016, 0, 63456, 37920, 37920, 63456, 0]],
        '誤' : [12, 13, 0, -2, 12, [0, 58304, 2624, 59968, 3008, 59392, 4064, 32, 61408, 40960, 42048, 59424, 0]],
        '護' : [12, 13, 0, -2, 12, [1152, 58496, 4064, 57984, 2016, 60544, 2016, 1152, 61408, 42048, 41856, 60512, 0]],
        '醐' : [12, 13, 0, -2, 12, [1024, 62944, 17696, 20256, 62944, 38176, 57120, 45536, 37152, 61728, 40736, 62048, 0]],
        '乞' : [12, 13, 0, -2, 12, [16384, 16384, 32736, 16384, 40832, 32896, 768, 3072, 12320, 16416, 16416, 16320, 0]],
        '鯉' : [12, 13, 0, -2, 12, [0, 29664, 37536, 9184, 64160, 43680, 64480, 43136, 64480, 128, 43136, 44000, 0]],
        '交' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 8320, 16448, 4352, 2560, 1024, 2560, 12672, 49248, 0]],
        '佼' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16384, 17472, 51232, 16384, 17472, 17024, 16768, 17984, 22560, 0]],
        '侯' : [12, 13, 0, -2, 12, [0, 12224, 8256, 16448, 24544, 50176, 18400, 18688, 24544, 17024, 17472, 22560, 0]],
        '候' : [12, 13, 0, -2, 12, [0, 10176, 8256, 20544, 24544, 53760, 21472, 21632, 22496, 20608, 20800, 17952, 0]],
        '倖' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16640, 24544, 50240, 17472, 24544, 16640, 20448, 16640, 16640, 0]],
        '光' : [12, 13, 0, -2, 12, [1024, 1024, 17472, 9344, 1024, 1024, 65504, 4608, 4608, 4640, 8736, 50144, 0]],
        '公' : [12, 13, 0, -2, 12, [0, 1920, 8320, 8320, 16448, 16448, 34848, 34848, 4224, 4224, 9024, 15424, 0]],
        '功' : [12, 13, 0, -2, 12, [256, 256, 61696, 12256, 8480, 8480, 8736, 8736, 9248, 13344, 49216, 384, 0]],
        '効' : [12, 13, 0, -2, 12, [8448, 8448, 63744, 2016, 20768, 35104, 288, 21024, 8736, 12320, 18496, 33152, 0]],
        '勾' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16416, 18464, 34848, 36896, 4640, 8736, 15904, 64, 384, 0]],
        '厚' : [12, 13, 0, -2, 12, [0, 32736, 16384, 20448, 18464, 20448, 18464, 20448, 16448, 40928, 32896, 1920, 0]],
        '口' : [12, 13, 0, -2, 12, [0, 0, 32736, 16416, 16416, 16416, 16416, 16416, 16416, 16416, 32736, 16416, 0]],
        '向' : [12, 13, 0, -2, 12, [512, 1024, 2048, 65504, 32800, 32800, 40736, 37152, 37152, 40736, 32800, 32992, 0]],
        '后' : [12, 13, 0, -2, 12, [0, 448, 32256, 16384, 16384, 32736, 16384, 16384, 24512, 36928, 36928, 8128, 0]],
        '喉' : [12, 13, 0, -2, 12, [0, 3008, 59456, 43072, 45024, 47616, 44000, 44160, 44000, 43136, 59712, 3616, 0]],
        '坑' : [12, 13, 0, -2, 12, [256, 16640, 20448, 16384, 57344, 18368, 17472, 17472, 17472, 17472, 59456, 2144, 0]],
        '垢' : [12, 13, 0, -2, 12, [0, 16608, 20224, 18432, 59392, 20448, 18432, 20480, 22496, 17440, 58400, 2016, 0]],
        '好' : [12, 13, 0, -2, 12, [0, 18400, 16416, 61504, 20608, 20608, 22496, 36992, 57472, 14464, 16512, 17280, 0]],
        '孔' : [12, 13, 0, -2, 12, [0, 63744, 2304, 2304, 4352, 8448, 8448, 64768, 8480, 8480, 8480, 24800, 0]],
        '孝' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1152, 1280, 65504, 2048, 14208, 49408, 16352, 512, 3584, 0]],
        '宏' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 34848, 34848, 32640, 4608, 4672, 9280, 9280, 18592, 20256, 0]],
        '工' : [12, 13, 0, -2, 12, [0, 0, 32704, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 65504, 0, 0]],
        '巧' : [12, 13, 0, -2, 12, [0, 4064, 61696, 8448, 8448, 8672, 8736, 8736, 9248, 13344, 49216, 384, 0]],
        '巷' : [12, 13, 0, -2, 12, [4352, 4352, 32704, 4352, 65504, 4352, 8064, 8544, 24320, 36864, 4160, 8128, 0]],
        '幸' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 65504, 4352, 4352, 65504, 1024, 32704, 1024, 1024, 0]],
        '広' : [12, 13, 0, -2, 12, [1024, 1024, 32736, 16384, 17408, 17408, 17408, 18496, 18496, 34880, 37280, 40480, 0]],
        '庚' : [12, 13, 0, -2, 12, [512, 512, 32736, 16896, 24544, 16928, 32736, 16928, 24544, 34048, 34944, 12384, 0]],
        '康' : [12, 13, 0, -2, 12, [512, 512, 32736, 16928, 32736, 16928, 24544, 25120, 21312, 35456, 37440, 9760, 0]],
        '弘' : [12, 13, 0, -2, 12, [0, 61696, 4352, 4352, 61696, 33280, 33280, 62016, 5184, 5184, 10400, 53024, 0]],
        '恒' : [12, 13, 0, -2, 12, [0, 12256, 8192, 10208, 42016, 46112, 42976, 42016, 42016, 10208, 8192, 12256, 0]],
        '慌' : [12, 13, 0, -2, 12, [1088, 9280, 12256, 9280, 41216, 45024, 41984, 42976, 40960, 9536, 9536, 10592, 0]],
        '抗' : [12, 13, 0, -2, 12, [256, 16640, 20448, 57344, 16384, 18368, 17472, 25664, 50240, 17472, 18496, 51296, 0]],
        '拘' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 59424, 18464, 18208, 17696, 25888, 50976, 16416, 16448, 49536, 0]],
        '控' : [12, 13, 0, -2, 12, [256, 16640, 24544, 62112, 21120, 17056, 19680, 16384, 61376, 16640, 16640, 57312, 0]],
        '攻' : [12, 13, 0, -2, 12, [1024, 1024, 2016, 62528, 9280, 10304, 10560, 8512, 8320, 12416, 49984, 3104, 0]],
        '昂' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 16256, 0, 62432, 33312, 36384, 62176, 512, 0]],
        '晃' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 16256, 17472, 9344, 65504, 2560, 12832, 49632, 0]],
        '更' : [12, 13, 0, -2, 12, [0, 65504, 1024, 16320, 9280, 16320, 9280, 16320, 2048, 6144, 9728, 49632, 0]],
        '杭' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61440, 8192, 10176, 25664, 25664, 42048, 42048, 10304, 10336, 0]],
        '校' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61440, 9280, 10272, 24576, 25664, 41600, 41344, 8768, 11296, 0]],
        '梗' : [12, 13, 0, -2, 12, [0, 12256, 8448, 65504, 10528, 12256, 26912, 28640, 41216, 42240, 8832, 11360, 0]],
        '構' : [12, 13, 0, -2, 12, [1152, 9344, 12256, 62592, 12256, 9344, 28640, 26912, 45024, 43296, 16352, 10272, 0]],
        '江' : [12, 13, 0, -2, 12, [0, 8160, 33024, 16640, 256, 33024, 16640, 256, 256, 8448, 16640, 40928, 0]],
        '洪' : [12, 13, 0, -2, 12, [1152, 1152, 33920, 24544, 1152, 33920, 17536, 8160, 16384, 17536, 34880, 36896, 0]],
        '浩' : [12, 13, 0, -2, 12, [128, 1152, 33920, 18400, 2176, 34944, 18400, 0, 2016, 9248, 17440, 34784, 0]],
        '港' : [12, 13, 0, -2, 12, [1152, 1152, 40928, 17536, 8160, 33856, 20448, 4160, 4032, 10240, 18464, 36832, 0]],
        '溝' : [12, 13, 0, -2, 12, [1152, 1152, 40928, 17536, 8160, 33920, 24544, 2336, 20448, 18720, 40928, 34848, 0]],
        '甲' : [12, 13, 0, -2, 12, [0, 32736, 17440, 17440, 32736, 17440, 17440, 17440, 32736, 1024, 1024, 1024, 0]],
        '皇' : [12, 13, 0, -2, 12, [1024, 2048, 16256, 8320, 16256, 8320, 8320, 32704, 1024, 16256, 1024, 65504, 0]],
        '硬' : [12, 13, 0, -2, 12, [0, 4064, 61696, 10208, 9504, 10208, 29984, 55264, 20736, 21760, 21120, 31840, 0]],
        '稿' : [12, 13, 0, -2, 12, [256, 6400, 61408, 9280, 10176, 61440, 12256, 26656, 27552, 43680, 43936, 10336, 0]],
        '糠' : [12, 13, 0, -2, 12, [128, 8320, 45024, 43168, 12256, 59552, 11232, 26752, 30112, 41664, 44192, 8576, 0]],
        '紅' : [12, 13, 0, -2, 12, [0, 10208, 16512, 34944, 61568, 8320, 18560, 63616, 8320, 43136, 43136, 10208, 0]],
        '紘' : [12, 13, 0, -2, 12, [256, 8448, 20448, 33280, 61952, 8832, 17536, 62592, 9472, 43296, 43552, 9184, 0]],
        '絞' : [12, 13, 0, -2, 12, [256, 8448, 20448, 32768, 58432, 10272, 16384, 62528, 8832, 41344, 41536, 11296, 0]],
        '綱' : [12, 13, 0, -2, 12, [0, 12256, 18592, 36000, 59680, 12256, 18976, 64032, 10912, 44960, 43040, 10336, 0]],
        '耕' : [12, 13, 0, -2, 12, [8256, 8768, 64064, 10208, 64064, 8768, 8768, 65504, 8768, 29248, 42048, 9280, 0]],
        '考' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1152, 1280, 65504, 2048, 16320, 53248, 8128, 64, 896, 0]],
        '肯' : [12, 13, 0, -2, 12, [1024, 10176, 9216, 9216, 65504, 8320, 16256, 8320, 16256, 8320, 8320, 8576, 0]],
        '肱' : [12, 13, 0, -2, 12, [256, 61696, 40928, 37376, 61952, 37504, 38016, 62592, 39168, 37152, 37408, 46048, 0]],
        '腔' : [12, 13, 0, -2, 12, [256, 61696, 40928, 37536, 62080, 37536, 40160, 61440, 38848, 37120, 37120, 49120, 0]],
        '膏' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 65504, 37152, 49056, 8320, 16256, 8320, 16256, 8320, 0]],
        '航' : [12, 13, 0, -2, 12, [8448, 16640, 63456, 36864, 53248, 37824, 64064, 37440, 53824, 37440, 37952, 46176, 0]],
        '荒' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 32704, 4096, 8064, 0, 9344, 9344, 50400, 0]],
        '行' : [12, 13, 0, -2, 12, [0, 10176, 16384, 32768, 0, 12256, 16512, 49280, 16512, 16512, 16512, 17280, 0]],
        '衡' : [12, 13, 0, -2, 12, [0, 12000, 20992, 33792, 8160, 13600, 24352, 54560, 24480, 17440, 18976, 20832, 0]],
        '講' : [12, 13, 0, -2, 12, [1152, 58496, 4064, 58496, 4064, 58496, 4064, 2336, 61408, 43296, 49120, 59424, 0]],
        '貢' : [12, 13, 0, -2, 12, [0, 32704, 1024, 1024, 65504, 8320, 16256, 8320, 16256, 8320, 16256, 49248, 0]],
        '購' : [12, 13, 0, -2, 12, [1152, 58496, 45024, 58496, 45024, 58496, 45024, 43296, 61408, 2336, 49120, 43040, 0]],
        '郊' : [12, 13, 0, -2, 12, [4096, 4576, 65312, 288, 18752, 34176, 320, 16672, 10528, 4384, 10688, 50432, 0]],
        '酵' : [12, 13, 0, -2, 12, [256, 63744, 10208, 8480, 63808, 45024, 43264, 56288, 35904, 64480, 34944, 63872, 0]],
        '鉱' : [12, 13, 0, -2, 12, [256, 12544, 18400, 33792, 62592, 9344, 62592, 9472, 43264, 43296, 8736, 64480, 0]],
        '砿' : [12, 13, 0, -2, 12, [256, 256, 63456, 9216, 9344, 9344, 29824, 54528, 21760, 22816, 21024, 29664, 0]],
        '鋼' : [12, 13, 0, -2, 12, [0, 12256, 18464, 35488, 60064, 12256, 64032, 10784, 43680, 44960, 10272, 63584, 0]],
        '閤' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 37152, 44704, 32800, 40736, 37152, 40800, 0]],
        '降' : [12, 13, 0, -2, 12, [0, 62432, 39968, 41536, 50048, 44128, 36992, 38880, 38016, 58496, 40928, 32896, 0]],
        '項' : [12, 13, 0, -2, 12, [0, 8160, 57600, 20448, 18464, 20448, 18464, 20448, 18464, 61408, 1088, 6176, 0]],
        '香' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 5376, 9344, 50240, 16288, 8320, 16256, 8320, 16256, 0]],
        '高' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 7936, 0, 65504, 32800, 40736, 37152, 40736, 32864, 0]],
        '鴻' : [12, 13, 0, -2, 12, [256, 512, 34752, 31808, 6080, 37952, 22496, 15360, 2016, 8224, 21792, 38240, 0]],
        '剛' : [12, 13, 0, -2, 12, [0, 65056, 33440, 43680, 43680, 65184, 41632, 41632, 43680, 64032, 33312, 34400, 0]],
        '劫' : [12, 13, 0, -2, 12, [8448, 8448, 63744, 10208, 8480, 8480, 63776, 8736, 8736, 18464, 34880, 63872, 0]],
        '号' : [12, 13, 0, -2, 12, [0, 16256, 8320, 8320, 16256, 0, 65504, 4096, 16320, 8256, 128, 1792, 0]],
        '合' : [12, 13, 0, -2, 12, [0, 3584, 4352, 8320, 16448, 49056, 0, 0, 16256, 8320, 8320, 16256, 0]],
        '壕' : [12, 13, 0, -2, 12, [256, 16640, 24544, 17536, 65504, 20512, 20448, 17664, 22944, 17088, 60576, 768, 0]],
        '拷' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61728, 8512, 12256, 8704, 14304, 60416, 14304, 8224, 24768, 0]],
        '濠' : [12, 13, 0, -2, 12, [512, 512, 40928, 17536, 8160, 36896, 20448, 1280, 22944, 17088, 40096, 33536, 0]],
        '豪' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 65504, 32800, 16256, 5152, 26176, 7040, 57952, 3072, 0]],
        '轟' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 9344, 16256, 9344, 65504, 19008, 31680, 19008, 65504, 8320, 0]],
        '麹' : [12, 13, 0, -2, 12, [9216, 9216, 65504, 9248, 64160, 8736, 65504, 21280, 22176, 41568, 12288, 53216, 0]],
        '克' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 16256, 8320, 8320, 16256, 4608, 4640, 8736, 50144, 0]],
        '刻' : [12, 13, 0, -2, 12, [4096, 4128, 64800, 4384, 8480, 17696, 10528, 4384, 25888, 2080, 13344, 49760, 0]],
        '告' : [12, 13, 0, -2, 12, [512, 8704, 8704, 16320, 16896, 33280, 65504, 0, 16320, 8256, 8256, 16320, 0]],
        '国' : [12, 13, 0, -2, 12, [0, 65504, 32800, 49056, 33824, 33824, 40736, 33952, 33824, 49056, 32800, 65504, 0]],
        '穀' : [12, 13, 0, -2, 12, [8192, 9152, 64064, 8768, 64064, 35936, 61440, 10208, 64032, 8768, 27008, 42592, 0]],
        '酷' : [12, 13, 0, -2, 12, [128, 64128, 8832, 9184, 64640, 43136, 45024, 55296, 35808, 64032, 35360, 64480, 0]],
        '鵠' : [12, 13, 0, -2, 12, [8448, 41472, 42944, 62528, 42944, 9280, 63456, 1024, 63456, 36896, 38176, 62816, 0]],
        '黒' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 17472, 32704, 1024, 65504, 0, 21056, 35104, 0]],
        '獄' : [12, 13, 0, -2, 12, [128, 11936, 41120, 24192, 16864, 44672, 8320, 24704, 44864, 10560, 10528, 53024, 0]],
        '漉' : [12, 13, 0, -2, 12, [512, 512, 40928, 21120, 8160, 37536, 24544, 5248, 22240, 21632, 42400, 44640, 0]],
        '腰' : [12, 13, 0, -2, 12, [0, 61408, 41600, 45024, 60064, 43680, 45024, 57856, 45024, 42048, 42880, 47200, 0]],
        '甑' : [12, 13, 0, -2, 12, [34816, 20448, 4352, 63744, 43968, 64064, 43584, 64320, 38080, 62528, 38880, 63520, 0]],
        '忽' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 18720, 35104, 4640, 9408, 0, 1024, 4672, 20512, 36736, 0]],
        '惚' : [12, 13, 0, -2, 12, [1024, 9216, 12256, 12960, 42144, 43296, 45600, 41024, 8704, 2368, 10272, 18304, 0]],
        '骨' : [12, 13, 0, -2, 12, [0, 16256, 8320, 10112, 9344, 65504, 41120, 49056, 8320, 16256, 8320, 8576, 0]],
        '狛' : [12, 13, 0, -2, 12, [256, 8448, 41472, 20448, 18464, 43040, 10272, 28640, 43040, 10272, 10272, 53216, 0]],
        '込' : [12, 13, 0, -2, 12, [0, 3840, 16640, 8448, 256, 640, 57984, 9280, 10272, 8192, 20480, 36832, 0]],
        '此' : [12, 13, 0, -2, 12, [512, 8704, 8704, 8704, 41568, 48000, 41472, 41472, 41472, 41504, 47648, 50144, 0]],
        '頃' : [12, 13, 0, -2, 12, [0, 4064, 33024, 34784, 62496, 34784, 33824, 34784, 37920, 30688, 576, 3104, 0]],
        '今' : [12, 13, 0, -2, 12, [0, 3840, 4224, 8256, 24480, 32768, 0, 16320, 64, 64, 384, 3584, 0]],
        '困' : [12, 13, 0, -2, 12, [0, 65504, 33824, 33824, 49056, 33824, 36384, 38176, 42144, 33824, 32800, 65504, 0]],
        '坤' : [12, 13, 0, -2, 12, [256, 16640, 20448, 18720, 59680, 20448, 18720, 18720, 20448, 16640, 57600, 256, 0]],
        '墾' : [12, 13, 0, -2, 12, [0, 64480, 21024, 65504, 21024, 44000, 19008, 45984, 1024, 32704, 1024, 65504, 0]],
        '婚' : [12, 13, 0, -2, 12, [0, 18400, 17536, 63456, 21632, 22112, 20480, 38880, 58400, 14304, 17440, 18400, 0]],
        '恨' : [12, 13, 0, -2, 12, [0, 10208, 9248, 10208, 46112, 44064, 42976, 42240, 42272, 9376, 9280, 10016, 0]],
        '懇' : [12, 13, 0, -2, 12, [0, 64480, 21024, 65504, 21024, 44000, 19008, 45984, 1024, 4672, 20512, 36736, 0]],
        '昏' : [12, 13, 0, -2, 12, [0, 32704, 16896, 32736, 16896, 22944, 57440, 16320, 8256, 16320, 8256, 16320, 0]],
        '昆' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 32704, 16896, 16992, 31616, 16896, 16928, 64480, 0]],
        '根' : [12, 13, 0, -2, 12, [0, 10208, 9248, 63456, 9248, 9248, 26592, 25856, 46368, 42144, 9280, 10016, 0]],
        '梱' : [12, 13, 0, -2, 12, [0, 12256, 10528, 63776, 12256, 10528, 26912, 27552, 44384, 43296, 10528, 12256, 0]],
        '混' : [12, 13, 0, -2, 12, [0, 4064, 34848, 20448, 2080, 36832, 18560, 2176, 3808, 10368, 18592, 36576, 0]],
        '痕' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 57280, 20544, 24512, 53312, 24512, 21088, 37248, 40032, 0]],
        '紺' : [12, 13, 0, -2, 12, [512, 8736, 16928, 38880, 61984, 8736, 18976, 64480, 8736, 43552, 43552, 9184, 0]],
        '艮' : [12, 13, 0, -2, 12, [0, 16320, 8256, 16320, 8256, 8256, 16320, 8704, 8736, 8512, 11392, 61536, 0]],
        '魂' : [12, 13, 0, -2, 12, [256, 512, 61408, 2336, 4064, 63776, 18720, 20448, 34112, 42304, 59680, 2528, 0]],
        '些' : [12, 13, 0, -2, 12, [512, 8800, 48000, 41472, 41504, 48672, 58336, 0, 16256, 0, 0, 65504, 0]],
        '佐' : [12, 13, 0, -2, 12, [512, 8704, 16352, 17408, 17408, 50176, 20448, 18560, 20608, 20608, 16512, 20448, 0]],
        '叉' : [12, 13, 0, -2, 12, [0, 32704, 64, 2112, 1088, 640, 128, 4352, 2560, 1536, 6528, 57440, 0]],
        '唆' : [12, 13, 0, -2, 12, [512, 576, 58432, 49120, 41600, 41632, 44128, 41920, 44096, 42112, 58240, 7264, 0]],
        '嵯' : [12, 13, 0, -2, 12, [1088, 9344, 12256, 8448, 42976, 41216, 45024, 41472, 42976, 63616, 128, 4064, 0]],
        '左' : [12, 13, 0, -2, 12, [2048, 2048, 65504, 4096, 4096, 4096, 8160, 8448, 8448, 16640, 16640, 8160, 0]],
        '差' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 32704, 1024, 65504, 4096, 16320, 49664, 512, 32736, 0]],
        '査' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 3328, 13440, 50272, 16256, 8320, 16256, 8320, 8320, 65504, 0]],
        '沙' : [12, 13, 0, -2, 12, [256, 256, 33024, 18752, 2336, 37120, 16640, 768, 32, 8256, 16768, 36352, 0]],
        '瑳' : [12, 13, 0, -2, 12, [1088, 1152, 61408, 16640, 18400, 57600, 20448, 16896, 18400, 18560, 57472, 4064, 0]],
        '砂' : [12, 13, 0, -2, 12, [256, 256, 61696, 9536, 9504, 10496, 28928, 54016, 20512, 20544, 20864, 30208, 0]],
        '詐' : [12, 13, 0, -2, 12, [1024, 58368, 2016, 58624, 2304, 59872, 256, 256, 57824, 41216, 41216, 57600, 0]],
        '鎖' : [12, 13, 0, -2, 12, [256, 13600, 17728, 33024, 63456, 9248, 63456, 9248, 47072, 40960, 8768, 62496, 0]],
        '裟' : [12, 13, 0, -2, 12, [256, 17728, 10528, 33536, 18656, 5888, 8704, 65504, 5184, 57920, 8576, 31840, 0]],
        '坐' : [12, 13, 0, -2, 12, [1024, 1024, 9344, 9344, 13504, 19744, 34336, 1024, 32704, 1024, 1024, 65504, 0]],
        '座' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 18560, 21824, 25120, 16896, 24544, 33280, 33280, 16352, 0]],
        '挫' : [12, 13, 0, -2, 12, [256, 16640, 18752, 59712, 18752, 21920, 21280, 16640, 61408, 16640, 16640, 57312, 0]],
        '債' : [12, 13, 0, -2, 12, [256, 8448, 16352, 16640, 20448, 49408, 24544, 18496, 20416, 18496, 20416, 28768, 0]],
        '催' : [12, 13, 0, -2, 12, [256, 10528, 10528, 20448, 17472, 53216, 22656, 28640, 18560, 20448, 18560, 20448, 0]],
        '再' : [12, 13, 0, -2, 12, [0, 65504, 1024, 32704, 17472, 32704, 17472, 17472, 65504, 16448, 16448, 16576, 0]],
        '最' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 65504, 18432, 32736, 19008, 19008, 31104, 52832, 0]],
        '哉' : [12, 13, 0, -2, 12, [8448, 8512, 63776, 8448, 10208, 63744, 320, 320, 63616, 34944, 35136, 65056, 0]],
        '塞' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 49056, 4352, 65504, 9344, 24384, 33824, 1024, 65504, 0]],
        '妻' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1152, 32736, 1152, 16256, 2048, 65504, 4224, 7936, 24768, 0]],
        '宰' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33824, 32704, 4352, 4352, 65504, 1024, 32704, 1024, 1024, 0]],
        '彩' : [12, 13, 0, -2, 12, [0, 64032, 8768, 38016, 17408, 32, 4160, 65152, 4096, 13344, 53824, 4224, 0]],
        '才' : [12, 13, 0, -2, 12, [512, 512, 512, 65504, 512, 640, 768, 1536, 6656, 57856, 512, 3584, 0]],
        '採' : [12, 13, 0, -2, 12, [0, 20384, 16928, 59712, 17472, 16384, 16640, 28640, 49920, 17728, 22816, 49408, 0]],
        '栽' : [12, 13, 0, -2, 12, [4352, 4416, 32032, 4352, 65504, 4224, 4256, 65184, 5184, 12864, 53664, 5664, 0]],
        '歳' : [12, 13, 0, -2, 12, [1024, 10176, 9216, 65504, 128, 32736, 16544, 32416, 18496, 43584, 43680, 39200, 0]],
        '済' : [12, 13, 0, -2, 12, [512, 512, 40928, 18496, 1152, 34688, 22624, 4032, 18496, 20416, 36928, 36928, 0]],
        '災' : [12, 13, 0, -2, 12, [0, 4672, 9344, 18688, 9344, 4672, 1024, 9280, 17536, 2560, 12672, 49248, 0]],
        '采' : [12, 13, 0, -2, 12, [0, 32704, 2048, 17472, 8320, 256, 1024, 65504, 3328, 13440, 50272, 1024, 0]],
        '犀' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 16896, 31456, 16896, 31456, 20352, 37376, 49120, 512, 0]],
        '砕' : [12, 13, 0, -2, 12, [512, 512, 63424, 8768, 8768, 9312, 29952, 53504, 24544, 20736, 20736, 28928, 0]],
        '砦' : [12, 13, 0, -2, 12, [512, 8704, 48096, 41472, 48672, 57824, 0, 65504, 4096, 16320, 57408, 16320, 0]],
        '祭' : [12, 13, 0, -2, 12, [0, 31712, 18464, 43328, 4224, 8256, 57248, 0, 32704, 1024, 17472, 35872, 0]],
        '斎' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 8320, 7936, 8320, 24416, 41024, 16320, 9280, 21824, 21824, 0]],
        '細' : [12, 13, 0, -2, 12, [0, 10208, 17696, 38176, 62752, 9504, 18400, 64800, 9504, 46368, 42272, 10208, 0]],
        '菜' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 18496, 9344, 256, 65504, 3328, 13440, 50272, 0]],
        '裁' : [12, 13, 0, -2, 12, [4352, 4416, 32032, 4352, 65504, 4224, 65184, 4256, 10816, 59968, 9376, 29472, 0]],
        '載' : [12, 13, 0, -2, 12, [4352, 4416, 32032, 4352, 65504, 4224, 65184, 21152, 32320, 21056, 65184, 4896, 0]],
        '際' : [12, 13, 0, -2, 12, [0, 61152, 45600, 43840, 51840, 42048, 43936, 40960, 45024, 49408, 35136, 37664, 0]],
        '剤' : [12, 13, 0, -2, 12, [4096, 4128, 65312, 17696, 18720, 14624, 50976, 32032, 17696, 32032, 33824, 33888, 0]],
        '在' : [12, 13, 0, -2, 12, [2048, 2048, 65504, 4352, 4352, 8448, 12256, 24832, 41216, 8448, 8448, 12256, 0]],
        '材' : [12, 13, 0, -2, 12, [8256, 8256, 8256, 63456, 8256, 8384, 24896, 25152, 46144, 41024, 8256, 8640, 0]],
        '罪' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 2560, 64480, 2560, 31680, 2560, 64480, 4608, 25088, 0]],
        '財' : [12, 13, 0, -2, 12, [64, 63552, 34880, 65504, 34880, 63552, 35008, 35136, 64064, 1088, 20544, 35264, 0]],
        '冴' : [12, 13, 0, -2, 12, [0, 8160, 33856, 17472, 1088, 8160, 64, 192, 17216, 23616, 32832, 33216, 0]],
        '坂' : [12, 13, 0, -2, 12, [0, 10208, 9216, 9216, 63424, 9280, 9280, 9536, 9536, 14464, 49472, 3616, 0]],
        '阪' : [12, 13, 0, -2, 12, [0, 59360, 41984, 41984, 51136, 42048, 42048, 43328, 43328, 49280, 33600, 35872, 0]],
        '堺' : [12, 13, 0, -2, 12, [0, 20448, 18720, 20448, 63776, 18720, 20448, 17472, 19040, 21056, 58432, 1088, 0]],
        '榊' : [12, 13, 0, -2, 12, [2176, 10368, 11232, 65184, 8864, 10208, 27296, 31392, 44000, 44160, 10368, 10368, 0]],
        '肴' : [12, 13, 0, -2, 12, [0, 14528, 1792, 6368, 59392, 8160, 4160, 16320, 53312, 8128, 4160, 4288, 0]],
        '咲' : [12, 13, 0, -2, 12, [2112, 1088, 57472, 45024, 41216, 41216, 49120, 41216, 41216, 57984, 1088, 6176, 0]],
        '崎' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8832, 42048, 43040, 45024, 40992, 42784, 42272, 63264, 96, 0]],
        '埼' : [12, 13, 0, -2, 12, [256, 16640, 20448, 17024, 58432, 18464, 20448, 16416, 20256, 18720, 61216, 96, 0]],
        '碕' : [12, 13, 0, -2, 12, [256, 256, 65504, 8832, 9280, 10272, 32736, 53280, 22304, 21792, 22304, 28768, 0]],
        '鷺' : [12, 13, 0, -2, 12, [0, 62400, 40000, 62080, 11232, 48704, 41536, 65472, 16512, 32736, 16448, 43712, 0]],
        '作' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 17664, 18688, 51680, 16640, 16640, 16864, 16640, 16640, 16640, 0]],
        '削' : [12, 13, 0, -2, 12, [8192, 8224, 42272, 43296, 8480, 64800, 34080, 64800, 34080, 64544, 33824, 35936, 0]],
        '咋' : [12, 13, 0, -2, 12, [1024, 1024, 63456, 38144, 39168, 37344, 37120, 37120, 37344, 37120, 61696, 256, 0]],
        '搾' : [12, 13, 0, -2, 12, [256, 16640, 24544, 62112, 17536, 22688, 18400, 27136, 54240, 16896, 17376, 49664, 0]],
        '昨' : [12, 13, 0, -2, 12, [1024, 1024, 63456, 38144, 39168, 37344, 61696, 37120, 37344, 37120, 61696, 256, 0]],
        '朔' : [12, 13, 0, -2, 12, [34816, 19424, 4640, 64032, 9184, 43552, 43552, 44000, 64032, 8736, 17440, 17504, 0]],
        '柵' : [12, 13, 0, -2, 12, [0, 12256, 10912, 64160, 10912, 10912, 28640, 27296, 43680, 43680, 10912, 10336, 0]],
        '窄' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 25152, 16352, 18432, 36800, 2048, 4064, 2048, 0]],
        '策' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 1024, 65504, 1024, 32704, 17472, 19776, 13440, 50272, 0]],
        '索' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 65504, 35104, 37408, 9280, 32704, 1024, 17472, 33824, 0]],
        '錯' : [12, 13, 0, -2, 12, [576, 12864, 18400, 33344, 64064, 10208, 63488, 10208, 46112, 42976, 9248, 65504, 0]],
        '桜' : [12, 13, 0, -2, 12, [0, 9504, 9504, 62784, 8256, 8704, 28640, 25152, 42048, 42112, 9088, 11360, 0]],
        '鮭' : [12, 13, 0, -2, 12, [128, 28800, 37856, 8320, 63616, 45024, 63616, 43136, 64480, 128, 43136, 45024, 0]],
        '笹' : [12, 13, 0, -2, 12, [512, 16896, 31712, 37952, 9344, 9344, 65504, 9344, 9344, 10112, 8192, 16352, 0]],
        '匙' : [12, 13, 0, -2, 12, [0, 64000, 35424, 64384, 35328, 64000, 544, 64032, 8672, 47104, 57344, 40928, 0]],
        '冊' : [12, 13, 0, -2, 12, [0, 32704, 19008, 19008, 19008, 19008, 65504, 19008, 19008, 19008, 19008, 16576, 0]],
        '刷' : [12, 13, 0, -2, 12, [0, 64544, 34080, 34080, 64800, 41248, 41248, 64800, 42272, 42016, 44064, 8288, 0]],
        '察' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 34976, 43680, 12672, 57184, 0, 65504, 1024, 17472, 35872, 0]],
        '拶' : [12, 13, 0, -2, 12, [0, 18720, 21056, 58496, 21056, 18720, 16896, 25568, 52256, 16960, 16768, 52736, 0]],
        '撮' : [12, 13, 0, -2, 12, [0, 20416, 18496, 61376, 18496, 24544, 18944, 28640, 51872, 19104, 20160, 56096, 0]],
        '擦' : [12, 13, 0, -2, 12, [256, 16640, 24544, 62112, 23200, 17472, 23456, 24576, 57312, 16640, 18752, 54048, 0]],
        '札' : [12, 13, 0, -2, 12, [0, 8704, 8704, 64000, 8704, 8704, 25088, 29184, 43552, 41504, 8736, 9184, 0]],
        '殺' : [12, 13, 0, -2, 12, [0, 3008, 21056, 8768, 21056, 33888, 8192, 63456, 8736, 29248, 43392, 9824, 0]],
        '薩' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 41536, 53216, 43648, 44000, 51840, 36992, 38880, 0]],
        '雑' : [12, 13, 0, -2, 12, [512, 16960, 62016, 22496, 21632, 40928, 9344, 62592, 10208, 29824, 44160, 10208, 0]],
        '皐' : [12, 13, 0, -2, 12, [1024, 2048, 32704, 16448, 32704, 16448, 32704, 1024, 62944, 1024, 65504, 1024, 0]],
        '鯖' : [12, 13, 0, -2, 12, [128, 28800, 38880, 8320, 64480, 43136, 65504, 43552, 64480, 544, 44000, 43552, 0]],
        '捌' : [12, 13, 0, -2, 12, [0, 24096, 21152, 62112, 24224, 18592, 20128, 27296, 53920, 21024, 16928, 52320, 0]],
        '錆' : [12, 13, 0, -2, 12, [128, 12416, 18400, 32896, 64480, 8320, 65504, 8736, 44000, 43552, 9184, 64032, 0]],
        '鮫' : [12, 13, 0, -2, 12, [256, 28928, 38880, 8192, 64064, 44064, 63488, 43552, 63808, 128, 43328, 44576, 0]],
        '皿' : [12, 13, 0, -2, 12, [0, 32704, 19008, 19008, 19008, 19008, 19008, 19008, 19008, 19008, 19008, 65504, 0]],
        '晒' : [12, 13, 0, -2, 12, [0, 8160, 57984, 41600, 45024, 43680, 60064, 43680, 43744, 44064, 59424, 4064, 0]],
        '三' : [12, 13, 0, -2, 12, [0, 0, 32704, 0, 0, 0, 16256, 0, 0, 0, 0, 65504, 0]],
        '傘' : [12, 13, 0, -2, 12, [0, 7936, 8320, 21824, 42144, 21824, 9344, 21824, 33824, 65504, 1024, 1024, 0]],
        '参' : [12, 13, 0, -2, 12, [2048, 2304, 4224, 32704, 1024, 65504, 2176, 12608, 50720, 6272, 768, 15360, 0]],
        '山' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1024, 1024, 33824, 33824, 33824, 33824, 33824, 33824, 65504, 0]],
        '惨' : [12, 13, 0, -2, 12, [512, 8768, 9280, 12256, 41216, 49120, 41600, 42080, 47488, 9760, 8384, 12032, 0]],
        '撒' : [12, 13, 0, -2, 12, [2688, 19072, 24288, 60064, 24480, 16416, 24224, 29344, 56896, 21056, 24224, 54048, 0]],
        '散' : [12, 13, 0, -2, 12, [20992, 20992, 64480, 21056, 64576, 1088, 63808, 35136, 63616, 34944, 63808, 36384, 0]],
        '桟' : [12, 13, 0, -2, 12, [256, 8480, 12256, 61696, 10208, 8448, 28640, 24864, 45376, 41344, 9856, 14432, 0]],
        '燦' : [12, 13, 0, -2, 12, [2048, 12000, 10272, 44704, 47776, 42304, 43296, 9536, 24544, 21248, 34112, 39200, 0]],
        '珊' : [12, 13, 0, -2, 12, [0, 4064, 60064, 19104, 19104, 60064, 24544, 19104, 19104, 19104, 60064, 2144, 0]],
        '産' : [12, 13, 0, -2, 12, [1024, 1024, 32736, 2304, 2304, 32736, 20992, 24512, 20992, 40896, 33280, 16352, 0]],
        '算' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 16256, 8320, 16256, 8320, 16256, 4352, 65504, 8448, 0]],
        '纂' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 16256, 8320, 65504, 10880, 54624, 32704, 9344, 50272, 0]],
        '蚕' : [12, 13, 0, -2, 12, [0, 65504, 1024, 32704, 2304, 5248, 16320, 58528, 9344, 16288, 1056, 65504, 0]],
        '讃' : [12, 13, 0, -2, 12, [1088, 58432, 3808, 58432, 3808, 58432, 4064, 1088, 59328, 42048, 42944, 63584, 0]],
        '賛' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8320, 64480, 16448, 49056, 8320, 16256, 8320, 16256, 49248, 0]],
        '酸' : [12, 13, 0, -2, 12, [512, 64064, 9280, 10208, 63808, 43328, 44640, 55744, 36416, 64064, 35200, 65120, 0]],
        '餐' : [12, 13, 0, -2, 12, [16384, 31712, 16416, 31040, 51584, 15968, 8320, 65472, 8352, 16256, 8768, 63968, 0]],
        '斬' : [12, 13, 0, -2, 12, [8192, 8288, 64384, 8704, 64000, 44000, 64064, 43584, 64064, 8768, 64576, 9280, 0]],
        '暫' : [12, 13, 0, -2, 12, [4096, 4576, 65280, 21984, 21824, 64832, 4672, 32704, 16448, 32704, 16448, 32704, 0]],
        '残' : [12, 13, 0, -2, 12, [256, 61728, 20448, 16640, 30688, 37120, 40928, 20768, 8512, 8576, 18048, 39008, 0]],
        '仕' : [12, 13, 0, -2, 12, [256, 8448, 8448, 16640, 20448, 49408, 16640, 16640, 16640, 16640, 16640, 20448, 0]],
        '仔' : [12, 13, 0, -2, 12, [0, 10208, 8224, 16448, 16512, 49280, 20448, 16512, 16512, 16512, 16512, 17280, 0]],
        '伺' : [12, 13, 0, -2, 12, [0, 12256, 8224, 16416, 20256, 49184, 20256, 18720, 18720, 20256, 16416, 16480, 0]],
        '使' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16640, 20448, 51488, 18720, 20448, 16640, 17152, 17536, 22624, 0]],
        '刺' : [12, 13, 0, -2, 12, [8192, 8224, 64800, 8480, 8480, 64800, 42272, 42272, 12576, 26656, 42016, 8288, 0]],
        '司' : [12, 13, 0, -2, 12, [0, 65472, 64, 64, 31808, 64, 64, 31808, 17472, 17472, 31808, 192, 0]],
        '史' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 17472, 17472, 17472, 32704, 1024, 13312, 3072, 13056, 49376, 0]],
        '嗣' : [12, 13, 0, -2, 12, [0, 64480, 34848, 63520, 928, 63520, 43936, 43680, 64160, 43936, 43040, 34912, 0]],
        '四' : [12, 13, 0, -2, 12, [0, 0, 65504, 37408, 37408, 37408, 41504, 49632, 32800, 32800, 65504, 32800, 0]],
        '士' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 65504, 1024, 1024, 1024, 1024, 1024, 1024, 32704, 0]],
        '始' : [12, 13, 0, -2, 12, [256, 16640, 16928, 62496, 22496, 20480, 20480, 38880, 58400, 13344, 17440, 18400, 0]],
        '姉' : [12, 13, 0, -2, 12, [256, 16640, 20448, 61696, 20736, 22496, 21792, 38176, 58656, 13600, 17760, 16640, 0]],
        '姿' : [12, 13, 0, -2, 12, [1024, 17408, 10208, 2336, 10496, 17024, 33888, 2048, 65504, 4224, 7936, 24768, 0]],
        '子' : [12, 13, 0, -2, 12, [0, 16256, 128, 256, 512, 1024, 65504, 1024, 1024, 1024, 1024, 7168, 0]],
        '屍' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 16384, 32736, 20736, 23840, 26048, 38144, 35104, 12512, 0]],
        '市' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 1024, 32704, 17472, 17472, 17472, 17472, 17600, 1024, 0]],
        '師' : [12, 13, 0, -2, 12, [8192, 20448, 61696, 37120, 38880, 62752, 34080, 62752, 38176, 38176, 38240, 61696, 0]],
        '志' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 1024, 32704, 0, 1024, 512, 4160, 20512, 36736, 0]],
        '思' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 17472, 32704, 0, 1024, 4672, 20512, 36736, 0]],
        '指' : [12, 13, 0, -2, 12, [0, 9312, 10112, 62464, 9248, 10208, 8192, 14304, 58400, 10208, 9248, 26592, 0]],
        '支' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 1024, 16256, 128, 4224, 2304, 1536, 6528, 57440, 0]],
        '孜' : [12, 13, 0, -2, 12, [512, 64000, 3040, 2624, 5184, 9280, 8512, 63808, 8320, 8320, 9024, 27680, 0]],
        '斯' : [12, 13, 0, -2, 12, [0, 18528, 64896, 18688, 30976, 18912, 31008, 18720, 64800, 288, 21024, 35360, 0]],
        '施' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 63744, 18784, 17824, 30496, 23840, 38240, 38144, 5152, 25568, 0]],
        '旨' : [12, 13, 0, -2, 12, [0, 16608, 32512, 16416, 16352, 0, 16320, 8256, 16320, 8256, 8256, 16320, 0]],
        '枝' : [12, 13, 0, -2, 12, [256, 8448, 16352, 57600, 8448, 12224, 24640, 25664, 42048, 41600, 9088, 15456, 0]],
        '止' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 18368, 17408, 17408, 17408, 17408, 17408, 65504, 0]],
        '死' : [12, 13, 0, -2, 12, [0, 65504, 8704, 8704, 14848, 19040, 19328, 43520, 4608, 4640, 8736, 50144, 0]],
        '氏' : [12, 13, 0, -2, 12, [0, 448, 32256, 16896, 16896, 16896, 32736, 16640, 16640, 16512, 19584, 61536, 0]],
        '獅' : [12, 13, 0, -2, 12, [1024, 11232, 44672, 19072, 19424, 44704, 10400, 28320, 43680, 10912, 10976, 52864, 0]],
        '祉' : [12, 13, 0, -2, 12, [256, 8448, 8448, 61696, 5376, 5600, 9472, 25856, 46336, 9472, 9472, 12256, 0]],
        '私' : [12, 13, 0, -2, 12, [0, 6400, 57600, 8448, 8448, 64000, 8704, 25152, 29760, 42048, 43168, 12064, 0]],
        '糸' : [12, 13, 0, -2, 12, [0, 2048, 4352, 8704, 5120, 2176, 4160, 65504, 1024, 9344, 17472, 33824, 0]],
        '紙' : [12, 13, 0, -2, 12, [0, 8288, 18304, 38016, 62592, 9344, 18400, 62528, 9280, 46144, 42400, 11808, 0]],
        '紫' : [12, 13, 0, -2, 12, [512, 8704, 48096, 41472, 48672, 58336, 4608, 9280, 65504, 1024, 17472, 33824, 0]],
        '肢' : [12, 13, 0, -2, 12, [256, 57600, 49120, 41216, 57600, 44992, 41024, 58432, 42048, 41600, 41856, 48224, 0]],
        '脂' : [12, 13, 0, -2, 12, [0, 62560, 38784, 37888, 62496, 38880, 36864, 63456, 37920, 38880, 37920, 47072, 0]],
        '至' : [12, 13, 0, -2, 12, [0, 65504, 2048, 4352, 8320, 32704, 1056, 1024, 16256, 1024, 1024, 65504, 0]],
        '視' : [12, 13, 0, -2, 12, [0, 10208, 9248, 63456, 5152, 6112, 9248, 26592, 45696, 8832, 9376, 14560, 0]],
        '詞' : [12, 13, 0, -2, 12, [0, 61408, 32, 57376, 3872, 57376, 3872, 2336, 59680, 44832, 40992, 57440, 0]],
        '詩' : [12, 13, 0, -2, 12, [256, 57600, 2016, 57600, 256, 61408, 64, 4064, 57408, 42048, 41536, 57536, 0]],
        '試' : [12, 13, 0, -2, 12, [256, 57632, 288, 61408, 128, 57472, 3968, 1088, 58432, 42048, 44832, 57376, 0]],
        '誌' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57600, 256, 61408, 0, 512, 57664, 43040, 43008, 59328, 0]],
        '諮' : [12, 13, 0, -2, 12, [256, 59648, 1504, 58016, 128, 59712, 4640, 0, 59360, 42016, 42016, 59360, 0]],
        '資' : [12, 13, 0, -2, 12, [1024, 17408, 10208, 2336, 4736, 17472, 49056, 8320, 16256, 8320, 16256, 49248, 0]],
        '賜' : [12, 13, 0, -2, 12, [0, 63424, 37952, 63424, 37952, 63424, 37888, 38880, 62112, 1184, 20768, 35520, 0]],
        '雌' : [12, 13, 0, -2, 12, [2304, 10560, 10560, 11232, 43584, 49120, 43584, 43584, 44000, 43584, 47680, 53216, 0]],
        '飼' : [12, 13, 0, -2, 12, [0, 26592, 36896, 8224, 63392, 36896, 63392, 38048, 62624, 42912, 36896, 59488, 0]],
        '歯' : [12, 13, 0, -2, 12, [1024, 10176, 9216, 65504, 9344, 38176, 33824, 49056, 35872, 38176, 42144, 65504, 0]],
        '事' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 9344, 9344, 32704, 1088, 65504, 1088, 32704, 1024, 7168, 0]],
        '似' : [12, 13, 0, -2, 12, [0, 10304, 10304, 19008, 18752, 51264, 18496, 18560, 19584, 28992, 16928, 19488, 0]],
        '侍' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16640, 16640, 57312, 16448, 24544, 16448, 17472, 16960, 16576, 0]],
        '児' : [12, 13, 0, -2, 12, [0, 20416, 18496, 18496, 20416, 18496, 18496, 20416, 2304, 2336, 4384, 57824, 0]],
        '字' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 48928, 256, 512, 1024, 65504, 1024, 1024, 7168, 0]],
        '寺' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 1024, 65504, 256, 65504, 256, 8448, 4352, 1792, 0]],
        '慈' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 8320, 20800, 8320, 18720, 64480, 1024, 4672, 20512, 36736, 0]],
        '持' : [12, 13, 0, -2, 12, [256, 16640, 20448, 57600, 16640, 20448, 16448, 28640, 49216, 17472, 16960, 49344, 0]],
        '時' : [12, 13, 0, -2, 12, [256, 256, 63456, 37120, 37120, 40928, 61504, 40928, 36928, 37952, 62016, 192, 0]],
        '次' : [12, 13, 0, -2, 12, [1024, 1024, 34784, 17696, 2336, 2368, 256, 256, 17024, 17024, 33856, 38944, 0]],
        '滋' : [12, 13, 0, -2, 12, [2112, 1088, 32896, 24544, 1088, 34944, 20768, 7648, 17472, 18560, 37152, 40416, 0]],
        '治' : [12, 13, 0, -2, 12, [512, 512, 33824, 18464, 4064, 32768, 16384, 4064, 2080, 10272, 18464, 36832, 0]],
        '爾' : [12, 13, 0, -2, 12, [0, 65504, 9344, 17472, 65504, 33824, 52832, 46496, 52832, 46496, 52832, 33824, 0]],
        '璽' : [12, 13, 0, -2, 12, [0, 65504, 17472, 65504, 46496, 52832, 46496, 52832, 1024, 32704, 1152, 65504, 0]],
        '痔' : [12, 13, 0, -2, 12, [512, 512, 32736, 16896, 57312, 16896, 24544, 49216, 24544, 16448, 34880, 33984, 0]],
        '磁' : [12, 13, 0, -2, 12, [2112, 1088, 61568, 12256, 9280, 10368, 28960, 57312, 21056, 21632, 22816, 28640, 0]],
        '示' : [12, 13, 0, -2, 12, [0, 32704, 0, 0, 0, 65504, 1024, 1024, 9344, 17472, 33824, 3072, 0]],
        '而' : [12, 13, 0, -2, 12, [0, 65504, 1024, 2048, 65504, 37408, 37408, 37408, 37408, 37408, 37408, 32864, 0]],
        '耳' : [12, 13, 0, -2, 12, [0, 65504, 8320, 16256, 8320, 8320, 16256, 8320, 8416, 16256, 57472, 128, 0]],
        '自' : [12, 13, 0, -2, 12, [512, 1024, 16320, 8256, 8256, 16320, 8256, 8256, 16320, 8256, 8256, 16320, 0]],
        '蒔' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 63456, 37120, 40928, 61504, 40928, 37952, 62016, 192, 0]],
        '辞' : [12, 13, 0, -2, 12, [128, 6272, 59360, 8768, 8768, 65504, 8320, 8320, 65504, 34944, 34944, 63616, 0]],
        '汐' : [12, 13, 0, -2, 12, [512, 512, 33760, 16928, 1056, 34848, 16960, 320, 16512, 16640, 34304, 38912, 0]],
        '鹿' : [12, 13, 0, -2, 12, [512, 512, 32736, 18688, 32736, 18720, 32736, 20736, 24032, 37120, 38688, 14816, 0]],
        '式' : [12, 13, 0, -2, 12, [256, 288, 288, 65504, 256, 128, 31872, 4224, 4160, 4160, 7712, 57376, 0]],
        '識' : [12, 13, 0, -2, 12, [1152, 58528, 8096, 60032, 2784, 65408, 160, 3744, 59968, 44608, 43680, 61216, 0]],
        '鴫' : [12, 13, 0, -2, 12, [128, 256, 64448, 43584, 43968, 43584, 64480, 43520, 44000, 43040, 64160, 2784, 0]],
        '竺' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 8320, 0, 0, 16256, 0, 0, 0, 65504, 0]],
        '軸' : [12, 13, 0, -2, 12, [128, 8320, 63616, 9184, 64160, 43680, 64160, 44000, 64160, 8864, 64160, 9184, 0]],
        '宍' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 1024, 1024, 65504, 0, 0, 8320, 16448, 32800, 0]],
        '雫' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 0, 65504, 1024, 1792, 1216, 1024, 0]],
        '七' : [12, 13, 0, -2, 12, [4096, 4096, 4096, 4320, 7936, 61440, 4096, 4096, 4096, 4160, 4160, 4032, 0]],
        '叱' : [12, 13, 0, -2, 12, [512, 512, 61952, 37472, 37760, 40448, 37376, 37376, 37376, 37408, 61984, 992, 0]],
        '執' : [12, 13, 0, -2, 12, [8448, 8448, 63744, 10176, 63808, 21824, 21312, 63808, 8896, 64064, 9248, 9248, 0]],
        '失' : [12, 13, 0, -2, 12, [1024, 9216, 9216, 16320, 17408, 33792, 65504, 1024, 1536, 2304, 12416, 49248, 0]],
        '嫉' : [12, 13, 0, -2, 12, [256, 16640, 18400, 62720, 24032, 22144, 21632, 40928, 58496, 10368, 18752, 17952, 0]],
        '室' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 49056, 2304, 4224, 32704, 1024, 16256, 1024, 65504, 0]],
        '悉' : [12, 13, 0, -2, 12, [0, 32704, 17472, 9344, 65504, 3328, 13440, 49248, 1024, 4672, 20512, 36736, 0]],
        '湿' : [12, 13, 0, -2, 12, [0, 4032, 34880, 20416, 2112, 36800, 16384, 1280, 21792, 19776, 34048, 40928, 0]],
        '漆' : [12, 13, 0, -2, 12, [256, 256, 40928, 17728, 2336, 37504, 17472, 2336, 17728, 16768, 35136, 37664, 0]],
        '疾' : [12, 13, 0, -2, 12, [512, 512, 32736, 18432, 53184, 18688, 20736, 57312, 16640, 17024, 35904, 45088, 0]],
        '質' : [12, 13, 0, -2, 12, [0, 31680, 16896, 32736, 37440, 37440, 16256, 8320, 16256, 8320, 16256, 49248, 0]],
        '実' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33824, 16256, 1024, 16256, 1024, 65504, 2560, 12672, 49248, 0]],
        '蔀' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 64992, 18720, 18752, 64832, 288, 64800, 34240, 64768, 0]],
        '篠' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 10208, 23104, 20864, 55136, 20736, 24544, 17728, 22816, 0]],
        '偲' : [12, 13, 0, -2, 12, [0, 12256, 10528, 20448, 18720, 51488, 20448, 16896, 16704, 21536, 25600, 17344, 0]],
        '柴' : [12, 13, 0, -2, 12, [512, 8704, 47712, 41856, 41472, 48672, 58336, 1024, 65504, 3328, 13440, 50272, 0]],
        '芝' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 1024, 32704, 128, 16640, 17920, 47104, 34784, 0]],
        '屡' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 21056, 32736, 19072, 21056, 49120, 34944, 3840, 28896, 0]],
        '蕊' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 4672, 20512, 36736, 0, 10560, 42272, 41216, 7392, 0]],
        '縞' : [12, 13, 0, -2, 12, [256, 8448, 20448, 33856, 59328, 8192, 20448, 59424, 11168, 43680, 43936, 10336, 0]],
        '舎' : [12, 13, 0, -2, 12, [0, 3584, 12672, 50272, 16256, 1024, 65504, 0, 16256, 8320, 8320, 16256, 0]],
        '写' : [12, 13, 0, -2, 12, [0, 65504, 36896, 40864, 4096, 8192, 16256, 128, 65504, 128, 256, 3584, 0]],
        '射' : [12, 13, 0, -2, 12, [8256, 16448, 63552, 36832, 63552, 34880, 64064, 35136, 63552, 10304, 18496, 39104, 0]],
        '捨' : [12, 13, 0, -2, 12, [0, 17344, 17440, 59520, 18400, 16512, 20448, 24576, 51168, 17440, 17440, 51168, 0]],
        '赦' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 8768, 9280, 64576, 20800, 54592, 21632, 21632, 37184, 46624, 0]],
        '斜' : [12, 13, 0, -2, 12, [0, 12352, 19008, 33088, 61504, 9280, 8768, 61536, 8640, 44608, 41024, 8256, 0]],
        '煮' : [12, 13, 0, -2, 12, [2048, 2112, 32640, 2304, 65504, 4224, 16256, 53376, 8064, 0, 21056, 35104, 0]],
        '社' : [12, 13, 0, -2, 12, [8320, 8320, 8320, 63616, 3040, 4224, 8320, 28800, 43136, 8320, 8320, 10208, 0]],
        '紗' : [12, 13, 0, -2, 12, [256, 8448, 16640, 34112, 58656, 10496, 16640, 62208, 8224, 45120, 41344, 11776, 0]],
        '者' : [12, 13, 0, -2, 12, [1024, 1024, 32736, 1088, 1152, 65504, 2048, 8128, 12352, 57280, 4160, 8128, 0]],
        '謝' : [12, 13, 0, -2, 12, [544, 58400, 3872, 59872, 3872, 59680, 4000, 2400, 65312, 42272, 43296, 62304, 0]],
        '車' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 17472, 32704, 17472, 32704, 1024, 65504, 1024, 0]],
        '遮' : [12, 13, 0, -2, 12, [256, 256, 40928, 21056, 8160, 4672, 62400, 8192, 10912, 10912, 20480, 36832, 0]],
        '蛇' : [12, 13, 0, -2, 12, [256, 8448, 12256, 63520, 43552, 43520, 43616, 64384, 8704, 10784, 10784, 64480, 0]],
        '邪' : [12, 13, 0, -2, 12, [0, 64992, 18720, 18752, 18816, 64832, 2336, 6432, 10528, 18880, 35072, 14592, 0]],
        '借' : [12, 13, 0, -2, 12, [576, 8768, 12256, 16960, 16960, 53216, 16384, 18400, 17440, 18400, 17440, 18400, 0]],
        '勺' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16416, 16416, 34848, 34848, 1056, 1056, 32, 64, 384, 0]],
        '尺' : [12, 13, 0, -2, 12, [0, 16320, 8256, 8256, 8256, 16320, 8704, 8704, 16640, 16640, 32896, 32864, 0]],
        '杓' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 62496, 10272, 10272, 25120, 24864, 45088, 40992, 8256, 8576, 0]],
        '灼' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 42016, 43040, 43040, 8736, 8480, 20512, 18464, 32832, 33152, 0]],
        '爵' : [12, 13, 0, -2, 12, [0, 65504, 9344, 65504, 37152, 65504, 34880, 65504, 35392, 63808, 36928, 60608, 0]],
        '酌' : [12, 13, 0, -2, 12, [512, 64000, 9184, 8736, 64544, 43040, 43552, 55584, 34848, 63520, 34880, 63872, 0]],
        '釈' : [12, 13, 0, -2, 12, [0, 7136, 57888, 10784, 45600, 9184, 64128, 8832, 12864, 27200, 42016, 9248, 0]],
        '錫' : [12, 13, 0, -2, 12, [0, 14272, 17472, 34752, 62528, 10208, 62464, 10208, 45728, 42144, 8480, 64192, 0]],
        '若' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 2048, 65504, 2048, 4096, 16320, 53312, 4160, 8128, 0]],
        '寂' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 40992, 41952, 14400, 8256, 64832, 8512, 43136, 43392, 26208, 0]],
        '弱' : [12, 13, 0, -2, 12, [0, 64480, 2080, 2080, 64480, 33280, 64480, 35360, 18720, 10400, 18720, 45760, 0]],
        '惹' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 2048, 32736, 4096, 16320, 53312, 8128, 4672, 20512, 36736, 0]],
        '主' : [12, 13, 0, -2, 12, [2048, 1024, 0, 65504, 1024, 1024, 1024, 32704, 1024, 1024, 1024, 65504, 0]],
        '取' : [12, 13, 0, -2, 12, [0, 64512, 19424, 30752, 18976, 18976, 31040, 18752, 18560, 31104, 51776, 3104, 0]],
        '守' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 33056, 256, 65504, 256, 8448, 4352, 256, 1792, 0]],
        '手' : [12, 13, 0, -2, 12, [0, 960, 31744, 1024, 1024, 32704, 1024, 1024, 65504, 1024, 1024, 7168, 0]],
        '朱' : [12, 13, 0, -2, 12, [1024, 9216, 9216, 16320, 17408, 33792, 65504, 1024, 3328, 13440, 50272, 1024, 0]],
        '殊' : [12, 13, 0, -2, 12, [128, 62592, 17536, 18400, 29824, 36992, 38880, 20608, 8576, 8896, 19616, 32896, 0]],
        '狩' : [12, 13, 0, -2, 12, [256, 8448, 45024, 18464, 18464, 41024, 12256, 24640, 42048, 8768, 8256, 49344, 0]],
        '珠' : [12, 13, 0, -2, 12, [128, 1152, 62592, 10208, 9344, 8320, 63456, 8320, 8576, 12992, 52384, 128, 0]],
        '種' : [12, 13, 0, -2, 12, [0, 8160, 57600, 12256, 10528, 65504, 10528, 28640, 24832, 45024, 41216, 12256, 0]],
        '腫' : [12, 13, 0, -2, 12, [0, 61408, 41216, 45024, 59680, 45024, 43296, 61408, 41216, 45024, 41216, 45024, 0]],
        '趣' : [12, 13, 0, -2, 12, [8192, 12256, 62720, 9696, 10016, 64800, 9536, 14144, 44448, 41248, 61440, 36832, 0]],
        '酒' : [12, 13, 0, -2, 12, [0, 4064, 33408, 17024, 4064, 35488, 19104, 3296, 2080, 12256, 18464, 36832, 0]],
        '首' : [12, 13, 0, -2, 12, [8320, 4224, 256, 65504, 2048, 16256, 8320, 16256, 8320, 16256, 8320, 16256, 0]],
        '儒' : [12, 13, 0, -2, 12, [0, 12256, 8448, 20448, 18720, 52064, 16384, 24544, 16640, 20448, 19104, 19104, 0]],
        '受' : [12, 13, 0, -2, 12, [0, 32576, 2112, 17536, 8448, 65504, 32800, 49056, 128, 4352, 3584, 29120, 0]],
        '呪' : [12, 13, 0, -2, 12, [0, 2016, 62496, 37920, 37920, 38880, 37504, 37504, 37504, 37536, 62624, 6368, 0]],
        '寿' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 16256, 1024, 65504, 2176, 65504, 8320, 18560, 1408, 0]],
        '授' : [12, 13, 0, -2, 12, [0, 24480, 16928, 59712, 17472, 24544, 20512, 28608, 50240, 17536, 17280, 56416, 0]],
        '樹' : [12, 13, 0, -2, 12, [512, 8736, 12064, 62176, 12064, 8224, 28576, 26976, 44832, 42272, 9504, 12128, 0]],
        '綬' : [12, 13, 0, -2, 12, [0, 12192, 16928, 35136, 58432, 12256, 18464, 63424, 9280, 46208, 41856, 11360, 0]],
        '需' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 5248, 65504, 2048, 32704, 19008, 19008, 19136, 0]],
        '囚' : [12, 13, 0, -2, 12, [0, 65504, 33824, 33824, 33824, 33824, 35360, 37152, 41120, 32800, 32800, 65504, 0]],
        '収' : [12, 13, 0, -2, 12, [0, 6112, 20512, 20512, 21024, 21024, 20800, 20800, 28800, 53632, 4672, 7200, 0]],
        '周' : [12, 13, 0, -2, 12, [0, 32736, 17440, 17440, 24352, 17440, 24480, 16416, 24352, 20768, 40736, 32864, 0]],
        '宗' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 40736, 0, 0, 32704, 1024, 17472, 33824, 3072, 0]],
        '就' : [12, 13, 0, -2, 12, [8448, 8480, 63776, 2016, 63744, 35072, 35200, 64128, 8832, 45728, 42144, 25824, 0]],
        '州' : [12, 13, 0, -2, 12, [0, 8736, 8736, 8736, 43680, 43680, 43680, 43680, 8736, 8736, 16928, 16416, 0]],
        '修' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 18496, 19008, 49536, 20832, 22016, 20672, 22272, 16480, 20352, 0]],
        '愁' : [12, 13, 0, -2, 12, [0, 6272, 58016, 8864, 63616, 8576, 30272, 40992, 1024, 4672, 20512, 36736, 0]],
        '拾' : [12, 13, 0, -2, 12, [0, 9152, 9248, 61440, 8192, 10208, 8192, 12288, 59360, 9248, 9248, 26592, 0]],
        '洲' : [12, 13, 0, -2, 12, [0, 2336, 35104, 18720, 2336, 40352, 19296, 2336, 18720, 18720, 37152, 36896, 0]],
        '秀' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 3328, 13440, 50272, 32512, 2496, 2112, 4160, 25472, 0]],
        '秋' : [12, 13, 0, -2, 12, [256, 6400, 57600, 9504, 9504, 59712, 10560, 24832, 29312, 41600, 42048, 14368, 0]],
        '終' : [12, 13, 0, -2, 12, [512, 8704, 17376, 33824, 59968, 8576, 18016, 61696, 8384, 44032, 41728, 8416, 0]],
        '繍' : [12, 13, 0, -2, 12, [256, 8448, 20448, 33056, 61408, 8480, 18400, 62784, 11168, 44384, 43936, 11616, 0]],
        '習' : [12, 13, 0, -2, 12, [0, 64480, 18720, 10400, 18720, 39520, 1024, 16320, 8256, 16320, 8256, 16320, 0]],
        '臭' : [12, 13, 0, -2, 12, [1024, 2048, 32704, 16448, 32704, 16448, 32704, 1024, 65504, 2560, 12672, 49248, 0]],
        '舟' : [12, 13, 0, -2, 12, [1024, 2048, 16320, 8256, 9280, 8768, 8256, 65504, 9280, 9280, 17472, 16576, 0]],
        '蒐' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 17472, 32704, 17472, 32704, 2688, 12832, 50144, 0]],
        '衆' : [12, 13, 0, -2, 12, [1024, 2048, 32704, 19008, 19008, 65504, 4352, 8736, 52544, 1152, 13376, 50208, 0]],
        '襲' : [12, 13, 0, -2, 12, [4608, 4608, 64480, 18944, 65504, 18464, 31712, 18944, 65504, 4672, 57728, 15456, 0]],
        '讐' : [12, 13, 0, -2, 12, [8448, 10560, 32736, 53888, 32736, 21120, 65504, 1024, 32704, 8320, 8320, 16256, 0]],
        '蹴' : [12, 13, 0, -2, 12, [1152, 62624, 40608, 37344, 65152, 10880, 10944, 44864, 46400, 42304, 42304, 65120, 0]],
        '輯' : [12, 13, 0, -2, 12, [8192, 9184, 64032, 9184, 63488, 45024, 64032, 44000, 64032, 9184, 65056, 8224, 0]],
        '週' : [12, 13, 0, -2, 12, [0, 4064, 18720, 11168, 2336, 4064, 59424, 11168, 10912, 9120, 20480, 36832, 0]],
        '酋' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 4352, 32704, 19008, 21056, 25024, 16448, 32704, 16448, 32704, 0]],
        '酬' : [12, 13, 0, -2, 12, [0, 64160, 8864, 8864, 64160, 45024, 43680, 55968, 35488, 64160, 36000, 63520, 0]],
        '集' : [12, 13, 0, -2, 12, [4608, 4608, 16320, 25600, 49024, 9216, 16256, 1024, 65504, 3328, 13440, 50272, 0]],
        '醜' : [12, 13, 0, -2, 12, [256, 64000, 12256, 10528, 65504, 43296, 43296, 57312, 35136, 64320, 36128, 63712, 0]],
        '什' : [12, 13, 0, -2, 12, [128, 8320, 8320, 16512, 20464, 49280, 16512, 16512, 16512, 16512, 16512, 16512, 0]],
        '住' : [12, 13, 0, -2, 12, [512, 8448, 8192, 24544, 16640, 49408, 16640, 20448, 16640, 16640, 16640, 24544, 0]],
        '充' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2048, 2304, 4224, 32704, 4640, 4608, 4608, 8736, 49632, 0]],
        '十' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1024, 65504, 1024, 1024, 1024, 1024, 1024, 1024, 0]],
        '従' : [12, 13, 0, -2, 12, [2112, 9280, 16512, 36832, 256, 8448, 18688, 51680, 18688, 18688, 20224, 28896, 0]],
        '戎' : [12, 13, 0, -2, 12, [512, 544, 65504, 512, 8736, 8736, 63808, 8512, 8320, 16768, 16960, 19488, 0]],
        '柔' : [12, 13, 0, -2, 12, [0, 16256, 256, 512, 32704, 3136, 13440, 1024, 65504, 3328, 13440, 50272, 0]],
        '汁' : [12, 13, 0, -2, 12, [128, 128, 32896, 16512, 4064, 32896, 16512, 128, 128, 8320, 16512, 32896, 0]],
        '渋' : [12, 13, 0, -2, 12, [256, 2304, 35296, 18688, 2304, 40928, 16384, 2112, 17536, 16384, 34880, 36896, 0]],
        '獣' : [12, 13, 0, -2, 12, [42112, 43168, 160, 63616, 44000, 63616, 43136, 63616, 320, 63808, 35360, 64544, 0]],
        '縦' : [12, 13, 0, -2, 12, [1088, 8768, 18560, 38880, 57472, 9344, 18560, 64224, 10880, 43648, 43904, 11360, 0]],
        '重' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 9344, 16256, 9344, 16256, 1024, 16320, 1024, 65504, 0]],
        '銃' : [12, 13, 0, -2, 12, [256, 12544, 20448, 33280, 62016, 9280, 65440, 8864, 45696, 41600, 9376, 63712, 0]],
        '叔' : [12, 13, 0, -2, 12, [8192, 9184, 14368, 8224, 8736, 64032, 8512, 43328, 43136, 43392, 8768, 27680, 0]],
        '夙' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 18496, 20288, 18752, 21824, 21312, 16672, 34336, 38944, 0]],
        '宿' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 49120, 8704, 20416, 51264, 20416, 18496, 18496, 20416, 0]],
        '淑' : [12, 13, 0, -2, 12, [2048, 2528, 36384, 18464, 2208, 48800, 18592, 2112, 23104, 19008, 34976, 39712, 0]],
        '祝' : [12, 13, 0, -2, 12, [0, 10208, 9248, 62496, 5152, 6112, 8832, 25216, 45696, 8864, 9376, 14560, 0]],
        '縮' : [12, 13, 0, -2, 12, [256, 8448, 24544, 36896, 63456, 9344, 19424, 64032, 11232, 43552, 43552, 11232, 0]],
        '粛' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1088, 65504, 1088, 32704, 21664, 17696, 32736, 38176, 42144, 0]],
        '塾' : [12, 13, 0, -2, 12, [8448, 8448, 65472, 18752, 32064, 4672, 64800, 9376, 1024, 32704, 1024, 65504, 0]],
        '熟' : [12, 13, 0, -2, 12, [8448, 8448, 65472, 18752, 32064, 4672, 64064, 9504, 25760, 0, 21056, 35104, 0]],
        '出' : [12, 13, 0, -2, 12, [1024, 1024, 17472, 17472, 17472, 32704, 1024, 1024, 33824, 33824, 33824, 65504, 0]],
        '術' : [12, 13, 0, -2, 12, [1024, 9568, 17664, 40704, 1248, 9248, 17440, 54560, 21792, 21792, 17440, 17504, 0]],
        '述' : [12, 13, 0, -2, 12, [256, 320, 16672, 12224, 256, 256, 58688, 10528, 8448, 8960, 20480, 36832, 0]],
        '俊' : [12, 13, 0, -2, 12, [512, 8768, 9280, 24544, 17024, 50336, 22624, 17344, 19520, 17536, 17280, 23648, 0]],
        '峻' : [12, 13, 0, -2, 12, [512, 8768, 9280, 16352, 41600, 41632, 44128, 41920, 44096, 42112, 58240, 7264, 0]],
        '春' : [12, 13, 0, -2, 12, [1024, 1024, 32736, 1024, 16320, 2304, 65504, 4224, 16320, 53408, 4224, 8064, 0]],
        '瞬' : [12, 13, 0, -2, 12, [0, 4064, 58656, 42304, 65504, 43040, 44864, 59872, 44864, 45376, 58336, 3136, 0]],
        '竣' : [12, 13, 0, -2, 12, [512, 16960, 17472, 65504, 8832, 41632, 44128, 41920, 44096, 9344, 62336, 3168, 0]],
        '舜' : [12, 13, 0, -2, 12, [0, 32704, 9280, 9344, 65504, 32800, 30848, 19392, 31360, 35456, 6112, 24704, 0]],
        '駿' : [12, 13, 0, -2, 12, [512, 62016, 42048, 65504, 41600, 62112, 44128, 62400, 7232, 54400, 4992, 27744, 0]],
        '准' : [12, 13, 0, -2, 12, [1024, 1152, 33920, 20448, 2304, 8160, 2304, 2304, 20448, 18688, 35072, 36832, 0]],
        '循' : [12, 13, 0, -2, 12, [0, 8288, 20352, 34944, 4064, 10368, 19424, 51744, 19424, 21024, 21024, 17376, 0]],
        '旬' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16416, 16416, 48672, 41504, 15904, 8736, 15904, 64, 384, 0]],
        '楯' : [12, 13, 0, -2, 12, [0, 8288, 12160, 63616, 12256, 10368, 27616, 27168, 44000, 45600, 8736, 9184, 0]],
        '殉' : [12, 13, 0, -2, 12, [1024, 62464, 18400, 17440, 30752, 38688, 38176, 22304, 9504, 10016, 16448, 33152, 0]],
        '淳' : [12, 13, 0, -2, 12, [512, 512, 40928, 16384, 4032, 34880, 20416, 128, 16640, 24544, 33024, 34560, 0]],
        '準' : [12, 13, 0, -2, 12, [1152, 17536, 12256, 39168, 20416, 2304, 10496, 20448, 33792, 65504, 1024, 1024, 0]],
        '潤' : [12, 13, 0, -2, 12, [0, 7904, 37536, 24288, 4768, 40672, 20512, 6048, 20768, 22432, 37152, 38816, 0]],
        '盾' : [12, 13, 0, -2, 12, [0, 448, 32256, 16896, 32736, 16896, 24512, 20544, 24512, 36928, 36928, 8128, 0]],
        '純' : [12, 13, 0, -2, 12, [256, 8448, 18400, 37120, 61728, 9504, 17696, 63456, 8448, 43264, 43296, 8672, 0]],
        '巡' : [12, 13, 0, -2, 12, [0, 1344, 17728, 9536, 2688, 2688, 58688, 8864, 8864, 8192, 20480, 36832, 0]],
        '遵' : [12, 13, 0, -2, 12, [1088, 1152, 40928, 17024, 4064, 2720, 60640, 10272, 12256, 9280, 21056, 36832, 0]],
        '醇' : [12, 13, 0, -2, 12, [128, 63616, 10208, 8192, 64480, 43552, 44000, 55360, 34944, 65504, 34944, 63872, 0]],
        '順' : [12, 13, 0, -2, 12, [0, 18400, 20736, 22496, 21536, 22496, 21536, 22496, 21536, 22496, 33344, 35872, 0]],
        '処' : [12, 13, 0, -2, 12, [16384, 16384, 31680, 19008, 19008, 35392, 37440, 21568, 9312, 24576, 38912, 34784, 0]],
        '初' : [12, 13, 0, -2, 12, [0, 20448, 16928, 61984, 4640, 4640, 8736, 17440, 58400, 18464, 18496, 16768, 0]],
        '所' : [12, 13, 0, -2, 12, [0, 63584, 896, 512, 31232, 19424, 19008, 31296, 16960, 17472, 33856, 32832, 0]],
        '暑' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 2080, 32704, 2176, 65504, 4160, 16320, 53312, 8128, 0]],
        '曙' : [12, 13, 0, -2, 12, [0, 4064, 60064, 45024, 41504, 45024, 57984, 49120, 42016, 45024, 58400, 2016, 0]],
        '渚' : [12, 13, 0, -2, 12, [256, 256, 36832, 16672, 320, 40928, 16896, 2016, 19488, 22496, 33824, 34784, 0]],
        '庶' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 18560, 32736, 18560, 34944, 36736, 0, 21056, 35104, 0]],
        '緒' : [12, 13, 0, -2, 12, [256, 8448, 18400, 37152, 61760, 10208, 18688, 64480, 9760, 44000, 43552, 9184, 0]],
        '署' : [12, 13, 0, -2, 12, [0, 32736, 18720, 32736, 2080, 32704, 2176, 65504, 4160, 16320, 53312, 8128, 0]],
        '書' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1088, 65504, 1088, 32704, 1024, 65504, 8320, 8320, 16256, 0]],
        '薯' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 19008, 32704, 2080, 32704, 2176, 65504, 12352, 53312, 8128, 0]],
        '藷' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 63456, 288, 61760, 2016, 61984, 40928, 37408, 62432, 0]],
        '諸' : [12, 13, 0, -2, 12, [256, 61696, 2016, 61728, 320, 63456, 256, 992, 63008, 39904, 37408, 62432, 0]],
        '助' : [12, 13, 0, -2, 12, [256, 61696, 37120, 40928, 61728, 37152, 37408, 61984, 37920, 37920, 36928, 63872, 0]],
        '叙' : [12, 13, 0, -2, 12, [0, 13280, 18464, 32800, 64032, 8736, 8512, 63808, 8320, 43136, 43328, 26144, 0]],
        '女' : [12, 13, 0, -2, 12, [2048, 2048, 2048, 65504, 4224, 4224, 4224, 8448, 12544, 3584, 4480, 57440, 0]],
        '序' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 20416, 16512, 16640, 24544, 16672, 33088, 33024, 1792, 0]],
        '徐' : [12, 13, 0, -2, 12, [0, 10112, 18496, 36896, 4064, 8448, 16640, 53216, 16640, 18752, 20768, 17152, 0]],
        '恕' : [12, 13, 0, -2, 12, [4096, 4096, 64992, 9504, 17696, 18720, 12768, 51200, 1024, 4160, 20512, 36736, 0]],
        '鋤' : [12, 13, 0, -2, 12, [128, 11904, 19072, 35808, 61088, 10912, 64160, 11936, 43680, 43808, 10784, 65216, 0]],
        '除' : [12, 13, 0, -2, 12, [0, 59264, 43072, 45088, 53216, 41216, 41216, 49120, 41216, 51520, 37152, 33536, 0]],
        '傷' : [12, 13, 0, -2, 12, [1024, 9216, 12256, 22592, 20416, 51264, 20448, 17408, 24544, 18720, 21024, 17600, 0]],
        '償' : [12, 13, 0, -2, 12, [256, 9536, 16352, 21664, 21632, 53216, 18464, 20448, 18464, 20448, 17472, 22560, 0]],
        '勝' : [12, 13, 0, -2, 12, [256, 58656, 42304, 45024, 57984, 45024, 42048, 59680, 47072, 41248, 41504, 44224, 0]],
        '匠' : [12, 13, 0, -2, 12, [0, 65504, 32768, 40832, 36864, 36864, 40896, 37120, 41216, 41216, 33024, 65504, 0]],
        '升' : [12, 13, 0, -2, 12, [0, 3200, 28800, 4224, 4224, 4224, 65504, 4224, 8320, 8320, 16512, 16512, 0]],
        '召' : [12, 13, 0, -2, 12, [0, 32736, 544, 1056, 6176, 24768, 0, 16320, 8256, 8256, 8256, 16320, 0]],
        '哨' : [12, 13, 0, -2, 12, [256, 256, 59680, 42304, 41216, 45024, 43040, 45024, 43040, 45024, 59424, 2144, 0]],
        '商' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 4352, 65504, 37152, 41248, 57312, 37152, 37152, 40800, 0]],
        '唱' : [12, 13, 0, -2, 12, [0, 1984, 62528, 38848, 37952, 38848, 36864, 40928, 36896, 40928, 61472, 8160, 0]],
        '嘗' : [12, 13, 0, -2, 12, [1024, 9344, 65504, 37152, 20736, 32640, 16416, 32736, 8320, 16256, 8320, 16256, 0]],
        '奨' : [12, 13, 0, -2, 12, [0, 6112, 38176, 20544, 6112, 21568, 37568, 1024, 65504, 2560, 12672, 49248, 0]],
        '妾' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 4352, 65504, 2048, 65504, 4224, 12544, 7936, 57568, 0]],
        '娼' : [12, 13, 0, -2, 12, [0, 18368, 17472, 63424, 21568, 22464, 20480, 40928, 59424, 12256, 18464, 20448, 0]],
        '宵' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 42144, 5376, 32704, 16448, 32704, 16448, 32704, 16448, 16576, 0]],
        '将' : [12, 13, 0, -2, 12, [0, 10208, 8224, 41536, 26944, 9216, 8256, 12256, 24640, 42048, 8768, 8384, 0]],
        '小' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1024, 17472, 17472, 17440, 33824, 33824, 1024, 1024, 7168, 0]],
        '少' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 9344, 9280, 17440, 33792, 3136, 128, 768, 3072, 28672, 0]],
        '尚' : [12, 13, 0, -2, 12, [1024, 1024, 17472, 9344, 1024, 65504, 32800, 40736, 37152, 37152, 40736, 32864, 0]],
        '庄' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 16640, 16640, 20448, 16640, 16640, 33024, 33024, 8160, 0]],
        '床' : [12, 13, 0, -2, 12, [512, 512, 32736, 16896, 16896, 32736, 16896, 17152, 18048, 35392, 45600, 512, 0]],
        '廠' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 41472, 46048, 41536, 64576, 34112, 62784, 38016, 62848, 36448, 0]],
        '彰' : [12, 13, 0, -2, 12, [4096, 4128, 65088, 9344, 65280, 17440, 31808, 17536, 31744, 4128, 65088, 4224, 0]],
        '承' : [12, 13, 0, -2, 12, [0, 16256, 512, 1024, 61344, 9408, 11904, 17472, 24384, 33824, 33824, 7168, 0]],
        '抄' : [12, 13, 0, -2, 12, [256, 16640, 16640, 58688, 18720, 16640, 16640, 25344, 49184, 16448, 16768, 52736, 0]],
        '招' : [12, 13, 0, -2, 12, [0, 10208, 8480, 61728, 8736, 9408, 8192, 14304, 58400, 9248, 9248, 26592, 0]],
        '掌' : [12, 13, 0, -2, 12, [1024, 9344, 65504, 37152, 4352, 32704, 1024, 16256, 1024, 65504, 1024, 7168, 0]],
        '捷' : [12, 13, 0, -2, 12, [256, 16640, 20448, 57632, 20448, 16672, 18400, 24832, 51680, 18688, 20224, 61664, 0]],
        '昇' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 32704, 4352, 4352, 65504, 4352, 8448, 49408, 0]],
        '昌' : [12, 13, 0, -2, 12, [0, 16320, 8256, 16320, 8256, 16320, 0, 32736, 16416, 32736, 16416, 32736, 0]],
        '昭' : [12, 13, 0, -2, 12, [0, 2016, 61728, 37152, 37408, 38080, 61440, 38880, 37920, 37920, 62496, 2016, 0]],
        '晶' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 16256, 0, 64480, 35360, 64480, 35360, 64480, 0]],
        '松' : [12, 13, 0, -2, 12, [0, 9152, 8256, 62528, 9248, 10272, 27168, 25088, 42048, 42048, 10400, 12064, 0]],
        '梢' : [12, 13, 0, -2, 12, [256, 8448, 9504, 62784, 8448, 10208, 25632, 26592, 46112, 42976, 9248, 9312, 0]],
        '樟' : [12, 13, 0, -2, 12, [128, 8320, 12256, 57920, 12256, 9248, 26592, 25632, 47072, 41088, 12256, 8320, 0]],
        '樵' : [12, 13, 0, -2, 12, [576, 8768, 10208, 64640, 10208, 9344, 26592, 29824, 42976, 40960, 10912, 10912, 0]],
        '沼' : [12, 13, 0, -2, 12, [0, 4064, 33312, 16928, 1056, 35008, 16384, 2016, 1056, 9248, 17440, 34784, 0]],
        '消' : [12, 13, 0, -2, 12, [256, 256, 35104, 17728, 256, 36832, 18464, 4064, 2080, 12256, 18464, 34912, 0]],
        '渉' : [12, 13, 0, -2, 12, [512, 2560, 35776, 18944, 8160, 33280, 19008, 4640, 17920, 16448, 33152, 36352, 0]],
        '湘' : [12, 13, 0, -2, 12, [4096, 5088, 37408, 32288, 5088, 37408, 21024, 7136, 22048, 21024, 37408, 37856, 0]],
        '焼' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8448, 49120, 42048, 42048, 12256, 21120, 21120, 33952, 39136, 0]],
        '焦' : [12, 13, 0, -2, 12, [4352, 4352, 16320, 8704, 32704, 41472, 16320, 8704, 16320, 0, 21056, 35104, 0]],
        '照' : [12, 13, 0, -2, 12, [0, 63456, 37152, 37408, 62656, 36864, 38880, 62496, 2016, 0, 21056, 35104, 0]],
        '症' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 57312, 16896, 16896, 54208, 20992, 20992, 37376, 49120, 0]],
        '省' : [12, 13, 0, -2, 12, [1024, 1024, 9280, 9248, 19712, 512, 4032, 12352, 57280, 4160, 4160, 8128, 0]],
        '硝' : [12, 13, 0, -2, 12, [256, 256, 62752, 9536, 8448, 10208, 29728, 55264, 21536, 22496, 21536, 29792, 0]],
        '礁' : [12, 13, 0, -2, 12, [576, 576, 63456, 9344, 12256, 9344, 30688, 54400, 22496, 20480, 23200, 29344, 0]],
        '祥' : [12, 13, 0, -2, 12, [1088, 8768, 8320, 63456, 4224, 4224, 10208, 24704, 45184, 10208, 8320, 8320, 0]],
        '称' : [12, 13, 0, -2, 12, [512, 6656, 58336, 9344, 9344, 61568, 8896, 25760, 26784, 41088, 41088, 9088, 0]],
        '章' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 4352, 65504, 8320, 16256, 8320, 16256, 1024, 65504, 1024, 0]],
        '笑' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 4096, 960, 31744, 1024, 65504, 2560, 12672, 49248, 0]],
        '粧' : [12, 13, 0, -2, 12, [256, 8448, 47072, 41984, 9344, 62592, 10208, 25728, 26752, 43136, 41088, 10208, 0]],
        '紹' : [12, 13, 0, -2, 12, [0, 10208, 16672, 37152, 61984, 9408, 16384, 63456, 9248, 46112, 42016, 10208, 0]],
        '肖' : [12, 13, 0, -2, 12, [1024, 1024, 17472, 9344, 1024, 32704, 16448, 32704, 16448, 32704, 16448, 16576, 0]],
        '菖' : [12, 13, 0, -2, 12, [4352, 65504, 4352, 16256, 8320, 16256, 8320, 32704, 16448, 32704, 16448, 32704, 0]],
        '蒋' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 12256, 41504, 26944, 9280, 12256, 24640, 42048, 8896, 0]],
        '蕉' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16320, 25088, 49088, 8704, 16320, 0, 21056, 35104, 0]],
        '衝' : [12, 13, 0, -2, 12, [0, 12000, 17408, 40704, 5600, 16160, 21792, 57120, 17440, 24352, 17440, 24416, 0]],
        '裳' : [12, 13, 0, -2, 12, [1024, 9344, 65504, 32800, 49056, 8320, 16256, 1024, 65504, 4672, 57728, 15456, 0]],
        '訟' : [12, 13, 0, -2, 12, [0, 58304, 64, 58432, 1056, 59424, 2592, 512, 58432, 42048, 43168, 61216, 0]],
        '証' : [12, 13, 0, -2, 12, [0, 61408, 256, 57600, 256, 58624, 1504, 1280, 58624, 42240, 42240, 61408, 0]],
        '詔' : [12, 13, 0, -2, 12, [0, 59360, 288, 57632, 544, 58560, 0, 2016, 58400, 42016, 42016, 59360, 0]],
        '詳' : [12, 13, 0, -2, 12, [2112, 58432, 128, 61408, 256, 57600, 2016, 256, 57600, 45024, 41216, 57600, 0]],
        '象' : [12, 13, 0, -2, 12, [0, 16128, 16896, 65472, 17472, 17472, 32704, 5248, 25856, 6784, 57952, 3072, 0]],
        '賞' : [12, 13, 0, -2, 12, [1024, 9344, 65504, 37152, 40736, 0, 16256, 8320, 16256, 8320, 16256, 49248, 0]],
        '醤' : [12, 13, 0, -2, 12, [0, 38880, 21792, 4160, 22496, 37952, 4672, 65504, 19008, 29632, 16448, 32704, 0]],
        '鉦' : [12, 13, 0, -2, 12, [0, 14304, 18560, 32896, 63616, 8832, 64224, 8832, 43648, 43648, 8832, 65504, 0]],
        '鍾' : [12, 13, 0, -2, 12, [0, 14304, 16640, 34784, 62752, 10208, 64800, 10208, 45312, 42976, 8448, 65504, 0]],
        '鐘' : [12, 13, 0, -2, 12, [256, 12544, 20448, 33344, 62016, 10208, 64800, 10208, 46368, 42976, 8448, 65504, 0]],
        '障' : [12, 13, 0, -2, 12, [256, 57600, 45024, 41536, 53216, 43040, 45024, 43040, 45024, 49408, 36832, 33024, 0]],
        '鞘' : [12, 13, 0, -2, 12, [128, 18560, 64160, 19136, 30848, 9184, 64032, 44000, 64032, 9184, 65056, 8800, 0]],
        '上' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 1984, 1024, 1024, 1024, 1024, 1024, 1024, 65504, 0]],
        '丈' : [12, 13, 0, -2, 12, [512, 512, 512, 65504, 512, 512, 4608, 2560, 1024, 2560, 12672, 49248, 0]],
        '丞' : [12, 13, 0, -2, 12, [0, 16256, 256, 512, 62752, 5440, 9344, 17472, 33824, 7168, 0, 65504, 0]],
        '乗' : [12, 13, 0, -2, 12, [0, 32704, 1024, 32704, 9344, 65504, 9344, 9344, 32704, 3328, 13440, 50272, 0]],
        '冗' : [12, 13, 0, -2, 12, [0, 65504, 32800, 32800, 7936, 4352, 4352, 4352, 4352, 4384, 8480, 49632, 0]],
        '剰' : [12, 13, 0, -2, 12, [0, 64544, 4384, 64800, 21792, 64800, 21792, 21792, 64800, 4128, 13344, 53856, 0]],
        '城' : [12, 13, 0, -2, 12, [128, 16544, 20448, 18560, 59520, 20128, 19104, 19104, 19008, 20032, 61600, 4896, 0]],
        '場' : [12, 13, 0, -2, 12, [0, 20416, 18496, 20416, 59456, 20448, 17408, 24544, 17568, 18720, 61984, 1216, 0]],
        '壌' : [12, 13, 0, -2, 12, [256, 16640, 24544, 17472, 61408, 21632, 20448, 17536, 24544, 17728, 63616, 3680, 0]],
        '嬢' : [12, 13, 0, -2, 12, [256, 16640, 20448, 62016, 22496, 23104, 22496, 37440, 61408, 12960, 19520, 18208, 0]],
        '常' : [12, 13, 0, -2, 12, [1024, 9344, 65504, 32800, 40736, 4352, 7936, 1024, 32704, 17472, 17472, 17600, 0]],
        '情' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8448, 42976, 41216, 45024, 42016, 42976, 9248, 10208, 9248, 0]],
        '擾' : [12, 13, 0, -2, 12, [0, 20448, 17472, 59328, 17472, 24544, 21792, 17472, 58336, 19520, 17280, 56416, 0]],
        '条' : [12, 13, 0, -2, 12, [0, 8064, 24704, 2304, 1536, 6528, 58464, 1024, 32704, 3328, 13440, 50272, 0]],
        '杖' : [12, 13, 0, -2, 12, [128, 8320, 8320, 63456, 8320, 8320, 25728, 25216, 45440, 41088, 9024, 11296, 0]],
        '浄' : [12, 13, 0, -2, 12, [0, 1920, 34944, 20736, 4064, 33056, 24544, 288, 20448, 16640, 33024, 34560, 0]],
        '状' : [12, 13, 0, -2, 12, [256, 8512, 8480, 41216, 28640, 8448, 8448, 8448, 25216, 41600, 9280, 14368, 0]],
        '畳' : [12, 13, 0, -2, 12, [0, 16256, 9344, 16256, 9344, 65504, 41120, 49056, 8320, 16256, 8320, 65504, 0]],
        '穣' : [12, 13, 0, -2, 12, [256, 6400, 61408, 9280, 12256, 62016, 12256, 25152, 28640, 41632, 44096, 10016, 0]],
        '蒸' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 7936, 58080, 9344, 17472, 49056, 0, 21056, 35104, 0]],
        '譲' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57920, 2016, 59968, 2016, 576, 61408, 41632, 44096, 59168, 0]],
        '醸' : [12, 13, 0, -2, 12, [256, 61696, 20448, 16960, 63456, 55872, 47072, 37440, 40928, 62112, 40000, 63264, 0]],
        '錠' : [12, 13, 0, -2, 12, [256, 12544, 20448, 34848, 61440, 10208, 63744, 8448, 46560, 42240, 9984, 63712, 0]],
        '嘱' : [12, 13, 0, -2, 12, [0, 8160, 61472, 40928, 38208, 38208, 38848, 37120, 40928, 59680, 11168, 10336, 0]],
        '埴' : [12, 13, 0, -2, 12, [256, 16640, 24544, 16640, 59328, 17472, 22464, 21568, 21568, 22464, 61440, 8160, 0]],
        '飾' : [12, 13, 0, -2, 12, [512, 29184, 35808, 9344, 63616, 35808, 64160, 35488, 64160, 37536, 35040, 62592, 0]],
        '拭' : [12, 13, 0, -2, 12, [256, 16672, 16672, 65504, 16512, 16512, 24448, 25664, 50240, 17472, 24352, 49184, 0]],
        '植' : [12, 13, 0, -2, 12, [128, 8320, 12256, 61568, 9184, 10784, 27616, 27168, 43552, 44000, 10240, 12256, 0]],
        '殖' : [12, 13, 0, -2, 12, [128, 61568, 20448, 16512, 29664, 37408, 37856, 23072, 10784, 11232, 18432, 36832, 0]],
        '燭' : [12, 13, 0, -2, 12, [0, 12256, 10912, 45024, 43008, 45024, 10912, 12192, 21024, 21152, 36768, 32864, 0]],
        '織' : [12, 13, 0, -2, 12, [1152, 9376, 24480, 35456, 60128, 16256, 16544, 61088, 10816, 44608, 43680, 12064, 0]],
        '職' : [12, 13, 0, -2, 12, [1152, 62624, 44960, 60032, 43744, 49024, 57504, 44704, 43584, 60992, 43680, 12064, 0]],
        '色' : [12, 13, 0, -2, 12, [0, 7936, 8448, 16896, 65472, 17472, 17472, 32704, 16384, 16416, 16416, 16352, 0]],
        '触' : [12, 13, 0, -2, 12, [128, 28800, 36992, 10208, 64672, 42144, 64672, 42976, 42112, 64672, 33952, 36832, 0]],
        '食' : [12, 13, 0, -2, 12, [0, 3584, 4352, 9344, 32704, 41120, 16256, 8320, 16288, 8768, 8576, 63584, 0]],
        '蝕' : [12, 13, 0, -2, 12, [128, 28800, 34944, 10208, 64672, 33952, 64672, 34784, 64640, 37024, 34976, 63456, 0]],
        '辱' : [12, 13, 0, -2, 12, [0, 32736, 16384, 32736, 21056, 37248, 48224, 128, 65504, 128, 4224, 2432, 0]],
        '尻' : [12, 13, 0, -2, 12, [0, 32736, 16416, 16416, 32736, 17408, 17408, 24512, 17472, 33856, 34880, 12384, 0]],
        '伸' : [12, 13, 0, -2, 12, [256, 8448, 12256, 18720, 18720, 53216, 18720, 18720, 20448, 16640, 16640, 16640, 0]],
        '信' : [12, 13, 0, -2, 12, [0, 10208, 8192, 20448, 16384, 51168, 16384, 16384, 18400, 17440, 17440, 18400, 0]],
        '侵' : [12, 13, 0, -2, 12, [0, 12224, 8256, 20416, 16448, 57312, 20512, 20416, 17472, 17536, 17280, 23648, 0]],
        '唇' : [12, 13, 0, -2, 12, [0, 32736, 16384, 32736, 21056, 37248, 48224, 0, 16320, 8256, 8256, 16320, 0]],
        '娠' : [12, 13, 0, -2, 12, [0, 24544, 20480, 63424, 20480, 20480, 24544, 38016, 62624, 9376, 17472, 20256, 0]],
        '寝' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 20416, 16448, 57312, 20512, 20416, 50240, 18304, 22624, 0]],
        '審' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 42144, 5376, 65504, 5376, 16256, 58592, 16256, 9344, 16256, 0]],
        '心' : [12, 13, 0, -2, 12, [0, 6144, 1536, 0, 0, 4160, 20544, 20512, 36896, 36992, 4224, 3840, 0]],
        '慎' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8448, 42976, 42016, 42976, 42016, 45024, 8192, 9280, 10272, 0]],
        '振' : [12, 13, 0, -2, 12, [0, 20448, 18432, 60352, 18432, 18432, 20448, 27264, 51872, 21152, 21056, 50976, 0]],
        '新' : [12, 13, 0, -2, 12, [8192, 8288, 64384, 18944, 18944, 65504, 8768, 64064, 12864, 27200, 42048, 9280, 0]],
        '晋' : [12, 13, 0, -2, 12, [0, 32704, 2560, 19008, 10880, 2560, 65504, 8320, 16256, 8320, 8320, 16256, 0]],
        '森' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 5376, 9344, 17472, 8320, 64480, 8320, 29120, 43680, 8320, 0]],
        '榛' : [12, 13, 0, -2, 12, [256, 8448, 16352, 57856, 12256, 9344, 28640, 28992, 45024, 41728, 9536, 14624, 0]],
        '浸' : [12, 13, 0, -2, 12, [0, 4032, 32832, 20416, 64, 40928, 20512, 4032, 1088, 9344, 17280, 40032, 0]],
        '深' : [12, 13, 0, -2, 12, [0, 8160, 37536, 21120, 1152, 39008, 16640, 8160, 17152, 17728, 39200, 33024, 0]],
        '申' : [12, 13, 0, -2, 12, [1024, 1024, 32736, 17440, 17440, 32736, 17440, 17440, 32736, 1024, 1024, 1024, 0]],
        '疹' : [12, 13, 0, -2, 12, [512, 512, 32736, 17664, 51328, 21312, 27680, 49536, 17920, 22624, 33152, 40448, 0]],
        '真' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 16256, 8320, 16256, 8320, 65504, 0, 12672, 49248, 0]],
        '神' : [12, 13, 0, -2, 12, [128, 8320, 10208, 62624, 5280, 6112, 9376, 25760, 47072, 8320, 8320, 8320, 0]],
        '秦' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2048, 16256, 4352, 65504, 9344, 32704, 36128, 5248, 25664, 0]],
        '紳' : [12, 13, 0, -2, 12, [256, 8448, 20448, 35104, 59680, 12256, 18720, 63776, 12256, 41216, 41216, 8448, 0]],
        '臣' : [12, 13, 0, -2, 12, [0, 32736, 16896, 16896, 32704, 16448, 16448, 32704, 16896, 16896, 16896, 32736, 0]],
        '芯' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 1024, 512, 4160, 20512, 36864, 4160, 4032, 0]],
        '薪' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 64608, 19328, 65024, 9184, 64064, 12864, 27200, 42048, 0]],
        '親' : [12, 13, 0, -2, 12, [8192, 9184, 64032, 19424, 18976, 65504, 8736, 64480, 12608, 26944, 41536, 11360, 0]],
        '診' : [12, 13, 0, -2, 12, [0, 58240, 1088, 59424, 192, 59136, 0, 192, 59136, 40992, 41152, 59136, 0]],
        '身' : [12, 13, 0, -2, 12, [1024, 2048, 16256, 8320, 16256, 8352, 16320, 8320, 65408, 2176, 12416, 50048, 0]],
        '辛' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 4352, 4608, 65504, 1024, 1024, 32704, 1024, 1024, 0]],
        '進' : [12, 13, 0, -2, 12, [1152, 1152, 34784, 19584, 6112, 1152, 59360, 9344, 9344, 10208, 20480, 36832, 0]],
        '針' : [12, 13, 0, -2, 12, [64, 14400, 17472, 32832, 64480, 8256, 64576, 8256, 43072, 43072, 8256, 64576, 0]],
        '震' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 32736, 16384, 32736, 21056, 37248, 48224, 0]],
        '人' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1024, 1024, 2560, 2560, 4352, 4352, 8320, 16448, 32800, 0]],
        '仁' : [12, 13, 0, -2, 12, [0, 8192, 12256, 16384, 16384, 49152, 16384, 16384, 16384, 16384, 20448, 16384, 0]],
        '刃' : [12, 13, 0, -2, 12, [0, 65504, 2080, 2080, 10272, 6176, 2080, 3104, 4640, 4128, 8256, 50048, 0]],
        '塵' : [12, 13, 0, -2, 12, [512, 512, 32736, 18720, 32736, 20736, 24000, 20768, 32224, 33280, 33280, 16352, 0]],
        '壬' : [12, 13, 0, -2, 12, [0, 960, 31744, 1024, 1024, 1024, 65504, 1024, 1024, 1024, 1024, 32704, 0]],
        '尋' : [12, 13, 0, -2, 12, [0, 16256, 128, 16256, 128, 64480, 8736, 64480, 128, 65504, 8320, 4480, 0]],
        '甚' : [12, 13, 0, -2, 12, [4224, 4224, 32736, 4224, 8064, 4224, 4224, 65504, 18688, 29152, 16384, 32736, 0]],
        '尽' : [12, 13, 0, -2, 12, [0, 16320, 8256, 8256, 16320, 8320, 8320, 19520, 17216, 32800, 39968, 896, 0]],
        '腎' : [12, 13, 0, -2, 12, [0, 64480, 40992, 63808, 34944, 65472, 41120, 16256, 8320, 16256, 8320, 8576, 0]],
        '訊' : [12, 13, 0, -2, 12, [0, 61376, 576, 57920, 576, 58304, 3648, 576, 57920, 41536, 42016, 63520, 0]],
        '迅' : [12, 13, 0, -2, 12, [0, 4032, 16960, 8768, 576, 4032, 57920, 8736, 8736, 8704, 20480, 36832, 0]],
        '陣' : [12, 13, 0, -2, 12, [256, 57600, 45024, 41216, 53216, 43296, 45024, 43296, 45024, 49408, 40928, 33024, 0]],
        '靭' : [12, 13, 0, -2, 12, [0, 22496, 63776, 20768, 29984, 8992, 63904, 43296, 64032, 8736, 63552, 8576, 0]],
        '笥' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 65504, 32, 32288, 32, 32288, 16928, 32352, 0]],
        '諏' : [12, 13, 0, -2, 12, [0, 65504, 2560, 61408, 2592, 60064, 3744, 2720, 59968, 44608, 47776, 58144, 0]],
        '須' : [12, 13, 0, -2, 12, [0, 12256, 16640, 34784, 1056, 10208, 17440, 34784, 1056, 10208, 16960, 35872, 0]],
        '酢' : [12, 13, 0, -2, 12, [1024, 64512, 10208, 9472, 63744, 43488, 43264, 55552, 35296, 63744, 35072, 63744, 0]],
        '図' : [12, 13, 0, -2, 12, [0, 65504, 34848, 33952, 41120, 37152, 34080, 33312, 36128, 45216, 32800, 65504, 0]],
        '厨' : [12, 13, 0, -2, 12, [0, 32736, 16384, 32544, 16416, 24544, 21024, 24096, 17696, 38048, 37920, 15968, 0]],
        '逗' : [12, 13, 0, -2, 12, [0, 4064, 16384, 10208, 1056, 1056, 59360, 8768, 8768, 10208, 20480, 36832, 0]],
        '吹' : [12, 13, 0, -2, 12, [1024, 1024, 63456, 38176, 39200, 37184, 37120, 37120, 37504, 62080, 1088, 6176, 0]],
        '垂' : [12, 13, 0, -2, 12, [0, 32704, 1024, 32704, 9344, 65504, 9344, 9344, 32704, 1024, 1024, 65504, 0]],
        '帥' : [12, 13, 0, -2, 12, [8448, 16640, 61696, 38880, 38176, 62752, 34080, 62752, 38176, 38176, 62816, 33024, 0]],
        '推' : [12, 13, 0, -2, 12, [512, 8768, 8768, 63456, 9344, 12256, 9344, 13440, 59360, 9344, 9344, 26592, 0]],
        '水' : [12, 13, 0, -2, 12, [1024, 1024, 1088, 62592, 5376, 5632, 9472, 9344, 17472, 33824, 1024, 7168, 0]],
        '炊' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 9504, 43296, 43328, 41216, 8448, 21120, 21120, 33856, 38944, 0]],
        '睡' : [12, 13, 0, -2, 12, [0, 4064, 57600, 45024, 58656, 49120, 42272, 58656, 45024, 41216, 57600, 4064, 0]],
        '粋' : [12, 13, 0, -2, 12, [512, 8704, 44992, 41536, 8768, 62560, 9472, 24832, 32736, 41216, 41216, 8448, 0]],
        '翠' : [12, 13, 0, -2, 12, [0, 64480, 18720, 10400, 18720, 35360, 32704, 8320, 21824, 1024, 65504, 1024, 0]],
        '衰' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 0, 16256, 8320, 65504, 8320, 16288, 4672, 57728, 15456, 0]],
        '遂' : [12, 13, 0, -2, 12, [1088, 1152, 40928, 17664, 6432, 1728, 63872, 9920, 14496, 8960, 20480, 36832, 0]],
        '酔' : [12, 13, 0, -2, 12, [256, 63744, 10176, 8512, 63808, 43616, 43648, 55424, 36832, 63616, 34944, 63616, 0]],
        '錐' : [12, 13, 0, -2, 12, [512, 12864, 19008, 34784, 62592, 10208, 64640, 9344, 47072, 42112, 9344, 65504, 0]],
        '錘' : [12, 13, 0, -2, 12, [0, 14304, 16640, 36832, 62752, 12256, 62752, 9504, 45024, 41216, 8448, 65504, 0]],
        '随' : [12, 13, 0, -2, 12, [256, 57600, 47072, 43264, 50144, 48672, 44000, 43552, 44000, 51744, 37888, 41952, 0]],
        '瑞' : [12, 13, 0, -2, 12, [256, 2336, 59680, 20448, 16384, 24544, 57856, 20448, 19104, 19104, 60064, 2144, 0]],
        '髄' : [12, 13, 0, -2, 12, [128, 28800, 23520, 29824, 20960, 64288, 36320, 28960, 24032, 29984, 22016, 23008, 0]],
        '崇' : [12, 13, 0, -2, 12, [1024, 17472, 32704, 1024, 65504, 32800, 16256, 0, 65504, 1024, 17472, 35872, 0]],
        '嵩' : [12, 13, 0, -2, 12, [1024, 17472, 32704, 1024, 65504, 4352, 7936, 0, 65504, 37152, 40736, 32864, 0]],
        '数' : [12, 13, 0, -2, 12, [8448, 42240, 27104, 65088, 8768, 26688, 42304, 63808, 18560, 51328, 12608, 52768, 0]],
        '枢' : [12, 13, 0, -2, 12, [0, 10208, 9216, 62528, 9792, 9536, 25728, 25728, 46400, 42528, 9216, 10208, 0]],
        '趨' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 9504, 10144, 63808, 10208, 15648, 42912, 41248, 62016, 36832, 0]],
        '雛' : [12, 13, 0, -2, 12, [16640, 16704, 31040, 44000, 64064, 12256, 19008, 29248, 44000, 64064, 19008, 39904, 0]],
        '据' : [12, 13, 0, -2, 12, [0, 20448, 18464, 61408, 18560, 20448, 18560, 26752, 53216, 21536, 21536, 51168, 0]],
        '杉' : [12, 13, 0, -2, 12, [0, 8288, 9088, 63488, 8192, 8288, 25472, 28672, 43008, 40960, 8288, 9088, 0]],
        '椙' : [12, 13, 0, -2, 12, [0, 10176, 9280, 63424, 9280, 10176, 24576, 28640, 43040, 45024, 10272, 12256, 0]],
        '菅' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 5376, 1024, 65504, 37152, 40736, 4096, 8064, 4224, 8064, 0]],
        '頗' : [12, 13, 0, -2, 12, [8192, 9184, 63616, 44000, 41504, 64480, 35360, 44000, 43552, 37856, 12864, 52256, 0]],
        '雀' : [12, 13, 0, -2, 12, [1024, 1024, 9344, 19520, 33568, 7168, 57856, 16352, 8704, 16320, 8704, 16352, 0]],
        '裾' : [12, 13, 0, -2, 12, [0, 18400, 17440, 63456, 5248, 6112, 9344, 25728, 55264, 18976, 18976, 17376, 0]],
        '澄' : [12, 13, 0, -2, 12, [0, 7808, 33440, 21152, 3136, 34752, 18464, 6080, 17472, 18368, 33920, 40928, 0]],
        '摺' : [12, 13, 0, -2, 12, [0, 24288, 19104, 58976, 19104, 21344, 16896, 28640, 51232, 20448, 18464, 53216, 0]],
        '寸' : [12, 13, 0, -2, 12, [256, 256, 256, 65504, 256, 256, 8448, 4352, 256, 256, 256, 1792, 0]],
        '世' : [12, 13, 0, -2, 12, [1024, 9344, 9344, 9344, 65504, 9344, 9344, 9344, 10112, 8192, 8192, 16320, 0]],
        '瀬' : [12, 13, 0, -2, 12, [2048, 3040, 48256, 18912, 16160, 43488, 32544, 2528, 19744, 23520, 43328, 36384, 0]],
        '畝' : [12, 13, 0, -2, 12, [8704, 8704, 9184, 65056, 544, 64544, 42048, 42048, 64640, 42304, 42528, 64544, 0]],
        '是' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 16256, 0, 65504, 1024, 10176, 15360, 50144, 0]],
        '凄' : [12, 13, 0, -2, 12, [512, 512, 40928, 16960, 8160, 576, 4032, 512, 24544, 17472, 34688, 39008, 0]],
        '制' : [12, 13, 0, -2, 12, [8192, 40992, 64800, 41248, 8480, 64800, 8480, 64800, 42272, 42016, 44064, 8288, 0]],
        '勢' : [12, 13, 0, -2, 12, [8448, 8448, 65472, 8512, 64832, 19008, 62752, 9248, 65472, 2112, 4160, 24960, 0]],
        '姓' : [12, 13, 0, -2, 12, [128, 17536, 17536, 63456, 21632, 20608, 20608, 38880, 57472, 12416, 16512, 20448, 0]],
        '征' : [12, 13, 0, -2, 12, [0, 12256, 16640, 33024, 256, 10496, 18912, 51456, 18688, 18688, 18688, 24544, 0]],
        '性' : [12, 13, 0, -2, 12, [128, 9344, 9344, 10208, 43136, 43136, 41088, 42976, 41088, 8320, 8320, 12256, 0]],
        '成' : [12, 13, 0, -2, 12, [256, 288, 32736, 16640, 16672, 31008, 18752, 18752, 18560, 22688, 33632, 35872, 0]],
        '政' : [12, 13, 0, -2, 12, [512, 512, 64480, 8768, 9280, 9280, 47424, 41280, 41088, 41088, 62272, 3104, 0]],
        '整' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 8768, 64832, 43392, 25152, 49120, 1024, 10112, 9216, 65504, 0]],
        '星' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 16256, 9216, 32704, 33792, 16256, 1024, 65504, 0]],
        '晴' : [12, 13, 0, -2, 12, [128, 128, 63456, 36992, 38880, 36992, 65504, 37920, 38880, 37920, 63456, 1056, 0]],
        '棲' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61728, 12256, 8480, 26592, 25088, 49120, 42048, 10112, 14432, 0]],
        '栖' : [12, 13, 0, -2, 12, [0, 12256, 8832, 62080, 12256, 10912, 27296, 27296, 44256, 43040, 10272, 12256, 0]],
        '正' : [12, 13, 0, -2, 12, [0, 65504, 1024, 1024, 1024, 1024, 10176, 9216, 9216, 9216, 9216, 65504, 0]],
        '清' : [12, 13, 0, -2, 12, [256, 256, 40928, 16640, 4064, 33024, 24544, 2080, 4064, 10272, 20448, 34848, 0]],
        '牲' : [12, 13, 0, -2, 12, [128, 42112, 42112, 63456, 42112, 41088, 8320, 14304, 57472, 8320, 8320, 12256, 0]],
        '生' : [12, 13, 0, -2, 12, [1024, 17408, 17408, 32704, 17408, 33792, 33792, 16320, 1024, 1024, 1024, 65504, 0]],
        '盛' : [12, 13, 0, -2, 12, [512, 576, 32736, 16640, 31040, 18560, 39232, 33312, 32704, 19008, 19008, 65504, 0]],
        '精' : [12, 13, 0, -2, 12, [128, 8320, 47072, 41088, 10208, 61568, 12256, 25632, 26592, 46112, 42976, 9248, 0]],
        '聖' : [12, 13, 0, -2, 12, [0, 65504, 18976, 31264, 18976, 31712, 51200, 16320, 1024, 16256, 1024, 65504, 0]],
        '声' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 0, 32704, 17472, 17472, 32704, 32768, 32768, 0]],
        '製' : [12, 13, 0, -2, 12, [4096, 37152, 64800, 37152, 64800, 37984, 4096, 65504, 5184, 57920, 8576, 31840, 0]],
        '西' : [12, 13, 0, -2, 12, [0, 65504, 4608, 4608, 65504, 37408, 37408, 41440, 49184, 32800, 32800, 65504, 0]],
        '誠' : [12, 13, 0, -2, 12, [128, 57504, 4064, 59520, 2176, 61088, 2720, 2720, 59968, 46656, 41120, 58144, 0]],
        '誓' : [12, 13, 0, -2, 12, [8192, 10208, 62464, 10208, 13440, 58496, 10368, 65504, 0, 32704, 8320, 16256, 0]],
        '請' : [12, 13, 0, -2, 12, [128, 57472, 2016, 57472, 2016, 57472, 4064, 1056, 59360, 42016, 42976, 58400, 0]],
        '逝' : [12, 13, 0, -2, 12, [2048, 2528, 40192, 18688, 2528, 3392, 55616, 19008, 19008, 22592, 40960, 40928, 0]],
        '醒' : [12, 13, 0, -2, 12, [0, 64480, 8736, 9184, 64032, 44000, 43648, 56288, 35968, 64480, 34944, 64480, 0]],
        '青' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 16256, 1024, 65504, 8320, 16256, 8320, 16256, 8320, 0]],
        '静' : [12, 13, 0, -2, 12, [8192, 9152, 64576, 8320, 64480, 8352, 65504, 34976, 64480, 34944, 63616, 35200, 0]],
        '斉' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 8320, 4352, 7936, 57568, 16256, 8320, 16256, 16512, 16512, 0]],
        '税' : [12, 13, 0, -2, 12, [1088, 6720, 57472, 10208, 9248, 64544, 10208, 25216, 29312, 41600, 42144, 14560, 0]],
        '脆' : [12, 13, 0, -2, 12, [0, 62400, 37952, 39040, 63456, 37888, 38368, 62752, 38240, 39168, 37152, 45536, 0]],
        '隻' : [12, 13, 0, -2, 12, [4352, 4352, 16320, 8704, 32704, 41472, 16320, 8704, 32704, 4224, 3840, 61664, 0]],
        '席' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 32736, 18560, 20352, 16896, 24512, 37440, 37440, 4800, 0]],
        '惜' : [12, 13, 0, -2, 12, [576, 8768, 12256, 8768, 41536, 45024, 40960, 42976, 42016, 10208, 9248, 10208, 0]],
        '戚' : [12, 13, 0, -2, 12, [256, 288, 32736, 18560, 20096, 18592, 32672, 18592, 23104, 43584, 34976, 6944, 0]],
        '斥' : [12, 13, 0, -2, 12, [0, 448, 32256, 16384, 16384, 32736, 16896, 19968, 17280, 33376, 33280, 512, 0]],
        '昔' : [12, 13, 0, -2, 12, [4352, 4352, 32704, 4352, 4352, 65504, 0, 16256, 8320, 16256, 8320, 16256, 0]],
        '析' : [12, 13, 0, -2, 12, [0, 8384, 9984, 62464, 9216, 10208, 25664, 25664, 42048, 43072, 10304, 8256, 0]],
        '石' : [12, 13, 0, -2, 12, [0, 65504, 1024, 1024, 1024, 2048, 8128, 12352, 53312, 4160, 4160, 8128, 0]],
        '積' : [12, 13, 0, -2, 12, [256, 6400, 61408, 8448, 12256, 57600, 16352, 25664, 26560, 42048, 42944, 14432, 0]],
        '籍' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 8768, 63456, 8768, 65504, 9248, 30688, 44064, 10208, 0]],
        '績' : [12, 13, 0, -2, 12, [256, 8448, 20448, 33024, 61408, 8448, 20448, 62528, 10176, 42048, 42944, 14432, 0]],
        '脊' : [12, 13, 0, -2, 12, [1024, 1024, 62944, 2560, 61920, 7936, 8320, 65504, 8320, 16256, 8320, 8576, 0]],
        '責' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 16256, 1024, 65504, 8320, 16256, 8320, 16256, 49248, 0]],
        '赤' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 1024, 65504, 2304, 2304, 18752, 35104, 4352, 25344, 0]],
        '跡' : [12, 13, 0, -2, 12, [256, 61696, 37120, 40928, 62080, 8832, 10912, 43680, 45728, 41600, 42112, 63872, 0]],
        '蹟' : [12, 13, 0, -2, 12, [256, 61696, 40928, 37120, 65504, 8448, 12256, 42048, 47040, 42048, 42944, 63584, 0]],
        '碩' : [12, 13, 0, -2, 12, [0, 4064, 61696, 10208, 9248, 10208, 29728, 55264, 21536, 22496, 21056, 31776, 0]],
        '切' : [12, 13, 0, -2, 12, [0, 20448, 16928, 16928, 61984, 16928, 16928, 17440, 17440, 14368, 2112, 384, 0]],
        '拙' : [12, 13, 0, -2, 12, [256, 16640, 18720, 59680, 18720, 20448, 16640, 24832, 51488, 18720, 18720, 53216, 0]],
        '接' : [12, 13, 0, -2, 12, [256, 16640, 20448, 58432, 17472, 24544, 16896, 24544, 58432, 19520, 17280, 52320, 0]],
        '摂' : [12, 13, 0, -2, 12, [0, 20448, 17472, 59328, 17472, 17472, 20448, 24640, 50240, 17024, 17472, 51232, 0]],
        '折' : [12, 13, 0, -2, 12, [0, 8384, 9984, 62464, 9216, 10208, 9280, 13376, 58432, 10304, 10304, 24640, 0]],
        '設' : [12, 13, 0, -2, 12, [0, 59264, 1152, 58496, 1184, 59616, 0, 4032, 58432, 42112, 41856, 60512, 0]],
        '窃' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 25024, 0, 20448, 61984, 16928, 17440, 13504, 0]],
        '節' : [12, 13, 0, -2, 12, [16896, 16896, 31712, 42112, 63968, 35104, 63776, 35104, 63776, 37152, 35168, 62720, 0]],
        '説' : [12, 13, 0, -2, 12, [1088, 62016, 128, 63456, 1056, 62496, 2016, 640, 62080, 37504, 38048, 63712, 0]],
        '雪' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 1024, 32704, 64, 32704, 64, 32704, 0]],
        '絶' : [12, 13, 0, -2, 12, [0, 9152, 17472, 36992, 63456, 9376, 17568, 63456, 9216, 46080, 42016, 10208, 0]],
        '舌' : [12, 13, 0, -2, 12, [0, 960, 31744, 1024, 1024, 65504, 1024, 1024, 16256, 8320, 8320, 16256, 0]],
        '蝉' : [12, 13, 0, -2, 12, [544, 10560, 9344, 65504, 42144, 42976, 42144, 64672, 10208, 10368, 12256, 63616, 0]],
        '仙' : [12, 13, 0, -2, 12, [256, 8448, 8448, 16640, 18720, 51488, 18720, 18720, 18720, 18720, 18720, 20448, 0]],
        '先' : [12, 13, 0, -2, 12, [1024, 9216, 9216, 16320, 17408, 33792, 65504, 2304, 2304, 2304, 4384, 57824, 0]],
        '千' : [12, 13, 0, -2, 12, [0, 960, 31744, 1024, 1024, 1024, 65504, 1024, 1024, 1024, 1024, 1024, 0]],
        '占' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1984, 1024, 1024, 1024, 32704, 16448, 16448, 16448, 32704, 0]],
        '宣' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 49056, 8320, 16256, 8320, 8320, 16256, 0, 65504, 0]],
        '専' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 16256, 9344, 16256, 9344, 65504, 128, 8320, 4992, 0]],
        '尖' : [12, 13, 0, -2, 12, [1024, 1024, 9344, 17472, 35872, 0, 1024, 65504, 1024, 2560, 12672, 49248, 0]],
        '川' : [12, 13, 0, -2, 12, [0, 16448, 17472, 17472, 17472, 17472, 17472, 17472, 17472, 33856, 32832, 32832, 0]],
        '戦' : [12, 13, 0, -2, 12, [8448, 38176, 18720, 63744, 45024, 63744, 43328, 43328, 63616, 8320, 63808, 9760, 0]],
        '扇' : [12, 13, 0, -2, 12, [0, 65504, 0, 32704, 16448, 32704, 16384, 32224, 17440, 38048, 42272, 3168, 0]],
        '撰' : [12, 13, 0, -2, 12, [0, 20192, 16928, 61152, 18560, 20192, 17024, 28640, 49792, 24544, 17472, 55328, 0]],
        '栓' : [12, 13, 0, -2, 12, [0, 9088, 9280, 63520, 12256, 8448, 24832, 28640, 41216, 41216, 8448, 12256, 0]],
        '栴' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 63488, 10176, 9536, 25920, 25920, 45024, 42048, 9280, 9408, 0]],
        '泉' : [12, 13, 0, -2, 12, [1024, 2048, 16256, 8320, 16256, 8320, 16256, 1088, 63104, 5376, 9344, 52320, 0]],
        '浅' : [12, 13, 0, -2, 12, [512, 544, 40928, 16896, 4032, 33280, 24544, 288, 16704, 16768, 34432, 39008, 0]],
        '洗' : [12, 13, 0, -2, 12, [256, 2304, 35072, 20448, 4352, 37120, 24544, 1152, 17536, 17536, 34944, 45280, 0]],
        '染' : [12, 13, 0, -2, 12, [512, 16896, 12160, 33408, 17568, 10464, 17408, 65504, 3328, 13440, 50272, 1024, 0]],
        '潜' : [12, 13, 0, -2, 12, [1152, 1152, 40928, 17536, 8160, 35136, 21024, 4032, 18496, 20416, 34880, 36800, 0]],
        '煎' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 19008, 31296, 19008, 31296, 18496, 22720, 0, 21056, 35104, 0]],
        '煽' : [12, 13, 0, -2, 12, [0, 12256, 8192, 44992, 43072, 44992, 43008, 12000, 21024, 19104, 37664, 42592, 0]],
        '旋' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 62464, 18400, 18720, 28928, 21984, 21760, 38144, 38656, 14560, 0]],
        '穿' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 32704, 4224, 4224, 65504, 2176, 12416, 50048, 0]],
        '箭' : [12, 13, 0, -2, 12, [16896, 16896, 31712, 42112, 8320, 65504, 35392, 64064, 35392, 64064, 34880, 39104, 0]],
        '線' : [12, 13, 0, -2, 12, [128, 8448, 18400, 37920, 63456, 9248, 18400, 61568, 9888, 41664, 42144, 10656, 0]],
        '繊' : [12, 13, 0, -2, 12, [1152, 9376, 24224, 33920, 58848, 16000, 19104, 64160, 10816, 43840, 48288, 8992, 0]],
        '羨' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 32704, 1024, 65504, 17696, 43328, 16640, 8832, 52320, 0]],
        '腺' : [12, 13, 0, -2, 12, [128, 61696, 38880, 37920, 63456, 37920, 38880, 61568, 38560, 37568, 38048, 47520, 0]],
        '舛' : [12, 13, 0, -2, 12, [16448, 16448, 30784, 20448, 19008, 43584, 43584, 10816, 8160, 4160, 8256, 49216, 0]],
        '船' : [12, 13, 0, -2, 12, [4096, 8640, 30784, 19008, 27168, 19488, 63488, 19424, 27168, 18976, 35360, 39904, 0]],
        '薦' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 19008, 32736, 20480, 24544, 32800, 38176, 9568, 0]],
        '詮' : [12, 13, 0, -2, 12, [0, 58304, 1056, 59392, 4064, 57600, 256, 2016, 57600, 41216, 41216, 61408, 0]],
        '賎' : [12, 13, 0, -2, 12, [256, 61728, 40928, 61696, 38880, 61696, 40928, 37152, 61760, 384, 21120, 35936, 0]],
        '践' : [12, 13, 0, -2, 12, [256, 61728, 40928, 37120, 63456, 8448, 10208, 47392, 41280, 41344, 42624, 63584, 0]],
        '選' : [12, 13, 0, -2, 12, [0, 7904, 16928, 12000, 2176, 4064, 57984, 12256, 8832, 11360, 20480, 36832, 0]],
        '遷' : [12, 13, 0, -2, 12, [0, 4064, 17024, 12256, 2720, 4064, 58528, 11136, 12832, 9184, 20480, 36832, 0]],
        '銭' : [12, 13, 0, -2, 12, [256, 12576, 20448, 33024, 63456, 8448, 65504, 8480, 43328, 43392, 9856, 63584, 0]],
        '銑' : [12, 13, 0, -2, 12, [128, 13440, 17536, 34784, 62592, 8320, 65504, 8512, 43328, 43328, 8768, 64608, 0]],
        '閃' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 32800, 33824, 33824, 35360, 37152, 32864, 0]],
        '鮮' : [12, 13, 0, -2, 12, [1088, 29248, 36992, 10208, 63616, 43136, 64480, 43136, 63616, 2016, 43136, 43136, 0]],
        '前' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 0, 32032, 17696, 32032, 17696, 32032, 17696, 17440, 19552, 0]],
        '善' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 16256, 1024, 32704, 9344, 65504, 8320, 8320, 16256, 0]],
        '漸' : [12, 13, 0, -2, 12, [2048, 2272, 49024, 18560, 16096, 43680, 32416, 10912, 32416, 18592, 48928, 35104, 0]],
        '然' : [12, 13, 0, -2, 12, [256, 31040, 18720, 20416, 43264, 37120, 4736, 9280, 51232, 0, 21056, 35104, 0]],
        '全' : [12, 13, 0, -2, 12, [0, 3584, 4352, 8320, 16448, 49056, 1024, 1024, 16256, 1024, 1024, 65504, 0]],
        '禅' : [12, 13, 0, -2, 12, [256, 9376, 8768, 63456, 5280, 6112, 9376, 25760, 47072, 8320, 12256, 8320, 0]],
        '繕' : [12, 13, 0, -2, 12, [1088, 9344, 20448, 33024, 61408, 8448, 20448, 58688, 16352, 42048, 42048, 10176, 0]],
        '膳' : [12, 13, 0, -2, 12, [1088, 58496, 45024, 41216, 61408, 41216, 45024, 58688, 49120, 42048, 42048, 42944, 0]],
        '糎' : [12, 13, 0, -2, 12, [0, 12256, 43008, 44000, 10912, 64480, 10912, 27616, 26752, 46048, 41088, 10208, 0]],
        '噌' : [12, 13, 0, -2, 12, [1088, 1152, 65504, 37152, 40928, 37152, 40928, 37952, 38848, 37952, 62528, 1984, 0]],
        '塑' : [12, 13, 0, -2, 12, [4096, 21472, 64032, 9184, 43552, 64480, 8736, 17504, 1024, 16256, 1024, 65504, 0]],
        '岨' : [12, 13, 0, -2, 12, [0, 10208, 9248, 9248, 10208, 42016, 42016, 42976, 42016, 62496, 1056, 4064, 0]],
        '措' : [12, 13, 0, -2, 12, [576, 8768, 10208, 62016, 8768, 12256, 8192, 14304, 58400, 10208, 9248, 26592, 0]],
        '曾' : [12, 13, 0, -2, 12, [0, 3968, 8256, 32736, 54432, 17696, 32736, 8256, 16320, 8256, 8256, 16320, 0]],
        '曽' : [12, 13, 0, -2, 12, [4224, 4352, 32736, 17440, 32736, 17440, 32736, 8256, 16320, 8256, 8256, 16320, 0]],
        '楚' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8576, 29376, 42144, 0, 65504, 1056, 10112, 15360, 50144, 0]],
        '狙' : [12, 13, 0, -2, 12, [0, 10208, 42016, 17440, 18400, 42016, 9248, 26592, 42016, 9248, 9248, 53216, 0]],
        '疏' : [12, 13, 0, -2, 12, [256, 61696, 8160, 4672, 9280, 42912, 47136, 42304, 42304, 42304, 43328, 61536, 0]],
        '疎' : [12, 13, 0, -2, 12, [256, 61696, 8160, 4352, 12256, 10528, 47392, 45024, 41216, 41856, 44352, 61728, 0]],
        '礎' : [12, 13, 0, -2, 12, [1088, 1088, 65504, 9408, 11616, 9280, 32736, 53536, 21952, 21760, 22272, 30944, 0]],
        '祖' : [12, 13, 0, -2, 12, [0, 9184, 8736, 64032, 3040, 4640, 8736, 29664, 43552, 8736, 8736, 10208, 0]],
        '租' : [12, 13, 0, -2, 12, [0, 7136, 57888, 8736, 9184, 64032, 8736, 25568, 29216, 43552, 41504, 10208, 0]],
        '粗' : [12, 13, 0, -2, 12, [0, 9184, 43552, 43552, 9184, 64032, 8736, 25568, 29216, 43552, 41504, 10208, 0]],
        '素' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 16256, 1024, 65504, 4608, 9280, 65504, 9344, 50272, 0]],
        '組' : [12, 13, 0, -2, 12, [0, 9184, 16928, 37408, 62432, 8736, 18976, 64480, 8736, 43552, 43552, 10208, 0]],
        '蘇' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4160, 64384, 43136, 65504, 43136, 63872, 704, 43168, 43136, 0]],
        '訴' : [12, 13, 0, -2, 12, [0, 61632, 1792, 62464, 1024, 63456, 1152, 1408, 62656, 39072, 36992, 61568, 0]],
        '阻' : [12, 13, 0, -2, 12, [0, 63456, 37920, 42016, 51168, 42016, 37920, 38880, 37920, 58400, 33824, 36832, 0]],
        '遡' : [12, 13, 0, -2, 12, [1024, 5600, 40736, 17888, 5408, 7968, 58848, 10528, 10784, 8800, 20480, 36832, 0]],
        '鼠' : [12, 13, 0, -2, 12, [4096, 25536, 16448, 29632, 16448, 32704, 17472, 30528, 17472, 17472, 30624, 52256, 0]],
        '僧' : [12, 13, 0, -2, 12, [1088, 9344, 16352, 20768, 24544, 53536, 24544, 18496, 20416, 18496, 18496, 20416, 0]],
        '創' : [12, 13, 0, -2, 12, [0, 14368, 17696, 64800, 17696, 32032, 17696, 32032, 16672, 48160, 42016, 15456, 0]],
        '双' : [12, 13, 0, -2, 12, [0, 64480, 2080, 2080, 18976, 10784, 4416, 6464, 9344, 16768, 34368, 6176, 0]],
        '叢' : [12, 13, 0, -2, 12, [2560, 10880, 65504, 4352, 32704, 1024, 65504, 18496, 31296, 19072, 31104, 52832, 0]],
        '倉' : [12, 13, 0, -2, 12, [0, 3840, 4224, 16320, 24672, 49088, 8256, 16320, 8192, 24512, 20544, 8128, 0]],
        '喪' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 21824, 21824, 30144, 1024, 65504, 9280, 8768, 8576, 63584, 0]],
        '壮' : [12, 13, 0, -2, 12, [256, 8448, 8448, 41216, 28640, 8448, 8448, 8448, 24832, 41216, 8448, 12256, 0]],
        '奏' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2048, 16256, 4352, 65504, 9344, 32704, 35360, 4352, 24768, 0]],
        '爽' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 21824, 9344, 21824, 9344, 17472, 2560, 12672, 49248, 0]],
        '宋' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 33824, 1024, 65504, 1024, 3328, 13440, 50272, 1024, 0]],
        '層' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 20768, 24544, 20768, 24544, 18496, 36800, 34880, 4032, 0]],
        '匝' : [12, 13, 0, -2, 12, [0, 65504, 33280, 33280, 40896, 37440, 37440, 37440, 37568, 33280, 32768, 65504, 0]],
        '惣' : [12, 13, 0, -2, 12, [1024, 41984, 45024, 62112, 42144, 14624, 57888, 8384, 1024, 4672, 20512, 36736, 0]],
        '想' : [12, 13, 0, -2, 12, [8192, 9184, 64032, 9184, 12832, 27168, 41952, 8192, 1024, 4672, 20512, 36736, 0]],
        '捜' : [12, 13, 0, -2, 12, [256, 16640, 24544, 61728, 24544, 20768, 24544, 24832, 57312, 17472, 17280, 56416, 0]],
        '掃' : [12, 13, 0, -2, 12, [0, 20416, 16448, 61376, 16448, 24544, 20768, 28640, 51520, 18752, 18880, 49408, 0]],
        '挿' : [12, 13, 0, -2, 12, [0, 20448, 16640, 57312, 16640, 20448, 18720, 28640, 51488, 20448, 16640, 49408, 0]],
        '掻' : [12, 13, 0, -2, 12, [0, 20416, 16448, 58496, 17280, 23648, 16640, 28640, 51488, 20416, 16672, 57312, 0]],
        '操' : [12, 13, 0, -2, 12, [0, 18368, 17472, 61408, 19104, 19104, 20192, 24832, 53216, 17152, 17728, 55584, 0]],
        '早' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 16448, 32704, 1024, 1024, 65504, 1024, 1024, 0]],
        '曹' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 19008, 32704, 19008, 32704, 8320, 16256, 8320, 8320, 16256, 0]],
        '巣' : [12, 13, 0, -2, 12, [2112, 17536, 8448, 16256, 9344, 16256, 9344, 9344, 65504, 3328, 13440, 50272, 0]],
        '槍' : [12, 13, 0, -2, 12, [0, 9152, 9248, 65504, 9248, 10208, 25632, 26592, 41984, 44000, 10784, 9184, 0]],
        '槽' : [12, 13, 0, -2, 12, [1152, 9344, 16352, 58528, 16352, 9376, 32736, 26688, 44992, 43072, 10304, 12224, 0]],
        '漕' : [12, 13, 0, -2, 12, [640, 640, 49120, 21152, 8160, 37536, 24544, 2112, 4032, 10304, 18496, 36800, 0]],
        '燥' : [12, 13, 0, -2, 12, [0, 10176, 9280, 12256, 43680, 43680, 44768, 8448, 24544, 17152, 34112, 39200, 0]],
        '争' : [12, 13, 0, -2, 12, [0, 8064, 8448, 16896, 32704, 1088, 65504, 1088, 32704, 1024, 1024, 7168, 0]],
        '痩' : [12, 13, 0, -2, 12, [512, 512, 32736, 20768, 57312, 20768, 24544, 49408, 24544, 17472, 33664, 40032, 0]],
        '相' : [12, 13, 0, -2, 12, [0, 10208, 9248, 62496, 10208, 9248, 25632, 26592, 46112, 42016, 9248, 10208, 0]],
        '窓' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37408, 41504, 18880, 4096, 16256, 1024, 4672, 20512, 36736, 0]],
        '糟' : [12, 13, 0, -2, 12, [1152, 9344, 49120, 42144, 16352, 58528, 16352, 26688, 28608, 43072, 43072, 12224, 0]],
        '総' : [12, 13, 0, -2, 12, [0, 9664, 17472, 34880, 59936, 9248, 18304, 61440, 8704, 43328, 43040, 10112, 0]],
        '綜' : [12, 13, 0, -2, 12, [256, 8448, 20448, 34848, 59328, 8192, 16384, 61408, 8448, 42304, 43296, 8960, 0]],
        '聡' : [12, 13, 0, -2, 12, [0, 62912, 42048, 59456, 43552, 42016, 59264, 40960, 41472, 59712, 43040, 10112, 0]],
        '草' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 8320, 16256, 8320, 16256, 1024, 65504, 1024, 0]],
        '荘' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 256, 8448, 41216, 28640, 8448, 8448, 24832, 41216, 12256, 0]],
        '葬' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 19072, 12832, 50144, 4352, 65504, 4352, 24832, 0]],
        '蒼' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 24768, 49056, 8320, 16256, 20608, 20608, 8064, 0]],
        '藻' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 5184, 20448, 10912, 36576, 16640, 4064, 8960, 17728, 35104, 0]],
        '装' : [12, 13, 0, -2, 12, [256, 37120, 22496, 4352, 20736, 38880, 0, 65504, 5184, 57920, 8576, 31840, 0]],
        '走' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 1024, 65504, 1024, 9216, 10176, 9216, 15360, 50144, 0]],
        '送' : [12, 13, 0, -2, 12, [2112, 1088, 16512, 12256, 256, 256, 61408, 8448, 8768, 11296, 20480, 36832, 0]],
        '遭' : [12, 13, 0, -2, 12, [640, 640, 40928, 19104, 4064, 2720, 61408, 9280, 9280, 10176, 20480, 36832, 0]],
        '鎗' : [12, 13, 0, -2, 12, [0, 13248, 17440, 36832, 62496, 10208, 62496, 10208, 43008, 44000, 8736, 64480, 0]],
        '霜' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 9216, 64480, 8736, 29664, 43552, 9184, 0]],
        '騒' : [12, 13, 0, -2, 12, [0, 64480, 40992, 63808, 41344, 63072, 41088, 64480, 2720, 43968, 2208, 14304, 0]],
        '像' : [12, 13, 0, -2, 12, [0, 10176, 10368, 24544, 18720, 51488, 20448, 17536, 22944, 18112, 22688, 17152, 0]],
        '増' : [12, 13, 0, -2, 12, [1088, 17536, 24544, 20768, 65504, 20768, 24544, 18496, 20416, 18496, 59456, 4032, 0]],
        '憎' : [12, 13, 0, -2, 12, [2112, 10368, 16352, 8736, 49120, 41504, 49120, 43072, 44992, 10304, 10304, 12224, 0]],
        '臓' : [12, 13, 0, -2, 12, [1152, 58496, 49120, 42048, 65504, 43648, 44960, 59552, 44992, 43584, 44960, 41248, 0]],
        '蔵' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4384, 32736, 21632, 24224, 21152, 24128, 21568, 40608, 33056, 0]],
        '贈' : [12, 13, 0, -2, 12, [1088, 62592, 40928, 61728, 40928, 61728, 40928, 37952, 63424, 1088, 21568, 38848, 0]],
        '造' : [12, 13, 0, -2, 12, [256, 2304, 36800, 18688, 256, 8160, 57344, 10176, 9280, 10176, 20480, 36832, 0]],
        '促' : [12, 13, 0, -2, 12, [0, 12256, 10272, 18464, 20448, 49408, 16640, 18912, 18688, 18688, 20224, 28896, 0]],
        '側' : [12, 13, 0, -2, 12, [0, 7712, 12960, 24224, 53920, 24224, 21152, 21152, 24224, 16416, 21024, 24928, 0]],
        '則' : [12, 13, 0, -2, 12, [0, 63520, 35104, 63776, 35104, 63776, 35104, 35104, 63776, 32, 18464, 33888, 0]],
        '即' : [12, 13, 0, -2, 12, [0, 63968, 35104, 35104, 63776, 35104, 35104, 63776, 33056, 37152, 35168, 62720, 0]],
        '息' : [12, 13, 0, -2, 12, [1024, 2048, 16256, 8320, 16256, 8320, 8320, 16256, 1024, 4672, 20512, 36736, 0]],
        '捉' : [12, 13, 0, -2, 12, [0, 20448, 18464, 59424, 20448, 16640, 16640, 27104, 51456, 18688, 20224, 61664, 0]],
        '束' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 17472, 17472, 32704, 3072, 5376, 9344, 50272, 0]],
        '測' : [12, 13, 0, -2, 12, [0, 7712, 37536, 24224, 4768, 40608, 21152, 4768, 24224, 16416, 37408, 41312, 0]],
        '足' : [12, 13, 0, -2, 12, [0, 16320, 8256, 8256, 16320, 1024, 1024, 10176, 9216, 9216, 15360, 50144, 0]],
        '速' : [12, 13, 0, -2, 12, [256, 256, 20448, 8448, 4064, 2336, 61408, 8960, 9536, 10528, 20480, 36832, 0]],
        '俗' : [12, 13, 0, -2, 12, [0, 9280, 10272, 16384, 16768, 49728, 17440, 18432, 18400, 17440, 17440, 18400, 0]],
        '属' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 21056, 21056, 24512, 16896, 49120, 41504, 12192, 8288, 0]],
        '賊' : [12, 13, 0, -2, 12, [128, 61600, 40928, 61568, 37504, 62112, 40864, 37536, 62016, 576, 20640, 35616, 0]],
        '族' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 64000, 19424, 29312, 21632, 22496, 36992, 36992, 4416, 26144, 0]],
        '続' : [12, 13, 0, -2, 12, [256, 8448, 20448, 33024, 59360, 8192, 20448, 63520, 8832, 41600, 42144, 14560, 0]],
        '卒' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 4352, 10880, 17472, 1024, 65504, 1024, 1024, 1024, 0]],
        '袖' : [12, 13, 0, -2, 12, [256, 16640, 16640, 61408, 10528, 10528, 18720, 57312, 26912, 18720, 18720, 20448, 0]],
        '其' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 7936, 4352, 7936, 4352, 65504, 0, 12672, 49248, 0]],
        '揃' : [12, 13, 0, -2, 12, [1088, 17536, 24544, 57344, 20000, 19104, 20128, 27296, 52896, 19104, 18976, 51808, 0]],
        '存' : [12, 13, 0, -2, 12, [2048, 2048, 65504, 4096, 6080, 8256, 16512, 49408, 24544, 16640, 16640, 17152, 0]],
        '孫' : [12, 13, 0, -2, 12, [0, 61408, 8448, 8704, 17472, 17024, 24864, 53216, 16640, 18752, 20768, 49408, 0]],
        '尊' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 2560, 32704, 19008, 29632, 16448, 65504, 128, 8320, 4480, 0]],
        '損' : [12, 13, 0, -2, 12, [0, 10208, 9248, 63456, 8192, 10208, 9248, 14304, 58400, 10208, 8768, 27680, 0]],
        '村' : [12, 13, 0, -2, 12, [8256, 8256, 8256, 63456, 8256, 8256, 25664, 25152, 45120, 41024, 8256, 8640, 0]],
        '遜' : [12, 13, 0, -2, 12, [0, 8160, 33920, 17728, 2176, 3360, 56288, 18560, 19104, 23200, 40960, 40928, 0]],
        '他' : [12, 13, 0, -2, 12, [512, 8704, 10944, 19264, 20032, 55872, 19008, 19136, 18944, 18976, 18464, 20448, 0]],
        '多' : [12, 13, 0, -2, 12, [1024, 1024, 3968, 28800, 2304, 1536, 6624, 57888, 3104, 576, 384, 15872, 0]],
        '太' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 1024, 2560, 2560, 4352, 4352, 8320, 17472, 33312, 0]],
        '汰' : [12, 13, 0, -2, 12, [256, 256, 33024, 24544, 256, 33024, 17024, 640, 17472, 17472, 35360, 37152, 0]],
        '詑' : [12, 13, 0, -2, 12, [256, 57600, 4064, 59424, 2080, 58368, 1216, 1792, 58368, 41984, 42016, 59360, 0]],
        '唾' : [12, 13, 0, -2, 12, [0, 4064, 57600, 49120, 43296, 49120, 43296, 43296, 49120, 41216, 57600, 4064, 0]],
        '堕' : [12, 13, 0, -2, 12, [256, 63456, 37120, 41952, 38432, 37856, 57888, 33376, 1024, 16256, 1024, 65504, 0]],
        '妥' : [12, 13, 0, -2, 12, [0, 32704, 64, 18560, 9344, 0, 2048, 65504, 4224, 12544, 3840, 28864, 0]],
        '惰' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8704, 42976, 43136, 42976, 42016, 42976, 9248, 10208, 9248, 0]],
        '打' : [12, 13, 0, -2, 12, [0, 10208, 8256, 61504, 8256, 8256, 8256, 12352, 57408, 8256, 8256, 25024, 0]],
        '柁' : [12, 13, 0, -2, 12, [256, 8448, 12256, 63520, 10272, 9216, 25792, 26368, 46080, 41984, 9248, 10208, 0]],
        '舵' : [12, 13, 0, -2, 12, [4352, 8448, 32736, 18464, 27168, 18944, 64096, 19328, 27136, 18944, 35360, 39904, 0]],
        '楕' : [12, 13, 0, -2, 12, [512, 8704, 12256, 61952, 10208, 10368, 26592, 25632, 47072, 42016, 10208, 9248, 0]],
        '陀' : [12, 13, 0, -2, 12, [256, 57600, 45024, 43040, 51232, 41984, 42176, 42752, 41984, 50176, 33824, 34784, 0]],
        '駄' : [12, 13, 0, -2, 12, [256, 63744, 41216, 65504, 41216, 63744, 41600, 64128, 3136, 43072, 2592, 12576, 0]],
        '騨' : [12, 13, 0, -2, 12, [1312, 62784, 40960, 63456, 42144, 63456, 42144, 62624, 6112, 53376, 6112, 24704, 0]],
        '体' : [12, 13, 0, -2, 12, [256, 8448, 8448, 24544, 16640, 49920, 17792, 18752, 20768, 20416, 16640, 16640, 0]],
        '堆' : [12, 13, 0, -2, 12, [512, 8768, 8768, 10208, 62592, 12256, 9344, 9344, 10208, 13440, 50304, 2016, 0]],
        '対' : [12, 13, 0, -2, 12, [8256, 8256, 8256, 63456, 4160, 4160, 37952, 21056, 8256, 12352, 18496, 32960, 0]],
        '耐' : [12, 13, 0, -2, 12, [64, 63552, 8256, 18400, 63552, 43072, 43584, 43328, 43072, 43072, 43072, 35008, 0]],
        '岱' : [12, 13, 0, -2, 12, [512, 8768, 8736, 24512, 49408, 16640, 16512, 17504, 1024, 33824, 33824, 65504, 0]],
        '帯' : [12, 13, 0, -2, 12, [1024, 9344, 65504, 9344, 16256, 0, 65504, 33824, 49056, 9344, 9600, 1024, 0]],
        '待' : [12, 13, 0, -2, 12, [256, 8448, 18400, 33024, 256, 12256, 16448, 53216, 16448, 17472, 16960, 16576, 0]],
        '怠' : [12, 13, 0, -2, 12, [2048, 2304, 4224, 65472, 32, 16256, 8320, 16256, 1024, 4672, 20512, 36736, 0]],
        '態' : [12, 13, 0, -2, 12, [8704, 16992, 35712, 64032, 35808, 64000, 35808, 64000, 35808, 4608, 20544, 36768, 0]],
        '戴' : [12, 13, 0, -2, 12, [4352, 4416, 32032, 4352, 65504, 21120, 32416, 21152, 65088, 9280, 65184, 17696, 0]],
        '替' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8320, 65504, 8832, 17472, 49056, 8320, 16256, 8320, 16256, 0]],
        '泰' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2048, 16256, 4352, 65504, 8320, 21824, 35872, 5376, 25792, 0]],
        '滞' : [12, 13, 0, -2, 12, [256, 2336, 40928, 18720, 4064, 32768, 24544, 4384, 20448, 18720, 35168, 33024, 0]],
        '胎' : [12, 13, 0, -2, 12, [256, 61696, 37408, 37920, 63456, 36864, 36864, 63456, 37920, 37920, 37920, 47072, 0]],
        '腿' : [12, 13, 0, -2, 12, [0, 58336, 45600, 44000, 57888, 41952, 47744, 60064, 43584, 43808, 46080, 41952, 0]],
        '苔' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 2048, 4352, 8320, 32704, 32, 16256, 8320, 8320, 16256, 0]],
        '袋' : [12, 13, 0, -2, 12, [512, 8736, 24544, 49664, 16640, 17632, 1024, 65504, 5184, 57920, 8576, 31840, 0]],
        '貸' : [12, 13, 0, -2, 12, [512, 4640, 12256, 57600, 8320, 16352, 8256, 16320, 8256, 16320, 4160, 24608, 0]],
        '退' : [12, 13, 0, -2, 12, [0, 1984, 17472, 10176, 1088, 1984, 58656, 9376, 9280, 10016, 20480, 36832, 0]],
        '逮' : [12, 13, 0, -2, 12, [256, 256, 36832, 16672, 8160, 288, 61408, 9504, 9024, 11552, 20480, 36832, 0]],
        '隊' : [12, 13, 0, -2, 12, [1088, 57920, 41088, 45024, 49408, 41760, 44224, 41344, 41664, 56480, 32928, 33536, 0]],
        '黛' : [12, 13, 0, -2, 12, [512, 8736, 24544, 49408, 32640, 9440, 16256, 9344, 65504, 0, 21056, 35104, 0]],
        '鯛' : [12, 13, 0, -2, 12, [0, 28640, 43296, 19360, 63776, 43936, 63520, 43936, 64160, 672, 43936, 43104, 0]],
        '代' : [12, 13, 0, -2, 12, [512, 8768, 8736, 16896, 24544, 49408, 16640, 16640, 16512, 16512, 16448, 16448, 0]],
        '台' : [12, 13, 0, -2, 12, [4096, 4224, 8320, 8256, 65472, 32, 32, 16256, 8320, 8320, 8320, 16256, 0]],
        '大' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 1024, 1024, 1024, 2560, 2560, 4352, 8320, 49248, 0]],
        '第' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 32704, 1088, 32704, 17408, 32736, 5152, 9248, 50368, 0]],
        '醍' : [12, 13, 0, -2, 12, [0, 64480, 8736, 9184, 64032, 44000, 43008, 57312, 34944, 64224, 35712, 64608, 0]],
        '題' : [12, 13, 0, -2, 12, [0, 64480, 34944, 64480, 35360, 64480, 544, 64480, 8512, 47648, 57344, 40928, 0]],
        '鷹' : [12, 13, 0, -2, 12, [0, 32736, 19072, 22496, 29312, 21472, 17408, 24512, 36928, 40928, 32, 21856, 0]],
        '滝' : [12, 13, 0, -2, 12, [512, 512, 40928, 17536, 1152, 40928, 21056, 8128, 21056, 24512, 33312, 33760, 0]],
        '瀧' : [12, 13, 0, -2, 12, [1152, 1152, 48864, 19072, 2784, 48928, 21216, 7808, 21216, 24192, 37536, 38624, 0]],
        '卓' : [12, 13, 0, -2, 12, [1024, 1024, 1984, 1024, 16256, 8320, 16256, 8320, 16256, 1024, 65504, 1024, 0]],
        '啄' : [12, 13, 0, -2, 12, [0, 8160, 57600, 41728, 44192, 41408, 42624, 47232, 41408, 42656, 63648, 768, 0]],
        '宅' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 33696, 15360, 2048, 4032, 63488, 2048, 2080, 4064, 0]],
        '托' : [12, 13, 0, -2, 12, [0, 16608, 20224, 57856, 16896, 16896, 20448, 25088, 49664, 16896, 16928, 50144, 0]],
        '択' : [12, 13, 0, -2, 12, [0, 18400, 17440, 62496, 17440, 18400, 17536, 25728, 50240, 17472, 18464, 51232, 0]],
        '拓' : [12, 13, 0, -2, 12, [0, 20448, 16640, 61696, 16640, 16896, 18400, 23584, 58400, 17440, 17440, 51168, 0]],
        '沢' : [12, 13, 0, -2, 12, [0, 2016, 33824, 17440, 1056, 34784, 17536, 1152, 1088, 9280, 18464, 34848, 0]],
        '濯' : [12, 13, 0, -2, 12, [0, 3808, 33312, 20192, 544, 36576, 17536, 4064, 22656, 20448, 34944, 36832, 0]],
        '琢' : [12, 13, 0, -2, 12, [0, 8160, 57600, 17152, 19616, 16832, 59008, 22656, 16832, 18080, 63648, 768, 0]],
        '託' : [12, 13, 0, -2, 12, [0, 57568, 3840, 57856, 512, 57856, 4064, 512, 57856, 41504, 41504, 58336, 0]],
        '鐸' : [12, 13, 0, -2, 12, [0, 16352, 19104, 36832, 61568, 10208, 64064, 8768, 47072, 41088, 10208, 63616, 0]],
        '濁' : [12, 13, 0, -2, 12, [0, 8160, 38176, 24544, 2048, 36832, 21024, 8096, 19104, 20384, 33440, 40800, 0]],
        '諾' : [12, 13, 0, -2, 12, [576, 57920, 4064, 57920, 256, 61408, 256, 512, 59360, 43552, 41504, 58336, 0]],
        '茸' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 8320, 16256, 8320, 16256, 8416, 65408, 128, 0]],
        '凧' : [12, 13, 0, -2, 12, [0, 32704, 17472, 17472, 24384, 21824, 21824, 21824, 21824, 22304, 33824, 33824, 0]],
        '蛸' : [12, 13, 0, -2, 12, [128, 8320, 9376, 64192, 43136, 44000, 43552, 64480, 8736, 11232, 10784, 64096, 0]],
        '只' : [12, 13, 0, -2, 12, [0, 32704, 16448, 16448, 16448, 16448, 16448, 32704, 0, 8320, 16448, 32800, 0]],
        '叩' : [12, 13, 0, -2, 12, [0, 63456, 37920, 37920, 37920, 37920, 37920, 37920, 37920, 62688, 1024, 1024, 0]],
        '但' : [12, 13, 0, -2, 12, [0, 10208, 9248, 17440, 18400, 50208, 17440, 17440, 18400, 16384, 16384, 20448, 0]],
        '達' : [12, 13, 0, -2, 12, [256, 256, 20448, 8448, 4064, 576, 57920, 12256, 8448, 12256, 20736, 36832, 0]],
        '辰' : [12, 13, 0, -2, 12, [0, 32736, 16384, 24512, 16384, 16384, 32736, 20992, 21024, 37184, 36992, 15456, 0]],
        '奪' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 10368, 19008, 40864, 29184, 8064, 4608, 65504, 8320, 4480, 0]],
        '脱' : [12, 13, 0, -2, 12, [1088, 62016, 36992, 38880, 62496, 37920, 38880, 62080, 37504, 37504, 38048, 47328, 0]],
        '巽' : [12, 13, 0, -2, 12, [0, 31680, 2112, 31680, 16928, 31712, 4352, 65504, 4352, 65504, 8320, 49248, 0]],
        '竪' : [12, 13, 0, -2, 12, [0, 64480, 40992, 63776, 35136, 63872, 41536, 42016, 65472, 4352, 4352, 65504, 0]],
        '辿' : [12, 13, 0, -2, 12, [256, 256, 16640, 10528, 2336, 2336, 59680, 10528, 10528, 12256, 20480, 36832, 0]],
        '棚' : [12, 13, 0, -2, 12, [0, 12000, 10912, 64160, 12000, 10912, 27296, 27296, 44768, 43680, 10912, 10912, 0]],
        '谷' : [12, 13, 0, -2, 12, [0, 4352, 8320, 16448, 3584, 4352, 8320, 16448, 49056, 8320, 8320, 16256, 0]],
        '狸' : [12, 13, 0, -2, 12, [0, 12256, 43296, 20448, 18720, 43296, 12256, 24832, 45024, 8448, 8448, 53216, 0]],
        '鱈' : [12, 13, 0, -2, 12, [0, 30688, 37120, 12256, 63776, 43872, 63488, 45024, 63520, 992, 43040, 45024, 0]],
        '樽' : [12, 13, 0, -2, 12, [1088, 9344, 12256, 62080, 12256, 10912, 27872, 26656, 49120, 41024, 9280, 8896, 0]],
        '誰' : [12, 13, 0, -2, 12, [512, 57920, 576, 59360, 1152, 61408, 1152, 1152, 59360, 42112, 42112, 59360, 0]],
        '丹' : [12, 13, 0, -2, 12, [0, 16320, 8256, 9280, 8768, 8256, 65504, 8256, 8256, 8256, 16448, 16832, 0]],
        '単' : [12, 13, 0, -2, 12, [2112, 17536, 8448, 32704, 17472, 32704, 17472, 17472, 32704, 1024, 65504, 1024, 0]],
        '嘆' : [12, 13, 0, -2, 12, [1152, 1152, 65504, 42112, 45024, 43296, 45024, 41216, 45024, 41216, 59008, 6240, 0]],
        '坦' : [12, 13, 0, -2, 12, [0, 10208, 9248, 9248, 63456, 9248, 9248, 9248, 10208, 12288, 49152, 4064, 0]],
        '担' : [12, 13, 0, -2, 12, [0, 18400, 17440, 62496, 18400, 17440, 17440, 25632, 51168, 16384, 16384, 53216, 0]],
        '探' : [12, 13, 0, -2, 12, [0, 24544, 21152, 62080, 17536, 22624, 16640, 20448, 58112, 17728, 22816, 49408, 0]],
        '旦' : [12, 13, 0, -2, 12, [0, 16256, 8320, 8320, 16256, 8320, 8320, 8320, 16256, 0, 0, 65504, 0]],
        '歎' : [12, 13, 0, -2, 12, [512, 20992, 65504, 21152, 64160, 43680, 63616, 8320, 64832, 8512, 21024, 35872, 0]],
        '淡' : [12, 13, 0, -2, 12, [0, 2336, 35136, 17152, 1152, 39008, 16640, 2336, 18752, 17152, 33920, 39008, 0]],
        '湛' : [12, 13, 0, -2, 12, [1088, 1088, 40928, 17472, 1984, 33856, 17472, 8160, 19072, 19680, 34816, 36832, 0]],
        '炭' : [12, 13, 0, -2, 12, [512, 16928, 32736, 0, 32736, 16384, 21024, 21056, 16896, 34048, 34944, 12384, 0]],
        '短' : [12, 13, 0, -2, 12, [0, 34784, 63488, 40960, 10208, 64544, 9248, 10208, 21056, 21056, 36992, 34784, 0]],
        '端' : [12, 13, 0, -2, 12, [256, 18720, 18720, 65504, 8192, 45024, 41216, 45024, 43680, 10912, 64160, 2144, 0]],
        '箪' : [12, 13, 0, -2, 12, [16896, 16896, 31712, 42112, 32704, 17472, 32704, 17472, 32704, 1024, 65504, 1024, 0]],
        '綻' : [12, 13, 0, -2, 12, [256, 8448, 20448, 34848, 57344, 12256, 16640, 61696, 9696, 42240, 42752, 14560, 0]],
        '耽' : [12, 13, 0, -2, 12, [256, 63744, 38880, 62752, 38176, 37152, 61824, 37504, 37504, 62112, 38048, 6368, 0]],
        '胆' : [12, 13, 0, -2, 12, [0, 62432, 37408, 37408, 62432, 37408, 37408, 61984, 37856, 36864, 36864, 47072, 0]],
        '蛋' : [12, 13, 0, -2, 12, [0, 65504, 1056, 10112, 15360, 50144, 1024, 16256, 9344, 16256, 1056, 65504, 0]],
        '誕' : [12, 13, 0, -2, 12, [0, 57568, 7040, 59520, 2688, 60128, 4736, 6784, 60032, 44000, 44032, 62432, 0]],
        '鍛' : [12, 13, 0, -2, 12, [0, 16320, 18752, 36672, 63808, 10592, 65024, 10688, 44864, 43328, 10624, 65120, 0]],
        '団' : [12, 13, 0, -2, 12, [0, 65504, 33056, 33056, 49056, 33056, 37152, 35104, 33056, 33568, 32800, 65504, 0]],
        '壇' : [12, 13, 0, -2, 12, [256, 16640, 24544, 19104, 60320, 18464, 20448, 17472, 18368, 17472, 58432, 8160, 0]],
        '弾' : [12, 13, 0, -2, 12, [544, 59712, 9344, 12256, 59680, 36832, 35104, 59680, 12256, 8448, 12256, 49408, 0]],
        '断' : [12, 13, 0, -2, 12, [0, 37984, 56192, 45568, 37376, 65504, 37440, 47680, 54848, 37440, 64576, 1088, 0]],
        '暖' : [12, 13, 0, -2, 12, [0, 8160, 58656, 42304, 45024, 41472, 61408, 41472, 42944, 47680, 57728, 3680, 0]],
        '檀' : [12, 13, 0, -2, 12, [256, 8448, 16352, 60064, 11168, 10272, 28640, 25664, 47040, 42048, 9280, 12256, 0]],
        '段' : [12, 13, 0, -2, 12, [0, 14208, 50304, 33920, 62624, 35040, 61440, 36800, 33856, 62592, 33664, 40032, 0]],
        '男' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 32704, 1024, 65504, 1056, 2080, 12352, 50048, 0]],
        '談' : [12, 13, 0, -2, 12, [0, 59680, 2368, 57600, 640, 60512, 256, 2336, 59712, 41728, 42112, 63584, 0]],
        '値' : [12, 13, 0, -2, 12, [128, 8320, 12256, 16512, 17376, 51744, 19424, 18976, 18976, 19424, 18432, 20448, 0]],
        '知' : [12, 13, 0, -2, 12, [16384, 16384, 32224, 37152, 37152, 4384, 65312, 4384, 4384, 10528, 17888, 33280, 0]],
        '地' : [12, 13, 0, -2, 12, [256, 8448, 9568, 9632, 63264, 11552, 9504, 9568, 9472, 13312, 50208, 2016, 0]],
        '弛' : [12, 13, 0, -2, 12, [256, 57600, 9568, 9632, 59168, 40224, 34080, 58720, 9472, 9216, 9248, 51168, 0]],
        '恥' : [12, 13, 0, -2, 12, [0, 63872, 36960, 61440, 37376, 37440, 62016, 38432, 39456, 61952, 37408, 5088, 0]],
        '智' : [12, 13, 0, -2, 12, [8192, 8192, 32224, 37152, 32544, 10720, 17408, 49088, 8256, 16320, 8256, 16320, 0]],
        '池' : [12, 13, 0, -2, 12, [512, 512, 35520, 19264, 3648, 39488, 19008, 2752, 18944, 18464, 34848, 36832, 0]],
        '痴' : [12, 13, 0, -2, 12, [1024, 1024, 32736, 20480, 57056, 21664, 17568, 65440, 17568, 17568, 35488, 45536, 0]],
        '稚' : [12, 13, 0, -2, 12, [512, 6720, 57920, 10208, 9344, 65504, 9344, 25728, 26592, 46208, 42112, 10208, 0]],
        '置' : [12, 13, 0, -2, 12, [0, 32736, 18720, 32736, 512, 65504, 2176, 20352, 18560, 20352, 16384, 32736, 0]],
        '致' : [12, 13, 0, -2, 12, [512, 64000, 9184, 16960, 33856, 64576, 8512, 8512, 63616, 8320, 9024, 64544, 0]],
        '蜘' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 64160, 43680, 43680, 44960, 64160, 8864, 9120, 9376, 63712, 0]],
        '遅' : [12, 13, 0, -2, 12, [0, 4064, 18464, 12256, 2336, 2368, 61408, 12416, 10208, 8320, 20480, 36832, 0]],
        '馳' : [12, 13, 0, -2, 12, [256, 63744, 42336, 62880, 42784, 64800, 42272, 62816, 5376, 54272, 5152, 25568, 0]],
        '築' : [12, 13, 0, -2, 12, [512, 16896, 31712, 38016, 64448, 8768, 62816, 1024, 65504, 3328, 13440, 50272, 0]],
        '畜' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 8704, 5248, 2112, 65504, 17472, 32704, 17472, 32704, 0]],
        '竹' : [12, 13, 0, -2, 12, [256, 16640, 16640, 31712, 21056, 37952, 37952, 4160, 4160, 4160, 4160, 4544, 0]],
        '筑' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 63424, 9280, 9792, 9536, 13376, 51264, 2144, 0]],
        '蓄' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 32704, 4608, 9280, 65504, 17472, 32704, 17472, 32704, 0]],
        '逐' : [12, 13, 0, -2, 12, [0, 4064, 16640, 8992, 3264, 384, 58048, 11424, 8352, 8960, 20480, 36832, 0]],
        '秩' : [12, 13, 0, -2, 12, [128, 7296, 58496, 10208, 9344, 61568, 12256, 24704, 24896, 45376, 41504, 11296, 0]],
        '窒' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 65504, 8320, 32704, 1024, 16256, 1024, 65504, 0]],
        '茶' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 3584, 12672, 50272, 16256, 1024, 9344, 17472, 0]],
        '嫡' : [12, 13, 0, -2, 12, [256, 16640, 24544, 58432, 17472, 24544, 20768, 40928, 61728, 6048, 13472, 22432, 0]],
        '着' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 32704, 1024, 65504, 4224, 16256, 53376, 4224, 8064, 0]],
        '中' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 33824, 33824, 33824, 65504, 1024, 1024, 1024, 1024, 0]],
        '仲' : [12, 13, 0, -2, 12, [256, 8448, 8448, 24544, 20768, 53536, 20768, 24544, 16640, 16640, 16640, 16640, 0]],
        '宙' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33824, 1024, 32704, 17472, 17472, 32704, 17472, 17472, 32704, 0]],
        '忠' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 17472, 17472, 32704, 0, 1024, 4608, 20544, 36896, 3968, 0]],
        '抽' : [12, 13, 0, -2, 12, [256, 16640, 16640, 61408, 18720, 18720, 18720, 28640, 51488, 18720, 18720, 53216, 0]],
        '昼' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16640, 24448, 37184, 40736, 4352, 7936, 0, 65504, 0]],
        '柱' : [12, 13, 0, -2, 12, [256, 8320, 8192, 63456, 8320, 8320, 24704, 26592, 45184, 41088, 8320, 10208, 0]],
        '注' : [12, 13, 0, -2, 12, [512, 256, 32768, 20448, 256, 33024, 16640, 4064, 256, 8448, 16640, 36832, 0]],
        '虫' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 32704, 17472, 17472, 17472, 32704, 1024, 1152, 1088, 65504, 0]],
        '衷' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 17472, 17472, 32704, 5152, 57920, 8576, 31840, 0]],
        '註' : [12, 13, 0, -2, 12, [512, 57600, 0, 61408, 256, 57600, 256, 4064, 57600, 41216, 41216, 61408, 0]],
        '酎' : [12, 13, 0, -2, 12, [64, 63552, 8256, 10208, 63552, 43072, 43584, 55616, 34880, 63552, 34880, 63936, 0]],
        '鋳' : [12, 13, 0, -2, 12, [256, 12544, 20448, 33024, 63456, 8448, 65504, 8768, 45024, 42048, 10816, 61632, 0]],
        '駐' : [12, 13, 0, -2, 12, [256, 63616, 40960, 64480, 41088, 63616, 41088, 64480, 2176, 43136, 2176, 13280, 0]],
        '樗' : [12, 13, 0, -2, 12, [0, 12256, 8448, 65504, 10528, 11104, 24832, 28640, 41984, 45024, 10272, 8640, 0]],
        '瀦' : [12, 13, 0, -2, 12, [128, 15488, 35808, 20608, 2208, 38880, 17664, 3552, 21792, 17888, 34080, 39392, 0]],
        '猪' : [12, 13, 0, -2, 12, [256, 8448, 45024, 16672, 16704, 45024, 8704, 26592, 44064, 10208, 9248, 51168, 0]],
        '苧' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 65504, 32800, 0, 32704, 1024, 1024, 7168, 0]],
        '著' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 5376, 1056, 32704, 1152, 65504, 4160, 16320, 53312, 8128, 0]],
        '貯' : [12, 13, 0, -2, 12, [256, 61696, 38880, 62496, 37920, 61440, 38880, 36992, 61568, 128, 20608, 35712, 0]],
        '丁' : [12, 13, 0, -2, 12, [0, 65504, 512, 512, 512, 512, 512, 512, 512, 512, 512, 7680, 0]],
        '兆' : [12, 13, 0, -2, 12, [512, 4608, 37408, 21056, 4736, 4608, 12928, 21056, 37408, 4608, 8736, 50144, 0]],
        '凋' : [12, 13, 0, -2, 12, [0, 8160, 37408, 21024, 8096, 4640, 8096, 20512, 22432, 38048, 42912, 8288, 0]],
        '喋' : [12, 13, 0, -2, 12, [512, 2624, 65504, 43584, 43968, 43008, 45024, 41216, 49120, 41728, 58688, 6432, 0]],
        '寵' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 41248, 64000, 19424, 64544, 35808, 64000, 35808, 64000, 35808, 0]],
        '帖' : [12, 13, 0, -2, 12, [256, 8448, 8448, 8672, 63744, 43264, 43264, 45024, 42016, 9248, 9248, 10208, 0]],
        '帳' : [12, 13, 0, -2, 12, [0, 10208, 9216, 10176, 64512, 42944, 41984, 45024, 42240, 9376, 9280, 10016, 0]],
        '庁' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 16384, 24544, 16640, 16640, 16640, 33024, 33024, 1792, 0]],
        '弔' : [12, 13, 0, -2, 12, [0, 32704, 1088, 1088, 32704, 17408, 17408, 65504, 33824, 1056, 1216, 1024, 0]],
        '張' : [12, 13, 0, -2, 12, [0, 59360, 9216, 10176, 58368, 34752, 33792, 61408, 9472, 9376, 9280, 50976, 0]],
        '彫' : [12, 13, 0, -2, 12, [0, 65056, 37440, 37504, 47616, 37408, 47680, 33408, 47616, 43552, 47680, 34432, 0]],
        '徴' : [12, 13, 0, -2, 12, [1152, 5248, 21728, 40864, 32, 16160, 17568, 50336, 24384, 17472, 17568, 24352, 0]],
        '懲' : [12, 13, 0, -2, 12, [1152, 21632, 40928, 1312, 24224, 50336, 24128, 16800, 1024, 4672, 20512, 36736, 0]],
        '挑' : [12, 13, 0, -2, 12, [256, 17664, 21792, 60736, 17664, 17664, 19840, 21824, 58656, 17664, 18720, 61664, 0]],
        '暢' : [12, 13, 0, -2, 12, [0, 10176, 9280, 65472, 42048, 65504, 41984, 45024, 62112, 9376, 10528, 8896, 0]],
        '朝' : [12, 13, 0, -2, 12, [8192, 9184, 64032, 8736, 64480, 35360, 64032, 35808, 64032, 8736, 64544, 9312, 0]],
        '潮' : [12, 13, 0, -2, 12, [2048, 2528, 48928, 18720, 7648, 38176, 23840, 5600, 23840, 18720, 48672, 35424, 0]],
        '牒' : [12, 13, 0, -2, 12, [256, 42304, 45024, 42304, 62912, 33792, 34784, 57600, 45024, 41728, 42304, 43296, 0]],
        '町' : [12, 13, 0, -2, 12, [0, 64512, 42976, 42048, 64576, 42048, 42048, 42048, 64576, 64, 64, 448, 0]],
        '眺' : [12, 13, 0, -2, 12, [128, 57984, 45728, 43712, 57984, 41600, 42688, 60064, 45696, 41600, 58528, 14432, 0]],
        '聴' : [12, 13, 0, -2, 12, [256, 61696, 45024, 57600, 45024, 43680, 60064, 45024, 41472, 59712, 43040, 10112, 0]],
        '脹' : [12, 13, 0, -2, 12, [0, 63456, 37888, 38848, 62464, 38848, 37888, 65504, 38144, 38048, 37952, 46880, 0]],
        '腸' : [12, 13, 0, -2, 12, [0, 63424, 37952, 38848, 62528, 38880, 37888, 65504, 37536, 38048, 39200, 45760, 0]],
        '蝶' : [12, 13, 0, -2, 12, [512, 10816, 12256, 64064, 43968, 43008, 45024, 61696, 12256, 8960, 9536, 63776, 0]],
        '調' : [12, 13, 0, -2, 12, [0, 61408, 2336, 59680, 2976, 59680, 2976, 2080, 60320, 43680, 43936, 59488, 0]],
        '諜' : [12, 13, 0, -2, 12, [256, 58688, 4064, 58688, 1472, 58368, 2016, 256, 61408, 41728, 42304, 63776, 0]],
        '超' : [12, 13, 0, -2, 12, [8192, 9184, 63648, 8480, 8800, 63488, 9184, 14880, 41504, 41952, 61440, 36832, 0]],
        '跳' : [12, 13, 0, -2, 12, [128, 62080, 39584, 38592, 62080, 8832, 8896, 42656, 47744, 41600, 42144, 63584, 0]],
        '銚' : [12, 13, 0, -2, 12, [128, 12928, 17056, 35520, 63104, 8832, 62144, 9888, 43648, 45696, 9376, 63584, 0]],
        '長' : [12, 13, 0, -2, 12, [0, 16320, 8192, 16320, 8192, 16320, 8192, 65504, 9280, 8768, 8576, 31840, 0]],
        '頂' : [12, 13, 0, -2, 12, [0, 4064, 61696, 20448, 18464, 20448, 18464, 20448, 18464, 20448, 50240, 6176, 0]],
        '鳥' : [12, 13, 0, -2, 12, [1024, 2048, 16256, 8320, 16256, 8320, 16352, 8192, 16352, 32, 21792, 38240, 0]],
        '勅' : [12, 13, 0, -2, 12, [256, 8448, 63744, 10208, 63776, 43296, 43296, 63776, 8736, 29216, 43072, 8576, 0]],
        '捗' : [12, 13, 0, -2, 12, [512, 18944, 19392, 59904, 24544, 16896, 19008, 21024, 50688, 16448, 16768, 52736, 0]],
        '直' : [12, 13, 0, -2, 12, [512, 512, 65504, 512, 4032, 18496, 20416, 18496, 18496, 20416, 16384, 32736, 0]],
        '朕' : [12, 13, 0, -2, 12, [1088, 62016, 36992, 40928, 61696, 37120, 40928, 61696, 37120, 37504, 37952, 47136, 0]],
        '沈' : [12, 13, 0, -2, 12, [256, 256, 36832, 18720, 2336, 35104, 16768, 640, 640, 8864, 17568, 39136, 0]],
        '珍' : [12, 13, 0, -2, 12, [0, 896, 58432, 18464, 16576, 18176, 57344, 16576, 18176, 16416, 57536, 3840, 0]],
        '賃' : [12, 13, 0, -2, 12, [0, 6112, 8448, 61408, 8448, 16352, 8256, 16320, 8256, 16320, 4160, 24608, 0]],
        '鎮' : [12, 13, 0, -2, 12, [256, 12544, 20448, 33024, 63456, 9248, 63456, 9248, 45024, 40960, 9280, 63520, 0]],
        '陳' : [12, 13, 0, -2, 12, [256, 57600, 45024, 41216, 53216, 43296, 45024, 43296, 45024, 49920, 34112, 39200, 0]],
        '津' : [12, 13, 0, -2, 12, [256, 256, 36832, 16672, 8160, 33056, 20448, 256, 20448, 16640, 40928, 33024, 0]],
        '墜' : [12, 13, 0, -2, 12, [1088, 58496, 45024, 49792, 44448, 41664, 56480, 33568, 1024, 16256, 1024, 65504, 0]],
        '椎' : [12, 13, 0, -2, 12, [512, 8768, 8768, 63456, 9344, 12256, 25728, 25728, 47072, 42112, 9344, 10208, 0]],
        '槌' : [12, 13, 0, -2, 12, [128, 8448, 10720, 62752, 8672, 8448, 28128, 25888, 42272, 42464, 10752, 12768, 0]],
        '追' : [12, 13, 0, -2, 12, [256, 512, 18368, 9280, 1984, 1024, 59328, 9280, 9280, 10176, 20480, 36832, 0]],
        '鎚' : [12, 13, 0, -2, 12, [128, 12544, 18912, 34080, 61920, 8448, 64992, 9504, 46368, 42464, 10752, 61920, 0]],
        '痛' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 53184, 16512, 24544, 53536, 24544, 20768, 40928, 37152, 0]],
        '通' : [12, 13, 0, -2, 12, [0, 4032, 16512, 12256, 2336, 4064, 59680, 12256, 10528, 10592, 20480, 36832, 0]],
        '塚' : [12, 13, 0, -2, 12, [0, 12256, 10272, 10208, 61696, 8992, 11456, 8576, 9920, 14496, 49312, 1792, 0]],
        '栂' : [12, 13, 0, -2, 12, [0, 10208, 9248, 62752, 9376, 12256, 25632, 25888, 43168, 43040, 12256, 8256, 0]],
        '掴' : [12, 13, 0, -2, 12, [0, 20448, 18464, 65504, 18720, 18720, 19360, 26976, 51488, 20448, 18464, 53216, 0]],
        '槻' : [12, 13, 0, -2, 12, [0, 9696, 9504, 65504, 9504, 16352, 25888, 25888, 44000, 45376, 8768, 11360, 0]],
        '佃' : [12, 13, 0, -2, 12, [0, 12256, 10528, 18720, 18720, 51488, 20448, 18720, 18720, 18720, 18720, 20448, 0]],
        '漬' : [12, 13, 0, -2, 12, [256, 256, 40928, 16640, 4064, 33024, 24544, 1088, 18368, 17472, 34752, 39008, 0]],
        '柘' : [12, 13, 0, -2, 12, [0, 12256, 8448, 61696, 8448, 8704, 26592, 27680, 46112, 42016, 9248, 10208, 0]],
        '辻' : [12, 13, 0, -2, 12, [256, 256, 16640, 8448, 4064, 256, 57600, 8448, 8448, 8448, 20480, 36832, 0]],
        '蔦' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 2048, 16256, 8320, 16352, 8192, 16352, 32, 21792, 38240, 0]],
        '綴' : [12, 13, 0, -2, 12, [0, 12000, 16928, 35488, 58432, 10912, 16384, 61152, 8736, 43680, 42048, 10912, 0]],
        '鍔' : [12, 13, 0, -2, 12, [0, 16096, 19104, 36576, 61440, 10208, 61440, 12256, 41472, 42976, 9248, 61888, 0]],
        '椿' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61696, 12256, 8768, 32736, 25632, 45024, 46112, 9248, 10208, 0]],
        '潰' : [12, 13, 0, -2, 12, [256, 256, 40928, 20768, 8160, 33024, 24544, 2112, 20416, 18496, 36800, 45152, 0]],
        '坪' : [12, 13, 0, -2, 12, [0, 24544, 16640, 16640, 59680, 17728, 16640, 16640, 24544, 16640, 57600, 256, 0]],
        '壷' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 16256, 0, 65504, 43680, 43680, 16256, 2560, 65504, 0]],
        '嬬' : [12, 13, 0, -2, 12, [0, 20448, 16640, 65504, 20768, 22368, 20480, 40928, 57856, 12256, 19104, 19104, 0]],
        '紬' : [12, 13, 0, -2, 12, [256, 8448, 16640, 36832, 59680, 10528, 18720, 65504, 10528, 43296, 43296, 12256, 0]],
        '爪' : [12, 13, 0, -2, 12, [0, 32736, 18560, 18560, 18560, 18560, 18560, 18496, 18496, 34880, 34848, 34848, 0]],
        '吊' : [12, 13, 0, -2, 12, [0, 16256, 8320, 8320, 16256, 1024, 1024, 65504, 33824, 33824, 33888, 1024, 0]],
        '釣' : [12, 13, 0, -2, 12, [512, 12800, 19424, 33824, 62496, 8224, 64032, 8480, 43040, 43040, 8256, 63872, 0]],
        '鶴' : [12, 13, 0, -2, 12, [128, 8448, 65472, 33856, 22464, 62528, 42976, 62464, 42976, 61472, 42272, 62816, 0]],
        '亭' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 8320, 16256, 0, 65504, 32800, 49056, 1024, 1024, 7168, 0]],
        '低' : [12, 13, 0, -2, 12, [0, 8288, 12160, 18560, 18560, 53216, 18496, 18496, 18464, 20000, 16384, 24544, 0]],
        '停' : [12, 13, 0, -2, 12, [512, 8704, 16352, 18496, 20416, 49152, 24544, 20512, 20448, 16640, 16640, 18176, 0]],
        '偵' : [12, 13, 0, -2, 12, [512, 8704, 9184, 16896, 20448, 51232, 20448, 18464, 20448, 16384, 17472, 22560, 0]],
        '剃' : [12, 13, 0, -2, 12, [18432, 18464, 4384, 64800, 9504, 64800, 41248, 64800, 9504, 25632, 42016, 14432, 0]],
        '貞' : [12, 13, 0, -2, 12, [1024, 1024, 1984, 1024, 16256, 8320, 16256, 8320, 16256, 8320, 16256, 49248, 0]],
        '呈' : [12, 13, 0, -2, 12, [0, 16256, 8320, 8320, 16256, 0, 32704, 1024, 16256, 1024, 1024, 65504, 0]],
        '堤' : [12, 13, 0, -2, 12, [0, 20416, 18496, 20416, 59456, 20416, 16384, 24544, 18688, 18880, 61184, 12512, 0]],
        '定' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 32768, 16320, 1024, 1024, 10176, 9216, 15360, 50144, 0]],
        '帝' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 4352, 65504, 33824, 49056, 9344, 9344, 9344, 9600, 0]],
        '底' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 24512, 20736, 24544, 20736, 20608, 40544, 32768, 16352, 0]],
        '庭' : [12, 13, 0, -2, 12, [1024, 1024, 32736, 16384, 31712, 18560, 21472, 30848, 34944, 37856, 6144, 26592, 0]],
        '廷' : [12, 13, 0, -2, 12, [0, 192, 59136, 8448, 16640, 36832, 57600, 8448, 41216, 20448, 28672, 36832, 0]],
        '弟' : [12, 13, 0, -2, 12, [8448, 4352, 512, 32704, 1088, 32704, 17408, 32736, 3104, 5152, 9248, 50368, 0]],
        '悌' : [12, 13, 0, -2, 12, [1088, 8768, 8320, 12256, 41248, 45024, 43264, 45024, 41248, 8992, 9504, 10688, 0]],
        '抵' : [12, 13, 0, -2, 12, [0, 8288, 10112, 62592, 9344, 10208, 9280, 13376, 58400, 10016, 8192, 28640, 0]],
        '挺' : [12, 13, 0, -2, 12, [0, 23648, 18304, 59520, 20608, 24544, 17536, 21632, 59520, 19424, 21504, 54240, 0]],
        '提' : [12, 13, 0, -2, 12, [0, 20416, 18496, 61376, 18496, 20416, 16384, 24544, 59648, 18880, 20224, 61664, 0]],
        '梯' : [12, 13, 0, -2, 12, [1088, 8768, 8320, 65504, 8480, 12256, 26880, 28640, 41248, 41760, 9504, 10688, 0]],
        '汀' : [12, 13, 0, -2, 12, [0, 8160, 32896, 16512, 128, 32896, 16512, 128, 128, 8320, 16512, 33664, 0]],
        '碇' : [12, 13, 0, -2, 12, [256, 256, 65504, 10272, 8192, 12256, 28928, 53504, 21984, 21760, 22272, 30944, 0]],
        '禎' : [12, 13, 0, -2, 12, [256, 8448, 8672, 61696, 6112, 5152, 10208, 25632, 47072, 8192, 8768, 11296, 0]],
        '程' : [12, 13, 0, -2, 12, [0, 8160, 58400, 9248, 10208, 61440, 10208, 24704, 30688, 41088, 41088, 10208, 0]],
        '締' : [12, 13, 0, -2, 12, [256, 8448, 20448, 33344, 57920, 12256, 18720, 63456, 9504, 46368, 42272, 9568, 0]],
        '艇' : [12, 13, 0, -2, 12, [8192, 20192, 62016, 37440, 54336, 38368, 65088, 37440, 55872, 38368, 38400, 47584, 0]],
        '訂' : [12, 13, 0, -2, 12, [0, 63456, 64, 61504, 64, 61504, 64, 64, 61504, 36928, 36928, 61888, 0]],
        '諦' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57920, 576, 61408, 2336, 2016, 58656, 42272, 42272, 58720, 0]],
        '蹄' : [12, 13, 0, -2, 12, [256, 61696, 40928, 37440, 62016, 12256, 10528, 42976, 46368, 42272, 42272, 62816, 0]],
        '逓' : [12, 13, 0, -2, 12, [0, 4064, 18688, 10496, 4064, 2304, 63456, 9504, 9504, 9568, 20480, 36832, 0]],
        '邸' : [12, 13, 0, -2, 12, [0, 3552, 61728, 37152, 37184, 64896, 35136, 35104, 34080, 58656, 448, 64768, 0]],
        '鄭' : [12, 13, 0, -2, 12, [17408, 18912, 64800, 20768, 64832, 42368, 56640, 34080, 65312, 8480, 20928, 36096, 0]],
        '釘' : [12, 13, 0, -2, 12, [0, 13280, 18496, 32832, 63552, 8256, 63552, 8256, 43072, 43072, 8256, 63936, 0]],
        '鼎' : [12, 13, 0, -2, 12, [0, 24384, 20800, 24384, 20800, 20800, 32704, 2560, 64480, 10816, 10816, 51776, 0]],
        '泥' : [12, 13, 0, -2, 12, [0, 4064, 34848, 18464, 4064, 34816, 18944, 2656, 19328, 18944, 37408, 37856, 0]],
        '摘' : [12, 13, 0, -2, 12, [256, 16640, 24544, 58496, 17536, 20448, 18720, 28640, 51488, 19360, 19104, 52128, 0]],
        '擢' : [12, 13, 0, -2, 12, [0, 20192, 16928, 61152, 16928, 20192, 17536, 28640, 55552, 20448, 18688, 53216, 0]],
        '敵' : [12, 13, 0, -2, 12, [4352, 4352, 64992, 18720, 18720, 65056, 37536, 65184, 37440, 47680, 43680, 47904, 0]],
        '滴' : [12, 13, 0, -2, 12, [256, 256, 40928, 17536, 1152, 36832, 18720, 4064, 2336, 11168, 19104, 35744, 0]],
        '的' : [12, 13, 0, -2, 12, [512, 16896, 17376, 62496, 37920, 36896, 61984, 37152, 36896, 36896, 61504, 384, 0]],
        '笛' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 1024, 32704, 17472, 17472, 32704, 17472, 17472, 32704, 0]],
        '適' : [12, 13, 0, -2, 12, [256, 256, 40928, 17536, 1152, 4064, 59680, 12256, 10912, 11168, 20480, 36832, 0]],
        '鏑' : [12, 13, 0, -2, 12, [256, 12544, 20448, 33344, 62016, 12256, 63776, 12256, 43296, 43936, 10912, 64416, 0]],
        '溺' : [12, 13, 0, -2, 12, [0, 7648, 33824, 17440, 7648, 37120, 24032, 1184, 19808, 21664, 34080, 35936, 0]],
        '哲' : [12, 13, 0, -2, 12, [8192, 9184, 64000, 9184, 14912, 58432, 9280, 0, 16320, 8256, 8256, 16320, 0]],
        '徹' : [12, 13, 0, -2, 12, [1152, 9344, 24544, 34976, 4640, 16160, 21152, 56992, 21056, 24128, 21152, 21280, 0]],
        '撤' : [12, 13, 0, -2, 12, [1152, 17536, 32736, 51360, 21024, 24352, 21152, 32416, 53824, 24128, 21152, 54048, 0]],
        '轍' : [12, 13, 0, -2, 12, [1152, 9344, 65504, 9376, 63648, 44960, 63776, 44960, 63808, 12096, 63904, 10528, 0]],
        '迭' : [12, 13, 0, -2, 12, [128, 1152, 17536, 10208, 2176, 2176, 59360, 8320, 8512, 9760, 20480, 36832, 0]],
        '鉄' : [12, 13, 0, -2, 12, [128, 12928, 19072, 33760, 62592, 9344, 64480, 8320, 43136, 43328, 8736, 64544, 0]],
        '典' : [12, 13, 0, -2, 12, [2560, 2560, 32704, 19008, 19008, 32704, 19008, 19008, 65504, 0, 12672, 49248, 0]],
        '填' : [12, 13, 0, -2, 12, [256, 16640, 24544, 16640, 61408, 18464, 20448, 18464, 24544, 16384, 58432, 2080, 0]],
        '天' : [12, 13, 0, -2, 12, [0, 65504, 1024, 1024, 1024, 32704, 1024, 1024, 2560, 4352, 8320, 49248, 0]],
        '展' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 18560, 32736, 18560, 18560, 32736, 37440, 37248, 15456, 0]],
        '店' : [12, 13, 0, -2, 12, [512, 512, 32736, 16896, 16896, 17376, 16896, 16896, 24544, 36896, 36896, 8160, 0]],
        '添' : [12, 13, 0, -2, 12, [0, 8128, 33280, 16896, 8160, 33920, 18496, 4640, 16896, 19104, 37536, 34304, 0]],
        '纏' : [12, 13, 0, -2, 12, [256, 8448, 20448, 35488, 60384, 10912, 20448, 59712, 10912, 47072, 41088, 12256, 0]],
        '甜' : [12, 13, 0, -2, 12, [0, 6688, 57888, 10208, 8736, 64032, 8736, 9184, 64032, 35360, 35360, 64480, 0]],
        '貼' : [12, 13, 0, -2, 12, [256, 61696, 37120, 61920, 37120, 61696, 37120, 38880, 62496, 1056, 21536, 38880, 0]],
        '転' : [12, 13, 0, -2, 12, [0, 9184, 64512, 8192, 64512, 42976, 64640, 42112, 64768, 8480, 65056, 9184, 0]],
        '顛' : [12, 13, 0, -2, 12, [8192, 10208, 63616, 9184, 64032, 35808, 64032, 35808, 64032, 992, 21056, 35872, 0]],
        '点' : [12, 13, 0, -2, 12, [1024, 1024, 1984, 1024, 1024, 16256, 8320, 8320, 16256, 0, 21056, 35104, 0]],
        '伝' : [12, 13, 0, -2, 12, [0, 10176, 8192, 16384, 16384, 53216, 16896, 16896, 17472, 17472, 18592, 20256, 0]],
        '殿' : [12, 13, 0, -2, 12, [0, 64448, 35392, 35392, 64064, 37472, 63488, 38848, 64064, 33344, 21376, 35936, 0]],
        '澱' : [12, 13, 0, -2, 12, [0, 7904, 37536, 21152, 7840, 38304, 24064, 5600, 24352, 16672, 38080, 41760, 0]],
        '田' : [12, 13, 0, -2, 12, [0, 65504, 33824, 33824, 33824, 33824, 65504, 33824, 33824, 33824, 33824, 65504, 0]],
        '電' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 38176, 32704, 17472, 32704, 17472, 32704, 1056, 2016, 0]],
        '兎' : [12, 13, 0, -2, 12, [0, 992, 64512, 1024, 32704, 17472, 17472, 32704, 4736, 4672, 8736, 49632, 0]],
        '吐' : [12, 13, 0, -2, 12, [256, 256, 61696, 37120, 40928, 37120, 37120, 37120, 37120, 37120, 61696, 4064, 0]],
        '堵' : [12, 13, 0, -2, 12, [256, 16640, 20448, 16672, 57664, 20448, 16896, 18400, 23584, 18400, 58400, 2016, 0]],
        '塗' : [12, 13, 0, -2, 12, [0, 18368, 10272, 34752, 16640, 8160, 17664, 35136, 1056, 16256, 1024, 65504, 0]],
        '妬' : [12, 13, 0, -2, 12, [0, 20448, 16640, 61696, 20736, 20992, 22496, 39968, 58400, 13344, 17440, 18400, 0]],
        '屠' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 17440, 24512, 17536, 32736, 18496, 40896, 43072, 4032, 0]],
        '徒' : [12, 13, 0, -2, 12, [256, 8448, 20448, 33024, 256, 12256, 16640, 49408, 18912, 18688, 20224, 28896, 0]],
        '斗' : [12, 13, 0, -2, 12, [256, 256, 4352, 2304, 16640, 8448, 4352, 480, 3840, 61696, 256, 256, 0]],
        '杜' : [12, 13, 0, -2, 12, [128, 8320, 8320, 61568, 10208, 8320, 24704, 24704, 45184, 41088, 8320, 10208, 0]],
        '渡' : [12, 13, 0, -2, 12, [256, 256, 40928, 21056, 8160, 37440, 21440, 4096, 22464, 17472, 33664, 40032, 0]],
        '登' : [12, 13, 0, -2, 12, [0, 31296, 2688, 35104, 20800, 16256, 16480, 49024, 8320, 16256, 4352, 65504, 0]],
        '菟' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 16640, 65472, 17472, 32704, 2688, 12832, 49632, 0]],
        '賭' : [12, 13, 0, -2, 12, [256, 61696, 38880, 61728, 37184, 63456, 37376, 38880, 64032, 992, 21024, 35808, 0]],
        '途' : [12, 13, 0, -2, 12, [0, 896, 17472, 10272, 1984, 256, 61408, 8448, 9024, 9504, 20480, 36832, 0]],
        '都' : [12, 13, 0, -2, 12, [8192, 8672, 64800, 9504, 10560, 64896, 8512, 32032, 50464, 32032, 17856, 32000, 0]],
        '鍍' : [12, 13, 0, -2, 12, [256, 12544, 20448, 35392, 65504, 10816, 64448, 10240, 43968, 43584, 8576, 65120, 0]],
        '砥' : [12, 13, 0, -2, 12, [0, 2016, 62592, 9344, 9344, 10208, 29760, 54336, 21536, 22304, 20480, 30688, 0]],
        '砺' : [12, 13, 0, -2, 12, [0, 2016, 62464, 9216, 10208, 9472, 29952, 53728, 21024, 21024, 21536, 29888, 0]],
        '努' : [12, 13, 0, -2, 12, [8192, 9184, 64544, 18720, 18752, 14528, 50976, 1024, 32704, 1088, 6208, 24960, 0]],
        '度' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 32736, 18560, 20352, 16384, 40896, 34944, 3840, 28896, 0]],
        '土' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 1024, 32704, 1024, 1024, 1024, 1024, 1024, 1024, 65504, 0]],
        '奴' : [12, 13, 0, -2, 12, [16384, 18400, 16416, 61472, 21024, 21024, 20800, 37184, 57472, 14720, 16960, 19488, 0]],
        '怒' : [12, 13, 0, -2, 12, [8192, 9184, 64544, 18720, 18752, 14528, 50976, 0, 1024, 4672, 20512, 36736, 0]],
        '倒' : [12, 13, 0, -2, 12, [0, 16160, 9376, 18592, 20640, 57248, 17568, 17568, 24224, 17440, 17440, 24416, 0]],
        '党' : [12, 13, 0, -2, 12, [1024, 9280, 9344, 65504, 32800, 49056, 8320, 8320, 16256, 4640, 8736, 50144, 0]],
        '冬' : [12, 13, 0, -2, 12, [2048, 2048, 3968, 4224, 26880, 1536, 6528, 58464, 768, 0, 14336, 1920, 0]],
        '凍' : [12, 13, 0, -2, 12, [256, 256, 40928, 16640, 4064, 2336, 4064, 2336, 20448, 17152, 34112, 39200, 0]],
        '刀' : [12, 13, 0, -2, 12, [0, 65504, 2080, 2080, 2080, 2080, 2080, 2080, 4128, 4128, 8256, 50048, 0]],
        '唐' : [12, 13, 0, -2, 12, [512, 512, 32736, 16928, 32736, 16928, 24544, 16384, 24512, 36928, 36928, 8128, 0]],
        '塔' : [12, 13, 0, -2, 12, [576, 8768, 12256, 8768, 61824, 8768, 9248, 11200, 8192, 14304, 50208, 2016, 0]],
        '塘' : [12, 13, 0, -2, 12, [128, 16512, 20448, 18592, 61408, 18592, 19424, 18432, 19424, 18976, 61984, 5088, 0]],
        '套' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 16256, 53344, 7936, 4096, 65504, 2048, 4160, 32704, 0]],
        '宕' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 0, 32704, 2048, 2048, 8128, 12352, 53312, 8128, 0]],
        '島' : [12, 13, 0, -2, 12, [1024, 2048, 16256, 8320, 16256, 8320, 16352, 8192, 16352, 4128, 21024, 32448, 0]],
        '嶋' : [12, 13, 0, -2, 12, [256, 512, 12224, 10304, 44992, 43072, 45024, 43008, 45024, 40992, 59936, 10944, 0]],
        '悼' : [12, 13, 0, -2, 12, [256, 8448, 8672, 8448, 47072, 42016, 42976, 42016, 42976, 8320, 12256, 8320, 0]],
        '投' : [12, 13, 0, -2, 12, [0, 18304, 17536, 58496, 17568, 18656, 16384, 28608, 50240, 17536, 17280, 56416, 0]],
        '搭' : [12, 13, 0, -2, 12, [576, 16960, 20448, 62016, 16768, 16960, 17440, 27584, 49152, 18400, 17440, 51168, 0]],
        '東' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 17472, 32704, 17472, 32704, 3328, 13440, 50272, 0]],
        '桃' : [12, 13, 0, -2, 12, [128, 8832, 10912, 63168, 8832, 8832, 25280, 30368, 43648, 41600, 9376, 14560, 0]],
        '梼' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61696, 12256, 8448, 28640, 25152, 49120, 42048, 10816, 8384, 0]],
        '棟' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61696, 12256, 10528, 28640, 31008, 45024, 41728, 9536, 14624, 0]],
        '盗' : [12, 13, 0, -2, 12, [1024, 17408, 10208, 2336, 10496, 17024, 33856, 2080, 32704, 19008, 19008, 65504, 0]],
        '淘' : [12, 13, 0, -2, 12, [2048, 2048, 36832, 20512, 7968, 33824, 24352, 1056, 21792, 24352, 32832, 33152, 0]],
        '湯' : [12, 13, 0, -2, 12, [0, 4032, 34880, 20416, 2112, 36832, 17408, 4064, 21152, 17568, 35104, 33472, 0]],
        '涛' : [12, 13, 0, -2, 12, [256, 256, 40928, 16640, 4064, 33024, 24544, 576, 24544, 18496, 37952, 33472, 0]],
        '灯' : [12, 13, 0, -2, 12, [0, 10208, 8256, 43072, 43072, 43072, 8256, 8256, 20544, 18496, 32832, 33216, 0]],
        '燈' : [12, 13, 0, -2, 12, [0, 12064, 8512, 43680, 42048, 42976, 10240, 10208, 21536, 22496, 33344, 36832, 0]],
        '当' : [12, 13, 0, -2, 12, [1024, 1024, 17440, 9280, 1024, 32736, 32, 32, 32736, 32, 32, 32736, 0]],
        '痘' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 57312, 16384, 20416, 51264, 20416, 17536, 33920, 40928, 0]],
        '祷' : [12, 13, 0, -2, 12, [256, 16640, 20448, 61696, 6112, 4352, 12256, 16960, 57312, 25664, 19008, 16576, 0]],
        '等' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 1024, 16320, 1024, 65504, 128, 32736, 4224, 2432, 0]],
        '答' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 7936, 8320, 24384, 32800, 16256, 8320, 16256, 0]],
        '筒' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 65504, 32800, 40736, 32800, 40736, 37152, 40800, 0]],
        '糖' : [12, 13, 0, -2, 12, [128, 8320, 45024, 43168, 12256, 63648, 11232, 26624, 27616, 45600, 41504, 9184, 0]],
        '統' : [12, 13, 0, -2, 12, [256, 8448, 20448, 37376, 62016, 9280, 18336, 64160, 8832, 41600, 42144, 14560, 0]],
        '到' : [12, 13, 0, -2, 12, [0, 64544, 8480, 16672, 34080, 64800, 8480, 8480, 64800, 8224, 8224, 64608, 0]],
        '董' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 32704, 9344, 16256, 9344, 9344, 32704, 1024, 65504, 0]],
        '蕩' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 36800, 18496, 4032, 34880, 24544, 1312, 18720, 37568, 0]],
        '藤' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4384, 62784, 38880, 62080, 40928, 62784, 37792, 40256, 45856, 0]],
        '討' : [12, 13, 0, -2, 12, [64, 61504, 64, 63456, 64, 61504, 1088, 576, 61504, 36928, 36928, 61888, 0]],
        '謄' : [12, 13, 0, -2, 12, [256, 62784, 40928, 37504, 65504, 37952, 39200, 63424, 36864, 40928, 37952, 47040, 0]],
        '豆' : [12, 13, 0, -2, 12, [0, 65504, 0, 16256, 8320, 8320, 8320, 16256, 4352, 4352, 4608, 65504, 0]],
        '踏' : [12, 13, 0, -2, 12, [128, 61568, 40608, 37568, 62624, 10656, 8192, 47072, 42016, 42976, 42016, 63456, 0]],
        '逃' : [12, 13, 0, -2, 12, [128, 640, 19104, 9920, 640, 704, 59040, 10880, 8864, 9440, 20480, 36832, 0]],
        '透' : [12, 13, 0, -2, 12, [0, 4064, 33024, 24544, 1344, 2336, 61312, 8928, 8736, 9408, 20480, 36832, 0]],
        '鐙' : [12, 13, 0, -2, 12, [0, 14112, 16704, 35488, 62528, 10208, 63488, 10208, 46112, 42976, 8768, 65504, 0]],
        '陶' : [12, 13, 0, -2, 12, [1024, 58368, 42976, 43040, 53152, 41504, 44960, 41504, 43680, 53152, 32800, 32960, 0]],
        '頭' : [12, 13, 0, -2, 12, [0, 63456, 128, 63456, 37920, 38880, 37920, 63456, 17440, 22496, 21056, 64544, 0]],
        '騰' : [12, 13, 0, -2, 12, [256, 58688, 45024, 41600, 61408, 43328, 45024, 59648, 45024, 40992, 43680, 43744, 0]],
        '闘' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 32800, 48288, 42912, 48288, 37536, 48544, 0]],
        '働' : [12, 13, 0, -2, 12, [128, 16256, 9344, 24544, 21664, 57248, 21664, 24480, 17568, 24352, 17440, 24256, 0]],
        '動' : [12, 13, 0, -2, 12, [256, 63744, 8448, 65504, 43296, 63776, 43296, 63776, 8736, 64032, 8256, 63872, 0]],
        '同' : [12, 13, 0, -2, 12, [0, 65504, 32800, 32800, 49056, 32800, 32800, 40736, 37152, 37152, 40736, 32864, 0]],
        '堂' : [12, 13, 0, -2, 12, [2112, 17536, 8448, 65504, 32800, 49056, 8320, 16256, 1024, 16256, 1024, 65504, 0]],
        '導' : [12, 13, 0, -2, 12, [1088, 1152, 40928, 16640, 4032, 59456, 12224, 18496, 65504, 128, 4224, 2432, 0]],
        '憧' : [12, 13, 0, -2, 12, [256, 8448, 16352, 9280, 42048, 49120, 43296, 45024, 43296, 12256, 8448, 12256, 0]],
        '撞' : [12, 13, 0, -2, 12, [256, 16640, 24544, 58432, 17472, 24544, 18720, 28640, 51488, 20448, 16640, 57312, 0]],
        '洞' : [12, 13, 0, -2, 12, [0, 8160, 36896, 20512, 6048, 36896, 20512, 6048, 21664, 22432, 36896, 36960, 0]],
        '瞳' : [12, 13, 0, -2, 12, [256, 256, 65504, 42048, 58432, 49120, 43296, 61408, 43296, 45024, 57600, 8160, 0]],
        '童' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 4352, 4352, 65504, 9344, 16256, 9344, 16256, 1024, 65504, 0]],
        '胴' : [12, 13, 0, -2, 12, [0, 61408, 43040, 43040, 60320, 43040, 43040, 60320, 43680, 43936, 43040, 43104, 0]],
        '萄' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8448, 32736, 40992, 16160, 18464, 65440, 2080, 18720, 32704, 0]],
        '道' : [12, 13, 0, -2, 12, [1088, 1152, 40928, 16640, 1984, 1088, 59328, 9280, 9280, 10176, 20480, 36832, 0]],
        '銅' : [12, 13, 0, -2, 12, [0, 16352, 18464, 34848, 64416, 10272, 63520, 11168, 43680, 43936, 10272, 63584, 0]],
        '峠' : [12, 13, 0, -2, 12, [256, 8448, 8640, 8448, 43264, 45024, 43008, 45024, 43264, 43392, 63808, 256, 0]],
        '鴇' : [12, 13, 0, -2, 12, [256, 33280, 34752, 62528, 34752, 37952, 63456, 17408, 18400, 61472, 19104, 19168, 0]],
        '匿' : [12, 13, 0, -2, 12, [0, 65504, 37120, 49088, 37120, 33792, 49088, 34816, 40832, 45184, 36992, 65504, 0]],
        '得' : [12, 13, 0, -2, 12, [0, 10208, 17440, 34784, 1056, 10208, 16448, 53216, 16448, 17472, 16960, 16576, 0]],
        '徳' : [12, 13, 0, -2, 12, [256, 8448, 20448, 33024, 4064, 10912, 19104, 53216, 16896, 18752, 26656, 18304, 0]],
        '涜' : [12, 13, 0, -2, 12, [256, 256, 40928, 16640, 4064, 32768, 24544, 4128, 17024, 17024, 33952, 39136, 0]],
        '特' : [12, 13, 0, -2, 12, [256, 41216, 45024, 61696, 41216, 45024, 8256, 16352, 57408, 9280, 8768, 8384, 0]],
        '督' : [12, 13, 0, -2, 12, [8192, 15328, 8224, 64800, 8512, 43200, 42272, 32704, 16448, 32704, 16448, 32704, 0]],
        '禿' : [12, 13, 0, -2, 12, [0, 960, 31744, 1024, 65504, 3328, 13440, 50272, 4608, 4608, 8736, 49632, 0]],
        '篤' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 16320, 8704, 16320, 8704, 16352, 32, 21792, 38240, 0]],
        '毒' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 1024, 65504, 9344, 65504, 17536, 32736, 256, 0]],
        '独' : [12, 13, 0, -2, 12, [256, 8448, 41216, 20448, 18720, 43296, 10528, 28640, 41216, 8480, 8480, 53216, 0]],
        '読' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57600, 2016, 57344, 4064, 2080, 57984, 41600, 42144, 63712, 0]],
        '栃' : [12, 13, 0, -2, 12, [0, 10208, 9216, 62464, 10208, 9472, 25856, 27104, 43552, 41504, 9248, 9408, 0]],
        '橡' : [12, 13, 0, -2, 12, [0, 10176, 10368, 65504, 10528, 10528, 28640, 25216, 44448, 41664, 11424, 8960, 0]],
        '凸' : [12, 13, 0, -2, 12, [0, 7936, 4352, 4352, 4352, 4352, 61920, 32800, 32800, 32800, 32800, 65504, 0]],
        '突' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 25024, 1024, 65504, 1024, 2560, 12672, 49248, 0]],
        '椴' : [12, 13, 0, -2, 12, [0, 12224, 10560, 65344, 10560, 10592, 28160, 27072, 44864, 47424, 10624, 11872, 0]],
        '届' : [12, 13, 0, -2, 12, [0, 32736, 16416, 16416, 32736, 16896, 24544, 21024, 24544, 37408, 37408, 8160, 0]],
        '鳶' : [12, 13, 0, -2, 12, [1024, 1152, 65504, 4608, 16256, 8416, 16256, 8320, 16352, 32, 21792, 38240, 0]],
        '苫' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 1984, 1024, 1024, 32704, 16448, 16448, 32704, 0]],
        '寅' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 49056, 1024, 16256, 9344, 16256, 9344, 16256, 49248, 0]],
        '酉' : [12, 13, 0, -2, 12, [0, 32736, 2560, 2560, 32736, 18976, 21024, 25056, 16416, 32736, 16416, 32736, 0]],
        '瀞' : [12, 13, 0, -2, 12, [2048, 2496, 48704, 18560, 7648, 34976, 32736, 5280, 24032, 21632, 40064, 38272, 0]],
        '噸' : [12, 13, 0, -2, 12, [2048, 3040, 60544, 48096, 43552, 44000, 43552, 49120, 43552, 44000, 59712, 3616, 0]],
        '屯' : [12, 13, 0, -2, 12, [1024, 1024, 2016, 64512, 1088, 17472, 17472, 32704, 17408, 1056, 1056, 992, 0]],
        '惇' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8192, 42976, 42016, 42976, 41024, 41088, 12256, 8320, 9088, 0]],
        '敦' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 576, 64064, 34880, 63808, 4416, 8320, 63616, 8512, 26144, 0]],
        '沌' : [12, 13, 0, -2, 12, [512, 512, 33760, 24064, 576, 37440, 21056, 8128, 4608, 544, 16928, 33760, 0]],
        '豚' : [12, 13, 0, -2, 12, [0, 61408, 41216, 41728, 60576, 41408, 42624, 63616, 41408, 42656, 47264, 41728, 0]],
        '遁' : [12, 13, 0, -2, 12, [0, 4064, 18560, 12256, 2176, 3040, 59936, 11232, 10784, 9184, 20480, 36832, 0]],
        '頓' : [12, 13, 0, -2, 12, [0, 10208, 14464, 59360, 9248, 42976, 42016, 65504, 9248, 10208, 8768, 7200, 0]],
        '呑' : [12, 13, 0, -2, 12, [0, 960, 31744, 1024, 65504, 2304, 4224, 16320, 57504, 8320, 8320, 16256, 0]],
        '曇' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 65504, 42272, 38048, 1024, 65504, 8320, 32576, 0]],
        '鈍' : [12, 13, 0, -2, 12, [256, 12544, 16864, 36608, 61728, 9504, 62752, 10208, 46336, 41216, 8480, 63968, 0]],
        '奈' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2304, 4224, 16192, 49184, 16256, 1024, 17472, 33824, 3072, 0]],
        '那' : [12, 13, 0, -2, 12, [0, 64992, 9504, 9504, 64832, 9600, 9536, 64800, 17696, 34080, 35264, 12544, 0]],
        '内' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 33824, 33824, 35360, 37152, 41120, 32800, 32800, 32992, 0]],
        '乍' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 9216, 17408, 18368, 33792, 33792, 2016, 1024, 1024, 1024, 0]],
        '凪' : [12, 13, 0, -2, 12, [0, 32704, 16448, 17472, 17472, 17472, 22336, 21568, 21568, 21536, 49056, 32800, 0]],
        '薙' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 33344, 64480, 42560, 9184, 64064, 9184, 21056, 35808, 0]],
        '謎' : [12, 13, 0, -2, 12, [128, 57472, 4768, 60096, 128, 58336, 6272, 2496, 60064, 43136, 44032, 62432, 0]],
        '灘' : [12, 13, 0, -2, 12, [256, 5440, 48960, 21984, 16192, 43488, 32576, 2368, 32736, 18752, 38208, 41952, 0]],
        '捺' : [12, 13, 0, -2, 12, [256, 16640, 24544, 58496, 18496, 22432, 16384, 24544, 57600, 18752, 20768, 49920, 0]],
        '鍋' : [12, 13, 0, -2, 12, [0, 14272, 17472, 34240, 62784, 9536, 65504, 10272, 43936, 43680, 11168, 63584, 0]],
        '楢' : [12, 13, 0, -2, 12, [576, 8832, 12256, 62080, 12256, 10912, 27296, 27872, 43040, 45024, 10272, 12256, 0]],
        '馴' : [12, 13, 0, -2, 12, [0, 64032, 41632, 64160, 41632, 64160, 41632, 64160, 2720, 43680, 3104, 13344, 0]],
        '縄' : [12, 13, 0, -2, 12, [0, 10176, 17728, 38848, 62784, 12256, 18720, 65504, 10528, 45024, 41216, 8416, 0]],
        '畷' : [12, 13, 0, -2, 12, [0, 2016, 63776, 44448, 43584, 44448, 63488, 45024, 43296, 44448, 64064, 1440, 0]],
        '南' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 65504, 37152, 37408, 49056, 33824, 49056, 33824, 33888, 0]],
        '楠' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61696, 12256, 10912, 27296, 28640, 43296, 45024, 10528, 10592, 0]],
        '軟' : [12, 13, 0, -2, 12, [512, 8704, 64480, 8864, 64160, 43680, 63616, 43136, 63808, 8512, 64032, 9248, 0]],
        '難' : [12, 13, 0, -2, 12, [256, 20800, 64832, 21472, 64064, 45024, 64064, 8768, 64480, 8768, 21056, 35808, 0]],
        '汝' : [12, 13, 0, -2, 12, [512, 512, 33280, 24544, 1088, 33856, 17472, 2112, 19584, 17280, 35904, 45088, 0]],
        '二' : [12, 13, 0, -2, 12, [0, 0, 0, 32704, 0, 0, 0, 0, 0, 0, 65504, 0, 0]],
        '尼' : [12, 13, 0, -2, 12, [0, 32704, 16448, 16448, 32704, 16384, 18432, 18624, 20224, 34816, 34848, 2016, 0]],
        '弐' : [12, 13, 0, -2, 12, [256, 31008, 288, 65504, 256, 256, 30848, 128, 64, 64, 3616, 61472, 0]],
        '迩' : [12, 13, 0, -2, 12, [1024, 1024, 34784, 18688, 4352, 256, 58688, 10528, 8448, 8960, 20480, 36832, 0]],
        '匂' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16416, 20512, 37664, 39968, 4128, 4384, 3872, 64, 384, 0]],
        '賑' : [12, 13, 0, -2, 12, [0, 65504, 36864, 63424, 36864, 61440, 40928, 38016, 62624, 1184, 21568, 36640, 0]],
        '肉' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33824, 35360, 37152, 42144, 33824, 35360, 37152, 32800, 32992, 0]],
        '虹' : [12, 13, 0, -2, 12, [8192, 10208, 8320, 63616, 43136, 43136, 43136, 63616, 8320, 10368, 10368, 65504, 0]],
        '廿' : [12, 13, 0, -2, 12, [8320, 8320, 8320, 65504, 8320, 8320, 8320, 8320, 8320, 8320, 8320, 16256, 0]],
        '日' : [12, 13, 0, -2, 12, [0, 16320, 8256, 8256, 8256, 8256, 16320, 8256, 8256, 8256, 8256, 16320, 0]],
        '乳' : [12, 13, 0, -2, 12, [0, 64128, 8832, 38016, 17536, 128, 30848, 2176, 4224, 65184, 4256, 12512, 0]],
        '入' : [12, 13, 0, -2, 12, [0, 15360, 1024, 1024, 1024, 1024, 2560, 2560, 4352, 8448, 16512, 32864, 0]],
        '如' : [12, 13, 0, -2, 12, [8192, 8192, 9184, 64032, 18976, 18976, 18976, 35360, 53792, 12832, 18976, 35808, 0]],
        '尿' : [12, 13, 0, -2, 12, [0, 32736, 16416, 16416, 32736, 16640, 23840, 17728, 17792, 35136, 45344, 768, 0]],
        '韮' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 2560, 31680, 2560, 2560, 31680, 2560, 2560, 65504, 0]],
        '任' : [12, 13, 0, -2, 12, [0, 8384, 12032, 16640, 16640, 49408, 24544, 16640, 16640, 16640, 16640, 20448, 0]],
        '妊' : [12, 13, 0, -2, 12, [0, 16576, 20224, 61696, 20736, 20736, 24544, 37120, 57600, 12544, 16640, 20448, 0]],
        '忍' : [12, 13, 0, -2, 12, [0, 32704, 1088, 5184, 2112, 5184, 24960, 0, 1024, 4672, 20512, 36736, 0]],
        '認' : [12, 13, 0, -2, 12, [0, 61408, 288, 58656, 544, 58656, 2240, 0, 57856, 43328, 43040, 59264, 0]],
        '濡' : [12, 13, 0, -2, 12, [0, 8160, 33024, 20448, 2336, 35680, 16640, 8160, 512, 12256, 19104, 35488, 0]],
        '禰' : [12, 13, 0, -2, 12, [0, 24544, 17728, 59680, 16352, 10528, 19872, 52064, 28064, 19296, 19872, 18720, 0]],
        '祢' : [12, 13, 0, -2, 12, [512, 8704, 9184, 62592, 5248, 4224, 8896, 29856, 43168, 8320, 8320, 9088, 0]],
        '寧' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37408, 20544, 36640, 0, 32704, 19008, 65504, 1024, 7168, 0]],
        '葱' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16320, 18752, 38208, 8768, 1152, 4672, 20512, 36736, 0]],
        '猫' : [12, 13, 0, -2, 12, [576, 8768, 45024, 16960, 16384, 45024, 10528, 26912, 45024, 10528, 10528, 53216, 0]],
        '熱' : [12, 13, 0, -2, 12, [4352, 4352, 65472, 4416, 64832, 21056, 47680, 9504, 62624, 0, 21056, 35104, 0]],
        '年' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16896, 33280, 16320, 8704, 8704, 65504, 512, 512, 512, 0]],
        '念' : [12, 13, 0, -2, 12, [0, 7936, 8320, 24384, 32800, 16256, 256, 512, 1024, 21056, 36896, 3968, 0]],
        '捻' : [12, 13, 0, -2, 12, [0, 17280, 17472, 60320, 20480, 18368, 16448, 24704, 49664, 18752, 18464, 51072, 0]],
        '撚' : [12, 13, 0, -2, 12, [128, 20128, 19104, 60384, 21120, 19072, 17536, 18752, 53792, 16384, 19008, 53536, 0]],
        '燃' : [12, 13, 0, -2, 12, [128, 11936, 10912, 44000, 45696, 43648, 9344, 10560, 21024, 16384, 35392, 37152, 0]],
        '粘' : [12, 13, 0, -2, 12, [256, 8448, 43264, 43488, 8448, 63744, 8448, 26592, 29728, 42016, 42016, 10208, 0]],
        '乃' : [12, 13, 0, -2, 12, [0, 32640, 2176, 2176, 2176, 2272, 4128, 4128, 8224, 8224, 16448, 17280, 0]],
        '廼' : [12, 13, 0, -2, 12, [0, 4064, 57984, 8832, 20448, 35488, 60064, 11488, 43040, 20448, 24576, 40928, 0]],
        '之' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 32704, 64, 128, 256, 8704, 9216, 22528, 34304, 33248, 0]],
        '埜' : [12, 13, 0, -2, 12, [8320, 8320, 65504, 8576, 29376, 42144, 8320, 1024, 16256, 1024, 1024, 65504, 0]],
        '嚢' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 9344, 65504, 37152, 16256, 4352, 65504, 4672, 57728, 15456, 0]],
        '悩' : [12, 13, 0, -2, 12, [0, 8736, 10528, 9280, 41024, 41984, 43680, 43296, 43680, 11360, 10272, 12256, 0]],
        '濃' : [12, 13, 0, -2, 12, [640, 640, 36832, 19104, 4064, 35488, 20448, 2048, 20448, 21824, 38016, 34400, 0]],
        '納' : [12, 13, 0, -2, 12, [256, 8448, 16640, 36832, 59680, 10528, 18720, 64160, 11360, 43040, 43040, 10336, 0]],
        '能' : [12, 13, 0, -2, 12, [8192, 16992, 35712, 64000, 544, 64480, 34816, 64096, 35712, 64000, 35360, 39904, 0]],
        '脳' : [12, 13, 0, -2, 12, [0, 57888, 43296, 42048, 57408, 41984, 43680, 59680, 43680, 44128, 43040, 45024, 0]],
        '膿' : [12, 13, 0, -2, 12, [640, 57984, 45024, 43680, 61408, 43680, 45024, 59392, 45024, 46400, 42112, 42592, 0]],
        '農' : [12, 13, 0, -2, 12, [2560, 2560, 32704, 19008, 32704, 19008, 32736, 16384, 32736, 37440, 37248, 15456, 0]],
        '覗' : [12, 13, 0, -2, 12, [0, 64480, 2592, 3040, 59936, 3040, 59936, 43552, 44000, 59712, 2624, 7264, 0]],
        '蚤' : [12, 13, 0, -2, 12, [0, 65504, 9280, 20864, 3584, 61920, 1024, 32704, 17472, 32704, 1056, 65504, 0]],
        '巴' : [12, 13, 0, -2, 12, [0, 65472, 33856, 33856, 33856, 33856, 65472, 32768, 32768, 32800, 32800, 32736, 0]],
        '把' : [12, 13, 0, -2, 12, [0, 20448, 18720, 59680, 18720, 18720, 20448, 26624, 51200, 18464, 18464, 53216, 0]],
        '播' : [12, 13, 0, -2, 12, [0, 20448, 17728, 57728, 20448, 17728, 18720, 24544, 51488, 20448, 18720, 53216, 0]],
        '覇' : [12, 13, 0, -2, 12, [0, 65504, 2560, 32704, 19008, 65504, 43552, 44000, 64032, 9184, 64544, 9312, 0]],
        '杷' : [12, 13, 0, -2, 12, [0, 10208, 9376, 62624, 9376, 9376, 26592, 25600, 46080, 42016, 9248, 10208, 0]],
        '波' : [12, 13, 0, -2, 12, [256, 256, 36832, 18720, 2304, 36800, 18496, 2624, 20800, 20608, 33600, 35872, 0]],
        '派' : [12, 13, 0, -2, 12, [0, 448, 40448, 20576, 6016, 38144, 21792, 5440, 21632, 21632, 42048, 42016, 0]],
        '琶' : [12, 13, 0, -2, 12, [0, 64480, 8320, 64480, 8320, 65504, 17472, 17472, 32704, 16384, 16416, 16352, 0]],
        '破' : [12, 13, 0, -2, 12, [128, 128, 63456, 9376, 9344, 10208, 29728, 54560, 21824, 22720, 20768, 30240, 0]],
        '婆' : [12, 13, 0, -2, 12, [256, 16640, 12256, 35104, 20416, 10816, 18816, 37440, 65504, 4224, 7936, 57568, 0]],
        '罵' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 17408, 32704, 17408, 32704, 17408, 32736, 16416, 38240, 0]],
        '芭' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 17472, 17472, 32704, 16384, 16384, 16416, 16352, 0]],
        '馬' : [12, 13, 0, -2, 12, [0, 32704, 17408, 32640, 17408, 32640, 17408, 32736, 32, 21792, 38176, 192, 0]],
        '俳' : [12, 13, 0, -2, 12, [128, 8832, 16096, 17024, 17024, 52960, 17024, 17024, 24192, 17120, 17536, 22656, 0]],
        '廃' : [12, 13, 0, -2, 12, [512, 512, 32736, 17696, 21824, 18560, 24512, 25888, 24544, 34048, 35104, 12768, 0]],
        '拝' : [12, 13, 0, -2, 12, [0, 20448, 16640, 57600, 20448, 16640, 16640, 24832, 53216, 16640, 16640, 49408, 0]],
        '排' : [12, 13, 0, -2, 12, [128, 17024, 24288, 57984, 17024, 20192, 17024, 25216, 56960, 17120, 17536, 55424, 0]],
        '敗' : [12, 13, 0, -2, 12, [512, 64000, 35808, 64064, 35392, 63552, 35136, 35136, 63616, 128, 21312, 35872, 0]],
        '杯' : [12, 13, 0, -2, 12, [0, 12256, 8320, 61568, 8448, 8448, 25408, 25920, 47392, 41248, 8448, 8448, 0]],
        '盃' : [12, 13, 0, -2, 12, [0, 65504, 512, 1024, 3200, 13376, 50208, 1024, 32704, 19008, 19008, 65504, 0]],
        '牌' : [12, 13, 0, -2, 12, [256, 41472, 45024, 43296, 65504, 35104, 35104, 61408, 41600, 42112, 45024, 41088, 0]],
        '背' : [12, 13, 0, -2, 12, [4608, 4704, 62336, 4608, 4640, 65504, 8256, 16320, 8256, 16320, 8256, 8384, 0]],
        '肺' : [12, 13, 0, -2, 12, [256, 57600, 45024, 41216, 57600, 45024, 43296, 59680, 43296, 43296, 43360, 41216, 0]],
        '輩' : [12, 13, 0, -2, 12, [512, 2560, 64480, 2560, 31680, 4608, 65504, 9344, 16256, 9344, 65504, 1024, 0]],
        '配' : [12, 13, 0, -2, 12, [0, 64480, 8224, 8224, 63520, 44000, 43520, 55808, 35328, 64032, 35360, 64480, 0]],
        '倍' : [12, 13, 0, -2, 12, [256, 4352, 8160, 8768, 8768, 61408, 8192, 8192, 10208, 9248, 9248, 10208, 0]],
        '培' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8768, 62016, 12256, 8192, 8192, 10208, 13344, 50208, 2016, 0]],
        '媒' : [12, 13, 0, -2, 12, [1088, 17472, 20448, 62528, 22464, 21568, 22464, 37120, 61408, 8960, 17728, 22816, 0]],
        '梅' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 63488, 12224, 9536, 25920, 28640, 43584, 43584, 12256, 8320, 0]],
        '楳' : [12, 13, 0, -2, 12, [1088, 9280, 12256, 62528, 10176, 9280, 26560, 24832, 45024, 41728, 9536, 14624, 0]],
        '煤' : [12, 13, 0, -2, 12, [1088, 9280, 12256, 42048, 44992, 46144, 10176, 8448, 24544, 17152, 34112, 39200, 0]],
        '狽' : [12, 13, 0, -2, 12, [0, 10208, 42016, 18400, 17440, 42976, 9248, 25632, 42976, 8192, 8768, 52256, 0]],
        '買' : [12, 13, 0, -2, 12, [0, 32736, 18720, 32736, 0, 16320, 8256, 16320, 8256, 16320, 4224, 24672, 0]],
        '売' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 0, 65504, 32800, 4608, 4608, 8736, 50144, 0]],
        '賠' : [12, 13, 0, -2, 12, [256, 61696, 38880, 62016, 37440, 65504, 36864, 36864, 63456, 1056, 42016, 38880, 0]],
        '陪' : [12, 13, 0, -2, 12, [256, 57600, 45024, 41536, 49728, 45024, 40960, 40960, 42976, 50208, 33824, 34784, 0]],
        '這' : [12, 13, 0, -2, 12, [0, 1984, 16384, 12256, 0, 1984, 57344, 10176, 9280, 10176, 20480, 36832, 0]],
        '蝿' : [12, 13, 0, -2, 12, [0, 10176, 9536, 65472, 42304, 45024, 43296, 65504, 10528, 12256, 10496, 63968, 0]],
        '秤' : [12, 13, 0, -2, 12, [0, 8160, 57600, 10528, 9536, 61696, 8448, 28640, 24832, 41216, 41216, 8448, 0]],
        '矧' : [12, 13, 0, -2, 12, [0, 34592, 63776, 41248, 10016, 64544, 9248, 10016, 20768, 20768, 37152, 34336, 0]],
        '萩' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 6272, 57472, 8864, 64192, 8320, 28800, 43328, 9760, 0]],
        '伯' : [12, 13, 0, -2, 12, [256, 8448, 8704, 20448, 18464, 51232, 18464, 20448, 18464, 18464, 18464, 20448, 0]],
        '剥' : [12, 13, 0, -2, 12, [0, 63520, 2336, 31008, 2336, 64800, 4384, 54560, 22816, 21536, 37408, 12384, 0]],
        '博' : [12, 13, 0, -2, 12, [256, 16672, 24544, 16640, 61408, 18720, 20448, 18720, 24544, 16448, 17472, 17088, 0]],
        '拍' : [12, 13, 0, -2, 12, [256, 16640, 16896, 61408, 18464, 18464, 18464, 28640, 51232, 18464, 18464, 53216, 0]],
        '柏' : [12, 13, 0, -2, 12, [256, 8448, 8704, 63456, 9248, 9248, 25632, 26592, 46112, 42016, 9248, 10208, 0]],
        '泊' : [12, 13, 0, -2, 12, [256, 256, 33280, 20448, 2080, 34848, 18464, 4064, 2080, 10272, 18464, 36832, 0]],
        '白' : [12, 13, 0, -2, 12, [512, 1024, 2048, 32704, 16448, 16448, 16448, 32704, 16448, 16448, 16448, 32704, 0]],
        '箔' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 256, 10208, 37920, 17440, 2016, 9248, 17440, 34784, 0]],
        '粕' : [12, 13, 0, -2, 12, [256, 8448, 45568, 42976, 9248, 62496, 9248, 26592, 29728, 42016, 42016, 10208, 0]],
        '舶' : [12, 13, 0, -2, 12, [8448, 16640, 61952, 38880, 54304, 37920, 64544, 38880, 54304, 37920, 37920, 47072, 0]],
        '薄' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 24544, 12864, 40896, 21056, 8160, 64, 18496, 33984, 0]],
        '迫' : [12, 13, 0, -2, 12, [256, 512, 18400, 9248, 1056, 2016, 58400, 9248, 10208, 8192, 20480, 36832, 0]],
        '曝' : [12, 13, 0, -2, 12, [0, 4064, 59424, 45024, 42112, 49120, 58496, 49120, 42048, 43296, 58688, 2848, 0]],
        '漠' : [12, 13, 0, -2, 12, [1088, 1088, 40928, 17472, 4064, 34848, 20448, 2080, 24544, 17024, 33856, 38944, 0]],
        '爆' : [12, 13, 0, -2, 12, [0, 12256, 10272, 45024, 42112, 49120, 42112, 16352, 21568, 18720, 34112, 35616, 0]],
        '縛' : [12, 13, 0, -2, 12, [256, 8480, 24544, 33024, 61408, 10528, 20448, 59680, 16352, 41024, 42048, 8896, 0]],
        '莫' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 8320, 16256, 8320, 65504, 2560, 12672, 49248, 0]],
        '駁' : [12, 13, 0, -2, 12, [0, 64032, 41280, 63616, 41280, 64032, 40960, 64032, 2368, 43136, 2368, 12832, 0]],
        '麦' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 16256, 1024, 65504, 4096, 16256, 53376, 3840, 28896, 0]],
        '函' : [12, 13, 0, -2, 12, [0, 65504, 2048, 3072, 42144, 38176, 33824, 38176, 42144, 52320, 32800, 65504, 0]],
        '箱' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 9184, 64032, 9184, 12832, 27616, 41504, 9184, 0]],
        '硲' : [12, 13, 0, -2, 12, [0, 576, 62496, 8192, 8576, 8768, 29728, 53248, 22496, 21536, 21536, 30688, 0]],
        '箸' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 2048, 32704, 2304, 65504, 4224, 16256, 53376, 8064, 0]],
        '肇' : [12, 13, 0, -2, 12, [512, 64000, 992, 32320, 18816, 19040, 32640, 33920, 49120, 1152, 65504, 1024, 0]],
        '筈' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 32704, 1024, 65504, 1024, 32704, 16448, 32704, 0]],
        '櫨' : [12, 13, 0, -2, 12, [960, 8704, 12256, 63776, 11200, 10496, 28640, 30016, 42304, 45024, 10912, 16352, 0]],
        '幡' : [12, 13, 0, -2, 12, [0, 12256, 9536, 8576, 65504, 42304, 43296, 49120, 43296, 45024, 10528, 12256, 0]],
        '肌' : [12, 13, 0, -2, 12, [0, 62400, 37440, 37440, 62016, 37440, 37440, 62016, 37440, 37440, 37952, 46176, 0]],
        '畑' : [12, 13, 0, -2, 12, [0, 10208, 9504, 44320, 46368, 42272, 10208, 9504, 21792, 19744, 34080, 34784, 0]],
        '畠' : [12, 13, 0, -2, 12, [512, 1024, 16320, 8256, 16320, 8256, 32736, 17440, 32736, 17440, 17440, 32736, 0]],
        '八' : [12, 13, 0, -2, 12, [0, 7936, 256, 256, 8448, 8320, 8320, 16512, 16448, 16448, 32800, 32800, 0]],
        '鉢' : [12, 13, 0, -2, 12, [128, 12416, 18560, 34784, 63616, 8576, 64192, 9376, 43136, 42976, 8320, 63616, 0]],
        '溌' : [12, 13, 0, -2, 12, [0, 7808, 33440, 21152, 3136, 34752, 19104, 4736, 20448, 17024, 33952, 39136, 0]],
        '発' : [12, 13, 0, -2, 12, [0, 31296, 2688, 35104, 20800, 16256, 21088, 37376, 32704, 4608, 8736, 50144, 0]],
        '醗' : [12, 13, 0, -2, 12, [0, 63104, 17056, 19104, 62528, 38848, 55968, 45696, 40928, 62080, 38048, 62688, 0]],
        '髪' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31840, 16768, 65120, 18816, 17408, 65504, 10368, 51072, 14432, 0]],
        '伐' : [12, 13, 0, -2, 12, [512, 8768, 8736, 16896, 24544, 49408, 16672, 16704, 16512, 16768, 17984, 22560, 0]],
        '罰' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 8192, 64800, 288, 63776, 288, 63776, 34848, 63584, 0]],
        '抜' : [12, 13, 0, -2, 12, [1024, 17408, 24544, 58368, 17408, 18368, 18496, 27200, 53568, 20608, 17216, 52256, 0]],
        '筏' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 8448, 8480, 20448, 49408, 16704, 16512, 17216, 19488, 0]],
        '閥' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 33824, 38048, 44960, 57888, 41760, 44256, 0]],
        '鳩' : [12, 13, 0, -2, 12, [256, 16896, 18368, 62528, 22464, 21568, 22496, 21504, 22496, 20512, 35488, 35552, 0]],
        '噺' : [12, 13, 0, -2, 12, [1024, 1120, 65408, 43264, 43264, 49120, 42272, 48928, 42272, 44576, 62752, 9248, 0]],
        '塙' : [12, 13, 0, -2, 12, [512, 16896, 24544, 17536, 59264, 16384, 24544, 20512, 22432, 21664, 63392, 4192, 0]],
        '蛤' : [12, 13, 0, -2, 12, [0, 8640, 8736, 64512, 43008, 44000, 43008, 63488, 9184, 10784, 10784, 64480, 0]],
        '隼' : [12, 13, 0, -2, 12, [4352, 4352, 16352, 25088, 49088, 8704, 16320, 8704, 8704, 65504, 512, 512, 0]],
        '伴' : [12, 13, 0, -2, 12, [256, 8448, 10528, 17728, 16640, 53216, 16640, 16640, 20448, 16640, 16640, 16640, 0]],
        '判' : [12, 13, 0, -2, 12, [8192, 8224, 42272, 43296, 8480, 64800, 8480, 8480, 64800, 8480, 8224, 8288, 0]],
        '半' : [12, 13, 0, -2, 12, [1024, 1024, 17472, 9344, 1024, 32704, 1024, 1024, 65504, 1024, 1024, 1024, 0]],
        '反' : [12, 13, 0, -2, 12, [0, 16352, 8192, 8192, 16320, 8256, 9280, 9280, 17024, 16768, 1600, 14368, 0]],
        '叛' : [12, 13, 0, -2, 12, [0, 10208, 44032, 46080, 10176, 62528, 9280, 10560, 59712, 8320, 17216, 19488, 0]],
        '帆' : [12, 13, 0, -2, 12, [0, 9152, 8768, 64064, 43584, 43840, 43712, 43584, 43584, 47680, 9280, 9312, 0]],
        '搬' : [12, 13, 0, -2, 12, [1024, 18880, 23872, 62784, 23872, 22112, 29696, 24544, 54560, 21824, 21696, 55072, 0]],
        '斑' : [12, 13, 0, -2, 12, [1024, 58848, 17472, 24128, 16960, 19008, 60384, 17472, 17472, 19008, 18496, 57824, 0]],
        '板' : [12, 13, 0, -2, 12, [0, 10208, 9216, 62464, 10176, 9280, 25664, 26944, 43328, 41088, 9024, 11296, 0]],
        '氾' : [12, 13, 0, -2, 12, [0, 4032, 34880, 18496, 2112, 34880, 18880, 2048, 2048, 10272, 18464, 36832, 0]],
        '汎' : [12, 13, 0, -2, 12, [0, 3968, 34944, 18560, 2176, 35968, 19072, 2176, 18560, 18592, 37024, 37088, 0]],
        '版' : [12, 13, 0, -2, 12, [0, 42976, 41984, 41984, 63424, 33856, 33856, 59712, 43328, 41088, 41792, 44064, 0]],
        '犯' : [12, 13, 0, -2, 12, [0, 10176, 42048, 17472, 17472, 42048, 9664, 25600, 41984, 9248, 9248, 51168, 0]],
        '班' : [12, 13, 0, -2, 12, [0, 62432, 19008, 19008, 19008, 19008, 60384, 19008, 16960, 16960, 17472, 62944, 0]],
        '畔' : [12, 13, 0, -2, 12, [128, 63616, 44192, 43680, 43136, 65504, 43136, 43136, 45024, 43136, 63616, 128, 0]],
        '繁' : [12, 13, 0, -2, 12, [16896, 16896, 32736, 54336, 32064, 21888, 65120, 4608, 9280, 65504, 9344, 50272, 0]],
        '般' : [12, 13, 0, -2, 12, [8192, 17344, 62016, 37440, 53824, 37984, 61440, 40928, 53792, 37440, 37248, 46688, 0]],
        '藩' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 20448, 9536, 40928, 17728, 2336, 12256, 18720, 36832, 0]],
        '販' : [12, 13, 0, -2, 12, [0, 63456, 37888, 62464, 38848, 62528, 37952, 38208, 62784, 128, 21312, 35872, 0]],
        '範' : [12, 13, 0, -2, 12, [16896, 16896, 31712, 38016, 64992, 42272, 64800, 42272, 64864, 8448, 64800, 8672, 0]],
        '釆' : [12, 13, 0, -2, 12, [0, 960, 31744, 17472, 9344, 1024, 65504, 1536, 3328, 13440, 50272, 1024, 0]],
        '煩' : [12, 13, 0, -2, 12, [0, 12256, 8448, 42976, 46112, 42976, 42016, 10208, 21536, 22496, 33856, 38944, 0]],
        '頒' : [12, 13, 0, -2, 12, [0, 26592, 4352, 38880, 33824, 63456, 21536, 22496, 37920, 38880, 4672, 27680, 0]],
        '飯' : [12, 13, 0, -2, 12, [0, 29664, 35328, 8704, 64480, 35360, 64160, 35488, 64064, 41536, 37024, 60192, 0]],
        '挽' : [12, 13, 0, -2, 12, [0, 18304, 18560, 61696, 20448, 18720, 18720, 28640, 49792, 17024, 17568, 55520, 0]],
        '晩' : [12, 13, 0, -2, 12, [0, 1920, 59520, 45312, 45024, 43296, 59680, 45024, 41600, 41600, 58528, 6368, 0]],
        '番' : [12, 13, 0, -2, 12, [0, 32704, 9344, 5376, 65504, 5248, 9280, 65440, 9344, 16256, 9344, 16256, 0]],
        '盤' : [12, 13, 0, -2, 12, [4096, 9152, 31296, 19552, 27584, 19008, 63872, 19040, 32704, 19008, 19008, 65504, 0]],
        '磐' : [12, 13, 0, -2, 12, [4096, 9152, 31296, 19552, 27584, 19008, 63872, 19008, 65504, 4160, 12352, 57280, 0]],
        '蕃' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 9344, 65504, 5248, 16320, 58528, 9344, 16256, 0]],
        '蛮' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2304, 18752, 37152, 16256, 9344, 9344, 16288, 1056, 65504, 0]],
        '匪' : [12, 13, 0, -2, 12, [0, 65504, 34048, 34048, 48608, 34048, 40384, 34048, 48608, 35072, 37120, 65504, 0]],
        '卑' : [12, 13, 0, -2, 12, [1024, 2048, 32704, 17472, 32704, 17472, 17472, 32704, 2304, 4352, 65504, 256, 0]],
        '否' : [12, 13, 0, -2, 12, [0, 65504, 512, 1024, 7296, 58432, 1056, 1024, 16256, 8320, 8320, 16256, 0]],
        '妃' : [12, 13, 0, -2, 12, [0, 18400, 16416, 61472, 20512, 22496, 21504, 37888, 58368, 13344, 17440, 18400, 0]],
        '庇' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 20736, 20736, 24032, 20736, 20736, 37120, 37152, 15840, 0]],
        '彼' : [12, 13, 0, -2, 12, [256, 8448, 20448, 35104, 2304, 12224, 18496, 51776, 20800, 20608, 17216, 19488, 0]],
        '悲' : [12, 13, 0, -2, 12, [512, 2560, 64480, 2560, 31680, 2560, 64480, 4608, 1024, 4672, 20512, 36736, 0]],
        '扉' : [12, 13, 0, -2, 12, [0, 65504, 0, 32736, 16416, 32736, 17664, 24000, 17664, 48608, 35072, 12544, 0]],
        '批' : [12, 13, 0, -2, 12, [256, 18688, 18688, 59680, 20288, 18816, 18688, 26880, 51456, 18720, 20256, 55776, 0]],
        '披' : [12, 13, 0, -2, 12, [256, 16640, 20448, 59680, 18688, 20416, 18496, 27200, 53568, 20608, 17216, 52256, 0]],
        '斐' : [12, 13, 0, -2, 12, [512, 2560, 64480, 2560, 64480, 2560, 4608, 65504, 4224, 4352, 3584, 29120, 0]],
        '比' : [12, 13, 0, -2, 12, [512, 16896, 16896, 16992, 31616, 16896, 16896, 16896, 16896, 16928, 23072, 58336, 0]],
        '泌' : [12, 13, 0, -2, 12, [0, 1024, 33344, 16448, 1152, 33920, 21792, 5408, 22048, 17408, 35904, 37824, 0]],
        '疲' : [12, 13, 0, -2, 12, [512, 512, 32736, 16640, 57312, 20768, 20736, 57280, 21568, 21568, 41856, 44128, 0]],
        '皮' : [12, 13, 0, -2, 12, [512, 512, 16352, 8736, 8704, 16320, 8256, 9280, 17536, 17152, 3200, 28768, 0]],
        '碑' : [12, 13, 0, -2, 12, [128, 256, 63456, 9504, 10208, 9504, 29984, 55264, 21120, 21632, 24544, 28800, 0]],
        '秘' : [12, 13, 0, -2, 12, [0, 6656, 57632, 8224, 8768, 62016, 10880, 27296, 27424, 41472, 42528, 10720, 0]],
        '緋' : [12, 13, 0, -2, 12, [128, 8832, 24288, 33408, 57984, 12000, 17024, 57984, 16000, 41696, 42112, 14464, 0]],
        '罷' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 18944, 64480, 35328, 64480, 35328, 64480, 35328, 39392, 0]],
        '肥' : [12, 13, 0, -2, 12, [0, 63456, 38176, 38176, 62752, 38176, 38880, 62464, 37888, 37920, 37920, 47072, 0]],
        '被' : [12, 13, 0, -2, 12, [256, 16640, 20448, 59680, 10496, 12224, 18496, 27200, 53568, 20608, 17216, 19488, 0]],
        '誹' : [12, 13, 0, -2, 12, [128, 57984, 7904, 57984, 640, 61152, 640, 640, 65152, 41696, 42112, 63616, 0]],
        '費' : [12, 13, 0, -2, 12, [4608, 32704, 4672, 65472, 37376, 65504, 8352, 16320, 8320, 16256, 8320, 49248, 0]],
        '避' : [12, 13, 0, -2, 12, [128, 7296, 38880, 21824, 7488, 5088, 56448, 22496, 21632, 23680, 40960, 40928, 0]],
        '非' : [12, 13, 0, -2, 12, [256, 2304, 63968, 2304, 2304, 31200, 2304, 2304, 63744, 2528, 4352, 24832, 0]],
        '飛' : [12, 13, 0, -2, 12, [0, 32544, 320, 5504, 58688, 9504, 9280, 65408, 9536, 9504, 17536, 17504, 0]],
        '樋' : [12, 13, 0, -2, 12, [0, 10176, 8320, 62432, 10912, 9184, 31392, 27616, 43680, 43680, 11264, 13280, 0]],
        '簸' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 19424, 65184, 19072, 31712, 18976, 64832, 18624, 34592, 0]],
        '備' : [12, 13, 0, -2, 12, [1152, 9344, 16352, 17536, 20448, 51200, 20448, 22816, 28640, 18720, 20448, 18720, 0]],
        '尾' : [12, 13, 0, -2, 12, [0, 32704, 16448, 16448, 32704, 17408, 24448, 17408, 32704, 17408, 33824, 34784, 0]],
        '微' : [12, 13, 0, -2, 12, [1152, 5248, 21728, 40864, 32, 16288, 19104, 51872, 19008, 21312, 20640, 17184, 0]],
        '枇' : [12, 13, 0, -2, 12, [128, 9344, 9344, 62592, 9440, 10112, 25728, 25728, 46208, 42112, 10144, 11488, 0]],
        '毘' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 32704, 16896, 16992, 31616, 16896, 16928, 64480, 0]],
        '琵' : [12, 13, 0, -2, 12, [0, 64480, 8320, 64480, 8320, 64480, 16896, 16992, 31616, 16896, 16928, 64480, 0]],
        '眉' : [12, 13, 0, -2, 12, [0, 32736, 16928, 32736, 16384, 24544, 20512, 24544, 20512, 40928, 36896, 8160, 0]],
        '美' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 16256, 1024, 65504, 1024, 65504, 2560, 12672, 49248, 0]],
        '鼻' : [12, 13, 0, -2, 12, [1024, 2048, 16256, 8320, 16256, 8320, 32704, 17472, 32704, 4352, 65504, 8448, 0]],
        '柊' : [12, 13, 0, -2, 12, [512, 8704, 9184, 62496, 10816, 8576, 26208, 28928, 41152, 44032, 8960, 8416, 0]],
        '稗' : [12, 13, 0, -2, 12, [256, 6656, 61408, 10528, 12256, 63776, 10528, 28640, 25216, 42112, 45024, 8320, 0]],
        '匹' : [12, 13, 0, -2, 12, [0, 65504, 35072, 35072, 35072, 37120, 57824, 32768, 32768, 32768, 65504, 32768, 0]],
        '疋' : [12, 13, 0, -2, 12, [0, 65504, 1056, 1088, 1024, 17408, 18304, 17408, 17408, 25600, 39936, 33760, 0]],
        '髭' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31840, 16768, 65120, 18816, 29952, 4576, 23808, 20768, 64992, 0]],
        '彦' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 4352, 32736, 16896, 23744, 17152, 40032, 33152, 15872, 0]],
        '膝' : [12, 13, 0, -2, 12, [256, 57600, 49120, 42304, 59680, 45696, 42048, 59680, 42304, 41344, 43328, 45856, 0]],
        '菱' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 5376, 32704, 1024, 65504, 4352, 16352, 53504, 3584, 29120, 0]],
        '肘' : [12, 13, 0, -2, 12, [64, 61504, 36928, 38880, 61504, 36928, 37952, 62016, 36928, 36928, 36928, 45504, 0]],
        '弼' : [12, 13, 0, -2, 12, [0, 61152, 9248, 9248, 61152, 35456, 35456, 61152, 10784, 10784, 10784, 52928, 0]],
        '必' : [12, 13, 0, -2, 12, [0, 6144, 1536, 0, 4160, 20800, 21024, 37920, 38944, 4224, 12416, 20224, 0]],
        '畢' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 32704, 9344, 65504, 9344, 9344, 32704, 1024, 0]],
        '筆' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42048, 16256, 1152, 65504, 1152, 16256, 1024, 65504, 1024, 0]],
        '逼' : [12, 13, 0, -2, 12, [0, 8160, 33856, 18368, 0, 4064, 59680, 12256, 10528, 12256, 20480, 36832, 0]],
        '桧' : [12, 13, 0, -2, 12, [0, 9152, 9248, 63488, 10208, 8192, 24576, 26592, 45312, 41504, 9248, 10208, 0]],
        '姫' : [12, 13, 0, -2, 12, [0, 18400, 17536, 62592, 22496, 21536, 21536, 38880, 58496, 13440, 17536, 18400, 0]],
        '媛' : [12, 13, 0, -2, 12, [0, 24544, 17696, 62784, 24544, 20736, 24544, 37376, 59328, 14912, 16768, 20064, 0]],
        '紐' : [12, 13, 0, -2, 12, [0, 10208, 16672, 37152, 61728, 8480, 18400, 61984, 8736, 45600, 41504, 12256, 0]],
        '百' : [12, 13, 0, -2, 12, [0, 65504, 1024, 2048, 32704, 16448, 16448, 32704, 16448, 16448, 16448, 32704, 0]],
        '謬' : [12, 13, 0, -2, 12, [0, 61152, 544, 60064, 1632, 59936, 384, 3648, 57760, 42496, 41152, 61184, 0]],
        '俵' : [12, 13, 0, -2, 12, [256, 8448, 16352, 16640, 20448, 49408, 24544, 16928, 17728, 23680, 17472, 20256, 0]],
        '彪' : [12, 13, 0, -2, 12, [4096, 7264, 4480, 32256, 21088, 32128, 20480, 24160, 16768, 37888, 37920, 26592, 0]],
        '標' : [12, 13, 0, -2, 12, [0, 12256, 8832, 65504, 10912, 12256, 24576, 28640, 40960, 45024, 9536, 11040, 0]],
        '氷' : [12, 13, 0, -2, 12, [1024, 1024, 17408, 9280, 1152, 62720, 5632, 9472, 9344, 17472, 33824, 7168, 0]],
        '漂' : [12, 13, 0, -2, 12, [0, 8160, 33408, 20448, 2720, 36832, 16384, 4064, 16384, 24544, 35136, 37664, 0]],
        '瓢' : [12, 13, 0, -2, 12, [0, 65504, 10560, 64832, 43328, 64832, 1344, 64832, 1344, 64960, 22304, 46112, 0]],
        '票' : [12, 13, 0, -2, 12, [0, 65504, 2560, 32704, 19008, 32704, 0, 32704, 0, 65504, 9344, 19520, 0]],
        '表' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 1024, 65504, 2560, 12832, 53568, 4224, 31840, 0]],
        '評' : [12, 13, 0, -2, 12, [0, 61408, 256, 57600, 2336, 58688, 256, 256, 61408, 41216, 41216, 57600, 0]],
        '豹' : [12, 13, 0, -2, 12, [512, 15872, 54240, 23072, 12832, 53280, 14880, 51488, 14368, 51232, 2112, 12672, 0]],
        '廟' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 36864, 65504, 42272, 48608, 42272, 48608, 37152, 65056, 37472, 0]],
        '描' : [12, 13, 0, -2, 12, [576, 16960, 20448, 57920, 16960, 20448, 18720, 26912, 53216, 18720, 18720, 53216, 0]],
        '病' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 57312, 16640, 24544, 53536, 21152, 23648, 36896, 36960, 0]],
        '秒' : [12, 13, 0, -2, 12, [256, 6400, 57600, 9536, 10528, 57600, 8448, 25344, 28704, 41024, 41344, 11776, 0]],
        '苗' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 4352, 32704, 17472, 17472, 32704, 17472, 17472, 32704, 0]],
        '錨' : [12, 13, 0, -2, 12, [576, 12864, 20448, 33344, 62016, 10208, 64800, 9504, 47072, 42272, 9504, 65504, 0]],
        '鋲' : [12, 13, 0, -2, 12, [0, 12480, 19200, 33280, 64480, 8768, 64064, 8768, 47072, 40960, 8768, 64544, 0]],
        '蒜' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 64480, 0, 65504, 4224, 21184, 38048, 12672, 0]],
        '蛭' : [12, 13, 0, -2, 12, [0, 10208, 8320, 63744, 43552, 44000, 43136, 63616, 9184, 10368, 10368, 65504, 0]],
        '鰭' : [12, 13, 0, -2, 12, [256, 28960, 38880, 8512, 65504, 43296, 64320, 44512, 64032, 992, 43552, 44000, 0]],
        '品' : [12, 13, 0, -2, 12, [0, 7936, 4352, 4352, 4352, 7936, 0, 64480, 35360, 35360, 35360, 64480, 0]],
        '彬' : [12, 13, 0, -2, 12, [0, 8736, 8768, 65408, 8704, 8736, 26432, 27264, 45568, 41504, 8768, 8832, 0]],
        '斌' : [12, 13, 0, -2, 12, [128, 20128, 16544, 57472, 16352, 41600, 41600, 19392, 19008, 43584, 35744, 39968, 0]],
        '浜' : [12, 13, 0, -2, 12, [0, 192, 36608, 18432, 4064, 34944, 18560, 2176, 24544, 16384, 34880, 36896, 0]],
        '瀕' : [12, 13, 0, -2, 12, [2048, 11232, 44160, 27104, 16160, 35296, 23328, 2528, 22816, 17888, 35136, 45600, 0]],
        '貧' : [12, 13, 0, -2, 12, [0, 5888, 8320, 32704, 34976, 4352, 16256, 8320, 16256, 8320, 16256, 49248, 0]],
        '賓' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 16000, 17472, 16256, 57472, 16256, 8320, 16256, 49248, 0]],
        '頻' : [12, 13, 0, -2, 12, [8192, 10208, 47232, 41952, 64032, 9184, 43552, 42976, 25120, 3040, 12608, 50720, 0]],
        '敏' : [12, 13, 0, -2, 12, [16640, 16640, 32224, 33088, 65088, 21568, 21824, 64832, 42112, 42112, 64832, 2592, 0]],
        '瓶' : [12, 13, 0, -2, 12, [34816, 20448, 4608, 64000, 21440, 21056, 64320, 21696, 21568, 21696, 38688, 38944, 0]],
        '不' : [12, 13, 0, -2, 12, [0, 65504, 256, 512, 1024, 3328, 13440, 50240, 1056, 1024, 1024, 1024, 0]],
        '付' : [12, 13, 0, -2, 12, [64, 8256, 8256, 20448, 16448, 49216, 17472, 16960, 16448, 16448, 16448, 16832, 0]],
        '埠' : [12, 13, 0, -2, 12, [128, 8448, 10208, 9248, 63456, 9216, 10208, 9248, 10208, 12416, 53216, 128, 0]],
        '夫' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 32704, 1024, 1024, 65504, 1024, 2560, 4352, 8320, 49248, 0]],
        '婦' : [12, 13, 0, -2, 12, [0, 20416, 16448, 61376, 16448, 24544, 20768, 40896, 59712, 10560, 18880, 16640, 0]],
        '富' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 49056, 4352, 4352, 32704, 17472, 32704, 17472, 32704, 0]],
        '冨' : [12, 13, 0, -2, 12, [0, 65504, 32800, 49056, 4352, 7936, 0, 32704, 17472, 32704, 17472, 32704, 0]],
        '布' : [12, 13, 0, -2, 12, [2048, 2048, 65504, 4096, 4608, 8704, 32704, 41536, 8768, 8768, 8896, 512, 0]],
        '府' : [12, 13, 0, -2, 12, [512, 512, 32736, 18496, 18496, 22496, 28736, 21056, 20800, 36928, 36928, 4288, 0]],
        '怖' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8704, 41600, 42112, 45024, 46240, 42144, 9376, 9440, 8320, 0]],
        '扶' : [12, 13, 0, -2, 12, [256, 16640, 16640, 61408, 16640, 16640, 20448, 24832, 49408, 17024, 17472, 55328, 0]],
        '敷' : [12, 13, 0, -2, 12, [8448, 9472, 64992, 8512, 65088, 42048, 64832, 42304, 65152, 9344, 17728, 39456, 0]],
        '斧' : [12, 13, 0, -2, 12, [0, 6528, 57440, 6528, 3584, 12672, 56928, 8192, 16352, 8448, 16640, 16640, 0]],
        '普' : [12, 13, 0, -2, 12, [4224, 4352, 32704, 2560, 19008, 10880, 2560, 65504, 8320, 16256, 8320, 16256, 0]],
        '浮' : [12, 13, 0, -2, 12, [0, 4064, 32800, 18752, 1344, 32768, 18368, 128, 256, 12256, 16640, 33536, 0]],
        '父' : [12, 13, 0, -2, 12, [0, 4352, 4352, 8320, 16448, 32800, 4352, 2560, 1024, 2560, 12672, 49248, 0]],
        '符' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 8320, 8256, 24544, 49216, 17472, 16960, 16448, 16576, 0]],
        '腐' : [12, 13, 0, -2, 12, [512, 512, 32736, 18496, 24544, 29248, 20800, 32736, 43680, 46432, 10400, 8288, 0]],
        '膚' : [12, 13, 0, -2, 12, [1920, 1024, 32736, 17440, 24512, 17408, 24544, 20768, 24544, 34880, 36800, 2112, 0]],
        '芙' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 32704, 1024, 65504, 1024, 2560, 12672, 49248, 0]],
        '譜' : [12, 13, 0, -2, 12, [1088, 58496, 4064, 57984, 2720, 59072, 640, 4064, 58400, 42976, 42016, 59360, 0]],
        '負' : [12, 13, 0, -2, 12, [0, 8064, 8448, 32704, 41024, 16320, 8256, 16320, 8256, 16320, 4224, 24672, 0]],
        '賦' : [12, 13, 0, -2, 12, [128, 61088, 41120, 57472, 49120, 57984, 41600, 43968, 59968, 2624, 43936, 39968, 0]],
        '赴' : [12, 13, 0, -2, 12, [8448, 8448, 63744, 8448, 8576, 64832, 8480, 14592, 41216, 41216, 61440, 36832, 0]],
        '阜' : [12, 13, 0, -2, 12, [1024, 2048, 16256, 8320, 16256, 8192, 16320, 8256, 16320, 1024, 65504, 1024, 0]],
        '附' : [12, 13, 0, -2, 12, [64, 58432, 42048, 44000, 55360, 43072, 43584, 43328, 43072, 51264, 34880, 35008, 0]],
        '侮' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 18432, 20416, 50496, 17728, 24544, 19008, 19008, 20448, 16512, 0]],
        '撫' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 60736, 17728, 24544, 17728, 25920, 53216, 16384, 19104, 53920, 0]],
        '武' : [12, 13, 0, -2, 12, [256, 31040, 288, 256, 65504, 4224, 4224, 23680, 20544, 20544, 24096, 57376, 0]],
        '舞' : [12, 13, 0, -2, 12, [16384, 32736, 43648, 65504, 10880, 65504, 8320, 15328, 51840, 10880, 6112, 57472, 0]],
        '葡' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 20736, 32736, 35360, 65440, 18720, 32544, 18720, 32544, 18880, 0]],
        '蕪' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 20736, 32736, 43648, 65504, 10880, 65504, 0, 21056, 35104, 0]],
        '部' : [12, 13, 0, -2, 12, [8192, 8672, 64800, 18720, 18752, 64896, 320, 288, 64800, 34080, 34240, 64768, 0]],
        '封' : [12, 13, 0, -2, 12, [8224, 8224, 63520, 9184, 8224, 64544, 8736, 8480, 63520, 8224, 8224, 64608, 0]],
        '楓' : [12, 13, 0, -2, 12, [0, 12256, 10272, 65504, 10784, 12192, 27296, 27296, 44960, 45600, 8864, 12192, 0]],
        '風' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 17472, 24384, 21824, 21824, 24384, 17440, 33952, 49056, 0]],
        '葺' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8320, 16256, 0, 65504, 8320, 16256, 8320, 16352, 49280, 0]],
        '蕗' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4608, 64480, 35360, 64832, 8640, 14880, 42976, 41504, 64480, 0]],
        '伏' : [12, 13, 0, -2, 12, [256, 8512, 8480, 16640, 24544, 49408, 16640, 16640, 17024, 17024, 17472, 22560, 0]],
        '副' : [12, 13, 0, -2, 12, [0, 64544, 288, 31008, 18720, 31008, 288, 64800, 42272, 64544, 42016, 64608, 0]],
        '復' : [12, 13, 0, -2, 12, [1024, 9216, 20448, 38976, 4032, 10304, 20416, 50176, 20448, 21568, 17280, 23648, 0]],
        '幅' : [12, 13, 0, -2, 12, [0, 12256, 8192, 10176, 62528, 42944, 40960, 45024, 43296, 45024, 10528, 12256, 0]],
        '服' : [12, 13, 0, -2, 12, [0, 63456, 37920, 37888, 63424, 37952, 37952, 62784, 38208, 36992, 37184, 46624, 0]],
        '福' : [12, 13, 0, -2, 12, [0, 20448, 16384, 63424, 5184, 10176, 16384, 53216, 26912, 20448, 18720, 20448, 0]],
        '腹' : [12, 13, 0, -2, 12, [512, 57856, 45024, 47168, 61376, 43072, 44992, 58368, 45024, 46144, 41856, 48224, 0]],
        '複' : [12, 13, 0, -2, 12, [1024, 17408, 20448, 63552, 12224, 10304, 20416, 50176, 28640, 21568, 17280, 23648, 0]],
        '覆' : [12, 13, 0, -2, 12, [0, 65504, 2560, 32704, 19008, 19008, 65504, 10304, 20448, 54336, 17280, 23648, 0]],
        '淵' : [12, 13, 0, -2, 12, [0, 2720, 35488, 19104, 3808, 34848, 20448, 2080, 20192, 19104, 37536, 37536, 0]],
        '弗' : [12, 13, 0, -2, 12, [4608, 4608, 65472, 4672, 32704, 20992, 20992, 65504, 37408, 4640, 8736, 49856, 0]],
        '払' : [12, 13, 0, -2, 12, [0, 16896, 16896, 61952, 16896, 16896, 17408, 25664, 50240, 18496, 18592, 53024, 0]],
        '沸' : [12, 13, 0, -2, 12, [1280, 1280, 40896, 17728, 8128, 38144, 21760, 8160, 17696, 17696, 35104, 45504, 0]],
        '仏' : [12, 13, 0, -2, 12, [0, 8704, 8704, 16896, 16896, 49664, 17408, 17472, 17472, 18496, 18592, 20256, 0]],
        '物' : [12, 13, 0, -2, 12, [2048, 43008, 45024, 60064, 45728, 42144, 9504, 14624, 59936, 8736, 8256, 8576, 0]],
        '鮒' : [12, 13, 0, -2, 12, [0, 28960, 37152, 9184, 64032, 44576, 64160, 43616, 64032, 544, 43552, 43616, 0]],
        '分' : [12, 13, 0, -2, 12, [0, 10112, 8320, 8256, 16448, 16416, 49056, 34944, 2176, 4224, 4224, 25344, 0]],
        '吻' : [12, 13, 0, -2, 12, [2048, 2048, 61408, 43680, 45728, 42144, 42272, 43296, 43552, 41504, 57408, 384, 0]],
        '噴' : [12, 13, 0, -2, 12, [256, 256, 61408, 41216, 42304, 49120, 42048, 45024, 43040, 45024, 58432, 6176, 0]],
        '墳' : [12, 13, 0, -2, 12, [256, 16640, 20448, 16640, 58688, 24544, 17472, 20448, 18464, 20448, 58432, 6176, 0]],
        '憤' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8448, 42304, 49120, 42048, 45024, 43040, 12256, 9280, 14368, 0]],
        '扮' : [12, 13, 0, -2, 12, [0, 18368, 16448, 59456, 18464, 20512, 24544, 17472, 58432, 17472, 18496, 51584, 0]],
        '焚' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8576, 29376, 42144, 0, 9280, 17536, 1024, 6912, 57568, 0]],
        '奮' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 8320, 21056, 49056, 25088, 49088, 8704, 32704, 16960, 32704, 0]],
        '粉' : [12, 13, 0, -2, 12, [0, 9152, 45120, 42048, 9248, 63520, 12256, 25152, 25152, 46144, 42048, 8576, 0]],
        '糞' : [12, 13, 0, -2, 12, [1024, 9344, 5376, 65504, 9344, 32704, 42144, 16256, 4352, 65504, 4352, 24768, 0]],
        '紛' : [12, 13, 0, -2, 12, [0, 9152, 16448, 33856, 58400, 10272, 20448, 62016, 8768, 46144, 42048, 8576, 0]],
        '雰' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 3840, 8320, 32704, 34976, 4224, 25344, 0]],
        '文' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 128, 4224, 4352, 2304, 2560, 1024, 6912, 57568, 0]],
        '聞' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 37152, 40736, 37152, 40736, 45344, 33120, 0]],
        '丙' : [12, 13, 0, -2, 12, [0, 65504, 1024, 1024, 65504, 33824, 33824, 35360, 37152, 41120, 32800, 32864, 0]],
        '併' : [12, 13, 0, -2, 12, [2112, 9280, 8320, 24544, 17472, 50240, 17472, 24544, 17472, 17472, 18496, 18496, 0]],
        '兵' : [12, 13, 0, -2, 12, [0, 384, 15872, 8192, 16320, 8448, 8448, 8448, 65504, 0, 12672, 49248, 0]],
        '塀' : [12, 13, 0, -2, 12, [0, 24544, 20512, 24544, 62016, 21120, 24544, 21056, 21056, 24544, 58432, 1088, 0]],
        '幣' : [12, 13, 0, -2, 12, [8704, 43520, 9184, 64576, 42304, 62848, 44640, 1024, 32704, 17472, 17472, 17600, 0]],
        '平' : [12, 13, 0, -2, 12, [0, 65504, 1024, 17472, 9344, 1024, 1024, 65504, 1024, 1024, 1024, 1024, 0]],
        '弊' : [12, 13, 0, -2, 12, [8704, 43520, 9184, 64576, 43328, 63872, 44640, 4352, 4352, 65504, 4352, 24832, 0]],
        '柄' : [12, 13, 0, -2, 12, [0, 12256, 8448, 61696, 12256, 10528, 26912, 27296, 44128, 43040, 10272, 10336, 0]],
        '並' : [12, 13, 0, -2, 12, [4224, 2176, 256, 32736, 2304, 2304, 18720, 10560, 2304, 2304, 2304, 32736, 0]],
        '蔽' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 43520, 44000, 8768, 64576, 43328, 63808, 43392, 44640, 0]],
        '閉' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 33312, 49056, 34336, 35360, 45600, 34400, 0]],
        '陛' : [12, 13, 0, -2, 12, [128, 59520, 44768, 43136, 51360, 44768, 41216, 41216, 45024, 49408, 33024, 36832, 0]],
        '米' : [12, 13, 0, -2, 12, [1024, 1024, 17472, 9344, 1024, 65504, 1024, 3328, 5248, 9280, 50208, 1024, 0]],
        '頁' : [12, 13, 0, -2, 12, [0, 65504, 1024, 16256, 8320, 16256, 8320, 16256, 8320, 16256, 8320, 49248, 0]],
        '僻' : [12, 13, 0, -2, 12, [128, 7296, 14304, 21824, 54592, 23872, 21472, 23680, 29824, 22496, 21632, 23680, 0]],
        '壁' : [12, 13, 0, -2, 12, [128, 63616, 36832, 64064, 34784, 63616, 36832, 63616, 33792, 16320, 1024, 65504, 0]],
        '癖' : [12, 13, 0, -2, 12, [512, 512, 32736, 21632, 55264, 23872, 20800, 57312, 29824, 22496, 38016, 40064, 0]],
        '碧' : [12, 13, 0, -2, 12, [128, 61696, 10208, 62496, 10208, 9248, 63456, 0, 65504, 4160, 12352, 57280, 0]],
        '別' : [12, 13, 0, -2, 12, [0, 63520, 35104, 35104, 63776, 16672, 31008, 18720, 35104, 34848, 2080, 28768, 0]],
        '瞥' : [12, 13, 0, -2, 12, [8704, 43520, 64480, 44608, 63872, 44896, 16448, 32704, 16448, 32704, 16448, 32704, 0]],
        '蔑' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 19008, 32704, 1152, 32736, 16960, 41344, 38496, 0]],
        '箆' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 32704, 17472, 19008, 20800, 16992, 31616, 16928, 64480, 0]],
        '偏' : [12, 13, 0, -2, 12, [0, 12256, 8192, 20448, 18464, 53216, 18432, 24544, 27296, 20448, 19104, 19104, 0]],
        '変' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2304, 18752, 35104, 4096, 8064, 24704, 2304, 3584, 61920, 0]],
        '片' : [12, 13, 0, -2, 12, [512, 8704, 8704, 8704, 16352, 8192, 8192, 16256, 8320, 8320, 16512, 16512, 0]],
        '篇' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 65504, 16448, 32704, 16384, 32736, 42272, 49120, 9504, 0]],
        '編' : [12, 13, 0, -2, 12, [0, 12256, 16384, 36832, 59424, 12256, 18432, 65504, 10912, 45024, 43680, 10912, 0]],
        '辺' : [12, 13, 0, -2, 12, [0, 4064, 16672, 8480, 288, 544, 57888, 9248, 9408, 8192, 20480, 36832, 0]],
        '返' : [12, 13, 0, -2, 12, [0, 2016, 17408, 10176, 1088, 1344, 59712, 10368, 8512, 9760, 20480, 36832, 0]],
        '遍' : [12, 13, 0, -2, 12, [0, 8160, 18464, 12256, 2048, 8160, 60064, 12256, 10912, 10912, 20480, 36832, 0]],
        '便' : [12, 13, 0, -2, 12, [0, 12256, 8448, 20448, 18720, 53216, 18720, 20448, 17664, 17152, 17536, 22624, 0]],
        '勉' : [12, 13, 0, -2, 12, [256, 28928, 37120, 10208, 63776, 43296, 43296, 64032, 21024, 20672, 36864, 36832, 0]],
        '娩' : [12, 13, 0, -2, 12, [0, 18304, 18560, 61696, 24544, 20768, 20768, 40928, 57984, 12928, 17568, 22752, 0]],
        '弁' : [12, 13, 0, -2, 12, [2048, 2048, 4224, 8256, 65504, 4352, 4352, 65504, 4352, 4352, 8448, 49408, 0]],
        '鞭' : [12, 13, 0, -2, 12, [0, 22496, 64640, 22496, 31392, 11232, 64160, 44000, 64128, 10624, 63808, 11808, 0]],
        '保' : [12, 13, 0, -2, 12, [0, 12256, 10272, 18464, 20448, 49408, 20448, 16640, 17280, 17728, 22816, 16640, 0]],
        '舗' : [12, 13, 0, -2, 12, [256, 28960, 36832, 8448, 63456, 9504, 63456, 1312, 63456, 38176, 38176, 62816, 0]],
        '鋪' : [12, 13, 0, -2, 12, [256, 12576, 20448, 33024, 63456, 9504, 63456, 9504, 47072, 42272, 9504, 62816, 0]],
        '圃' : [12, 13, 0, -2, 12, [0, 65504, 33952, 49056, 33824, 49056, 42144, 49056, 42144, 49056, 42144, 65504, 0]],
        '捕' : [12, 13, 0, -2, 12, [256, 16672, 20448, 57600, 20448, 18720, 20448, 26912, 53216, 18720, 18720, 51552, 0]],
        '歩' : [12, 13, 0, -2, 12, [1024, 1024, 10176, 9216, 9216, 65504, 9344, 17472, 3072, 192, 1792, 30720, 0]],
        '甫' : [12, 13, 0, -2, 12, [1024, 1056, 65504, 1024, 32704, 17472, 32704, 17472, 32704, 17472, 17472, 17600, 0]],
        '補' : [12, 13, 0, -2, 12, [256, 16672, 20448, 61696, 8160, 10528, 20448, 51488, 28640, 18720, 18720, 18784, 0]],
        '輔' : [12, 13, 0, -2, 12, [256, 8480, 65504, 8448, 65504, 43296, 65504, 43296, 65504, 10528, 63776, 10592, 0]],
        '穂' : [12, 13, 0, -2, 12, [256, 6400, 61408, 8448, 12256, 63776, 12256, 26912, 28640, 41216, 46144, 10144, 0]],
        '募' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 8320, 65504, 8320, 32704, 34976, 4224, 25344, 0]],
        '墓' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 8320, 65504, 4352, 9344, 57184, 1024, 32704, 0]],
        '慕' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 8320, 65504, 4224, 9280, 50208, 9536, 19776, 0]],
        '戊' : [12, 13, 0, -2, 12, [256, 288, 32736, 16640, 16640, 16640, 16672, 16704, 16512, 16768, 34368, 38944, 0]],
        '暮' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 8320, 65504, 4352, 16256, 53600, 4352, 7936, 0]],
        '母' : [12, 13, 0, -2, 12, [0, 16256, 8320, 10368, 9344, 65504, 16512, 18560, 33920, 32896, 65504, 256, 0]],
        '簿' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 8160, 21056, 16320, 37440, 24544, 64, 18496, 33984, 0]],
        '菩' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 32704, 4352, 4352, 65504, 0, 16256, 8320, 16256, 0]],
        '倣' : [12, 13, 0, -2, 12, [2304, 10496, 10720, 24352, 18720, 51232, 20128, 19104, 21056, 21056, 17056, 19744, 0]],
        '俸' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16896, 20448, 50304, 24544, 18752, 22496, 16640, 20448, 16640, 0]],
        '包' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16416, 24352, 33056, 33056, 16160, 8384, 8192, 8224, 16352, 0]],
        '呆' : [12, 13, 0, -2, 12, [0, 16256, 8320, 8320, 16256, 1024, 65504, 1024, 3072, 13696, 50272, 1024, 0]],
        '報' : [12, 13, 0, -2, 12, [8192, 9184, 64032, 8704, 65504, 18976, 19104, 65184, 8768, 63552, 8608, 9760, 0]],
        '奉' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2048, 32704, 4352, 65504, 9344, 24384, 33824, 32704, 1024, 0]],
        '宝' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 32800, 16256, 1024, 1024, 16192, 1056, 1024, 65504, 0]],
        '峰' : [12, 13, 0, -2, 12, [0, 9184, 11296, 8768, 41920, 44192, 41952, 43136, 44000, 43136, 65504, 128, 0]],
        '峯' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 4352, 26880, 3584, 12672, 50272, 16256, 1024, 65504, 1024, 0]],
        '崩' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 0, 31712, 18976, 31712, 18976, 31712, 18976, 35360, 39520, 0]],
        '庖' : [12, 13, 0, -2, 12, [512, 512, 32736, 18432, 20448, 20512, 28576, 16544, 24480, 36928, 36896, 8160, 0]],
        '抱' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 63520, 12192, 10400, 8352, 14240, 58432, 9216, 9248, 26592, 0]],
        '捧' : [12, 13, 0, -2, 12, [256, 16640, 20448, 57856, 20448, 17536, 24544, 18752, 55264, 16640, 20448, 49408, 0]],
        '放' : [12, 13, 0, -2, 12, [8704, 8704, 9184, 64064, 9280, 9280, 14656, 18752, 18560, 34944, 35136, 13856, 0]],
        '方' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2048, 2048, 4032, 2112, 4160, 4160, 8256, 8320, 1792, 0]],
        '朋' : [12, 13, 0, -2, 12, [0, 31712, 18976, 18976, 31712, 18976, 18976, 31712, 18976, 18976, 35360, 39520, 0]],
        '法' : [12, 13, 0, -2, 12, [256, 256, 36832, 16640, 256, 33024, 20448, 512, 1088, 9280, 18848, 36384, 0]],
        '泡' : [12, 13, 0, -2, 12, [1024, 1024, 34784, 18464, 4000, 34976, 16544, 1952, 1088, 9216, 17440, 34784, 0]],
        '烹' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 8320, 16256, 0, 32704, 512, 1024, 1024, 21056, 35104, 0]],
        '砲' : [12, 13, 0, -2, 12, [1024, 1024, 63456, 10272, 12192, 10400, 28832, 55200, 21568, 21504, 21536, 30688, 0]],
        '縫' : [12, 13, 0, -2, 12, [0, 9184, 19488, 37440, 62400, 9376, 23520, 59520, 11232, 43136, 44032, 13280, 0]],
        '胞' : [12, 13, 0, -2, 12, [1024, 62464, 38880, 38944, 63392, 37024, 37024, 63392, 37952, 37888, 37920, 47072, 0]],
        '芳' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 5376, 1024, 65504, 4096, 8128, 4160, 8256, 8320, 768, 0]],
        '萌' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 64480, 35360, 64480, 35360, 35808, 64032, 1056, 6368, 0]],
        '蓬' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 2112, 38016, 18304, 6496, 59328, 8448, 12256, 20736, 36832, 0]],
        '蜂' : [12, 13, 0, -2, 12, [0, 9184, 11296, 64064, 43968, 44192, 44000, 63616, 9184, 10368, 12256, 63616, 0]],
        '褒' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 10304, 20416, 49152, 24544, 18752, 21792, 8768, 57728, 15456, 0]],
        '訪' : [12, 13, 0, -2, 12, [256, 61696, 4064, 61696, 256, 61920, 544, 544, 62496, 37920, 36928, 61824, 0]],
        '豊' : [12, 13, 0, -2, 12, [2560, 2560, 32704, 19008, 32704, 19008, 65504, 8320, 16256, 4352, 4352, 65504, 0]],
        '邦' : [12, 13, 0, -2, 12, [4096, 4576, 65312, 4384, 64832, 4480, 4416, 65312, 4384, 4384, 8640, 49408, 0]],
        '鋒' : [12, 13, 0, -2, 12, [0, 13280, 19488, 33344, 62400, 11424, 63456, 8320, 44000, 43136, 10208, 63616, 0]],
        '飽' : [12, 13, 0, -2, 12, [512, 29184, 35808, 9248, 64416, 34976, 63648, 35744, 64064, 41472, 37408, 60384, 0]],
        '鳳' : [12, 13, 0, -2, 12, [0, 32704, 17472, 24384, 20800, 24384, 20800, 24384, 20544, 24480, 32928, 38304, 0]],
        '鵬' : [12, 13, 0, -2, 12, [128, 63744, 43968, 43584, 64448, 43584, 44000, 64000, 44000, 43040, 43680, 43744, 0]],
        '乏' : [12, 13, 0, -2, 12, [0, 960, 31744, 1024, 1024, 65472, 128, 256, 8704, 9216, 22528, 34784, 0]],
        '亡' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8160, 0]],
        '傍' : [12, 13, 0, -2, 12, [256, 8448, 16352, 17536, 17536, 57312, 21024, 20448, 17408, 18400, 18464, 18624, 0]],
        '剖' : [12, 13, 0, -2, 12, [8192, 8224, 64800, 18720, 18720, 64800, 288, 288, 64800, 33824, 33824, 64608, 0]],
        '坊' : [12, 13, 0, -2, 12, [256, 16640, 24544, 16896, 57856, 17376, 17440, 17440, 18464, 18464, 57408, 384, 0]],
        '妨' : [12, 13, 0, -2, 12, [256, 16640, 20448, 61696, 20736, 20960, 21024, 37408, 58400, 13344, 16448, 16768, 0]],
        '帽' : [12, 13, 0, -2, 12, [0, 10176, 9280, 10176, 64576, 42944, 40960, 45024, 43040, 45024, 10272, 12256, 0]],
        '忘' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 8192, 8192, 8192, 8128, 1024, 4608, 20544, 36896, 3968, 0]],
        '忙' : [12, 13, 0, -2, 12, [256, 8448, 8448, 12256, 41984, 41984, 41984, 41984, 41984, 9216, 9216, 9184, 0]],
        '房' : [12, 13, 0, -2, 12, [0, 65504, 0, 32704, 16448, 32704, 18432, 32736, 20480, 40896, 41024, 8576, 0]],
        '暴' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 65504, 4352, 65504, 9344, 54624, 9344, 19520, 0]],
        '望' : [12, 13, 0, -2, 12, [8192, 9184, 65056, 17376, 16928, 15328, 1056, 32704, 1024, 16256, 1024, 65504, 0]],
        '某' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 7936, 4352, 7936, 1024, 65504, 3328, 13440, 50272, 0]],
        '棒' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61952, 12256, 9344, 32736, 26944, 47008, 41216, 12256, 8448, 0]],
        '冒' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 16416, 32736, 0, 16320, 8256, 16320, 8256, 16320, 0]],
        '紡' : [12, 13, 0, -2, 12, [256, 8448, 20448, 37120, 61696, 8672, 16928, 61984, 9248, 46112, 41024, 8576, 0]],
        '肪' : [12, 13, 0, -2, 12, [256, 61696, 40928, 37120, 61696, 37344, 37408, 61984, 37920, 37920, 36928, 45440, 0]],
        '膨' : [12, 13, 0, -2, 12, [1024, 58400, 48960, 42112, 65280, 40992, 44864, 59776, 44800, 43552, 43584, 49024, 0]],
        '謀' : [12, 13, 0, -2, 12, [1088, 58432, 4064, 58432, 1984, 58432, 1984, 256, 61408, 41728, 42304, 63776, 0]],
        '貌' : [12, 13, 0, -2, 12, [128, 14592, 55264, 23584, 14304, 54304, 5152, 14304, 53888, 4736, 5280, 25824, 0]],
        '貿' : [12, 13, 0, -2, 12, [0, 63456, 33056, 35104, 64032, 9408, 16256, 8320, 16256, 8320, 16256, 49248, 0]],
        '鉾' : [12, 13, 0, -2, 12, [256, 12800, 17440, 34784, 61600, 9344, 63456, 9344, 43136, 45024, 8320, 63616, 0]],
        '防' : [12, 13, 0, -2, 12, [256, 57600, 49120, 41472, 49664, 41952, 42016, 42016, 43040, 51232, 32832, 33152, 0]],
        '吠' : [12, 13, 0, -2, 12, [256, 320, 57632, 41216, 49120, 41216, 41216, 41216, 41600, 41600, 58432, 6176, 0]],
        '頬' : [12, 13, 0, -2, 12, [8192, 10208, 63616, 10208, 44064, 47072, 9248, 65504, 9248, 10208, 21056, 35872, 0]],
        '北' : [12, 13, 0, -2, 12, [512, 4608, 4608, 4704, 62336, 4608, 4608, 4608, 12800, 53792, 4640, 5088, 0]],
        '僕' : [12, 13, 0, -2, 12, [1280, 9536, 16352, 17536, 17536, 57312, 16640, 20448, 16640, 24544, 17536, 22624, 0]],
        '卜' : [12, 13, 0, -2, 12, [0, 4096, 4096, 4096, 4096, 6144, 5632, 4480, 4096, 4096, 4096, 4096, 0]],
        '墨' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 32704, 1024, 65504, 18752, 49056, 1024, 65504, 0]],
        '撲' : [12, 13, 0, -2, 12, [1280, 21824, 24544, 58496, 17536, 24544, 16640, 28640, 49408, 24544, 17536, 55392, 0]],
        '朴' : [12, 13, 0, -2, 12, [8448, 8448, 8448, 63744, 8448, 8576, 24896, 28960, 43280, 41216, 8448, 8448, 0]],
        '牧' : [12, 13, 0, -2, 12, [512, 41472, 41952, 64064, 42048, 42048, 8512, 14656, 57472, 8320, 9024, 11296, 0]],
        '睦' : [12, 13, 0, -2, 12, [256, 57600, 45024, 41216, 65504, 41600, 41632, 60640, 41216, 45024, 57600, 8160, 0]],
        '穆' : [12, 13, 0, -2, 12, [256, 6656, 61408, 10272, 12256, 61696, 9536, 27424, 28864, 42784, 41152, 12032, 0]],
        '釦' : [12, 13, 0, -2, 12, [0, 13280, 18976, 33312, 64032, 8736, 64032, 8736, 43552, 43552, 8736, 64480, 0]],
        '勃' : [12, 13, 0, -2, 12, [8448, 8448, 63744, 10208, 63776, 35104, 28960, 4640, 8736, 64544, 9280, 24960, 0]],
        '没' : [12, 13, 0, -2, 12, [0, 1920, 33920, 17536, 1184, 35040, 16384, 4032, 17472, 17536, 33664, 40032, 0]],
        '殆' : [12, 13, 0, -2, 12, [512, 61952, 17440, 17440, 30688, 36864, 36864, 22496, 9248, 9248, 17440, 34784, 0]],
        '堀' : [12, 13, 0, -2, 12, [0, 20448, 18464, 20448, 59520, 19104, 19104, 19424, 18560, 21664, 62624, 2016, 0]],
        '幌' : [12, 13, 0, -2, 12, [0, 10208, 9248, 10208, 64544, 42976, 43296, 42304, 45024, 8832, 9376, 14560, 0]],
        '奔' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2304, 13440, 50272, 16256, 5376, 4352, 65504, 8448, 8448, 0]],
        '本' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 1024, 3584, 5376, 9344, 17472, 40736, 1024, 1024, 0]],
        '翻' : [12, 13, 0, -2, 12, [0, 65504, 43296, 8480, 64928, 25440, 43296, 63776, 43872, 64928, 43296, 64352, 0]],
        '凡' : [12, 13, 0, -2, 12, [0, 16128, 8448, 8448, 8448, 10496, 9472, 8448, 16640, 16672, 33056, 33248, 0]],
        '盆' : [12, 13, 0, -2, 12, [0, 10112, 8320, 16448, 49056, 2176, 4224, 8448, 32704, 19008, 19008, 65504, 0]],
        '摩' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 32224, 19072, 22720, 28576, 16896, 49120, 33280, 3584, 0]],
        '磨' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 32224, 23232, 26784, 24544, 34816, 40928, 12320, 24544, 0]],
        '魔' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 32224, 23232, 27296, 24512, 21056, 40896, 34080, 14816, 0]],
        '麻' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 18560, 32224, 18560, 18816, 23232, 44192, 34944, 2176, 0]],
        '埋' : [12, 13, 0, -2, 12, [0, 20448, 18720, 20448, 59680, 18720, 20448, 16640, 20448, 16640, 57600, 4064, 0]],
        '妹' : [12, 13, 0, -2, 12, [256, 16640, 20448, 61696, 20736, 24544, 20736, 37120, 58240, 9536, 22816, 16640, 0]],
        '昧' : [12, 13, 0, -2, 12, [256, 256, 65504, 37120, 37120, 40928, 61696, 37120, 37760, 40256, 61728, 256, 0]],
        '枚' : [12, 13, 0, -2, 12, [8704, 8704, 9184, 64064, 9280, 9280, 24896, 28992, 43136, 41088, 9024, 11296, 0]],
        '毎' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16384, 24512, 4672, 4672, 65504, 9280, 9280, 32736, 128, 0]],
        '哩' : [12, 13, 0, -2, 12, [0, 4064, 59680, 45024, 43296, 43296, 45024, 41216, 45024, 41216, 57600, 4064, 0]],
        '槙' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61696, 10208, 9248, 26592, 25632, 45024, 40960, 9280, 10272, 0]],
        '幕' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 8320, 65504, 9344, 32704, 42144, 9344, 9600, 0]],
        '膜' : [12, 13, 0, -2, 12, [576, 62016, 40928, 37440, 63456, 37920, 38880, 62496, 40928, 37120, 37440, 46112, 0]],
        '枕' : [12, 13, 0, -2, 12, [256, 8448, 12256, 63776, 10528, 8480, 24960, 25216, 45696, 41632, 9376, 14560, 0]],
        '鮪' : [12, 13, 0, -2, 12, [256, 28928, 38880, 8448, 64480, 44576, 64480, 43552, 64480, 544, 43552, 43616, 0]],
        '柾' : [12, 13, 0, -2, 12, [0, 12256, 8448, 61696, 8448, 9472, 26080, 25856, 46336, 42240, 9472, 12256, 0]],
        '鱒' : [12, 13, 0, -2, 12, [576, 29312, 40928, 8832, 65504, 43680, 64736, 43040, 65504, 32, 43552, 43360, 0]],
        '桝' : [12, 13, 0, -2, 12, [64, 9280, 10048, 62944, 9536, 10560, 26944, 25920, 46048, 41536, 9280, 14400, 0]],
        '亦' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 2304, 2304, 18752, 18752, 35104, 37152, 4352, 25344, 0]],
        '俣' : [12, 13, 0, -2, 12, [0, 12256, 10272, 18464, 20448, 49152, 20448, 16640, 20448, 16640, 18048, 22624, 0]],
        '又' : [12, 13, 0, -2, 12, [0, 32704, 64, 64, 4160, 4224, 4224, 2304, 2560, 1536, 6528, 57440, 0]],
        '抹' : [12, 13, 0, -2, 12, [256, 16640, 24544, 57600, 16640, 20448, 16640, 24832, 50048, 17728, 22816, 49408, 0]],
        '末' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 1024, 32704, 1024, 3328, 5248, 9280, 50208, 1024, 0]],
        '沫' : [12, 13, 0, -2, 12, [256, 256, 40928, 16640, 256, 36832, 16640, 256, 17280, 17728, 39200, 33024, 0]],
        '迄' : [12, 13, 0, -2, 12, [1024, 1024, 18400, 9216, 3008, 2176, 57600, 8736, 9248, 10176, 20480, 36832, 0]],
        '侭' : [12, 13, 0, -2, 12, [0, 12256, 10272, 18464, 20448, 51264, 18496, 21056, 20768, 20512, 17920, 16832, 0]],
        '繭' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 42272, 55264, 44320, 63456, 42272, 64864, 42912, 0]],
        '麿' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 32224, 23232, 26784, 24512, 20544, 49120, 40992, 16352, 0]],
        '万' : [12, 13, 0, -2, 12, [0, 65504, 2048, 2048, 2048, 4032, 2112, 4160, 4160, 8256, 8320, 1792, 0]],
        '慢' : [12, 13, 0, -2, 12, [0, 12224, 10304, 12224, 43072, 49120, 42144, 49120, 43072, 9280, 9088, 15456, 0]],
        '満' : [12, 13, 0, -2, 12, [1152, 1152, 40928, 17536, 8160, 33024, 24544, 4384, 21792, 22496, 36896, 36960, 0]],
        '漫' : [12, 13, 0, -2, 12, [0, 4032, 34880, 20416, 2112, 40928, 21152, 8160, 18496, 17472, 33664, 40032, 0]],
        '蔓' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8320, 16256, 8320, 65504, 37152, 65504, 8320, 7936, 57568, 0]],
        '味' : [12, 13, 0, -2, 12, [256, 256, 61408, 41216, 41216, 49120, 41216, 41216, 41856, 58688, 6432, 256, 0]],
        '未' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 1024, 65504, 1024, 3328, 5248, 9280, 50208, 1024, 0]],
        '魅' : [12, 13, 0, -2, 12, [8320, 16512, 64480, 43136, 65504, 43136, 43456, 64160, 21632, 22144, 36896, 40928, 0]],
        '巳' : [12, 13, 0, -2, 12, [0, 32704, 16448, 16448, 16448, 32704, 16384, 16384, 16384, 16416, 16416, 16352, 0]],
        '箕' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42048, 4224, 32736, 4224, 8064, 4224, 65520, 4224, 24672, 0]],
        '岬' : [12, 13, 0, -2, 12, [0, 12256, 10528, 10528, 45024, 43296, 43296, 45024, 41216, 43264, 63744, 256, 0]],
        '密' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33824, 20800, 37408, 3136, 29632, 1024, 17472, 17472, 32704, 0]],
        '蜜' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33824, 20800, 37408, 16320, 25728, 9344, 16288, 1056, 65504, 0]],
        '湊' : [12, 13, 0, -2, 12, [256, 256, 40928, 16896, 4064, 33920, 24544, 2368, 22496, 16640, 34432, 39008, 0]],
        '蓑' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 8320, 65504, 8320, 16288, 4672, 57728, 15456, 0]],
        '稔' : [12, 13, 0, -2, 12, [0, 7040, 58432, 10272, 10176, 61440, 10176, 24640, 25216, 43264, 43072, 10016, 0]],
        '脈' : [12, 13, 0, -2, 12, [0, 57568, 44800, 43104, 60288, 43648, 43680, 60064, 43584, 45632, 41504, 41504, 0]],
        '妙' : [12, 13, 0, -2, 12, [256, 16640, 16640, 62784, 22816, 20736, 20736, 37632, 57376, 12352, 16768, 19968, 0]],
        '粍' : [12, 13, 0, -2, 12, [0, 8640, 44544, 41568, 9088, 60928, 8800, 25472, 32256, 41472, 41504, 8672, 0]],
        '民' : [12, 13, 0, -2, 12, [0, 32704, 16448, 16448, 32704, 16896, 16896, 32736, 16640, 16640, 19616, 61536, 0]],
        '眠' : [12, 13, 0, -2, 12, [0, 2016, 62496, 37920, 63456, 38016, 38016, 63456, 37952, 37952, 62496, 1824, 0]],
        '務' : [12, 13, 0, -2, 12, [256, 63744, 2528, 22048, 8512, 63872, 11872, 10496, 30688, 41248, 8736, 25792, 0]],
        '夢' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 19008, 65504, 34848, 49056, 2176, 1792, 30720, 0]],
        '無' : [12, 13, 0, -2, 12, [8192, 8192, 32736, 43648, 10880, 65504, 10880, 10880, 65504, 0, 21056, 35104, 0]],
        '牟' : [12, 13, 0, -2, 12, [2048, 2304, 4224, 65472, 1056, 9216, 16320, 17408, 33792, 65504, 1024, 1024, 0]],
        '矛' : [12, 13, 0, -2, 12, [0, 16256, 128, 4352, 2560, 65504, 544, 1600, 6784, 57856, 512, 3584, 0]],
        '霧' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 31712, 5696, 63872, 14176, 53824, 13504, 0]],
        '鵡' : [12, 13, 0, -2, 12, [4224, 54528, 6080, 4672, 65472, 10816, 11232, 14848, 44000, 43040, 48416, 50528, 0]],
        '椋' : [12, 13, 0, -2, 12, [128, 8320, 12256, 61440, 10208, 9248, 25632, 26592, 45184, 42144, 10400, 8576, 0]],
        '婿' : [12, 13, 0, -2, 12, [0, 20448, 16672, 62912, 21760, 22496, 23584, 38880, 58400, 14304, 17440, 17504, 0]],
        '娘' : [12, 13, 0, -2, 12, [256, 16640, 18400, 62496, 22496, 21536, 22496, 38176, 58688, 13440, 17472, 18208, 0]],
        '冥' : [12, 13, 0, -2, 12, [0, 65504, 32800, 49056, 8320, 16256, 8320, 16256, 1024, 65504, 8320, 49248, 0]],
        '名' : [12, 13, 0, -2, 12, [1024, 1024, 4032, 4160, 26752, 1280, 512, 4064, 14368, 51232, 2080, 4064, 0]],
        '命' : [12, 13, 0, -2, 12, [0, 3840, 4224, 8256, 32672, 32768, 31712, 18976, 18976, 18976, 31456, 512, 0]],
        '明' : [12, 13, 0, -2, 12, [0, 2016, 62496, 37920, 38880, 62496, 37920, 38880, 62496, 1056, 2080, 12512, 0]],
        '盟' : [12, 13, 0, -2, 12, [0, 63456, 37920, 63456, 37920, 38880, 62496, 2144, 32704, 19008, 19008, 65504, 0]],
        '迷' : [12, 13, 0, -2, 12, [256, 256, 18720, 9536, 256, 4064, 58112, 9536, 10528, 8448, 20480, 36832, 0]],
        '銘' : [12, 13, 0, -2, 12, [256, 12544, 18912, 33312, 64032, 9536, 63616, 9184, 44576, 41504, 8736, 64480, 0]],
        '鳴' : [12, 13, 0, -2, 12, [256, 512, 61376, 43072, 44992, 43072, 45024, 43008, 45024, 57376, 5408, 5472, 0]],
        '姪' : [12, 13, 0, -2, 12, [0, 20448, 16640, 61952, 21536, 22496, 20608, 36992, 59360, 12416, 16512, 20448, 0]],
        '牝' : [12, 13, 0, -2, 12, [512, 41472, 41472, 64032, 41536, 41600, 8960, 14848, 57856, 8736, 8736, 9184, 0]],
        '滅' : [12, 13, 0, -2, 12, [128, 160, 40928, 20608, 8064, 38048, 23968, 5280, 23104, 21184, 41248, 42528, 0]],
        '免' : [12, 13, 0, -2, 12, [0, 7936, 8448, 16896, 65472, 17472, 17472, 32704, 4608, 4640, 8736, 50144, 0]],
        '棉' : [12, 13, 0, -2, 12, [128, 8448, 10208, 62496, 10208, 9248, 26592, 28928, 45024, 43296, 10592, 8448, 0]],
        '綿' : [12, 13, 0, -2, 12, [128, 8448, 18400, 37920, 63456, 9248, 18400, 61696, 12256, 43296, 43360, 8448, 0]],
        '緬' : [12, 13, 0, -2, 12, [0, 12256, 16640, 33280, 61408, 10912, 19360, 64160, 11168, 43680, 43680, 12256, 0]],
        '面' : [12, 13, 0, -2, 12, [0, 65504, 1024, 2048, 65504, 37152, 40736, 37152, 40736, 37152, 37152, 65504, 0]],
        '麺' : [12, 13, 0, -2, 12, [8192, 12256, 61696, 8704, 65504, 9376, 63392, 17568, 29856, 57312, 12288, 53216, 0]],
        '摸' : [12, 13, 0, -2, 12, [576, 8768, 12256, 62016, 10208, 9248, 10208, 13344, 61408, 8448, 8768, 27680, 0]],
        '模' : [12, 13, 0, -2, 12, [576, 8768, 12256, 62016, 10208, 9248, 26592, 29728, 45024, 41216, 8768, 11296, 0]],
        '茂' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 5376, 1088, 32736, 16896, 16960, 16768, 16640, 34432, 39008, 0]],
        '妄' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 8192, 8192, 8128, 1024, 65504, 4224, 12544, 3840, 28864, 0]],
        '孟' : [12, 13, 0, -2, 12, [0, 16256, 256, 512, 1024, 65504, 1024, 1024, 32704, 19008, 19008, 65504, 0]],
        '毛' : [12, 13, 0, -2, 12, [0, 896, 31744, 2240, 3840, 30720, 2144, 3968, 63488, 2080, 2080, 2016, 0]],
        '猛' : [12, 13, 0, -2, 12, [0, 10176, 41024, 16512, 16640, 45024, 8448, 24832, 45024, 10912, 10912, 57312, 0]],
        '盲' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 16384, 16384, 16320, 0, 32704, 16448, 32704, 16448, 32704, 0]],
        '網' : [12, 13, 0, -2, 12, [0, 12256, 18464, 35488, 60064, 12256, 18720, 65504, 10784, 43552, 43936, 10336, 0]],
        '耗' : [12, 13, 0, -2, 12, [0, 8640, 11776, 62048, 9088, 65024, 8800, 62336, 11776, 29184, 41504, 9184, 0]],
        '蒙' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 32800, 32704, 4640, 26176, 7040, 57952, 3072, 0]],
        '儲' : [12, 13, 0, -2, 12, [128, 15488, 8672, 23680, 16544, 56800, 16512, 16864, 24352, 21984, 21792, 24032, 0]],
        '木' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 1024, 3584, 5376, 9344, 17472, 33824, 1024, 1024, 0]],
        '黙' : [12, 13, 0, -2, 12, [128, 64672, 42144, 65504, 42112, 64640, 8512, 64032, 1056, 0, 21056, 35104, 0]],
        '目' : [12, 13, 0, -2, 12, [0, 16320, 8256, 8256, 16320, 8256, 8256, 16320, 8256, 8256, 8256, 16320, 0]],
        '杢' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 3328, 13440, 50272, 0, 32704, 1024, 1024, 65504, 0]],
        '勿' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 18720, 18720, 35104, 37152, 4640, 8736, 17440, 2112, 384, 0]],
        '餅' : [12, 13, 0, -2, 12, [1088, 29248, 34944, 10208, 64064, 35392, 64064, 36832, 64064, 37440, 35392, 62528, 0]],
        '尤' : [12, 13, 0, -2, 12, [1024, 1088, 1056, 65472, 1024, 1024, 2560, 2560, 4608, 4640, 8736, 49632, 0]],
        '戻' : [12, 13, 0, -2, 12, [0, 65504, 0, 32704, 16448, 32704, 16896, 32736, 33280, 34048, 6272, 24672, 0]],
        '籾' : [12, 13, 0, -2, 12, [0, 12256, 41504, 41504, 10784, 63008, 8736, 25888, 29856, 43040, 43072, 8576, 0]],
        '貰' : [12, 13, 0, -2, 12, [2304, 18688, 65504, 18688, 32704, 0, 16256, 8320, 16256, 8320, 16256, 49248, 0]],
        '問' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 32800, 32800, 40736, 37152, 37152, 40800, 0]],
        '悶' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 32800, 34848, 33824, 53408, 36960, 36640, 0]],
        '紋' : [12, 13, 0, -2, 12, [256, 8448, 16640, 40928, 58432, 9280, 17472, 62080, 8832, 41216, 42624, 14432, 0]],
        '門' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 32800, 32800, 32800, 32800, 32800, 32864, 0]],
        '匁' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16928, 21024, 35360, 33824, 1056, 2592, 4384, 8256, 384, 0]],
        '也' : [12, 13, 0, -2, 12, [1024, 1024, 9408, 10048, 15424, 58432, 9280, 9408, 9216, 9248, 8224, 8160, 0]],
        '冶' : [12, 13, 0, -2, 12, [512, 512, 33824, 18464, 12256, 0, 0, 4064, 2080, 10272, 18464, 36832, 0]],
        '夜' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 9216, 10176, 19008, 18752, 54400, 21120, 16640, 18048, 22624, 0]],
        '爺' : [12, 13, 0, -2, 12, [0, 6912, 57568, 6912, 3584, 64480, 18976, 31296, 18976, 31264, 51904, 2560, 0]],
        '耶' : [12, 13, 0, -2, 12, [0, 65504, 18976, 31296, 19072, 19072, 31296, 18976, 18976, 31264, 51904, 2560, 0]],
        '野' : [12, 13, 0, -2, 12, [0, 64480, 43040, 63808, 43136, 45024, 63648, 8352, 63680, 8320, 8320, 63872, 0]],
        '弥' : [12, 13, 0, -2, 12, [512, 61952, 5088, 5248, 62592, 32896, 33472, 62624, 6304, 4224, 4224, 25472, 0]],
        '矢' : [12, 13, 0, -2, 12, [8192, 8192, 16320, 9216, 17408, 33792, 65504, 1024, 1536, 2304, 12416, 49248, 0]],
        '厄' : [12, 13, 0, -2, 12, [0, 32736, 16384, 16384, 20416, 18496, 18496, 18880, 18432, 18464, 34848, 34784, 0]],
        '役' : [12, 13, 0, -2, 12, [0, 10112, 17536, 33920, 1184, 10464, 16384, 53184, 17472, 17536, 17280, 23648, 0]],
        '約' : [12, 13, 0, -2, 12, [1024, 9216, 18400, 33824, 59424, 10272, 16928, 61728, 8224, 43040, 43072, 8576, 0]],
        '薬' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 40736, 20800, 40736, 4352, 65504, 3328, 13440, 50272, 0]],
        '訳' : [12, 13, 0, -2, 12, [0, 62432, 544, 61984, 544, 62432, 640, 640, 62016, 37440, 37920, 62496, 0]],
        '躍' : [12, 13, 0, -2, 12, [0, 63328, 37152, 38752, 61728, 10080, 8768, 47072, 44160, 42976, 42112, 63456, 0]],
        '靖' : [12, 13, 0, -2, 12, [256, 16640, 20448, 61696, 10208, 41216, 45024, 42016, 42976, 9248, 63456, 1056, 0]],
        '柳' : [12, 13, 0, -2, 12, [0, 12000, 10400, 63648, 10912, 10912, 27296, 28320, 43680, 41696, 9344, 9344, 0]],
        '薮' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 43520, 44000, 29248, 43584, 64832, 18752, 29056, 36448, 0]],
        '鑓' : [12, 13, 0, -2, 12, [128, 12416, 19424, 33440, 64160, 9184, 64832, 9696, 46368, 42464, 11264, 62432, 0]],
        '愉' : [12, 13, 0, -2, 12, [0, 10176, 10272, 14304, 40960, 44704, 43680, 44704, 43680, 11936, 10784, 10848, 0]],
        '愈' : [12, 13, 0, -2, 12, [0, 7936, 8320, 32576, 51808, 31296, 19008, 22720, 1024, 4672, 20512, 36736, 0]],
        '油' : [12, 13, 0, -2, 12, [256, 256, 33024, 20448, 2336, 35104, 18720, 4064, 2336, 10528, 18720, 36832, 0]],
        '癒' : [12, 13, 0, -2, 12, [1024, 1024, 32736, 20608, 65344, 19040, 31296, 51776, 23744, 512, 20544, 40736, 0]],
        '諭' : [12, 13, 0, -2, 12, [0, 59328, 2080, 63456, 0, 61088, 2720, 3744, 60064, 44704, 43552, 60000, 0]],
        '輸' : [12, 13, 0, -2, 12, [0, 10176, 63520, 10208, 63488, 44704, 64160, 44704, 64160, 11936, 64032, 10848, 0]],
        '唯' : [12, 13, 0, -2, 12, [512, 576, 62016, 38880, 38016, 40928, 38016, 38016, 38880, 38016, 62592, 2016, 0]],
        '佑' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16640, 16640, 49664, 18400, 19488, 29728, 17440, 17440, 18400, 0]],
        '優' : [12, 13, 0, -2, 12, [0, 16352, 10304, 12224, 18496, 32736, 51872, 21568, 18400, 23616, 17280, 23648, 0]],
        '勇' : [12, 13, 0, -2, 12, [0, 32640, 256, 32704, 17472, 32704, 17472, 17472, 65504, 2080, 12320, 49600, 0]],
        '友' : [12, 13, 0, -2, 12, [2048, 2048, 65504, 2048, 3968, 4224, 4224, 10368, 17664, 33536, 3200, 28768, 0]],
        '宥' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 34848, 32704, 4096, 16320, 24640, 49088, 8256, 16320, 8256, 0]],
        '幽' : [12, 13, 0, -2, 12, [1024, 1024, 38048, 42272, 50720, 44384, 38048, 42272, 65504, 33824, 33824, 65504, 0]],
        '悠' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 18496, 53824, 20864, 21056, 19488, 512, 4416, 20512, 36736, 0]],
        '憂' : [12, 13, 0, -2, 12, [0, 65504, 8320, 16256, 8320, 65504, 37408, 20800, 36768, 28800, 3840, 61664, 0]],
        '揖' : [12, 13, 0, -2, 12, [0, 10208, 9248, 63456, 8192, 12256, 9248, 14304, 58400, 10208, 11296, 24608, 0]],
        '有' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2048, 2048, 8128, 12352, 57280, 4160, 8128, 4160, 4288, 0]],
        '柚' : [12, 13, 0, -2, 12, [256, 8448, 8448, 65504, 10528, 10528, 26912, 28640, 43296, 43296, 10528, 12256, 0]],
        '湧' : [12, 13, 0, -2, 12, [0, 4032, 32896, 20448, 2336, 36832, 18720, 2336, 24544, 16928, 33824, 39104, 0]],
        '涌' : [12, 13, 0, -2, 12, [0, 4032, 32896, 20448, 2336, 36832, 18720, 2336, 4064, 10528, 18720, 35168, 0]],
        '猶' : [12, 13, 0, -2, 12, [1088, 9344, 45024, 17024, 20448, 43680, 10912, 27872, 43040, 12256, 10272, 53216, 0]],
        '猷' : [12, 13, 0, -2, 12, [18560, 20640, 64672, 20608, 65504, 42112, 42112, 56448, 34112, 64832, 34336, 64544, 0]],
        '由' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 32736, 17440, 17440, 17440, 32736, 17440, 17440, 17440, 32736, 0]],
        '祐' : [12, 13, 0, -2, 12, [128, 8320, 10208, 63616, 2176, 4352, 9184, 26144, 47648, 8736, 8736, 9184, 0]],
        '裕' : [12, 13, 0, -2, 12, [0, 8768, 9248, 61440, 4480, 4672, 9248, 24576, 46048, 10784, 8736, 9184, 0]],
        '誘' : [12, 13, 0, -2, 12, [0, 61408, 256, 61408, 768, 60736, 288, 4032, 57920, 41568, 42016, 63680, 0]],
        '遊' : [12, 13, 0, -2, 12, [2304, 2304, 40928, 18688, 4064, 2592, 59968, 13280, 8768, 11456, 20480, 36832, 0]],
        '邑' : [12, 13, 0, -2, 12, [0, 16320, 8256, 16320, 0, 32736, 16928, 16928, 32736, 16384, 16416, 16352, 0]],
        '郵' : [12, 13, 0, -2, 12, [0, 64992, 4384, 64832, 21888, 65344, 21792, 21792, 64800, 4544, 4352, 64768, 0]],
        '雄' : [12, 13, 0, -2, 12, [512, 8768, 8768, 65504, 9344, 12256, 17536, 21632, 22496, 38016, 42112, 16352, 0]],
        '融' : [12, 13, 0, -2, 12, [128, 64640, 18560, 31712, 672, 65184, 43680, 45024, 49792, 47776, 37536, 38880, 0]],
        '夕' : [12, 13, 0, -2, 12, [2048, 2048, 4032, 4160, 4160, 10304, 17536, 640, 256, 512, 3072, 28672, 0]],
        '予' : [12, 13, 0, -2, 12, [0, 32640, 256, 512, 1024, 65504, 1088, 1152, 1024, 1024, 1024, 7168, 0]],
        '余' : [12, 13, 0, -2, 12, [0, 3584, 4352, 8320, 24384, 33824, 1024, 32704, 1024, 17472, 33824, 7168, 0]],
        '与' : [12, 13, 0, -2, 12, [4096, 4096, 8128, 4096, 4096, 8064, 128, 65504, 128, 128, 256, 3584, 0]],
        '誉' : [12, 13, 0, -2, 12, [1024, 9344, 65504, 4352, 9344, 32704, 32800, 16256, 0, 16256, 8320, 16256, 0]],
        '輿' : [12, 13, 0, -2, 12, [1024, 50272, 49056, 54624, 40736, 54624, 49056, 33824, 65504, 0, 16448, 32800, 0]],
        '預' : [12, 13, 0, -2, 12, [0, 63456, 4352, 42976, 17440, 63456, 21536, 22496, 25632, 18400, 16960, 52256, 0]],
        '傭' : [12, 13, 0, -2, 12, [256, 8448, 12256, 18720, 20448, 51488, 19424, 18688, 24544, 26912, 20448, 18720, 0]],
        '幼' : [12, 13, 0, -2, 12, [256, 16640, 16640, 36832, 37152, 20768, 8736, 8736, 17440, 21536, 36928, 61824, 0]],
        '妖' : [12, 13, 0, -2, 12, [0, 16576, 18176, 61696, 20736, 20736, 24544, 37120, 57600, 12928, 17472, 22560, 0]],
        '容' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 41120, 20032, 4352, 8320, 32704, 41120, 8320, 16256, 0]],
        '庸' : [12, 13, 0, -2, 12, [512, 512, 32736, 16928, 32736, 16928, 24544, 16896, 32736, 41504, 49120, 8736, 0]],
        '揚' : [12, 13, 0, -2, 12, [0, 20416, 18496, 61376, 18496, 20448, 18432, 24544, 58528, 18720, 21024, 50368, 0]],
        '揺' : [12, 13, 0, -2, 12, [0, 20448, 16416, 59968, 19008, 20448, 16640, 28640, 49408, 18720, 18720, 53216, 0]],
        '擁' : [12, 13, 0, -2, 12, [512, 16896, 24544, 62752, 18752, 21472, 24128, 19424, 53824, 24544, 19008, 54240, 0]],
        '曜' : [12, 13, 0, -2, 12, [0, 3808, 57888, 44768, 41504, 44768, 58496, 45024, 47360, 45024, 59648, 4064, 0]],
        '楊' : [12, 13, 0, -2, 12, [0, 12224, 10304, 65472, 10304, 12256, 26624, 32736, 42144, 43296, 12832, 9408, 0]],
        '様' : [12, 13, 0, -2, 12, [1088, 9344, 12256, 61568, 10208, 8320, 28640, 24704, 46752, 41664, 9376, 10656, 0]],
        '洋' : [12, 13, 0, -2, 12, [2112, 1088, 32896, 24544, 256, 33024, 20448, 256, 16640, 24544, 33024, 33024, 0]],
        '溶' : [12, 13, 0, -2, 12, [512, 512, 40928, 21664, 2112, 37664, 17536, 2112, 24544, 18496, 34880, 36800, 0]],
        '熔' : [12, 13, 0, -2, 12, [512, 8704, 16352, 9376, 43072, 45856, 42112, 10304, 24544, 18496, 34880, 36800, 0]],
        '用' : [12, 13, 0, -2, 12, [0, 32736, 17440, 17440, 32736, 17440, 17440, 32736, 17440, 17440, 33824, 34016, 0]],
        '窯' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 29632, 2304, 16256, 1024, 65504, 0, 21056, 35104, 0]],
        '羊' : [12, 13, 0, -2, 12, [8320, 4224, 256, 65504, 1024, 1024, 32704, 1024, 1024, 65504, 1024, 1024, 0]],
        '耀' : [12, 13, 0, -2, 12, [0, 10080, 43296, 46944, 41248, 10080, 62016, 22496, 21632, 22496, 40064, 34784, 0]],
        '葉' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 9344, 32704, 9344, 10112, 8192, 65504, 3328, 13440, 50272, 0]],
        '蓉' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 65504, 41120, 20032, 37152, 16256, 24768, 41120, 16256, 0]],
        '要' : [12, 13, 0, -2, 12, [0, 65504, 2560, 32704, 19008, 19008, 32704, 2048, 65504, 4224, 7936, 24768, 0]],
        '謡' : [12, 13, 0, -2, 12, [0, 61408, 32, 59968, 2624, 61408, 256, 4064, 57600, 43296, 43296, 61408, 0]],
        '踊' : [12, 13, 0, -2, 12, [0, 63424, 36992, 37120, 65504, 10528, 12256, 47392, 45024, 43296, 43296, 63840, 0]],
        '遥' : [12, 13, 0, -2, 12, [0, 4064, 16416, 10816, 4064, 256, 61408, 8448, 9504, 10208, 20480, 36832, 0]],
        '陽' : [12, 13, 0, -2, 12, [0, 61376, 43072, 44992, 51264, 45024, 43008, 49120, 42144, 51488, 37408, 33984, 0]],
        '養' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 32704, 4352, 16256, 57568, 16256, 9408, 8960, 63712, 0]],
        '慾' : [12, 13, 0, -2, 12, [256, 18688, 34272, 12960, 19072, 64640, 18752, 31264, 1024, 4672, 20512, 36736, 0]],
        '抑' : [12, 13, 0, -2, 12, [0, 17152, 19680, 59680, 18720, 18720, 18720, 26912, 52000, 23840, 16736, 49408, 0]],
        '欲' : [12, 13, 0, -2, 12, [512, 20992, 35808, 672, 12960, 18592, 33920, 128, 63808, 35136, 35360, 64544, 0]],
        '沃' : [12, 13, 0, -2, 12, [0, 192, 36608, 16640, 256, 33024, 24544, 256, 16640, 17024, 33856, 38944, 0]],
        '浴' : [12, 13, 0, -2, 12, [0, 1088, 34848, 16384, 384, 33344, 17440, 2048, 2016, 9248, 17440, 34784, 0]],
        '翌' : [12, 13, 0, -2, 12, [0, 64480, 18720, 10400, 18720, 35360, 6240, 1024, 32704, 4352, 4352, 65504, 0]],
        '翼' : [12, 13, 0, -2, 12, [0, 64480, 18720, 10400, 18720, 49120, 9344, 32704, 4352, 65504, 8320, 49248, 0]],
        '淀' : [12, 13, 0, -2, 12, [256, 256, 40928, 20512, 0, 36832, 16640, 256, 18912, 18688, 36608, 45280, 0]],
        '羅' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 8704, 21056, 42976, 19008, 64480, 8768, 43584, 44000, 0]],
        '螺' : [12, 13, 0, -2, 12, [0, 12256, 10528, 65504, 43296, 45024, 43328, 64160, 10208, 10368, 10912, 64160, 0]],
        '裸' : [12, 13, 0, -2, 12, [0, 20448, 18720, 61408, 10528, 12256, 16640, 57312, 25344, 17728, 22816, 16640, 0]],
        '来' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 9344, 5376, 1024, 65504, 3328, 13440, 50272, 1024, 0]],
        '莱' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 32704, 9344, 5376, 65504, 3328, 13440, 50272, 0]],
        '頼' : [12, 13, 0, -2, 12, [8192, 10208, 63616, 9184, 64032, 44000, 43552, 64480, 8736, 29664, 43328, 9760, 0]],
        '雷' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 1024, 32704, 17472, 32704, 17472, 32704, 0]],
        '洛' : [12, 13, 0, -2, 12, [1024, 1024, 34752, 18496, 5248, 33536, 17536, 2144, 24512, 18496, 34880, 36800, 0]],
        '絡' : [12, 13, 0, -2, 12, [512, 8704, 17376, 33824, 59968, 8576, 16960, 62496, 12256, 42016, 42016, 10208, 0]],
        '落' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 18368, 10304, 33920, 18304, 2144, 8128, 18496, 36800, 0]],
        '酪' : [12, 13, 0, -2, 12, [512, 61952, 17376, 17440, 64064, 37248, 53824, 46112, 40928, 62496, 37920, 63456, 0]],
        '乱' : [12, 13, 0, -2, 12, [0, 3328, 61696, 8448, 8448, 64768, 8448, 8448, 64768, 34080, 34080, 64736, 0]],
        '卵' : [12, 13, 0, -2, 12, [0, 3552, 28960, 16672, 26016, 21856, 17696, 23840, 58656, 1376, 2304, 12544, 0]],
        '嵐' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 16448, 32704, 17472, 24384, 21824, 24384, 17568, 49056, 32800, 0]],
        '欄' : [12, 13, 0, -2, 12, [0, 12000, 10912, 65248, 10912, 12000, 26912, 28640, 43680, 43936, 11616, 10528, 0]],
        '濫' : [12, 13, 0, -2, 12, [0, 8000, 38016, 24544, 4352, 40704, 21728, 5120, 24544, 19104, 35488, 40928, 0]],
        '藍' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 41472, 64480, 35840, 63968, 40960, 65472, 19008, 19008, 65504, 0]],
        '蘭' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 64480, 35360, 64480, 35360, 65504, 38176, 40736, 42208, 0]],
        '覧' : [12, 13, 0, -2, 12, [512, 64000, 41952, 64512, 35296, 65152, 41088, 65408, 8320, 16256, 4640, 58336, 0]],
        '利' : [12, 13, 0, -2, 12, [0, 3104, 61728, 4384, 4384, 64800, 4384, 12576, 22816, 37920, 4128, 4192, 0]],
        '吏' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 17472, 17472, 32704, 1024, 3072, 4864, 57568, 0]],
        '履' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 21504, 26592, 19520, 21568, 26592, 43584, 41856, 11360, 0]],
        '李' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 3328, 13440, 50272, 16256, 512, 1024, 65504, 1024, 7168, 0]],
        '梨' : [12, 13, 0, -2, 12, [0, 64800, 8480, 64800, 12576, 27936, 41056, 1024, 65504, 3328, 13440, 50272, 0]],
        '理' : [12, 13, 0, -2, 12, [0, 4064, 59680, 20448, 18720, 18720, 61408, 16640, 20448, 16640, 57600, 4064, 0]],
        '璃' : [12, 13, 0, -2, 12, [256, 256, 65504, 17024, 18720, 19104, 61408, 16896, 24544, 21536, 63392, 4192, 0]],
        '痢' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 56864, 18592, 32416, 51360, 19616, 23072, 43296, 34912, 0]],
        '裏' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 9344, 16256, 9344, 32704, 1024, 65504, 4672, 57728, 15456, 0]],
        '裡' : [12, 13, 0, -2, 12, [0, 20448, 18720, 61408, 10528, 10528, 20448, 49408, 28640, 16640, 16640, 20448, 0]],
        '里' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 17472, 32704, 1024, 32704, 1024, 1024, 65504, 0]],
        '離' : [12, 13, 0, -2, 12, [4352, 4416, 64832, 11232, 37440, 44000, 65088, 4672, 65504, 41536, 47680, 34784, 0]],
        '陸' : [12, 13, 0, -2, 12, [256, 57600, 45024, 41216, 53216, 41600, 41632, 44256, 41216, 53216, 33024, 40928, 0]],
        '律' : [12, 13, 0, -2, 12, [256, 8448, 20448, 33056, 8160, 8480, 20448, 49408, 20448, 16640, 24544, 16640, 0]],
        '率' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 17472, 10880, 1024, 18752, 40736, 1024, 65504, 1024, 1024, 0]],
        '立' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 0, 4224, 4224, 4224, 4352, 4352, 256, 65504, 0]],
        '葎' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 20448, 33056, 12256, 16672, 53216, 16640, 20448, 16640, 0]],
        '掠' : [12, 13, 0, -2, 12, [256, 16640, 24544, 57344, 20416, 18496, 18496, 28608, 49408, 18752, 20768, 49920, 0]],
        '略' : [12, 13, 0, -2, 12, [512, 512, 64480, 44064, 43584, 43392, 64064, 44064, 44000, 43552, 64032, 992, 0]],
        '劉' : [12, 13, 0, -2, 12, [0, 56864, 35488, 43680, 54944, 15520, 17056, 64928, 4256, 65056, 21536, 65120, 0]],
        '流' : [12, 13, 0, -2, 12, [256, 256, 40928, 16960, 1088, 36768, 16416, 1344, 1344, 9536, 18752, 35168, 0]],
        '溜' : [12, 13, 0, -2, 12, [0, 7648, 37024, 21152, 5408, 39264, 16384, 4064, 2336, 12256, 18720, 36832, 0]],
        '琉' : [12, 13, 0, -2, 12, [512, 512, 61408, 16960, 17472, 20384, 57376, 17728, 17728, 17728, 59712, 2400, 0]],
        '留' : [12, 13, 0, -2, 12, [0, 63456, 33056, 37152, 35360, 62656, 0, 32704, 17472, 32704, 17472, 32704, 0]],
        '硫' : [12, 13, 0, -2, 12, [512, 512, 65504, 8768, 9280, 10144, 28704, 54592, 21824, 21824, 22848, 29024, 0]],
        '粒' : [12, 13, 0, -2, 12, [256, 8448, 41216, 45024, 8192, 62528, 9280, 25664, 29824, 42112, 41088, 12256, 0]],
        '隆' : [12, 13, 0, -2, 12, [0, 59328, 47168, 42112, 50048, 48224, 43264, 45024, 53504, 38848, 33024, 40928, 0]],
        '竜' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 4352, 65504, 9344, 16256, 9344, 16288, 1056, 2016, 0]],
        '龍' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 37376, 64480, 32, 64480, 35328, 64480, 35328, 64032, 35808, 0]],
        '侶' : [12, 13, 0, -2, 12, [0, 12256, 10272, 18464, 20448, 49408, 16896, 20448, 18464, 18464, 18464, 20448, 0]],
        '慮' : [12, 13, 0, -2, 12, [1920, 1024, 32736, 17440, 24512, 21056, 24512, 21056, 40896, 35072, 10304, 18336, 0]],
        '旅' : [12, 13, 0, -2, 12, [17408, 17408, 18400, 63616, 18688, 17024, 32416, 19104, 19008, 35392, 35360, 12832, 0]],
        '虜' : [12, 13, 0, -2, 12, [1920, 1024, 32736, 17440, 24512, 21056, 24512, 21056, 32736, 33824, 34848, 12480, 0]],
        '了' : [12, 13, 0, -2, 12, [0, 65472, 128, 256, 512, 1024, 1024, 1024, 1024, 1024, 1024, 7168, 0]],
        '亮' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 8320, 8320, 16256, 0, 65504, 37408, 4608, 8736, 50144, 0]],
        '僚' : [12, 13, 0, -2, 12, [256, 8448, 16352, 21568, 20448, 51232, 24544, 26656, 20448, 16640, 18752, 21280, 0]],
        '両' : [12, 13, 0, -2, 12, [0, 65504, 1024, 1024, 65504, 33824, 42144, 42144, 42144, 49056, 32800, 32864, 0]],
        '凌' : [12, 13, 0, -2, 12, [256, 256, 40928, 16640, 8160, 1152, 6240, 1984, 22592, 17536, 33664, 40032, 0]],
        '寮' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33824, 32704, 20800, 16256, 53600, 7936, 1024, 17472, 35872, 0]],
        '料' : [12, 13, 0, -2, 12, [0, 8256, 45632, 41280, 8256, 62528, 8768, 24672, 29120, 44608, 41024, 8256, 0]],
        '梁' : [12, 13, 0, -2, 12, [0, 10176, 37184, 17760, 2368, 8768, 17536, 65504, 3328, 13440, 50272, 1024, 0]],
        '涼' : [12, 13, 0, -2, 12, [256, 256, 40928, 16384, 4032, 34880, 18496, 4032, 16640, 18752, 37152, 33536, 0]],
        '猟' : [12, 13, 0, -2, 12, [512, 10528, 42048, 20416, 18752, 44992, 10560, 26944, 44992, 10560, 10528, 51488, 0]],
        '療' : [12, 13, 0, -2, 12, [512, 512, 32736, 16896, 65504, 18592, 24512, 59552, 20352, 16896, 37440, 42528, 0]],
        '瞭' : [12, 13, 0, -2, 12, [256, 256, 65504, 43584, 59360, 42016, 45024, 62496, 42976, 41216, 59712, 4896, 0]],
        '稜' : [12, 13, 0, -2, 12, [256, 6400, 61408, 8448, 16352, 57984, 11360, 25536, 27712, 42112, 41856, 11360, 0]],
        '糧' : [12, 13, 0, -2, 12, [0, 10176, 46144, 42944, 9280, 65504, 10528, 28640, 26912, 45024, 41216, 12256, 0]],
        '良' : [12, 13, 0, -2, 12, [1024, 1024, 16256, 8320, 16256, 8320, 8320, 16256, 8736, 8512, 11392, 61536, 0]],
        '諒' : [12, 13, 0, -2, 12, [256, 57600, 8160, 57344, 4032, 59456, 2112, 4032, 57600, 43328, 45344, 58112, 0]],
        '遼' : [12, 13, 0, -2, 12, [256, 256, 40928, 17472, 6080, 3168, 63424, 8448, 10560, 13088, 20480, 36832, 0]],
        '量' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 65504, 9344, 16256, 9344, 32704, 1024, 65504, 0]],
        '陵' : [12, 13, 0, -2, 12, [256, 57600, 45024, 41216, 57312, 41600, 44128, 41920, 44096, 50304, 33664, 40032, 0]],
        '領' : [12, 13, 0, -2, 12, [0, 26592, 37120, 2016, 62496, 2016, 62496, 22496, 21536, 26592, 16960, 19488, 0]],
        '力' : [12, 13, 0, -2, 12, [2048, 2048, 2048, 65504, 2080, 2080, 2080, 2080, 4128, 4128, 8256, 50048, 0]],
        '緑' : [12, 13, 0, -2, 12, [0, 10176, 16448, 38848, 61504, 12256, 16672, 60736, 9600, 43328, 45344, 9984, 0]],
        '倫' : [12, 13, 0, -2, 12, [0, 9088, 9280, 18464, 22496, 49152, 20448, 19104, 19104, 20448, 19104, 19104, 0]],
        '厘' : [12, 13, 0, -2, 12, [0, 32736, 16384, 24544, 20768, 24544, 20768, 24544, 16640, 40928, 33024, 16352, 0]],
        '林' : [12, 13, 0, -2, 12, [8320, 8320, 8320, 63456, 8320, 8320, 29056, 27328, 42144, 43136, 8320, 8320, 0]],
        '淋' : [12, 13, 0, -2, 12, [1152, 1152, 33920, 24544, 1152, 33920, 19840, 5824, 1184, 9344, 17536, 33920, 0]],
        '燐' : [12, 13, 0, -2, 12, [256, 9504, 9536, 45024, 42304, 43296, 10048, 9696, 21824, 19424, 33344, 35904, 0]],
        '琳' : [12, 13, 0, -2, 12, [1152, 1152, 58496, 24544, 17536, 17536, 60800, 22208, 17568, 17536, 58496, 1152, 0]],
        '臨' : [12, 13, 0, -2, 12, [1024, 1024, 63456, 43008, 43968, 62016, 37824, 61440, 44768, 43680, 64160, 3808, 0]],
        '輪' : [12, 13, 0, -2, 12, [0, 9088, 62528, 10272, 65504, 43008, 65504, 43680, 64160, 12256, 64160, 10912, 0]],
        '隣' : [12, 13, 0, -2, 12, [256, 58656, 42304, 45024, 50496, 43296, 44608, 44000, 47680, 51168, 33856, 38976, 0]],
        '鱗' : [12, 13, 0, -2, 12, [256, 29984, 38208, 10208, 63808, 42272, 65344, 42464, 64832, 992, 43584, 44096, 0]],
        '麟' : [12, 13, 0, -2, 12, [8448, 9504, 64832, 42976, 64832, 42272, 65344, 38368, 64832, 37856, 37440, 56384, 0]],
        '瑠' : [12, 13, 0, -2, 12, [0, 3552, 59552, 19104, 19232, 19808, 57344, 20448, 18720, 20448, 59680, 4064, 0]],
        '塁' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 32704, 49248, 13696, 50272, 16256, 1024, 65504, 0]],
        '涙' : [12, 13, 0, -2, 12, [0, 8160, 32768, 20448, 2080, 36832, 18688, 4064, 20736, 20864, 33344, 35872, 0]],
        '累' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 32704, 4608, 9280, 65504, 1024, 9344, 50272, 0]],
        '類' : [12, 13, 0, -2, 12, [0, 47072, 8320, 65504, 9248, 30688, 44064, 10208, 64544, 10208, 21056, 35872, 0]],
        '令' : [12, 13, 0, -2, 12, [0, 3840, 4224, 8256, 32672, 32768, 0, 32704, 1088, 1088, 1408, 1024, 0]],
        '伶' : [12, 13, 0, -2, 12, [0, 9088, 9280, 18464, 20416, 49152, 16384, 24544, 16928, 16928, 17088, 16896, 0]],
        '例' : [12, 13, 0, -2, 12, [0, 16160, 10400, 18592, 20128, 53920, 21152, 19104, 17568, 17440, 18464, 28768, 0]],
        '冷' : [12, 13, 0, -2, 12, [0, 1920, 34880, 20512, 4032, 0, 0, 24544, 16928, 33312, 33472, 512, 0]],
        '励' : [12, 13, 0, -2, 12, [256, 64768, 33024, 65504, 41248, 41248, 48416, 42272, 42272, 9760, 17440, 22720, 0]],
        '嶺' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 18560, 34784, 64032, 992, 64032, 19424, 18432, 23104, 17440, 0]],
        '怜' : [12, 13, 0, -2, 12, [0, 9088, 9280, 10272, 44992, 40960, 40960, 45024, 41504, 8736, 8896, 8704, 0]],
        '玲' : [12, 13, 0, -2, 12, [0, 896, 58432, 18464, 20416, 16384, 57344, 20448, 16928, 16928, 58048, 512, 0]],
        '礼' : [12, 13, 0, -2, 12, [0, 8704, 8704, 64000, 2560, 4608, 8704, 25088, 45600, 10784, 8736, 8672, 0]],
        '苓' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 3584, 12672, 57184, 0, 32704, 2112, 2112, 2432, 0]],
        '鈴' : [12, 13, 0, -2, 12, [0, 12736, 16928, 33792, 63456, 8192, 63488, 10208, 45344, 41248, 8544, 63744, 0]],
        '隷' : [12, 13, 0, -2, 12, [256, 8448, 65504, 8480, 63456, 288, 63456, 288, 64320, 9600, 43328, 45856, 0]],
        '零' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 40608, 12672, 57184, 0, 32704, 2112, 2432, 0]],
        '霊' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 32704, 0, 65504, 19008, 10880, 65504, 0]],
        '麗' : [12, 13, 0, -2, 12, [0, 64480, 0, 64480, 43680, 1024, 32736, 18720, 32736, 20736, 38688, 47584, 0]],
        '齢' : [12, 13, 0, -2, 12, [8192, 8640, 47648, 41984, 64480, 20480, 43008, 64480, 43296, 55584, 35168, 63744, 0]],
        '暦' : [12, 13, 0, -2, 12, [0, 32736, 18560, 32736, 18816, 23232, 26784, 24512, 20544, 40896, 36928, 8128, 0]],
        '歴' : [12, 13, 0, -2, 12, [0, 32736, 18560, 32736, 18816, 23232, 27808, 19072, 16896, 37824, 37376, 16352, 0]],
        '列' : [12, 13, 0, -2, 12, [0, 64544, 8480, 8480, 15648, 17696, 25888, 38176, 2336, 2080, 12320, 49248, 0]],
        '劣' : [12, 13, 0, -2, 12, [1024, 9344, 9280, 19488, 33152, 7680, 58368, 16352, 1056, 2080, 12352, 50048, 0]],
        '烈' : [12, 13, 0, -2, 12, [0, 64544, 8480, 15648, 25888, 38176, 2336, 4128, 24672, 0, 21056, 35104, 0]],
        '裂' : [12, 13, 0, -2, 12, [0, 64800, 8480, 15648, 25888, 37920, 2144, 4608, 65504, 4672, 57728, 15456, 0]],
        '廉' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 24544, 17696, 32736, 17696, 24544, 34048, 38208, 9504, 0]],
        '恋' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2304, 18752, 35104, 4352, 8192, 1024, 4672, 20512, 36736, 0]],
        '憐' : [12, 13, 0, -2, 12, [256, 9504, 9536, 12256, 42304, 43296, 44608, 44000, 47680, 10208, 9280, 14400, 0]],
        '漣' : [12, 13, 0, -2, 12, [128, 36992, 20448, 672, 992, 39584, 19424, 2176, 20448, 18560, 37888, 41952, 0]],
        '煉' : [12, 13, 0, -2, 12, [256, 8448, 16352, 41216, 45024, 43296, 12256, 10528, 24544, 17152, 34112, 39200, 0]],
        '簾' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 32736, 17696, 32736, 17696, 24544, 34048, 38208, 9504, 0]],
        '練' : [12, 13, 0, -2, 12, [256, 8448, 20448, 33024, 61408, 10528, 20448, 63776, 12256, 41728, 42304, 14624, 0]],
        '聯' : [12, 13, 0, -2, 12, [544, 62528, 43680, 58432, 43680, 45024, 57984, 43680, 43680, 61152, 41600, 11392, 0]],
        '蓮' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 20448, 10528, 4064, 59680, 12256, 8448, 20480, 36832, 0]],
        '連' : [12, 13, 0, -2, 12, [256, 256, 20448, 8448, 4064, 2336, 61408, 10528, 12256, 8448, 20480, 36832, 0]],
        '錬' : [12, 13, 0, -2, 12, [256, 12544, 20448, 33024, 65504, 10528, 65504, 10528, 45024, 41728, 9536, 63776, 0]],
        '呂' : [12, 13, 0, -2, 12, [0, 16320, 8256, 8256, 8256, 16320, 1024, 32736, 16416, 16416, 16416, 32736, 0]],
        '魯' : [12, 13, 0, -2, 12, [0, 16128, 16896, 65472, 17472, 32704, 21056, 49056, 16448, 32704, 16448, 32704, 0]],
        '櫓' : [12, 13, 0, -2, 12, [0, 9152, 9344, 65504, 10528, 12256, 27296, 30688, 42016, 42976, 9248, 10208, 0]],
        '炉' : [12, 13, 0, -2, 12, [0, 10208, 8192, 43008, 47072, 42016, 9248, 10208, 21504, 21504, 34816, 34816, 0]],
        '賂' : [12, 13, 0, -2, 12, [512, 61952, 37856, 62496, 39488, 61824, 37440, 37920, 64480, 544, 21024, 35808, 0]],
        '路' : [12, 13, 0, -2, 12, [512, 61952, 37856, 37920, 64064, 8576, 8768, 46112, 41952, 41504, 41504, 64480, 0]],
        '露' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 62432, 40512, 61824, 11872, 41536, 64448, 0]],
        '労' : [12, 13, 0, -2, 12, [4096, 2112, 17536, 8448, 65504, 33824, 1024, 32704, 2112, 2112, 4160, 25472, 0]],
        '婁' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 17472, 65504, 17472, 65504, 33824, 65504, 4224, 7936, 57568, 0]],
        '廊' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 36864, 48608, 42272, 48448, 42304, 48416, 43296, 42432, 47872, 0]],
        '弄' : [12, 13, 0, -2, 12, [0, 65504, 1024, 16256, 1024, 1024, 65504, 4352, 65504, 4352, 8448, 49408, 0]],
        '朗' : [12, 13, 0, -2, 12, [8192, 9184, 64032, 35360, 64480, 35360, 35360, 64480, 33312, 37408, 35360, 62560, 0]],
        '楼' : [12, 13, 0, -2, 12, [256, 9504, 9536, 65504, 9536, 10528, 25088, 32736, 42048, 44160, 9088, 11360, 0]],
        '榔' : [12, 13, 0, -2, 12, [1024, 9696, 12064, 63776, 12096, 10624, 26944, 28448, 47392, 43808, 11712, 10496, 0]],
        '浪' : [12, 13, 0, -2, 12, [512, 512, 36800, 18496, 4032, 34880, 18496, 4032, 2336, 10560, 18560, 36448, 0]],
        '漏' : [12, 13, 0, -2, 12, [0, 4064, 34848, 20448, 2048, 36832, 18688, 4064, 18720, 19296, 38304, 37152, 0]],
        '牢' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33824, 42016, 16320, 17408, 33792, 65504, 1024, 1024, 1024, 0]],
        '狼' : [12, 13, 0, -2, 12, [256, 8448, 44992, 18496, 20416, 43072, 10304, 28608, 43296, 10560, 10368, 52832, 0]],
        '篭' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 1024, 65504, 4352, 65504, 17472, 32704, 17440, 2016, 0]],
        '老' : [12, 13, 0, -2, 12, [1024, 1088, 32736, 1152, 1280, 65504, 2048, 14784, 52736, 2048, 2080, 4064, 0]],
        '聾' : [12, 13, 0, -2, 12, [8448, 8448, 64992, 18688, 65504, 18464, 31712, 18944, 65504, 8320, 16352, 49280, 0]],
        '蝋' : [12, 13, 0, -2, 12, [544, 10528, 9280, 65472, 43328, 44992, 43328, 63808, 12224, 10560, 10528, 63776, 0]],
        '郎' : [12, 13, 0, -2, 12, [8192, 9184, 64032, 35360, 64064, 35456, 35392, 64032, 33312, 37408, 35520, 62976, 0]],
        '六' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 0, 0, 8320, 8320, 16448, 16448, 32800, 32800, 0]],
        '麓' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 21312, 38176, 32736, 18720, 32736, 41216, 48608, 8448, 32224, 0]],
        '禄' : [12, 13, 0, -2, 12, [0, 18368, 16448, 63424, 4160, 8160, 8480, 19776, 58752, 18752, 20768, 18176, 0]],
        '肋' : [12, 13, 0, -2, 12, [256, 61696, 37120, 38880, 61728, 37152, 37152, 61984, 37408, 37920, 37952, 45440, 0]],
        '録' : [12, 13, 0, -2, 12, [0, 14272, 16448, 34752, 61504, 12256, 61728, 11584, 42368, 43328, 12576, 59136, 0]],
        '論' : [12, 13, 0, -2, 12, [0, 58240, 1088, 59424, 4064, 57344, 4064, 2720, 60064, 45024, 43680, 60064, 0]],
        '倭' : [12, 13, 0, -2, 12, [0, 12256, 8448, 24544, 17152, 50496, 18720, 24544, 17472, 17536, 17280, 23648, 0]],
        '和' : [12, 13, 0, -2, 12, [0, 3072, 61920, 4384, 4384, 65312, 4384, 6432, 13600, 21280, 37152, 4576, 0]],
        '話' : [12, 13, 0, -2, 12, [0, 61536, 1920, 61568, 128, 63456, 128, 128, 63456, 37920, 37920, 63456, 0]],
        '歪' : [12, 13, 0, -2, 12, [0, 65504, 512, 3072, 13696, 50272, 1024, 32704, 1024, 10112, 9216, 65504, 0]],
        '賄' : [12, 13, 0, -2, 12, [128, 61568, 40928, 61696, 37120, 62432, 38432, 39904, 61984, 992, 21024, 35424, 0]],
        '脇' : [12, 13, 0, -2, 12, [512, 57856, 44992, 41536, 57920, 42048, 49120, 60064, 43680, 43680, 45856, 42592, 0]],
        '惑' : [12, 13, 0, -2, 12, [256, 288, 65504, 256, 31040, 18560, 18752, 31264, 256, 4160, 20512, 36736, 0]],
        '枠' : [12, 13, 0, -2, 12, [512, 8704, 12224, 62016, 8768, 9312, 25856, 24832, 49120, 41216, 8448, 8448, 0]],
        '鷲' : [12, 13, 0, -2, 12, [4352, 4384, 65504, 17536, 32128, 4768, 21728, 49024, 8320, 16352, 16416, 38240, 0]],
        '亙' : [12, 13, 0, -2, 12, [0, 65504, 2048, 2048, 3968, 5248, 4736, 10496, 9472, 512, 512, 65504, 0]],
        '亘' : [12, 13, 0, -2, 12, [0, 65504, 0, 16256, 8320, 16256, 8320, 8320, 16256, 0, 0, 65504, 0]],
        '鰐' : [12, 13, 0, -2, 12, [0, 30688, 38176, 10208, 63488, 45024, 63488, 45024, 63744, 992, 43552, 43200, 0]],
        '詫' : [12, 13, 0, -2, 12, [256, 57600, 4064, 59424, 448, 60928, 512, 992, 60928, 41472, 41504, 58336, 0]],
        '藁' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 65504, 20800, 24384, 1024, 65504, 3328, 13440, 50272, 0]],
        '蕨' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32736, 18944, 32736, 21152, 21664, 31872, 41280, 42528, 0]],
        '椀' : [12, 13, 0, -2, 12, [256, 8448, 12256, 63520, 12000, 10912, 27296, 27296, 46816, 41600, 9376, 14560, 0]],
        '湾' : [12, 13, 0, -2, 12, [512, 512, 40928, 21824, 1312, 40928, 16416, 4064, 18432, 24544, 36896, 33216, 0]],
        '碗' : [12, 13, 0, -2, 12, [256, 256, 65504, 10272, 12000, 10912, 31392, 54944, 21216, 21120, 21664, 30944, 0]],
        '腕' : [12, 13, 0, -2, 12, [256, 57600, 49120, 43040, 61152, 43680, 43680, 60064, 46816, 41600, 42144, 47328, 0]],
        '弌' : [12, 13, 0, -2, 12, [512, 544, 544, 65504, 256, 256, 128, 128, 64, 64576, 32, 32, 0]],
        '丐' : [12, 13, 0, -2, 12, [0, 65504, 1024, 1024, 18368, 17408, 17408, 32736, 32800, 32800, 64, 896, 0]],
        '丕' : [12, 13, 0, -2, 12, [0, 65504, 256, 512, 1024, 3200, 13376, 50208, 1024, 1024, 0, 65504, 0]],
        '个' : [12, 13, 0, -2, 12, [3584, 4352, 8320, 17472, 33824, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 0]],
        '丱' : [12, 13, 0, -2, 12, [256, 2304, 35104, 35104, 35104, 35104, 35104, 63968, 35104, 2304, 4352, 24832, 0]],
        '丶' : [12, 13, 0, -2, 12, [0, 0, 0, 0, 0, 6144, 1536, 256, 0, 0, 0, 0, 0]],
        '丼' : [12, 13, 0, -2, 12, [4352, 4352, 4352, 65504, 4352, 6400, 5376, 4352, 65504, 8448, 16640, 16640, 0]],
        '丿' : [12, 13, 0, -2, 12, [0, 512, 512, 512, 512, 512, 512, 512, 1024, 1024, 6144, 24576, 0]],
        '乂' : [12, 13, 0, -2, 12, [0, 128, 8320, 8320, 4352, 4352, 2560, 2560, 1024, 2560, 12672, 49248, 0]],
        '乖' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 1024, 5408, 62784, 5504, 5376, 13600, 54496, 1024, 0]],
        '乘' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 5376, 62816, 5504, 13600, 54496, 3584, 13696, 50272, 0]],
        '亂' : [12, 13, 0, -2, 12, [0, 65152, 42112, 37504, 31872, 2176, 65152, 41600, 65152, 43680, 37536, 44768, 0]],
        '亅' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 15360, 0]],
        '豫' : [12, 13, 0, -2, 12, [0, 62400, 5248, 45024, 18720, 63776, 20448, 17536, 22944, 18112, 22688, 49920, 0]],
        '亊' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 9344, 5376, 32704, 1088, 65504, 1088, 32704, 1024, 7168, 0]],
        '舒' : [12, 13, 0, -2, 12, [0, 13280, 18464, 33088, 63616, 10208, 63648, 8352, 63680, 34944, 34944, 63872, 0]],
        '弍' : [12, 13, 0, -2, 12, [512, 544, 544, 65504, 256, 256, 30848, 128, 64, 64, 64544, 32, 0]],
        '于' : [12, 13, 0, -2, 12, [0, 32704, 1024, 1024, 1024, 1024, 65504, 1024, 1024, 1024, 1024, 7168, 0]],
        '亞' : [12, 13, 0, -2, 12, [0, 65504, 2560, 2560, 31680, 16448, 16448, 31680, 2560, 2560, 2560, 65504, 0]],
        '亟' : [12, 13, 0, -2, 12, [0, 65504, 2048, 4096, 7648, 62496, 38176, 38208, 62656, 1824, 2048, 65504, 0]],
        '亠' : [12, 13, 0, -2, 12, [0, 0, 1024, 1024, 1024, 1024, 65504, 0, 0, 0, 0, 0, 0]],
        '亢' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 0, 0, 7936, 4352, 4352, 4352, 4384, 8480, 49376, 0]],
        '亰' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 0, 16256, 8320, 16256, 8320, 16256, 1024, 17472, 35872, 0]],
        '亳' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 7936, 0, 65504, 33312, 48128, 1984, 31776, 2016, 0]],
        '亶' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 20800, 24384, 16448, 32704, 8320, 16256, 8320, 8320, 65504, 0]],
        '从' : [12, 13, 0, -2, 12, [0, 8448, 8448, 8448, 8448, 8448, 21120, 21120, 21568, 33856, 34848, 34848, 0]],
        '仍' : [12, 13, 0, -2, 12, [0, 16256, 9344, 17536, 17536, 50400, 17440, 17440, 18464, 18464, 20544, 20864, 0]],
        '仄' : [12, 13, 0, -2, 12, [0, 32736, 16384, 16640, 16640, 16640, 16640, 16640, 17024, 33408, 33856, 6176, 0]],
        '仆' : [12, 13, 0, -2, 12, [256, 4352, 4352, 8448, 8448, 25024, 41264, 8448, 8448, 8448, 8448, 8448, 0]],
        '仂' : [12, 13, 0, -2, 12, [512, 8704, 8704, 20448, 16928, 49696, 16928, 17440, 17440, 18464, 18496, 16768, 0]],
        '仗' : [12, 13, 0, -2, 12, [128, 8320, 8320, 20448, 16512, 49280, 17536, 17024, 16768, 16512, 17216, 19488, 0]],
        '仞' : [12, 13, 0, -2, 12, [0, 12256, 8736, 16928, 18976, 50720, 16928, 17696, 17696, 18464, 18496, 16768, 0]],
        '仭' : [12, 13, 0, -2, 12, [0, 12256, 8736, 18976, 17952, 49696, 17184, 17568, 17504, 18464, 18496, 16768, 0]],
        '仟' : [12, 13, 0, -2, 12, [0, 8416, 12032, 16640, 16640, 49408, 24544, 16640, 16640, 16640, 16640, 16640, 0]],
        '价' : [12, 13, 0, -2, 12, [0, 9088, 9280, 18464, 20480, 50240, 17472, 17472, 17472, 17472, 18496, 18496, 0]],
        '伉' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16384, 16384, 51136, 17472, 17472, 17472, 17472, 18496, 18528, 0]],
        '佚' : [12, 13, 0, -2, 12, [256, 10496, 10496, 20448, 20736, 53504, 20448, 16640, 16640, 17024, 17472, 22560, 0]],
        '估' : [12, 13, 0, -2, 12, [256, 8448, 8448, 20448, 16640, 49408, 16640, 20448, 18464, 18464, 18464, 20448, 0]],
        '佛' : [12, 13, 0, -2, 12, [640, 8832, 16352, 17056, 20448, 51840, 19072, 20448, 17056, 17056, 17600, 22656, 0]],
        '佝' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 18464, 18464, 50976, 17696, 17696, 18208, 16416, 16448, 16768, 0]],
        '佗' : [12, 13, 0, -2, 12, [256, 8448, 12256, 18464, 18464, 50176, 17600, 18176, 17408, 17408, 17440, 17376, 0]],
        '佇' : [12, 13, 0, -2, 12, [256, 8448, 12256, 18464, 18464, 49152, 20448, 16512, 16512, 16512, 16512, 17280, 0]],
        '佶' : [12, 13, 0, -2, 12, [128, 8320, 12256, 16512, 16512, 51168, 16384, 16384, 18400, 17440, 17440, 18400, 0]],
        '侈' : [12, 13, 0, -2, 12, [256, 8448, 9184, 19488, 16960, 49536, 19968, 17376, 19488, 16960, 16768, 24064, 0]],
        '侏' : [12, 13, 0, -2, 12, [256, 10496, 10496, 20448, 20736, 53504, 20448, 16640, 17280, 17728, 22816, 16640, 0]],
        '侘' : [12, 13, 0, -2, 12, [512, 8704, 16352, 20512, 20896, 52736, 16896, 17376, 24064, 16896, 16928, 17376, 0]],
        '佻' : [12, 13, 0, -2, 12, [256, 9472, 9504, 21824, 19712, 50432, 17792, 19776, 21792, 17664, 18720, 28896, 0]],
        '佩' : [12, 13, 0, -2, 12, [0, 8128, 12352, 24384, 54336, 21568, 24384, 21824, 21824, 21792, 25888, 17440, 0]],
        '佰' : [12, 13, 0, -2, 12, [0, 12256, 8320, 16640, 18400, 50208, 17440, 18400, 17440, 17440, 17440, 18400, 0]],
        '侑' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16896, 16896, 51168, 17440, 20448, 21536, 18400, 17440, 17504, 0]],
        '佯' : [12, 13, 0, -2, 12, [1088, 8768, 8320, 20448, 16640, 49408, 20448, 16640, 16640, 24544, 16640, 16640, 0]],
        '來' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 9344, 9344, 21824, 33824, 3584, 13696, 50272, 1024, 0]],
        '侖' : [12, 13, 0, -2, 12, [0, 7936, 8320, 32704, 32800, 32704, 19008, 19008, 32704, 19008, 19008, 19136, 0]],
        '儘' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16672, 24544, 49440, 20448, 19008, 20768, 20448, 19104, 24544, 0]],
        '俔' : [12, 13, 0, -2, 12, [0, 12256, 10272, 20448, 18464, 53216, 18464, 20448, 17024, 17024, 17568, 22752, 0]],
        '俟' : [12, 13, 0, -2, 12, [512, 8832, 9280, 24544, 17408, 51168, 18688, 18688, 24544, 17024, 17472, 22560, 0]],
        '俎' : [12, 13, 0, -2, 12, [0, 9184, 8736, 8736, 13280, 18976, 33312, 9184, 8736, 12832, 18976, 34784, 0]],
        '俘' : [12, 13, 0, -2, 12, [0, 10208, 9216, 18752, 18720, 49152, 17376, 16448, 16512, 20448, 16512, 16768, 0]],
        '俛' : [12, 13, 0, -2, 12, [0, 10112, 10368, 20736, 20448, 51488, 18720, 20448, 17024, 17024, 17568, 22752, 0]],
        '俑' : [12, 13, 0, -2, 12, [0, 12224, 8320, 16640, 20448, 51488, 20448, 18720, 20448, 18720, 18720, 18784, 0]],
        '俚' : [12, 13, 0, -2, 12, [0, 12256, 10528, 20448, 18720, 51488, 20448, 16640, 20448, 16640, 16640, 20448, 0]],
        '俐' : [12, 13, 0, -2, 12, [0, 8992, 15520, 17568, 17568, 57248, 17568, 19616, 20000, 21792, 21536, 17504, 0]],
        '俤' : [12, 13, 0, -2, 12, [2176, 9344, 8448, 24512, 16960, 57280, 20992, 24544, 17952, 18976, 21184, 16896, 0]],
        '俥' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16640, 20448, 51488, 20448, 18720, 20448, 16640, 20448, 16640, 0]],
        '倚' : [12, 13, 0, -2, 12, [256, 8448, 12256, 17024, 17472, 51232, 24544, 16416, 18208, 17696, 18208, 16480, 0]],
        '倨' : [12, 13, 0, -2, 12, [0, 12256, 10272, 20448, 18560, 53216, 18560, 18560, 20448, 21536, 21536, 18400, 0]],
        '倔' : [12, 13, 0, -2, 12, [0, 12256, 10272, 20448, 18560, 51872, 19104, 19424, 18560, 21664, 21664, 18400, 0]],
        '倪' : [12, 13, 0, -2, 12, [256, 8704, 11488, 18464, 20192, 51232, 18464, 20448, 17024, 17024, 17568, 22752, 0]],
        '倥' : [12, 13, 0, -2, 12, [256, 8448, 16352, 21152, 21120, 49824, 19680, 16384, 18368, 16640, 16640, 24544, 0]],
        '倅' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16384, 17472, 50240, 19104, 16640, 16640, 24544, 16640, 16640, 0]],
        '伜' : [12, 13, 0, -2, 12, [512, 8704, 12224, 16960, 16960, 50272, 17664, 16640, 24544, 16640, 16640, 16640, 0]],
        '俶' : [12, 13, 0, -2, 12, [1024, 9440, 10016, 17440, 17568, 57248, 17568, 17472, 21824, 21824, 17568, 19744, 0]],
        '倡' : [12, 13, 0, -2, 12, [0, 10176, 9280, 18368, 17472, 51136, 16384, 20448, 18464, 20448, 18464, 20448, 0]],
        '倩' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16640, 20448, 49408, 24544, 18720, 18720, 20448, 18464, 18528, 0]],
        '倬' : [12, 13, 0, -2, 12, [512, 8704, 9184, 16896, 20448, 51232, 20448, 18464, 20448, 16640, 24544, 16640, 0]],
        '俾' : [12, 13, 0, -2, 12, [256, 8704, 12256, 18720, 20448, 51488, 18720, 20448, 17024, 17536, 24544, 16512, 0]],
        '俯' : [12, 13, 0, -2, 12, [512, 8704, 16352, 21056, 21056, 55264, 23616, 21824, 21696, 21568, 25664, 17600, 0]],
        '們' : [12, 13, 0, -2, 12, [0, 12000, 10912, 20192, 19104, 51872, 20192, 18464, 18464, 18464, 18464, 18528, 0]],
        '倆' : [12, 13, 0, -2, 12, [0, 16352, 8704, 16896, 24544, 53792, 21408, 23200, 23200, 22368, 21024, 21088, 0]],
        '偃' : [12, 13, 0, -2, 12, [0, 12256, 10784, 19424, 18976, 52192, 18560, 20448, 18752, 18880, 18976, 20448, 0]],
        '假' : [12, 13, 0, -2, 12, [0, 12256, 10528, 18720, 20320, 51200, 18912, 20000, 18592, 20128, 18624, 19232, 0]],
        '會' : [12, 13, 0, -2, 12, [0, 7936, 8320, 32704, 32800, 32704, 21824, 32704, 8320, 16256, 8320, 16256, 0]],
        '偕' : [12, 13, 0, -2, 12, [128, 10368, 12000, 18560, 18560, 52832, 16640, 20448, 18464, 20448, 18464, 20448, 0]],
        '偐' : [12, 13, 0, -2, 12, [256, 8448, 12256, 17472, 17280, 50240, 20448, 18976, 18624, 21248, 20576, 18304, 0]],
        '偈' : [12, 13, 0, -2, 12, [0, 12224, 10304, 20416, 18496, 53184, 17408, 20448, 22816, 19104, 20256, 16576, 0]],
        '做' : [12, 13, 0, -2, 12, [2304, 10496, 10720, 24352, 18720, 51232, 18592, 24224, 21056, 21056, 21152, 24352, 0]],
        '偖' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16672, 16704, 57312, 16960, 18400, 19488, 22496, 17440, 18400, 0]],
        '偬' : [12, 13, 0, -2, 12, [1024, 9216, 12256, 21152, 19616, 52000, 21152, 16448, 16896, 18752, 26656, 18304, 0]],
        '偸' : [12, 13, 0, -2, 12, [0, 12160, 9280, 18464, 22496, 49152, 24224, 21152, 24384, 21312, 24224, 21152, 0]],
        '傀' : [12, 13, 0, -2, 12, [256, 8704, 12256, 18720, 20448, 51488, 18720, 20448, 17728, 17728, 18720, 18656, 0]],
        '傚' : [12, 13, 0, -2, 12, [1152, 9344, 16352, 16544, 19104, 53536, 16416, 16704, 19008, 17472, 19104, 20768, 0]],
        '傅' : [12, 13, 0, -2, 12, [256, 8480, 12256, 16640, 20448, 51488, 20448, 18720, 16448, 24544, 17472, 17088, 0]],
        '傴' : [12, 13, 0, -2, 12, [0, 12256, 10240, 19392, 19008, 52160, 18432, 20192, 19104, 20192, 18432, 20448, 0]],
        '傲' : [12, 13, 0, -2, 12, [1152, 9344, 16096, 17568, 24480, 50208, 24480, 18592, 20032, 21056, 17056, 19744, 0]],
        '僉' : [12, 13, 0, -2, 12, [0, 7936, 8320, 32704, 32800, 31680, 19008, 19008, 31680, 8320, 20800, 35360, 0]],
        '僊' : [12, 13, 0, -2, 12, [0, 12256, 8832, 20448, 19104, 53216, 16640, 24544, 17472, 20416, 21536, 18400, 0]],
        '傳' : [12, 13, 0, -2, 12, [256, 8448, 16352, 18720, 20448, 51488, 24544, 16448, 24544, 16448, 17472, 17088, 0]],
        '僂' : [12, 13, 0, -2, 12, [256, 8448, 12256, 18720, 24544, 51488, 20448, 18720, 24544, 17472, 18304, 22624, 0]],
        '僖' : [12, 13, 0, -2, 12, [256, 8448, 16352, 16640, 20448, 51232, 20448, 17472, 24544, 18464, 18464, 20448, 0]],
        '僞' : [12, 13, 0, -2, 12, [0, 12256, 10816, 20768, 24512, 51264, 20448, 18464, 20448, 20512, 21792, 19136, 0]],
        '僥' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16640, 24544, 50240, 24288, 17472, 24544, 17024, 17568, 22752, 0]],
        '僭' : [12, 13, 0, -2, 12, [1152, 9344, 16352, 21632, 24544, 51520, 21088, 20416, 18496, 20416, 18496, 20416, 0]],
        '僣' : [12, 13, 0, -2, 12, [1152, 9344, 16352, 17536, 24544, 51520, 21024, 20416, 18496, 20416, 18496, 20416, 0]],
        '僮' : [12, 13, 0, -2, 12, [256, 8448, 16352, 17472, 17472, 57312, 18720, 20448, 18720, 20448, 16640, 24544, 0]],
        '價' : [12, 13, 0, -2, 12, [0, 16352, 8832, 24544, 21152, 57312, 18496, 20416, 18496, 20416, 17472, 22560, 0]],
        '僵' : [12, 13, 0, -2, 12, [0, 16352, 10528, 20448, 18720, 53216, 16384, 24544, 18720, 20448, 18720, 24544, 0]],
        '儉' : [12, 13, 0, -2, 12, [0, 10176, 10272, 22496, 16384, 52960, 19104, 19104, 20192, 17472, 19104, 20768, 0]],
        '儁' : [12, 13, 0, -2, 12, [1152, 9344, 12256, 18688, 24544, 51456, 20448, 16384, 20192, 19104, 19360, 18528, 0]],
        '儂' : [12, 13, 0, -2, 12, [640, 8832, 12256, 19104, 20448, 51872, 20448, 18432, 20448, 21824, 21632, 18016, 0]],
        '儖' : [12, 13, 0, -2, 12, [0, 12096, 10880, 20448, 18688, 52992, 19168, 18944, 20448, 19104, 19104, 24544, 0]],
        '儕' : [12, 13, 0, -2, 12, [512, 8704, 16352, 21824, 22336, 60832, 18496, 20416, 18496, 20416, 20544, 20544, 0]],
        '儔' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16640, 24544, 49184, 20416, 16640, 24544, 18976, 19104, 20064, 0]],
        '儚' : [12, 13, 0, -2, 12, [1152, 9344, 16352, 17536, 20448, 51872, 24544, 21536, 18368, 23104, 16768, 19968, 0]],
        '儡' : [12, 13, 0, -2, 12, [0, 12224, 10560, 20416, 18752, 53184, 16384, 24544, 21152, 24544, 21152, 24544, 0]],
        '儺' : [12, 13, 0, -2, 12, [256, 10560, 16192, 18912, 24384, 54752, 24384, 18752, 32736, 18752, 21824, 25568, 0]],
        '儷' : [12, 13, 0, -2, 12, [0, 12000, 8192, 24288, 19104, 49408, 20448, 19104, 20448, 21632, 22432, 19680, 0]],
        '儼' : [12, 13, 0, -2, 12, [0, 12000, 10912, 24544, 21632, 57056, 21152, 24352, 21152, 24224, 29376, 17184, 0]],
        '儻' : [12, 13, 0, -2, 12, [256, 9536, 16352, 17568, 20448, 51488, 20448, 16640, 20448, 16640, 24544, 19104, 0]],
        '儿' : [12, 13, 0, -2, 12, [0, 8704, 8704, 8704, 8704, 8704, 8704, 8704, 16928, 16928, 33312, 33760, 0]],
        '兀' : [12, 13, 0, -2, 12, [0, 65504, 4608, 4608, 4608, 4608, 8704, 8704, 16928, 16928, 33312, 33760, 0]],
        '兒' : [12, 13, 0, -2, 12, [1024, 2048, 29120, 16448, 31680, 16448, 16448, 32704, 4608, 4640, 8736, 50144, 0]],
        '兌' : [12, 13, 0, -2, 12, [0, 10112, 8320, 16448, 32704, 41120, 41120, 16256, 4608, 4640, 8736, 50144, 0]],
        '兔' : [12, 13, 0, -2, 12, [0, 7936, 8448, 16896, 65472, 17472, 17472, 32704, 5376, 5280, 9248, 51168, 0]],
        '兢' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8320, 64480, 35360, 35360, 64480, 20800, 20800, 37440, 39520, 0]],
        '竸' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 19008, 65504, 19008, 31680, 19008, 31680, 21120, 38048, 40160, 0]],
        '兩' : [12, 13, 0, -2, 12, [0, 65504, 1024, 1024, 65504, 33824, 46496, 38048, 46496, 52832, 33824, 33888, 0]],
        '兪' : [12, 13, 0, -2, 12, [0, 32512, 4224, 16224, 16384, 63776, 35104, 59968, 35392, 59968, 35104, 39200, 0]],
        '兮' : [12, 13, 0, -2, 12, [0, 10112, 8320, 16448, 32704, 36896, 36896, 8064, 8320, 8320, 256, 1536, 0]],
        '冀' : [12, 13, 0, -2, 12, [2560, 2656, 64384, 2592, 65504, 9344, 16256, 9344, 32704, 4352, 65504, 8320, 0]],
        '冂' : [12, 13, 0, -2, 12, [0, 65504, 32800, 32800, 32800, 32800, 32800, 32800, 32800, 32800, 32800, 32992, 0]],
        '囘' : [12, 13, 0, -2, 12, [0, 65504, 32800, 32800, 40736, 37152, 40736, 36896, 37152, 40736, 32800, 32864, 0]],
        '册' : [12, 13, 0, -2, 12, [0, 31680, 19008, 19008, 19008, 65504, 19008, 19008, 19008, 19008, 35392, 39616, 0]],
        '冉' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 17472, 17472, 32704, 17472, 17472, 65504, 16448, 16448, 16576, 0]],
        '冏' : [12, 13, 0, -2, 12, [0, 65504, 35360, 35360, 37408, 41440, 57120, 37152, 37152, 40736, 32800, 32864, 0]],
        '冑' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33824, 65504, 33824, 65504, 16448, 24384, 16448, 24384, 16576, 0]],
        '冓' : [12, 13, 0, -2, 12, [4352, 4352, 32704, 4352, 65504, 1024, 32704, 17472, 32704, 17472, 65504, 16448, 0]],
        '冕' : [12, 13, 0, -2, 12, [0, 32704, 16448, 24384, 16448, 24384, 8704, 65472, 17472, 32704, 4640, 58336, 0]],
        '冖' : [12, 13, 0, -2, 12, [0, 0, 0, 65504, 32800, 32800, 32800, 0, 0, 0, 0, 0, 0]],
        '冤' : [12, 13, 0, -2, 12, [0, 65504, 32800, 40736, 8704, 32704, 50240, 17472, 32704, 2688, 12832, 49632, 0]],
        '冦' : [12, 13, 0, -2, 12, [0, 65504, 32800, 33056, 63936, 256, 2016, 64064, 21376, 21600, 36896, 36832, 0]],
        '冢' : [12, 13, 0, -2, 12, [0, 65504, 32800, 49056, 9216, 1088, 6784, 29440, 5760, 6720, 57888, 3072, 0]],
        '冩' : [12, 13, 0, -2, 12, [0, 65504, 34848, 45984, 8320, 15232, 8320, 8320, 32736, 32800, 10784, 19136, 0]],
        '冪' : [12, 13, 0, -2, 12, [0, 65504, 37152, 32704, 8320, 16256, 8320, 65504, 9344, 32704, 42144, 9600, 0]],
        '冫' : [12, 13, 0, -2, 12, [0, 4096, 2048, 1024, 0, 0, 0, 128, 256, 512, 3072, 12288, 0]],
        '决' : [12, 13, 0, -2, 12, [256, 256, 36832, 16672, 288, 288, 8160, 256, 16640, 17024, 33856, 38944, 0]],
        '冱' : [12, 13, 0, -2, 12, [0, 8160, 33280, 16896, 9152, 1088, 1088, 1984, 17472, 16448, 32832, 40928, 0]],
        '冲' : [12, 13, 0, -2, 12, [256, 256, 33024, 20448, 2336, 2336, 2336, 4064, 16640, 16640, 33024, 33024, 0]],
        '冰' : [12, 13, 0, -2, 12, [256, 256, 33024, 23840, 1344, 1408, 1408, 2368, 18752, 20768, 37152, 33536, 0]],
        '况' : [12, 13, 0, -2, 12, [0, 4064, 34848, 18464, 2080, 4064, 640, 640, 17024, 17056, 33952, 39136, 0]],
        '冽' : [12, 13, 0, -2, 12, [0, 16160, 34976, 18592, 3744, 4768, 4768, 2720, 17568, 17440, 34848, 45152, 0]],
        '凅' : [12, 13, 0, -2, 12, [0, 4064, 35104, 18720, 4064, 2336, 2976, 2720, 19104, 19360, 34848, 36832, 0]],
        '凉' : [12, 13, 0, -2, 12, [256, 256, 40928, 16384, 4032, 2112, 2112, 4032, 16640, 18752, 37152, 33536, 0]],
        '凛' : [12, 13, 0, -2, 12, [256, 256, 40928, 19104, 2976, 2080, 4064, 0, 24544, 16640, 35136, 37664, 0]],
        '几' : [12, 13, 0, -2, 12, [0, 16128, 8448, 8448, 8448, 8448, 8448, 8448, 16640, 16672, 33056, 33248, 0]],
        '處' : [12, 13, 0, -2, 12, [3968, 2048, 32736, 18464, 32640, 18464, 20448, 20480, 24000, 46432, 35840, 13280, 0]],
        '凩' : [12, 13, 0, -2, 12, [0, 32704, 17472, 17472, 32704, 17472, 19520, 20032, 21824, 21664, 33824, 33824, 0]],
        '凭' : [12, 13, 0, -2, 12, [0, 12256, 8448, 24544, 49408, 16640, 20448, 0, 7936, 4352, 4384, 57824, 0]],
        '凰' : [12, 13, 0, -2, 12, [0, 32704, 17472, 24384, 20800, 24384, 20800, 24384, 17472, 24352, 33824, 49056, 0]],
        '凵' : [12, 13, 0, -2, 12, [0, 32800, 32800, 32800, 32800, 32800, 32800, 32800, 32800, 32800, 32800, 65504, 0]],
        '凾' : [12, 13, 0, -2, 12, [0, 32704, 512, 33824, 34784, 62496, 38176, 38240, 62880, 34400, 34848, 65504, 0]],
        '刄' : [12, 13, 0, -2, 12, [0, 65472, 2112, 18496, 18496, 43072, 36928, 7232, 9184, 8256, 16512, 17152, 0]],
        '刋' : [12, 13, 0, -2, 12, [0, 3104, 61728, 4384, 4384, 4384, 65312, 4384, 4384, 8224, 8224, 16480, 0]],
        '刔' : [12, 13, 0, -2, 12, [8192, 8224, 64800, 9504, 9504, 9504, 65312, 8480, 8480, 20512, 34848, 33888, 0]],
        '刎' : [12, 13, 0, -2, 12, [16384, 16416, 32032, 54560, 21792, 21792, 21792, 38176, 42272, 9248, 17440, 22624, 0]],
        '刧' : [12, 13, 0, -2, 12, [8192, 10208, 63776, 8480, 8480, 8480, 63776, 16928, 16928, 34848, 34880, 63872, 0]],
        '刪' : [12, 13, 0, -2, 12, [0, 31776, 21792, 21792, 21792, 21792, 65312, 21792, 21792, 21536, 21536, 21600, 0]],
        '刮' : [12, 13, 0, -2, 12, [0, 6176, 57632, 8480, 8480, 64800, 8480, 8480, 63776, 34848, 34848, 63584, 0]],
        '刳' : [12, 13, 0, -2, 12, [4096, 4128, 64800, 18720, 34080, 64800, 288, 64800, 16672, 64544, 33824, 6240, 0]],
        '刹' : [12, 13, 0, -2, 12, [0, 17440, 10528, 4384, 10528, 50464, 4384, 64800, 12576, 22560, 37920, 4192, 0]],
        '剏' : [12, 13, 0, -2, 12, [34816, 20448, 4384, 63776, 21280, 20768, 20896, 64096, 21024, 21536, 37952, 37248, 0]],
        '剄' : [12, 13, 0, -2, 12, [0, 64544, 21792, 21792, 43296, 21792, 21792, 288, 64800, 8224, 8224, 64608, 0]],
        '剋' : [12, 13, 0, -2, 12, [8192, 8480, 64800, 8480, 63776, 35104, 35104, 63520, 20576, 20480, 36896, 36832, 0]],
        '剌' : [12, 13, 0, -2, 12, [8192, 8224, 64800, 8480, 64800, 42272, 42272, 64800, 12576, 26656, 42016, 8288, 0]],
        '剞' : [12, 13, 0, -2, 12, [4096, 4128, 64800, 4384, 10528, 17696, 65312, 1312, 58656, 42016, 58400, 3168, 0]],
        '剔' : [12, 13, 0, -2, 12, [0, 63520, 35104, 63776, 35104, 63776, 16672, 64800, 21792, 37920, 9248, 22624, 0]],
        '剪' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 19008, 27200, 23104, 18624, 0, 65504, 2080, 12320, 49344, 0]],
        '剴' : [12, 13, 0, -2, 12, [8192, 42016, 64800, 288, 64800, 288, 64800, 34080, 64800, 18464, 18464, 64608, 0]],
        '剩' : [12, 13, 0, -2, 12, [0, 65056, 4256, 65184, 21664, 54944, 21664, 54944, 6304, 13344, 53792, 4192, 0]],
        '剳' : [12, 13, 0, -2, 12, [18432, 18464, 64800, 18720, 12576, 18720, 34080, 63776, 288, 64544, 33824, 64608, 0]],
        '剿' : [12, 13, 0, -2, 12, [21504, 43040, 21792, 64800, 42272, 64800, 42272, 64800, 8480, 26656, 42016, 8288, 0]],
        '剽' : [12, 13, 0, -2, 12, [0, 65056, 10400, 65184, 43680, 65184, 160, 65184, 160, 65056, 21536, 45664, 0]],
        '劍' : [12, 13, 0, -2, 12, [0, 31776, 33440, 64672, 160, 61088, 43680, 43680, 61088, 17440, 43552, 37216, 0]],
        '劔' : [12, 13, 0, -2, 12, [0, 30688, 35104, 63776, 288, 64288, 43424, 43360, 64032, 21024, 43072, 42368, 0]],
        '劒' : [12, 13, 0, -2, 12, [0, 30688, 35104, 63776, 800, 64800, 43296, 43296, 64032, 21024, 43072, 42368, 0]],
        '剱' : [12, 13, 0, -2, 12, [0, 30688, 35104, 63776, 8992, 63776, 43424, 43360, 64032, 8736, 20544, 36224, 0]],
        '劈' : [12, 13, 0, -2, 12, [256, 61696, 38880, 62016, 36832, 61696, 38880, 37120, 65504, 2080, 12320, 49344, 0]],
        '劑' : [12, 13, 0, -2, 12, [2048, 2080, 65440, 21792, 23968, 46752, 17056, 32416, 17056, 32288, 33312, 33376, 0]],
        '辨' : [12, 13, 0, -2, 12, [8320, 8832, 64480, 19008, 19008, 64480, 10880, 8832, 64480, 8832, 17536, 18560, 0]],
        '辧' : [12, 13, 0, -2, 12, [8320, 16000, 60384, 19008, 19008, 60384, 10880, 10880, 62432, 21120, 17024, 19584, 0]],
        '劬' : [12, 13, 0, -2, 12, [16640, 16640, 30976, 36832, 35104, 59680, 43296, 43296, 59936, 2592, 4160, 24960, 0]],
        '劭' : [12, 13, 0, -2, 12, [256, 63744, 18688, 20448, 35104, 45344, 288, 63776, 35360, 35360, 34880, 63872, 0]],
        '劼' : [12, 13, 0, -2, 12, [8448, 8448, 63744, 10208, 8480, 63776, 288, 288, 64032, 35360, 34880, 63872, 0]],
        '劵' : [12, 13, 0, -2, 12, [1024, 9344, 17472, 65504, 2304, 65504, 4224, 16320, 51360, 2176, 4224, 25344, 0]],
        '勁' : [12, 13, 0, -2, 12, [256, 64768, 20736, 22496, 43296, 21792, 21792, 288, 64032, 8736, 8256, 63872, 0]],
        '勍' : [12, 13, 0, -2, 12, [4352, 4352, 63744, 2016, 31008, 18720, 18720, 31264, 4640, 21536, 37952, 12672, 0]],
        '勗' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 16256, 256, 65504, 37152, 61728, 37408, 64704, 0]],
        '勞' : [12, 13, 0, -2, 12, [4352, 21792, 38208, 10880, 17472, 65504, 33824, 32704, 2112, 2112, 4160, 25472, 0]],
        '勣' : [12, 13, 0, -2, 12, [8448, 8448, 63744, 10208, 63776, 8480, 63776, 35360, 64032, 34848, 63552, 36224, 0]],
        '勦' : [12, 13, 0, -2, 12, [21760, 43264, 20736, 65504, 43296, 63776, 43296, 64032, 8736, 26656, 42048, 8576, 0]],
        '飭' : [12, 13, 0, -2, 12, [512, 29184, 35808, 62720, 1280, 64480, 35104, 63776, 33056, 64032, 33312, 63680, 0]],
        '勠' : [12, 13, 0, -2, 12, [256, 64768, 42240, 28640, 46368, 9504, 14624, 50464, 6688, 25120, 6208, 57728, 0]],
        '勳' : [12, 13, 0, -2, 12, [256, 63744, 8448, 65504, 43296, 63776, 8480, 63776, 8736, 64032, 20544, 43392, 0]],
        '勵' : [12, 13, 0, -2, 12, [256, 64768, 35072, 65504, 38176, 64800, 38176, 65056, 37408, 64544, 42048, 46464, 0]],
        '勸' : [12, 13, 0, -2, 12, [20736, 20736, 63744, 45024, 43296, 63776, 20768, 63776, 37408, 64032, 36928, 63872, 0]],
        '勹' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16416, 16416, 32800, 32800, 32, 32, 32, 64, 896, 0]],
        '匆' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 18720, 18720, 35104, 45344, 7712, 9120, 17440, 2112, 384, 0]],
        '匈' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16416, 25120, 38176, 35104, 21792, 25376, 16672, 32544, 192, 0]],
        '甸' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16416, 32544, 43296, 48928, 10528, 10528, 16160, 64, 384, 0]],
        '匍' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 18976, 32544, 34848, 65312, 18720, 32544, 18720, 19232, 192, 0]],
        '匐' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16416, 32544, 41504, 65312, 18720, 32544, 18720, 32544, 192, 0]],
        '匏' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 21536, 35744, 64160, 672, 64416, 16992, 64000, 35360, 12768, 0]],
        '匕' : [12, 13, 0, -2, 12, [4096, 4096, 4096, 4192, 4992, 7168, 61440, 4096, 4128, 4128, 4128, 4064, 0]],
        '匚' : [12, 13, 0, -2, 12, [0, 65504, 32768, 32768, 32768, 32768, 32768, 32768, 32768, 32768, 32768, 65504, 0]],
        '匣' : [12, 13, 0, -2, 12, [0, 65504, 32768, 40896, 37440, 40896, 37440, 40896, 33280, 33280, 33280, 65504, 0]],
        '匯' : [12, 13, 0, -2, 12, [0, 65504, 33280, 41600, 38848, 44288, 38848, 34048, 38144, 42944, 32768, 65504, 0]],
        '匱' : [12, 13, 0, -2, 12, [0, 65504, 33280, 40832, 37504, 49088, 36992, 40832, 36992, 40832, 41024, 65504, 0]],
        '匳' : [12, 13, 0, -2, 12, [0, 65504, 35072, 40832, 41024, 56800, 38208, 40384, 34944, 38208, 41504, 65504, 0]],
        '匸' : [12, 13, 0, -2, 12, [0, 65504, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8192, 8160, 0]],
        '區' : [12, 13, 0, -2, 12, [0, 65504, 32768, 36608, 35072, 36608, 32768, 48576, 42304, 48576, 32768, 65504, 0]],
        '卆' : [12, 13, 0, -2, 12, [2048, 2048, 65280, 4352, 4352, 8480, 50400, 1024, 65504, 1024, 1024, 1024, 0]],
        '卅' : [12, 13, 0, -2, 12, [0, 17472, 17472, 17472, 17472, 65504, 17472, 17472, 17472, 17472, 33856, 33856, 0]],
        '丗' : [12, 13, 0, -2, 12, [0, 17472, 17472, 17472, 17472, 65504, 17472, 17472, 17472, 17472, 17472, 32704, 0]],
        '卉' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 5248, 4224, 4224, 65504, 4224, 4224, 8320, 49280, 0]],
        '卍' : [12, 13, 0, -2, 12, [0, 64544, 1056, 1056, 1056, 1056, 65504, 33792, 33792, 33792, 33792, 34784, 0]],
        '凖' : [12, 13, 0, -2, 12, [1152, 1152, 36832, 22784, 4032, 2304, 18688, 36832, 1024, 65504, 1024, 1024, 0]],
        '卞' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 1024, 1792, 1216, 1024, 1024, 1024, 1024, 1024, 0]],
        '卩' : [12, 13, 0, -2, 12, [0, 16256, 8320, 8320, 8320, 8320, 8320, 8320, 8320, 10112, 8192, 8192, 0]],
        '卮' : [12, 13, 0, -2, 12, [0, 448, 32256, 16384, 32736, 16384, 20352, 18560, 18560, 35200, 34848, 2016, 0]],
        '夘' : [12, 13, 0, -2, 12, [16384, 17376, 31264, 18976, 18976, 43552, 39456, 2592, 4640, 4832, 8704, 49664, 0]],
        '卻' : [12, 13, 0, -2, 12, [0, 29152, 35104, 1312, 288, 28960, 35104, 1312, 63776, 35168, 35072, 63744, 0]],
        '卷' : [12, 13, 0, -2, 12, [1024, 9344, 9344, 24384, 35104, 65504, 4224, 16192, 57632, 8960, 8256, 16320, 0]],
        '厂' : [12, 13, 0, -2, 12, [0, 32736, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 32768, 32768, 0]],
        '厖' : [12, 13, 0, -2, 12, [0, 32736, 18464, 19136, 32256, 18528, 18816, 19456, 21600, 38272, 42016, 9184, 0]],
        '厠' : [12, 13, 0, -2, 12, [0, 32736, 16384, 24096, 21152, 24224, 21152, 21152, 24224, 32800, 37408, 8544, 0]],
        '厦' : [12, 13, 0, -2, 12, [0, 32736, 16896, 24544, 20512, 24544, 20512, 24544, 18496, 46144, 34688, 14432, 0]],
        '厥' : [12, 13, 0, -2, 12, [0, 32736, 17664, 18688, 32224, 21152, 21152, 21632, 32064, 37184, 41504, 9248, 0]],
        '厮' : [12, 13, 0, -2, 12, [0, 65504, 42048, 65408, 42240, 48608, 42304, 42304, 65344, 33088, 42048, 16960, 0]],
        '厰' : [12, 13, 0, -2, 12, [0, 32736, 20736, 23008, 21792, 20768, 32416, 17056, 31296, 43584, 47776, 1824, 0]],
        '厶' : [12, 13, 0, -2, 12, [0, 2048, 2048, 2048, 2048, 4224, 4224, 4160, 8256, 8608, 65056, 32, 0]],
        '參' : [12, 13, 0, -2, 12, [2048, 2304, 4224, 32704, 20800, 64480, 4736, 11328, 49952, 7296, 768, 15360, 0]],
        '簒' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 16256, 8320, 16256, 8320, 65504, 21568, 43168, 24448, 0]],
        '雙' : [12, 13, 0, -2, 12, [8448, 10560, 32736, 51776, 32736, 19008, 65504, 0, 32704, 4224, 3840, 61664, 0]],
        '叟' : [12, 13, 0, -2, 12, [2048, 4096, 26048, 17472, 30144, 17472, 32704, 1024, 65504, 8320, 7936, 57568, 0]],
        '曼' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 65504, 37152, 65504, 8320, 4352, 7936, 57568, 0]],
        '燮' : [12, 13, 0, -2, 12, [8320, 11904, 45472, 44704, 8320, 24384, 35360, 2560, 65504, 8320, 7936, 57568, 0]],
        '叮' : [12, 13, 0, -2, 12, [0, 4064, 61504, 36928, 36928, 36928, 36928, 36928, 36928, 36928, 61504, 448, 0]],
        '叨' : [12, 13, 0, -2, 12, [0, 4064, 61728, 37152, 37152, 37152, 37408, 37408, 37920, 37920, 61504, 384, 0]],
        '叭' : [12, 13, 0, -2, 12, [0, 960, 61504, 36928, 37440, 37440, 37440, 37440, 37920, 37920, 63520, 2080, 0]],
        '叺' : [12, 13, 0, -2, 12, [0, 1920, 61568, 36992, 36992, 36992, 37184, 37184, 37440, 37408, 62496, 1056, 0]],
        '吁' : [12, 13, 0, -2, 12, [0, 2016, 61568, 36992, 36992, 36992, 40928, 36992, 36992, 36992, 61568, 896, 0]],
        '吽' : [12, 13, 0, -2, 12, [128, 2176, 59520, 45024, 43136, 43136, 41088, 49120, 41088, 41088, 57472, 128, 0]],
        '呀' : [12, 13, 0, -2, 12, [0, 2016, 61504, 37952, 37952, 37952, 38880, 37184, 37184, 62016, 1088, 6336, 0]],
        '听' : [12, 13, 0, -2, 12, [0, 96, 63360, 37888, 37888, 38880, 37952, 37952, 37952, 37952, 63552, 2112, 0]],
        '吭' : [12, 13, 0, -2, 12, [256, 256, 65504, 36864, 36864, 38848, 37952, 37952, 37952, 37952, 63552, 2144, 0]],
        '吼' : [12, 13, 0, -2, 12, [0, 8000, 57664, 41536, 42048, 42048, 42816, 48192, 42048, 42048, 58432, 3168, 0]],
        '吮' : [12, 13, 0, -2, 12, [512, 512, 58432, 42048, 49056, 40992, 42112, 42112, 42112, 58528, 2208, 12512, 0]],
        '吶' : [12, 13, 0, -2, 12, [1792, 256, 61408, 43296, 43296, 43296, 43680, 44128, 43040, 43040, 59424, 2144, 0]],
        '吩' : [12, 13, 0, -2, 12, [0, 1472, 58432, 43072, 43040, 40992, 45024, 42048, 42048, 42048, 59456, 2432, 0]],
        '吝' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 8320, 4352, 3584, 12672, 49248, 16256, 8320, 8320, 16256, 0]],
        '呎' : [12, 13, 0, -2, 12, [0, 2016, 62496, 37920, 37920, 38880, 38016, 38016, 37952, 37952, 63520, 2080, 0]],
        '咏' : [12, 13, 0, -2, 12, [256, 128, 61440, 38784, 37024, 40640, 37504, 37504, 38080, 38048, 63648, 2432, 0]],
        '呵' : [12, 13, 0, -2, 12, [0, 8160, 57408, 41024, 44608, 43584, 43584, 43584, 44608, 41024, 57408, 448, 0]],
        '咎' : [12, 13, 0, -2, 12, [256, 30976, 18688, 19072, 46144, 38944, 9984, 49376, 16256, 8320, 8320, 16256, 0]],
        '呟' : [12, 13, 0, -2, 12, [256, 256, 63456, 36992, 37120, 37408, 37952, 37504, 37120, 37408, 62496, 2016, 0]],
        '呱' : [12, 13, 0, -2, 12, [0, 4064, 60032, 43648, 43648, 43648, 43584, 43584, 43584, 43680, 63392, 4128, 0]],
        '呷' : [12, 13, 0, -2, 12, [0, 4064, 59680, 43296, 45024, 43296, 43296, 45024, 41216, 41216, 57600, 256, 0]],
        '呰' : [12, 13, 0, -2, 12, [512, 8704, 47712, 41856, 41472, 48672, 57824, 0, 16256, 8320, 8320, 16256, 0]],
        '咒' : [12, 13, 0, -2, 12, [0, 31680, 19008, 19008, 31680, 0, 7936, 4352, 4352, 4384, 8480, 49376, 0]],
        '呻' : [12, 13, 0, -2, 12, [256, 256, 61408, 43296, 43296, 45024, 43296, 43296, 45024, 41216, 57600, 256, 0]],
        '咀' : [12, 13, 0, -2, 12, [0, 992, 61984, 37408, 37856, 37408, 37408, 37856, 37408, 37408, 61984, 2016, 0]],
        '呶' : [12, 13, 0, -2, 12, [2048, 2528, 59424, 48800, 43680, 43680, 43680, 43680, 48192, 42816, 59552, 2336, 0]],
        '咄' : [12, 13, 0, -2, 12, [256, 256, 59680, 43296, 43296, 45024, 41216, 41216, 43296, 43296, 59680, 4064, 0]],
        '咐' : [12, 13, 0, -2, 12, [64, 1088, 58432, 44000, 47168, 43072, 43584, 43328, 43072, 43072, 59456, 2240, 0]],
        '咆' : [12, 13, 0, -2, 12, [1024, 1024, 63456, 38944, 38816, 38048, 38048, 38816, 37984, 37888, 62496, 992, 0]],
        '哇' : [12, 13, 0, -2, 12, [256, 256, 63456, 37120, 37120, 40928, 37120, 37120, 38880, 37120, 61696, 4064, 0]],
        '咢' : [12, 13, 0, -2, 12, [0, 31680, 19008, 31680, 0, 32704, 0, 65504, 4096, 16320, 8256, 384, 0]],
        '咸' : [12, 13, 0, -2, 12, [128, 160, 32736, 16512, 32384, 16512, 23712, 21664, 21568, 23616, 33184, 34336, 0]],
        '咥' : [12, 13, 0, -2, 12, [0, 4064, 61696, 37376, 37920, 38880, 36992, 36992, 38880, 36992, 61568, 4064, 0]],
        '咬' : [12, 13, 0, -2, 12, [256, 256, 65504, 36864, 37952, 38944, 36864, 37952, 37504, 37248, 62016, 3104, 0]],
        '哄' : [12, 13, 0, -2, 12, [1152, 1152, 58496, 49120, 42112, 42112, 42112, 49120, 40960, 42112, 59456, 4128, 0]],
        '哈' : [12, 13, 0, -2, 12, [0, 960, 62496, 38912, 36864, 38880, 36864, 36864, 38880, 37920, 62496, 2016, 0]],
        '咨' : [12, 13, 0, -2, 12, [1024, 1024, 59360, 2336, 2368, 8960, 17536, 34912, 16256, 8320, 8320, 16256, 0]],
        '咫' : [12, 13, 0, -2, 12, [0, 32224, 17696, 17696, 17696, 32224, 20480, 20800, 18976, 18432, 34304, 33248, 0]],
        '哂' : [12, 13, 0, -2, 12, [0, 4064, 57984, 41600, 45024, 43680, 43680, 43680, 43744, 44064, 59424, 4064, 0]],
        '咤' : [12, 13, 0, -2, 12, [512, 512, 65504, 36896, 37280, 40448, 37376, 37856, 40448, 37376, 61984, 992, 0]],
        '咾' : [12, 13, 0, -2, 12, [512, 512, 65504, 37440, 37504, 40928, 37376, 40128, 38656, 37888, 62496, 2016, 0]],
        '咼' : [12, 13, 0, -2, 12, [0, 16256, 8320, 10112, 9344, 9344, 65504, 32800, 40736, 37152, 40736, 32864, 0]],
        '哘' : [12, 13, 0, -2, 12, [0, 1504, 59392, 45056, 41472, 42464, 44096, 46144, 42048, 42048, 58432, 1216, 0]],
        '哥' : [12, 13, 0, -2, 12, [0, 65504, 9344, 9344, 15488, 128, 65504, 128, 15488, 9344, 15488, 384, 0]],
        '哦' : [12, 13, 0, -2, 12, [128, 7840, 58528, 42112, 42976, 48256, 42144, 42656, 48192, 42048, 58784, 3616, 0]],
        '唏' : [12, 13, 0, -2, 12, [0, 1632, 61824, 38464, 37120, 40928, 37376, 38880, 40224, 38176, 62816, 256, 0]],
        '唔' : [12, 13, 0, -2, 12, [0, 4064, 61696, 38880, 37408, 37408, 40928, 36864, 38880, 37920, 62496, 2016, 0]],
        '哽' : [12, 13, 0, -2, 12, [0, 8160, 57600, 45024, 43296, 45024, 43296, 45024, 42240, 41728, 58496, 6240, 0]],
        '哮' : [12, 13, 0, -2, 12, [256, 256, 63456, 37152, 37184, 40928, 37376, 38336, 38976, 38880, 61568, 896, 0]],
        '哭' : [12, 13, 0, -2, 12, [0, 31680, 19008, 19008, 31680, 1152, 1088, 65504, 1024, 2560, 12672, 49248, 0]],
        '哺' : [12, 13, 0, -2, 12, [256, 288, 61408, 41216, 45024, 43296, 45024, 43296, 45024, 43296, 59680, 2400, 0]],
        '哢' : [12, 13, 0, -2, 12, [0, 4064, 61696, 38880, 37120, 40928, 37440, 37440, 40928, 37440, 62528, 1088, 0]],
        '唹' : [12, 13, 0, -2, 12, [2048, 2240, 59680, 48640, 43008, 43392, 44640, 45568, 41472, 41472, 58752, 6240, 0]],
        '啀' : [12, 13, 0, -2, 12, [0, 4064, 59520, 44000, 43136, 43136, 45024, 43136, 44000, 43136, 61568, 6112, 0]],
        '啣' : [12, 13, 0, -2, 12, [2048, 2272, 61344, 46240, 42144, 49056, 42144, 42656, 38048, 38112, 63360, 6272, 0]],
        '啌' : [12, 13, 0, -2, 12, [256, 256, 65504, 37536, 37504, 37536, 40160, 36864, 38848, 37120, 61696, 4064, 0]],
        '售' : [12, 13, 0, -2, 12, [4352, 4352, 16352, 25088, 49088, 8704, 16320, 8704, 16352, 8256, 8256, 16320, 0]],
        '啜' : [12, 13, 0, -2, 12, [0, 3808, 57888, 43680, 42048, 43680, 40960, 44768, 41504, 43680, 58432, 2720, 0]],
        '啅' : [12, 13, 0, -2, 12, [256, 256, 61920, 37120, 38880, 37920, 38880, 37920, 38880, 37120, 65504, 256, 0]],
        '啖' : [12, 13, 0, -2, 12, [0, 2336, 59712, 41216, 41600, 44128, 41216, 43296, 43328, 41728, 58496, 6240, 0]],
        '啗' : [12, 13, 0, -2, 12, [0, 960, 62528, 39040, 37120, 38496, 37920, 37920, 38624, 37920, 62496, 2016, 0]],
        '唸' : [12, 13, 0, -2, 12, [0, 1920, 59456, 45088, 44992, 40960, 44992, 41024, 41600, 43264, 59456, 10144, 0]],
        '唳' : [12, 13, 0, -2, 12, [0, 4064, 57344, 45024, 43040, 45024, 43264, 45024, 43264, 43392, 57920, 3104, 0]],
        '啝' : [12, 13, 0, -2, 12, [0, 992, 64672, 42144, 42144, 49056, 42144, 42144, 42656, 44512, 62592, 9312, 0]],
        '喙' : [12, 13, 0, -2, 12, [0, 2016, 58432, 44992, 43136, 45024, 41600, 44192, 41408, 42656, 63648, 768, 0]],
        '喀' : [12, 13, 0, -2, 12, [512, 512, 65504, 42144, 42144, 41728, 42112, 47200, 44992, 43072, 59456, 4032, 0]],
        '咯' : [12, 13, 0, -2, 12, [1024, 1024, 59328, 43072, 46208, 41728, 42112, 47200, 44992, 43072, 59456, 4032, 0]],
        '喊' : [12, 13, 0, -2, 12, [128, 160, 61408, 43136, 44928, 43168, 44704, 43680, 44608, 43072, 61856, 5664, 0]],
        '喟' : [12, 13, 0, -2, 12, [0, 4064, 59680, 45024, 43296, 45024, 42048, 42944, 42048, 42944, 58432, 1216, 0]],
        '啻' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 65504, 33824, 49056, 9344, 1024, 32704, 16448, 32704, 0]],
        '啾' : [12, 13, 0, -2, 12, [128, 896, 64640, 42144, 42144, 49056, 42112, 42176, 44352, 46400, 58912, 1056, 0]],
        '喘' : [12, 13, 0, -2, 12, [256, 2336, 59680, 45024, 40960, 45024, 41216, 45024, 43680, 43680, 60064, 2144, 0]],
        '喞' : [12, 13, 0, -2, 12, [1024, 2272, 65184, 37536, 40608, 37536, 40608, 37024, 40608, 37024, 65248, 4224, 0]],
        '單' : [12, 13, 0, -2, 12, [0, 31680, 19008, 19008, 32704, 9344, 16256, 9344, 16256, 1024, 65504, 1024, 0]],
        '啼' : [12, 13, 0, -2, 12, [256, 256, 65504, 42112, 42112, 49120, 41248, 45024, 43296, 43296, 59744, 256, 0]],
        '喃' : [12, 13, 0, -2, 12, [256, 256, 61408, 41216, 45024, 43680, 43680, 45024, 43296, 45024, 59680, 2400, 0]],
        '喩' : [12, 13, 0, -2, 12, [0, 8064, 58432, 43040, 47072, 40960, 44704, 43680, 44864, 43840, 61088, 2720, 0]],
        '喇' : [12, 13, 0, -2, 12, [2048, 2080, 65440, 43168, 48800, 43680, 43680, 48800, 43168, 48160, 60192, 2144, 0]],
        '喨' : [12, 13, 0, -2, 12, [256, 256, 65504, 38016, 38784, 36864, 40928, 36896, 37504, 37504, 62624, 6368, 0]],
        '嗚' : [12, 13, 0, -2, 12, [256, 512, 63424, 37952, 38848, 37888, 38880, 37888, 38880, 36896, 62752, 5472, 0]],
        '嗅' : [12, 13, 0, -2, 12, [256, 512, 61408, 43040, 45024, 43040, 45024, 41280, 49120, 41216, 57984, 3168, 0]],
        '嗟' : [12, 13, 0, -2, 12, [1088, 1152, 65504, 37120, 38880, 37120, 40928, 37376, 38880, 39040, 61568, 4064, 0]],
        '嗄' : [12, 13, 0, -2, 12, [0, 4064, 61696, 38880, 37920, 38880, 37920, 38880, 37952, 39488, 61824, 3680, 0]],
        '嗜' : [12, 13, 0, -2, 12, [256, 288, 61408, 41280, 49120, 42560, 43648, 47072, 42016, 42976, 58400, 2016, 0]],
        '嗤' : [12, 13, 0, -2, 12, [256, 2336, 61408, 41472, 49120, 41216, 45024, 43296, 45024, 41216, 57632, 4064, 0]],
        '嗔' : [12, 13, 0, -2, 12, [1024, 1120, 59264, 42016, 41952, 43584, 43968, 43584, 45024, 40960, 57920, 3104, 0]],
        '嘔' : [12, 13, 0, -2, 12, [0, 4064, 59392, 43968, 43584, 43968, 43008, 44768, 43680, 44768, 59392, 4064, 0]],
        '嗷' : [12, 13, 0, -2, 12, [1152, 1152, 65504, 42144, 49056, 42016, 49056, 43168, 44608, 45632, 58016, 3360, 0]],
        '嘖' : [12, 13, 0, -2, 12, [256, 256, 65504, 41216, 45024, 41216, 49120, 42048, 42944, 42048, 59328, 6176, 0]],
        '嗾' : [12, 13, 0, -2, 12, [1152, 1152, 65504, 43264, 43488, 44864, 43584, 44000, 45632, 41536, 58016, 3360, 0]],
        '嗽' : [12, 13, 0, -2, 12, [2304, 2304, 65504, 43168, 48800, 43680, 48768, 43136, 47424, 44352, 59936, 3104, 0]],
        '嘛' : [12, 13, 0, -2, 12, [512, 512, 65504, 42112, 42112, 49120, 42112, 42144, 44448, 46752, 50336, 17536, 0]],
        '嗹' : [12, 13, 0, -2, 12, [128, 4224, 60384, 41088, 46048, 43680, 41952, 47776, 45024, 43136, 60416, 13280, 0]],
        '噎' : [12, 13, 0, -2, 12, [512, 512, 65504, 41472, 49120, 40992, 44992, 43072, 44992, 42112, 58496, 8160, 0]],
        '噐' : [12, 13, 0, -2, 12, [0, 31680, 19008, 31680, 0, 32704, 1024, 65504, 0, 31680, 19008, 31680, 0]],
        '營' : [12, 13, 0, -2, 12, [4352, 21792, 38208, 10880, 17472, 65504, 37152, 40736, 1024, 16256, 8320, 16256, 0]],
        '嘴' : [12, 13, 0, -2, 12, [640, 2688, 60384, 43648, 43680, 49120, 43136, 49120, 43296, 45024, 59424, 12384, 0]],
        '嘶' : [12, 13, 0, -2, 12, [2560, 2784, 65408, 43648, 44768, 43680, 44704, 43680, 49056, 41120, 59936, 4384, 0]],
        '嘲' : [12, 13, 0, -2, 12, [2048, 2528, 65312, 43296, 48608, 42272, 48416, 42464, 48416, 43296, 65056, 2656, 0]],
        '嘸' : [12, 13, 0, -2, 12, [1024, 1024, 61408, 47776, 43680, 49120, 43680, 43680, 49120, 40960, 60064, 4768, 0]],
        '噫' : [12, 13, 0, -2, 12, [256, 256, 61408, 41536, 45024, 43040, 45024, 43040, 45024, 42240, 62528, 5024, 0]],
        '噤' : [12, 13, 0, -2, 12, [1152, 1152, 65504, 42368, 44736, 46240, 44992, 40960, 49120, 41216, 59712, 4896, 0]],
        '嘯' : [12, 13, 0, -2, 12, [512, 512, 61376, 41536, 49120, 41536, 49088, 43680, 47840, 42784, 64224, 10912, 0]],
        '噬' : [12, 13, 0, -2, 12, [2176, 2176, 61152, 46400, 42048, 40960, 49120, 42304, 42304, 43936, 57600, 8160, 0]],
        '噪' : [12, 13, 0, -2, 12, [0, 1984, 58432, 45024, 43680, 43680, 44768, 41216, 45024, 41728, 58688, 6432, 0]],
        '嚆' : [12, 13, 0, -2, 12, [1152, 1152, 65504, 42112, 41472, 49120, 42112, 42112, 49120, 43168, 61344, 8288, 0]],
        '嚀' : [12, 13, 0, -2, 12, [512, 512, 65504, 42272, 46144, 42912, 40960, 45024, 43680, 49120, 57600, 1792, 0]],
        '嚊' : [12, 13, 0, -2, 12, [256, 1984, 58432, 42944, 42048, 45024, 43296, 45024, 43296, 49120, 57920, 3136, 0]],
        '嚠' : [12, 13, 0, -2, 12, [0, 14240, 58016, 43680, 48544, 41120, 44704, 45472, 44704, 42016, 62752, 8032, 0]],
        '嚔' : [12, 13, 0, -2, 12, [512, 4032, 57856, 49120, 43680, 44928, 42048, 49120, 41472, 43968, 60928, 12768, 0]],
        '嚏' : [12, 13, 0, -2, 12, [512, 4032, 57856, 49120, 43680, 44928, 43648, 49120, 41504, 43904, 60928, 12768, 0]],
        '嚥' : [12, 13, 0, -2, 12, [2304, 2304, 65504, 43264, 44864, 41056, 44864, 43328, 44896, 40960, 62784, 5408, 0]],
        '嚮' : [12, 13, 0, -2, 12, [0, 20192, 43680, 19136, 44704, 59552, 20192, 34944, 65504, 37152, 40736, 32864, 0]],
        '嚶' : [12, 13, 0, -2, 12, [0, 3808, 60064, 44768, 43680, 44768, 43328, 45600, 49120, 42048, 59264, 6240, 0]],
        '嚴' : [12, 13, 0, -2, 12, [0, 32224, 17696, 32736, 18560, 32480, 21280, 24224, 21152, 24224, 45760, 33568, 0]],
        '囂' : [12, 13, 0, -2, 12, [0, 31680, 19008, 65504, 8320, 16256, 8320, 16256, 16448, 64480, 19008, 31680, 0]],
        '嚼' : [12, 13, 0, -2, 12, [0, 4064, 58688, 49120, 42144, 49120, 43552, 44768, 43296, 44704, 59424, 3680, 0]],
        '囁' : [12, 13, 0, -2, 12, [0, 4064, 58432, 42944, 42048, 49120, 43680, 44768, 43680, 44768, 64416, 544, 0]],
        '囃' : [12, 13, 0, -2, 12, [1280, 1344, 65344, 43488, 43840, 46560, 42304, 48960, 42464, 44864, 62784, 9696, 0]],
        '囀' : [12, 13, 0, -2, 12, [2176, 2176, 65504, 43136, 49120, 43680, 49120, 43680, 49120, 43296, 65184, 2144, 0]],
        '囈' : [12, 13, 0, -2, 12, [2176, 2176, 65504, 43264, 49088, 46400, 43456, 48480, 43520, 49120, 59456, 16288, 0]],
        '囎' : [12, 13, 0, -2, 12, [0, 15808, 58912, 49120, 42272, 48992, 42272, 42976, 48672, 41952, 59936, 18400, 0]],
        '囑' : [12, 13, 0, -2, 12, [0, 16352, 57376, 49120, 43328, 49120, 42144, 49120, 43680, 44960, 49824, 24416, 0]],
        '囓' : [12, 13, 0, -2, 12, [2048, 2528, 65184, 43296, 48672, 43968, 43520, 49120, 42272, 49120, 58656, 16352, 0]],
        '囗' : [12, 13, 0, -2, 12, [0, 65504, 32800, 32800, 32800, 32800, 32800, 32800, 32800, 32800, 32800, 65504, 0]],
        '囮' : [12, 13, 0, -2, 12, [0, 65504, 33824, 37920, 38048, 46368, 54816, 37920, 38048, 38816, 32800, 65504, 0]],
        '囹' : [12, 13, 0, -2, 12, [0, 65504, 35360, 37152, 41120, 57184, 32800, 49056, 33952, 34208, 33824, 65504, 0]],
        '圀' : [12, 13, 0, -2, 12, [0, 65504, 35360, 37536, 41888, 33824, 49056, 34848, 36640, 35104, 37152, 65504, 0]],
        '囿' : [12, 13, 0, -2, 12, [0, 65504, 34848, 49056, 34848, 40736, 45344, 57120, 37152, 37664, 32800, 65504, 0]],
        '圄' : [12, 13, 0, -2, 12, [0, 65504, 32800, 49056, 34848, 48928, 37152, 65504, 41120, 49056, 32800, 65504, 0]],
        '圉' : [12, 13, 0, -2, 12, [0, 65504, 33824, 49056, 33824, 65504, 37152, 49056, 33824, 49056, 33824, 65504, 0]],
        '圈' : [12, 13, 0, -2, 12, [0, 65504, 38176, 42144, 49056, 35360, 65504, 37152, 44704, 34848, 35104, 65504, 0]],
        '國' : [12, 13, 0, -2, 12, [0, 65504, 33440, 65504, 33312, 47776, 43680, 47392, 33056, 47776, 33952, 65504, 0]],
        '圍' : [12, 13, 0, -2, 12, [0, 65504, 33824, 48928, 35104, 65504, 37152, 48928, 42016, 49056, 33824, 65504, 0]],
        '圓' : [12, 13, 0, -2, 12, [0, 65504, 37152, 37152, 49056, 41120, 49056, 41120, 49056, 37152, 41120, 65504, 0]],
        '團' : [12, 13, 0, -2, 12, [0, 65504, 33824, 49056, 38176, 40736, 38176, 65504, 33056, 37152, 35104, 65504, 0]],
        '圖' : [12, 13, 0, -2, 12, [0, 65504, 37152, 40736, 33824, 65504, 43680, 44704, 41120, 49056, 32800, 65504, 0]],
        '嗇' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 9344, 9344, 21824, 33824, 65504, 20800, 24384, 16448, 32704, 0]],
        '圜' : [12, 13, 0, -2, 12, [0, 65504, 43680, 49056, 32800, 65504, 37152, 40736, 35360, 45728, 37152, 65504, 0]],
        '圦' : [12, 13, 0, -2, 12, [0, 10112, 8320, 8320, 61568, 8320, 8512, 8512, 8768, 12832, 50208, 6176, 0]],
        '圷' : [12, 13, 0, -2, 12, [0, 12256, 8448, 8448, 61696, 8576, 8512, 8480, 8448, 12544, 49408, 256, 0]],
        '圸' : [12, 13, 0, -2, 12, [256, 16640, 16640, 16640, 59680, 18720, 18720, 18720, 18720, 18720, 59680, 4064, 0]],
        '坎' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 17696, 59680, 18752, 16640, 16640, 17024, 17024, 58432, 6176, 0]],
        '圻' : [12, 13, 0, -2, 12, [0, 16576, 18176, 17408, 58368, 18400, 17472, 17472, 17472, 18496, 59456, 64, 0]],
        '址' : [12, 13, 0, -2, 12, [256, 16640, 16640, 18688, 59648, 18912, 18688, 18688, 18688, 18688, 59648, 8160, 0]],
        '坏' : [12, 13, 0, -2, 12, [0, 20448, 16512, 16512, 61696, 16640, 17216, 17728, 18720, 16672, 61696, 256, 0]],
        '坩' : [12, 13, 0, -2, 12, [1024, 17440, 17440, 20448, 58400, 17440, 17440, 18400, 17440, 17440, 62496, 2016, 0]],
        '埀' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 5376, 30144, 5408, 62944, 5120, 32704, 1024, 65504, 0]],
        '垈' : [12, 13, 0, -2, 12, [512, 8768, 12256, 16640, 49408, 16512, 17504, 1024, 16256, 1024, 1024, 65504, 0]],
        '坡' : [12, 13, 0, -2, 12, [256, 16640, 20448, 18720, 59648, 20416, 18496, 19008, 20800, 20608, 58176, 3104, 0]],
        '坿' : [12, 13, 0, -2, 12, [64, 17472, 17472, 19424, 63552, 18496, 19008, 18752, 18496, 18496, 59456, 2240, 0]],
        '垉' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 10272, 65440, 9376, 9376, 10144, 9312, 13312, 50208, 2016, 0]],
        '垓' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8448, 61984, 9280, 8832, 8480, 9792, 12416, 49984, 3104, 0]],
        '垠' : [12, 13, 0, -2, 12, [0, 10208, 9248, 10208, 62496, 9248, 10208, 9472, 9504, 13472, 50240, 1824, 0]],
        '垳' : [12, 13, 0, -2, 12, [0, 17888, 18432, 16384, 57856, 17888, 19520, 21568, 17472, 17472, 58432, 1216, 0]],
        '垤' : [12, 13, 0, -2, 12, [0, 12256, 8448, 8704, 62496, 10208, 8320, 8320, 10208, 12416, 49280, 4064, 0]],
        '垪' : [12, 13, 0, -2, 12, [2112, 17472, 16512, 20448, 62528, 17472, 17472, 24544, 17472, 17472, 59456, 2112, 0]],
        '垰' : [12, 13, 0, -2, 12, [256, 8448, 8640, 8448, 61696, 12256, 8192, 12256, 8448, 12672, 49472, 256, 0]],
        '埃' : [12, 13, 0, -2, 12, [512, 8768, 9280, 12256, 62464, 10208, 9344, 10368, 12256, 12416, 49984, 3104, 0]],
        '埆' : [12, 13, 0, -2, 12, [0, 18368, 18560, 24544, 59680, 20448, 18720, 18720, 20448, 18464, 61472, 4320, 0]],
        '埔' : [12, 13, 0, -2, 12, [256, 16672, 20448, 16640, 61408, 18720, 20448, 18720, 20448, 18720, 59680, 2400, 0]],
        '埒' : [12, 13, 0, -2, 12, [0, 10208, 9216, 9536, 63776, 10240, 8256, 12256, 8256, 13376, 49728, 192, 0]],
        '埓' : [12, 13, 0, -2, 12, [0, 8416, 12032, 9344, 62176, 12032, 8256, 12256, 8256, 13376, 49728, 192, 0]],
        '堊' : [12, 13, 0, -2, 12, [0, 65504, 2560, 31680, 16448, 31680, 2560, 65504, 1024, 32704, 1024, 65504, 0]],
        '埖' : [12, 13, 0, -2, 12, [1152, 17536, 24544, 17536, 57344, 17696, 17728, 18816, 22784, 18688, 59680, 2528, 0]],
        '埣' : [12, 13, 0, -2, 12, [256, 16640, 24544, 17472, 58432, 19104, 20768, 16640, 24544, 16640, 57600, 256, 0]],
        '堋' : [12, 13, 0, -2, 12, [0, 20192, 19104, 19104, 61152, 19104, 19104, 20192, 19104, 19104, 60064, 2720, 0]],
        '堙' : [12, 13, 0, -2, 12, [0, 20448, 17024, 20448, 60064, 19104, 20448, 16640, 20448, 16640, 57600, 4064, 0]],
        '堝' : [12, 13, 0, -2, 12, [0, 20416, 18496, 19392, 59968, 24544, 20512, 22432, 21664, 21664, 63392, 4192, 0]],
        '塲' : [12, 13, 0, -2, 12, [1024, 17408, 20448, 22592, 61376, 18496, 20448, 17408, 24544, 18720, 61984, 1216, 0]],
        '堡' : [12, 13, 0, -2, 12, [0, 12224, 10304, 20416, 49408, 24544, 17728, 22816, 1024, 16256, 1024, 65504, 0]],
        '塢' : [12, 13, 0, -2, 12, [256, 16896, 20416, 18496, 61376, 18432, 20448, 18432, 20448, 16416, 62752, 5472, 0]],
        '塋' : [12, 13, 0, -2, 12, [4352, 21792, 38208, 10880, 17472, 65504, 33824, 33824, 16256, 1024, 1024, 65504, 0]],
        '塰' : [12, 13, 0, -2, 12, [1024, 17408, 12256, 39232, 20448, 4672, 24544, 32896, 1024, 32704, 1024, 65504, 0]],
        '毀' : [12, 13, 0, -2, 12, [8192, 17344, 39488, 35392, 55872, 35936, 63488, 10208, 64032, 8768, 8576, 65120, 0]],
        '塒' : [12, 13, 0, -2, 12, [128, 16512, 24544, 21632, 62592, 22496, 23616, 22496, 21568, 22080, 64832, 192, 0]],
        '堽' : [12, 13, 0, -2, 12, [0, 20448, 19104, 19104, 61408, 16384, 20448, 16640, 18912, 18688, 59648, 4064, 0]],
        '塹' : [12, 13, 0, -2, 12, [4096, 4576, 65280, 21984, 32064, 21824, 65088, 4672, 1024, 32704, 1024, 65504, 0]],
        '墅' : [12, 13, 0, -2, 12, [0, 64480, 43072, 63616, 45024, 63648, 8320, 64896, 1024, 32704, 1024, 65504, 0]],
        '墹' : [12, 13, 0, -2, 12, [0, 24288, 21152, 24288, 62112, 24288, 20512, 22432, 21664, 22432, 62624, 6048, 0]],
        '墟' : [12, 13, 0, -2, 12, [512, 17344, 16896, 24544, 62496, 24512, 21504, 22464, 23200, 22208, 57984, 4064, 0]],
        '墫' : [12, 13, 0, -2, 12, [0, 17856, 18464, 24544, 60064, 19680, 18464, 20448, 16448, 24544, 58432, 704, 0]],
        '墺' : [12, 13, 0, -2, 12, [256, 16896, 20448, 19808, 60320, 19808, 19360, 19808, 16640, 24544, 58432, 6176, 0]],
        '壞' : [12, 13, 0, -2, 12, [256, 16640, 24544, 19104, 60064, 20448, 16640, 19808, 17024, 23712, 58432, 3872, 0]],
        '墻' : [12, 13, 0, -2, 12, [256, 16640, 24544, 18752, 59712, 21920, 24832, 24544, 21664, 22432, 61472, 8160, 0]],
        '墸' : [12, 13, 0, -2, 12, [1152, 17536, 24544, 17536, 57632, 20448, 16704, 24544, 17440, 20448, 62496, 2016, 0]],
        '墮' : [12, 13, 0, -2, 12, [256, 61408, 41472, 51168, 43136, 47072, 42016, 51168, 33824, 32704, 1024, 65504, 0]],
        '壅' : [12, 13, 0, -2, 12, [1024, 65504, 8768, 18400, 44160, 22496, 62592, 10208, 17408, 32704, 1024, 65504, 0]],
        '壓' : [12, 13, 0, -2, 12, [0, 65504, 42112, 48288, 33760, 48256, 42368, 48704, 9248, 32704, 512, 65504, 0]],
        '壑' : [12, 13, 0, -2, 12, [4096, 65504, 42272, 23200, 9376, 32576, 42144, 15648, 1024, 32704, 1024, 65504, 0]],
        '壗' : [12, 13, 0, -2, 12, [256, 16640, 20448, 16672, 61408, 16672, 20448, 19008, 20768, 20448, 60064, 8160, 0]],
        '壙' : [12, 13, 0, -2, 12, [256, 16640, 24544, 21120, 65504, 21120, 24544, 21824, 22464, 21824, 59328, 6176, 0]],
        '壘' : [12, 13, 0, -2, 12, [0, 16256, 9344, 16256, 9344, 65504, 43680, 64480, 1024, 32704, 1024, 65504, 0]],
        '壥' : [12, 13, 0, -2, 12, [0, 24544, 21824, 22464, 61696, 24544, 21824, 23200, 20736, 61408, 8448, 8160, 0]],
        '壜' : [12, 13, 0, -2, 12, [0, 20416, 18496, 20416, 59456, 24544, 21024, 22240, 16384, 24544, 58432, 4000, 0]],
        '壤' : [12, 13, 0, -2, 12, [256, 16640, 24544, 19104, 61152, 17024, 24544, 17024, 24544, 17728, 63616, 3680, 0]],
        '壟' : [12, 13, 0, -2, 12, [8704, 9184, 64000, 19424, 64544, 19424, 31232, 19424, 1024, 16256, 1024, 65504, 0]],
        '壯' : [12, 13, 0, -2, 12, [128, 36992, 36992, 36992, 65504, 4224, 4224, 61568, 20608, 20608, 36992, 38880, 0]],
        '壺' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 65504, 35360, 48032, 8320, 15232, 2560, 2560, 65504, 0]],
        '壹' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 65504, 32800, 49056, 8320, 16256, 4352, 4352, 65504, 0]],
        '壻' : [12, 13, 0, -2, 12, [0, 24544, 16672, 17856, 58624, 24544, 18464, 20448, 18464, 20448, 59424, 2144, 0]],
        '壼' : [12, 13, 0, -2, 12, [1024, 65504, 1024, 65504, 32800, 49056, 32800, 16256, 8320, 15232, 2560, 65504, 0]],
        '壽' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 65504, 32, 32640, 1088, 65504, 20544, 21568, 29376, 0]],
        '夂' : [12, 13, 0, -2, 12, [2048, 2048, 4032, 4160, 4160, 8320, 18560, 1280, 512, 1280, 6272, 57440, 0]],
        '夊' : [12, 13, 0, -2, 12, [2048, 2048, 4032, 18496, 10304, 4224, 10368, 17664, 512, 1280, 6272, 57440, 0]],
        '夐' : [12, 13, 0, -2, 12, [0, 8064, 8448, 32704, 51776, 20800, 32704, 16448, 32704, 53376, 3840, 61664, 0]],
        '夛' : [12, 13, 0, -2, 12, [0, 16256, 128, 16256, 128, 65504, 2048, 8128, 26688, 1152, 768, 31744, 0]],
        '梦' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8576, 29376, 42144, 10368, 8128, 26688, 1152, 768, 31744, 0]],
        '夥' : [12, 13, 0, -2, 12, [256, 63744, 43488, 65056, 43328, 63872, 9728, 63968, 9760, 28992, 41344, 11776, 0]],
        '夬' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 32704, 1088, 1088, 1088, 65504, 2560, 4352, 8320, 49248, 0]],
        '夭' : [12, 13, 0, -2, 12, [0, 960, 31744, 1024, 1024, 65504, 1024, 1024, 2560, 4352, 8320, 49248, 0]],
        '夲' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2304, 4224, 9280, 50208, 1024, 32704, 1024, 1024, 1024, 0]],
        '夸' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 8320, 24384, 32800, 32704, 4096, 16320, 8256, 896, 0]],
        '夾' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 9344, 9344, 21824, 33824, 2560, 4352, 8320, 49248, 0]],
        '竒' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 4352, 4352, 65504, 64, 15936, 8768, 8768, 15936, 192, 0]],
        '奕' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2304, 18752, 35104, 4352, 1024, 65504, 2560, 12672, 49248, 0]],
        '奐' : [12, 13, 0, -2, 12, [0, 8064, 8448, 32704, 51776, 21440, 24640, 17472, 65504, 2560, 12672, 49248, 0]],
        '奎' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4224, 9280, 57248, 1024, 32704, 1024, 16256, 1024, 65504, 0]],
        '奚' : [12, 13, 0, -2, 12, [0, 32736, 18560, 33856, 4608, 9344, 32704, 1024, 65504, 2560, 12672, 49248, 0]],
        '奘' : [12, 13, 0, -2, 12, [4224, 36992, 65504, 4224, 61568, 22496, 36864, 1024, 65504, 2560, 12672, 49248, 0]],
        '奢' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4224, 9280, 57248, 1280, 65504, 4160, 16320, 53312, 8128, 0]],
        '奠' : [12, 13, 0, -2, 12, [0, 12160, 19008, 49056, 10880, 13184, 8320, 16256, 8320, 65504, 4352, 57568, 0]],
        '奧' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 16960, 23872, 21824, 32704, 19520, 21824, 65504, 4352, 57568, 0]],
        '奬' : [12, 13, 0, -2, 12, [0, 37344, 63136, 4416, 63456, 21568, 37568, 1024, 65504, 1024, 6912, 57568, 0]],
        '奩' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4224, 8256, 32736, 51328, 20352, 16384, 24000, 21824, 32736, 0]],
        '奸' : [12, 13, 0, -2, 12, [0, 20448, 16640, 61696, 20736, 20736, 24544, 37120, 57600, 12544, 16640, 16640, 0]],
        '妁' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 62496, 22560, 20512, 21024, 37152, 57376, 12320, 16448, 16768, 0]],
        '妝' : [12, 13, 0, -2, 12, [256, 37120, 37120, 40928, 62016, 4672, 4672, 62528, 22144, 20864, 37440, 39968, 0]],
        '佞' : [12, 13, 0, -2, 12, [0, 12224, 8192, 16384, 24544, 49664, 16896, 24544, 17472, 19584, 17280, 23648, 0]],
        '侫' : [12, 13, 0, -2, 12, [256, 8448, 16352, 18432, 18432, 51136, 16896, 24544, 17472, 19584, 17280, 23648, 0]],
        '妣' : [12, 13, 0, -2, 12, [128, 17536, 17536, 62624, 22464, 21632, 21632, 38016, 58496, 13440, 18336, 19680, 0]],
        '妲' : [12, 13, 0, -2, 12, [0, 17376, 16928, 61984, 21472, 21024, 21024, 37408, 58336, 14336, 16384, 18400, 0]],
        '姆' : [12, 13, 0, -2, 12, [0, 18368, 17472, 63040, 21824, 24544, 21568, 38464, 59712, 10304, 20448, 16512, 0]],
        '姨' : [12, 13, 0, -2, 12, [256, 16640, 20448, 61696, 22496, 20768, 22496, 38144, 59360, 14624, 17024, 19552, 0]],
        '姜' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 16256, 1024, 65504, 2048, 65504, 4224, 7936, 24768, 0]],
        '妍' : [12, 13, 0, -2, 12, [0, 20448, 16960, 62016, 21056, 24544, 21056, 37440, 57920, 12864, 17472, 17472, 0]],
        '姙' : [12, 13, 0, -2, 12, [0, 17120, 16960, 62528, 21568, 23616, 21984, 37952, 58432, 13376, 17472, 17888, 0]],
        '姚' : [12, 13, 0, -2, 12, [128, 17024, 19104, 63168, 21120, 21120, 21120, 38592, 60064, 8832, 17568, 22752, 0]],
        '娥' : [12, 13, 0, -2, 12, [128, 20128, 17568, 62592, 22496, 23680, 21664, 38048, 58944, 15424, 17824, 20000, 0]],
        '娟' : [12, 13, 0, -2, 12, [0, 18400, 17440, 63456, 20480, 22496, 21536, 38880, 58400, 14304, 17440, 17504, 0]],
        '娑' : [12, 13, 0, -2, 12, [256, 16640, 9536, 35104, 16640, 4928, 8320, 18688, 65504, 4224, 7936, 57568, 0]],
        '娜' : [12, 13, 0, -2, 12, [0, 24288, 19104, 65184, 19104, 19136, 24224, 35488, 60064, 12992, 21120, 19584, 0]],
        '娉' : [12, 13, 0, -2, 12, [256, 16640, 18400, 62752, 22496, 21792, 24544, 37376, 58336, 13344, 17440, 16576, 0]],
        '娚' : [12, 13, 0, -2, 12, [0, 20448, 18720, 65504, 18720, 20448, 20736, 40928, 57888, 12832, 17440, 22720, 0]],
        '婀' : [12, 13, 0, -2, 12, [0, 20448, 18976, 64032, 19680, 19104, 19104, 39584, 60128, 11296, 18464, 18528, 0]],
        '婬' : [12, 13, 0, -2, 12, [0, 18400, 17408, 62784, 22816, 20480, 22496, 36992, 59360, 12416, 16512, 18400, 0]],
        '婉' : [12, 13, 0, -2, 12, [256, 16640, 24544, 61472, 24288, 21152, 21152, 39584, 59104, 12928, 17568, 22752, 0]],
        '娵' : [12, 13, 0, -2, 12, [0, 24544, 18944, 65504, 18976, 19104, 20128, 39584, 59968, 11840, 23200, 17184, 0]],
        '娶' : [12, 13, 0, -2, 12, [0, 64480, 18464, 31264, 19008, 31104, 52832, 2048, 65504, 4224, 7936, 57568, 0]],
        '婢' : [12, 13, 0, -2, 12, [128, 16640, 18400, 62752, 22496, 21792, 22496, 37504, 58496, 16352, 16512, 16512, 0]],
        '婪' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8576, 29376, 42144, 8320, 2048, 65504, 4224, 7936, 57568, 0]],
        '媚' : [12, 13, 0, -2, 12, [0, 24544, 20768, 65504, 20480, 22496, 21536, 38880, 58400, 14304, 17440, 18400, 0]],
        '媼' : [12, 13, 0, -2, 12, [0, 18368, 17728, 63168, 21568, 22464, 20480, 40928, 60064, 10912, 19104, 24544, 0]],
        '媾' : [12, 13, 0, -2, 12, [576, 16960, 20448, 62016, 24544, 20608, 22496, 38048, 59360, 13472, 20448, 17440, 0]],
        '嫋' : [12, 13, 0, -2, 12, [0, 20192, 16928, 61984, 20192, 18560, 20192, 37408, 59232, 10912, 21280, 19648, 0]],
        '嫂' : [12, 13, 0, -2, 12, [256, 17664, 18784, 63776, 19808, 18720, 20448, 37120, 65504, 9280, 18304, 22624, 0]],
        '媽' : [12, 13, 0, -2, 12, [0, 18400, 17536, 63456, 21632, 22496, 21632, 38880, 57376, 13600, 21792, 16576, 0]],
        '嫣' : [12, 13, 0, -2, 12, [0, 20448, 16640, 62912, 21760, 24544, 21504, 38848, 58368, 14304, 16416, 19136, 0]],
        '嫗' : [12, 13, 0, -2, 12, [0, 20448, 18432, 64448, 19008, 19392, 18432, 40672, 60064, 12000, 18432, 20448, 0]],
        '嫦' : [12, 13, 0, -2, 12, [256, 17728, 24544, 61472, 22464, 21568, 22464, 37120, 61408, 10528, 18784, 16640, 0]],
        '嫩' : [12, 13, 0, -2, 12, [1152, 17536, 24544, 58528, 24352, 21792, 21792, 40864, 58432, 11840, 21664, 17696, 0]],
        '嫖' : [12, 13, 0, -2, 12, [0, 20448, 17024, 65504, 21152, 24544, 20480, 38880, 61440, 12256, 17728, 19232, 0]],
        '嫺' : [12, 13, 0, -2, 12, [0, 20192, 19104, 65248, 19104, 20192, 18464, 39840, 60064, 11168, 19104, 19616, 0]],
        '嫻' : [12, 13, 0, -2, 12, [0, 20192, 19104, 65248, 19104, 20192, 18720, 39840, 59680, 11168, 19808, 18720, 0]],
        '嬌' : [12, 13, 0, -2, 12, [0, 20448, 16640, 65504, 21120, 22208, 23456, 36864, 61408, 10912, 19360, 18528, 0]],
        '嬋' : [12, 13, 0, -2, 12, [0, 20192, 19104, 64160, 24544, 21792, 22496, 38176, 59360, 12544, 20448, 16640, 0]],
        '嬖' : [12, 13, 0, -2, 12, [256, 61696, 38880, 62016, 36832, 61696, 38880, 37120, 65504, 4224, 7936, 57568, 0]],
        '嬲' : [12, 13, 0, -2, 12, [2048, 64480, 43680, 65504, 43680, 64480, 19072, 64480, 21664, 23200, 37152, 45760, 0]],
        '嫐' : [12, 13, 0, -2, 12, [0, 24192, 21632, 65504, 21664, 24224, 17568, 40736, 59840, 10560, 20768, 22048, 0]],
        '嬪' : [12, 13, 0, -2, 12, [256, 16640, 24544, 61472, 22496, 22848, 22496, 40000, 59328, 13376, 18368, 22624, 0]],
        '嬶' : [12, 13, 0, -2, 12, [256, 16896, 18368, 62528, 22464, 21568, 24544, 39200, 61408, 8768, 24544, 17472, 0]],
        '嬾' : [12, 13, 0, -2, 12, [1024, 17888, 24224, 58528, 24320, 21984, 24352, 38368, 58656, 12256, 21824, 17952, 0]],
        '孃' : [12, 13, 0, -2, 12, [256, 16640, 20448, 62752, 22496, 21120, 24544, 37504, 61408, 8864, 23616, 18208, 0]],
        '孅' : [12, 13, 0, -2, 12, [2688, 19104, 21920, 59520, 24544, 19072, 23456, 35488, 64320, 10816, 19104, 24352, 0]],
        '孀' : [12, 13, 0, -2, 12, [0, 20448, 16640, 65504, 20768, 22368, 21504, 40672, 58528, 12000, 21920, 17632, 0]],
        '孑' : [12, 13, 0, -2, 12, [0, 32640, 128, 256, 512, 1120, 1920, 64512, 1024, 1024, 1024, 3072, 0]],
        '孕' : [12, 13, 0, -2, 12, [0, 32640, 1152, 2272, 12320, 57120, 704, 1024, 65504, 1024, 1024, 7168, 0]],
        '孚' : [12, 13, 0, -2, 12, [0, 32704, 18560, 33856, 32800, 16256, 256, 512, 65504, 1024, 1024, 7168, 0]],
        '孛' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 65504, 32800, 49056, 512, 1024, 65504, 1024, 7168, 0]],
        '孥' : [12, 13, 0, -2, 12, [8192, 9184, 64544, 18720, 18752, 14528, 50464, 16256, 256, 65504, 1024, 7168, 0]],
        '孩' : [12, 13, 0, -2, 12, [256, 61696, 8160, 4352, 8736, 9280, 12928, 57632, 9792, 8320, 9024, 27680, 0]],
        '孰' : [12, 13, 0, -2, 12, [8448, 8448, 63744, 22464, 28992, 1344, 64320, 4416, 8896, 64064, 9280, 25696, 0]],
        '孳' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 10560, 21120, 8448, 19008, 32704, 256, 65504, 1024, 7168, 0]],
        '孵' : [12, 13, 0, -2, 12, [0, 61408, 35488, 43680, 43520, 61152, 43552, 43584, 60384, 11840, 18496, 18624, 0]],
        '學' : [12, 13, 0, -2, 12, [2560, 25792, 19008, 30144, 19008, 65504, 32800, 40736, 512, 65504, 1024, 7168, 0]],
        '斈' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 3584, 12672, 49248, 16256, 512, 65504, 1024, 7168, 0]],
        '孺' : [12, 13, 0, -2, 12, [0, 61408, 8448, 12256, 18720, 19296, 24832, 53216, 16896, 20448, 19104, 51872, 0]],
        '宀' : [12, 13, 0, -2, 12, [0, 1024, 1024, 1024, 65504, 32800, 32800, 32800, 0, 0, 0, 0, 0]],
        '它' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 40992, 8192, 9088, 15360, 8192, 8256, 8256, 8128, 0]],
        '宦' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 49120, 41504, 16256, 8320, 16256, 8704, 8704, 16320, 0]],
        '宸' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 32736, 16384, 24512, 16384, 32736, 37440, 37248, 15456, 0]],
        '寃' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 49056, 16640, 65472, 17472, 32704, 2688, 12832, 49632, 0]],
        '寇' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33056, 29120, 256, 1984, 64064, 20864, 22080, 36896, 36832, 0]],
        '寉' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 37408, 16320, 25088, 49088, 8704, 16320, 8704, 16352, 0]],
        '寔' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 40736, 4352, 4352, 65504, 1024, 10112, 15360, 50144, 0]],
        '寐' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 20736, 22496, 28928, 8160, 62208, 21824, 39200, 37632, 0]],
        '寤' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 36896, 22496, 20736, 30656, 4672, 65504, 21568, 37952, 38848, 0]],
        '實' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 42144, 65504, 9344, 16256, 8320, 16256, 8320, 16256, 49248, 0]],
        '寢' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32800, 22464, 20544, 32736, 4160, 65504, 21664, 37760, 40032, 0]],
        '寞' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 65504, 8320, 16256, 8320, 65504, 4352, 8320, 49248, 0]],
        '寥' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 43296, 23200, 10528, 19968, 12672, 50784, 6272, 768, 15360, 0]],
        '寫' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 34848, 13184, 8320, 15232, 8320, 32736, 32800, 10912, 19136, 0]],
        '寰' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 43680, 49056, 0, 65504, 8320, 16288, 4672, 57728, 15456, 0]],
        '寶' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37536, 31680, 4768, 65504, 8320, 16256, 8320, 16256, 49248, 0]],
        '寳' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 42144, 29376, 9376, 65408, 8320, 16256, 8320, 16256, 49248, 0]],
        '尅' : [12, 13, 0, -2, 12, [8256, 8256, 63552, 9184, 63552, 35392, 35136, 63552, 20672, 20480, 36896, 36832, 0]],
        '將' : [12, 13, 0, -2, 12, [0, 37344, 38432, 37184, 62592, 4928, 7232, 63456, 20544, 21568, 37440, 37056, 0]],
        '專' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 9344, 16256, 9344, 32640, 1088, 65504, 128, 8320, 4992, 0]],
        '對' : [12, 13, 0, -2, 12, [32, 21536, 55328, 21472, 64544, 18464, 18720, 64672, 8224, 64544, 8224, 64608, 0]],
        '尓' : [12, 13, 0, -2, 12, [8192, 8192, 8192, 16352, 16896, 33280, 512, 8832, 16960, 33312, 512, 3584, 0]],
        '尠' : [12, 13, 0, -2, 12, [256, 18688, 63744, 19776, 31008, 18688, 18688, 64256, 43040, 52288, 33152, 65024, 0]],
        '尢' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 65504, 2560, 2560, 2560, 2560, 4608, 4640, 8736, 50144, 0]],
        '尨' : [12, 13, 0, -2, 12, [8192, 10336, 9600, 8192, 65120, 10624, 10240, 10336, 18816, 18432, 34848, 34784, 0]],
        '尸' : [12, 13, 0, -2, 12, [0, 16352, 8224, 8224, 8224, 16352, 8192, 8192, 16384, 16384, 32768, 32768, 0]],
        '尹' : [12, 13, 0, -2, 12, [0, 32704, 1088, 1088, 65504, 1088, 1088, 32704, 1024, 1024, 2048, 28672, 0]],
        '屁' : [12, 13, 0, -2, 12, [0, 32736, 16416, 16416, 32736, 20736, 20768, 23872, 20864, 37120, 37152, 15584, 0]],
        '屆' : [12, 13, 0, -2, 12, [0, 32736, 16416, 16416, 32736, 16640, 20768, 22496, 20768, 38880, 36896, 8160, 0]],
        '屎' : [12, 13, 0, -2, 12, [0, 32736, 16416, 16416, 32736, 21056, 19072, 32736, 17920, 35456, 45664, 512, 0]],
        '屓' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 20544, 24512, 20544, 24512, 20544, 40896, 34880, 12320, 0]],
        '屐' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 20608, 26592, 18560, 22464, 29248, 37440, 37248, 5728, 0]],
        '屏' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 18496, 17536, 24544, 17536, 24544, 33920, 34944, 12416, 0]],
        '孱' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 16640, 24512, 16896, 32736, 18496, 48608, 34880, 6336, 0]],
        '屬' : [12, 13, 0, -2, 12, [0, 65504, 32800, 65504, 42112, 49088, 43584, 49120, 59680, 48928, 2336, 32448, 0]],
        '屮' : [12, 13, 0, -2, 12, [1024, 1024, 33824, 33824, 33824, 33824, 33824, 65504, 33824, 1024, 2048, 28672, 0]],
        '乢' : [12, 13, 0, -2, 12, [512, 8704, 8704, 8704, 43520, 43520, 43520, 43520, 43552, 43552, 64032, 480, 0]],
        '屶' : [12, 13, 0, -2, 12, [1024, 17472, 17472, 17472, 32704, 0, 65504, 2080, 2080, 4128, 8256, 50048, 0]],
        '屹' : [12, 13, 0, -2, 12, [512, 8704, 9184, 8704, 44032, 44000, 43072, 43136, 43264, 43552, 64032, 480, 0]],
        '岌' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 0, 65472, 8256, 8320, 16352, 18464, 17472, 33664, 48224, 0]],
        '岑' : [12, 13, 0, -2, 12, [1024, 33824, 33824, 65504, 4352, 8320, 24384, 32800, 16256, 128, 256, 1536, 0]],
        '岔' : [12, 13, 0, -2, 12, [0, 5888, 8320, 16448, 49056, 2176, 4224, 26368, 1024, 33824, 33824, 65504, 0]],
        '妛' : [12, 13, 0, -2, 12, [1024, 17472, 17472, 32704, 0, 65504, 2048, 65504, 4224, 12544, 3584, 29120, 0]],
        '岫' : [12, 13, 0, -2, 12, [256, 8448, 8448, 12256, 43296, 43296, 43296, 45024, 43296, 43296, 63776, 4064, 0]],
        '岻' : [12, 13, 0, -2, 12, [0, 8288, 10112, 9344, 42112, 42976, 42048, 42048, 42016, 42784, 61440, 4064, 0]],
        '岶' : [12, 13, 0, -2, 12, [256, 8448, 8704, 10208, 42016, 42016, 42016, 42976, 42016, 42016, 64544, 2016, 0]],
        '岼' : [12, 13, 0, -2, 12, [0, 12256, 8448, 10528, 42304, 41216, 41216, 45024, 43264, 43264, 63744, 256, 0]],
        '岷' : [12, 13, 0, -2, 12, [0, 10208, 9248, 9248, 42976, 42112, 42112, 42976, 42048, 42048, 64544, 1824, 0]],
        '峅' : [12, 13, 0, -2, 12, [256, 8704, 9248, 12256, 40960, 41536, 41536, 45024, 41536, 41536, 64576, 1088, 0]],
        '岾' : [12, 13, 0, -2, 12, [256, 8448, 8448, 8672, 43264, 43264, 43264, 45024, 43040, 43040, 63520, 4064, 0]],
        '峇' : [12, 13, 0, -2, 12, [1024, 33824, 33824, 65504, 4352, 8320, 24384, 32800, 16256, 8320, 8320, 16256, 0]],
        '峙' : [12, 13, 0, -2, 12, [256, 8448, 10208, 8448, 41216, 45024, 41024, 45024, 41024, 42048, 62016, 192, 0]],
        '峩' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 0, 64064, 8736, 65472, 8480, 14656, 57472, 9024, 27680, 0]],
        '峽' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8448, 42304, 42304, 43936, 43296, 41216, 41600, 62528, 6176, 0]],
        '峺' : [12, 13, 0, -2, 12, [0, 12256, 8448, 12256, 43296, 45024, 43296, 45024, 42240, 41728, 62592, 6240, 0]],
        '峭' : [12, 13, 0, -2, 12, [256, 8448, 9536, 10528, 41216, 42976, 42016, 42976, 42016, 42976, 64544, 1120, 0]],
        '嶌' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 2048, 16256, 8320, 16256, 8320, 16352, 32, 21792, 38240, 0]],
        '峪' : [12, 13, 0, -2, 12, [0, 8768, 9248, 8192, 41344, 41536, 42016, 43008, 42976, 42016, 64544, 2016, 0]],
        '崋' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 0, 32704, 9344, 65504, 9344, 32704, 1024, 65504, 1024, 0]],
        '崕' : [12, 13, 0, -2, 12, [0, 12256, 10368, 11232, 43136, 43136, 45024, 43136, 44000, 43136, 61568, 6112, 0]],
        '崗' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 0, 65504, 37152, 37408, 65504, 33824, 42144, 49056, 32864, 0]],
        '嵜' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 1024, 32704, 4352, 4352, 65504, 64, 15936, 8768, 16064, 0]],
        '崟' : [12, 13, 0, -2, 12, [1024, 33824, 33824, 65504, 4352, 8320, 24384, 33824, 32704, 9344, 5376, 65504, 0]],
        '崛' : [12, 13, 0, -2, 12, [0, 12256, 10272, 12256, 43136, 43680, 43680, 44000, 43136, 44192, 62624, 6112, 0]],
        '崑' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 16448, 32704, 16448, 32704, 16896, 31712, 16896, 16928, 63968, 0]],
        '崔' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 4224, 4352, 16352, 8704, 32704, 41472, 16320, 8704, 16352, 0]],
        '崢' : [12, 13, 0, -2, 12, [0, 10208, 9536, 10528, 43008, 42976, 41120, 45024, 41120, 42976, 63616, 896, 0]],
        '崚' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8448, 45024, 41600, 44128, 41920, 44096, 42112, 62336, 7264, 0]],
        '崙' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 4352, 8320, 32704, 32800, 32704, 19008, 32704, 19008, 19136, 0]],
        '崘' : [12, 13, 0, -2, 12, [0, 9088, 9280, 10272, 47072, 40960, 45024, 43680, 43680, 45024, 64160, 2720, 0]],
        '嵌' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 512, 37376, 64480, 37536, 38048, 62592, 36992, 37184, 63008, 0]],
        '嵒' : [12, 13, 0, -2, 12, [0, 16256, 8320, 8320, 65504, 35360, 35360, 64480, 1024, 33824, 33824, 65504, 0]],
        '嵎' : [12, 13, 0, -2, 12, [0, 12256, 10528, 12256, 43296, 45024, 41216, 45024, 43552, 43680, 65440, 2144, 0]],
        '嵋' : [12, 13, 0, -2, 12, [0, 12256, 10528, 12256, 43008, 44000, 43552, 44000, 43552, 44000, 61984, 5088, 0]],
        '嵬' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 2048, 32704, 17472, 32704, 17472, 32704, 2688, 12832, 50144, 0]],
        '嵳' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 4352, 65504, 1024, 32704, 1024, 65504, 8704, 49664, 16352, 0]],
        '嵶' : [12, 13, 0, -2, 12, [0, 12000, 8736, 8736, 44768, 43136, 44768, 43680, 42592, 43680, 62240, 1632, 0]],
        '嶇' : [12, 13, 0, -2, 12, [0, 12256, 10240, 11200, 43584, 43968, 43008, 44768, 43680, 44768, 63488, 4064, 0]],
        '嶄' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 8192, 65504, 43520, 64000, 44000, 64064, 8768, 64576, 9280, 0]],
        '嶂' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8768, 45024, 43040, 45024, 43040, 45024, 41216, 65504, 256, 0]],
        '嶢' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8448, 45024, 42048, 44768, 42048, 45024, 41600, 62112, 3296, 0]],
        '嶝' : [12, 13, 0, -2, 12, [0, 12064, 8512, 10912, 42048, 42976, 43008, 42976, 42016, 42976, 62016, 4064, 0]],
        '嶬' : [12, 13, 0, -2, 12, [1088, 9344, 12256, 8448, 42976, 41216, 45024, 42112, 45024, 42144, 65088, 1440, 0]],
        '嶮' : [12, 13, 0, -2, 12, [0, 10176, 10272, 14304, 40960, 44768, 43680, 43680, 44768, 42048, 60064, 4384, 0]],
        '嶽' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 0, 44704, 16544, 45024, 8320, 28288, 43648, 11072, 52768, 0]],
        '嶐' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 1024, 59328, 47168, 50304, 42880, 47456, 53184, 33024, 40928, 0]],
        '嶷' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 32768, 64480, 32832, 31872, 37856, 64640, 4832, 10880, 51168, 0]],
        '嶼' : [12, 13, 0, -2, 12, [0, 11872, 11168, 10784, 44640, 43936, 43168, 44768, 43680, 49120, 58432, 6176, 0]],
        '巉' : [12, 13, 0, -2, 12, [0, 10176, 10368, 16352, 43040, 45024, 42112, 49120, 43296, 45024, 62080, 3296, 0]],
        '巍' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 8448, 65504, 25888, 47072, 9504, 65504, 19072, 29344, 36064, 0]],
        '巓' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 32768, 64480, 32896, 31712, 43552, 48096, 43552, 65504, 18720, 0]],
        '巒' : [12, 13, 0, -2, 12, [1024, 24384, 41120, 20032, 61920, 20032, 60128, 20032, 1024, 33824, 33824, 65504, 0]],
        '巖' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 19008, 32736, 18560, 32736, 21152, 24224, 21056, 40608, 45856, 0]],
        '巛' : [12, 13, 0, -2, 12, [0, 8736, 8736, 17472, 17472, 34944, 34944, 17472, 17472, 8736, 8736, 8736, 0]],
        '巫' : [12, 13, 0, -2, 12, [0, 65504, 1024, 1024, 9344, 9344, 9344, 21824, 36384, 1024, 1024, 65504, 0]],
        '已' : [12, 13, 0, -2, 12, [0, 32640, 128, 128, 16512, 32640, 16384, 16384, 16448, 16448, 16448, 16256, 0]],
        '巵' : [12, 13, 0, -2, 12, [0, 448, 32256, 16384, 32736, 16384, 24512, 21056, 24512, 36864, 36896, 4064, 0]],
        '帋' : [12, 13, 0, -2, 12, [0, 32704, 17408, 32736, 16896, 16768, 62560, 1024, 32704, 17472, 17600, 1024, 0]],
        '帚' : [12, 13, 0, -2, 12, [0, 16256, 128, 65504, 128, 65504, 33824, 49056, 9344, 9344, 9600, 1024, 0]],
        '帙' : [12, 13, 0, -2, 12, [128, 9344, 9344, 10208, 63616, 43136, 45040, 43136, 43328, 8512, 8736, 11296, 0]],
        '帑' : [12, 13, 0, -2, 12, [8192, 9184, 64544, 18720, 18752, 14528, 50976, 1024, 32704, 17472, 17472, 17600, 0]],
        '帛' : [12, 13, 0, -2, 12, [1024, 2048, 32704, 16448, 32704, 16448, 32704, 1024, 65504, 33824, 33824, 33888, 0]],
        '帶' : [12, 13, 0, -2, 12, [2560, 10880, 65504, 10880, 52960, 0, 65504, 33824, 49056, 9344, 9344, 9600, 0]],
        '帷' : [12, 13, 0, -2, 12, [512, 8768, 8768, 10208, 62592, 45024, 42112, 42112, 42976, 9344, 9344, 10208, 0]],
        '幄' : [12, 13, 0, -2, 12, [0, 12256, 10272, 12256, 63488, 45024, 43584, 44000, 43136, 11232, 8320, 10208, 0]],
        '幃' : [12, 13, 0, -2, 12, [128, 8320, 10208, 8480, 65504, 43552, 44000, 43136, 42976, 9344, 12256, 8320, 0]],
        '幀' : [12, 13, 0, -2, 12, [256, 8448, 8672, 8448, 65504, 42016, 42976, 42016, 42016, 10208, 8768, 11296, 0]],
        '幎' : [12, 13, 0, -2, 12, [0, 12256, 10272, 12256, 62528, 42944, 42048, 42944, 41216, 12256, 8768, 11296, 0]],
        '幗' : [12, 13, 0, -2, 12, [0, 12256, 10528, 12256, 63648, 44704, 43744, 44704, 43168, 12128, 10528, 12256, 0]],
        '幔' : [12, 13, 0, -2, 12, [0, 10176, 9280, 10176, 62528, 45024, 43680, 45024, 42048, 8768, 9088, 11360, 0]],
        '幟' : [12, 13, 0, -2, 12, [1152, 9376, 16288, 10880, 60128, 49024, 41120, 44704, 43584, 11840, 10912, 12064, 0]],
        '幢' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8768, 62016, 45024, 42272, 42976, 42272, 10208, 8448, 12256, 0]],
        '幤' : [12, 13, 0, -2, 12, [8704, 43520, 9184, 64576, 38208, 62848, 34400, 1024, 32704, 17472, 17472, 17600, 0]],
        '幇' : [12, 13, 0, -2, 12, [8256, 8256, 64480, 8256, 62528, 8768, 63680, 1024, 32704, 17472, 17472, 17600, 0]],
        '幵' : [12, 13, 0, -2, 12, [0, 64480, 8320, 8320, 8320, 8320, 65504, 8320, 8320, 8320, 8320, 8320, 0]],
        '并' : [12, 13, 0, -2, 12, [8320, 4224, 256, 32704, 4352, 4352, 4352, 65504, 4352, 4352, 8448, 49408, 0]],
        '幺' : [12, 13, 0, -2, 12, [0, 2048, 4096, 8320, 16640, 8704, 5120, 2176, 4224, 8320, 16704, 32320, 0]],
        '麼' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 32736, 18560, 23200, 27296, 18560, 37120, 35392, 16352, 0]],
        '广' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 32768, 32768, 0]],
        '庠' : [12, 13, 0, -2, 12, [512, 512, 32736, 17472, 17536, 24544, 16640, 20448, 16640, 40928, 33024, 256, 0]],
        '廁' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 24224, 21152, 24224, 21152, 21152, 40608, 37408, 8544, 0]],
        '廂' : [12, 13, 0, -2, 12, [512, 512, 32736, 18432, 19424, 32288, 19424, 23072, 23520, 44576, 43552, 3040, 0]],
        '廈' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 32736, 16640, 24544, 20512, 24544, 43072, 34688, 14432, 0]],
        '廐' : [12, 13, 0, -2, 12, [512, 512, 32736, 18432, 24544, 21056, 24128, 21472, 20544, 40640, 37184, 7776, 0]],
        '廏' : [12, 13, 0, -2, 12, [512, 512, 32736, 18432, 24512, 20800, 24416, 20992, 21472, 40224, 37312, 7712, 0]],
        '廖' : [12, 13, 0, -2, 12, [512, 512, 32736, 21664, 28000, 21664, 25344, 19584, 29536, 35840, 33152, 7680, 0]],
        '廣' : [12, 13, 0, -2, 12, [512, 512, 32736, 17536, 24544, 17536, 32736, 18752, 20416, 35136, 36800, 12384, 0]],
        '廝' : [12, 13, 0, -2, 12, [512, 512, 32736, 18432, 32224, 18688, 31200, 18720, 32544, 33056, 43296, 17440, 0]],
        '廚' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 36928, 64576, 37856, 64576, 34368, 64832, 43072, 10304, 64704, 0]],
        '廛' : [12, 13, 0, -2, 12, [512, 512, 32736, 21056, 24512, 21056, 32736, 18560, 21088, 44928, 33280, 16352, 0]],
        '廢' : [12, 13, 0, -2, 12, [512, 512, 65504, 38208, 59520, 48608, 58688, 48736, 41920, 47680, 35200, 15968, 0]],
        '廡' : [12, 13, 0, -2, 12, [512, 512, 32736, 18432, 24544, 30016, 24544, 21824, 32736, 32768, 38208, 9504, 0]],
        '廨' : [12, 13, 0, -2, 12, [512, 512, 65504, 34976, 64800, 42592, 64768, 42464, 42624, 64640, 34784, 35968, 0]],
        '廩' : [12, 13, 0, -2, 12, [512, 512, 32736, 21824, 22336, 20544, 24512, 16896, 32736, 34432, 35392, 12832, 0]],
        '廬' : [12, 13, 0, -2, 12, [512, 512, 32736, 20992, 24544, 21024, 22464, 21824, 21824, 45024, 43680, 16352, 0]],
        '廱' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 43328, 54240, 47680, 45024, 64064, 44000, 64064, 33344, 64480, 0]],
        '廳' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37120, 65504, 37536, 65504, 20480, 65504, 20800, 62752, 6592, 0]],
        '廰' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 33024, 65504, 37536, 62112, 40928, 61952, 37184, 62496, 6016, 0]],
        '廴' : [12, 13, 0, -2, 12, [0, 61440, 4096, 8192, 16384, 61440, 4096, 36864, 24576, 8192, 22528, 34784, 0]],
        '廸' : [12, 13, 0, -2, 12, [256, 256, 57600, 12256, 18720, 35104, 61408, 10528, 43296, 20448, 24576, 40928, 0]],
        '廾' : [12, 13, 0, -2, 12, [4224, 4224, 4224, 4224, 65504, 4224, 4224, 4224, 4224, 4224, 8320, 49280, 0]],
        '弃' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4096, 8320, 32704, 4384, 4352, 65504, 4352, 8448, 49408, 0]],
        '弉' : [12, 13, 0, -2, 12, [4224, 36992, 63456, 4224, 61568, 22496, 37120, 4352, 65504, 4352, 8448, 49408, 0]],
        '彝' : [12, 13, 0, -2, 12, [0, 8128, 8320, 65504, 43328, 29344, 45024, 8864, 4352, 65504, 4352, 24832, 0]],
        '彜' : [12, 13, 0, -2, 12, [0, 8128, 8320, 65504, 43584, 30688, 43584, 9408, 4352, 65504, 4352, 24832, 0]],
        '弋' : [12, 13, 0, -2, 12, [4096, 4352, 4224, 4096, 65504, 2048, 2048, 2048, 1024, 1088, 576, 448, 0]],
        '弑' : [12, 13, 0, -2, 12, [128, 2208, 22496, 8320, 20608, 36736, 8832, 64064, 8768, 29248, 44960, 8224, 0]],
        '弖' : [12, 13, 0, -2, 12, [0, 32704, 64, 64, 32704, 16384, 65504, 32800, 32, 960, 0, 65504, 0]],
        '弩' : [12, 13, 0, -2, 12, [8192, 9184, 64800, 18880, 14880, 53184, 64, 16320, 8192, 32736, 16416, 960, 0]],
        '弭' : [12, 13, 0, -2, 12, [0, 65504, 4640, 5088, 61984, 33312, 33760, 61984, 4640, 5088, 5664, 57376, 0]],
        '弸' : [12, 13, 0, -2, 12, [0, 61152, 10912, 10912, 61152, 35488, 35488, 61152, 10912, 10912, 10912, 51872, 0]],
        '彁' : [12, 13, 0, -2, 12, [0, 65504, 9536, 9536, 59200, 32832, 40928, 58688, 9536, 10048, 8256, 49344, 0]],
        '彈' : [12, 13, 0, -2, 12, [0, 61152, 10912, 12000, 57344, 36832, 35104, 61408, 10528, 10528, 16352, 49408, 0]],
        '彌' : [12, 13, 0, -2, 12, [0, 65504, 9536, 10528, 61408, 36192, 35744, 60768, 11168, 11616, 11168, 51488, 0]],
        '彎' : [12, 13, 0, -2, 12, [1024, 24384, 41120, 20032, 61920, 20032, 51808, 32704, 16384, 65504, 32800, 960, 0]],
        '弯' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 18752, 35104, 32704, 64, 32704, 16384, 65504, 32800, 960, 0]],
        '彑' : [12, 13, 0, -2, 12, [1024, 1024, 1024, 2048, 3968, 4224, 4352, 8448, 15872, 512, 1024, 65504, 0]],
        '彖' : [12, 13, 0, -2, 12, [0, 4032, 4224, 16128, 512, 65504, 17472, 6784, 59136, 6784, 57952, 3072, 0]],
        '彗' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8320, 64480, 8320, 0, 16256, 128, 65504, 128, 16256, 0]],
        '彙' : [12, 13, 0, -2, 12, [0, 8064, 8448, 65504, 42144, 49056, 9344, 9344, 65504, 3328, 13440, 50272, 0]],
        '彡' : [12, 13, 0, -2, 12, [0, 384, 3584, 28672, 0, 192, 1792, 14336, 0, 96, 896, 31744, 0]],
        '彭' : [12, 13, 0, -2, 12, [8192, 8288, 64896, 8192, 64512, 96, 64896, 33792, 64512, 18432, 18528, 64896, 0]],
        '彳' : [12, 13, 0, -2, 12, [0, 384, 1536, 14336, 96, 384, 3584, 29184, 512, 512, 512, 512, 0]],
        '彷' : [12, 13, 0, -2, 12, [512, 8704, 20448, 33280, 512, 9184, 17440, 50208, 18464, 18464, 16448, 16768, 0]],
        '徃' : [12, 13, 0, -2, 12, [128, 9344, 17536, 34784, 2176, 10368, 16512, 51168, 16512, 16512, 16512, 20448, 0]],
        '徂' : [12, 13, 0, -2, 12, [0, 10208, 17440, 33824, 2016, 9248, 17440, 51168, 17440, 17440, 17440, 20448, 0]],
        '彿' : [12, 13, 0, -2, 12, [640, 8832, 20448, 33440, 4064, 10880, 19072, 53216, 17056, 17056, 17600, 22656, 0]],
        '徊' : [12, 13, 0, -2, 12, [0, 12256, 18464, 34848, 2976, 10912, 19104, 51872, 19360, 18464, 18464, 20448, 0]],
        '很' : [12, 13, 0, -2, 12, [0, 10208, 17440, 34784, 5152, 9248, 18400, 50432, 17696, 17568, 17472, 18208, 0]],
        '徑' : [12, 13, 0, -2, 12, [0, 10208, 17056, 33440, 5440, 8864, 17056, 49152, 18400, 16512, 16512, 18400, 0]],
        '徇' : [12, 13, 0, -2, 12, [2048, 10240, 20448, 36896, 4128, 3872, 18720, 53024, 18720, 20256, 16448, 16768, 0]],
        '從' : [12, 13, 0, -2, 12, [1088, 9280, 18560, 38208, 544, 8448, 16640, 51680, 18688, 18688, 20224, 28896, 0]],
        '徙' : [12, 13, 0, -2, 12, [256, 8448, 18912, 35072, 2304, 16352, 16640, 49408, 18912, 18688, 20224, 28896, 0]],
        '徘' : [12, 13, 0, -2, 12, [128, 8832, 24288, 33408, 640, 12000, 17024, 49792, 24192, 17120, 17536, 22656, 0]],
        '徠' : [12, 13, 0, -2, 12, [256, 8448, 24544, 33024, 1344, 9536, 19232, 53536, 17280, 17728, 22816, 16640, 0]],
        '徨' : [12, 13, 0, -2, 12, [128, 8448, 18400, 33824, 6112, 9248, 18400, 49280, 18400, 16512, 16512, 20448, 0]],
        '徭' : [12, 13, 0, -2, 12, [0, 9184, 19488, 33088, 1408, 11776, 17376, 51456, 20448, 16640, 18720, 20448, 0]],
        '徼' : [12, 13, 0, -2, 12, [1152, 10368, 24288, 37536, 7968, 12832, 24224, 51360, 20032, 21056, 17056, 19744, 0]],
        '忖' : [12, 13, 0, -2, 12, [32, 8224, 8224, 10208, 40992, 45088, 43552, 41248, 40992, 8224, 8224, 8416, 0]],
        '忻' : [12, 13, 0, -2, 12, [0, 8384, 9984, 9216, 41984, 47072, 44096, 42048, 42048, 10304, 10304, 8256, 0]],
        '忤' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 10368, 43136, 41088, 41088, 45024, 41088, 8320, 8320, 8320, 0]],
        '忸' : [12, 13, 0, -2, 12, [0, 10208, 8480, 8480, 41248, 41248, 45024, 41504, 41504, 8736, 8736, 12256, 0]],
        '忱' : [12, 13, 0, -2, 12, [256, 8448, 12256, 10528, 43296, 43296, 41344, 41600, 41600, 8864, 9376, 14560, 0]],
        '忝' : [12, 13, 0, -2, 12, [0, 896, 31744, 1024, 65504, 2560, 12672, 50272, 1024, 9536, 17568, 3072, 0]],
        '悳' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2112, 20416, 18496, 20416, 16384, 32736, 4672, 20512, 36736, 0]],
        '忿' : [12, 13, 0, -2, 12, [0, 5888, 8320, 16448, 49056, 2176, 4224, 25344, 1024, 4672, 20512, 36736, 0]],
        '怡' : [12, 13, 0, -2, 12, [0, 8448, 8704, 9248, 42976, 45056, 43008, 42976, 42016, 9248, 9248, 10208, 0]],
        '恠' : [12, 13, 0, -2, 12, [512, 8704, 12256, 8704, 41536, 42048, 42464, 44096, 46144, 9280, 9280, 10208, 0]],
        '怙' : [12, 13, 0, -2, 12, [128, 8320, 8320, 10208, 41088, 45184, 43136, 42976, 42016, 9248, 9248, 10208, 0]],
        '怐' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 10272, 43040, 42784, 42272, 42272, 42784, 8224, 8256, 8576, 0]],
        '怩' : [12, 13, 0, -2, 12, [0, 12256, 10272, 10272, 45024, 43008, 43520, 43616, 43904, 12800, 8736, 8672, 0]],
        '怎' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 18432, 20416, 34816, 36800, 2048, 1024, 4672, 20512, 36736, 0]],
        '怱' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 18720, 47392, 4000, 4640, 9408, 512, 4160, 20512, 36736, 0]],
        '怛' : [12, 13, 0, -2, 12, [0, 10208, 9248, 9248, 42976, 46112, 44064, 42016, 42976, 8192, 8192, 10208, 0]],
        '怕' : [12, 13, 0, -2, 12, [256, 8448, 8704, 10208, 42016, 46112, 44064, 42976, 42016, 9248, 9248, 10208, 0]],
        '怫' : [12, 13, 0, -2, 12, [640, 8832, 12256, 8864, 45024, 43648, 43648, 45024, 41632, 8864, 9408, 14464, 0]],
        '怦' : [12, 13, 0, -2, 12, [0, 12256, 8448, 8448, 42304, 43296, 41216, 41216, 45024, 8448, 8448, 8448, 0]],
        '怏' : [12, 13, 0, -2, 12, [256, 8448, 12256, 10528, 43296, 43296, 43296, 49120, 41216, 8832, 9280, 14368, 0]],
        '怺' : [12, 13, 0, -2, 12, [512, 8448, 8192, 12032, 41248, 48448, 42368, 42240, 42368, 10560, 12576, 8960, 0]],
        '恚' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 65504, 1024, 16256, 1024, 65504, 4672, 20512, 36736, 0]],
        '恁' : [12, 13, 0, -2, 12, [0, 12256, 8448, 16640, 57312, 16640, 16640, 20448, 1024, 4640, 20496, 36800, 0]],
        '恪' : [12, 13, 0, -2, 12, [512, 8704, 9184, 9248, 43584, 41344, 41536, 42016, 45024, 9248, 9248, 10208, 0]],
        '恷' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16640, 50048, 17728, 22816, 0, 1024, 4672, 20512, 36736, 0]],
        '恟' : [12, 13, 0, -2, 12, [2048, 10240, 12256, 12576, 43552, 42016, 43680, 45472, 41120, 16288, 8224, 8384, 0]],
        '恊' : [12, 13, 0, -2, 12, [512, 8704, 12224, 8768, 41536, 42048, 43136, 49120, 43680, 10912, 13088, 9824, 0]],
        '恆' : [12, 13, 0, -2, 12, [0, 12256, 8704, 8704, 41952, 42272, 42144, 43584, 43328, 8320, 8320, 12256, 0]],
        '恍' : [12, 13, 0, -2, 12, [256, 8448, 10528, 9536, 41216, 41216, 49120, 41600, 41600, 8832, 9376, 14560, 0]],
        '恣' : [12, 13, 0, -2, 12, [1024, 1024, 59360, 2336, 2304, 8832, 17472, 34848, 1024, 4672, 20512, 36736, 0]],
        '恃' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8448, 41216, 45024, 41024, 45024, 41024, 9280, 8768, 8384, 0]],
        '恤' : [12, 13, 0, -2, 12, [256, 8448, 8704, 12256, 43680, 43680, 43680, 43680, 43680, 10912, 10912, 16352, 0]],
        '恂' : [12, 13, 0, -2, 12, [2048, 10240, 12256, 10272, 45088, 44832, 43296, 44832, 43296, 12064, 8256, 8576, 0]],
        '恬' : [12, 13, 0, -2, 12, [0, 8416, 12032, 8448, 41216, 45024, 41216, 41216, 42976, 9248, 9248, 10208, 0]],
        '恫' : [12, 13, 0, -2, 12, [0, 12256, 10272, 10272, 43936, 43040, 43936, 43680, 43680, 11168, 10272, 10336, 0]],
        '恙' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 16256, 1024, 65504, 0, 1024, 4672, 20512, 36736, 0]],
        '悁' : [12, 13, 0, -2, 12, [0, 10208, 9248, 10208, 40960, 47072, 44064, 42976, 42016, 10208, 9248, 9312, 0]],
        '悍' : [12, 13, 0, -2, 12, [0, 10208, 9248, 10208, 42016, 47072, 40960, 42976, 41088, 12256, 8320, 8320, 0]],
        '惧' : [12, 13, 0, -2, 12, [0, 10208, 9248, 10208, 42016, 47072, 42016, 42016, 45024, 8192, 8768, 11296, 0]],
        '悃' : [12, 13, 0, -2, 12, [0, 12256, 10528, 10528, 45024, 43296, 43808, 44448, 43360, 10528, 10272, 12256, 0]],
        '悚' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8448, 45024, 43296, 43296, 45024, 41728, 9536, 14624, 8448, 0]],
        '悄' : [12, 13, 0, -2, 12, [256, 8448, 9536, 10528, 41216, 47072, 42016, 42976, 42016, 10208, 9248, 9312, 0]],
        '悛' : [12, 13, 0, -2, 12, [512, 8768, 9280, 16352, 41600, 41632, 44128, 41920, 44096, 9344, 9088, 15456, 0]],
        '悖' : [12, 13, 0, -2, 12, [256, 8448, 12256, 8448, 45024, 43040, 42944, 41088, 41216, 12256, 8448, 8960, 0]],
        '悗' : [12, 13, 0, -2, 12, [0, 10112, 10368, 12544, 45024, 43296, 43296, 45024, 41600, 8832, 9376, 14560, 0]],
        '悒' : [12, 13, 0, -2, 12, [0, 10176, 9280, 10176, 40960, 45024, 43296, 43296, 45024, 10240, 10272, 12256, 0]],
        '悧' : [12, 13, 0, -2, 12, [0, 8992, 15520, 9376, 42144, 49056, 42144, 44192, 46752, 9632, 9248, 9312, 0]],
        '悋' : [12, 13, 0, -2, 12, [128, 8320, 12256, 9248, 41536, 41344, 44640, 40960, 42976, 9248, 9248, 10208, 0]],
        '惡' : [12, 13, 0, -2, 12, [0, 65504, 2560, 31680, 16448, 31680, 2560, 65504, 1024, 4672, 20512, 36736, 0]],
        '悸' : [12, 13, 0, -2, 12, [0, 12256, 8448, 16352, 42304, 43296, 47040, 41088, 41216, 12256, 8448, 8960, 0]],
        '惠' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 17472, 32704, 17472, 32704, 1056, 65504, 4672, 20512, 36736, 0]],
        '惓' : [12, 13, 0, -2, 12, [256, 9536, 10528, 12256, 41536, 45024, 42048, 45024, 46144, 9664, 9248, 10208, 0]],
        '悴' : [12, 13, 0, -2, 12, [256, 8448, 16352, 9280, 42048, 43680, 43296, 41216, 49120, 8448, 8448, 8448, 0]],
        '忰' : [12, 13, 0, -2, 12, [512, 8704, 12224, 8768, 41536, 42080, 42240, 41216, 49120, 8448, 8448, 8448, 0]],
        '悽' : [12, 13, 0, -2, 12, [256, 8448, 16352, 8480, 45024, 41248, 42976, 41472, 49120, 9280, 10112, 14432, 0]],
        '惆' : [12, 13, 0, -2, 12, [0, 12256, 10528, 11168, 43296, 43936, 43296, 43040, 43936, 10912, 11168, 10336, 0]],
        '悵' : [12, 13, 0, -2, 12, [0, 10208, 9216, 10176, 41984, 42944, 46080, 45024, 42240, 9376, 9280, 10016, 0]],
        '惘' : [12, 13, 0, -2, 12, [0, 12256, 10272, 10912, 43680, 45024, 43296, 45024, 43552, 10784, 11168, 10336, 0]],
        '慍' : [12, 13, 0, -2, 12, [0, 10176, 9536, 9920, 42048, 42944, 40960, 45024, 43680, 10912, 10912, 16352, 0]],
        '愕' : [12, 13, 0, -2, 12, [0, 12000, 10912, 12000, 40960, 45024, 40960, 49120, 41984, 12256, 10272, 8640, 0]],
        '愆' : [12, 13, 0, -2, 12, [0, 21472, 34816, 9184, 20544, 50240, 18496, 20928, 1024, 4672, 20512, 36736, 0]],
        '惶' : [12, 13, 0, -2, 12, [128, 8448, 10208, 9248, 42976, 46112, 45024, 41088, 42976, 8320, 8320, 12256, 0]],
        '惷' : [12, 13, 0, -2, 12, [1024, 1024, 32736, 1024, 16320, 2304, 65504, 20800, 40736, 4608, 20544, 36640, 0]],
        '愀' : [12, 13, 0, -2, 12, [128, 9088, 15488, 9376, 42144, 49056, 42112, 42176, 44352, 13632, 9760, 9248, 0]],
        '惴' : [12, 13, 0, -2, 12, [256, 10528, 10528, 12256, 40960, 45024, 41472, 45024, 43680, 10912, 10912, 10336, 0]],
        '惺' : [12, 13, 0, -2, 12, [0, 10208, 9248, 10208, 46112, 45024, 42112, 42976, 43136, 10208, 8320, 12256, 0]],
        '愃' : [12, 13, 0, -2, 12, [256, 8448, 12256, 10272, 42976, 42016, 47072, 44064, 42016, 10208, 8192, 12256, 0]],
        '愡' : [12, 13, 0, -2, 12, [1024, 9216, 12256, 12960, 44704, 42400, 43296, 41024, 8704, 2368, 10272, 18304, 0]],
        '惻' : [12, 13, 0, -2, 12, [0, 11808, 10912, 11936, 43680, 44704, 43680, 43680, 44704, 8224, 10784, 12640, 0]],
        '惱' : [12, 13, 0, -2, 12, [0, 10528, 12864, 9344, 45632, 43296, 45024, 43680, 43296, 10912, 10272, 12256, 0]],
        '愍' : [12, 13, 0, -2, 12, [512, 64000, 35808, 64064, 38208, 64832, 37248, 51808, 1024, 4672, 20512, 36736, 0]],
        '愎' : [12, 13, 0, -2, 12, [512, 8704, 10208, 11296, 42976, 46112, 45024, 41472, 42976, 10784, 8640, 11808, 0]],
        '慇' : [12, 13, 0, -2, 12, [0, 64448, 33344, 62528, 38880, 64064, 35200, 45664, 1024, 4672, 20512, 36736, 0]],
        '愾' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 10240, 43968, 40960, 44992, 43584, 41536, 12224, 9760, 10912, 0]],
        '愨' : [12, 13, 0, -2, 12, [8192, 9152, 64064, 9280, 64480, 35392, 384, 29280, 1024, 4672, 20512, 36736, 0]],
        '愧' : [12, 13, 0, -2, 12, [256, 8704, 12256, 10528, 45024, 43296, 43296, 45024, 42304, 9536, 10528, 10720, 0]],
        '慊' : [12, 13, 0, -2, 12, [576, 9856, 15328, 8832, 45024, 41632, 49120, 41632, 45024, 8832, 9920, 10912, 0]],
        '愿' : [12, 13, 0, -2, 12, [0, 32736, 20544, 24512, 20544, 24512, 33280, 37440, 9760, 4096, 20544, 36640, 0]],
        '愼' : [12, 13, 0, -2, 12, [1024, 9312, 10112, 9248, 41952, 43552, 44000, 43552, 45024, 8192, 8768, 11296, 0]],
        '愬' : [12, 13, 0, -2, 12, [4096, 21472, 64032, 9184, 43552, 64480, 8736, 17504, 512, 4160, 20512, 36736, 0]],
        '愴' : [12, 13, 0, -2, 12, [0, 9152, 9248, 12256, 42016, 47072, 42016, 42976, 41984, 11232, 10784, 9184, 0]],
        '愽' : [12, 13, 0, -2, 12, [256, 8480, 12256, 8448, 45024, 43296, 45024, 43296, 41024, 16352, 9280, 8896, 0]],
        '慂' : [12, 13, 0, -2, 12, [0, 20416, 8320, 36832, 18720, 4064, 10528, 18784, 33280, 4416, 20512, 36736, 0]],
        '慄' : [12, 13, 0, -2, 12, [0, 12256, 8832, 12256, 43680, 43680, 45024, 41216, 45024, 8960, 9536, 14624, 0]],
        '慳' : [12, 13, 0, -2, 12, [0, 12256, 10912, 12192, 43328, 44864, 43680, 44960, 41216, 10208, 8448, 12256, 0]],
        '慷' : [12, 13, 0, -2, 12, [128, 8320, 12256, 10400, 45024, 43168, 44000, 43136, 46496, 8896, 11424, 8576, 0]],
        '慘' : [12, 13, 0, -2, 12, [512, 9280, 12256, 9344, 43296, 45024, 41600, 42080, 47488, 9760, 8384, 12032, 0]],
        '慙' : [12, 13, 0, -2, 12, [4096, 4576, 65280, 21984, 32064, 21824, 65088, 4672, 1024, 4672, 20512, 36736, 0]],
        '慚' : [12, 13, 0, -2, 12, [2048, 10720, 16128, 10496, 49120, 43296, 48928, 43296, 48928, 10272, 16160, 10272, 0]],
        '慫' : [12, 13, 0, -2, 12, [1088, 9280, 19648, 37664, 8704, 27584, 44544, 12768, 1024, 4672, 20512, 36736, 0]],
        '慴' : [12, 13, 0, -2, 12, [0, 16096, 10912, 9824, 43680, 45920, 41472, 45024, 43040, 12256, 10272, 12256, 0]],
        '慯' : [12, 13, 0, -2, 12, [1024, 9216, 12256, 14400, 44992, 43072, 45024, 41984, 45024, 9376, 10528, 8896, 0]],
        '慥' : [12, 13, 0, -2, 12, [64, 10560, 9536, 12768, 43584, 41952, 44096, 42464, 42272, 9696, 11264, 13280, 0]],
        '慱' : [12, 13, 0, -2, 12, [256, 8448, 16352, 10528, 45024, 43296, 45024, 41248, 49120, 8256, 9280, 8896, 0]],
        '慟' : [12, 13, 0, -2, 12, [128, 12160, 8832, 16352, 43680, 44960, 43680, 44960, 41632, 12192, 8736, 16224, 0]],
        '慝' : [12, 13, 0, -2, 12, [0, 65504, 35072, 49088, 35072, 49088, 38016, 42112, 65504, 4608, 20544, 36640, 0]],
        '慓' : [12, 13, 0, -2, 12, [0, 12256, 8832, 12256, 43680, 45024, 40960, 45024, 40960, 12256, 9536, 11040, 0]],
        '慵' : [12, 13, 0, -2, 12, [256, 8448, 12256, 10528, 45024, 43296, 45024, 43264, 45024, 14624, 12256, 10528, 0]],
        '憙' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 4352, 65504, 8320, 16256, 4608, 20544, 36640, 0]],
        '憖' : [12, 13, 0, -2, 12, [4224, 4256, 65504, 21632, 39552, 12416, 21824, 37408, 1024, 4672, 20512, 36736, 0]],
        '憇' : [12, 13, 0, -2, 12, [576, 64064, 10208, 64064, 9152, 64064, 35392, 64448, 1024, 4672, 20512, 36736, 0]],
        '憬' : [12, 13, 0, -2, 12, [0, 10208, 9248, 10208, 42016, 42976, 45184, 45024, 41504, 9184, 9344, 10656, 0]],
        '憔' : [12, 13, 0, -2, 12, [576, 8768, 10208, 9344, 45024, 42112, 42976, 42112, 42976, 8192, 10912, 12960, 0]],
        '憚' : [12, 13, 0, -2, 12, [0, 12000, 10912, 10912, 45024, 43296, 45024, 43296, 45024, 8448, 16352, 8448, 0]],
        '憊' : [12, 13, 0, -2, 12, [1152, 9344, 16352, 17536, 53216, 22816, 28640, 18720, 19808, 4608, 20544, 36640, 0]],
        '憑' : [12, 13, 0, -2, 12, [0, 20448, 10496, 4032, 2304, 20448, 36896, 9504, 1216, 4096, 20544, 36640, 0]],
        '憫' : [12, 13, 0, -2, 12, [0, 12000, 10912, 12000, 43680, 44768, 43296, 45024, 43680, 10528, 10912, 11360, 0]],
        '憮' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 11584, 42304, 49120, 42304, 42304, 45024, 8192, 10912, 12960, 0]],
        '懌' : [12, 13, 0, -2, 12, [0, 12256, 10912, 10912, 45024, 41216, 45024, 42048, 49120, 8448, 12256, 8448, 0]],
        '懊' : [12, 13, 0, -2, 12, [256, 8704, 12256, 11616, 43936, 44384, 43936, 44384, 41216, 16352, 8768, 11296, 0]],
        '應' : [12, 13, 0, -2, 12, [512, 512, 32736, 19008, 22496, 29824, 22496, 21632, 38880, 35072, 10304, 18208, 0]],
        '懷' : [12, 13, 0, -2, 12, [256, 8448, 16352, 10912, 43680, 45024, 41216, 44384, 41600, 15520, 9280, 12064, 0]],
        '懈' : [12, 13, 0, -2, 12, [0, 12256, 12960, 9376, 48992, 43584, 49120, 43584, 43584, 16352, 8768, 9792, 0]],
        '懃' : [12, 13, 0, -2, 12, [18560, 18560, 65504, 18592, 64672, 42272, 64800, 8224, 64704, 4608, 20544, 36640, 0]],
        '懆' : [12, 13, 0, -2, 12, [0, 10176, 9280, 12256, 43680, 43680, 44768, 41216, 45024, 8960, 9536, 14624, 0]],
        '憺' : [12, 13, 0, -2, 12, [0, 10176, 10368, 16352, 43328, 44640, 43136, 45024, 43008, 14304, 9248, 10208, 0]],
        '懋' : [12, 13, 0, -2, 12, [0, 12160, 8832, 65504, 9856, 27328, 45728, 8832, 1024, 4672, 20512, 36736, 0]],
        '罹' : [12, 13, 0, -2, 12, [0, 32736, 18720, 32736, 8768, 10208, 44160, 47072, 42112, 42976, 9344, 10208, 0]],
        '懍' : [12, 13, 0, -2, 12, [256, 8448, 16352, 10912, 43936, 43040, 45024, 41216, 45024, 8960, 9536, 14624, 0]],
        '懦' : [12, 13, 0, -2, 12, [0, 12256, 8448, 12256, 43296, 43872, 41216, 45024, 41472, 12256, 10912, 10912, 0]],
        '懣' : [12, 13, 0, -2, 12, [1152, 17536, 16352, 33920, 24544, 5408, 6816, 20832, 33792, 4672, 20512, 36736, 0]],
        '懶' : [12, 13, 0, -2, 12, [2048, 11232, 16032, 10592, 48640, 44000, 48672, 44000, 44576, 15328, 10560, 11808, 0]],
        '懺' : [12, 13, 0, -2, 12, [2688, 10912, 13728, 10368, 49120, 43648, 48032, 43680, 47936, 10816, 10912, 16160, 0]],
        '懴' : [12, 13, 0, -2, 12, [1152, 9376, 16288, 9344, 49120, 43648, 48032, 43680, 47936, 10816, 10912, 16160, 0]],
        '懿' : [12, 13, 0, -2, 12, [8448, 8448, 64992, 8864, 63616, 36160, 8736, 63744, 35488, 64032, 22016, 64448, 0]],
        '懽' : [12, 13, 0, -2, 12, [1152, 9344, 16352, 10912, 44768, 42112, 45024, 47360, 45024, 10496, 10496, 12256, 0]],
        '懼' : [12, 13, 0, -2, 12, [0, 12000, 10912, 12000, 43680, 44768, 42240, 45024, 47360, 12256, 10496, 12256, 0]],
        '懾' : [12, 13, 0, -2, 12, [0, 12256, 9280, 10176, 42048, 49120, 43680, 44768, 43680, 12000, 15264, 8736, 0]],
        '戀' : [12, 13, 0, -2, 12, [1024, 24384, 41120, 20032, 61920, 20032, 60128, 20032, 1024, 4672, 20512, 36736, 0]],
        '戈' : [12, 13, 0, -2, 12, [2048, 2176, 2112, 2048, 65504, 1024, 1088, 1152, 768, 1568, 6432, 57568, 0]],
        '戉' : [12, 13, 0, -2, 12, [512, 544, 32736, 16896, 16960, 16960, 17024, 20736, 24864, 17056, 35936, 12320, 0]],
        '戍' : [12, 13, 0, -2, 12, [512, 544, 32736, 16896, 16960, 16960, 21120, 18688, 16672, 17056, 35936, 45088, 0]],
        '戌' : [12, 13, 0, -2, 12, [512, 544, 32736, 16896, 16960, 16960, 31360, 16640, 16672, 17056, 35936, 45088, 0]],
        '戔' : [12, 13, 0, -2, 12, [1024, 9184, 65024, 4480, 3616, 61920, 1024, 9184, 65024, 4480, 3616, 61920, 0]],
        '戛' : [12, 13, 0, -2, 12, [0, 65504, 1024, 16256, 8320, 16256, 8320, 4224, 65504, 4224, 3872, 61664, 0]],
        '戞' : [12, 13, 0, -2, 12, [0, 65504, 1024, 16256, 8320, 16256, 8320, 59616, 49056, 2176, 1824, 63712, 0]],
        '戡' : [12, 13, 0, -2, 12, [128, 18592, 65184, 18560, 31712, 18560, 18592, 65184, 43072, 52416, 33568, 64544, 0]],
        '截' : [12, 13, 0, -2, 12, [4224, 4256, 65184, 4224, 65504, 9344, 32416, 51360, 32320, 18496, 18592, 32544, 0]],
        '戮' : [12, 13, 0, -2, 12, [128, 65184, 43680, 30336, 44000, 8832, 14496, 50336, 6720, 24640, 6304, 58144, 0]],
        '戰' : [12, 13, 0, -2, 12, [128, 64672, 42144, 64640, 992, 64640, 42144, 64672, 42048, 42048, 64672, 8992, 0]],
        '戲' : [12, 13, 0, -2, 12, [4224, 7840, 4256, 32384, 21472, 32384, 20640, 32672, 41536, 48704, 5280, 32544, 0]],
        '戳' : [12, 13, 0, -2, 12, [128, 65184, 43680, 30336, 44000, 9344, 32416, 51360, 32320, 18496, 18592, 32544, 0]],
        '扁' : [12, 13, 0, -2, 12, [0, 65504, 0, 32704, 16448, 32704, 16384, 32736, 25888, 49120, 42272, 9568, 0]],
        '扎' : [12, 13, 0, -2, 12, [0, 8704, 8704, 64000, 8704, 8704, 8704, 12800, 57888, 8736, 8736, 25056, 0]],
        '扞' : [12, 13, 0, -2, 12, [0, 20448, 16640, 57600, 16640, 16640, 20448, 24832, 49408, 16640, 16640, 49408, 0]],
        '扣' : [12, 13, 0, -2, 12, [0, 10208, 9248, 64544, 9248, 9248, 9248, 13344, 58400, 9248, 9248, 26592, 0]],
        '扛' : [12, 13, 0, -2, 12, [0, 18400, 16512, 61568, 16512, 16512, 16512, 24704, 49280, 16512, 16512, 51168, 0]],
        '扠' : [12, 13, 0, -2, 12, [0, 20448, 16416, 61984, 16672, 16416, 16960, 24896, 49280, 16768, 17984, 55328, 0]],
        '扨' : [12, 13, 0, -2, 12, [0, 20448, 16928, 59936, 17952, 16928, 17184, 25760, 50272, 18464, 18496, 49536, 0]],
        '扼' : [12, 13, 0, -2, 12, [0, 20448, 18432, 63488, 19392, 19008, 19008, 27200, 51904, 20992, 21024, 50144, 0]],
        '抂' : [12, 13, 0, -2, 12, [0, 20448, 16640, 61696, 16640, 16640, 20448, 24832, 49408, 16640, 16640, 53216, 0]],
        '抉' : [12, 13, 0, -2, 12, [256, 16640, 18400, 61728, 16672, 16672, 20448, 24832, 49408, 17024, 17472, 55328, 0]],
        '找' : [12, 13, 0, -2, 12, [256, 8512, 8480, 61696, 12256, 8448, 8512, 12608, 57472, 8320, 9024, 27680, 0]],
        '抒' : [12, 13, 0, -2, 12, [0, 20416, 16448, 62080, 16640, 20448, 16672, 24864, 49472, 16640, 16640, 49920, 0]],
        '抓' : [12, 13, 0, -2, 12, [0, 20448, 19072, 64128, 19072, 19072, 19008, 27200, 51776, 18976, 21024, 53280, 0]],
        '抖' : [12, 13, 0, -2, 12, [0, 16448, 16960, 61760, 16448, 17472, 16960, 24672, 49600, 20032, 16448, 49216, 0]],
        '拔' : [12, 13, 0, -2, 12, [512, 16928, 16928, 65504, 16896, 16928, 17696, 25920, 51328, 18560, 17216, 52256, 0]],
        '抃' : [12, 13, 0, -2, 12, [256, 16640, 16640, 61408, 16640, 16640, 16768, 24896, 49440, 16640, 16640, 49408, 0]],
        '抔' : [12, 13, 0, -2, 12, [0, 20448, 16512, 61568, 16640, 16640, 17216, 25920, 51488, 16672, 16640, 49408, 0]],
        '拗' : [12, 13, 0, -2, 12, [128, 18560, 18560, 62432, 21152, 19104, 17568, 25888, 51488, 18976, 21024, 57024, 0]],
        '拑' : [12, 13, 0, -2, 12, [1024, 17440, 17440, 65504, 17440, 17440, 17440, 26592, 50208, 17440, 17440, 51168, 0]],
        '抻' : [12, 13, 0, -2, 12, [256, 16640, 20448, 63776, 18720, 20448, 18720, 26912, 53216, 16640, 16640, 49408, 0]],
        '拏' : [12, 13, 0, -2, 12, [8192, 9184, 64544, 18720, 18752, 12480, 57120, 1024, 16256, 1024, 65504, 1024, 0]],
        '拿' : [12, 13, 0, -2, 12, [0, 7936, 8320, 32704, 37152, 4352, 32704, 1024, 16256, 1024, 65504, 1024, 0]],
        '拆' : [12, 13, 0, -2, 12, [0, 16576, 18176, 62464, 17408, 18400, 17536, 25984, 50368, 18592, 18560, 49280, 0]],
        '擔' : [12, 13, 0, -2, 12, [0, 18368, 18560, 65504, 18752, 20064, 18560, 28640, 51200, 22496, 21536, 51168, 0]],
        '拈' : [12, 13, 0, -2, 12, [256, 8448, 8448, 63968, 8448, 8448, 8448, 14304, 58400, 9248, 9248, 26592, 0]],
        '拜' : [12, 13, 0, -2, 12, [0, 61408, 16640, 16640, 63456, 16640, 16640, 61696, 20448, 16640, 33024, 33024, 0]],
        '拌' : [12, 13, 0, -2, 12, [256, 16640, 17728, 59680, 16640, 20448, 16640, 24832, 53216, 16640, 16640, 49408, 0]],
        '拊' : [12, 13, 0, -2, 12, [64, 17472, 17472, 60384, 22592, 18496, 19008, 26944, 51264, 18496, 18496, 51392, 0]],
        '拂' : [12, 13, 0, -2, 12, [640, 17024, 24544, 58016, 20448, 19072, 19072, 28640, 49824, 17056, 17600, 55424, 0]],
        '拇' : [12, 13, 0, -2, 12, [0, 20416, 18496, 59968, 18752, 24544, 18496, 19008, 53568, 20544, 24544, 49280, 0]],
        '抛' : [12, 13, 0, -2, 12, [2176, 18560, 18560, 65504, 19104, 19104, 19232, 27168, 51904, 18944, 21024, 54240, 0]],
        '拉' : [12, 13, 0, -2, 12, [256, 16640, 16640, 65504, 16384, 17472, 17472, 25664, 50304, 17536, 16512, 57312, 0]],
        '挌' : [12, 13, 0, -2, 12, [512, 16896, 17376, 62496, 19008, 16768, 16960, 25632, 53216, 17440, 17440, 51168, 0]],
        '拮' : [12, 13, 0, -2, 12, [128, 16512, 20448, 61568, 16512, 18400, 16384, 24576, 51168, 17440, 17440, 51168, 0]],
        '拱' : [12, 13, 0, -2, 12, [1152, 17536, 17536, 65504, 17536, 17536, 17536, 25728, 57312, 16384, 18496, 53280, 0]],
        '挧' : [12, 13, 0, -2, 12, [0, 24544, 16928, 57888, 18016, 19104, 16928, 26208, 51872, 21280, 16928, 50784, 0]],
        '挂' : [12, 13, 0, -2, 12, [256, 16640, 18400, 61696, 16640, 20448, 16640, 24832, 51168, 16640, 16640, 53216, 0]],
        '挈' : [12, 13, 0, -2, 12, [8192, 15328, 57632, 14624, 57888, 12832, 61376, 1024, 16256, 1024, 65504, 1024, 0]],
        '拯' : [12, 13, 0, -2, 12, [0, 20416, 16512, 57600, 23840, 17728, 17792, 26944, 51488, 17152, 16384, 57312, 0]],
        '拵' : [12, 13, 0, -2, 12, [512, 16896, 24544, 58368, 18368, 18496, 18560, 22656, 60384, 18560, 18560, 51584, 0]],
        '捐' : [12, 13, 0, -2, 12, [0, 10208, 9248, 63456, 8192, 10208, 9248, 14304, 58400, 10208, 9248, 25696, 0]],
        '挾' : [12, 13, 0, -2, 12, [256, 16640, 20448, 57600, 17728, 17728, 19360, 26912, 49408, 17024, 17472, 55328, 0]],
        '捍' : [12, 13, 0, -2, 12, [0, 10208, 9248, 63456, 9248, 10208, 8192, 14304, 57472, 12256, 8320, 24704, 0]],
        '搜' : [12, 13, 0, -2, 12, [256, 17664, 18784, 59680, 19808, 18720, 20448, 24832, 53216, 17472, 17280, 56416, 0]],
        '捏' : [12, 13, 0, -2, 12, [0, 10208, 9248, 63456, 9248, 9248, 10208, 12416, 59360, 8320, 8320, 28640, 0]],
        '掖' : [12, 13, 0, -2, 12, [256, 16640, 24544, 59648, 18880, 21056, 21056, 30016, 54592, 20608, 20864, 54880, 0]],
        '掎' : [12, 13, 0, -2, 12, [256, 16640, 20448, 62080, 17472, 18464, 20448, 24608, 53024, 18720, 20256, 49248, 0]],
        '掀' : [12, 13, 0, -2, 12, [256, 23808, 20960, 62112, 24224, 21664, 21632, 29824, 54592, 21824, 22048, 50208, 0]],
        '掫' : [12, 13, 0, -2, 12, [0, 24544, 18944, 65504, 18976, 19104, 20128, 27296, 51776, 20032, 23200, 49952, 0]],
        '捶' : [12, 13, 0, -2, 12, [0, 20448, 16640, 61408, 17696, 24544, 17696, 25888, 53216, 16640, 16640, 53216, 0]],
        '掣' : [12, 13, 0, -2, 12, [4096, 37152, 64800, 37152, 64800, 37920, 48992, 1024, 16256, 1024, 65504, 1024, 0]],
        '掏' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 59424, 20384, 16928, 20384, 25120, 51872, 20384, 16416, 49344, 0]],
        '掉' : [12, 13, 0, -2, 12, [512, 16896, 17376, 57856, 20448, 18464, 20448, 26656, 53216, 16640, 24544, 49408, 0]],
        '掟' : [12, 13, 0, -2, 12, [256, 16640, 20448, 59424, 16384, 20448, 16640, 24832, 51680, 18688, 20224, 61664, 0]],
        '掵' : [12, 13, 0, -2, 12, [0, 18304, 18496, 61472, 20448, 16384, 24032, 29984, 54560, 21792, 23904, 49408, 0]],
        '捫' : [12, 13, 0, -2, 12, [0, 20192, 19104, 61152, 19104, 20192, 18464, 26656, 51232, 18464, 18464, 51296, 0]],
        '捩' : [12, 13, 0, -2, 12, [0, 24544, 16384, 61408, 18464, 20448, 18688, 28640, 53504, 20864, 16960, 52256, 0]],
        '掾' : [12, 13, 0, -2, 12, [0, 18400, 17472, 61376, 18560, 24544, 17664, 25504, 52416, 17312, 23712, 49920, 0]],
        '揩' : [12, 13, 0, -2, 12, [128, 18560, 20192, 63616, 18592, 20192, 16640, 28640, 51232, 20448, 18464, 53216, 0]],
        '揀' : [12, 13, 0, -2, 12, [256, 16640, 24544, 57600, 20448, 18720, 19808, 26912, 53216, 17152, 17728, 55584, 0]],
        '揆' : [12, 13, 0, -2, 12, [0, 24192, 17056, 60224, 17536, 18496, 22432, 16640, 61408, 16640, 18048, 55392, 0]],
        '揣' : [12, 13, 0, -2, 12, [256, 18720, 18720, 61408, 16384, 20448, 16640, 28640, 51872, 19104, 19104, 51296, 0]],
        '揉' : [12, 13, 0, -2, 12, [0, 18368, 16512, 61696, 20448, 17184, 19776, 24832, 53216, 17152, 17728, 55584, 0]],
        '插' : [12, 13, 0, -2, 12, [0, 20448, 16640, 65504, 16640, 17664, 18784, 26912, 52576, 18720, 18720, 53216, 0]],
        '揶' : [12, 13, 0, -2, 12, [0, 24544, 19104, 61088, 19104, 19136, 20128, 27296, 51872, 20128, 23232, 49792, 0]],
        '揄' : [12, 13, 0, -2, 12, [0, 24448, 17472, 59424, 22496, 16384, 20128, 27296, 53056, 19264, 20128, 51872, 0]],
        '搖' : [12, 13, 0, -2, 12, [0, 17376, 19488, 57664, 17792, 19968, 17376, 26880, 53216, 16640, 18720, 53216, 0]],
        '搴' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 16256, 4352, 65504, 9280, 57248, 1024, 65504, 1024, 0]],
        '搆' : [12, 13, 0, -2, 12, [1152, 17536, 24544, 58496, 24544, 17536, 20448, 26912, 53216, 18720, 24544, 51232, 0]],
        '搓' : [12, 13, 0, -2, 12, [1088, 17536, 20448, 61696, 18400, 16640, 20448, 25088, 51168, 18560, 16512, 53216, 0]],
        '搦' : [12, 13, 0, -2, 12, [0, 24032, 17440, 58400, 24032, 20736, 24032, 25760, 52576, 21664, 17696, 55488, 0]],
        '搶' : [12, 13, 0, -2, 12, [0, 9152, 9248, 65504, 9248, 10208, 9248, 14304, 58368, 11232, 10784, 25568, 0]],
        '攝' : [12, 13, 0, -2, 12, [0, 20448, 17472, 63424, 17472, 24544, 19104, 28384, 51872, 20192, 23456, 49696, 0]],
        '搗' : [12, 13, 0, -2, 12, [256, 16896, 20416, 63552, 20416, 18496, 20448, 26624, 53216, 17440, 21792, 57184, 0]],
        '搨' : [12, 13, 0, -2, 12, [0, 20416, 18496, 61376, 18496, 20416, 16384, 24544, 60064, 22368, 19104, 54112, 0]],
        '搏' : [12, 13, 0, -2, 12, [256, 16672, 24544, 57600, 20448, 18720, 20448, 26912, 49216, 24544, 17472, 49856, 0]],
        '摧' : [12, 13, 0, -2, 12, [256, 18720, 18720, 61408, 17472, 20448, 22656, 20448, 59520, 20448, 18560, 53216, 0]],
        '摯' : [12, 13, 0, -2, 12, [8448, 8448, 65472, 18752, 64320, 8640, 32352, 1024, 16256, 1024, 65504, 1024, 0]],
        '摶' : [12, 13, 0, -2, 12, [256, 16640, 24544, 59680, 20448, 18720, 20448, 24864, 57312, 16448, 17472, 49856, 0]],
        '摎' : [12, 13, 0, -2, 12, [0, 24544, 19104, 59232, 19104, 21024, 17280, 27712, 49568, 17920, 16576, 52992, 0]],
        '攪' : [12, 13, 0, -2, 12, [0, 20192, 18720, 61152, 18720, 24544, 21568, 18368, 58432, 18368, 17568, 55520, 0]],
        '撕' : [12, 13, 0, -2, 12, [0, 21600, 21888, 65280, 21984, 23872, 21824, 21824, 65344, 16704, 21568, 57920, 0]],
        '撓' : [12, 13, 0, -2, 12, [256, 16640, 20448, 57600, 24544, 17472, 24288, 25664, 57312, 17024, 17568, 55520, 0]],
        '撥' : [12, 13, 0, -2, 12, [0, 24192, 19104, 58688, 18560, 24032, 17728, 32352, 54208, 23872, 17600, 56096, 0]],
        '撩' : [12, 13, 0, -2, 12, [256, 16640, 24544, 59968, 18400, 19488, 22496, 17440, 59360, 16640, 18752, 54048, 0]],
        '撈' : [12, 13, 0, -2, 12, [1152, 21920, 22176, 58496, 18752, 24544, 21024, 16896, 61408, 16928, 17440, 55488, 0]],
        '撼' : [12, 13, 0, -2, 12, [128, 16544, 24544, 61568, 22432, 21824, 22336, 20640, 53504, 17472, 21536, 58240, 0]],
        '據' : [12, 13, 0, -2, 12, [512, 17344, 16896, 61408, 18720, 20416, 18688, 27616, 52352, 21408, 23744, 49952, 0]],
        '擒' : [12, 13, 0, -2, 12, [0, 18368, 18720, 65504, 18720, 19104, 20448, 24832, 53216, 18976, 19360, 51232, 0]],
        '擅' : [12, 13, 0, -2, 12, [256, 16640, 24544, 60064, 19360, 18464, 20448, 25664, 51136, 17472, 17472, 57312, 0]],
        '擇' : [12, 13, 0, -2, 12, [0, 20448, 19104, 64160, 20448, 16640, 20448, 25664, 57312, 16640, 20448, 49408, 0]],
        '撻' : [12, 13, 0, -2, 12, [128, 20608, 19424, 57472, 22496, 16672, 18400, 22656, 60384, 18560, 19456, 62432, 0]],
        '擘' : [12, 13, 0, -2, 12, [256, 61696, 38880, 62016, 36832, 61696, 40928, 58368, 16256, 1024, 65504, 1024, 0]],
        '擂' : [12, 13, 0, -2, 12, [0, 20448, 16640, 65504, 18720, 19296, 16640, 28640, 51488, 20448, 18720, 53216, 0]],
        '擱' : [12, 13, 0, -2, 12, [0, 24288, 21152, 65248, 21152, 24288, 21408, 31904, 54048, 23776, 21664, 55200, 0]],
        '擧' : [12, 13, 0, -2, 12, [2048, 61152, 34848, 61152, 33312, 65504, 9280, 57248, 1024, 65504, 1024, 7168, 0]],
        '舉' : [12, 13, 0, -2, 12, [2048, 61152, 34848, 61152, 33312, 65504, 9280, 57248, 1024, 65504, 1024, 1024, 0]],
        '擠' : [12, 13, 0, -2, 12, [512, 16896, 32736, 50464, 32192, 19104, 23264, 16928, 61408, 18464, 20448, 55328, 0]],
        '擡' : [12, 13, 0, -2, 12, [256, 16640, 24544, 57600, 20416, 18496, 24544, 29216, 54432, 20416, 16640, 57312, 0]],
        '抬' : [12, 13, 0, -2, 12, [256, 8448, 8736, 62496, 10208, 8192, 8192, 14304, 58400, 9248, 9248, 26592, 0]],
        '擣' : [12, 13, 0, -2, 12, [256, 16640, 20448, 57600, 24544, 16416, 20416, 24832, 57312, 18976, 19104, 52832, 0]],
        '擯' : [12, 13, 0, -2, 12, [512, 16896, 24544, 61472, 20448, 21056, 20416, 22592, 61376, 18496, 20416, 61536, 0]],
        '攬' : [12, 13, 0, -2, 12, [256, 24320, 21984, 65280, 21984, 24224, 18912, 28480, 51264, 20416, 17568, 55520, 0]],
        '擶' : [12, 13, 0, -2, 12, [2176, 18560, 20192, 62784, 17536, 24544, 21024, 32416, 53920, 24224, 21024, 54880, 0]],
        '擴' : [12, 13, 0, -2, 12, [128, 16512, 24544, 62016, 24544, 21056, 22496, 29856, 55264, 21664, 18400, 55328, 0]],
        '擲' : [12, 13, 0, -2, 12, [0, 22240, 24992, 24224, 59552, 24256, 23200, 22176, 53920, 32672, 19136, 61824, 0]],
        '擺' : [12, 13, 0, -2, 12, [0, 24544, 21664, 65504, 18560, 21216, 24192, 29280, 56960, 21216, 24192, 53984, 0]],
        '攀' : [12, 13, 0, -2, 12, [8320, 10880, 62944, 27328, 46496, 10880, 65504, 9280, 57248, 1024, 65504, 1024, 0]],
        '擽' : [12, 13, 0, -2, 12, [256, 19008, 22432, 60608, 22432, 23776, 18304, 24832, 57312, 17152, 17728, 55584, 0]],
        '攘' : [12, 13, 0, -2, 12, [256, 16640, 24544, 60064, 20192, 17024, 24544, 25216, 53216, 17056, 23616, 50976, 0]],
        '攜' : [12, 13, 0, -2, 12, [256, 20768, 24544, 58496, 20448, 22784, 20448, 26880, 53216, 19104, 19360, 51296, 0]],
        '攅' : [12, 13, 0, -2, 12, [1088, 17472, 24288, 58432, 24288, 17472, 24544, 25664, 51136, 17472, 18368, 55328, 0]],
        '攤' : [12, 13, 0, -2, 12, [256, 18752, 24384, 59872, 24384, 21984, 24384, 25920, 57312, 17728, 19264, 53728, 0]],
        '攣' : [12, 13, 0, -2, 12, [1024, 17472, 44704, 16448, 44704, 19008, 61152, 17472, 16256, 1024, 65504, 1024, 0]],
        '攫' : [12, 13, 0, -2, 12, [0, 20192, 19104, 65248, 19104, 19104, 24544, 26880, 53216, 17472, 17280, 56416, 0]],
        '攴' : [12, 13, 0, -2, 12, [1024, 1024, 1984, 1024, 1024, 32704, 64, 4224, 2304, 1536, 6528, 57440, 0]],
        '攵' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 8320, 16512, 18560, 34944, 35072, 1280, 1536, 6528, 57440, 0]],
        '攷' : [12, 13, 0, -2, 12, [512, 64000, 9184, 8768, 9280, 31808, 18752, 35136, 34944, 2176, 4416, 26144, 0]],
        '收' : [12, 13, 0, -2, 12, [1024, 5120, 38880, 37952, 37952, 37952, 37184, 37184, 45184, 53376, 4928, 7200, 0]],
        '攸' : [12, 13, 0, -2, 12, [512, 8704, 9184, 16960, 21568, 54336, 20800, 20800, 20608, 20608, 17216, 19488, 0]],
        '畋' : [12, 13, 0, -2, 12, [512, 512, 64480, 43584, 43584, 44096, 63808, 43328, 43136, 43136, 63808, 1568, 0]],
        '效' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 576, 21056, 34880, 320, 2368, 20608, 8320, 20800, 35360, 0]],
        '敖' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 8768, 64576, 8256, 63808, 16704, 30848, 34944, 35136, 13856, 0]],
        '敕' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 8768, 64064, 44096, 43328, 63808, 8320, 28800, 43328, 9760, 0]],
        '敍' : [12, 13, 0, -2, 12, [256, 12544, 18912, 33024, 63744, 10208, 8224, 64032, 8512, 43136, 43328, 26144, 0]],
        '敘' : [12, 13, 0, -2, 12, [512, 12800, 19424, 33344, 64576, 9280, 8512, 63808, 8320, 43136, 43328, 26144, 0]],
        '敞' : [12, 13, 0, -2, 12, [8704, 8704, 44000, 42560, 8768, 64576, 34112, 62784, 38016, 62592, 34112, 36384, 0]],
        '敝' : [12, 13, 0, -2, 12, [8704, 8704, 44000, 42560, 8768, 64576, 42304, 46400, 46208, 46208, 42304, 44576, 0]],
        '敲' : [12, 13, 0, -2, 12, [8448, 8448, 64992, 18688, 30976, 992, 64544, 34080, 62784, 38016, 62784, 34336, 0]],
        '數' : [12, 13, 0, -2, 12, [256, 32000, 21984, 64832, 22080, 31808, 21824, 64832, 17536, 51328, 14656, 50720, 0]],
        '斂' : [12, 13, 0, -2, 12, [512, 29184, 35808, 64064, 576, 64576, 43328, 43328, 63616, 20608, 43328, 44576, 0]],
        '斃' : [12, 13, 0, -2, 12, [8704, 43520, 9184, 64064, 44352, 63872, 43584, 65504, 18944, 43968, 4640, 58336, 0]],
        '變' : [12, 13, 0, -2, 12, [1024, 17472, 44704, 16448, 44704, 16448, 61152, 19008, 16352, 53376, 3840, 61664, 0]],
        '斛' : [12, 13, 0, -2, 12, [0, 28736, 37440, 8512, 63552, 43584, 63808, 43104, 43456, 65088, 34880, 38976, 0]],
        '斟' : [12, 13, 0, -2, 12, [0, 18496, 65088, 18752, 30784, 19008, 18752, 64608, 43456, 52800, 32832, 64576, 0]],
        '斫' : [12, 13, 0, -2, 12, [0, 192, 63232, 9216, 9216, 10208, 29824, 54400, 21632, 22656, 20608, 28800, 0]],
        '斷' : [12, 13, 0, -2, 12, [0, 43104, 54144, 43520, 53760, 65504, 43584, 53824, 43584, 53824, 64576, 1088, 0]],
        '旃' : [12, 13, 0, -2, 12, [512, 8704, 9184, 64512, 10208, 8864, 15008, 20448, 18976, 35360, 35360, 12896, 0]],
        '旆' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 62720, 18688, 20448, 28928, 22496, 38176, 38176, 5472, 24832, 0]],
        '旁' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 4352, 4352, 65504, 34848, 32704, 2048, 4032, 4160, 24960, 0]],
        '旄' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 64512, 10176, 8448, 14816, 20224, 18912, 36608, 35104, 12768, 0]],
        '旌' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 63616, 10880, 9184, 14976, 19584, 19424, 34944, 34944, 13280, 0]],
        '旒' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 63744, 20448, 16960, 30688, 20480, 38208, 38208, 5440, 25952, 0]],
        '旛' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 63488, 20448, 17056, 30688, 21120, 38048, 40928, 5280, 26592, 0]],
        '旙' : [12, 13, 0, -2, 12, [0, 20448, 17568, 62080, 20448, 17088, 29856, 24544, 38048, 38880, 5280, 26592, 0]],
        '无' : [12, 13, 0, -2, 12, [0, 32704, 1024, 1024, 1024, 65504, 1024, 1024, 2560, 4640, 8736, 50144, 0]],
        '旡' : [12, 13, 0, -2, 12, [0, 32704, 1024, 17408, 17408, 32736, 33792, 33792, 2560, 4640, 8736, 50144, 0]],
        '旱' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 32704, 0, 32704, 1024, 65504, 1024, 1024, 0]],
        '杲' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 16448, 32704, 1024, 65504, 3328, 13440, 50272, 0]],
        '昊' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 16448, 65504, 1024, 32704, 1024, 6912, 57568, 0]],
        '昃' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 32704, 0, 32736, 16896, 16896, 36224, 45152, 0]],
        '旻' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 32704, 1024, 65504, 4352, 4352, 3584, 61920, 0]],
        '杳' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 3328, 13440, 50240, 16288, 8320, 16256, 8320, 16256, 0]],
        '昵' : [12, 13, 0, -2, 12, [0, 2016, 62496, 37920, 38880, 37888, 62752, 38208, 38272, 38144, 63776, 2272, 0]],
        '昶' : [12, 13, 0, -2, 12, [4096, 2528, 288, 31008, 2528, 60192, 11552, 10528, 18912, 19456, 35584, 39136, 0]],
        '昴' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 16256, 0, 60384, 35360, 35360, 62176, 4608, 0]],
        '昜' : [12, 13, 0, -2, 12, [0, 16320, 8256, 16320, 8256, 32736, 4096, 16352, 18720, 35104, 4640, 9408, 0]],
        '晏' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 65504, 34848, 65504, 4352, 12544, 3584, 29120, 0]],
        '晄' : [12, 13, 0, -2, 12, [256, 256, 59680, 42304, 41216, 41216, 65504, 41600, 41600, 41600, 58528, 6368, 0]],
        '晉' : [12, 13, 0, -2, 12, [0, 65504, 4224, 8448, 19008, 31680, 0, 65504, 8320, 16256, 8320, 16256, 0]],
        '晁' : [12, 13, 0, -2, 12, [0, 16256, 8320, 16256, 8320, 8320, 49056, 19008, 10880, 19008, 4640, 58336, 0]],
        '晞' : [12, 13, 0, -2, 12, [0, 1632, 61824, 38464, 37152, 40928, 62080, 38880, 40096, 38048, 62688, 128, 0]],
        '晝' : [12, 13, 0, -2, 12, [1024, 1024, 16256, 1152, 65504, 1152, 65504, 8320, 16256, 8320, 8320, 65504, 0]],
        '晤' : [12, 13, 0, -2, 12, [0, 4064, 61696, 38880, 37408, 37408, 65504, 36864, 38880, 37920, 62496, 2016, 0]],
        '晧' : [12, 13, 0, -2, 12, [128, 1152, 63456, 38016, 36992, 38880, 61568, 36864, 38880, 37920, 62496, 2016, 0]],
        '晨' : [12, 13, 0, -2, 12, [0, 32736, 16416, 32736, 16416, 32736, 16384, 24512, 16384, 32736, 41280, 48352, 0]],
        '晟' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 32736, 16640, 31008, 18752, 18560, 39232, 34336, 0]],
        '晢' : [12, 13, 0, -2, 12, [0, 9184, 64000, 9184, 14912, 57920, 9280, 32704, 16448, 32704, 16448, 32704, 0]],
        '晰' : [12, 13, 0, -2, 12, [0, 2144, 59776, 48896, 43264, 43488, 60704, 47904, 43296, 43552, 59936, 2080, 0]],
        '暃' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 65504, 2560, 31680, 2560, 64480, 4608, 25088, 0]],
        '暈' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 65504, 42144, 16256, 9344, 9344, 65504, 1024, 0]],
        '暎' : [12, 13, 0, -2, 12, [1088, 1088, 61408, 42304, 41216, 45024, 59680, 43296, 49120, 41600, 58432, 6176, 0]],
        '暉' : [12, 13, 0, -2, 12, [0, 8160, 61728, 45024, 43296, 45024, 59680, 43296, 45024, 41216, 65504, 256, 0]],
        '暄' : [12, 13, 0, -2, 12, [256, 256, 65504, 36896, 38848, 37952, 63424, 37952, 37952, 38848, 61440, 4064, 0]],
        '暘' : [12, 13, 0, -2, 12, [0, 1984, 62528, 38848, 37952, 38880, 62464, 40928, 37536, 38048, 63776, 704, 0]],
        '暝' : [12, 13, 0, -2, 12, [0, 4064, 63520, 38880, 37920, 38880, 62496, 38880, 37120, 40928, 62016, 3104, 0]],
        '曁' : [12, 13, 0, -2, 12, [8192, 18400, 64128, 35456, 65504, 33408, 38048, 65504, 8320, 16256, 8320, 65504, 0]],
        '暹' : [12, 13, 0, -2, 12, [0, 20416, 10304, 4032, 34880, 20448, 6400, 61376, 10496, 12256, 20480, 36832, 0]],
        '曉' : [12, 13, 0, -2, 12, [256, 256, 61408, 41216, 49120, 42048, 61152, 42048, 49120, 41600, 58528, 6368, 0]],
        '暾' : [12, 13, 0, -2, 12, [1152, 1152, 65504, 41120, 44832, 43552, 61088, 41632, 42048, 48960, 58528, 3360, 0]],
        '暼' : [12, 13, 0, -2, 12, [8704, 43520, 9184, 64576, 43328, 63872, 43616, 32704, 16448, 32704, 16448, 32704, 0]],
        '曄' : [12, 13, 0, -2, 12, [576, 576, 61408, 41536, 45024, 42272, 65504, 42272, 45024, 41216, 61408, 256, 0]],
        '暸' : [12, 13, 0, -2, 12, [256, 256, 65504, 43584, 42976, 42016, 61408, 46112, 42976, 41216, 59712, 4896, 0]],
        '曖' : [12, 13, 0, -2, 12, [0, 8160, 58656, 42304, 49120, 42144, 63424, 41984, 44992, 46144, 58240, 7264, 0]],
        '曚' : [12, 13, 0, -2, 12, [1152, 1152, 65504, 42112, 45024, 43040, 61376, 41600, 44448, 41664, 60576, 768, 0]],
        '曠' : [12, 13, 0, -2, 12, [256, 256, 65504, 42112, 49120, 42112, 61408, 43296, 45024, 59680, 12256, 12320, 0]],
        '昿' : [12, 13, 0, -2, 12, [128, 128, 63456, 37888, 38016, 38016, 62592, 38144, 38176, 39200, 61984, 992, 0]],
        '曦' : [12, 13, 0, -2, 12, [1088, 1152, 65504, 41216, 49120, 42112, 65504, 43680, 47264, 44608, 58016, 3360, 0]],
        '曩' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 65504, 19008, 64480, 4352, 65504, 20800, 48352, 0]],
        '曰' : [12, 13, 0, -2, 12, [0, 32704, 16448, 16448, 16448, 16448, 32320, 16448, 16448, 16448, 16448, 32704, 0]],
        '曵' : [12, 13, 0, -2, 12, [1088, 1056, 32704, 17472, 32704, 17472, 32704, 576, 640, 768, 3360, 61664, 0]],
        '曷' : [12, 13, 0, -2, 12, [0, 16320, 8256, 16320, 8256, 16320, 4096, 16352, 49696, 9504, 10272, 16064, 0]],
        '朏' : [12, 13, 0, -2, 12, [256, 57600, 43296, 43296, 59680, 45024, 41216, 57600, 43296, 43296, 43296, 45024, 0]],
        '朖' : [12, 13, 0, -2, 12, [256, 61696, 38880, 37920, 63456, 37920, 38880, 62720, 38176, 38048, 37952, 46880, 0]],
        '朞' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 7936, 4352, 65504, 4224, 16320, 53408, 8064, 24704, 0]],
        '朦' : [12, 13, 0, -2, 12, [1152, 58496, 49120, 42112, 61408, 43040, 44992, 57984, 44448, 41664, 44192, 41728, 0]],
        '朧' : [12, 13, 0, -2, 12, [1152, 58496, 49120, 43648, 60128, 48928, 43744, 61056, 43744, 44672, 43680, 43744, 0]],
        '霸' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 5248, 65504, 21024, 64480, 43552, 65504, 8736, 0]],
        '朮' : [12, 13, 0, -2, 12, [1152, 1088, 1024, 65504, 1024, 9344, 9344, 9344, 9344, 17568, 17632, 1024, 0]],
        '朿' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 17472, 17472, 17600, 3584, 13696, 50272, 1024, 0]],
        '朶' : [12, 13, 0, -2, 12, [0, 32640, 1152, 1248, 2080, 12320, 50368, 1024, 65504, 3328, 13440, 50272, 0]],
        '杁' : [12, 13, 0, -2, 12, [0, 10112, 8320, 63616, 8320, 8320, 24896, 24896, 45632, 41536, 9248, 9248, 0]],
        '朸' : [12, 13, 0, -2, 12, [256, 8448, 8448, 63456, 8480, 8480, 24864, 25120, 45600, 42016, 9280, 8576, 0]],
        '朷' : [12, 13, 0, -2, 12, [0, 10208, 8480, 63776, 8480, 8480, 24864, 25120, 45600, 42016, 9280, 8576, 0]],
        '杆' : [12, 13, 0, -2, 12, [0, 10208, 8320, 63616, 8320, 8320, 26592, 28800, 43136, 41088, 8320, 8320, 0]],
        '杞' : [12, 13, 0, -2, 12, [0, 10208, 8224, 61472, 8224, 10208, 25600, 25600, 46080, 42016, 9248, 9184, 0]],
        '杠' : [12, 13, 0, -2, 12, [0, 10208, 8320, 63616, 8320, 8320, 24704, 28800, 43136, 41088, 8320, 10208, 0]],
        '杙' : [12, 13, 0, -2, 12, [256, 8512, 8480, 61696, 12256, 8320, 24704, 28800, 43072, 41024, 8224, 8224, 0]],
        '杣' : [12, 13, 0, -2, 12, [256, 8448, 8448, 61696, 10528, 10528, 26912, 26912, 43296, 43296, 10528, 12256, 0]],
        '杤' : [12, 13, 0, -2, 12, [0, 12256, 8704, 61952, 8704, 9184, 25632, 29728, 43040, 43040, 8256, 8576, 0]],
        '枉' : [12, 13, 0, -2, 12, [0, 10208, 8320, 63616, 8320, 8320, 26592, 24704, 45184, 41088, 8320, 10208, 0]],
        '杰' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 3584, 13696, 50272, 1024, 1024, 0, 21056, 35104, 0]],
        '枩' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 3328, 13440, 50272, 1024, 12672, 49248, 2048, 4160, 32704, 0]],
        '杼' : [12, 13, 0, -2, 12, [0, 12224, 8256, 62080, 8448, 12256, 24864, 24864, 45376, 41216, 8448, 8960, 0]],
        '杪' : [12, 13, 0, -2, 12, [256, 8448, 8448, 62784, 10528, 8448, 24832, 24832, 45856, 41024, 8576, 11776, 0]],
        '枌' : [12, 13, 0, -2, 12, [0, 9152, 8256, 62528, 9248, 10272, 28640, 25152, 45632, 42048, 9280, 8576, 0]],
        '枋' : [12, 13, 0, -2, 12, [512, 8704, 12256, 61952, 8704, 9184, 25632, 29728, 43040, 43040, 8256, 8576, 0]],
        '枦' : [12, 13, 0, -2, 12, [0, 10208, 8192, 61440, 10208, 9248, 25632, 26592, 46080, 41984, 10240, 10240, 0]],
        '枡' : [12, 13, 0, -2, 12, [0, 8640, 11840, 62016, 8768, 8768, 28640, 25152, 45632, 41536, 9280, 9280, 0]],
        '枅' : [12, 13, 0, -2, 12, [0, 12256, 8768, 62016, 8768, 12256, 25152, 25152, 45632, 41536, 9280, 9280, 0]],
        '枷' : [12, 13, 0, -2, 12, [1024, 9216, 9696, 65312, 9504, 9504, 25888, 29984, 43296, 43296, 8672, 9728, 0]],
        '柯' : [12, 13, 0, -2, 12, [0, 12256, 8224, 61472, 10016, 9504, 25888, 25888, 46880, 40992, 8224, 8416, 0]],
        '枴' : [12, 13, 0, -2, 12, [0, 10176, 9280, 62528, 10176, 8192, 24576, 28640, 45600, 41504, 9248, 9408, 0]],
        '柬' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 25792, 21824, 17472, 32704, 3328, 13440, 50272, 0]],
        '枳' : [12, 13, 0, -2, 12, [0, 10208, 9248, 64544, 9248, 9248, 26592, 24576, 45632, 41536, 9248, 9248, 0]],
        '柩' : [12, 13, 0, -2, 12, [0, 12256, 10496, 63936, 10816, 11328, 26752, 27008, 43584, 44064, 10240, 12256, 0]],
        '枸' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 63520, 10272, 10016, 25888, 25888, 46880, 40992, 8256, 8576, 0]],
        '柤' : [12, 13, 0, -2, 12, [0, 9184, 8736, 64032, 9184, 8736, 25120, 29664, 43552, 41504, 8736, 12256, 0]],
        '柞' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 62720, 10496, 10720, 24832, 24832, 45536, 41216, 8448, 8448, 0]],
        '柝' : [12, 13, 0, -2, 12, [0, 8384, 9984, 62464, 9216, 10208, 25728, 25984, 46272, 43168, 10368, 8320, 0]],
        '柢' : [12, 13, 0, -2, 12, [0, 10208, 9344, 62592, 9344, 10208, 25664, 25664, 46112, 42784, 8192, 12256, 0]],
        '柮' : [12, 13, 0, -2, 12, [128, 8320, 9376, 62624, 9376, 10208, 24704, 24704, 46240, 42144, 9376, 10208, 0]],
        '枹' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 63520, 10144, 9376, 25760, 26528, 46176, 41984, 9248, 10208, 0]],
        '柎' : [12, 13, 0, -2, 12, [64, 8768, 8768, 62944, 11328, 9280, 26176, 25920, 46144, 42048, 9280, 9408, 0]],
        '柆' : [12, 13, 0, -2, 12, [256, 8448, 8448, 65504, 8256, 9280, 25664, 25664, 46208, 42112, 8320, 12256, 0]],
        '柧' : [12, 13, 0, -2, 12, [0, 12256, 10880, 64128, 10880, 10880, 27200, 27200, 43584, 43680, 14240, 8224, 0]],
        '檜' : [12, 13, 0, -2, 12, [0, 10176, 10272, 63456, 8192, 12256, 27296, 28640, 46112, 42976, 9248, 10208, 0]],
        '栞' : [12, 13, 0, -2, 12, [0, 64480, 8320, 64480, 8320, 9344, 1024, 65504, 3328, 13440, 50272, 1024, 0]],
        '框' : [12, 13, 0, -2, 12, [0, 12256, 10240, 64480, 10368, 10368, 27616, 26752, 43136, 44000, 10240, 12256, 0]],
        '栩' : [12, 13, 0, -2, 12, [0, 12000, 8736, 61984, 9824, 10912, 25120, 26208, 43680, 45856, 8736, 9824, 0]],
        '桀' : [12, 13, 0, -2, 12, [128, 30848, 19424, 19072, 43648, 38880, 4224, 9216, 65504, 3328, 13440, 50272, 0]],
        '桍' : [12, 13, 0, -2, 12, [256, 8448, 12256, 62016, 9248, 12256, 24576, 28640, 41472, 42976, 9248, 8384, 0]],
        '栲' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61728, 8512, 12256, 25088, 26592, 44032, 47072, 8224, 8384, 0]],
        '桎' : [12, 13, 0, -2, 12, [0, 12256, 8448, 61952, 9248, 10208, 24704, 24704, 47072, 41088, 8320, 12256, 0]],
        '梳' : [12, 13, 0, -2, 12, [512, 8704, 12256, 62016, 9280, 12192, 24608, 25920, 46400, 42304, 10560, 10592, 0]],
        '栫' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61952, 9152, 9280, 25728, 27776, 47072, 42112, 9344, 9600, 0]],
        '桙' : [12, 13, 0, -2, 12, [512, 8768, 9280, 63456, 8320, 9344, 26592, 25728, 45184, 45024, 8320, 8320, 0]],
        '档' : [12, 13, 0, -2, 12, [256, 8448, 10528, 62784, 8448, 12256, 24608, 24608, 45024, 40992, 8224, 12256, 0]],
        '桷' : [12, 13, 0, -2, 12, [0, 9152, 9280, 63616, 10208, 9504, 26592, 25888, 47072, 42016, 10272, 10336, 0]],
        '桿' : [12, 13, 0, -2, 12, [0, 10208, 9248, 63456, 9248, 10208, 24576, 26592, 45184, 45024, 8320, 8320, 0]],
        '梟' : [12, 13, 0, -2, 12, [1024, 2048, 32640, 16512, 32736, 16384, 32704, 1088, 65504, 3328, 13440, 50272, 0]],
        '梏' : [12, 13, 0, -2, 12, [128, 9344, 10208, 62592, 8320, 10208, 24704, 24576, 47072, 42016, 9248, 10208, 0]],
        '梭' : [12, 13, 0, -2, 12, [512, 8768, 9280, 65504, 9344, 9376, 30816, 25536, 44096, 42112, 9088, 11360, 0]],
        '梔' : [12, 13, 0, -2, 12, [0, 10176, 9216, 63456, 9216, 10208, 25760, 25760, 47072, 43520, 10784, 9184, 0]],
        '條' : [12, 13, 0, -2, 12, [512, 8704, 9184, 21568, 23104, 54144, 23904, 20736, 24544, 21248, 17728, 22816, 0]],
        '梛' : [12, 13, 0, -2, 12, [0, 16352, 9376, 58528, 12192, 9408, 25760, 32672, 42144, 42144, 10432, 10624, 0]],
        '梃' : [12, 13, 0, -2, 12, [0, 12000, 8768, 62528, 10304, 12256, 25152, 27200, 43584, 42464, 9728, 14816, 0]],
        '檮' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61696, 12256, 8224, 28608, 24832, 49120, 43552, 10912, 11872, 0]],
        '梹' : [12, 13, 0, -2, 12, [0, 8384, 9984, 62464, 10208, 9280, 25664, 25664, 49120, 40960, 9280, 10272, 0]],
        '桴' : [12, 13, 0, -2, 12, [0, 10208, 9216, 63808, 10528, 8192, 25568, 24640, 45184, 42976, 8320, 8576, 0]],
        '梵' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8320, 29120, 43680, 8320, 7936, 4352, 6432, 9504, 49632, 0]],
        '梠' : [12, 13, 0, -2, 12, [0, 10208, 9248, 62496, 10208, 8448, 25088, 28640, 43040, 43040, 10272, 12256, 0]],
        '梺' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8576, 29376, 42144, 65504, 1024, 1792, 1216, 1024, 1024, 0]],
        '椏' : [12, 13, 0, -2, 12, [0, 12256, 8832, 62080, 12000, 10272, 26656, 28384, 41600, 41600, 8832, 12256, 0]],
        '梍' : [12, 13, 0, -2, 12, [128, 8448, 10208, 62496, 10208, 9248, 26592, 29184, 41568, 44928, 8736, 9184, 0]],
        '桾' : [12, 13, 0, -2, 12, [0, 10208, 8480, 63456, 8480, 8480, 26592, 25088, 47072, 43552, 8736, 9184, 0]],
        '椁' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61440, 10208, 9248, 26592, 24640, 45184, 45024, 8320, 9088, 0]],
        '棊' : [12, 13, 0, -2, 12, [4352, 4352, 32704, 4352, 7936, 4352, 65504, 9344, 32704, 35872, 13696, 50272, 0]],
        '椈' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 62496, 10912, 10912, 25120, 28640, 41760, 42656, 10784, 8384, 0]],
        '棘' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8320, 64480, 43680, 43680, 45760, 8320, 29120, 43680, 9344, 0]],
        '椢' : [12, 13, 0, -2, 12, [0, 12256, 10272, 65504, 10528, 10528, 27552, 26976, 43296, 45024, 10272, 12256, 0]],
        '椦' : [12, 13, 0, -2, 12, [256, 9536, 10528, 61408, 8768, 12256, 25664, 26912, 47072, 41248, 8736, 11456, 0]],
        '棡' : [12, 13, 0, -2, 12, [0, 12256, 10272, 64160, 10912, 12256, 27168, 27296, 43680, 44960, 10272, 10336, 0]],
        '椌' : [12, 13, 0, -2, 12, [256, 8448, 12256, 64160, 10880, 8864, 27872, 24576, 47040, 41216, 8448, 12256, 0]],
        '棍' : [12, 13, 0, -2, 12, [0, 10208, 9248, 63456, 9248, 10208, 25728, 25728, 46816, 42112, 9376, 12000, 0]],
        '棔' : [12, 13, 0, -2, 12, [0, 10208, 9344, 65504, 9344, 9824, 24576, 26592, 46112, 42976, 9248, 10208, 0]],
        '棧' : [12, 13, 0, -2, 12, [1024, 9248, 16352, 58496, 8992, 11488, 25728, 32736, 42048, 41600, 8992, 15584, 0]],
        '棕' : [12, 13, 0, -2, 12, [256, 8448, 12256, 63520, 10176, 8192, 24576, 28640, 45312, 42304, 10528, 8960, 0]],
        '椶' : [12, 13, 0, -2, 12, [0, 10912, 10528, 64160, 12256, 8832, 27744, 25536, 44096, 42112, 9088, 15456, 0]],
        '椒' : [12, 13, 0, -2, 12, [1024, 9440, 10016, 62496, 9376, 16288, 25760, 25760, 44608, 46400, 9376, 11552, 0]],
        '椄' : [12, 13, 0, -2, 12, [256, 8448, 12256, 62528, 9280, 12256, 25088, 28640, 42048, 44096, 9088, 11360, 0]],
        '棗' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 17472, 20032, 13696, 50272, 32704, 17472, 20032, 13696, 50272, 0]],
        '棣' : [12, 13, 0, -2, 12, [256, 8448, 10208, 61728, 12256, 8480, 26592, 28960, 43840, 42368, 10560, 13088, 0]],
        '椥' : [12, 13, 0, -2, 12, [2048, 10464, 12192, 62624, 9376, 9376, 32672, 25760, 42144, 43680, 12768, 8320, 0]],
        '棹' : [12, 13, 0, -2, 12, [256, 8448, 8672, 63744, 10208, 9248, 26592, 25632, 47072, 41216, 12256, 8448, 0]],
        '棠' : [12, 13, 0, -2, 12, [1024, 9344, 65504, 32800, 40736, 4352, 7936, 1024, 65504, 3328, 13440, 50272, 0]],
        '棯' : [12, 13, 0, -2, 12, [0, 9088, 9280, 59424, 10176, 8192, 26560, 28736, 41600, 43264, 10304, 10144, 0]],
        '椨' : [12, 13, 0, -2, 12, [256, 8448, 12256, 63776, 10528, 10976, 28192, 27296, 43616, 43552, 12832, 8800, 0]],
        '椪' : [12, 13, 0, -2, 12, [2112, 9280, 8320, 61408, 8832, 8832, 27296, 26304, 45696, 41600, 8832, 12256, 0]],
        '椚' : [12, 13, 0, -2, 12, [0, 12000, 10912, 65248, 10912, 12000, 26656, 26656, 43040, 43040, 10272, 10336, 0]],
        '椣' : [12, 13, 0, -2, 12, [640, 8832, 12256, 64160, 10912, 12256, 27296, 27296, 49120, 40960, 9280, 10272, 0]],
        '椡' : [12, 13, 0, -2, 12, [0, 12064, 8864, 62624, 10400, 12192, 25760, 29856, 44960, 42016, 9248, 12128, 0]],
        '棆' : [12, 13, 0, -2, 12, [0, 9088, 9280, 63520, 12256, 8192, 28640, 27296, 43680, 45024, 10912, 10912, 0]],
        '楹' : [12, 13, 0, -2, 12, [0, 12160, 9344, 63456, 9376, 11040, 29888, 24576, 45024, 43680, 10912, 16352, 0]],
        '楷' : [12, 13, 0, -2, 12, [128, 10368, 12000, 63616, 10400, 12000, 24832, 28640, 43040, 45024, 10272, 12256, 0]],
        '楜' : [12, 13, 0, -2, 12, [1024, 9696, 9504, 65312, 9696, 9504, 28448, 27104, 47392, 43296, 12064, 8800, 0]],
        '楸' : [12, 13, 0, -2, 12, [128, 9088, 15488, 58528, 9376, 16288, 25728, 25792, 44352, 46400, 9760, 9248, 0]],
        '楫' : [12, 13, 0, -2, 12, [0, 10176, 9280, 62528, 12256, 9280, 26560, 25664, 47040, 42080, 12224, 8256, 0]],
        '楔' : [12, 13, 0, -2, 12, [1024, 9216, 12256, 62624, 11936, 9504, 28256, 25856, 49120, 41600, 9280, 14368, 0]],
        '楾' : [12, 13, 0, -2, 12, [128, 8448, 10208, 64544, 10208, 9248, 26592, 24704, 46752, 41664, 9376, 10624, 0]],
        '楮' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61728, 8512, 12256, 25152, 26592, 44064, 47072, 9248, 10208, 0]],
        '椹' : [12, 13, 0, -2, 12, [1088, 9280, 12256, 62528, 10176, 9280, 25664, 28640, 43648, 44256, 10240, 12256, 0]],
        '楴' : [12, 13, 0, -2, 12, [256, 8448, 16352, 58496, 9344, 16352, 24864, 28640, 43296, 43296, 10592, 8448, 0]],
        '椽' : [12, 13, 0, -2, 12, [0, 10208, 9280, 61376, 10368, 16352, 25856, 25504, 44224, 41888, 15520, 8960, 0]],
        '楙' : [12, 13, 0, -2, 12, [0, 11904, 8832, 60384, 9344, 16256, 25280, 26304, 43680, 45728, 8832, 9856, 0]],
        '椰' : [12, 13, 0, -2, 12, [0, 16096, 10912, 61088, 10912, 10944, 28320, 27296, 43680, 44704, 15040, 8832, 0]],
        '楡' : [12, 13, 0, -2, 12, [0, 12160, 9280, 59424, 16320, 8192, 28320, 27296, 44864, 43840, 11936, 10912, 0]],
        '楞' : [12, 13, 0, -2, 12, [0, 12256, 10912, 64160, 12256, 8448, 28640, 25088, 46048, 41504, 9248, 9408, 0]],
        '楝' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61696, 12256, 11616, 26912, 28640, 41728, 42304, 14624, 8448, 0]],
        '榁' : [12, 13, 0, -2, 12, [256, 8448, 16352, 57376, 12256, 8832, 25664, 28640, 45312, 42944, 8448, 12256, 0]],
        '楪' : [12, 13, 0, -2, 12, [256, 9536, 12256, 62784, 9664, 9216, 26592, 28928, 45024, 41728, 9536, 14624, 0]],
        '榲' : [12, 13, 0, -2, 12, [0, 10176, 9536, 63168, 9280, 10176, 24576, 28640, 43680, 43680, 10912, 16352, 0]],
        '榮' : [12, 13, 0, -2, 12, [4352, 21792, 38208, 10880, 17472, 65504, 33824, 49056, 1024, 7552, 58464, 1024, 0]],
        '槐' : [12, 13, 0, -2, 12, [256, 8704, 12256, 63776, 12256, 10528, 26912, 28640, 46400, 42304, 10528, 10464, 0]],
        '榿' : [12, 13, 0, -2, 12, [256, 10528, 12256, 61440, 12256, 8192, 26592, 25632, 47072, 41536, 8768, 12256, 0]],
        '槁' : [12, 13, 0, -2, 12, [256, 8448, 12256, 62528, 10176, 8192, 28640, 26656, 43936, 43680, 11168, 10336, 0]],
        '槓' : [12, 13, 0, -2, 12, [0, 10208, 8448, 61408, 8192, 10208, 25632, 26592, 46112, 42976, 8768, 11296, 0]],
        '榾' : [12, 13, 0, -2, 12, [0, 10176, 9280, 62912, 9536, 12256, 26656, 26560, 46144, 42944, 9280, 9408, 0]],
        '槎' : [12, 13, 0, -2, 12, [1088, 9344, 12256, 61696, 10208, 8448, 28640, 25088, 42976, 43136, 8320, 12256, 0]],
        '寨' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 49024, 4352, 65504, 9344, 57184, 3072, 13696, 50272, 0]],
        '槊' : [12, 13, 0, -2, 12, [4096, 21472, 64032, 9184, 43552, 64480, 8736, 17408, 65504, 3328, 13440, 50272, 0]],
        '槝' : [12, 13, 0, -2, 12, [256, 8704, 10176, 64576, 10176, 9280, 26592, 25600, 47072, 41504, 10912, 12192, 0]],
        '榻' : [12, 13, 0, -2, 12, [0, 10176, 9280, 63424, 9280, 10176, 24576, 32736, 43680, 46944, 10912, 13152, 0]],
        '槃' : [12, 13, 0, -2, 12, [4096, 9152, 31296, 19520, 27616, 19008, 63872, 19008, 65504, 3328, 13440, 50272, 0]],
        '榧' : [12, 13, 0, -2, 12, [0, 12256, 10560, 63808, 12128, 10560, 26944, 28512, 43328, 43584, 10240, 12256, 0]],
        '樮' : [12, 13, 0, -2, 12, [0, 12256, 8832, 65504, 10912, 10912, 28640, 24832, 42272, 43328, 8832, 11360, 0]],
        '榑' : [12, 13, 0, -2, 12, [256, 8480, 12256, 61696, 12256, 10528, 28640, 26912, 41024, 45024, 9280, 8896, 0]],
        '榠' : [12, 13, 0, -2, 12, [0, 12256, 10272, 63456, 9248, 10208, 25632, 26592, 45312, 45024, 8768, 11296, 0]],
        '榜' : [12, 13, 0, -2, 12, [256, 8448, 12256, 62016, 8768, 12256, 26912, 26592, 45568, 41952, 9248, 9408, 0]],
        '榕' : [12, 13, 0, -2, 12, [256, 8448, 16352, 58528, 10304, 13088, 25728, 26688, 49120, 43072, 10304, 12224, 0]],
        '榴' : [12, 13, 0, -2, 12, [0, 11744, 10400, 64160, 11040, 11616, 24576, 28640, 47392, 45024, 10528, 12256, 0]],
        '槞' : [12, 13, 0, -2, 12, [256, 8448, 12256, 62016, 8768, 16352, 26912, 28640, 43296, 45024, 8448, 8672, 0]],
        '槨' : [12, 13, 0, -2, 12, [1024, 9440, 16288, 57504, 12192, 10432, 28576, 25248, 42144, 49056, 9408, 11392, 0]],
        '樂' : [12, 13, 0, -2, 12, [1024, 18496, 49056, 20800, 40736, 61920, 7936, 1024, 65504, 3328, 13440, 50272, 0]],
        '樛' : [12, 13, 0, -2, 12, [0, 12256, 10912, 63328, 10912, 12832, 25472, 27712, 41376, 42496, 8384, 12032, 0]],
        '槿' : [12, 13, 0, -2, 12, [576, 8768, 12256, 62016, 10208, 9376, 25760, 26592, 45184, 42976, 8320, 12256, 0]],
        '權' : [12, 13, 0, -2, 12, [576, 8768, 16352, 60064, 12000, 8768, 26592, 27776, 47072, 42112, 9344, 10208, 0]],
        '槹' : [12, 13, 0, -2, 12, [256, 8704, 12256, 63520, 12256, 10272, 28640, 24832, 44384, 41216, 12256, 8448, 0]],
        '槲' : [12, 13, 0, -2, 12, [0, 11808, 12960, 65120, 10784, 16160, 27296, 27168, 48736, 41888, 8736, 9760, 0]],
        '槧' : [12, 13, 0, -2, 12, [4096, 4576, 65280, 21984, 32064, 21824, 65088, 5184, 65504, 3328, 13440, 50272, 0]],
        '樅' : [12, 13, 0, -2, 12, [1088, 9280, 9280, 60064, 12576, 9344, 26752, 31456, 43648, 43648, 11136, 11488, 0]],
        '榱' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61440, 10176, 9280, 28640, 25664, 47040, 41632, 11328, 10016, 0]],
        '樞' : [12, 13, 0, -2, 12, [0, 12256, 10240, 64448, 10816, 11200, 26624, 28384, 43680, 44768, 10240, 12256, 0]],
        '槭' : [12, 13, 0, -2, 12, [128, 8352, 12256, 64128, 11136, 10912, 28576, 27296, 45632, 42816, 10912, 8992, 0]],
        '樔' : [12, 13, 0, -2, 12, [1344, 10880, 9536, 65504, 10528, 12256, 26912, 28640, 41728, 42304, 14624, 8448, 0]],
        '槫' : [12, 13, 0, -2, 12, [256, 8448, 16352, 59680, 12256, 10528, 28608, 24864, 45024, 41024, 9280, 8896, 0]],
        '樊' : [12, 13, 0, -2, 12, [8320, 10880, 62944, 10880, 30144, 43680, 13696, 1024, 65504, 2560, 12672, 49248, 0]],
        '樒' : [12, 13, 0, -2, 12, [256, 8448, 16352, 57888, 10560, 13472, 25856, 27584, 41216, 43296, 10528, 12256, 0]],
        '櫁' : [12, 13, 0, -2, 12, [256, 8448, 16352, 57888, 10560, 13472, 25856, 28608, 43328, 44992, 8480, 16352, 0]],
        '樣' : [12, 13, 0, -2, 12, [1088, 9344, 12256, 61568, 10208, 8320, 28640, 24832, 44448, 42176, 10400, 12672, 0]],
        '樓' : [12, 13, 0, -2, 12, [256, 8448, 12256, 59680, 16352, 10528, 28640, 26912, 45024, 42048, 10112, 14432, 0]],
        '橄' : [12, 13, 0, -2, 12, [128, 11904, 9440, 65440, 10912, 11808, 27296, 28320, 43584, 44608, 15008, 8992, 0]],
        '樌' : [12, 13, 0, -2, 12, [0, 10208, 9376, 65504, 9376, 10208, 25632, 26592, 46112, 42976, 8768, 11296, 0]],
        '橲' : [12, 13, 0, -2, 12, [256, 8448, 10208, 61696, 12256, 9248, 26592, 29248, 45024, 42016, 9248, 10208, 0]],
        '樶' : [12, 13, 0, -2, 12, [0, 12224, 10304, 65472, 10304, 16352, 27136, 28640, 43680, 43680, 11968, 15136, 0]],
        '橸' : [12, 13, 0, -2, 12, [0, 10176, 9280, 63424, 9280, 10176, 24576, 28384, 47776, 44768, 10912, 12000, 0]],
        '橇' : [12, 13, 0, -2, 12, [0, 12224, 9216, 65472, 9248, 10208, 24576, 28384, 42048, 49120, 9280, 10080, 0]],
        '橢' : [12, 13, 0, -2, 12, [128, 11904, 11232, 64128, 11744, 10816, 27616, 27168, 44000, 44576, 11232, 10784, 0]],
        '橙' : [12, 13, 0, -2, 12, [0, 12064, 8512, 64160, 9280, 10208, 26624, 26592, 46112, 42976, 8768, 12256, 0]],
        '橦' : [12, 13, 0, -2, 12, [256, 8448, 12256, 62016, 8768, 12256, 25888, 26592, 46368, 42976, 8448, 12256, 0]],
        '橈' : [12, 13, 0, -2, 12, [256, 8448, 12256, 57600, 16352, 9280, 28384, 25664, 49120, 41600, 9376, 14560, 0]],
        '樸' : [12, 13, 0, -2, 12, [640, 10912, 12256, 62016, 8768, 12256, 24832, 26592, 45312, 45024, 8768, 11296, 0]],
        '樢' : [12, 13, 0, -2, 12, [256, 8704, 10176, 62528, 10176, 9280, 26592, 25600, 47072, 40992, 10912, 10976, 0]],
        '檐' : [12, 13, 0, -2, 12, [0, 10176, 10368, 65504, 10560, 11872, 26752, 28640, 43008, 47072, 9248, 10208, 0]],
        '檍' : [12, 13, 0, -2, 12, [256, 8448, 12256, 62016, 12256, 10272, 28640, 26656, 45024, 42240, 13376, 9120, 0]],
        '檠' : [12, 13, 0, -2, 12, [512, 18944, 65504, 19008, 32064, 54656, 30304, 1024, 65504, 3328, 13440, 50272, 0]],
        '檄' : [12, 13, 0, -2, 12, [1152, 10368, 12000, 64160, 12064, 10784, 32416, 26784, 44608, 45632, 8864, 11552, 0]],
        '檢' : [12, 13, 0, -2, 12, [0, 10176, 10272, 63456, 8192, 12000, 27296, 27296, 44768, 42048, 10912, 12576, 0]],
        '檣' : [12, 13, 0, -2, 12, [256, 8448, 12256, 62784, 9536, 11168, 24832, 28640, 43680, 43936, 10272, 12256, 0]],
        '檗' : [12, 13, 0, -2, 12, [256, 61696, 38880, 62016, 36832, 61696, 38880, 37120, 65504, 3328, 13440, 50272, 0]],
        '蘗' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 18752, 31712, 16512, 64480, 18560, 65504, 3328, 13440, 50272, 0]],
        '檻' : [12, 13, 0, -2, 12, [128, 12160, 10880, 65504, 10496, 12032, 27360, 25088, 45024, 43680, 10912, 16352, 0]],
        '櫃' : [12, 13, 0, -2, 12, [0, 12256, 10368, 64480, 10912, 12256, 27168, 27616, 43552, 44000, 10560, 12256, 0]],
        '櫂' : [12, 13, 0, -2, 12, [0, 12256, 10912, 63328, 10912, 8736, 25728, 26592, 44160, 47072, 9344, 10208, 0]],
        '檸' : [12, 13, 0, -2, 12, [256, 8448, 16352, 58656, 13376, 10144, 24576, 28640, 43680, 49120, 8448, 9984, 0]],
        '檳' : [12, 13, 0, -2, 12, [256, 8448, 12256, 63520, 10208, 10560, 26592, 27712, 47040, 42048, 10176, 14432, 0]],
        '檬' : [12, 13, 0, -2, 12, [1152, 9344, 16352, 58496, 12256, 10272, 28608, 25216, 44448, 41664, 11424, 8960, 0]],
        '櫞' : [12, 13, 0, -2, 12, [0, 9696, 10528, 62400, 11328, 10208, 26752, 32480, 42304, 44768, 13632, 9408, 0]],
        '櫑' : [12, 13, 0, -2, 12, [0, 10176, 9536, 63424, 9536, 10176, 24576, 28640, 43680, 45024, 10912, 12256, 0]],
        '櫟' : [12, 13, 0, -2, 12, [256, 10816, 14240, 60608, 14240, 15584, 26496, 24832, 49120, 41728, 9536, 14624, 0]],
        '檪' : [12, 13, 0, -2, 12, [256, 8704, 14240, 60608, 10112, 11456, 30624, 24832, 49120, 41728, 9536, 14624, 0]],
        '櫚' : [12, 13, 0, -2, 12, [0, 12000, 10912, 65248, 10912, 12256, 27296, 27552, 43296, 43936, 10912, 11168, 0]],
        '櫪' : [12, 13, 0, -2, 12, [0, 12256, 10816, 65504, 10816, 12000, 27456, 26752, 43744, 45696, 8832, 12256, 0]],
        '櫻' : [12, 13, 0, -2, 12, [0, 12000, 10912, 65248, 10912, 12000, 26944, 29216, 45024, 41536, 9088, 11360, 0]],
        '欅' : [12, 13, 0, -2, 12, [256, 11616, 10656, 60768, 10912, 16352, 25920, 26912, 47040, 41216, 12256, 8448, 0]],
        '蘖' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 18752, 31712, 16512, 31712, 18560, 65504, 3328, 13440, 50272, 0]],
        '櫺' : [12, 13, 0, -2, 12, [0, 12256, 8448, 65504, 10528, 11680, 27488, 24576, 45024, 43680, 10912, 12256, 0]],
        '欒' : [12, 13, 0, -2, 12, [1024, 17472, 44704, 16448, 44704, 16448, 61152, 19008, 65504, 3328, 13440, 50272, 0]],
        '欖' : [12, 13, 0, -2, 12, [256, 16128, 9696, 65280, 9696, 16032, 27104, 28480, 43072, 44992, 9376, 14560, 0]],
        '鬱' : [12, 13, 0, -2, 12, [2048, 11904, 62944, 28352, 42144, 65504, 44192, 55072, 19648, 29440, 17504, 32128, 0]],
        '欟' : [12, 13, 0, -2, 12, [2560, 10976, 16288, 60128, 10912, 15328, 25760, 28576, 47840, 44864, 10816, 12128, 0]],
        '欸' : [12, 13, 0, -2, 12, [8704, 16896, 35808, 64160, 17056, 31904, 20608, 36992, 64832, 4416, 10784, 50208, 0]],
        '欷' : [12, 13, 0, -2, 12, [512, 19968, 13280, 52896, 8864, 65184, 20608, 31872, 54592, 21824, 24096, 4640, 0]],
        '盜' : [12, 13, 0, -2, 12, [1024, 17408, 10208, 35104, 18688, 8832, 17472, 34848, 32704, 19008, 19008, 65504, 0]],
        '欹' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 8864, 21152, 35488, 64640, 2176, 59712, 43328, 59936, 7200, 0]],
        '飮' : [12, 13, 0, -2, 12, [512, 29184, 35808, 64160, 672, 64160, 34944, 63616, 33088, 63808, 33312, 64544, 0]],
        '歇' : [12, 13, 0, -2, 12, [512, 64000, 35808, 64160, 35488, 64160, 16512, 64640, 42304, 38208, 58912, 3104, 0]],
        '歃' : [12, 13, 0, -2, 12, [512, 64000, 5088, 65184, 4768, 56480, 38016, 38016, 56640, 38208, 38432, 64544, 0]],
        '歉' : [12, 13, 0, -2, 12, [18944, 37376, 65504, 21152, 65184, 21664, 64640, 21632, 64832, 20800, 55840, 21536, 0]],
        '歐' : [12, 13, 0, -2, 12, [512, 64000, 33760, 47776, 43680, 47776, 32896, 63616, 43328, 63808, 33312, 64544, 0]],
        '歙' : [12, 13, 0, -2, 12, [512, 29184, 35808, 65184, 35488, 64160, 128, 64640, 9536, 46400, 28192, 46112, 0]],
        '歔' : [12, 13, 0, -2, 12, [8704, 14848, 9184, 65184, 41632, 64672, 41088, 48256, 43328, 60736, 10784, 64544, 0]],
        '歛' : [12, 13, 0, -2, 12, [512, 29184, 35808, 64160, 672, 64672, 43136, 43136, 63808, 20800, 43552, 44064, 0]],
        '歟' : [12, 13, 0, -2, 12, [8704, 60928, 46048, 59040, 47776, 52896, 43648, 43648, 65344, 320, 18976, 33824, 0]],
        '歡' : [12, 13, 0, -2, 12, [8704, 8704, 65504, 43680, 61088, 9376, 32384, 51328, 32064, 18752, 18976, 31776, 0]],
        '歸' : [12, 13, 0, -2, 12, [8192, 18368, 61504, 38880, 63552, 36832, 63520, 12256, 9504, 46368, 42336, 63744, 0]],
        '歹' : [12, 13, 0, -2, 12, [0, 32736, 2048, 2048, 4032, 4160, 26688, 1152, 640, 256, 1536, 30720, 0]],
        '歿' : [12, 13, 0, -2, 12, [512, 61952, 17376, 16928, 29728, 37984, 36864, 22496, 8736, 8768, 16768, 36448, 0]],
        '殀' : [12, 13, 0, -2, 12, [0, 61632, 20224, 16640, 28928, 37120, 40928, 20736, 8448, 8832, 17472, 38944, 0]],
        '殄' : [12, 13, 0, -2, 12, [0, 62336, 17472, 18464, 28864, 38656, 36864, 20672, 9984, 8224, 16576, 36608, 0]],
        '殃' : [12, 13, 0, -2, 12, [256, 61696, 18400, 17696, 29984, 38176, 38176, 24544, 8448, 8832, 17472, 38944, 0]],
        '殍' : [12, 13, 0, -2, 12, [0, 63456, 17408, 18752, 31008, 36864, 37856, 20544, 8320, 10208, 16512, 33152, 0]],
        '殘' : [12, 13, 0, -2, 12, [1024, 62496, 20448, 16960, 29088, 40544, 38016, 24544, 13376, 8832, 17184, 40160, 0]],
        '殕' : [12, 13, 0, -2, 12, [256, 61696, 20448, 16960, 29248, 40928, 36864, 20480, 10208, 9248, 17440, 34784, 0]],
        '殞' : [12, 13, 0, -2, 12, [0, 63456, 17440, 18400, 28672, 38880, 37920, 22496, 9248, 10208, 16960, 35872, 0]],
        '殤' : [12, 13, 0, -2, 12, [512, 61952, 18400, 19520, 30656, 37952, 38880, 20992, 12256, 9376, 18720, 33472, 0]],
        '殪' : [12, 13, 0, -2, 12, [256, 61696, 20448, 16640, 32736, 36896, 38848, 21568, 10176, 8832, 17024, 40928, 0]],
        '殫' : [12, 13, 0, -2, 12, [0, 61152, 19104, 19104, 32480, 36864, 40928, 22816, 12256, 10528, 24544, 33024, 0]],
        '殯' : [12, 13, 0, -2, 12, [256, 61696, 20448, 18464, 30688, 39232, 38880, 23616, 10176, 9280, 18368, 39008, 0]],
        '殲' : [12, 13, 0, -2, 12, [2688, 60064, 21920, 18560, 32736, 43584, 47936, 27232, 15168, 10816, 19104, 48928, 0]],
        '殱' : [12, 13, 0, -2, 12, [1152, 58528, 24480, 17536, 32736, 43584, 47936, 27232, 15168, 10816, 19104, 48928, 0]],
        '殳' : [12, 13, 0, -2, 12, [0, 7936, 4352, 4384, 8480, 49376, 0, 32640, 4224, 4352, 3584, 61920, 0]],
        '殷' : [12, 13, 0, -2, 12, [0, 64448, 33344, 64064, 35392, 64608, 34816, 65504, 33312, 64064, 35200, 13920, 0]],
        '殼' : [12, 13, 0, -2, 12, [8192, 9152, 64064, 8768, 64064, 35936, 61440, 2016, 29216, 21056, 39296, 34400, 0]],
        '毆' : [12, 13, 0, -2, 12, [0, 64448, 33344, 47680, 43584, 48224, 32768, 64480, 43552, 64064, 33152, 65120, 0]],
        '毋' : [12, 13, 0, -2, 12, [0, 16320, 9280, 9280, 9280, 65504, 18560, 18560, 18560, 18560, 65504, 256, 0]],
        '毓' : [12, 13, 0, -2, 12, [16640, 16640, 32736, 33344, 62528, 57248, 36896, 64832, 38208, 54592, 38208, 63840, 0]],
        '毟' : [12, 13, 0, -2, 12, [1024, 1024, 17472, 35872, 384, 3584, 61664, 3840, 30944, 3840, 63520, 2016, 0]],
        '毬' : [12, 13, 0, -2, 12, [256, 12608, 49440, 24512, 26912, 50496, 20736, 25920, 51488, 17152, 16416, 16352, 0]],
        '毫' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 4352, 65504, 34848, 4032, 30720, 4032, 63520, 2016, 0]],
        '毳' : [12, 13, 0, -2, 12, [0, 4064, 63488, 4032, 63520, 4064, 0, 64480, 16640, 65504, 16640, 31200, 0]],
        '毯' : [12, 13, 0, -2, 12, [256, 13600, 51520, 17024, 21568, 24864, 50496, 21824, 25216, 50240, 18464, 16352, 0]],
        '麾' : [12, 13, 0, -2, 12, [512, 512, 32736, 18688, 32736, 23360, 28000, 17280, 23648, 18304, 48160, 33760, 0]],
        '氈' : [12, 13, 0, -2, 12, [4096, 4192, 65408, 43648, 47840, 33664, 65152, 17632, 32640, 17536, 17568, 65248, 0]],
        '氓' : [12, 13, 0, -2, 12, [0, 18400, 17440, 17440, 63456, 33920, 33920, 34784, 33856, 33856, 29728, 1824, 0]],
        '气' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16384, 24448, 0, 32704, 64, 64, 64, 32, 32, 0]],
        '氛' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16384, 24448, 0, 32704, 8768, 16192, 21056, 4640, 9760, 0]],
        '氤' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16384, 24448, 0, 32704, 18752, 32576, 18752, 21792, 32544, 0]],
        '氣' : [12, 13, 0, -2, 12, [8192, 8192, 16352, 16384, 24448, 0, 32704, 10816, 2112, 32576, 10784, 18720, 0]],
        '汞' : [12, 13, 0, -2, 12, [0, 32704, 1024, 1024, 65504, 1024, 29760, 5760, 9472, 17536, 33888, 7168, 0]],
        '汕' : [12, 13, 0, -2, 12, [256, 256, 33024, 16640, 2336, 35104, 18720, 2336, 2336, 10528, 18720, 36832, 0]],
        '汢' : [12, 13, 0, -2, 12, [256, 256, 33024, 16640, 4064, 33024, 16640, 256, 256, 8448, 16640, 36832, 0]],
        '汪' : [12, 13, 0, -2, 12, [0, 4064, 33024, 16640, 256, 33024, 20448, 256, 256, 8448, 16640, 36832, 0]],
        '沂' : [12, 13, 0, -2, 12, [0, 192, 36608, 18432, 2048, 36832, 18560, 2176, 18560, 20608, 36992, 32896, 0]],
        '沍' : [12, 13, 0, -2, 12, [0, 4064, 33024, 16896, 992, 33824, 17472, 2112, 3968, 8320, 16640, 40928, 0]],
        '沚' : [12, 13, 0, -2, 12, [256, 256, 33024, 16640, 256, 35296, 18688, 2304, 18688, 18688, 35072, 40928, 0]],
        '沁' : [12, 13, 0, -2, 12, [0, 1536, 33152, 16384, 1024, 33856, 21568, 5152, 21536, 17408, 33856, 33728, 0]],
        '沛' : [12, 13, 0, -2, 12, [256, 256, 36832, 16640, 256, 36832, 18720, 2336, 2336, 10528, 18784, 33024, 0]],
        '汾' : [12, 13, 0, -2, 12, [0, 1984, 32832, 18496, 2080, 36896, 24544, 1088, 17472, 18496, 34880, 33152, 0]],
        '汨' : [12, 13, 0, -2, 12, [0, 2016, 33824, 17440, 1056, 33824, 18400, 1056, 1056, 9248, 17440, 34784, 0]],
        '汳' : [12, 13, 0, -2, 12, [0, 2016, 33792, 17408, 1984, 33856, 17472, 2368, 2368, 8320, 17216, 35872, 0]],
        '沒' : [12, 13, 0, -2, 12, [1024, 1024, 34784, 17440, 2080, 35008, 16384, 4032, 1088, 9344, 17280, 40032, 0]],
        '沐' : [12, 13, 0, -2, 12, [256, 256, 33024, 24544, 256, 33024, 17152, 1408, 18752, 20768, 33024, 33024, 0]],
        '泄' : [12, 13, 0, -2, 12, [256, 2368, 35136, 18752, 8160, 35136, 18752, 2368, 2496, 10240, 18432, 36832, 0]],
        '泱' : [12, 13, 0, -2, 12, [256, 256, 36832, 18720, 2336, 35104, 18720, 8160, 256, 8832, 17472, 38944, 0]],
        '泓' : [12, 13, 0, -2, 12, [0, 7296, 33920, 17536, 7296, 36992, 20608, 7424, 17696, 17696, 34080, 39392, 0]],
        '沽' : [12, 13, 0, -2, 12, [256, 256, 33024, 20448, 256, 33024, 16640, 4064, 2080, 10272, 18464, 36832, 0]],
        '泗' : [12, 13, 0, -2, 12, [0, 4064, 35488, 19104, 2720, 35488, 19104, 3296, 2080, 10272, 20448, 34848, 0]],
        '泅' : [12, 13, 0, -2, 12, [0, 4064, 35104, 18720, 2336, 35104, 18720, 2720, 3168, 10272, 18464, 36832, 0]],
        '泝' : [12, 13, 0, -2, 12, [0, 192, 36608, 18432, 2048, 36832, 18560, 2432, 18624, 20640, 36992, 32896, 0]],
        '沮' : [12, 13, 0, -2, 12, [0, 2016, 33824, 17440, 2016, 33824, 17440, 2016, 1056, 9248, 17440, 36832, 0]],
        '沱' : [12, 13, 0, -2, 12, [512, 512, 40928, 20512, 4128, 33792, 17600, 1792, 1024, 9216, 17440, 33760, 0]],
        '沾' : [12, 13, 0, -2, 12, [512, 512, 33280, 17376, 512, 33280, 16896, 4064, 2080, 10272, 18464, 36832, 0]],
        '沺' : [12, 13, 0, -2, 12, [0, 4064, 35104, 18720, 2336, 35104, 20448, 2336, 2336, 10528, 18720, 36832, 0]],
        '泛' : [12, 13, 0, -2, 12, [0, 224, 36608, 16640, 256, 36832, 16416, 64, 18560, 18688, 38400, 37344, 0]],
        '泯' : [12, 13, 0, -2, 12, [0, 4064, 34848, 18464, 4064, 34944, 18560, 4064, 2112, 10304, 18464, 36640, 0]],
        '泙' : [12, 13, 0, -2, 12, [0, 8160, 33024, 16640, 1344, 35104, 16640, 256, 24544, 16640, 33024, 33024, 0]],
        '泪' : [12, 13, 0, -2, 12, [0, 4064, 34848, 18464, 4064, 34848, 18464, 4064, 18464, 18464, 34848, 36832, 0]],
        '洟' : [12, 13, 0, -2, 12, [512, 512, 49120, 16960, 8128, 37376, 24544, 4640, 16960, 17664, 34944, 45152, 0]],
        '衍' : [12, 13, 0, -2, 12, [0, 8672, 18432, 33792, 0, 10720, 17472, 49216, 16448, 16960, 17472, 18624, 0]],
        '洶' : [12, 13, 0, -2, 12, [2048, 2048, 36832, 20768, 10784, 33952, 19104, 4512, 16544, 24480, 32800, 32960, 0]],
        '洫' : [12, 13, 0, -2, 12, [128, 256, 33280, 20448, 2720, 35488, 19104, 2720, 19104, 19104, 35488, 40928, 0]],
        '洽' : [12, 13, 0, -2, 12, [0, 960, 33824, 18432, 0, 34784, 16384, 0, 2016, 9248, 17440, 34784, 0]],
        '洸' : [12, 13, 0, -2, 12, [256, 256, 35104, 17728, 256, 33024, 24544, 640, 640, 8832, 17568, 39136, 0]],
        '洙' : [12, 13, 0, -2, 12, [256, 2304, 35072, 20448, 4352, 37120, 20448, 256, 17280, 17728, 39200, 33024, 0]],
        '洵' : [12, 13, 0, -2, 12, [2048, 2048, 36832, 20512, 4128, 36640, 18720, 3872, 18720, 20256, 32832, 33152, 0]],
        '洳' : [12, 13, 0, -2, 12, [2048, 2048, 35040, 24224, 2720, 35488, 19104, 4768, 23712, 18080, 35232, 37088, 0]],
        '洒' : [12, 13, 0, -2, 12, [0, 8160, 33408, 17024, 4064, 35488, 19104, 2720, 3296, 10272, 18464, 36832, 0]],
        '洌' : [12, 13, 0, -2, 12, [0, 16160, 34976, 18592, 3744, 37536, 21152, 2720, 17568, 17440, 34848, 45152, 0]],
        '浣' : [12, 13, 0, -2, 12, [256, 256, 40928, 20512, 4064, 32768, 16384, 8160, 17024, 17024, 33952, 39136, 0]],
        '涓' : [12, 13, 0, -2, 12, [0, 4064, 34848, 20448, 0, 36832, 18464, 4064, 2080, 12256, 18464, 34912, 0]],
        '浤' : [12, 13, 0, -2, 12, [512, 512, 40928, 21024, 4640, 36832, 17536, 1152, 1280, 10528, 18976, 35808, 0]],
        '浚' : [12, 13, 0, -2, 12, [1024, 1088, 34880, 24544, 640, 33952, 22624, 960, 19520, 17536, 33664, 40032, 0]],
        '浹' : [12, 13, 0, -2, 12, [256, 256, 40928, 16640, 1344, 34112, 19360, 4384, 16640, 17024, 33856, 38944, 0]],
        '浙' : [12, 13, 0, -2, 12, [0, 2144, 35200, 23808, 2304, 35296, 18720, 3360, 22816, 18976, 35360, 38944, 0]],
        '涎' : [12, 13, 0, -2, 12, [0, 7648, 33856, 18496, 4160, 40288, 17728, 5440, 18752, 18912, 37888, 37856, 0]],
        '涕' : [12, 13, 0, -2, 12, [4224, 2176, 33024, 24512, 576, 40896, 20992, 8160, 17952, 18976, 37568, 33280, 0]],
        '濤' : [12, 13, 0, -2, 12, [256, 256, 36832, 16640, 8160, 32800, 20416, 256, 24544, 18976, 35488, 36448, 0]],
        '涅' : [12, 13, 0, -2, 12, [0, 4064, 34848, 20448, 2080, 34848, 20448, 256, 4064, 8448, 16640, 36832, 0]],
        '淹' : [12, 13, 0, -2, 12, [256, 256, 40928, 17472, 2336, 40928, 18720, 4064, 2336, 12256, 16640, 33248, 0]],
        '渕' : [12, 13, 0, -2, 12, [4352, 2336, 33312, 24480, 1184, 33952, 24480, 1184, 17568, 18976, 37152, 41056, 0]],
        '渊' : [12, 13, 0, -2, 12, [0, 2336, 35104, 19808, 2976, 35104, 20448, 2336, 18848, 19296, 38176, 37152, 0]],
        '涵' : [12, 13, 0, -2, 12, [0, 8160, 33024, 16896, 5920, 37216, 21920, 4896, 21920, 22880, 37152, 40928, 0]],
        '淇' : [12, 13, 0, -2, 12, [1088, 1088, 40928, 17472, 1984, 33856, 18368, 1088, 24544, 16384, 33856, 38944, 0]],
        '淦' : [12, 13, 0, -2, 12, [0, 1920, 34880, 20512, 4064, 33024, 20448, 256, 18720, 17728, 33024, 40928, 0]],
        '涸' : [12, 13, 0, -2, 12, [0, 4064, 35104, 18720, 4064, 35104, 19360, 2720, 2720, 11168, 18464, 36832, 0]],
        '淆' : [12, 13, 0, -2, 12, [0, 3296, 33536, 23744, 288, 40928, 16896, 2016, 19488, 22496, 33824, 33888, 0]],
        '淬' : [12, 13, 0, -2, 12, [256, 256, 40928, 17472, 1088, 35488, 20768, 256, 24544, 16640, 33024, 33024, 0]],
        '淞' : [12, 13, 0, -2, 12, [0, 2496, 34880, 23872, 2336, 35360, 19104, 6272, 3200, 10528, 18720, 35808, 0]],
        '淌' : [12, 13, 0, -2, 12, [256, 256, 34112, 18720, 256, 36832, 18464, 2976, 2720, 10912, 19360, 34912, 0]],
        '淨' : [12, 13, 0, -2, 12, [0, 4064, 35392, 20768, 4096, 36832, 16672, 8160, 16672, 20448, 33024, 34560, 0]],
        '淒' : [12, 13, 0, -2, 12, [512, 512, 40928, 16960, 8160, 33344, 20416, 512, 24544, 17472, 34688, 39008, 0]],
        '淅' : [12, 13, 0, -2, 12, [0, 2144, 35200, 24320, 2304, 35296, 19744, 6944, 10528, 2592, 18976, 34848, 0]],
        '淺' : [12, 13, 0, -2, 12, [1024, 1056, 40928, 17536, 800, 36064, 17536, 8160, 17472, 17024, 33568, 40160, 0]],
        '淙' : [12, 13, 0, -2, 12, [256, 256, 40928, 20512, 4032, 32768, 16384, 8160, 16640, 18752, 37152, 33536, 0]],
        '淤' : [12, 13, 0, -2, 12, [2048, 2240, 35104, 24064, 2048, 34944, 20032, 2592, 20992, 21120, 33344, 35872, 0]],
        '淕' : [12, 13, 0, -2, 12, [256, 256, 36832, 16640, 8160, 33408, 17056, 3296, 256, 12256, 16640, 40928, 0]],
        '淪' : [12, 13, 0, -2, 12, [0, 896, 33856, 18464, 4064, 32768, 20448, 2720, 2720, 12256, 19104, 35488, 0]],
        '淮' : [12, 13, 0, -2, 12, [1024, 1152, 33920, 20448, 2304, 40928, 18688, 2304, 4064, 10496, 18688, 36832, 0]],
        '渭' : [12, 13, 0, -2, 12, [0, 8160, 37152, 24544, 4384, 40928, 18496, 4032, 2112, 12224, 18496, 35008, 0]],
        '湮' : [12, 13, 0, -2, 12, [0, 8160, 33408, 20448, 2720, 35488, 20448, 256, 4064, 8448, 16640, 40928, 0]],
        '渮' : [12, 13, 0, -2, 12, [2176, 2176, 40928, 18560, 0, 40928, 16448, 3648, 2624, 11840, 16448, 32960, 0]],
        '渙' : [12, 13, 0, -2, 12, [0, 1984, 34944, 24544, 2720, 35488, 19680, 2336, 24544, 17024, 33856, 38944, 0]],
        '湲' : [12, 13, 0, -2, 12, [0, 4064, 35392, 20768, 4064, 33280, 24544, 512, 18368, 23104, 33152, 36448, 0]],
        '湟' : [12, 13, 0, -2, 12, [256, 512, 36800, 18496, 4032, 34880, 18496, 8160, 16640, 20416, 33024, 40928, 0]],
        '渾' : [12, 13, 0, -2, 12, [0, 8160, 37152, 24544, 2336, 36832, 18720, 2336, 20448, 16640, 40928, 33024, 0]],
        '渣' : [12, 13, 0, -2, 12, [512, 512, 40928, 19072, 4672, 36832, 18496, 4032, 18496, 20416, 32768, 40928, 0]],
        '湫' : [12, 13, 0, -2, 12, [128, 896, 40064, 17568, 1184, 40864, 17536, 3264, 19776, 21824, 38432, 33824, 0]],
        '渫' : [12, 13, 0, -2, 12, [512, 2624, 40928, 19008, 3008, 34816, 20448, 256, 24544, 17152, 34112, 39200, 0]],
        '湶' : [12, 13, 0, -2, 12, [256, 512, 36800, 18496, 4032, 34880, 20416, 288, 23872, 17792, 35136, 37664, 0]],
        '湍' : [12, 13, 0, -2, 12, [256, 2336, 35104, 20448, 0, 36832, 16896, 4064, 2720, 10912, 19104, 34912, 0]],
        '渟' : [12, 13, 0, -2, 12, [512, 512, 40928, 18496, 4032, 32768, 24544, 4128, 20448, 16640, 33024, 34560, 0]],
        '湃' : [12, 13, 0, -2, 12, [0, 8160, 34944, 18560, 7136, 34944, 18560, 7296, 19424, 18560, 36992, 36992, 0]],
        '渺' : [12, 13, 0, -2, 12, [128, 7296, 38016, 21696, 7840, 38016, 21632, 7552, 5120, 13344, 23744, 33536, 0]],
        '湎' : [12, 13, 0, -2, 12, [0, 8160, 33280, 16896, 8160, 38048, 22432, 5280, 22432, 21664, 38048, 40928, 0]],
        '渤' : [12, 13, 0, -2, 12, [2176, 2176, 48768, 18912, 16032, 41632, 23712, 1184, 18720, 32544, 34848, 39104, 0]],
        '滿' : [12, 13, 0, -2, 12, [2112, 2112, 40928, 18496, 4032, 33024, 24544, 4384, 23968, 21856, 39712, 37216, 0]],
        '渝' : [12, 13, 0, -2, 12, [0, 8064, 33856, 18464, 6112, 32768, 24224, 4768, 8000, 4928, 24224, 37536, 0]],
        '游' : [12, 13, 0, -2, 12, [2304, 2304, 35296, 24064, 2528, 34848, 20032, 2624, 21472, 21056, 33344, 36032, 0]],
        '溂' : [12, 13, 0, -2, 12, [2048, 2080, 48800, 18592, 16032, 43680, 27296, 16032, 18592, 23072, 43296, 34912, 0]],
        '溪' : [12, 13, 0, -2, 12, [0, 4064, 35392, 20768, 1152, 35104, 24544, 256, 24544, 17024, 33856, 38944, 0]],
        '溘' : [12, 13, 0, -2, 12, [256, 256, 36832, 16640, 8160, 33280, 17472, 4064, 16384, 20448, 35488, 40928, 0]],
        '滉' : [12, 13, 0, -2, 12, [0, 4064, 34848, 20448, 2080, 36832, 18720, 1344, 24544, 17024, 33952, 39136, 0]],
        '溷' : [12, 13, 0, -2, 12, [0, 8160, 36896, 24544, 4640, 38240, 22944, 4896, 21920, 22880, 37408, 40928, 0]],
        '滓' : [12, 13, 0, -2, 12, [256, 256, 40928, 20768, 256, 40928, 17472, 1088, 24544, 16640, 40928, 33024, 0]],
        '溽' : [12, 13, 0, -2, 12, [0, 8160, 36864, 24544, 5440, 38016, 24160, 4160, 24544, 20544, 33856, 33472, 0]],
        '溯' : [12, 13, 0, -2, 12, [1024, 9696, 43296, 32544, 2528, 43296, 26912, 16352, 2336, 10528, 21024, 37472, 0]],
        '滄' : [12, 13, 0, -2, 12, [0, 960, 33824, 20448, 6176, 36832, 18464, 4064, 18432, 22496, 37920, 34784, 0]],
        '溲' : [12, 13, 0, -2, 12, [256, 1280, 39264, 20768, 7520, 37152, 23904, 256, 24544, 17472, 33664, 40032, 0]],
        '滔' : [12, 13, 0, -2, 12, [0, 4064, 34816, 21056, 4384, 32768, 16896, 3296, 2080, 12000, 18464, 36832, 0]],
        '滕' : [12, 13, 0, -2, 12, [256, 58688, 43296, 45024, 57920, 49120, 42048, 59680, 46400, 42368, 43328, 45856, 0]],
        '溏' : [12, 13, 0, -2, 12, [256, 256, 40928, 20768, 8160, 37152, 22496, 4352, 22496, 25632, 42016, 34784, 0]],
        '溥' : [12, 13, 0, -2, 12, [256, 288, 40928, 16640, 4064, 35104, 20448, 2336, 16448, 24544, 33856, 33472, 0]],
        '滂' : [12, 13, 0, -2, 12, [512, 512, 40928, 17536, 1152, 40928, 21024, 4064, 17408, 18400, 34848, 35008, 0]],
        '溟' : [12, 13, 0, -2, 12, [0, 8160, 36896, 20416, 2112, 36800, 18496, 4032, 16640, 24544, 33856, 38944, 0]],
        '潁' : [12, 13, 0, -2, 12, [0, 33760, 39040, 58336, 35360, 31712, 8736, 59360, 10784, 29664, 43328, 26144, 0]],
        '漑' : [12, 13, 0, -2, 12, [2048, 5088, 40512, 21056, 7744, 37856, 24128, 4160, 23744, 20800, 37440, 40032, 0]],
        '灌' : [12, 13, 0, -2, 12, [1152, 1152, 40928, 19104, 3808, 33920, 20448, 6400, 20448, 18688, 35072, 36832, 0]],
        '滬' : [12, 13, 0, -2, 12, [0, 8160, 32768, 20448, 2080, 36832, 19008, 4064, 19616, 22496, 37888, 34784, 0]],
        '滸' : [12, 13, 0, -2, 12, [256, 7424, 33248, 23872, 320, 40000, 16448, 992, 23616, 21568, 37952, 40000, 0]],
        '滾' : [12, 13, 0, -2, 12, [256, 256, 40928, 17536, 2112, 40928, 18496, 4032, 17696, 22848, 34944, 36448, 0]],
        '漿' : [12, 13, 0, -2, 12, [0, 37344, 63136, 320, 63456, 21568, 37440, 1216, 65056, 5440, 9344, 52320, 0]],
        '滲' : [12, 13, 0, -2, 12, [512, 1088, 40928, 17536, 2336, 40928, 17024, 1120, 22912, 17952, 32960, 36608, 0]],
        '漱' : [12, 13, 0, -2, 12, [2304, 2304, 48608, 19104, 16032, 43680, 27264, 16000, 18752, 22848, 43552, 35872, 0]],
        '滯' : [12, 13, 0, -2, 12, [1344, 1344, 40928, 17728, 7008, 32768, 24544, 4384, 24544, 18720, 35168, 33024, 0]],
        '漲' : [12, 13, 0, -2, 12, [0, 15328, 35328, 19392, 14848, 41920, 25088, 16352, 19072, 19104, 35392, 45856, 0]],
        '滌' : [12, 13, 0, -2, 12, [512, 2560, 35808, 21568, 4416, 46464, 22112, 5248, 22496, 21888, 37568, 40096, 0]],
        '漾' : [12, 13, 0, -2, 12, [1088, 1152, 40928, 16640, 4064, 33024, 24544, 256, 23968, 17600, 34976, 37248, 0]],
        '漓' : [12, 13, 0, -2, 12, [256, 256, 40928, 17024, 2336, 35488, 20448, 512, 24544, 21152, 38816, 36960, 0]],
        '滷' : [12, 13, 0, -2, 12, [512, 960, 33280, 24544, 4384, 37984, 21152, 6432, 21152, 21600, 37152, 40928, 0]],
        '澆' : [12, 13, 0, -2, 12, [256, 256, 36832, 16640, 8160, 33856, 20192, 1088, 24544, 17024, 33952, 39136, 0]],
        '潺' : [12, 13, 0, -2, 12, [0, 8160, 36896, 24544, 4224, 40928, 20736, 6112, 20768, 28640, 41536, 34496, 0]],
        '潸' : [12, 13, 0, -2, 12, [1152, 1152, 40928, 17536, 3488, 38624, 17536, 4064, 2080, 12256, 18464, 34912, 0]],
        '澁' : [12, 13, 0, -2, 12, [256, 256, 35264, 18688, 2304, 40928, 17536, 1152, 22240, 21632, 38016, 49120, 0]],
        '澀' : [12, 13, 0, -2, 12, [0, 16352, 38048, 22176, 9504, 42272, 22720, 1152, 22240, 21632, 38016, 49120, 0]],
        '潯' : [12, 13, 0, -2, 12, [0, 4032, 32832, 24544, 64, 40928, 18720, 7648, 16448, 24544, 33856, 33472, 0]],
        '潛' : [12, 13, 0, -2, 12, [1152, 1152, 40928, 21632, 8160, 35136, 21088, 4032, 18496, 20416, 34880, 36800, 0]],
        '濳' : [12, 13, 0, -2, 12, [1152, 5248, 40928, 17536, 8160, 35136, 21088, 4032, 18496, 20416, 34880, 36800, 0]],
        '潭' : [12, 13, 0, -2, 12, [0, 8160, 33920, 24544, 4768, 37536, 24544, 2112, 20416, 18496, 40928, 33024, 0]],
        '澂' : [12, 13, 0, -2, 12, [1152, 5248, 38112, 24480, 32, 40736, 16544, 8096, 17472, 24128, 33952, 40736, 0]],
        '潼' : [12, 13, 0, -2, 12, [256, 256, 40928, 17472, 1088, 40928, 18720, 4064, 18720, 20448, 33024, 40928, 0]],
        '潘' : [12, 13, 0, -2, 12, [0, 8160, 35136, 17792, 8160, 34112, 18720, 8160, 18720, 20448, 35104, 36832, 0]],
        '澎' : [12, 13, 0, -2, 12, [1024, 1056, 48960, 17536, 7936, 32800, 24384, 4480, 24320, 18976, 35392, 49024, 0]],
        '澑' : [12, 13, 0, -2, 12, [0, 8160, 33408, 24288, 4768, 40672, 17024, 8160, 20768, 24544, 37152, 40928, 0]],
        '濂' : [12, 13, 0, -2, 12, [256, 256, 40928, 21632, 8160, 37536, 24544, 4768, 22496, 25216, 42688, 35488, 0]],
        '潦' : [12, 13, 0, -2, 12, [256, 256, 49120, 21664, 4032, 39008, 20416, 2112, 4032, 8448, 18752, 37664, 0]],
        '澳' : [12, 13, 0, -2, 12, [256, 512, 36832, 19808, 2976, 36192, 19360, 3424, 16640, 24544, 33856, 38944, 0]],
        '澣' : [12, 13, 0, -2, 12, [2048, 2496, 48672, 18432, 16352, 41536, 32320, 9184, 32320, 18496, 48704, 34880, 0]],
        '澡' : [12, 13, 0, -2, 12, [0, 1984, 33856, 20448, 2720, 35488, 20192, 256, 24544, 17152, 34112, 39200, 0]],
        '澤' : [12, 13, 0, -2, 12, [0, 4064, 35488, 19104, 4064, 33024, 20448, 1088, 24544, 16640, 36832, 33024, 0]],
        '澹' : [12, 13, 0, -2, 12, [0, 1984, 34944, 24544, 2368, 36448, 18560, 4064, 18432, 22496, 37920, 34784, 0]],
        '濆' : [12, 13, 0, -2, 12, [256, 256, 40928, 16640, 1344, 40928, 17472, 4064, 18464, 20448, 33856, 38944, 0]],
        '澪' : [12, 13, 0, -2, 12, [0, 8160, 33024, 24544, 4384, 38752, 17472, 4064, 20480, 20448, 33312, 33376, 0]],
        '濟' : [12, 13, 0, -2, 12, [512, 512, 49120, 21824, 5952, 44448, 18496, 4032, 18496, 20416, 36928, 36928, 0]],
        '濕' : [12, 13, 0, -2, 12, [0, 4064, 34848, 20448, 2080, 36832, 17472, 2720, 20192, 16384, 35488, 37536, 0]],
        '濬' : [12, 13, 0, -2, 12, [512, 960, 33280, 24544, 4384, 35648, 21664, 4032, 22624, 20416, 34880, 36800, 0]],
        '濔' : [12, 13, 0, -2, 12, [0, 8160, 35136, 20768, 8160, 37152, 23392, 5536, 23392, 21920, 39776, 37152, 0]],
        '濘' : [12, 13, 0, -2, 12, [256, 256, 40928, 21792, 3136, 38816, 16384, 4064, 19104, 24544, 33024, 34560, 0]],
        '濱' : [12, 13, 0, -2, 12, [512, 512, 40928, 20512, 4064, 37440, 20416, 6208, 20416, 18496, 36800, 45152, 0]],
        '濮' : [12, 13, 0, -2, 12, [640, 4768, 40928, 21056, 12864, 40928, 20736, 6112, 20736, 24544, 37440, 39968, 0]],
        '濛' : [12, 13, 0, -2, 12, [1152, 1152, 40928, 17536, 8160, 36896, 24512, 1152, 22944, 17088, 40096, 33536, 0]],
        '瀉' : [12, 13, 0, -2, 12, [512, 512, 40928, 21024, 7392, 34880, 19904, 2112, 24544, 16416, 38176, 42336, 0]],
        '瀋' : [12, 13, 0, -2, 12, [512, 512, 40928, 20512, 8160, 35136, 17792, 8128, 18784, 24512, 43328, 36800, 0]],
        '濺' : [12, 13, 0, -2, 12, [256, 7456, 38880, 23840, 5312, 40736, 21824, 6112, 23840, 16704, 38080, 41760, 0]],
        '瀑' : [12, 13, 0, -2, 12, [0, 4064, 34848, 20448, 640, 36832, 17024, 8160, 1088, 10528, 17728, 35104, 0]],
        '瀁' : [12, 13, 0, -2, 12, [1088, 1152, 40928, 16640, 8160, 33856, 20448, 5184, 1984, 9568, 17536, 36448, 0]],
        '瀏' : [12, 13, 0, -2, 12, [0, 7072, 37536, 22176, 6304, 36512, 20896, 11936, 1184, 11808, 21792, 40800, 0]],
        '濾' : [12, 13, 0, -2, 12, [896, 512, 40928, 21024, 8128, 38208, 22464, 5440, 22464, 17664, 37952, 41888, 0]],
        '瀛' : [12, 13, 0, -2, 12, [256, 256, 49120, 20480, 8160, 34880, 24544, 5280, 24480, 21728, 40864, 46368, 0]],
        '瀚' : [12, 13, 0, -2, 12, [2048, 3008, 48160, 19424, 15648, 42848, 32160, 9504, 32096, 19360, 48416, 35680, 0]],
        '潴' : [12, 13, 0, -2, 12, [128, 1152, 38368, 18560, 2208, 38880, 17536, 3552, 22304, 17888, 34080, 39392, 0]],
        '瀝' : [12, 13, 0, -2, 12, [0, 8160, 38592, 21056, 8160, 37440, 22240, 6976, 20736, 21952, 34048, 40928, 0]],
        '瀘' : [12, 13, 0, -2, 12, [960, 512, 40928, 20768, 6080, 37120, 22496, 5440, 21824, 20448, 35488, 40928, 0]],
        '瀟' : [12, 13, 0, -2, 12, [1152, 1152, 40928, 16672, 8160, 33056, 20448, 2336, 20448, 18720, 36576, 39584, 0]],
        '瀰' : [12, 13, 0, -2, 12, [0, 16352, 34112, 18720, 16352, 41248, 28000, 15264, 19808, 19360, 36192, 45344, 0]],
        '瀾' : [12, 13, 0, -2, 12, [0, 7904, 37536, 24288, 4768, 40672, 20768, 8160, 21792, 22496, 38176, 39264, 0]],
        '瀲' : [12, 13, 0, -2, 12, [128, 7808, 41440, 24224, 160, 40736, 21792, 5536, 24384, 19008, 38048, 43808, 0]],
        '灑' : [12, 13, 0, -2, 12, [0, 7904, 32768, 24288, 2720, 33024, 20448, 2720, 20448, 21632, 38816, 36064, 0]],
        '灣' : [12, 13, 0, -2, 12, [512, 5952, 34848, 22336, 10400, 38720, 21856, 8128, 16448, 24544, 36896, 33216, 0]],
        '炙' : [12, 13, 0, -2, 12, [0, 4064, 29248, 2432, 1536, 63488, 1024, 9280, 17536, 2560, 12672, 49248, 0]],
        '炒' : [12, 13, 0, -2, 12, [256, 8448, 8448, 42304, 43296, 41216, 8448, 8960, 20512, 20544, 33152, 36352, 0]],
        '炯' : [12, 13, 0, -2, 12, [0, 12256, 10272, 43040, 43936, 43680, 10912, 10912, 23456, 18464, 34848, 34912, 0]],
        '烱' : [12, 13, 0, -2, 12, [0, 12256, 10912, 43680, 43680, 44256, 10272, 11168, 23200, 19360, 34848, 34912, 0]],
        '炬' : [12, 13, 0, -2, 12, [0, 10208, 9216, 44032, 47072, 42016, 9248, 10208, 21504, 19456, 33792, 34784, 0]],
        '炸' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 42240, 43264, 43488, 8448, 8448, 20960, 18688, 35072, 33024, 0]],
        '炳' : [12, 13, 0, -2, 12, [0, 12256, 8448, 41216, 45024, 43296, 10528, 10528, 23200, 19552, 34848, 34912, 0]],
        '炮' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 43040, 44960, 42144, 9376, 10144, 21600, 19456, 33824, 33760, 0]],
        '烟' : [12, 13, 0, -2, 12, [0, 12256, 10528, 43296, 45024, 43296, 10528, 10528, 23200, 19552, 34848, 36832, 0]],
        '烋' : [12, 13, 0, -2, 12, [256, 8448, 12256, 16640, 49408, 17280, 17728, 22816, 16640, 0, 21056, 35104, 0]],
        '烝' : [12, 13, 0, -2, 12, [0, 32704, 128, 256, 63008, 5440, 9344, 50272, 16256, 0, 21056, 35104, 0]],
        '烙' : [12, 13, 0, -2, 12, [512, 8704, 9184, 42016, 43584, 41344, 8768, 11296, 22496, 21536, 33824, 34784, 0]],
        '焉' : [12, 13, 0, -2, 12, [0, 65504, 1024, 10112, 9216, 65504, 8192, 16320, 8192, 16352, 21024, 35168, 0]],
        '烽' : [12, 13, 0, -2, 12, [0, 9184, 11296, 41536, 41920, 44192, 9184, 8320, 22496, 20608, 38880, 32896, 0]],
        '焜' : [12, 13, 0, -2, 12, [0, 12256, 10272, 45024, 43040, 45024, 10368, 10368, 24288, 18560, 34976, 36576, 0]],
        '焙' : [12, 13, 0, -2, 12, [256, 8448, 12256, 41536, 41536, 45024, 8192, 8192, 22496, 19488, 33824, 34784, 0]],
        '煥' : [12, 13, 0, -2, 12, [0, 10176, 10368, 49120, 43680, 43680, 11488, 10528, 24544, 17024, 33856, 38944, 0]],
        '煕' : [12, 13, 0, -2, 12, [0, 64480, 37408, 47648, 44000, 43520, 47648, 37408, 64480, 0, 21056, 35104, 0]],
        '熈' : [12, 13, 0, -2, 12, [0, 48608, 43296, 48416, 42464, 42240, 48416, 43296, 48608, 0, 21056, 35104, 0]],
        '煦' : [12, 13, 0, -2, 12, [1024, 62464, 38880, 38944, 63264, 38176, 38176, 38688, 61632, 0, 21056, 35104, 0]],
        '煢' : [12, 13, 0, -2, 12, [4352, 21792, 38208, 10880, 17472, 65504, 32800, 32672, 2176, 65440, 2208, 2144, 0]],
        '煌' : [12, 13, 0, -2, 12, [128, 8448, 10208, 44064, 47072, 42016, 9248, 10208, 20608, 22496, 32896, 36832, 0]],
        '煖' : [12, 13, 0, -2, 12, [0, 12256, 9504, 9536, 45024, 41472, 45024, 8704, 22464, 19008, 33152, 36448, 0]],
        '煬' : [12, 13, 0, -2, 12, [0, 12224, 10304, 44992, 43072, 45024, 9216, 12256, 21152, 17568, 35104, 33472, 0]],
        '熏' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 21824, 17472, 32704, 1024, 65504, 0, 21056, 35104, 0]],
        '燻' : [12, 13, 0, -2, 12, [0, 12256, 8448, 49120, 44448, 43296, 12256, 8448, 24544, 16384, 35488, 37536, 0]],
        '熄' : [12, 13, 0, -2, 12, [128, 8448, 10208, 44064, 47072, 42016, 9248, 10208, 20736, 21568, 37920, 33664, 0]],
        '熕' : [12, 13, 0, -2, 12, [0, 12256, 8448, 42976, 44064, 47072, 9248, 10208, 21536, 22496, 33344, 35872, 0]],
        '熨' : [12, 13, 0, -2, 12, [64, 64576, 34784, 64576, 36928, 21568, 45760, 0, 9280, 17536, 6912, 57568, 0]],
        '熬' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 8768, 64576, 9536, 30848, 35136, 13856, 0, 21056, 35104, 0]],
        '燗' : [12, 13, 0, -2, 12, [0, 12000, 10912, 44768, 43680, 44768, 43936, 10912, 23456, 19104, 35744, 36512, 0]],
        '熹' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 4352, 65504, 8320, 16256, 0, 21056, 35104, 0]],
        '熾' : [12, 13, 0, -2, 12, [1024, 9376, 16288, 43648, 43744, 49024, 8352, 11936, 23104, 20032, 35488, 36640, 0]],
        '燒' : [12, 13, 0, -2, 12, [256, 8448, 12256, 41216, 49120, 42048, 12000, 9280, 24544, 17024, 33952, 39136, 0]],
        '燉' : [12, 13, 0, -2, 12, [1152, 9344, 16352, 41120, 44832, 43552, 11936, 8864, 17472, 24384, 33952, 36128, 0]],
        '燔' : [12, 13, 0, -2, 12, [0, 12256, 10560, 42368, 49120, 42304, 10528, 16352, 18720, 20448, 35104, 36832, 0]],
        '燎' : [12, 13, 0, -2, 12, [256, 8448, 16352, 43584, 42976, 44064, 14304, 9248, 22496, 16640, 35136, 37664, 0]],
        '燠' : [12, 13, 0, -2, 12, [256, 8448, 12256, 44384, 43936, 44384, 11168, 11616, 20736, 24544, 33856, 38944, 0]],
        '燬' : [12, 13, 0, -2, 12, [0, 11744, 10528, 44320, 43296, 44832, 9216, 9696, 24224, 17568, 33984, 40736, 0]],
        '燧' : [12, 13, 0, -2, 12, [0, 9152, 12352, 44064, 42976, 45312, 10912, 9664, 23200, 18816, 35840, 46048, 0]],
        '燵' : [12, 13, 0, -2, 12, [128, 8320, 13280, 43136, 42976, 45344, 11232, 8320, 23520, 18560, 35840, 46048, 0]],
        '燼' : [12, 13, 0, -2, 12, [0, 12256, 8480, 49120, 41248, 45024, 10816, 12576, 24544, 19104, 35488, 40928, 0]],
        '燹' : [12, 13, 0, -2, 12, [0, 64480, 16640, 43680, 29120, 43680, 24960, 1024, 9280, 17536, 6912, 57568, 0]],
        '燿' : [12, 13, 0, -2, 12, [0, 12256, 8864, 43872, 42656, 43808, 9344, 10208, 21632, 20448, 33920, 34784, 0]],
        '爍' : [12, 13, 0, -2, 12, [256, 10784, 10176, 43680, 42944, 43680, 12256, 8448, 24544, 17152, 34112, 39200, 0]],
        '爐' : [12, 13, 0, -2, 12, [960, 8704, 12256, 43296, 43968, 43264, 12256, 13632, 21824, 20448, 35488, 40928, 0]],
        '爛' : [12, 13, 0, -2, 12, [0, 12000, 10912, 44768, 43680, 44768, 10400, 12256, 23200, 19424, 35488, 36000, 0]],
        '爨' : [12, 13, 0, -2, 12, [0, 28352, 19008, 28352, 19008, 65504, 43680, 17472, 65504, 42144, 6912, 57568, 0]],
        '爭' : [12, 13, 0, -2, 12, [0, 32704, 18560, 33856, 32736, 1088, 65504, 1088, 32704, 1024, 1024, 7168, 0]],
        '爬' : [12, 13, 0, -2, 12, [0, 65504, 43680, 43680, 44000, 43520, 43552, 43488, 41984, 41984, 41728, 41184, 0]],
        '爰' : [12, 13, 0, -2, 12, [0, 32704, 18560, 33856, 49056, 1024, 65504, 2048, 8064, 59520, 1792, 30944, 0]],
        '爲' : [12, 13, 0, -2, 12, [0, 32704, 18560, 33856, 49056, 8320, 16320, 8256, 16352, 16416, 38176, 9568, 0]],
        '爻' : [12, 13, 0, -2, 12, [0, 24640, 6528, 1536, 6528, 57440, 4352, 4352, 2560, 1024, 6912, 57568, 0]],
        '爼' : [12, 13, 0, -2, 12, [0, 35808, 21024, 8736, 21472, 35360, 544, 35808, 21024, 8736, 21024, 36832, 0]],
        '爿' : [12, 13, 0, -2, 12, [128, 8320, 8320, 8320, 16256, 128, 128, 32640, 4224, 4224, 8320, 49280, 0]],
        '牀' : [12, 13, 0, -2, 12, [128, 36992, 36992, 40928, 61568, 4224, 4480, 62144, 21664, 22656, 36992, 36992, 0]],
        '牆' : [12, 13, 0, -2, 12, [256, 41216, 45024, 42304, 58688, 11168, 8448, 61408, 43680, 43936, 43040, 45024, 0]],
        '牋' : [12, 13, 0, -2, 12, [512, 41504, 45024, 41536, 61856, 36448, 33920, 61408, 42048, 41600, 41760, 48352, 0]],
        '牘' : [12, 13, 0, -2, 12, [256, 41216, 45024, 41216, 65504, 35488, 36832, 58432, 42944, 42048, 42944, 47200, 0]],
        '牴' : [12, 13, 0, -2, 12, [0, 41056, 42880, 62592, 42112, 42976, 9280, 13376, 58400, 10016, 8192, 12256, 0]],
        '牾' : [12, 13, 0, -2, 12, [0, 42976, 41216, 63456, 41504, 41504, 10208, 12288, 59360, 9248, 9248, 10208, 0]],
        '犂' : [12, 13, 0, -2, 12, [512, 64000, 5088, 64800, 12576, 23072, 38080, 17408, 32704, 33792, 65504, 1024, 0]],
        '犁' : [12, 13, 0, -2, 12, [0, 64800, 4384, 64800, 12576, 22816, 37984, 17408, 32704, 33792, 65504, 1024, 0]],
        '犇' : [12, 13, 0, -2, 12, [1024, 17408, 32704, 17408, 65504, 1024, 9344, 41600, 64480, 8320, 64480, 8320, 0]],
        '犒' : [12, 13, 0, -2, 12, [256, 41216, 45024, 62528, 42944, 40960, 12256, 14368, 60320, 10912, 11168, 10336, 0]],
        '犖' : [12, 13, 0, -2, 12, [4352, 21792, 38208, 10880, 17472, 65504, 42016, 49056, 9216, 17408, 65504, 1024, 0]],
        '犢' : [12, 13, 0, -2, 12, [256, 41216, 45024, 61696, 45024, 43680, 12256, 13376, 59328, 9280, 10176, 14432, 0]],
        '犧' : [12, 13, 0, -2, 12, [1088, 42112, 45024, 61696, 45024, 42112, 16352, 10912, 63648, 11840, 8864, 11552, 0]],
        '犹' : [12, 13, 0, -2, 12, [256, 8480, 41248, 20448, 16640, 41344, 8832, 25216, 42112, 9376, 10400, 51424, 0]],
        '犲' : [12, 13, 0, -2, 12, [64, 8256, 41024, 20448, 16448, 41152, 8512, 25152, 42048, 10304, 8256, 49600, 0]],
        '狃' : [12, 13, 0, -2, 12, [0, 12224, 41536, 16960, 16960, 41536, 12256, 25664, 42048, 9280, 9280, 57312, 0]],
        '狆' : [12, 13, 0, -2, 12, [256, 8448, 41216, 20448, 18720, 43296, 10528, 28640, 41216, 8448, 8448, 49408, 0]],
        '狄' : [12, 13, 0, -2, 12, [256, 8448, 41216, 17696, 17696, 43328, 10560, 24832, 41600, 8832, 9280, 55328, 0]],
        '狎' : [12, 13, 0, -2, 12, [0, 12256, 43296, 18720, 20448, 43296, 10528, 26912, 45024, 8448, 8448, 49408, 0]],
        '狒' : [12, 13, 0, -2, 12, [640, 8832, 45024, 17056, 20448, 43648, 10880, 28640, 41632, 8864, 9408, 55424, 0]],
        '狢' : [12, 13, 0, -2, 12, [512, 8704, 41952, 17440, 19008, 41344, 8768, 25632, 45024, 9248, 9248, 51168, 0]],
        '狠' : [12, 13, 0, -2, 12, [0, 12224, 43072, 20416, 18496, 43072, 12224, 26912, 43328, 10368, 10304, 52768, 0]],
        '狡' : [12, 13, 0, -2, 12, [256, 8448, 45024, 16384, 17472, 43040, 8192, 25664, 41600, 8576, 8768, 52256, 0]],
        '狹' : [12, 13, 0, -2, 12, [256, 8448, 45024, 16640, 17728, 42304, 11168, 26912, 41216, 8832, 9280, 55328, 0]],
        '狷' : [12, 13, 0, -2, 12, [0, 10208, 42016, 18400, 16384, 42976, 9248, 26592, 42016, 10208, 9248, 50272, 0]],
        '倏' : [12, 13, 0, -2, 12, [512, 8704, 9184, 21568, 23104, 54144, 23904, 20800, 24544, 20736, 18048, 22624, 0]],
        '猗' : [12, 13, 0, -2, 12, [256, 8448, 45024, 17024, 17472, 43040, 12256, 24608, 42784, 9504, 10016, 49248, 0]],
        '猊' : [12, 13, 0, -2, 12, [256, 9824, 42016, 17440, 18144, 42016, 9248, 26592, 41600, 8832, 9376, 55520, 0]],
        '猜' : [12, 13, 0, -2, 12, [256, 8448, 45024, 16640, 20448, 41216, 12256, 26912, 43296, 12256, 10272, 51296, 0]],
        '猖' : [12, 13, 0, -2, 12, [0, 10176, 42048, 18368, 17472, 42944, 8192, 28640, 43040, 12256, 10272, 53216, 0]],
        '猝' : [12, 13, 0, -2, 12, [256, 8448, 45024, 17472, 17472, 43680, 10528, 24832, 45024, 8448, 8448, 49408, 0]],
        '猴' : [12, 13, 0, -2, 12, [0, 10176, 42048, 18496, 20448, 47616, 11232, 26752, 45024, 10368, 10560, 52768, 0]],
        '猯' : [12, 13, 0, -2, 12, [256, 10528, 43296, 20448, 16384, 45024, 8704, 28640, 43680, 10912, 10912, 51296, 0]],
        '猩' : [12, 13, 0, -2, 12, [0, 10208, 42016, 18400, 17440, 42976, 9344, 26592, 43136, 10208, 8320, 53216, 0]],
        '猥' : [12, 13, 0, -2, 12, [0, 12256, 43296, 20448, 18720, 45024, 8192, 28640, 42272, 9536, 9344, 52832, 0]],
        '猾' : [12, 13, 0, -2, 12, [0, 10176, 42048, 17856, 17728, 45024, 10272, 26560, 42048, 10176, 9280, 50368, 0]],
        '獎' : [12, 13, 0, -2, 12, [0, 37344, 63136, 4416, 63456, 21568, 37568, 1056, 65504, 2560, 12672, 49248, 0]],
        '獏' : [12, 13, 0, -2, 12, [576, 8768, 45024, 16960, 18400, 42016, 10208, 25632, 45024, 8448, 8768, 52256, 0]],
        '默' : [12, 13, 0, -2, 12, [128, 64672, 42144, 60544, 47072, 42112, 64640, 8320, 64832, 320, 43552, 43552, 0]],
        '獗' : [12, 13, 0, -2, 12, [0, 16352, 42240, 21984, 24224, 43168, 10912, 32384, 43328, 10560, 12832, 53792, 0]],
        '獪' : [12, 13, 0, -2, 12, [0, 10176, 43040, 22496, 16384, 45024, 10912, 28640, 42048, 10176, 9280, 51136, 0]],
        '獨' : [12, 13, 0, -2, 12, [0, 12256, 43680, 20448, 17408, 45024, 12832, 28576, 43680, 12192, 8864, 53088, 0]],
        '獰' : [12, 13, 0, -2, 12, [256, 8448, 49120, 17696, 21568, 42912, 8192, 28640, 43680, 16352, 8448, 49920, 0]],
        '獸' : [12, 13, 0, -2, 12, [128, 64672, 42144, 64640, 44000, 63616, 43136, 64640, 320, 63808, 35360, 64544, 0]],
        '獵' : [12, 13, 0, -2, 12, [2336, 4672, 43296, 20448, 19104, 43296, 12256, 26944, 44480, 10560, 11680, 56096, 0]],
        '獻' : [12, 13, 0, -2, 12, [14464, 8352, 64672, 42112, 65504, 41088, 64640, 43136, 64832, 42304, 56864, 37920, 0]],
        '獺' : [12, 13, 0, -2, 12, [2048, 11232, 48288, 18784, 32256, 44000, 15904, 27616, 44576, 15328, 10560, 51744, 0]],
        '珈' : [12, 13, 0, -2, 12, [2048, 2048, 59616, 24224, 19104, 19104, 60064, 21152, 21152, 17056, 58016, 3296, 0]],
        '玳' : [12, 13, 0, -2, 12, [128, 672, 62112, 9344, 10208, 11392, 62592, 9280, 9280, 13376, 50208, 1056, 0]],
        '珎' : [12, 13, 0, -2, 12, [1024, 1024, 63456, 9344, 10368, 10368, 62144, 9376, 10400, 8320, 61568, 896, 0]],
        '玻' : [12, 13, 0, -2, 12, [256, 256, 61408, 18720, 18688, 20416, 59456, 19008, 18752, 20608, 58176, 3104, 0]],
        '珀' : [12, 13, 0, -2, 12, [256, 256, 61952, 10208, 9248, 9248, 62496, 10208, 9248, 13344, 50208, 2016, 0]],
        '珥' : [12, 13, 0, -2, 12, [0, 8160, 59456, 20416, 18496, 18496, 61376, 18496, 18528, 20416, 63552, 64, 0]],
        '珮' : [12, 13, 0, -2, 12, [0, 8128, 61504, 22464, 20800, 22464, 62784, 21824, 21824, 21792, 58656, 8480, 0]],
        '珞' : [12, 13, 0, -2, 12, [512, 512, 62432, 9248, 10816, 8576, 62016, 9248, 12256, 13344, 50208, 2016, 0]],
        '璢' : [12, 13, 0, -2, 12, [0, 4064, 57984, 20192, 19104, 20192, 57984, 20448, 18720, 20448, 59680, 4064, 0]],
        '琅' : [12, 13, 0, -2, 12, [256, 256, 63456, 9248, 10208, 9248, 63456, 9472, 9504, 13472, 50240, 1824, 0]],
        '瑯' : [12, 13, 0, -2, 12, [1024, 1504, 65312, 18720, 20288, 18816, 59712, 20256, 18720, 19232, 64960, 2304, 0]],
        '琥' : [12, 13, 0, -2, 12, [512, 960, 57856, 20448, 18976, 20416, 59904, 19424, 18560, 21120, 58016, 3296, 0]],
        '珸' : [12, 13, 0, -2, 12, [0, 2016, 61696, 10208, 8736, 8736, 63456, 8192, 10208, 13344, 50208, 2016, 0]],
        '琲' : [12, 13, 0, -2, 12, [128, 640, 61152, 17024, 17024, 20192, 57984, 17024, 20096, 17120, 58496, 6272, 0]],
        '琺' : [12, 13, 0, -2, 12, [128, 128, 60384, 17536, 16512, 20608, 60384, 16640, 18688, 18720, 61984, 5088, 0]],
        '瑕' : [12, 13, 0, -2, 12, [0, 8160, 61984, 21024, 24288, 20480, 61920, 24096, 20640, 24224, 61632, 4896, 0]],
        '琿' : [12, 13, 0, -2, 12, [0, 8160, 61728, 24544, 18720, 20448, 59680, 18720, 20448, 16640, 65504, 256, 0]],
        '瑟' : [12, 13, 0, -2, 12, [0, 64480, 8320, 64480, 8320, 64480, 2048, 1152, 20800, 37408, 3136, 29632, 0]],
        '瑙' : [12, 13, 0, -2, 12, [0, 2336, 53824, 25728, 21056, 18720, 61408, 19104, 18720, 19104, 60448, 4064, 0]],
        '瑁' : [12, 13, 0, -2, 12, [0, 4064, 59424, 19360, 18464, 19360, 57344, 20448, 18464, 20448, 59424, 4064, 0]],
        '瑜' : [12, 13, 0, -2, 12, [0, 8064, 58432, 18464, 22496, 16384, 65184, 21152, 24384, 21312, 65184, 4768, 0]],
        '瑩' : [12, 13, 0, -2, 12, [4352, 21792, 38208, 10880, 17472, 65504, 32800, 49056, 1024, 16256, 1088, 65504, 0]],
        '瑰' : [12, 13, 0, -2, 12, [256, 512, 61408, 18720, 20448, 18720, 59680, 20448, 17728, 17728, 59680, 2528, 0]],
        '瑣' : [12, 13, 0, -2, 12, [256, 256, 58688, 18720, 16640, 20448, 59424, 20448, 18464, 20448, 57920, 3104, 0]],
        '瑪' : [12, 13, 0, -2, 12, [0, 4064, 59648, 20416, 18688, 20416, 59648, 20448, 16416, 21792, 58656, 192, 0]],
        '瑶' : [12, 13, 0, -2, 12, [0, 4064, 57376, 17728, 17728, 18400, 57600, 20448, 16640, 18720, 59680, 4064, 0]],
        '瑾' : [12, 13, 0, -2, 12, [1152, 1152, 65504, 17536, 20448, 18720, 59680, 20448, 16640, 20448, 57600, 8160, 0]],
        '璋' : [12, 13, 0, -2, 12, [256, 256, 61408, 16960, 20448, 18464, 61408, 18464, 20448, 16640, 61408, 256, 0]],
        '璞' : [12, 13, 0, -2, 12, [1280, 5440, 65504, 17536, 17536, 24544, 57600, 20448, 16640, 24544, 58432, 6176, 0]],
        '璧' : [12, 13, 0, -2, 12, [128, 61568, 38880, 62016, 33344, 63456, 36992, 65504, 1024, 16256, 1088, 65504, 0]],
        '瓊' : [12, 13, 0, -2, 12, [0, 1984, 59520, 24544, 19104, 19520, 59360, 17440, 18400, 23104, 57728, 3680, 0]],
        '瓏' : [12, 13, 0, -2, 12, [2176, 2176, 65248, 21632, 24544, 16416, 65248, 21120, 24288, 21120, 65184, 4832, 0]],
        '瓔' : [12, 13, 0, -2, 12, [0, 3808, 60064, 20192, 19104, 20192, 58496, 18720, 24544, 17472, 59264, 6240, 0]],
        '珱' : [12, 13, 0, -2, 12, [0, 2336, 59680, 18752, 16448, 16896, 61408, 16960, 17472, 17536, 58240, 3168, 0]],
        '瓠' : [12, 13, 0, -2, 12, [8192, 10208, 64832, 21824, 36160, 62784, 1344, 64832, 17728, 64864, 35744, 12320, 0]],
        '瓣' : [12, 13, 0, -2, 12, [16448, 24384, 62944, 38176, 21792, 62784, 21984, 21824, 62528, 21984, 44864, 41280, 0]],
        '瓧' : [12, 13, 0, -2, 12, [128, 64640, 8320, 8320, 31712, 18560, 26752, 22656, 18560, 34976, 38944, 59360, 0]],
        '瓩' : [12, 13, 0, -2, 12, [0, 64608, 9088, 8320, 30848, 19424, 26752, 22656, 18560, 34976, 38944, 59360, 0]],
        '瓮' : [12, 13, 0, -2, 12, [0, 3840, 8320, 51296, 4224, 65504, 4096, 8064, 4224, 11392, 9120, 64736, 0]],
        '瓲' : [12, 13, 0, -2, 12, [256, 64768, 8544, 10112, 31008, 18720, 28640, 22784, 18720, 35040, 38912, 59360, 0]],
        '瓰' : [12, 13, 0, -2, 12, [0, 63936, 8256, 8736, 31776, 19424, 26912, 22816, 18976, 35520, 38912, 59360, 0]],
        '瓱' : [12, 13, 0, -2, 12, [0, 63584, 9088, 8448, 31712, 18688, 27104, 24320, 18720, 35040, 38912, 59360, 0]],
        '瓸' : [12, 13, 0, -2, 12, [0, 64480, 8320, 9184, 31264, 19424, 27168, 23072, 19424, 34816, 38944, 59360, 0]],
        '瓷' : [12, 13, 0, -2, 12, [1024, 17408, 10208, 2336, 10880, 17472, 65504, 4096, 8064, 10368, 10144, 63712, 0]],
        '甄' : [12, 13, 0, -2, 12, [0, 65504, 10496, 64768, 43968, 43584, 65344, 8896, 64064, 9280, 9664, 65120, 0]],
        '甃' : [12, 13, 0, -2, 12, [0, 62752, 9536, 63744, 27264, 42048, 65504, 4096, 8064, 10368, 10144, 63712, 0]],
        '甅' : [12, 13, 0, -2, 12, [0, 64480, 8704, 9184, 31392, 19424, 27296, 23520, 19584, 35808, 38912, 59360, 0]],
        '甌' : [12, 13, 0, -2, 12, [0, 65504, 33024, 47360, 43968, 47680, 33600, 64192, 43584, 64576, 34240, 65120, 0]],
        '甎' : [12, 13, 0, -2, 12, [8192, 10208, 64768, 43264, 64448, 43584, 64320, 10944, 65088, 1088, 17856, 11872, 0]],
        '甍' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 19008, 65504, 36896, 8064, 10368, 10144, 63712, 0]],
        '甕' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 42144, 20416, 63744, 12224, 18688, 65504, 9344, 17312, 64736, 0]],
        '甓' : [12, 13, 0, -2, 12, [256, 61696, 38880, 62016, 36832, 61696, 38880, 37120, 65504, 9344, 17312, 64736, 0]],
        '甞' : [12, 13, 0, -2, 12, [1024, 9344, 65504, 37152, 37152, 7936, 8320, 65504, 8320, 16256, 8320, 16256, 0]],
        '甦' : [12, 13, 0, -2, 12, [128, 63616, 8832, 64480, 44160, 63616, 44000, 63616, 41088, 26592, 12288, 53216, 0]],
        '甬' : [12, 13, 0, -2, 12, [0, 32704, 128, 256, 32736, 17440, 32736, 17440, 32736, 17440, 17440, 17504, 0]],
        '甼' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 17472, 32704, 0, 65504, 512, 512, 7680, 0]],
        '畄' : [12, 13, 0, -2, 12, [2048, 17440, 8768, 4224, 32736, 17440, 17440, 32736, 17440, 17440, 17440, 32736, 0]],
        '畍' : [12, 13, 0, -2, 12, [0, 384, 64064, 44064, 43008, 43584, 64064, 43584, 43584, 43584, 64576, 1088, 0]],
        '畊' : [12, 13, 0, -2, 12, [576, 576, 64064, 45024, 43584, 43584, 64064, 45024, 43584, 43584, 64576, 1088, 0]],
        '畉' : [12, 13, 0, -2, 12, [128, 128, 63616, 45024, 43136, 43136, 65504, 43136, 43328, 43328, 64032, 3104, 0]],
        '畛' : [12, 13, 0, -2, 12, [0, 896, 64576, 43040, 43200, 43776, 63488, 43200, 43776, 43040, 63680, 1792, 0]],
        '畆' : [12, 13, 0, -2, 12, [8192, 8320, 64640, 128, 64640, 42112, 42112, 64768, 42272, 42272, 42272, 65504, 0]],
        '畚' : [12, 13, 0, -2, 12, [2048, 2304, 4224, 32704, 1024, 65504, 4224, 16320, 58528, 16256, 9344, 16256, 0]],
        '畩' : [12, 13, 0, -2, 12, [256, 256, 65504, 43264, 43264, 43680, 64160, 44704, 43584, 43584, 64032, 1952, 0]],
        '畤' : [12, 13, 0, -2, 12, [256, 256, 65504, 43264, 43264, 45024, 63552, 45024, 43072, 44096, 64064, 192, 0]],
        '畧' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 17472, 32704, 8320, 52992, 12512, 57472, 16256, 0]],
        '畫' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1088, 65504, 1088, 65504, 9344, 16256, 9344, 9344, 65504, 0]],
        '畭' : [12, 13, 0, -2, 12, [0, 960, 64544, 43008, 45024, 43136, 63616, 45024, 43136, 43680, 64672, 384, 0]],
        '畸' : [12, 13, 0, -2, 12, [128, 128, 65504, 43136, 43328, 43552, 65504, 43040, 43936, 43680, 64416, 96, 0]],
        '當' : [12, 13, 0, -2, 12, [1024, 9344, 65504, 37152, 37152, 7936, 0, 32704, 17472, 32704, 17472, 32704, 0]],
        '疆' : [12, 13, 0, -2, 12, [0, 65504, 2720, 64480, 33440, 64480, 18432, 61408, 19104, 60384, 2720, 30688, 0]],
        '疇' : [12, 13, 0, -2, 12, [256, 256, 65504, 43264, 45024, 43040, 65472, 43264, 45024, 43552, 64160, 3680, 0]],
        '畴' : [12, 13, 0, -2, 12, [128, 128, 65504, 43136, 44000, 43136, 65504, 43296, 45024, 43296, 64160, 608, 0]],
        '疊' : [12, 13, 0, -2, 12, [0, 16256, 9344, 16256, 9344, 65504, 42144, 65504, 41120, 16256, 8320, 65504, 0]],
        '疉' : [12, 13, 0, -2, 12, [0, 16256, 9344, 16256, 9344, 65504, 42144, 65504, 37024, 15616, 512, 65504, 0]],
        '疂' : [12, 13, 0, -2, 12, [0, 16256, 9344, 16256, 57568, 6912, 8320, 65504, 41120, 16256, 8320, 65504, 0]],
        '疔' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 49152, 24544, 16640, 49408, 16640, 16640, 33024, 34560, 0]],
        '疚' : [12, 13, 0, -2, 12, [1024, 1024, 32736, 17408, 51072, 17536, 18560, 53376, 16640, 17024, 35904, 45088, 0]],
        '疝' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 49408, 16640, 20768, 53536, 20768, 20768, 37152, 40928, 0]],
        '疥' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 50944, 18560, 20544, 58528, 17536, 17536, 34944, 45184, 0]],
        '疣' : [12, 13, 0, -2, 12, [512, 512, 32736, 16896, 49696, 24544, 16896, 49664, 17664, 17696, 35104, 45280, 0]],
        '痂' : [12, 13, 0, -2, 12, [512, 512, 32736, 18432, 51424, 24224, 19104, 51872, 21152, 21152, 33440, 36064, 0]],
        '疳' : [12, 13, 0, -2, 12, [512, 512, 32736, 18496, 51264, 32736, 18496, 51264, 20416, 18496, 34880, 36800, 0]],
        '痃' : [12, 13, 0, -2, 12, [512, 512, 32736, 16896, 57312, 16896, 17472, 51328, 17664, 16928, 33824, 40928, 0]],
        '疵' : [12, 13, 0, -2, 12, [512, 512, 32736, 17536, 50304, 21664, 22464, 54400, 21632, 21632, 38816, 47328, 0]],
        '疽' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 53184, 18496, 20416, 51264, 20416, 18496, 34880, 49120, 0]],
        '疸' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 53184, 18496, 20416, 51264, 18496, 20416, 32768, 49120, 0]],
        '疼' : [12, 13, 0, -2, 12, [512, 512, 32736, 18432, 53184, 18496, 21632, 49920, 19584, 29536, 35840, 33664, 0]],
        '疱' : [12, 13, 0, -2, 12, [512, 512, 32736, 18432, 53184, 18496, 24384, 59712, 20288, 18624, 34848, 36832, 0]],
        '痍' : [12, 13, 0, -2, 12, [512, 512, 32736, 16896, 57312, 16928, 24544, 53760, 24544, 17696, 34944, 45152, 0]],
        '痊' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 57216, 17472, 18464, 55264, 16640, 20416, 33024, 40928, 0]],
        '痒' : [12, 13, 0, -2, 12, [512, 512, 32736, 17472, 50304, 24544, 16640, 53216, 16640, 24544, 33024, 33024, 0]],
        '痙' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 57312, 18720, 21056, 51488, 16384, 20448, 33024, 40928, 0]],
        '痣' : [12, 13, 0, -2, 12, [512, 512, 32736, 16896, 57312, 16896, 24544, 49664, 16704, 17440, 37888, 41920, 0]],
        '痞' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 65504, 19072, 21056, 57888, 24512, 20544, 36928, 40896, 0]],
        '痾' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 57312, 21536, 21984, 55584, 21792, 21984, 38944, 36960, 0]],
        '痿' : [12, 13, 0, -2, 12, [512, 512, 32736, 16640, 57312, 17152, 17728, 51488, 32736, 17472, 34688, 39008, 0]],
        '痼' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 57312, 21024, 24544, 53792, 22432, 21664, 38048, 40928, 0]],
        '瘁' : [12, 13, 0, -2, 12, [512, 512, 32736, 16640, 57312, 17472, 17472, 51872, 20768, 24544, 33024, 33024, 0]],
        '痰' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 51488, 20800, 18048, 55392, 16640, 18752, 34432, 47200, 0]],
        '痺' : [12, 13, 0, -2, 12, [512, 512, 32736, 21056, 57280, 21056, 21056, 57280, 17664, 18688, 49120, 33024, 0]],
        '痲' : [12, 13, 0, -2, 12, [512, 512, 32736, 20736, 53504, 32736, 20736, 54592, 21824, 21824, 38240, 37120, 0]],
        '痳' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 51328, 32736, 18560, 51584, 23232, 27808, 34944, 34944, 0]],
        '瘋' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 65472, 17472, 24384, 54592, 24384, 17440, 33952, 49056, 0]],
        '瘍' : [12, 13, 0, -2, 12, [512, 512, 32736, 18496, 53184, 18496, 20448, 50176, 24544, 17568, 35104, 37568, 0]],
        '瘉' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 57280, 18464, 24512, 62112, 24224, 21312, 40608, 37536, 0]],
        '瘟' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 53184, 18752, 19136, 51264, 24544, 21664, 38048, 49120, 0]],
        '瘧' : [12, 13, 0, -2, 12, [512, 512, 32736, 16896, 57312, 21024, 22464, 53760, 22496, 23552, 41984, 42976, 0]],
        '瘠' : [12, 13, 0, -2, 12, [512, 512, 32736, 16896, 64224, 16896, 20352, 61536, 24512, 20544, 40896, 36928, 0]],
        '瘡' : [12, 13, 0, -2, 12, [512, 512, 32736, 17536, 51776, 24544, 26688, 53184, 18496, 20448, 37920, 38880, 0]],
        '瘢' : [12, 13, 0, -2, 12, [512, 512, 32736, 18432, 56768, 21824, 24416, 54272, 32736, 21792, 38080, 38688, 0]],
        '瘤' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 56800, 20640, 21792, 57312, 20768, 24544, 37152, 40928, 0]],
        '瘴' : [12, 13, 0, -2, 12, [512, 512, 32736, 16640, 57312, 17536, 24544, 51264, 20416, 18496, 49120, 33024, 0]],
        '瘰' : [12, 13, 0, -2, 12, [512, 512, 32736, 21056, 57280, 21056, 24512, 50304, 18720, 32736, 37440, 41504, 0]],
        '瘻' : [12, 13, 0, -2, 12, [512, 512, 32736, 21056, 65504, 21056, 24512, 49664, 32736, 18496, 36736, 45152, 0]],
        '癇' : [12, 13, 0, -2, 12, [512, 512, 32736, 21152, 57056, 21152, 24544, 54432, 22432, 21664, 38816, 38048, 0]],
        '癈' : [12, 13, 0, -2, 12, [512, 512, 32736, 21664, 51264, 24032, 25920, 56640, 21472, 23840, 33984, 39712, 0]],
        '癆' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 51872, 21824, 25120, 57312, 21024, 24544, 33824, 39104, 0]],
        '癜' : [12, 13, 0, -2, 12, [512, 512, 32736, 16384, 57056, 21152, 24480, 54272, 24544, 21792, 48832, 41760, 0]],
        '癘' : [12, 13, 0, -2, 12, [512, 512, 32736, 17536, 57312, 17536, 20416, 51776, 24544, 21024, 37536, 38816, 0]],
        '癡' : [12, 13, 0, -2, 12, [512, 512, 32736, 20480, 56800, 20512, 24128, 59872, 24128, 18784, 38208, 41952, 0]],
        '癢' : [12, 13, 0, -2, 12, [512, 512, 32736, 16896, 57312, 16896, 32736, 51328, 24512, 26912, 34944, 40544, 0]],
        '癨' : [12, 13, 0, -2, 12, [512, 512, 32736, 16896, 65504, 21152, 19008, 57312, 28928, 24544, 37120, 40928, 0]],
        '癩' : [12, 13, 0, -2, 12, [512, 512, 32736, 17568, 56992, 17696, 24544, 54560, 24544, 17696, 36320, 38176, 0]],
        '癪' : [12, 13, 0, -2, 12, [512, 512, 32736, 17536, 56288, 18560, 32736, 51744, 20448, 23072, 44000, 35872, 0]],
        '癧' : [12, 13, 0, -2, 12, [512, 512, 32736, 21632, 57312, 21632, 24000, 54944, 20736, 21952, 42240, 45024, 0]],
        '癬' : [12, 13, 0, -2, 12, [512, 512, 32736, 20768, 64832, 22496, 31872, 55264, 31872, 17376, 43648, 43648, 0]],
        '癰' : [12, 13, 0, -2, 12, [512, 512, 32736, 21632, 59872, 22336, 32224, 54592, 21984, 32064, 41280, 48608, 0]],
        '癲' : [12, 13, 0, -2, 12, [512, 512, 32736, 20608, 57312, 21792, 24032, 54560, 32544, 16864, 38208, 41504, 0]],
        '癶' : [12, 13, 0, -2, 12, [0, 0, 31296, 2624, 19072, 10528, 4416, 8320, 16448, 32800, 0, 0, 0]],
        '癸' : [12, 13, 0, -2, 12, [0, 31296, 2688, 35104, 20800, 16256, 17504, 33792, 32704, 2560, 12672, 49248, 0]],
        '發' : [12, 13, 0, -2, 12, [0, 64064, 2688, 20768, 8512, 64448, 2656, 63488, 34784, 64064, 2496, 30240, 0]],
        '皀' : [12, 13, 0, -2, 12, [1024, 2048, 32704, 16448, 32704, 16448, 32704, 16384, 17280, 31776, 16416, 16352, 0]],
        '皃' : [12, 13, 0, -2, 12, [1024, 2048, 32704, 16448, 32704, 16448, 16448, 32704, 4608, 4640, 8736, 49632, 0]],
        '皈' : [12, 13, 0, -2, 12, [0, 18400, 17408, 62464, 38848, 37952, 62528, 38208, 39232, 36992, 62272, 3104, 0]],
        '皋' : [12, 13, 0, -2, 12, [1024, 2048, 32704, 16448, 32704, 16448, 65504, 4352, 9344, 32704, 33824, 1024, 0]],
        '皎' : [12, 13, 0, -2, 12, [256, 16640, 20448, 61440, 37952, 38944, 61440, 37952, 37504, 37248, 62016, 3104, 0]],
        '皖' : [12, 13, 0, -2, 12, [256, 16640, 24544, 61472, 38880, 36864, 61440, 40928, 37504, 37504, 62624, 6368, 0]],
        '皓' : [12, 13, 0, -2, 12, [128, 17536, 17536, 63456, 38016, 39040, 63456, 36864, 38880, 37920, 62496, 2016, 0]],
        '皙' : [12, 13, 0, -2, 12, [4096, 5088, 64000, 13280, 21056, 38464, 2048, 32704, 16448, 32704, 16448, 32704, 0]],
        '皚' : [12, 13, 0, -2, 12, [256, 18720, 20448, 61440, 38880, 36864, 63456, 37920, 38880, 37440, 62016, 4064, 0]],
        '皰' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 43552, 42912, 64160, 35488, 43936, 43616, 37376, 10784, 52192, 0]],
        '皴' : [12, 13, 0, -2, 12, [128, 18560, 35808, 65184, 21120, 21472, 36384, 31392, 51872, 19008, 12448, 53024, 0]],
        '皸' : [12, 13, 0, -2, 12, [128, 64640, 42976, 64160, 43648, 64480, 43552, 43680, 64160, 9280, 64672, 8992, 0]],
        '皹' : [12, 13, 0, -2, 12, [8192, 10208, 64672, 44000, 41632, 64480, 35488, 43680, 44000, 36992, 12256, 51328, 0]],
        '皺' : [12, 13, 0, -2, 12, [8320, 8320, 32736, 41632, 64128, 9184, 17440, 32416, 41632, 64064, 8864, 19744, 0]],
        '盂' : [12, 13, 0, -2, 12, [0, 32704, 1024, 1024, 65504, 1024, 7168, 0, 32704, 19008, 19008, 65504, 0]],
        '盍' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 65504, 4096, 8320, 32704, 32, 32704, 19008, 65504, 0]],
        '盖' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 16256, 1024, 65504, 0, 32704, 19008, 19008, 65504, 0]],
        '盒' : [12, 13, 0, -2, 12, [0, 7936, 8320, 24384, 32800, 16256, 8320, 16256, 0, 32704, 19008, 65504, 0]],
        '盞' : [12, 13, 0, -2, 12, [4096, 4352, 4064, 63616, 3840, 30880, 4064, 63616, 1312, 32736, 19008, 65504, 0]],
        '盡' : [12, 13, 0, -2, 12, [1024, 1024, 32640, 1152, 65504, 1152, 65504, 21056, 35104, 32704, 19008, 65504, 0]],
        '盥' : [12, 13, 0, -2, 12, [1024, 58592, 34080, 65248, 33824, 61152, 38176, 42144, 32704, 19008, 19008, 65504, 0]],
        '盧' : [12, 13, 0, -2, 12, [1024, 1984, 1024, 32736, 17440, 24512, 17408, 24512, 21056, 49120, 42272, 32736, 0]],
        '盪' : [12, 13, 0, -2, 12, [0, 20416, 10304, 36800, 18496, 8160, 1184, 18720, 37440, 32704, 19008, 65504, 0]],
        '蘯' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 9280, 38848, 17472, 8160, 17568, 35104, 32704, 19008, 65504, 0]],
        '盻' : [12, 13, 0, -2, 12, [0, 960, 61504, 37952, 63456, 39456, 37376, 62432, 37920, 37920, 61504, 384, 0]],
        '眈' : [12, 13, 0, -2, 12, [256, 256, 63456, 38176, 62752, 37152, 37248, 62080, 37504, 37504, 62624, 6368, 0]],
        '眇' : [12, 13, 0, -2, 12, [256, 256, 61696, 38208, 63776, 37120, 37120, 62208, 36896, 36928, 61824, 3584, 0]],
        '眄' : [12, 13, 0, -2, 12, [0, 2016, 61696, 37120, 62944, 38176, 38176, 62752, 38880, 36896, 61504, 384, 0]],
        '眩' : [12, 13, 0, -2, 12, [256, 256, 65504, 36992, 61696, 37408, 37952, 62080, 37120, 37408, 62496, 4064, 0]],
        '眤' : [12, 13, 0, -2, 12, [0, 2016, 62496, 37920, 63456, 37888, 38176, 62784, 38272, 38144, 63776, 2272, 0]],
        '眞' : [12, 13, 0, -2, 12, [16384, 16832, 32256, 16448, 16320, 36992, 40832, 36992, 65504, 0, 12672, 49248, 0]],
        '眥' : [12, 13, 0, -2, 12, [512, 8704, 48096, 41472, 48672, 57824, 1024, 16320, 8256, 16320, 8256, 16320, 0]],
        '眦' : [12, 13, 0, -2, 12, [256, 2304, 59648, 43296, 61248, 43392, 43264, 59648, 43264, 43296, 61216, 14560, 0]],
        '眛' : [12, 13, 0, -2, 12, [256, 256, 65504, 37120, 61696, 40928, 37120, 61696, 37760, 38208, 63776, 256, 0]],
        '眷' : [12, 13, 0, -2, 12, [1024, 9344, 24384, 35360, 4352, 65504, 8320, 32704, 41120, 16256, 8320, 16256, 0]],
        '眸' : [12, 13, 0, -2, 12, [512, 512, 62496, 38880, 61568, 38016, 38880, 62592, 36992, 40928, 61568, 128, 0]],
        '睇' : [12, 13, 0, -2, 12, [1088, 576, 57472, 45024, 57632, 45024, 43264, 61408, 41760, 42272, 59840, 256, 0]],
        '睚' : [12, 13, 0, -2, 12, [0, 4064, 59520, 44000, 59520, 43136, 45024, 59520, 44000, 45184, 57472, 4064, 0]],
        '睨' : [12, 13, 0, -2, 12, [128, 256, 63072, 37920, 63200, 37920, 37920, 63456, 37504, 37504, 62624, 6368, 0]],
        '睫' : [12, 13, 0, -2, 12, [256, 256, 61408, 41248, 65504, 41248, 45024, 57600, 43488, 43264, 61184, 12512, 0]],
        '睛' : [12, 13, 0, -2, 12, [128, 128, 63456, 36992, 63456, 36992, 40928, 62624, 38048, 38880, 62496, 1120, 0]],
        '睥' : [12, 13, 0, -2, 12, [128, 256, 63456, 38048, 63456, 38048, 38048, 63456, 37504, 38016, 65504, 128, 0]],
        '睿' : [12, 13, 0, -2, 12, [2048, 3968, 2048, 65504, 32800, 24384, 35360, 16256, 57568, 16256, 8320, 16256, 0]],
        '睾' : [12, 13, 0, -2, 12, [1024, 2048, 32704, 19008, 32704, 1024, 65504, 4352, 65504, 1024, 32704, 1024, 0]],
        '睹' : [12, 13, 0, -2, 12, [256, 256, 63456, 37152, 61760, 38880, 37184, 62432, 38432, 39904, 61984, 992, 0]],
        '瞎' : [12, 13, 0, -2, 12, [256, 256, 65504, 37152, 63456, 37120, 38848, 61696, 40928, 37952, 62528, 1984, 0]],
        '瞋' : [12, 13, 0, -2, 12, [1024, 1120, 59264, 42016, 58336, 43552, 44000, 59936, 45024, 40960, 57920, 3104, 0]],
        '瞑' : [12, 13, 0, -2, 12, [0, 4064, 63520, 38880, 62496, 38880, 37920, 63456, 37120, 40928, 62016, 3104, 0]],
        '瞠' : [12, 13, 0, -2, 12, [256, 1344, 65504, 36896, 63456, 37952, 37952, 63424, 37120, 38880, 61696, 4064, 0]],
        '瞞' : [12, 13, 0, -2, 12, [1088, 1088, 65504, 42048, 59328, 41216, 45024, 59680, 44448, 43872, 59680, 2400, 0]],
        '瞰' : [12, 13, 0, -2, 12, [128, 8064, 58592, 49056, 60064, 44576, 43680, 61088, 43584, 43584, 65184, 800, 0]],
        '瞶' : [12, 13, 0, -2, 12, [256, 256, 61408, 43296, 61408, 41216, 49120, 58432, 42944, 42048, 59328, 6240, 0]],
        '瞹' : [12, 13, 0, -2, 12, [0, 8160, 58656, 42304, 65504, 42272, 47040, 58368, 44992, 46144, 58240, 7264, 0]],
        '瞿' : [12, 13, 0, -2, 12, [0, 31200, 18720, 31200, 18720, 31200, 4608, 16352, 25088, 49088, 8704, 16352, 0]],
        '瞼' : [12, 13, 0, -2, 12, [0, 1984, 59424, 47072, 57344, 44768, 43680, 60064, 44768, 42048, 60064, 4384, 0]],
        '瞽' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8320, 65504, 35104, 20800, 65440, 8320, 16256, 8320, 16256, 0]],
        '瞻' : [12, 13, 0, -2, 12, [0, 1984, 59520, 49120, 59712, 44640, 43136, 61408, 43008, 47072, 58400, 2016, 0]],
        '矇' : [12, 13, 0, -2, 12, [1152, 1152, 65504, 42112, 61408, 43040, 44992, 57984, 44448, 41664, 60576, 768, 0]],
        '矍' : [12, 13, 0, -2, 12, [0, 31200, 18720, 18720, 32736, 8704, 32704, 41472, 16352, 4224, 3840, 61664, 0]],
        '矗' : [12, 13, 0, -2, 12, [512, 512, 65504, 512, 24512, 20544, 20544, 65504, 9504, 48608, 42272, 65504, 0]],
        '矚' : [12, 13, 0, -2, 12, [0, 16352, 57376, 49120, 59712, 49120, 42144, 65504, 43680, 44960, 49824, 24416, 0]],
        '矜' : [12, 13, 0, -2, 12, [0, 63936, 2592, 21504, 9184, 63488, 10240, 11232, 28704, 40992, 8256, 24960, 0]],
        '矣' : [12, 13, 0, -2, 12, [2048, 2304, 4224, 65472, 4128, 8064, 9216, 17408, 65504, 2560, 12672, 49248, 0]],
        '矮' : [12, 13, 0, -2, 12, [0, 36832, 61696, 45024, 8960, 62784, 10528, 12256, 21568, 21632, 33664, 35936, 0]],
        '矼' : [12, 13, 0, -2, 12, [0, 2016, 63616, 8320, 8320, 8320, 30848, 51328, 18560, 18560, 18560, 32736, 0]],
        '砌' : [12, 13, 0, -2, 12, [0, 1504, 62624, 9376, 11936, 9376, 29856, 54560, 21792, 21792, 22048, 29376, 0]],
        '砒' : [12, 13, 0, -2, 12, [128, 1152, 62592, 9376, 10176, 9344, 29824, 54400, 21632, 21664, 22432, 31968, 0]],
        '礦' : [12, 13, 0, -2, 12, [256, 256, 65504, 10816, 12256, 10816, 32736, 54432, 22496, 21664, 22496, 30752, 0]],
        '砠' : [12, 13, 0, -2, 12, [0, 992, 64032, 8736, 9184, 8736, 31264, 52192, 18976, 18976, 18976, 32736, 0]],
        '礪' : [12, 13, 0, -2, 12, [0, 4064, 63808, 12256, 10912, 11232, 31392, 53920, 22496, 21792, 21920, 29792, 0]],
        '硅' : [12, 13, 0, -2, 12, [256, 256, 63456, 8448, 8448, 12256, 28928, 53504, 22496, 20736, 20736, 32736, 0]],
        '碎' : [12, 13, 0, -2, 12, [256, 256, 65504, 9280, 9280, 10912, 28960, 53504, 24544, 20736, 20736, 28928, 0]],
        '硴' : [12, 13, 0, -2, 12, [1152, 1152, 65504, 9344, 8192, 8832, 29856, 56512, 21632, 21632, 21664, 29920, 0]],
        '碆' : [12, 13, 0, -2, 12, [256, 16640, 12256, 35104, 20416, 10816, 18816, 37440, 65504, 4160, 12352, 57280, 0]],
        '硼' : [12, 13, 0, -2, 12, [0, 3808, 64160, 10912, 12000, 10912, 31392, 51872, 20192, 19104, 19104, 31392, 0]],
        '碚' : [12, 13, 0, -2, 12, [256, 256, 65504, 8768, 8768, 12256, 28672, 53248, 22496, 21536, 21536, 30688, 0]],
        '碌' : [12, 13, 0, -2, 12, [0, 2016, 62528, 12224, 10368, 12256, 28800, 54944, 20672, 21152, 21664, 31104, 0]],
        '碣' : [12, 13, 0, -2, 12, [0, 4032, 63552, 12224, 10304, 12224, 29696, 57312, 21024, 21792, 24352, 28864, 0]],
        '碵' : [12, 13, 0, -2, 12, [256, 256, 63968, 8448, 10208, 9248, 30688, 54304, 21536, 22496, 21056, 31776, 0]],
        '碪' : [12, 13, 0, -2, 12, [1088, 1088, 65504, 9280, 10176, 9280, 29760, 57312, 21120, 23776, 20480, 32736, 0]],
        '碯' : [12, 13, 0, -2, 12, [0, 2336, 62016, 9344, 8768, 8480, 30688, 54432, 21792, 22176, 21536, 30688, 0]],
        '磑' : [12, 13, 0, -2, 12, [128, 1184, 63456, 8192, 10208, 8192, 30688, 54304, 22496, 21056, 21056, 30688, 0]],
        '磆' : [12, 13, 0, -2, 12, [0, 1984, 64576, 9664, 9536, 12256, 30752, 55232, 21568, 22464, 21568, 29888, 0]],
        '磋' : [12, 13, 0, -2, 12, [1088, 1152, 65504, 8448, 10208, 8448, 32736, 53760, 22496, 22656, 20608, 30688, 0]],
        '磔' : [12, 13, 0, -2, 12, [0, 1856, 62944, 11584, 11072, 8928, 31808, 53504, 24544, 21248, 21824, 31008, 0]],
        '碾' : [12, 13, 0, -2, 12, [0, 2016, 64544, 10208, 9536, 10208, 30016, 54592, 22496, 23200, 21056, 30496, 0]],
        '碼' : [12, 13, 0, -2, 12, [0, 4064, 63744, 12224, 10496, 12224, 30976, 57312, 20512, 21792, 21792, 28864, 0]],
        '磅' : [12, 13, 0, -2, 12, [256, 256, 65504, 8768, 8768, 12256, 30752, 55264, 20992, 21472, 21536, 29888, 0]],
        '磊' : [12, 13, 0, -2, 12, [0, 65504, 2048, 4096, 16320, 53312, 4160, 65504, 8448, 31712, 52512, 31200, 0]],
        '磬' : [12, 13, 0, -2, 12, [4096, 5056, 65088, 4704, 65472, 21568, 32128, 33344, 65504, 4160, 12352, 57280, 0]],
        '磧' : [12, 13, 0, -2, 12, [256, 256, 65504, 8448, 12256, 8448, 32736, 54336, 22464, 21568, 22464, 30816, 0]],
        '磚' : [12, 13, 0, -2, 12, [256, 256, 65504, 10528, 12256, 10528, 32736, 53536, 24544, 20544, 21568, 29376, 0]],
        '磽' : [12, 13, 0, -2, 12, [256, 256, 65504, 8448, 12256, 9280, 32480, 54336, 24544, 21120, 21664, 30944, 0]],
        '磴' : [12, 13, 0, -2, 12, [0, 1824, 61760, 10912, 9280, 12256, 28672, 55264, 21536, 22496, 21056, 32736, 0]],
        '礇' : [12, 13, 0, -2, 12, [256, 512, 65504, 11616, 11168, 11616, 31648, 56672, 20736, 24544, 21056, 31776, 0]],
        '礒' : [12, 13, 0, -2, 12, [1088, 1152, 65504, 8448, 10208, 8448, 32736, 54528, 24544, 21664, 24128, 30112, 0]],
        '礑' : [12, 13, 0, -2, 12, [256, 1344, 65504, 10912, 10912, 9088, 30720, 53216, 18720, 20448, 18720, 32736, 0]],
        '礙' : [12, 13, 0, -2, 12, [0, 2528, 65056, 10304, 10208, 10304, 32576, 54336, 24416, 21824, 23360, 29152, 0]],
        '礬' : [12, 13, 0, -2, 12, [8320, 10880, 62944, 27328, 42144, 10880, 65504, 4224, 16320, 53408, 12416, 24448, 0]],
        '礫' : [12, 13, 0, -2, 12, [256, 1344, 64416, 9920, 11168, 9920, 31648, 53504, 24544, 21248, 21824, 31008, 0]],
        '祀' : [12, 13, 0, -2, 12, [0, 64480, 544, 544, 64032, 9184, 8704, 43520, 43520, 43552, 43552, 9184, 0]],
        '祠' : [12, 13, 0, -2, 12, [0, 63456, 32, 32, 63392, 8224, 8224, 42784, 46368, 42272, 42784, 8288, 0]],
        '祗' : [12, 13, 0, -2, 12, [0, 61536, 1920, 1152, 64640, 10208, 9280, 42048, 46112, 42784, 40960, 12256, 0]],
        '祟' : [12, 13, 0, -2, 12, [1024, 17472, 32704, 1024, 33824, 65504, 0, 32704, 0, 65504, 17472, 35872, 0]],
        '祚' : [12, 13, 0, -2, 12, [1024, 64512, 2016, 1280, 63744, 8672, 8448, 43264, 43488, 43264, 43264, 8448, 0]],
        '祕' : [12, 13, 0, -2, 12, [0, 61952, 288, 32, 62016, 8768, 10880, 43680, 43808, 41472, 42528, 10720, 0]],
        '祓' : [12, 13, 0, -2, 12, [512, 61984, 4064, 512, 61984, 9504, 9536, 42304, 43136, 43136, 41792, 11296, 0]],
        '祺' : [12, 13, 0, -2, 12, [1088, 62528, 4064, 1088, 63424, 9280, 10176, 42048, 49120, 40960, 42048, 10272, 0]],
        '祿' : [12, 13, 0, -2, 12, [0, 63456, 1088, 4032, 63616, 12256, 8480, 44352, 41344, 42304, 43296, 13056, 0]],
        '禊' : [12, 13, 0, -2, 12, [1024, 62944, 7840, 1184, 65184, 9504, 12128, 42240, 49120, 41600, 42048, 14368, 0]],
        '禝' : [12, 13, 0, -2, 12, [0, 65504, 2336, 4064, 63776, 12256, 8832, 44128, 41920, 44096, 41856, 15456, 0]],
        '禧' : [12, 13, 0, -2, 12, [128, 61568, 4064, 128, 65504, 9248, 10208, 41536, 45024, 42016, 42016, 10208, 0]],
        '齋' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2560, 62944, 21824, 49056, 8256, 16320, 9280, 21824, 21696, 0]],
        '禪' : [12, 13, 0, -2, 12, [0, 61152, 2720, 2720, 65504, 10528, 12256, 43296, 45024, 41216, 49120, 8448, 0]],
        '禮' : [12, 13, 0, -2, 12, [640, 62080, 4064, 2720, 65504, 10912, 12256, 42048, 42944, 41600, 41600, 12256, 0]],
        '禳' : [12, 13, 0, -2, 12, [256, 61696, 4064, 2720, 65248, 8832, 12000, 41600, 45024, 41632, 48192, 10016, 0]],
        '禹' : [12, 13, 0, -2, 12, [0, 65504, 1024, 16256, 9344, 16256, 1024, 65504, 33824, 33952, 49056, 32864, 0]],
        '禺' : [12, 13, 0, -2, 12, [0, 32704, 17472, 32704, 17472, 32704, 1024, 65504, 33824, 33952, 49056, 32864, 0]],
        '秉' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 1088, 65504, 1088, 32704, 3328, 13440, 50272, 1024, 0]],
        '秕' : [12, 13, 0, -2, 12, [256, 6400, 59648, 10528, 12096, 63872, 10496, 26880, 26880, 43264, 44832, 14560, 0]],
        '秧' : [12, 13, 0, -2, 12, [256, 6400, 61408, 10528, 10528, 59680, 10528, 32736, 24832, 41600, 42048, 14368, 0]],
        '秬' : [12, 13, 0, -2, 12, [0, 8160, 58368, 9216, 10176, 64576, 9280, 26560, 29696, 44032, 41984, 10208, 0]],
        '秡' : [12, 13, 0, -2, 12, [512, 6688, 61408, 8704, 8736, 62752, 9536, 25920, 26752, 43136, 41792, 11296, 0]],
        '秣' : [12, 13, 0, -2, 12, [256, 6400, 61408, 8448, 8448, 63456, 8448, 24832, 25472, 42304, 47392, 8448, 0]],
        '稈' : [12, 13, 0, -2, 12, [0, 8160, 58400, 10208, 9248, 62496, 10208, 24576, 30688, 41088, 42976, 8320, 0]],
        '稍' : [12, 13, 0, -2, 12, [128, 6272, 58048, 9376, 8320, 63456, 9248, 26592, 25632, 47072, 42016, 9312, 0]],
        '稘' : [12, 13, 0, -2, 12, [1088, 7232, 59360, 9280, 10176, 62528, 10176, 25664, 32736, 40960, 42048, 10272, 0]],
        '稙' : [12, 13, 0, -2, 12, [128, 6272, 61408, 8320, 9184, 64032, 11232, 27168, 27168, 44000, 43008, 12256, 0]],
        '稠' : [12, 13, 0, -2, 12, [0, 8160, 59680, 11168, 10528, 64416, 10528, 26656, 27552, 43680, 43936, 10336, 0]],
        '稟' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 10880, 11904, 8320, 16256, 1024, 65504, 3328, 13440, 50272, 0]],
        '禀' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 10880, 11904, 8320, 16256, 0, 65504, 1024, 17472, 35872, 0]],
        '稱' : [12, 13, 0, -2, 12, [0, 8160, 58688, 9504, 10496, 65504, 10528, 28640, 26912, 49120, 43040, 10336, 0]],
        '稻' : [12, 13, 0, -2, 12, [0, 8160, 58688, 9504, 10240, 63744, 9824, 25632, 26464, 46112, 42016, 10208, 0]],
        '稾' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 8320, 65504, 37152, 49056, 1024, 65504, 3328, 13440, 50272, 0]],
        '稷' : [12, 13, 0, -2, 12, [0, 8160, 59680, 12256, 10528, 65504, 8832, 27744, 25536, 44096, 41856, 15456, 0]],
        '穃' : [12, 13, 0, -2, 12, [256, 6400, 61408, 10912, 9280, 63904, 8768, 25632, 28640, 46112, 42016, 10208, 0]],
        '穗' : [12, 13, 0, -2, 12, [256, 6400, 61408, 10528, 12256, 63776, 12256, 24864, 32736, 42240, 46144, 9120, 0]],
        '穉' : [12, 13, 0, -2, 12, [0, 8160, 59424, 12256, 10368, 65248, 10368, 27264, 27616, 46208, 42976, 8320, 0]],
        '穡' : [12, 13, 0, -2, 12, [256, 6400, 61408, 9536, 9536, 64416, 8480, 28640, 27296, 43936, 43040, 12256, 0]],
        '穢' : [12, 13, 0, -2, 12, [256, 7648, 58624, 12256, 8320, 65504, 10400, 28576, 27200, 43840, 45728, 11552, 0]],
        '穩' : [12, 13, 0, -2, 12, [0, 8160, 58688, 10528, 12256, 57632, 16352, 24608, 28640, 42240, 46144, 9120, 0]],
        '龝' : [12, 13, 0, -2, 12, [0, 7104, 58496, 12256, 10912, 61152, 14976, 28640, 31392, 45024, 47616, 9184, 0]],
        '穰' : [12, 13, 0, -2, 12, [256, 6400, 61408, 10912, 12000, 62080, 12000, 25216, 28640, 41632, 48192, 10016, 0]],
        '穹' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 25536, 128, 32640, 16384, 32704, 16448, 896, 0]],
        '穽' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 25536, 4352, 32704, 4352, 65504, 4352, 24832, 0]],
        '窈' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 2560, 29632, 8448, 20448, 63776, 8480, 18976, 64192, 0]],
        '窗' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37408, 25536, 2048, 65504, 35104, 45600, 36384, 37152, 65504, 0]],
        '窕' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 25536, 0, 19008, 10880, 19008, 37408, 25056, 0]],
        '窘' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 32704, 2176, 65504, 2176, 16320, 57408, 16320, 0]],
        '窖' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37408, 26560, 17408, 32704, 33792, 65504, 17472, 16448, 32704, 0]],
        '窩' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 32704, 8320, 10112, 9344, 65504, 37152, 40800, 0]],
        '竈' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 4608, 65504, 19008, 64480, 35360, 64480, 35328, 4064, 0]],
        '窰' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37408, 26560, 14464, 5376, 65504, 9216, 65504, 17472, 32704, 0]],
        '窶' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37408, 28608, 9344, 65504, 9344, 65504, 4224, 7936, 57568, 0]],
        '竅' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37408, 31680, 18688, 31712, 19008, 64832, 18752, 35200, 13920, 0]],
        '竄' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 29632, 16448, 32704, 17472, 30528, 17472, 30624, 52256, 0]],
        '窿' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 62400, 44096, 41344, 50784, 48256, 42976, 49280, 36832, 0]],
        '邃' : [12, 13, 0, -2, 12, [512, 512, 24544, 9504, 39360, 17024, 8160, 58656, 14784, 9888, 22784, 36832, 0]],
        '竇' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 29632, 1024, 65504, 19008, 32704, 8320, 16256, 49248, 0]],
        '竊' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 35360, 29632, 41216, 47040, 9280, 65504, 27168, 43936, 10336, 0]],
        '竍' : [12, 13, 0, -2, 12, [128, 8320, 8320, 64640, 2176, 19424, 18560, 18560, 20608, 4224, 64640, 128, 0]],
        '竏' : [12, 13, 0, -2, 12, [0, 9184, 8320, 64640, 2176, 18560, 19424, 18560, 20608, 4224, 64640, 128, 0]],
        '竕' : [12, 13, 0, -2, 12, [0, 17344, 16448, 62528, 9248, 43040, 45024, 41536, 41536, 9280, 62528, 384, 0]],
        '竓' : [12, 13, 0, -2, 12, [0, 16832, 19968, 61952, 9184, 44544, 41472, 41952, 44544, 8704, 61984, 992, 0]],
        '站' : [12, 13, 0, -2, 12, [256, 8448, 8448, 63968, 4352, 20736, 20736, 22496, 21536, 5152, 64544, 2016, 0]],
        '竚' : [12, 13, 0, -2, 12, [256, 16640, 20448, 63520, 10272, 40960, 45024, 41088, 41088, 8320, 61568, 896, 0]],
        '竝' : [12, 13, 0, -2, 12, [256, 16640, 16640, 65504, 8192, 42048, 42048, 42048, 42112, 9344, 61568, 4064, 0]],
        '竡' : [12, 13, 0, -2, 12, [0, 20448, 16512, 61696, 10208, 42016, 42016, 42976, 42016, 9248, 62496, 2016, 0]],
        '竢' : [12, 13, 0, -2, 12, [512, 16960, 17472, 65504, 9216, 42976, 42112, 43136, 45024, 8320, 62272, 3104, 0]],
        '竦' : [12, 13, 0, -2, 12, [256, 16640, 20448, 61696, 12256, 43296, 43296, 45024, 41728, 9536, 63776, 256, 0]],
        '竭' : [12, 13, 0, -2, 12, [0, 20416, 18496, 65472, 10304, 44992, 43008, 45024, 42016, 10528, 65312, 192, 0]],
        '竰' : [12, 13, 0, -2, 12, [0, 20448, 18432, 64480, 10912, 44000, 43680, 44000, 43136, 11232, 61568, 2016, 0]],
        '笂' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 8320, 2048, 32512, 2304, 10496, 4384, 10528, 50400, 0]],
        '笏' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 8320, 4096, 16352, 18720, 35104, 4384, 8736, 17600, 0]],
        '笊' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 8320, 0, 32736, 17472, 17472, 17472, 33824, 33824, 0]],
        '笆' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 32704, 17472, 17472, 32704, 16384, 16416, 16352, 0]],
        '笳' : [12, 13, 0, -2, 12, [512, 16896, 31712, 38016, 8192, 8672, 64800, 9504, 9504, 17696, 34080, 14816, 0]],
        '笘' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 9216, 1920, 1024, 1024, 32704, 16448, 16448, 32704, 0]],
        '笙' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 9216, 16320, 17408, 33792, 16256, 1024, 1024, 65504, 0]],
        '笞' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 2048, 4224, 65472, 32, 16256, 8320, 8320, 16256, 0]],
        '笵' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 10176, 37952, 17472, 1216, 9216, 17440, 34784, 0]],
        '笨' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 1024, 65504, 3328, 5248, 9280, 57248, 1024, 1024, 0]],
        '笶' : [12, 13, 0, -2, 12, [512, 16896, 31712, 38016, 8192, 16320, 17408, 33792, 65504, 2560, 12672, 49248, 0]],
        '筐' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 65504, 33280, 40896, 33280, 49120, 32768, 65504, 0]],
        '筺' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 65504, 33280, 40896, 33312, 49120, 32768, 65504, 0]],
        '笄' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 32704, 4352, 4352, 65504, 4352, 8448, 49408, 0]],
        '筍' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 8256, 32736, 16416, 48672, 41504, 15904, 8736, 16064, 0]],
        '笋' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 32704, 1088, 65504, 1088, 32704, 1024, 6144, 0]],
        '筌' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 32512, 4224, 16320, 50208, 7936, 1024, 65504, 0]],
        '筅' : [12, 13, 0, -2, 12, [512, 16896, 31712, 38016, 9216, 16320, 17408, 33792, 65504, 4608, 8736, 50144, 0]],
        '筵' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 4192, 63360, 8320, 17632, 62592, 8160, 12288, 53216, 0]],
        '筥' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 16256, 8320, 16256, 2048, 32704, 16448, 32704, 0]],
        '筴' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 1024, 65504, 9344, 9344, 21824, 35360, 12672, 49248, 0]],
        '筧' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 16256, 8320, 16256, 8320, 16256, 4640, 58336, 0]],
        '筰' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 128, 9216, 10208, 18944, 52192, 16896, 17376, 16896, 0]],
        '筱' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 8704, 17376, 53824, 21568, 21824, 20800, 20864, 18016, 0]],
        '筬' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 288, 32736, 16640, 31008, 18752, 18560, 39232, 34336, 0]],
        '筮' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 32704, 9344, 9344, 21824, 36384, 1024, 65504, 0]],
        '箝' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 9248, 65504, 9248, 14304, 58400, 9248, 26592, 0]],
        '箘' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 65504, 33824, 49056, 35872, 38176, 42144, 65504, 0]],
        '箟' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 32704, 16448, 32704, 16896, 31680, 16928, 64480, 0]],
        '箍' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 20448, 63616, 19424, 19104, 60064, 18560, 53216, 0]],
        '箜' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 1024, 65504, 37408, 41888, 17408, 16256, 1024, 65504, 0]],
        '箚' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 128, 31776, 33056, 64800, 288, 64800, 33824, 64608, 0]],
        '箋' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 4064, 30848, 3840, 61600, 4064, 30848, 1824, 63712, 0]],
        '箒' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 16320, 64, 65504, 64, 65504, 42144, 42144, 9600, 0]],
        '箏' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 16352, 17472, 49056, 1152, 65504, 1152, 16256, 1024, 0]],
        '筝' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 8064, 8448, 32704, 1088, 65504, 1088, 32704, 1024, 0]],
        '箙' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 31712, 18976, 31232, 19424, 31264, 19104, 35008, 39712, 0]],
        '篋' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 0, 65504, 33792, 65504, 42112, 54592, 35360, 65504, 0]],
        '篁' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 2048, 16256, 8320, 16256, 8320, 32704, 1024, 65504, 0]],
        '篌' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 9152, 16448, 55264, 21632, 22496, 22656, 20800, 17952, 0]],
        '篏' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 640, 37376, 64480, 38048, 62624, 36992, 37184, 63008, 0]],
        '箴' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 288, 32736, 16640, 32032, 16704, 31872, 42304, 48672, 0]],
        '篆' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 8064, 8448, 65504, 5248, 25856, 6784, 57952, 3072, 0]],
        '篝' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 4352, 32704, 4352, 65504, 17472, 65504, 16448, 16576, 0]],
        '篩' : [12, 13, 0, -2, 12, [512, 16896, 31712, 38016, 8192, 64480, 34944, 64480, 33440, 64160, 35488, 63616, 0]],
        '簑' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 32704, 8320, 65504, 8320, 16288, 4672, 57728, 15456, 0]],
        '簔' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 32704, 4352, 65504, 4352, 32736, 4672, 57728, 15456, 0]],
        '篦' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 32704, 19008, 20800, 32704, 16896, 31680, 16928, 63968, 0]],
        '篥' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 32704, 19008, 32704, 1024, 65504, 3328, 13440, 50272, 0]],
        '籠' : [12, 13, 0, -2, 12, [512, 16896, 31712, 38016, 9184, 64000, 37856, 64544, 35808, 64000, 35360, 39392, 0]],
        '簀' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 32704, 1024, 65504, 8320, 16256, 8320, 16256, 49248, 0]],
        '簇' : [12, 13, 0, -2, 12, [512, 16896, 31712, 38016, 18400, 64000, 17376, 29824, 38880, 36992, 4416, 26144, 0]],
        '簓' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 32, 65216, 37376, 47648, 37568, 47616, 43552, 47808, 0]],
        '篳' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 32704, 17472, 32704, 9344, 65504, 9344, 65504, 1024, 0]],
        '篷' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 9152, 37952, 16768, 3680, 57472, 10208, 12416, 53216, 0]],
        '簗' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 12224, 37440, 19040, 33920, 65504, 3328, 13440, 50272, 0]],
        '簍' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 65504, 9344, 32704, 17472, 65504, 4224, 7936, 57568, 0]],
        '篶' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 32704, 1024, 10176, 9216, 65504, 32, 21792, 38336, 0]],
        '簣' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 16320, 9344, 65504, 8320, 16256, 8320, 16256, 49248, 0]],
        '簧' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 32704, 4352, 65504, 9344, 16256, 9344, 16256, 49248, 0]],
        '簪' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 64480, 37440, 64480, 20800, 37472, 32704, 16448, 32704, 0]],
        '簟' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 65504, 19008, 32704, 8320, 16256, 8320, 65504, 1024, 0]],
        '簷' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 16320, 16512, 65504, 18560, 29280, 24512, 36928, 40896, 0]],
        '簫' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 8064, 57568, 8064, 17440, 32736, 17440, 62944, 38176, 0]],
        '簽' : [12, 13, 0, -2, 12, [512, 16896, 31712, 44608, 4352, 16256, 49248, 31680, 19008, 31680, 8832, 52320, 0]],
        '籌' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 65504, 32, 32640, 1088, 65504, 20544, 21568, 29376, 0]],
        '籃' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 512, 64480, 41984, 64480, 34816, 65472, 19008, 65504, 0]],
        '籔' : [12, 13, 0, -2, 12, [512, 16896, 31712, 38144, 32224, 21824, 65088, 21824, 65344, 17536, 14656, 50720, 0]],
        '籏' : [12, 13, 0, -2, 12, [512, 16896, 31712, 38016, 18400, 64064, 17344, 31296, 20448, 34816, 35392, 13344, 0]],
        '籀' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 7648, 20640, 62752, 24544, 18720, 61408, 18720, 53216, 0]],
        '籐' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 2368, 65504, 37504, 65504, 38208, 63904, 38208, 47904, 0]],
        '籘' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 2368, 65504, 37504, 65504, 39584, 62784, 40928, 46400, 0]],
        '籟' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 64480, 8480, 64480, 43552, 64480, 8736, 27072, 42528, 0]],
        '籤' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 10496, 21792, 65504, 10368, 61088, 10304, 10400, 65312, 0]],
        '籖' : [12, 13, 0, -2, 12, [512, 16896, 31712, 38016, 32000, 4384, 65504, 10368, 61088, 10304, 10400, 65312, 0]],
        '籥' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 7936, 8320, 32704, 43680, 32704, 19008, 32704, 19008, 0]],
        '籬' : [12, 13, 0, -2, 12, [512, 16896, 31712, 42112, 65088, 44000, 64064, 10208, 64064, 44000, 47680, 35808, 0]],
        '籵' : [12, 13, 0, -2, 12, [128, 8320, 43136, 43136, 8320, 63456, 8320, 24704, 28800, 43136, 41088, 8320, 0]],
        '粃' : [12, 13, 0, -2, 12, [128, 9344, 46208, 42144, 10176, 64640, 9344, 25728, 29824, 42144, 42912, 11488, 0]],
        '粐' : [12, 13, 0, -2, 12, [0, 9184, 43008, 43008, 9184, 64032, 8736, 25568, 29184, 41472, 41984, 9216, 0]],
        '粤' : [12, 13, 0, -2, 12, [1024, 2048, 32704, 21824, 32704, 19520, 21824, 65504, 8192, 32704, 16448, 896, 0]],
        '粭' : [12, 13, 0, -2, 12, [0, 9152, 46112, 40960, 8192, 63456, 8192, 24576, 30688, 42016, 42016, 10208, 0]],
        '粢' : [12, 13, 0, -2, 12, [1024, 17408, 10208, 2336, 10496, 17024, 42080, 5248, 65504, 3328, 13440, 50272, 0]],
        '粫' : [12, 13, 0, -2, 12, [0, 12256, 45312, 41472, 12256, 64160, 10912, 27296, 27296, 43680, 43680, 10336, 0]],
        '粡' : [12, 13, 0, -2, 12, [0, 12256, 47136, 43936, 10272, 63520, 11168, 27296, 27296, 43936, 43040, 10336, 0]],
        '粨' : [12, 13, 0, -2, 12, [0, 12256, 45312, 41472, 10208, 64544, 9248, 26592, 29728, 42016, 42016, 10208, 0]],
        '粳' : [12, 13, 0, -2, 12, [0, 12256, 45312, 45024, 10528, 65504, 10528, 28640, 25856, 41728, 42112, 14432, 0]],
        '粲' : [12, 13, 0, -2, 12, [8192, 15328, 8224, 14912, 51584, 13920, 58496, 5376, 65504, 3328, 13440, 50272, 0]],
        '粱' : [12, 13, 0, -2, 12, [0, 10176, 37184, 17760, 10560, 16960, 42176, 5376, 65504, 3328, 13440, 50272, 0]],
        '粮' : [12, 13, 0, -2, 12, [256, 8448, 47072, 42016, 10208, 64544, 10208, 25856, 29984, 42144, 42048, 10016, 0]],
        '粹' : [12, 13, 0, -2, 12, [256, 8448, 45024, 46144, 9280, 60064, 12576, 24832, 28640, 41216, 41216, 8448, 0]],
        '粽' : [12, 13, 0, -2, 12, [256, 8448, 45024, 43040, 10176, 61440, 8192, 28640, 28928, 42304, 43296, 8960, 0]],
        '糀' : [12, 13, 0, -2, 12, [576, 8768, 45024, 41536, 8192, 62080, 9344, 27872, 29824, 42112, 42144, 9440, 0]],
        '糅' : [12, 13, 0, -2, 12, [0, 10176, 45184, 41216, 12256, 62240, 11584, 24832, 28640, 41728, 42304, 14624, 0]],
        '糂' : [12, 13, 0, -2, 12, [1088, 9280, 45024, 42048, 10176, 62528, 9280, 28640, 27264, 44256, 43008, 12256, 0]],
        '糘' : [12, 13, 0, -2, 12, [256, 8448, 45024, 43040, 12224, 63776, 9024, 27776, 25024, 42656, 47264, 8960, 0]],
        '糒' : [12, 13, 0, -2, 12, [576, 8768, 45024, 45632, 10208, 62464, 10208, 25760, 28640, 46240, 42976, 9376, 0]],
        '糜' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 32736, 19104, 27296, 18560, 21056, 49120, 35456, 12896, 0]],
        '糢' : [12, 13, 0, -2, 12, [576, 8768, 45024, 45632, 10208, 62496, 10208, 25632, 28640, 41216, 41536, 11296, 0]],
        '鬻' : [12, 13, 0, -2, 12, [1024, 54624, 19488, 40768, 50272, 21792, 65504, 8320, 65504, 37152, 61152, 33824, 0]],
        '糯' : [12, 13, 0, -2, 12, [0, 12256, 41216, 45024, 10528, 64352, 8192, 28640, 24832, 45024, 43680, 10912, 0]],
        '糲' : [12, 13, 0, -2, 12, [0, 12256, 43328, 45024, 10912, 64480, 10912, 27616, 26752, 47072, 42144, 9696, 0]],
        '糴' : [12, 13, 0, -2, 12, [0, 61408, 10912, 14176, 51872, 13088, 44160, 10208, 64640, 14304, 27776, 42976, 0]],
        '糶' : [12, 13, 0, -2, 12, [0, 45024, 64160, 10080, 43680, 64288, 9344, 45024, 9344, 65504, 25728, 45024, 0]],
        '糺' : [12, 13, 0, -2, 12, [0, 8704, 16896, 35328, 61952, 8704, 18944, 64000, 8736, 43552, 43552, 9184, 0]],
        '紆' : [12, 13, 0, -2, 12, [0, 10208, 16512, 32896, 61568, 12256, 16512, 63616, 8320, 43136, 43136, 8576, 0]],
        '紂' : [12, 13, 0, -2, 12, [64, 8256, 16448, 34784, 61504, 8256, 19008, 63808, 8256, 43072, 43072, 8384, 0]],
        '紜' : [12, 13, 0, -2, 12, [0, 10176, 16384, 32768, 61440, 12256, 16896, 61952, 9280, 42048, 43168, 12064, 0]],
        '紕' : [12, 13, 0, -2, 12, [256, 10496, 18688, 35104, 65344, 10624, 18688, 63744, 10496, 43296, 44832, 14560, 0]],
        '紊' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 3584, 13696, 51552, 4608, 65504, 1024, 9344, 50272, 0]],
        '絅' : [12, 13, 0, -2, 12, [0, 12256, 18464, 34848, 64416, 10912, 19104, 64160, 11168, 43040, 43040, 10336, 0]],
        '絋' : [12, 13, 0, -2, 12, [256, 8448, 18400, 33792, 62592, 9344, 17536, 62720, 9472, 43296, 43552, 11232, 0]],
        '紮' : [12, 13, 0, -2, 12, [256, 8448, 64768, 12576, 26912, 42208, 4608, 9248, 65504, 1024, 9344, 50272, 0]],
        '紲' : [12, 13, 0, -2, 12, [0, 9536, 17728, 34112, 61408, 9536, 17728, 62784, 9664, 46080, 41984, 10208, 0]],
        '紿' : [12, 13, 0, -2, 12, [512, 8704, 17440, 33824, 63456, 8192, 16384, 63456, 9248, 46112, 42016, 10208, 0]],
        '紵' : [12, 13, 0, -2, 12, [256, 8448, 18400, 33824, 62496, 8192, 18400, 63616, 8320, 43136, 43136, 9088, 0]],
        '絆' : [12, 13, 0, -2, 12, [256, 8448, 17728, 35104, 61696, 12256, 16640, 61696, 12256, 43264, 43264, 8448, 0]],
        '絳' : [12, 13, 0, -2, 12, [0, 9184, 19488, 33344, 62336, 11360, 16512, 63456, 9344, 42112, 45024, 8320, 0]],
        '絖' : [12, 13, 0, -2, 12, [256, 8448, 18720, 34112, 61696, 8448, 20448, 62080, 8832, 41600, 42144, 14560, 0]],
        '絎' : [12, 13, 0, -2, 12, [0, 9696, 18432, 36864, 57856, 9696, 19520, 62528, 9280, 46144, 42048, 9408, 0]],
        '絲' : [12, 13, 0, -2, 12, [0, 8448, 16896, 33824, 63456, 8448, 16928, 65504, 8448, 42304, 43296, 8448, 0]],
        '絨' : [12, 13, 0, -2, 12, [128, 8352, 20448, 32896, 62592, 9376, 20384, 62624, 9280, 43072, 43168, 8992, 0]],
        '絮' : [12, 13, 0, -2, 12, [8192, 8672, 64800, 18720, 18720, 14816, 52480, 4672, 65504, 1024, 9344, 50272, 0]],
        '絏' : [12, 13, 0, -2, 12, [256, 8448, 20448, 35104, 61408, 10528, 20448, 61728, 8512, 41344, 42656, 14432, 0]],
        '絣' : [12, 13, 0, -2, 12, [2112, 9280, 16512, 36832, 62016, 8768, 16960, 65504, 8768, 45632, 42048, 9280, 0]],
        '經' : [12, 13, 0, -2, 12, [0, 10208, 16384, 33440, 62784, 8864, 19104, 63488, 10208, 43136, 43136, 10208, 0]],
        '綉' : [12, 13, 0, -2, 12, [0, 10208, 16640, 36832, 62208, 9536, 18720, 63360, 8928, 45600, 42016, 9408, 0]],
        '絛' : [12, 13, 0, -2, 12, [512, 8704, 9184, 21568, 23104, 54144, 23648, 21120, 21792, 24544, 17728, 22816, 0]],
        '綏' : [12, 13, 0, -2, 12, [0, 12256, 18432, 37440, 61728, 8704, 16896, 65504, 9280, 44160, 41856, 15456, 0]],
        '絽' : [12, 13, 0, -2, 12, [0, 10208, 17440, 33824, 63456, 8448, 16896, 63456, 9248, 46112, 42016, 10208, 0]],
        '綛' : [12, 13, 0, -2, 12, [0, 12256, 16672, 34080, 61984, 9504, 18624, 61440, 8704, 43328, 43040, 10112, 0]],
        '綺' : [12, 13, 0, -2, 12, [256, 8448, 20448, 33408, 62528, 10272, 20448, 61472, 10016, 46368, 42784, 8288, 0]],
        '綮' : [12, 13, 0, -2, 12, [512, 64000, 992, 32320, 19008, 31104, 34400, 37120, 8768, 65504, 9344, 50272, 0]],
        '綣' : [12, 13, 0, -2, 12, [256, 9536, 18720, 36832, 62016, 12256, 17472, 61408, 13376, 42432, 42016, 10208, 0]],
        '綵' : [12, 13, 0, -2, 12, [0, 12256, 18432, 37440, 61728, 8192, 16640, 65504, 8960, 42304, 47392, 8448, 0]],
        '緇' : [12, 13, 0, -2, 12, [0, 8864, 17056, 34112, 62784, 8864, 17056, 63456, 9376, 47072, 42144, 10208, 0]],
        '綽' : [12, 13, 0, -2, 12, [256, 8448, 16864, 33024, 63456, 9248, 18400, 62496, 10208, 45312, 45024, 8448, 0]],
        '綫' : [12, 13, 0, -2, 12, [512, 8736, 20448, 33344, 61856, 11872, 17536, 65504, 9280, 41600, 41760, 15584, 0]],
        '總' : [12, 13, 0, -2, 12, [256, 8704, 20448, 35104, 61088, 10528, 19104, 65504, 8704, 43328, 43040, 10112, 0]],
        '綢' : [12, 13, 0, -2, 12, [0, 12256, 18720, 35744, 59680, 11168, 18720, 63520, 11168, 43680, 43936, 10336, 0]],
        '綯' : [12, 13, 0, -2, 12, [1024, 9216, 18400, 34848, 61344, 8736, 20384, 61984, 10912, 44960, 40992, 8384, 0]],
        '緜' : [12, 13, 0, -2, 12, [8192, 18400, 63744, 35360, 63808, 34944, 63776, 10208, 64768, 42304, 44320, 8448, 0]],
        '綸' : [12, 13, 0, -2, 12, [0, 9088, 17472, 34848, 61408, 8192, 20448, 64160, 10912, 45024, 43680, 10912, 0]],
        '綟' : [12, 13, 0, -2, 12, [0, 12256, 16384, 34784, 62496, 10208, 17536, 63456, 10368, 43136, 41280, 9760, 0]],
        '綰' : [12, 13, 0, -2, 12, [256, 8448, 20448, 34848, 63424, 9280, 18368, 62464, 10208, 46112, 42016, 10208, 0]],
        '緘' : [12, 13, 0, -2, 12, [128, 8352, 20448, 34944, 65408, 10400, 20128, 60064, 11840, 43072, 45472, 9760, 0]],
        '緝' : [12, 13, 0, -2, 12, [0, 10208, 17440, 34784, 61440, 12256, 17440, 63456, 9248, 42976, 44064, 8224, 0]],
        '緤' : [12, 13, 0, -2, 12, [512, 10816, 24544, 35392, 60352, 10240, 20448, 61696, 12256, 41728, 42304, 14624, 0]],
        '緞' : [12, 13, 0, -2, 12, [0, 12224, 18752, 36672, 59712, 10592, 19968, 63936, 12096, 43328, 43392, 11872, 0]],
        '緻' : [12, 13, 0, -2, 12, [128, 16256, 17632, 33952, 63776, 12064, 17568, 62624, 11840, 42048, 42144, 16160, 0]],
        '緲' : [12, 13, 0, -2, 12, [128, 11904, 19072, 35520, 61088, 10880, 19072, 65408, 10784, 43552, 44608, 8576, 0]],
        '緡' : [12, 13, 0, -2, 12, [0, 12256, 18464, 36832, 59520, 12256, 18496, 65504, 9248, 47072, 42016, 10208, 0]],
        '縅' : [12, 13, 0, -2, 12, [256, 8480, 20448, 34944, 61312, 10912, 20384, 60064, 10816, 47936, 42144, 11040, 0]],
        '縊' : [12, 13, 0, -2, 12, [0, 10176, 16448, 37952, 63392, 10272, 19392, 61504, 9248, 45024, 43680, 16352, 0]],
        '縣' : [12, 13, 0, -2, 12, [0, 16352, 42240, 48672, 42304, 48256, 42272, 65504, 4224, 21152, 38048, 4224, 0]],
        '縡' : [12, 13, 0, -2, 12, [256, 8448, 24544, 37152, 61408, 9280, 17472, 65504, 8448, 45024, 41216, 8448, 0]],
        '縒' : [12, 13, 0, -2, 12, [1088, 9344, 20448, 33024, 63456, 8448, 20448, 61952, 10208, 43136, 41088, 12256, 0]],
        '縱' : [12, 13, 0, -2, 12, [1088, 9280, 17472, 35488, 61728, 9344, 18560, 64224, 10880, 43648, 43904, 11360, 0]],
        '縟' : [12, 13, 0, -2, 12, [0, 12256, 18432, 36832, 60064, 10816, 20256, 59456, 12256, 45120, 42048, 8896, 0]],
        '縉' : [12, 13, 0, -2, 12, [0, 12256, 16928, 33856, 60064, 12256, 16384, 63456, 9248, 47072, 42016, 10208, 0]],
        '縋' : [12, 13, 0, -2, 12, [64, 10368, 17888, 33056, 59872, 9472, 16864, 64800, 9504, 42464, 44032, 13280, 0]],
        '縢' : [12, 13, 0, -2, 12, [256, 58688, 43296, 45024, 57920, 45024, 42304, 60064, 45376, 45024, 42304, 47392, 0]],
        '繆' : [12, 13, 0, -2, 12, [0, 16352, 19104, 37728, 59040, 10784, 17280, 62528, 10656, 42496, 41152, 9984, 0]],
        '繦' : [12, 13, 0, -2, 12, [0, 15840, 17696, 34272, 60544, 11232, 19104, 65184, 10208, 42112, 42144, 15328, 0]],
        '縻' : [12, 13, 0, -2, 12, [512, 512, 32736, 18560, 32736, 19104, 27296, 18688, 21056, 32736, 35392, 45600, 0]],
        '縵' : [12, 13, 0, -2, 12, [0, 10176, 17472, 34752, 62528, 12256, 19104, 65504, 9280, 45632, 41856, 11360, 0]],
        '縹' : [12, 13, 0, -2, 12, [0, 12256, 17024, 36832, 60064, 12256, 16384, 61408, 8192, 49120, 42304, 11040, 0]],
        '繃' : [12, 13, 0, -2, 12, [256, 10528, 20448, 32768, 61152, 10912, 20192, 64160, 10912, 44768, 43680, 10912, 0]],
        '縷' : [12, 13, 0, -2, 12, [256, 8448, 20448, 35104, 61408, 10528, 20448, 63776, 12256, 42048, 42880, 14432, 0]],
        '縲' : [12, 13, 0, -2, 12, [0, 12256, 18720, 36832, 59680, 12256, 16960, 62592, 12256, 41216, 42304, 10528, 0]],
        '縺' : [12, 13, 0, -2, 12, [128, 10368, 18400, 36992, 60384, 8864, 23520, 60064, 11232, 43136, 44032, 13280, 0]],
        '繧' : [12, 13, 0, -2, 12, [0, 12256, 16640, 36832, 59680, 11104, 16384, 63456, 8192, 45024, 42048, 12192, 0]],
        '繝' : [12, 13, 0, -2, 12, [0, 12000, 19104, 36576, 60064, 12000, 19360, 64160, 11168, 43680, 43936, 11936, 0]],
        '繖' : [12, 13, 0, -2, 12, [2688, 10880, 24288, 35488, 65440, 8224, 20128, 64160, 11840, 43584, 44704, 11040, 0]],
        '繞' : [12, 13, 0, -2, 12, [256, 8448, 20448, 33024, 61408, 9280, 20192, 62528, 12256, 41600, 42144, 14560, 0]],
        '繙' : [12, 13, 0, -2, 12, [0, 12256, 18752, 34176, 61408, 9536, 18720, 65504, 10528, 45024, 43296, 12256, 0]],
        '繚' : [12, 13, 0, -2, 12, [256, 8448, 24544, 35392, 59360, 9248, 20448, 62496, 10208, 41216, 43328, 13088, 0]],
        '繹' : [12, 13, 0, -2, 12, [0, 12256, 19104, 35488, 61408, 8448, 20448, 58432, 16352, 41216, 45024, 8448, 0]],
        '繪' : [12, 13, 0, -2, 12, [0, 10176, 18464, 38880, 57344, 12256, 19104, 65504, 9248, 47072, 42016, 10208, 0]],
        '繩' : [12, 13, 0, -2, 12, [0, 12256, 19104, 36576, 57984, 12000, 19104, 65248, 10912, 44768, 41600, 9184, 0]],
        '繼' : [12, 13, 0, -2, 12, [0, 10816, 19616, 35392, 59680, 12256, 18432, 64064, 11424, 43584, 43296, 12256, 0]],
        '繻' : [12, 13, 0, -2, 12, [0, 12256, 16640, 36832, 59680, 11104, 16384, 65504, 8704, 45024, 43680, 10912, 0]],
        '纃' : [12, 13, 0, -2, 12, [512, 8704, 24544, 38208, 63296, 11680, 17472, 63424, 9280, 42944, 43072, 10304, 0]],
        '緕' : [12, 13, 0, -2, 12, [256, 8448, 24544, 33856, 62336, 9280, 19552, 63424, 9280, 42944, 43072, 10304, 0]],
        '繽' : [12, 13, 0, -2, 12, [512, 8704, 24544, 36896, 61408, 12864, 20416, 63552, 12224, 43072, 44992, 12384, 0]],
        '辮' : [12, 13, 0, -2, 12, [8320, 9344, 63968, 20800, 24384, 62944, 10368, 65504, 9344, 62944, 21632, 17536, 0]],
        '繿' : [12, 13, 0, -2, 12, [128, 12160, 19072, 36832, 59648, 12032, 19168, 61952, 12256, 43680, 43680, 16352, 0]],
        '纈' : [12, 13, 0, -2, 12, [1024, 9696, 24192, 34272, 58656, 16352, 16672, 61920, 12064, 43488, 43328, 12064, 0]],
        '纉' : [12, 13, 0, -2, 12, [1088, 9280, 20192, 33856, 61152, 9280, 20448, 62528, 10176, 42048, 42944, 14432, 0]],
        '續' : [12, 13, 0, -2, 12, [256, 8448, 24544, 33024, 61408, 10912, 20448, 62528, 10176, 42048, 42944, 14432, 0]],
        '纒' : [12, 13, 0, -2, 12, [0, 12256, 19104, 35808, 60064, 11232, 19776, 60064, 10496, 47072, 41216, 12256, 0]],
        '纐' : [12, 13, 0, -2, 12, [1024, 9696, 24192, 33248, 60192, 12768, 16672, 61920, 11040, 42464, 43584, 12576, 0]],
        '纓' : [12, 13, 0, -2, 12, [0, 12000, 19104, 36576, 60064, 12000, 17536, 63776, 12256, 41536, 41856, 11360, 0]],
        '纔' : [12, 13, 0, -2, 12, [0, 10176, 18560, 40928, 59520, 12000, 17472, 65504, 10528, 45024, 41600, 11488, 0]],
        '纖' : [12, 13, 0, -2, 12, [2688, 10912, 21920, 34944, 65504, 10880, 19360, 64160, 11072, 43584, 43680, 16160, 0]],
        '纎' : [12, 13, 0, -2, 12, [1152, 9376, 24480, 33920, 65504, 10880, 19360, 64160, 11072, 43584, 43680, 16160, 0]],
        '纛' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 9280, 65504, 17472, 32704, 10560, 43680, 65504, 21120, 38048, 0]],
        '纜' : [12, 13, 0, -2, 12, [256, 16128, 21984, 40704, 62944, 16032, 18912, 65344, 10304, 44992, 42144, 14560, 0]],
        '缸' : [12, 13, 0, -2, 12, [0, 34784, 63616, 41088, 8320, 63616, 8320, 8320, 43136, 43136, 43136, 65504, 0]],
        '缺' : [12, 13, 0, -2, 12, [256, 33024, 65504, 41248, 8480, 63776, 10208, 8448, 41216, 41600, 42048, 63520, 0]],
        '罅' : [12, 13, 0, -2, 12, [448, 33024, 65504, 42272, 10176, 64768, 10208, 9376, 43648, 44000, 43136, 64384, 0]],
        '罌' : [12, 13, 0, -2, 12, [0, 31680, 19008, 31680, 19008, 31680, 20800, 49056, 9216, 65504, 17472, 32704, 0]],
        '罍' : [12, 13, 0, -2, 12, [0, 7936, 5376, 65504, 43680, 64480, 43680, 65504, 9216, 65504, 17472, 32704, 0]],
        '罎' : [12, 13, 0, -2, 12, [0, 34752, 62528, 45024, 10528, 64352, 8192, 12256, 40960, 45024, 42048, 65440, 0]],
        '罐' : [12, 13, 0, -2, 12, [1152, 33920, 65504, 43680, 12000, 62080, 10208, 11392, 42976, 42112, 42112, 63456, 0]],
        '网' : [12, 13, 0, -2, 12, [0, 65504, 37152, 37152, 38176, 54048, 45472, 39264, 38176, 41504, 41504, 32864, 0]],
        '罕' : [12, 13, 0, -2, 12, [0, 65504, 35360, 4608, 25536, 0, 16256, 1024, 1024, 65504, 1024, 1024, 0]],
        '罔' : [12, 13, 0, -2, 12, [0, 65504, 32800, 37152, 37408, 65504, 33824, 40736, 36896, 36896, 40736, 32864, 0]],
        '罘' : [12, 13, 0, -2, 12, [0, 32704, 19008, 19008, 32704, 0, 65504, 512, 1280, 7296, 58464, 1024, 0]],
        '罟' : [12, 13, 0, -2, 12, [0, 32704, 19008, 19008, 32704, 1024, 65504, 1024, 16256, 8320, 8320, 16256, 0]],
        '罠' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 0, 32704, 16448, 32704, 16896, 32736, 16640, 63712, 0]],
        '罨' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 2048, 65504, 9344, 32704, 42144, 16256, 9248, 2016, 0]],
        '罩' : [12, 13, 0, -2, 12, [0, 65504, 37152, 37152, 65504, 1024, 1984, 1024, 32704, 16448, 65504, 1024, 0]],
        '罧' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 8320, 8320, 64480, 8320, 12672, 27328, 42144, 8320, 0]],
        '罸' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 64, 61504, 2016, 61504, 1088, 62016, 36928, 61632, 0]],
        '羂' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 8768, 22496, 42016, 22496, 64544, 10208, 46112, 42080, 0]],
        '羆' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 16896, 35808, 64000, 35808, 64000, 35296, 21056, 35104, 0]],
        '羃' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 8320, 16256, 8320, 65504, 9344, 32704, 42144, 9600, 0]],
        '羈' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 21120, 64480, 43648, 43648, 64480, 8224, 64160, 10976, 0]],
        '羇' : [12, 13, 0, -2, 12, [0, 65504, 35104, 65504, 20608, 65504, 43328, 43552, 65504, 8864, 64416, 8288, 0]],
        '羌' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 16256, 1024, 1024, 65504, 4608, 4608, 8736, 49632, 0]],
        '羔' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 16256, 1024, 1024, 65504, 0, 0, 21056, 35104, 0]],
        '羞' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 16256, 1024, 65504, 5248, 16352, 51328, 2176, 32736, 0]],
        '羝' : [12, 13, 0, -2, 12, [4096, 36960, 42880, 64640, 9344, 63456, 9280, 9280, 64544, 10016, 16384, 20448, 0]],
        '羚' : [12, 13, 0, -2, 12, [4096, 37312, 41504, 64512, 10208, 61440, 8192, 10208, 63776, 8480, 16736, 16640, 0]],
        '羣' : [12, 13, 0, -2, 12, [0, 32704, 2112, 2112, 65504, 4128, 16352, 53504, 16320, 1024, 65504, 1024, 0]],
        '羯' : [12, 13, 0, -2, 12, [4096, 38848, 42048, 65472, 9280, 63424, 9216, 12256, 64032, 10528, 20256, 16576, 0]],
        '羲' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 32704, 4384, 65504, 21632, 37536, 64576, 9376, 23328, 0]],
        '羹' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 65504, 21056, 35104, 32704, 1024, 65504, 4352, 57568, 0]],
        '羮' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 1024, 65504, 18752, 49056, 1024, 65504, 2560, 12672, 49248, 0]],
        '羶' : [12, 13, 0, -2, 12, [4352, 37120, 45024, 64160, 19360, 59424, 20448, 17472, 63424, 17472, 33856, 40928, 0]],
        '羸' : [12, 13, 0, -2, 12, [1024, 65504, 32768, 32704, 8320, 65504, 43680, 60064, 49056, 58528, 48928, 42272, 0]],
        '譱' : [12, 13, 0, -2, 12, [4224, 65504, 1024, 32704, 1024, 65504, 17472, 62944, 1024, 62944, 38176, 62944, 0]],
        '翅' : [12, 13, 0, -2, 12, [0, 20192, 16928, 63072, 19104, 57888, 9824, 43680, 41504, 18016, 45056, 36832, 0]],
        '翆' : [12, 13, 0, -2, 12, [0, 64480, 18720, 10400, 18720, 35360, 32640, 4256, 25824, 1024, 65504, 1024, 0]],
        '翊' : [12, 13, 0, -2, 12, [0, 20192, 16928, 61984, 9824, 43680, 41504, 42592, 43680, 8992, 61984, 1632, 0]],
        '翕' : [12, 13, 0, -2, 12, [0, 3584, 12672, 57184, 0, 16256, 8320, 65504, 18720, 43680, 18720, 39520, 0]],
        '翔' : [12, 13, 0, -2, 12, [4096, 40672, 41504, 62240, 19104, 58976, 16928, 18016, 60064, 21280, 33312, 34400, 0]],
        '翡' : [12, 13, 0, -2, 12, [2560, 2560, 64480, 2560, 31680, 2560, 62432, 4608, 64480, 43680, 18720, 39520, 0]],
        '翦' : [12, 13, 0, -2, 12, [4224, 4352, 65504, 19008, 27200, 23104, 18496, 64480, 18720, 43680, 18720, 39520, 0]],
        '翩' : [12, 13, 0, -2, 12, [0, 65504, 288, 63776, 35680, 64928, 33056, 64352, 44448, 63776, 43296, 43872, 0]],
        '翳' : [12, 13, 0, -2, 12, [0, 64448, 45632, 55264, 47648, 37312, 43552, 65504, 18720, 43680, 18720, 39520, 0]],
        '翹' : [12, 13, 0, -2, 12, [8192, 10080, 63776, 9056, 64928, 20768, 56160, 21920, 63776, 21344, 36864, 40928, 0]],
        '飜' : [12, 13, 0, -2, 12, [0, 65472, 43360, 10048, 64864, 25856, 44992, 62816, 42304, 64864, 43328, 63840, 0]],
        '耆' : [12, 13, 0, -2, 12, [2048, 2176, 32704, 2560, 65504, 10368, 51488, 16352, 8256, 16320, 8256, 16320, 0]],
        '耄' : [12, 13, 0, -2, 12, [2048, 2176, 32704, 2560, 65504, 10368, 51488, 4064, 30720, 4064, 63488, 4064, 0]],
        '耋' : [12, 13, 0, -2, 12, [2048, 2176, 32704, 2560, 65504, 10368, 18720, 65504, 4352, 16256, 1024, 65504, 0]],
        '耒' : [12, 13, 0, -2, 12, [1024, 1984, 31744, 1024, 16256, 1024, 1024, 65504, 3328, 13440, 50272, 1024, 0]],
        '耘' : [12, 13, 0, -2, 12, [8192, 15296, 57344, 8192, 63488, 10208, 8448, 63744, 8768, 29248, 42144, 10016, 0]],
        '耙' : [12, 13, 0, -2, 12, [8192, 16352, 58656, 9504, 62752, 9504, 10208, 64512, 9216, 29696, 42016, 10208, 0]],
        '耜' : [12, 13, 0, -2, 12, [8192, 15328, 57888, 8736, 64480, 8704, 8704, 64480, 8736, 29216, 43552, 9184, 0]],
        '耡' : [12, 13, 0, -2, 12, [8320, 16000, 60032, 12256, 64160, 10912, 11936, 64160, 10912, 27168, 44832, 14528, 0]],
        '耨' : [12, 13, 0, -2, 12, [8192, 16352, 59392, 12256, 64160, 10816, 10016, 61504, 12256, 28736, 42048, 8896, 0]],
        '耿' : [12, 13, 0, -2, 12, [256, 63744, 37120, 62752, 38176, 38208, 62784, 37120, 37504, 62080, 37952, 6176, 0]],
        '耻' : [12, 13, 0, -2, 12, [128, 63616, 36992, 61568, 38016, 38112, 62592, 38016, 38016, 62592, 38016, 8160, 0]],
        '聊' : [12, 13, 0, -2, 12, [0, 62432, 40096, 61600, 37536, 37536, 62112, 37536, 40608, 62176, 38016, 6272, 0]],
        '聆' : [12, 13, 0, -2, 12, [0, 63936, 37408, 62464, 38880, 36864, 61440, 38880, 37152, 61728, 37216, 4352, 0]],
        '聒' : [12, 13, 0, -2, 12, [0, 63584, 38784, 61568, 36992, 38880, 61568, 36992, 38880, 62496, 37920, 6112, 0]],
        '聘' : [12, 13, 0, -2, 12, [256, 61696, 45024, 59680, 45024, 43296, 65504, 41472, 41952, 58400, 42016, 8384, 0]],
        '聚' : [12, 13, 0, -2, 12, [0, 65504, 18464, 31264, 19008, 31104, 51776, 7200, 58688, 3200, 13376, 50208, 0]],
        '聟' : [12, 13, 0, -2, 12, [16384, 32736, 37408, 65056, 11232, 17408, 65504, 8320, 16256, 8416, 65408, 128, 0]],
        '聢' : [12, 13, 0, -2, 12, [256, 61696, 40928, 61472, 36864, 40928, 61696, 37120, 38368, 62720, 38656, 6368, 0]],
        '聨' : [12, 13, 0, -2, 12, [0, 62016, 42112, 59680, 45024, 42112, 59680, 45024, 41536, 65504, 41536, 11328, 0]],
        '聳' : [12, 13, 0, -2, 12, [2176, 18560, 38208, 8992, 51648, 20224, 63968, 8320, 16256, 8416, 16256, 49280, 0]],
        '聲' : [12, 13, 0, -2, 12, [4096, 65472, 4672, 64608, 22400, 21600, 65408, 41088, 16256, 8416, 16256, 49280, 0]],
        '聰' : [12, 13, 0, -2, 12, [256, 61952, 45024, 59680, 44704, 43296, 60064, 45024, 41472, 59712, 43040, 10112, 0]],
        '聶' : [12, 13, 0, -2, 12, [0, 65504, 8320, 16256, 8320, 65504, 19008, 31680, 19008, 31680, 52800, 2112, 0]],
        '聹' : [12, 13, 0, -2, 12, [256, 57600, 49120, 57632, 42048, 47008, 57344, 45024, 43680, 65504, 41216, 9984, 0]],
        '聽' : [12, 13, 0, -2, 12, [256, 61696, 40928, 61696, 40928, 62624, 40928, 20480, 65504, 21760, 62528, 5024, 0]],
        '聿' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1088, 65504, 1088, 16320, 1024, 32704, 1024, 65504, 1024, 0]],
        '肄' : [12, 13, 0, -2, 12, [128, 39040, 59360, 34976, 64480, 16544, 31712, 36992, 65504, 4224, 11232, 50304, 0]],
        '肆' : [12, 13, 0, -2, 12, [128, 63616, 34784, 61600, 34784, 61600, 33760, 63616, 34784, 41088, 38880, 59520, 0]],
        '肅' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1088, 65504, 1088, 16320, 38176, 62944, 33824, 62944, 38176, 0]],
        '肛' : [12, 13, 0, -2, 12, [0, 63456, 36992, 36992, 61568, 36992, 36992, 61568, 36992, 36992, 36992, 47072, 0]],
        '肓' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 32768, 32768, 32704, 8320, 16256, 8320, 16256, 8320, 8576, 0]],
        '肚' : [12, 13, 0, -2, 12, [128, 61568, 36992, 36992, 61568, 40928, 36992, 61568, 36992, 36992, 36992, 45184, 0]],
        '肭' : [12, 13, 0, -2, 12, [1792, 57600, 41216, 45024, 59680, 43296, 43296, 60064, 44128, 43040, 43040, 43104, 0]],
        '冐' : [12, 13, 0, -2, 12, [0, 65504, 32800, 49056, 32800, 16256, 8320, 16256, 8320, 16256, 8320, 8576, 0]],
        '肬' : [12, 13, 0, -2, 12, [256, 61728, 37152, 40928, 61696, 37120, 37248, 62080, 37504, 37504, 38048, 47328, 0]],
        '胛' : [12, 13, 0, -2, 12, [0, 61408, 43296, 43296, 61408, 43296, 43296, 59680, 45024, 41216, 41216, 41216, 0]],
        '胥' : [12, 13, 0, -2, 12, [0, 65504, 1056, 10112, 15360, 59360, 8256, 16320, 8256, 16320, 8256, 8384, 0]],
        '胙' : [12, 13, 0, -2, 12, [1024, 62464, 38880, 38144, 63744, 37344, 37120, 61696, 37344, 37120, 37120, 45312, 0]],
        '胝' : [12, 13, 0, -2, 12, [0, 63456, 38016, 38016, 62592, 38880, 37952, 62528, 37920, 38688, 36864, 47072, 0]],
        '胄' : [12, 13, 0, -2, 12, [1024, 32736, 17440, 32736, 17440, 32736, 8256, 16320, 8256, 16320, 8256, 8384, 0]],
        '胚' : [12, 13, 0, -2, 12, [0, 65504, 36992, 36992, 61696, 37632, 38208, 63776, 37120, 37120, 36864, 49120, 0]],
        '胖' : [12, 13, 0, -2, 12, [256, 57600, 42304, 43296, 57600, 45024, 41216, 57600, 49120, 41216, 41216, 41216, 0]],
        '脉' : [12, 13, 0, -2, 12, [512, 57600, 40960, 44800, 57632, 48448, 42368, 58624, 43392, 43328, 45344, 41728, 0]],
        '胯' : [12, 13, 0, -2, 12, [256, 57600, 45024, 42048, 59424, 45024, 40960, 61408, 41984, 45024, 43040, 41152, 0]],
        '胱' : [12, 13, 0, -2, 12, [256, 57600, 43296, 42304, 57600, 41216, 49120, 57984, 41600, 41600, 42144, 47328, 0]],
        '脛' : [12, 13, 0, -2, 12, [0, 63456, 36864, 37536, 62784, 37536, 37536, 61440, 38880, 36992, 36992, 47072, 0]],
        '脩' : [12, 13, 0, -2, 12, [1024, 9216, 10208, 18496, 19008, 50048, 23648, 22464, 21568, 22464, 21568, 17600, 0]],
        '脣' : [12, 13, 0, -2, 12, [0, 32736, 16384, 32736, 21056, 37248, 48224, 8256, 16320, 8256, 16320, 8256, 0]],
        '脯' : [12, 13, 0, -2, 12, [256, 57632, 45024, 41216, 61408, 43296, 45024, 59680, 45024, 43296, 43296, 43360, 0]],
        '腋' : [12, 13, 0, -2, 12, [256, 57600, 49120, 42240, 59840, 47680, 43584, 60736, 43328, 43136, 43392, 44640, 0]],
        '隋' : [12, 13, 0, -2, 12, [256, 57600, 45024, 41472, 51168, 43136, 47072, 42016, 42976, 50208, 34784, 33824, 0]],
        '腆' : [12, 13, 0, -2, 12, [640, 57984, 45024, 43680, 60064, 45024, 43680, 60064, 49120, 40960, 42048, 43040, 0]],
        '脾' : [12, 13, 0, -2, 12, [256, 57856, 45024, 43296, 61408, 43296, 43296, 61408, 41600, 42112, 49120, 41088, 0]],
        '腓' : [12, 13, 0, -2, 12, [128, 57984, 48864, 41600, 57984, 44768, 41600, 57984, 48768, 41696, 42112, 47232, 0]],
        '腑' : [12, 13, 0, -2, 12, [512, 57856, 49120, 42048, 58432, 44000, 47168, 59968, 43328, 43072, 43072, 43200, 0]],
        '胼' : [12, 13, 0, -2, 12, [2112, 62528, 36992, 40928, 62016, 37440, 37440, 65504, 37440, 37440, 37952, 46144, 0]],
        '腱' : [12, 13, 0, -2, 12, [128, 57472, 49120, 42144, 59360, 43168, 44512, 58496, 47072, 43136, 44032, 46048, 0]],
        '腮' : [12, 13, 0, -2, 12, [0, 61408, 43296, 45024, 59680, 43296, 45024, 57856, 41280, 42016, 46080, 41920, 0]],
        '腥' : [12, 13, 0, -2, 12, [0, 63456, 37920, 38880, 62496, 38880, 38016, 63456, 39040, 38880, 36992, 47072, 0]],
        '腦' : [12, 13, 0, -2, 12, [0, 59680, 45632, 42112, 62016, 43296, 45024, 60064, 43296, 43680, 44064, 45024, 0]],
        '腴' : [12, 13, 0, -2, 12, [256, 58624, 43488, 43296, 60768, 43296, 43296, 61408, 41216, 41600, 42048, 47136, 0]],
        '膃' : [12, 13, 0, -2, 12, [0, 59328, 42304, 42688, 58432, 42944, 40960, 61408, 43680, 43680, 43680, 49120, 0]],
        '膈' : [12, 13, 0, -2, 12, [0, 61408, 40960, 42944, 58432, 42944, 40960, 61408, 43680, 44128, 43936, 43296, 0]],
        '膊' : [12, 13, 0, -2, 12, [256, 57632, 49120, 41216, 61408, 43296, 45024, 59680, 41024, 49120, 42048, 41664, 0]],
        '膀' : [12, 13, 0, -2, 12, [256, 61696, 40928, 37440, 62016, 40928, 37152, 63456, 37376, 37856, 37920, 46272, 0]],
        '膂' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 18048, 31392, 35392, 65504, 16448, 32704, 16448, 32704, 16448, 0]],
        '膠' : [12, 13, 0, -2, 12, [0, 65504, 43680, 42848, 60064, 45600, 41856, 60480, 41376, 42496, 41152, 44800, 0]],
        '膕' : [12, 13, 0, -2, 12, [0, 61408, 43168, 45024, 59552, 44704, 43744, 61088, 43168, 44896, 43296, 45024, 0]],
        '膤' : [12, 13, 0, -2, 12, [0, 61408, 41216, 45024, 59680, 43872, 40960, 61408, 40992, 45024, 40992, 45024, 0]],
        '膣' : [12, 13, 0, -2, 12, [256, 57600, 49120, 41632, 58496, 49120, 43072, 61408, 41216, 45024, 41216, 49120, 0]],
        '腟' : [12, 13, 0, -2, 12, [256, 57600, 49120, 40992, 61408, 41600, 42048, 61408, 41216, 45024, 41216, 49120, 0]],
        '膓' : [12, 13, 0, -2, 12, [1024, 58368, 45024, 47168, 61376, 43072, 45024, 58368, 49120, 42144, 43296, 45760, 0]],
        '膩' : [12, 13, 0, -2, 12, [128, 57504, 49120, 41088, 61312, 41088, 49088, 59712, 44864, 43296, 44832, 45472, 0]],
        '膰' : [12, 13, 0, -2, 12, [0, 61408, 43328, 42368, 65504, 42304, 43296, 65504, 43296, 45024, 43296, 45024, 0]],
        '膵' : [12, 13, 0, -2, 12, [1088, 58432, 49120, 42048, 57600, 49120, 42048, 60064, 41216, 49120, 41216, 41216, 0]],
        '膾' : [12, 13, 0, -2, 12, [0, 59328, 43040, 47072, 57344, 45024, 43680, 61408, 42016, 42976, 42016, 42976, 0]],
        '膸' : [12, 13, 0, -2, 12, [256, 61696, 45024, 41472, 63456, 43136, 42976, 64032, 44000, 43552, 44032, 46048, 0]],
        '膽' : [12, 13, 0, -2, 12, [0, 59328, 43136, 49120, 59712, 44640, 43136, 61408, 43008, 45024, 46112, 42976, 0]],
        '臀' : [12, 13, 0, -2, 12, [0, 64448, 35392, 64608, 43904, 64608, 43968, 16448, 32704, 16448, 32704, 16448, 0]],
        '臂' : [12, 13, 0, -2, 12, [256, 61696, 38880, 62016, 40928, 37120, 65504, 16448, 32704, 16448, 32704, 16448, 0]],
        '膺' : [12, 13, 0, -2, 12, [512, 512, 32736, 19008, 22496, 31872, 22496, 21632, 24544, 36896, 40928, 4128, 0]],
        '臉' : [12, 13, 0, -2, 12, [0, 59328, 43040, 47072, 57344, 44768, 43680, 60064, 44768, 42048, 43680, 45344, 0]],
        '臍' : [12, 13, 0, -2, 12, [512, 57856, 49120, 42304, 63296, 44448, 43072, 61376, 43072, 44992, 43072, 45120, 0]],
        '臑' : [12, 13, 0, -2, 12, [0, 61408, 41216, 45024, 59680, 43872, 40960, 61408, 41472, 45024, 43680, 43680, 0]],
        '臙' : [12, 13, 0, -2, 12, [2304, 59648, 49120, 43264, 61248, 41056, 44864, 59712, 44896, 40960, 46400, 42272, 0]],
        '臘' : [12, 13, 0, -2, 12, [2336, 62016, 43296, 45024, 60064, 43296, 45024, 59712, 44480, 43296, 44448, 47904, 0]],
        '臈' : [12, 13, 0, -2, 12, [1152, 58496, 49120, 43072, 61376, 43072, 44992, 58368, 45024, 47648, 43296, 44896, 0]],
        '臚' : [12, 13, 0, -2, 12, [960, 57856, 45024, 43296, 60352, 43264, 45024, 62784, 42304, 45024, 43680, 49120, 0]],
        '臟' : [12, 13, 0, -2, 12, [1152, 58496, 49120, 42144, 61408, 47744, 44960, 59552, 44992, 47680, 44960, 43296, 0]],
        '臠' : [12, 13, 0, -2, 12, [1024, 24384, 41120, 20032, 61920, 20032, 19008, 65504, 37152, 42144, 35360, 37216, 0]],
        '臧' : [12, 13, 0, -2, 12, [128, 160, 49120, 41088, 61312, 10912, 61344, 43328, 44864, 10816, 20384, 16672, 0]],
        '臺' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 8320, 65504, 35104, 4224, 32704, 1024, 65504, 0]],
        '臻' : [12, 13, 0, -2, 12, [256, 61696, 12256, 16896, 40928, 62592, 20448, 20800, 65504, 17152, 19776, 61728, 0]],
        '臾' : [12, 13, 0, -2, 12, [1024, 13312, 50656, 33824, 62944, 33824, 33824, 62944, 1024, 2560, 12672, 49248, 0]],
        '舁' : [12, 13, 0, -2, 12, [4096, 25536, 16448, 29632, 16448, 16448, 32704, 4352, 65504, 4352, 8448, 49408, 0]],
        '舂' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2048, 32704, 4352, 64480, 8320, 31680, 41120, 8320, 16256, 0]],
        '舅' : [12, 13, 0, -2, 12, [0, 31200, 16416, 31200, 16416, 32736, 9280, 16320, 9280, 32736, 2080, 29120, 0]],
        '與' : [12, 13, 0, -2, 12, [2048, 61152, 34848, 59616, 36384, 58080, 35360, 35360, 65504, 0, 16448, 32800, 0]],
        '舊' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8704, 32704, 41472, 8704, 31712, 16448, 31680, 16448, 32704, 0]],
        '舍' : [12, 13, 0, -2, 12, [0, 3584, 12672, 49248, 16256, 1024, 65504, 1024, 16256, 8320, 8320, 16256, 0]],
        '舐' : [12, 13, 0, -2, 12, [0, 6240, 59264, 9344, 9344, 64640, 10208, 9280, 62528, 37952, 38304, 65056, 0]],
        '舖' : [12, 13, 0, -2, 12, [256, 28960, 36832, 256, 63456, 9504, 63456, 9504, 63456, 38176, 38176, 62816, 0]],
        '舩' : [12, 13, 0, -2, 12, [8192, 17344, 61504, 37952, 54304, 38176, 63776, 37120, 53760, 37408, 37920, 47072, 0]],
        '舫' : [12, 13, 0, -2, 12, [8448, 16640, 65504, 37120, 53504, 37344, 64032, 37408, 54304, 37920, 36928, 45440, 0]],
        '舸' : [12, 13, 0, -2, 12, [8192, 20448, 61472, 36896, 55072, 38176, 64800, 38176, 55072, 36896, 36896, 45280, 0]],
        '舳' : [12, 13, 0, -2, 12, [8448, 16640, 61696, 38880, 54560, 38176, 64800, 38880, 54560, 38176, 38176, 47072, 0]],
        '艀' : [12, 13, 0, -2, 12, [8192, 18400, 62464, 39232, 53536, 36864, 64480, 36928, 53376, 38880, 36992, 45440, 0]],
        '艙' : [12, 13, 0, -2, 12, [8192, 17344, 62496, 40928, 54304, 38880, 62496, 38880, 54272, 39904, 37408, 46048, 0]],
        '艘' : [12, 13, 0, -2, 12, [8320, 16768, 63200, 38048, 55008, 38048, 65504, 36992, 55264, 37440, 37248, 48736, 0]],
        '艝' : [12, 13, 0, -2, 12, [8192, 20448, 61696, 40928, 53536, 38752, 61440, 38880, 53280, 38880, 36896, 47072, 0]],
        '艚' : [12, 13, 0, -2, 12, [9344, 17536, 65504, 37536, 57312, 37536, 65504, 37952, 55232, 37952, 37952, 47040, 0]],
        '艟' : [12, 13, 0, -2, 12, [8448, 16640, 65504, 37440, 53824, 40928, 62752, 38880, 54560, 38880, 37120, 49120, 0]],
        '艤' : [12, 13, 0, -2, 12, [9280, 17536, 65504, 37120, 55264, 37120, 65504, 37888, 57312, 38048, 40512, 46496, 0]],
        '艢' : [12, 13, 0, -2, 12, [8448, 16640, 65504, 38208, 54592, 39840, 61728, 40928, 54432, 38816, 36896, 49120, 0]],
        '艨' : [12, 13, 0, -2, 12, [9344, 17536, 65504, 38016, 57312, 36896, 63424, 38144, 55712, 37568, 40096, 45824, 0]],
        '艪' : [12, 13, 0, -2, 12, [8192, 17344, 62592, 40928, 54432, 38880, 64160, 38880, 54304, 38880, 37920, 47072, 0]],
        '艫' : [12, 13, 0, -2, 12, [9152, 16896, 65504, 37408, 55232, 37376, 63456, 38208, 54592, 40928, 37536, 49120, 0]],
        '舮' : [12, 13, 0, -2, 12, [8192, 18400, 61440, 36864, 54240, 37408, 64032, 37856, 53760, 37376, 37888, 46080, 0]],
        '艱' : [12, 13, 0, -2, 12, [0, 21472, 65056, 21472, 64032, 43552, 64480, 8832, 65184, 8864, 21056, 35616, 0]],
        '艷' : [12, 13, 0, -2, 12, [0, 43968, 65088, 43648, 65504, 43680, 65184, 17376, 32256, 10752, 10784, 65504, 0]],
        '艸' : [12, 13, 0, -2, 12, [4224, 4224, 4224, 38048, 38048, 38048, 38048, 65504, 38016, 4224, 8320, 49280, 0]],
        '艾' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 8320, 4352, 2560, 1024, 2560, 12672, 49248, 0]],
        '芍' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 4096, 16352, 16416, 32800, 4128, 2080, 64, 896, 0]],
        '芒' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 1024, 65504, 8192, 8192, 8192, 8192, 8128, 0]],
        '芫' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 16256, 0, 65504, 4608, 4608, 8736, 50144, 0]],
        '芟' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 7936, 4384, 57568, 16256, 4224, 4352, 3840, 61664, 0]],
        '芻' : [12, 13, 0, -2, 12, [16384, 16384, 32736, 34848, 43296, 16160, 20512, 32736, 34848, 43296, 16160, 4288, 0]],
        '芬' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 5888, 8320, 16448, 49056, 2176, 4224, 25344, 0]],
        '苡' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 9280, 8768, 8320, 8320, 14656, 49696, 3104, 0]],
        '苣' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32736, 16384, 32704, 16448, 32704, 16384, 16384, 32736, 0]],
        '苟' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 4096, 16352, 16416, 48160, 9248, 9248, 15424, 384, 0]],
        '苒' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 32704, 17472, 32704, 17472, 65504, 16448, 16576, 0]],
        '苴' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 16256, 8320, 16256, 8320, 16256, 8320, 65504, 0]],
        '苳' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4224, 10496, 50688, 2432, 13408, 49664, 0, 896, 7168, 0]],
        '苺' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 10368, 9344, 65504, 18560, 17536, 32736, 256, 0]],
        '莓' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8448, 32736, 43136, 9344, 65504, 18560, 17536, 32736, 256, 0]],
        '范' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16384, 12224, 34880, 18496, 2496, 10240, 18464, 36832, 0]],
        '苻' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 4160, 8256, 24544, 49216, 18496, 17472, 16448, 16576, 0]],
        '苹' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 32704, 9344, 17472, 1024, 65504, 1024, 1024, 0]],
        '苞' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 4096, 16352, 25120, 41504, 16064, 8192, 8224, 16352, 0]],
        '茆' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 3072, 29152, 17696, 17696, 17696, 64800, 2400, 12544, 0]],
        '苜' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 16448, 32704, 16448, 32704, 16448, 16448, 32704, 0]],
        '茉' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 65504, 1024, 32704, 3328, 13440, 50272, 1024, 0]],
        '苙' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 1024, 32704, 4352, 4352, 4608, 512, 65504, 0]],
        '茵' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 33824, 49056, 33824, 35360, 45472, 32800, 65504, 0]],
        '茴' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 32800, 40736, 37152, 37152, 40736, 32800, 65504, 0]],
        '茖' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4224, 10496, 17920, 6528, 57440, 16256, 8320, 8320, 16256, 0]],
        '茲' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 8320, 16640, 35360, 20800, 8320, 18720, 35360, 64480, 0]],
        '茱' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 9216, 16320, 17408, 33792, 65504, 3328, 13440, 50272, 0]],
        '荀' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 4096, 16352, 16416, 48160, 9248, 15392, 9280, 15744, 0]],
        '茹' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8448, 8192, 64992, 9504, 9504, 17696, 26912, 6432, 58848, 0]],
        '荐' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 4096, 10176, 24704, 41216, 12256, 8448, 8960, 0]],
        '荅' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 3584, 4352, 16256, 16448, 49056, 8320, 8320, 16256, 0]],
        '茯' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 4384, 8480, 24544, 49408, 16640, 17024, 17472, 22560, 0]],
        '茫' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16640, 12256, 34816, 18432, 2048, 10240, 18432, 34784, 0]],
        '茗' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 4032, 4160, 26752, 1280, 8160, 12320, 53280, 8160, 0]],
        '茘' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 2112, 4160, 8320, 64480, 10400, 18720, 39520, 0]],
        '莅' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 4352, 16352, 17472, 50240, 17472, 17536, 16512, 24544, 0]],
        '莚' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 61408, 8448, 16832, 59648, 10496, 20448, 45056, 36832, 0]],
        '莪' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4224, 63648, 9184, 64640, 8352, 11328, 61504, 8608, 26144, 0]],
        '莟' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 2560, 12672, 57184, 256, 512, 32704, 16448, 16448, 32704, 0]],
        '莢' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 65504, 9344, 9344, 21824, 2560, 12672, 49248, 0]],
        '莖' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 17472, 34944, 17472, 8736, 32704, 1024, 65504, 0]],
        '茣' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 24512, 20544, 32704, 1088, 65504, 2560, 12672, 49248, 0]],
        '莎' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16640, 9536, 35104, 16640, 768, 8224, 16576, 36608, 0]],
        '莇' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 512, 61952, 40928, 61984, 37408, 61984, 37920, 63680, 0]],
        '莊' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4224, 36992, 61568, 6112, 61568, 20608, 20608, 36992, 38880, 0]],
        '荼' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 3584, 12672, 57184, 1024, 32704, 1024, 17472, 35872, 0]],
        '莵' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 17472, 17472, 32704, 2688, 2688, 4640, 58336, 0]],
        '荳' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 32704, 8320, 8320, 16256, 4352, 4352, 65504, 0]],
        '荵' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 1088, 9280, 6208, 25728, 4672, 20512, 36736, 0]],
        '莠' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 1024, 65504, 9280, 24352, 35264, 4160, 24960, 0]],
        '莉' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 3104, 28960, 4384, 64800, 12576, 22816, 37920, 4192, 0]],
        '莨' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 8320, 16256, 8320, 16288, 8768, 8576, 63584, 0]],
        '菴' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 65504, 5376, 9344, 32704, 42144, 16256, 9248, 2016, 0]],
        '萓' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 65504, 32800, 40736, 4352, 7936, 4352, 65504, 0]],
        '菫' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 9344, 9344, 16256, 1024, 32704, 1024, 65504, 0]],
        '菎' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 16448, 32704, 16448, 32704, 16896, 31712, 16896, 16928, 64480, 0]],
        '菽' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 8192, 15328, 8224, 64800, 8512, 43328, 43392, 26208, 0]],
        '萃' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 65504, 8320, 21824, 33824, 65504, 1024, 1024, 0]],
        '菘' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 8640, 9280, 62528, 10528, 10528, 25088, 42016, 10208, 0]],
        '萋' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1152, 32736, 1152, 16256, 2048, 65504, 4224, 7936, 24768, 0]],
        '菁' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 16256, 1024, 65504, 9344, 9344, 16256, 8320, 8576, 0]],
        '菷' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 128, 65504, 128, 16256, 0, 65504, 42144, 42144, 9600, 0]],
        '萇' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16320, 8192, 16320, 8192, 65504, 8768, 8576, 64608, 0]],
        '菠' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 20448, 10528, 35072, 20416, 2112, 4672, 20864, 34400, 0]],
        '菲' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4608, 2560, 64480, 2560, 31680, 2560, 64480, 4608, 25088, 0]],
        '萍' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4096, 20448, 8448, 34112, 18720, 256, 8160, 16640, 33024, 0]],
        '萢' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 18432, 12256, 36896, 20256, 352, 12032, 18464, 36832, 0]],
        '萠' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 0, 64480, 35360, 60320, 35360, 60320, 35360, 39520, 0]],
        '莽' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 2048, 65504, 8320, 18752, 35104, 32704, 4352, 24832, 0]],
        '萸' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 58848, 33824, 62944, 33824, 58592, 2560, 12672, 49248, 0]],
        '蔆' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 20448, 8448, 40928, 17536, 7136, 1088, 18304, 39008, 0]],
        '菻' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 8320, 8320, 64480, 8320, 12672, 27328, 42144, 8320, 0]],
        '葭' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 64480, 34848, 64480, 32768, 64480, 33312, 63936, 34336, 0]],
        '萪' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 6720, 57664, 9280, 64064, 8288, 29120, 42560, 8256, 0]],
        '萼' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 19008, 19008, 32704, 0, 65504, 4096, 16320, 8256, 384, 0]],
        '蕚' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 32704, 19008, 19008, 65504, 4096, 16320, 8256, 384, 0]],
        '蒄' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 32800, 63456, 64, 64064, 20800, 36896, 40928, 0]],
        '葷' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 33824, 32704, 9344, 16256, 9344, 65504, 1024, 0]],
        '葫' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8448, 9184, 64032, 9184, 8736, 64480, 35360, 35360, 64608, 0]],
        '蒭' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8448, 32736, 42272, 16160, 10304, 32736, 42272, 16160, 2240, 0]],
        '葮' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 63424, 33856, 62528, 34912, 34752, 62528, 33664, 40032, 0]],
        '蒂' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 32704, 4352, 65504, 33824, 49056, 9344, 9344, 9600, 0]],
        '葩' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 20736, 18400, 62624, 38048, 38048, 63456, 37888, 37920, 63456, 0]],
        '葆' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 12256, 18464, 53216, 16640, 24544, 17152, 17728, 22816, 0]],
        '萬' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 9344, 16256, 9344, 65504, 33952, 49056, 32864, 0]],
        '葯' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 9472, 17408, 63456, 9248, 18464, 64032, 8480, 43040, 43200, 0]],
        '葹' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 21504, 18400, 64768, 17888, 30496, 40288, 38144, 5152, 26592, 0]],
        '萵' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 16256, 8320, 10112, 9344, 65504, 37152, 40736, 32864, 0]],
        '蓊' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 9344, 51552, 16256, 0, 64480, 43680, 18720, 39520, 0]],
        '葢' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 65504, 4352, 9344, 65504, 19008, 19008, 65504, 0]],
        '蒹' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 2304, 4608, 65504, 4736, 65504, 4736, 16256, 21056, 37408, 0]],
        '蒿' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 65504, 4352, 7936, 0, 65504, 37152, 40800, 0]],
        '蒟' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8448, 8448, 64480, 5152, 21408, 21152, 21408, 4128, 63680, 0]],
        '蓙' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 512, 32736, 18560, 18560, 21824, 25120, 40896, 33280, 16352, 0]],
        '蓍' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 2304, 32704, 10496, 51744, 16352, 8320, 16256, 8320, 16256, 0]],
        '蒻' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 64480, 2080, 64480, 33280, 64480, 43680, 18720, 45760, 0]],
        '蓚' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8704, 9184, 23616, 54144, 21088, 23680, 21280, 20672, 18176, 0]],
        '蓐' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 16384, 32736, 21056, 37248, 48224, 128, 65504, 4224, 2432, 0]],
        '蓁' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 2048, 16256, 4352, 65504, 9344, 32704, 36128, 5248, 25664, 0]],
        '蓆' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 18560, 32736, 18560, 20352, 16896, 24512, 37440, 37440, 4800, 0]],
        '蓖' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 21824, 19008, 21824, 32704, 16896, 31712, 16896, 20000, 62432, 0]],
        '蒡' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 32704, 4352, 4352, 65504, 34848, 4032, 4160, 24960, 0]],
        '蔡' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 35392, 20800, 8320, 57184, 0, 65504, 1024, 17472, 35872, 0]],
        '蓿' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 1024, 65504, 41504, 12224, 18496, 53184, 18496, 20416, 0]],
        '蓴' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 9344, 16256, 9344, 32640, 1088, 65504, 128, 8320, 4992, 0]],
        '蔗' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 512, 32736, 18560, 32736, 18560, 36736, 32768, 10816, 18720, 0]],
        '蔘' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 2560, 4352, 24384, 35360, 64480, 9856, 55392, 768, 15360, 0]],
        '蔬' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 256, 63456, 4672, 9280, 10144, 45088, 42304, 42304, 63840, 0]],
        '蔟' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 21504, 18400, 64000, 19424, 29824, 38880, 36992, 4416, 26144, 0]],
        '蔕' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 10880, 65504, 10880, 52960, 0, 65504, 42144, 9344, 9600, 0]],
        '蔔' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 20736, 32736, 41504, 41504, 32544, 18720, 32544, 18720, 32704, 0]],
        '蓼' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 64480, 43680, 18720, 39520, 9600, 55392, 768, 15360, 0]],
        '蕀' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 64480, 8320, 64480, 43680, 45472, 27328, 42144, 8320, 0]],
        '蕣' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 17472, 65504, 41120, 15296, 51840, 14304, 49280, 0]],
        '蕘' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 65504, 8320, 31680, 8320, 65504, 2560, 12832, 50144, 0]],
        '蕈' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 65504, 19008, 32704, 8320, 16256, 8320, 65504, 1024, 0]],
        '蕁' : [12, 13, 0, -2, 12, [4352, 65504, 4352, 16256, 128, 65504, 128, 31712, 8736, 65504, 8320, 4480, 0]],
        '蘂' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 10880, 18496, 7936, 41504, 48032, 1024, 65504, 5376, 58592, 0]],
        '蕋' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 10176, 9216, 65504, 8320, 47840, 41600, 41600, 64480, 0]],
        '蕕' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 9280, 45024, 21120, 45024, 10912, 27296, 44256, 10272, 53216, 0]],
        '薀' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 20448, 10528, 35488, 20448, 0, 12256, 19104, 40928, 0]],
        '薤' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 62080, 20192, 29312, 21120, 48864, 4736, 8832, 53216, 0]],
        '薈' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8320, 32704, 54624, 17472, 32704, 8320, 16256, 8320, 16256, 0]],
        '薑' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 9344, 16256, 9344, 65504, 17472, 32704, 17472, 17472, 65504, 0]],
        '薊' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 18688, 64544, 42272, 64800, 42272, 64800, 288, 43040, 42080, 0]],
        '薨' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 19008, 32704, 0, 65504, 18976, 43968, 10752, 4640, 58336, 0]],
        '蕭' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1152, 65504, 1152, 49056, 38176, 62944, 33824, 62944, 38176, 0]],
        '薔' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 32736, 9344, 21824, 65504, 20800, 24384, 16448, 32704, 0]],
        '薛' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 16512, 63456, 37440, 62016, 36832, 61568, 38880, 36992, 61568, 0]],
        '藪' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 21120, 65248, 21280, 32416, 21152, 65344, 17472, 14496, 50976, 0]],
        '薇' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1280, 21984, 40768, 8256, 24384, 51776, 19072, 20864, 22112, 0]],
        '薜' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4224, 63456, 37440, 62016, 36832, 61568, 38880, 36992, 61568, 0]],
        '蕷' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8448, 63456, 8320, 18400, 62496, 22496, 25632, 18368, 55328, 0]],
        '蕾' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 65504, 42272, 5248, 32704, 17472, 32704, 17472, 32704, 0]],
        '薐' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 61408, 8448, 65504, 9344, 15328, 25664, 42880, 14432, 0]],
        '藉' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8768, 14304, 57920, 10208, 64032, 9184, 29216, 43552, 9184, 0]],
        '薺' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 2560, 62944, 21824, 38176, 16320, 8256, 16320, 16448, 16448, 0]],
        '藏' : [12, 13, 0, -2, 12, [4352, 4384, 65504, 8320, 48768, 59552, 16032, 58016, 48704, 43072, 32416, 17184, 0]],
        '薹' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 32704, 8320, 65504, 35104, 4224, 32704, 1024, 65504, 0]],
        '藐' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 20736, 23520, 29216, 54240, 14880, 52192, 14720, 51872, 13536, 0]],
        '藕' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8448, 14304, 58656, 10208, 62752, 12256, 27168, 43936, 10336, 0]],
        '藝' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8448, 64448, 20800, 44864, 29408, 9216, 65504, 4224, 32576, 0]],
        '藥' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 42144, 24384, 45472, 24384, 37152, 65504, 3328, 13440, 50272, 0]],
        '藜' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8448, 64480, 9376, 28960, 44640, 13696, 58592, 13696, 50272, 0]],
        '藹' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 2112, 61376, 2112, 61376, 1024, 61408, 47648, 43296, 61280, 0]],
        '蘊' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8448, 20448, 59680, 10912, 20448, 61440, 12256, 43680, 49120, 0]],
        '蘓' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4672, 59360, 9504, 65504, 9504, 14304, 24576, 43680, 12960, 0]],
        '蘋' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8448, 48096, 41088, 64480, 8736, 44000, 25120, 7136, 58912, 0]],
        '藾' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8448, 64480, 8480, 64480, 43552, 64480, 12832, 27616, 42528, 0]],
        '藺' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 4352, 64480, 35360, 64480, 40736, 46112, 40736, 37920, 40800, 0]],
        '蘆' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 1024, 32736, 17440, 24512, 21120, 21120, 49120, 43680, 32736, 0]],
        '蘢' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8704, 64480, 36896, 64480, 35328, 64480, 35328, 64032, 35808, 0]],
        '蘚' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 21056, 63808, 45024, 63616, 44000, 63616, 2016, 43136, 42112, 0]],
        '蘰' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 9280, 18368, 62528, 12256, 19104, 65504, 9280, 41856, 44128, 0]],
        '蘿' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 19008, 32704, 21056, 42976, 19584, 63456, 9344, 46208, 42976, 0]],
        '虍' : [12, 13, 0, -2, 12, [1024, 1984, 1024, 32736, 16416, 17408, 17504, 18304, 31744, 17408, 33824, 34784, 0]],
        '乕' : [12, 13, 0, -2, 12, [0, 32736, 16896, 32736, 16896, 32736, 16896, 24544, 21024, 37408, 37472, 512, 0]],
        '虔' : [12, 13, 0, -2, 12, [1024, 1984, 1024, 32736, 18464, 20416, 16896, 32736, 18560, 34944, 34560, 14560, 0]],
        '號' : [12, 13, 0, -2, 12, [256, 61920, 37120, 38880, 62752, 1984, 64768, 17888, 62528, 37184, 4416, 26208, 0]],
        '虧' : [12, 13, 0, -2, 12, [8192, 15328, 8320, 64640, 42976, 47360, 41216, 64992, 41504, 64032, 40992, 64704, 0]],
        '虱' : [12, 13, 0, -2, 12, [0, 65472, 64, 65344, 2112, 32576, 18752, 18752, 32576, 2080, 2336, 65312, 0]],
        '蚓' : [12, 13, 0, -2, 12, [0, 10144, 8352, 63648, 43936, 43552, 43552, 64416, 8352, 10400, 10400, 65312, 0]],
        '蚣' : [12, 13, 0, -2, 12, [0, 9152, 8256, 64064, 43552, 44064, 43296, 63744, 8448, 10784, 10784, 64480, 0]],
        '蚩' : [12, 13, 0, -2, 12, [1024, 33824, 65504, 2048, 4096, 65504, 1024, 16256, 9344, 16256, 1056, 65504, 0]],
        '蚪' : [12, 13, 0, -2, 12, [0, 8256, 8768, 63808, 43072, 44096, 43584, 63584, 8640, 11840, 10304, 63552, 0]],
        '蚋' : [12, 13, 0, -2, 12, [768, 8448, 8448, 65504, 43296, 43296, 43296, 64160, 11360, 10272, 10272, 63584, 0]],
        '蚌' : [12, 13, 0, -2, 12, [128, 8320, 10208, 63616, 43136, 44000, 43136, 63616, 10208, 10368, 10368, 63616, 0]],
        '蚶' : [12, 13, 0, -2, 12, [0, 8736, 8736, 65504, 43552, 43552, 43552, 64480, 8736, 10784, 10784, 64480, 0]],
        '蚯' : [12, 13, 0, -2, 12, [0, 8384, 8960, 64000, 43520, 44000, 43584, 64064, 8768, 10816, 10816, 65504, 0]],
        '蛄' : [12, 13, 0, -2, 12, [128, 8320, 8320, 65504, 43136, 43136, 43136, 64480, 8736, 10784, 10784, 64480, 0]],
        '蛆' : [12, 13, 0, -2, 12, [0, 9184, 8736, 64032, 44000, 43552, 43552, 64480, 8736, 10784, 10784, 65504, 0]],
        '蚰' : [12, 13, 0, -2, 12, [128, 8320, 8320, 64480, 43680, 43680, 43680, 64480, 8864, 10912, 10912, 64480, 0]],
        '蛉' : [12, 13, 0, -2, 12, [0, 8640, 8736, 64512, 44000, 43008, 43008, 64480, 8480, 10528, 10592, 63744, 0]],
        '蠣' : [12, 13, 0, -2, 12, [0, 10208, 9344, 65504, 42304, 42944, 42304, 64832, 10208, 10528, 11168, 63584, 0]],
        '蚫' : [12, 13, 0, -2, 12, [512, 8704, 9184, 64032, 44960, 43680, 43680, 64416, 8800, 10752, 10784, 64480, 0]],
        '蛔' : [12, 13, 0, -2, 12, [0, 12256, 10272, 63520, 43936, 43680, 43680, 64160, 11168, 10272, 10272, 65504, 0]],
        '蛞' : [12, 13, 0, -2, 12, [0, 8288, 9088, 63616, 43136, 45024, 43136, 63616, 9184, 10784, 10784, 64480, 0]],
        '蛩' : [12, 13, 0, -2, 12, [0, 63424, 9280, 9792, 9536, 13376, 50272, 16256, 9344, 16256, 1056, 65504, 0]],
        '蛬' : [12, 13, 0, -2, 12, [4352, 4352, 32704, 4352, 65504, 5248, 9280, 65440, 9344, 16256, 1056, 65504, 0]],
        '蛟' : [12, 13, 0, -2, 12, [256, 8448, 12256, 63488, 43584, 44064, 43008, 64576, 8832, 10624, 10816, 64544, 0]],
        '蛛' : [12, 13, 0, -2, 12, [128, 9344, 9344, 65504, 43136, 43136, 45024, 63616, 8576, 10944, 11424, 63616, 0]],
        '蛯' : [12, 13, 0, -2, 12, [512, 8704, 10208, 64064, 43648, 45024, 43264, 64096, 9088, 10752, 10784, 64480, 0]],
        '蜒' : [12, 13, 0, -2, 12, [0, 12000, 8768, 64064, 44096, 43488, 44608, 64064, 8768, 11232, 11776, 63968, 0]],
        '蜆' : [12, 13, 0, -2, 12, [0, 9184, 8736, 64480, 43552, 44000, 43552, 64480, 8320, 10624, 10912, 64736, 0]],
        '蜈' : [12, 13, 0, -2, 12, [0, 9184, 10784, 64032, 44000, 43008, 45024, 63648, 10208, 10368, 10560, 65056, 0]],
        '蜀' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 16384, 32736, 34848, 48928, 10528, 16160, 2336, 32448, 0]],
        '蜃' : [12, 13, 0, -2, 12, [0, 32736, 16384, 32736, 21056, 37248, 37984, 16256, 9344, 16256, 1056, 65504, 0]],
        '蛻' : [12, 13, 0, -2, 12, [0, 8640, 8256, 64032, 45024, 43552, 43552, 64480, 8320, 10624, 10912, 64736, 0]],
        '蜑' : [12, 13, 0, -2, 12, [0, 61408, 8448, 17888, 58624, 12256, 20480, 49120, 9344, 16256, 1056, 65504, 0]],
        '蜉' : [12, 13, 0, -2, 12, [0, 10208, 9216, 63808, 43296, 43008, 44000, 63552, 8320, 12256, 10368, 63872, 0]],
        '蜍' : [12, 13, 0, -2, 12, [0, 8640, 8736, 64512, 44000, 43136, 43136, 65504, 8320, 10912, 11424, 63872, 0]],
        '蛹' : [12, 13, 0, -2, 12, [0, 10176, 8320, 63744, 45024, 43296, 45024, 63776, 12256, 10528, 10528, 63840, 0]],
        '蜊' : [12, 13, 0, -2, 12, [0, 8608, 9760, 64160, 43680, 44960, 43680, 64160, 9888, 11168, 10784, 64096, 0]],
        '蜴' : [12, 13, 0, -2, 12, [0, 10176, 9280, 65472, 42048, 42944, 41984, 65504, 8864, 9376, 10528, 64192, 0]],
        '蜿' : [12, 13, 0, -2, 12, [256, 8448, 12256, 63520, 44768, 43680, 43680, 64160, 9952, 10880, 10912, 64736, 0]],
        '蜷' : [12, 13, 0, -2, 12, [256, 9536, 10528, 65504, 43328, 45024, 43584, 65504, 8768, 10944, 10784, 64480, 0]],
        '蜻' : [12, 13, 0, -2, 12, [128, 8320, 10208, 63616, 44000, 43136, 45024, 64160, 8864, 11232, 10784, 64096, 0]],
        '蜥' : [12, 13, 0, -2, 12, [0, 8928, 8832, 65408, 43648, 43744, 44704, 64416, 8864, 10912, 11040, 65056, 0]],
        '蜩' : [12, 13, 0, -2, 12, [0, 12256, 10528, 64416, 43296, 43936, 43296, 64416, 10912, 10912, 11168, 63584, 0]],
        '蜚' : [12, 13, 0, -2, 12, [512, 2560, 64480, 2560, 31680, 2560, 65504, 9344, 9344, 16256, 1056, 65504, 0]],
        '蝠' : [12, 13, 0, -2, 12, [0, 10208, 8192, 64448, 43584, 43968, 43008, 65504, 10528, 12256, 10528, 65504, 0]],
        '蝟' : [12, 13, 0, -2, 12, [0, 12256, 10528, 65504, 43296, 45024, 42048, 65472, 9280, 10176, 9280, 64704, 0]],
        '蝸' : [12, 13, 0, -2, 12, [0, 10176, 9280, 64960, 42304, 45024, 43040, 64416, 10912, 10912, 11168, 63584, 0]],
        '蝌' : [12, 13, 0, -2, 12, [0, 8608, 9760, 64160, 43616, 44832, 43680, 64032, 9824, 11168, 10784, 64032, 0]],
        '蝎' : [12, 13, 0, -2, 12, [0, 10176, 9280, 65472, 42048, 42944, 41984, 65504, 9248, 10528, 12064, 63680, 0]],
        '蝴' : [12, 13, 0, -2, 12, [1024, 9696, 9504, 65312, 42464, 42272, 44832, 63968, 10528, 10528, 12064, 64096, 0]],
        '蝗' : [12, 13, 0, -2, 12, [128, 8448, 9184, 64032, 44000, 43552, 43552, 65504, 8320, 11232, 10368, 65504, 0]],
        '蝨' : [12, 13, 0, -2, 12, [0, 65472, 1088, 65344, 2080, 4256, 64480, 43680, 64480, 8320, 10400, 64480, 0]],
        '蝮' : [12, 13, 0, -2, 12, [512, 8704, 10208, 64544, 42976, 42016, 42976, 64512, 10208, 10816, 10624, 65120, 0]],
        '蝙' : [12, 13, 0, -2, 12, [0, 12256, 8192, 65504, 43040, 45024, 43008, 65504, 10912, 12256, 10912, 64160, 0]],
        '蝓' : [12, 13, 0, -2, 12, [0, 12160, 9280, 63520, 45024, 43008, 44704, 64160, 12096, 11072, 11936, 64160, 0]],
        '蝣' : [12, 13, 0, -2, 12, [2304, 10496, 10720, 65280, 43744, 43040, 44576, 64064, 11232, 10816, 8768, 64704, 0]],
        '蝪' : [12, 13, 0, -2, 12, [0, 10176, 9280, 65472, 42048, 42976, 41984, 65504, 8864, 9376, 10528, 64192, 0]],
        '蠅' : [12, 13, 0, -2, 12, [0, 12256, 10912, 65248, 41600, 44768, 43680, 65248, 10912, 12000, 10880, 64480, 0]],
        '螢' : [12, 13, 0, -2, 12, [4352, 21792, 38208, 10880, 17472, 65504, 33824, 16256, 9344, 16256, 1056, 65504, 0]],
        '螟' : [12, 13, 0, -2, 12, [0, 12256, 10272, 65504, 42016, 42976, 42016, 65504, 8448, 12256, 8768, 64544, 0]],
        '螂' : [12, 13, 0, -2, 12, [1024, 9696, 12064, 63776, 44864, 43392, 43328, 65312, 10528, 11040, 11712, 63744, 0]],
        '螯' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 9280, 64832, 8320, 32704, 42144, 9344, 16256, 1056, 65504, 0]],
        '蟋' : [12, 13, 0, -2, 12, [0, 10208, 9504, 63808, 45024, 43264, 43840, 64800, 8320, 10752, 10816, 64416, 0]],
        '螽' : [12, 13, 0, -2, 12, [0, 8064, 24832, 3584, 12672, 52832, 8320, 65504, 43680, 62400, 10400, 64480, 0]],
        '蟀' : [12, 13, 0, -2, 12, [256, 8448, 10208, 63776, 43584, 43264, 43584, 65440, 8448, 12256, 10496, 63744, 0]],
        '蟐' : [12, 13, 0, -2, 12, [256, 9536, 12256, 63520, 45024, 42048, 42944, 63744, 12256, 10528, 10528, 63840, 0]],
        '雖' : [12, 13, 0, -2, 12, [256, 63808, 35136, 64480, 8768, 65504, 43584, 43584, 64480, 8768, 10816, 64480, 0]],
        '螫' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 9280, 64832, 20608, 54592, 32672, 9344, 16256, 1056, 65504, 0]],
        '蟄' : [12, 13, 0, -2, 12, [8448, 8448, 64448, 20800, 65344, 8896, 65440, 9376, 9344, 16256, 1056, 65504, 0]],
        '螳' : [12, 13, 0, -2, 12, [256, 9536, 12256, 63520, 45024, 42048, 42048, 65472, 8448, 10176, 8448, 65504, 0]],
        '蟇' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8320, 16256, 8320, 65504, 5504, 13632, 57120, 1088, 65504, 0]],
        '蟆' : [12, 13, 0, -2, 12, [576, 8768, 12256, 62016, 45024, 43040, 45024, 63520, 12256, 8448, 8768, 64544, 0]],
        '螻' : [12, 13, 0, -2, 12, [256, 8448, 12256, 63776, 45024, 43296, 45024, 63776, 12256, 9280, 10112, 63584, 0]],
        '蟯' : [12, 13, 0, -2, 12, [256, 8448, 12256, 61696, 45024, 42048, 44768, 62528, 12256, 8832, 9344, 63712, 0]],
        '蟲' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 17472, 32704, 1056, 65504, 43680, 43680, 62400, 10400, 64480, 0]],
        '蟠' : [12, 13, 0, -2, 12, [0, 12256, 10560, 62848, 45024, 42304, 43296, 65504, 10528, 12256, 10528, 65504, 0]],
        '蠏' : [12, 13, 0, -2, 12, [0, 12256, 12960, 58528, 48992, 43584, 49120, 59968, 10816, 16352, 8768, 63040, 0]],
        '蠍' : [12, 13, 0, -2, 12, [256, 12032, 10720, 65312, 43296, 44864, 43072, 65344, 9536, 11680, 10528, 64288, 0]],
        '蟾' : [12, 13, 0, -2, 12, [0, 10176, 10368, 65504, 43328, 44640, 43136, 65504, 10240, 10208, 9248, 65504, 0]],
        '蟶' : [12, 13, 0, -2, 12, [0, 12256, 10912, 65184, 43680, 44768, 43520, 65504, 8448, 10176, 8448, 65504, 0]],
        '蟷' : [12, 13, 0, -2, 12, [256, 9536, 12256, 64160, 43680, 43904, 43008, 65504, 10528, 12256, 10528, 65504, 0]],
        '蠎' : [12, 13, 0, -2, 12, [576, 8768, 12256, 63744, 45024, 43584, 45024, 63744, 8768, 12256, 8768, 64576, 0]],
        '蟒' : [12, 13, 0, -2, 12, [576, 8768, 12256, 64064, 43264, 45024, 43584, 64544, 8768, 12256, 8768, 64576, 0]],
        '蠑' : [12, 13, 0, -2, 12, [2176, 10912, 10912, 62784, 41504, 45024, 43296, 65504, 8448, 9024, 9504, 63744, 0]],
        '蠖' : [12, 13, 0, -2, 12, [1152, 9344, 12256, 64128, 42976, 44160, 42976, 62592, 12256, 9280, 9088, 64608, 0]],
        '蠕' : [12, 13, 0, -2, 12, [0, 12256, 8448, 65504, 43296, 43872, 43008, 65504, 8704, 12256, 10912, 64160, 0]],
        '蠢' : [12, 13, 0, -2, 12, [1024, 32704, 4352, 65504, 12608, 57120, 4352, 65504, 43680, 62400, 10400, 64480, 0]],
        '蠡' : [12, 13, 0, -2, 12, [0, 8064, 8448, 65504, 9536, 51840, 4672, 65504, 43680, 62400, 10400, 64480, 0]],
        '蠱' : [12, 13, 0, -2, 12, [1024, 32704, 17472, 32704, 1056, 65504, 43680, 62400, 10400, 65504, 19008, 65504, 0]],
        '蠶' : [12, 13, 0, -2, 12, [0, 64480, 20800, 64480, 20800, 49120, 8320, 65504, 43680, 62400, 10400, 64480, 0]],
        '蠹' : [12, 13, 0, -2, 12, [1024, 32704, 9344, 65504, 36896, 32704, 8256, 65504, 43680, 62400, 10400, 64480, 0]],
        '蠧' : [12, 13, 0, -2, 12, [1024, 32704, 1024, 65504, 36896, 32704, 8256, 65504, 43680, 62400, 10400, 64480, 0]],
        '蠻' : [12, 13, 0, -2, 12, [1024, 24384, 41120, 20032, 41120, 61152, 19008, 32704, 17472, 32704, 1056, 65504, 0]],
        '衄' : [12, 13, 0, -2, 12, [0, 10208, 16672, 63776, 43296, 43296, 45024, 43552, 43552, 47648, 57888, 4064, 0]],
        '衂' : [12, 13, 0, -2, 12, [0, 10208, 16672, 64800, 43808, 43296, 43424, 43360, 43552, 47648, 58400, 1216, 0]],
        '衒' : [12, 13, 0, -2, 12, [1024, 9440, 24320, 33280, 1024, 10464, 20512, 51744, 17440, 18720, 20768, 24416, 0]],
        '衙' : [12, 13, 0, -2, 12, [0, 12128, 17408, 40704, 2304, 10720, 24352, 49184, 20256, 18720, 18720, 20320, 0]],
        '衞' : [12, 13, 0, -2, 12, [1024, 9440, 24320, 35072, 16128, 2528, 24352, 50208, 24352, 21792, 21792, 21856, 0]],
        '衢' : [12, 13, 0, -2, 12, [0, 16224, 21760, 40704, 5376, 16352, 18976, 57120, 29216, 24352, 21024, 24416, 0]],
        '衫' : [12, 13, 0, -2, 12, [0, 8288, 9088, 63488, 2048, 4192, 9088, 26624, 45056, 10240, 8288, 9088, 0]],
        '袁' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 65504, 8320, 8320, 16256, 5152, 57920, 8576, 31840, 0]],
        '衾' : [12, 13, 0, -2, 12, [0, 3584, 12672, 57184, 0, 32704, 64, 1152, 65504, 4672, 57728, 15456, 0]],
        '袞' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 16256, 24768, 41120, 16256, 5152, 57920, 8576, 31840, 0]],
        '衵' : [12, 13, 0, -2, 12, [0, 18400, 17440, 62496, 5152, 5152, 10208, 17440, 54304, 25632, 21536, 18400, 0]],
        '衽' : [12, 13, 0, -2, 12, [0, 16608, 20224, 61696, 4352, 4352, 12256, 16640, 53504, 24832, 20736, 20448, 0]],
        '袵' : [12, 13, 0, -2, 12, [0, 17504, 18304, 59520, 10368, 14464, 20448, 51328, 26752, 22656, 18560, 19424, 0]],
        '衲' : [12, 13, 0, -2, 12, [1792, 16640, 16640, 61408, 10528, 10528, 18720, 51872, 27744, 22560, 18464, 18528, 0]],
        '袂' : [12, 13, 0, -2, 12, [256, 16640, 20448, 61728, 4384, 4384, 12256, 16640, 57600, 21120, 17472, 22560, 0]],
        '袗' : [12, 13, 0, -2, 12, [0, 17280, 17472, 63520, 4288, 5888, 8192, 16576, 55040, 24608, 20672, 18176, 0]],
        '袒' : [12, 13, 0, -2, 12, [0, 18400, 17440, 62496, 6112, 5152, 9248, 17440, 55264, 24576, 20480, 20448, 0]],
        '袮' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 62592, 6272, 4224, 9408, 17568, 51360, 26752, 16512, 17280, 0]],
        '袙' : [12, 13, 0, -2, 12, [256, 16640, 16896, 63456, 5152, 5152, 9248, 18400, 54304, 25632, 21536, 18400, 0]],
        '袢' : [12, 13, 0, -2, 12, [256, 16640, 17728, 59680, 8448, 12256, 16640, 49408, 28640, 20736, 16640, 16640, 0]],
        '袍' : [12, 13, 0, -2, 12, [1024, 17408, 18400, 63520, 6048, 5280, 9376, 18336, 54368, 25600, 21536, 18400, 0]],
        '袤' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4352, 2560, 65504, 3136, 13696, 50208, 6720, 57728, 15456, 0]],
        '袰' : [12, 13, 0, -2, 12, [0, 16320, 9280, 65504, 17472, 32736, 1152, 65504, 5184, 57920, 8576, 31840, 0]],
        '袿' : [12, 13, 0, -2, 12, [256, 16640, 18400, 61696, 4352, 8160, 8448, 16640, 55264, 24832, 20736, 20448, 0]],
        '袱' : [12, 13, 0, -2, 12, [128, 17568, 17568, 59520, 12256, 14464, 18560, 51328, 26944, 22848, 18976, 19488, 0]],
        '裃' : [12, 13, 0, -2, 12, [256, 16640, 16832, 61696, 4352, 8160, 8192, 20448, 53504, 24960, 20800, 16640, 0]],
        '裄' : [12, 13, 0, -2, 12, [0, 17888, 18432, 61440, 8192, 9696, 18496, 55360, 26688, 18496, 18496, 18880, 0]],
        '裔' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 4672, 57664, 8320, 65504, 37152, 49120, 53536, 37152, 40800, 0]],
        '裘' : [12, 13, 0, -2, 12, [1024, 1088, 65504, 17472, 9344, 3072, 13696, 50272, 32704, 4672, 57728, 15456, 0]],
        '裙' : [12, 13, 0, -2, 12, [0, 20448, 16672, 65504, 4384, 4384, 12256, 16896, 59360, 19488, 21536, 18400, 0]],
        '裝' : [12, 13, 0, -2, 12, [128, 34944, 65504, 2176, 63616, 11232, 18432, 65504, 5184, 57920, 8576, 31840, 0]],
        '裹' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 9344, 16256, 9344, 65504, 5376, 58592, 6720, 57728, 15456, 0]],
        '褂' : [12, 13, 0, -2, 12, [1024, 17472, 24384, 58432, 9280, 16192, 17504, 50256, 24384, 25664, 17472, 24384, 0]],
        '裼' : [12, 13, 0, -2, 12, [0, 18368, 17472, 63424, 5184, 6080, 9216, 20448, 53920, 25760, 18720, 17088, 0]],
        '裴' : [12, 13, 0, -2, 12, [512, 2560, 64480, 2560, 31680, 2560, 4608, 65504, 5184, 57920, 8576, 31840, 0]],
        '裨' : [12, 13, 0, -2, 12, [128, 16640, 18400, 62752, 6112, 5408, 9504, 18400, 57984, 21632, 20448, 16512, 0]],
        '裲' : [12, 13, 0, -2, 12, [0, 24544, 16640, 61408, 10528, 10528, 19872, 52640, 27488, 22816, 18720, 18784, 0]],
        '褄' : [12, 13, 0, -2, 12, [256, 16640, 20448, 61728, 8160, 4384, 12256, 16640, 57312, 25664, 18304, 22624, 0]],
        '褌' : [12, 13, 0, -2, 12, [0, 20448, 18720, 65504, 5408, 6112, 9504, 17696, 59360, 20736, 20448, 16640, 0]],
        '褊' : [12, 13, 0, -2, 12, [0, 20448, 16384, 61408, 10272, 12256, 18432, 53216, 27296, 24544, 19104, 19104, 0]],
        '褓' : [12, 13, 0, -2, 12, [0, 19424, 18976, 59936, 15328, 10368, 20448, 51328, 27008, 23232, 19616, 18560, 0]],
        '襃' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 17440, 30688, 16512, 18400, 31360, 5280, 57920, 8576, 31840, 0]],
        '褞' : [12, 13, 0, -2, 12, [0, 18368, 17728, 63168, 5184, 6080, 8192, 20448, 60064, 23200, 19104, 24544, 0]],
        '褥' : [12, 13, 0, -2, 12, [0, 20448, 18432, 61408, 10912, 10816, 20256, 51264, 28640, 20544, 17472, 17088, 0]],
        '褪' : [12, 13, 0, -2, 12, [0, 18912, 17696, 57824, 10528, 9696, 16704, 52544, 25888, 21920, 17920, 23008, 0]],
        '褫' : [12, 13, 0, -2, 12, [0, 20448, 18560, 61408, 10880, 11232, 19072, 51936, 27136, 21952, 20800, 22112, 0]],
        '襁' : [12, 13, 0, -2, 12, [128, 23808, 17952, 63456, 7296, 5088, 12960, 24224, 51168, 25728, 17568, 23520, 0]],
        '襄' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 19008, 31680, 4352, 65504, 4352, 65504, 4672, 57728, 15456, 0]],
        '褻' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 8448, 64448, 20800, 43840, 62176, 9248, 14912, 57728, 15456, 0]],
        '褶' : [12, 13, 0, -2, 12, [0, 24288, 19104, 58976, 10912, 13152, 16896, 53216, 26656, 24544, 18464, 20448, 0]],
        '褸' : [12, 13, 0, -2, 12, [256, 16640, 20448, 59680, 16352, 10528, 20448, 51488, 28640, 21568, 18304, 22624, 0]],
        '襌' : [12, 13, 0, -2, 12, [0, 20192, 19104, 60064, 12256, 10528, 20448, 51488, 28640, 16640, 24544, 16640, 0]],
        '褝' : [12, 13, 0, -2, 12, [256, 9376, 8768, 63456, 5280, 6112, 9376, 25760, 45024, 12416, 12256, 8320, 0]],
        '襠' : [12, 13, 0, -2, 12, [256, 17728, 24544, 62624, 5280, 6016, 8192, 20448, 51488, 28640, 22816, 20448, 0]],
        '襞' : [12, 13, 0, -2, 12, [128, 61568, 38880, 62016, 36832, 61568, 38880, 36992, 65504, 4672, 57728, 15456, 0]],
        '襦' : [12, 13, 0, -2, 12, [0, 20448, 16640, 65504, 4384, 5984, 8192, 20448, 53504, 28640, 23200, 19104, 0]],
        '襤' : [12, 13, 0, -2, 12, [128, 20352, 19072, 61408, 10496, 12032, 19168, 55808, 28640, 23200, 19104, 24544, 0]],
        '襭' : [12, 13, 0, -2, 12, [2048, 19424, 23680, 60384, 10784, 16352, 16928, 58336, 24096, 21472, 21056, 24096, 0]],
        '襪' : [12, 13, 0, -2, 12, [640, 17024, 24544, 62080, 8160, 4768, 16352, 17472, 57312, 21056, 22912, 26208, 0]],
        '襯' : [12, 13, 0, -2, 12, [3072, 16864, 24352, 62944, 5408, 16352, 18720, 65504, 19520, 23232, 26944, 19040, 0]],
        '襴' : [12, 13, 0, -2, 12, [0, 24288, 21152, 65248, 4768, 16096, 20768, 57312, 29984, 22432, 21792, 22880, 0]],
        '襷' : [12, 13, 0, -2, 12, [0, 23392, 21024, 64352, 5408, 16352, 18752, 55200, 24832, 24544, 16640, 17152, 0]],
        '襾' : [12, 13, 0, -2, 12, [0, 65504, 4352, 4352, 4352, 65504, 37152, 37152, 40736, 32800, 32800, 32800, 0]],
        '覃' : [12, 13, 0, -2, 12, [0, 65504, 4352, 65504, 37152, 37152, 16256, 8320, 16256, 8320, 65504, 1024, 0]],
        '覈' : [12, 13, 0, -2, 12, [0, 65504, 4352, 65504, 37152, 32512, 18912, 64064, 9536, 31040, 35200, 13920, 0]],
        '覊' : [12, 13, 0, -2, 12, [0, 65504, 4352, 65504, 37152, 65504, 43648, 44000, 64128, 9184, 63520, 9568, 0]],
        '覓' : [12, 13, 0, -2, 12, [0, 32736, 18560, 33856, 49056, 8320, 16256, 8320, 16256, 2592, 12832, 50144, 0]],
        '覘' : [12, 13, 0, -2, 12, [0, 10208, 9248, 16352, 9248, 10208, 9248, 65504, 34944, 35200, 35488, 64736, 0]],
        '覡' : [12, 13, 0, -2, 12, [0, 65504, 9248, 10208, 42016, 45024, 62496, 42976, 8832, 8832, 64672, 2272, 0]],
        '覩' : [12, 13, 0, -2, 12, [8192, 9184, 64032, 10208, 10784, 65504, 18976, 64480, 34944, 63872, 35488, 64736, 0]],
        '覦' : [12, 13, 0, -2, 12, [0, 64480, 9760, 17376, 48672, 992, 59936, 44000, 62592, 46464, 60064, 43744, 0]],
        '覬' : [12, 13, 0, -2, 12, [8192, 44000, 64032, 992, 64032, 992, 64032, 35808, 63616, 20864, 21152, 64736, 0]],
        '覯' : [12, 13, 0, -2, 12, [20480, 21472, 64032, 21472, 65056, 9184, 64032, 44000, 63616, 43392, 64160, 36064, 0]],
        '覲' : [12, 13, 0, -2, 12, [20480, 21472, 65056, 21472, 64032, 44000, 43552, 64480, 8320, 63872, 8864, 64736, 0]],
        '覺' : [12, 13, 0, -2, 12, [2560, 25792, 19008, 30144, 19008, 65504, 37152, 40736, 4352, 7936, 4640, 58336, 0]],
        '覽' : [12, 13, 0, -2, 12, [1024, 63456, 41984, 64480, 37536, 65504, 41088, 65408, 8320, 16256, 4640, 58336, 0]],
        '覿' : [12, 13, 0, -2, 12, [8192, 9184, 65056, 9184, 65056, 44000, 65056, 19424, 30848, 18816, 31392, 52448, 0]],
        '觀' : [12, 13, 0, -2, 12, [10240, 11232, 65056, 44000, 60960, 21472, 32288, 54240, 32064, 20800, 21056, 31840, 0]],
        '觚' : [12, 13, 0, -2, 12, [0, 30688, 38208, 9536, 64832, 42304, 64832, 42304, 42304, 64800, 34720, 35872, 0]],
        '觜' : [12, 13, 0, -2, 12, [512, 8704, 48096, 41472, 41504, 65504, 17472, 32704, 17472, 32704, 32832, 32960, 0]],
        '觝' : [12, 13, 0, -2, 12, [0, 28768, 38784, 9344, 64640, 42112, 65504, 42048, 42048, 64576, 34208, 36384, 0]],
        '觧' : [12, 13, 0, -2, 12, [1088, 29248, 36992, 10208, 63616, 43136, 64480, 43136, 43136, 65504, 34944, 39040, 0]],
        '觴' : [12, 13, 0, -2, 12, [1024, 29696, 38880, 10304, 65472, 43072, 65504, 41984, 45024, 64160, 36128, 39616, 0]],
        '觸' : [12, 13, 0, -2, 12, [0, 32736, 35488, 12256, 61440, 45024, 64032, 44960, 43680, 65440, 33440, 36704, 0]],
        '訃' : [12, 13, 0, -2, 12, [0, 63744, 256, 63744, 256, 63872, 320, 288, 63744, 35072, 35072, 63744, 0]],
        '訖' : [12, 13, 0, -2, 12, [512, 61952, 992, 62464, 1024, 63456, 64, 128, 61696, 37408, 37920, 63456, 0]],
        '訐' : [12, 13, 0, -2, 12, [0, 63456, 128, 61568, 128, 61568, 2016, 128, 61568, 36992, 36992, 61568, 0]],
        '訌' : [12, 13, 0, -2, 12, [0, 63456, 128, 61568, 128, 61568, 128, 128, 61568, 36992, 36992, 63456, 0]],
        '訛' : [12, 13, 0, -2, 12, [256, 58624, 1280, 59680, 6464, 59776, 2304, 2304, 59648, 43296, 43296, 59872, 0]],
        '訝' : [12, 13, 0, -2, 12, [0, 63456, 64, 62016, 576, 63456, 64, 192, 62272, 40000, 36928, 61888, 0]],
        '訥' : [12, 13, 0, -2, 12, [1792, 57600, 256, 61408, 2336, 59680, 2336, 2720, 60512, 43040, 43040, 59488, 0]],
        '訶' : [12, 13, 0, -2, 12, [0, 61408, 64, 57408, 3648, 59968, 2624, 2624, 60992, 41024, 41024, 57792, 0]],
        '詁' : [12, 13, 0, -2, 12, [128, 61568, 128, 63456, 128, 61568, 128, 2016, 62496, 37920, 37920, 63456, 0]],
        '詛' : [12, 13, 0, -2, 12, [0, 62432, 544, 61984, 992, 61984, 544, 992, 61984, 37408, 37408, 63456, 0]],
        '詒' : [12, 13, 0, -2, 12, [256, 61696, 544, 62496, 2016, 61440, 0, 2016, 62496, 37920, 37920, 63456, 0]],
        '詆' : [12, 13, 0, -2, 12, [0, 61536, 1920, 62592, 1152, 63456, 1088, 1088, 62496, 38688, 36864, 63456, 0]],
        '詈' : [12, 13, 0, -2, 12, [0, 65504, 37152, 65504, 0, 16256, 0, 65504, 0, 16256, 8320, 16256, 0]],
        '詼' : [12, 13, 0, -2, 12, [512, 57856, 8160, 58368, 1152, 59040, 2720, 2752, 59520, 41344, 41536, 60448, 0]],
        '詭' : [12, 13, 0, -2, 12, [0, 58304, 1088, 59520, 2016, 58368, 1504, 1312, 58720, 43264, 43296, 57824, 0]],
        '詬' : [12, 13, 0, -2, 12, [0, 59360, 1024, 58368, 2016, 58368, 1024, 3040, 59936, 43552, 41504, 58336, 0]],
        '詢' : [12, 13, 0, -2, 12, [2048, 59392, 4064, 59424, 4128, 61216, 2336, 3872, 59680, 44832, 41024, 57728, 0]],
        '誅' : [12, 13, 0, -2, 12, [256, 59648, 2304, 61408, 2304, 57600, 8160, 256, 58240, 42304, 47392, 57600, 0]],
        '誂' : [12, 13, 0, -2, 12, [256, 58624, 5408, 60736, 1280, 58624, 3456, 5440, 58656, 42240, 43296, 61920, 0]],
        '誄' : [12, 13, 0, -2, 12, [256, 57824, 3840, 57600, 2016, 57600, 256, 4064, 58112, 42304, 47392, 57600, 0]],
        '誨' : [12, 13, 0, -2, 12, [1024, 58368, 2016, 59392, 4032, 58688, 1344, 8160, 59968, 43584, 45024, 57472, 0]],
        '誡' : [12, 13, 0, -2, 12, [128, 57504, 4064, 57472, 2688, 60064, 8096, 2720, 59968, 43584, 41120, 58144, 0]],
        '誑' : [12, 13, 0, -2, 12, [0, 59360, 5248, 59520, 2176, 62592, 1504, 3200, 62592, 42112, 42112, 64480, 0]],
        '誥' : [12, 13, 0, -2, 12, [128, 58496, 2016, 59520, 2176, 59360, 128, 128, 59360, 42016, 42016, 59360, 0]],
        '誦' : [12, 13, 0, -2, 12, [0, 61376, 64, 57472, 4064, 59680, 4064, 2336, 61408, 43296, 43296, 59744, 0]],
        '誚' : [12, 13, 0, -2, 12, [256, 57600, 1344, 59680, 256, 59360, 1056, 2016, 58400, 42976, 42016, 58464, 0]],
        '誣' : [12, 13, 0, -2, 12, [0, 61408, 256, 57600, 1344, 58688, 1344, 2976, 59680, 41216, 41216, 61408, 0]],
        '諄' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57344, 2016, 58400, 2016, 64, 57472, 45024, 41088, 58240, 0]],
        '諍' : [12, 13, 0, -2, 12, [0, 59360, 1344, 59680, 2048, 59360, 160, 4064, 57504, 42976, 41088, 58240, 0]],
        '諂' : [12, 13, 0, -2, 12, [0, 58336, 1056, 59456, 256, 58976, 1056, 1056, 59232, 42016, 42016, 59360, 0]],
        '諚' : [12, 13, 0, -2, 12, [256, 57600, 4064, 59424, 0, 61408, 256, 256, 59872, 43264, 44800, 61664, 0]],
        '諫' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57600, 4064, 59680, 3424, 2336, 61408, 41728, 42304, 63776, 0]],
        '諳' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57920, 576, 61408, 0, 2016, 58400, 42976, 42016, 59360, 0]],
        '諧' : [12, 13, 0, -2, 12, [128, 59520, 3808, 59520, 2176, 61152, 256, 2016, 58400, 42976, 42016, 59360, 0]],
        '諤' : [12, 13, 0, -2, 12, [0, 61152, 2720, 61152, 0, 61408, 0, 4064, 57856, 42976, 42016, 57536, 0]],
        '諱' : [12, 13, 0, -2, 12, [128, 57472, 4064, 57632, 4064, 58400, 2016, 128, 59360, 42112, 49120, 57472, 0]],
        '謔' : [12, 13, 0, -2, 12, [256, 57792, 256, 61408, 2336, 61376, 2336, 3040, 59904, 45024, 45568, 58336, 0]],
        '諠' : [12, 13, 0, -2, 12, [256, 57600, 4064, 59424, 1984, 58432, 1984, 1088, 58432, 42944, 40960, 61408, 0]],
        '諢' : [12, 13, 0, -2, 12, [0, 61408, 2336, 61408, 1312, 59360, 1312, 1312, 59360, 41216, 49120, 57600, 0]],
        '諷' : [12, 13, 0, -2, 12, [0, 61408, 2080, 61408, 2592, 61344, 2720, 4000, 61984, 41632, 44960, 57376, 0]],
        '諞' : [12, 13, 0, -2, 12, [0, 61408, 0, 61408, 2080, 61408, 2048, 8160, 60064, 45024, 43680, 60064, 0]],
        '諛' : [12, 13, 0, -2, 12, [256, 58624, 2528, 59680, 3424, 59680, 2336, 4064, 57600, 41600, 42048, 63520, 0]],
        '謌' : [12, 13, 0, -2, 12, [0, 61408, 1344, 58688, 1856, 57408, 4064, 64, 59200, 42304, 42816, 57536, 0]],
        '謇' : [12, 13, 0, -2, 12, [1024, 65504, 37152, 32704, 4352, 65504, 9344, 24384, 32800, 16256, 8320, 16256, 0]],
        '謚' : [12, 13, 0, -2, 12, [0, 58816, 1088, 59424, 2976, 57344, 1472, 1088, 59424, 45024, 43680, 65504, 0]],
        '諡' : [12, 13, 0, -2, 12, [0, 58816, 1088, 59424, 4000, 57856, 1984, 1088, 57472, 45024, 43680, 65504, 0]],
        '謖' : [12, 13, 0, -2, 12, [0, 61408, 2336, 61408, 2336, 61408, 640, 3168, 58304, 44096, 41856, 64608, 0]],
        '謐' : [12, 13, 0, -2, 12, [0, 59200, 64, 60032, 4768, 58144, 1536, 6624, 57344, 45024, 43680, 65504, 0]],
        '謗' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57920, 576, 61408, 2336, 2016, 57856, 41952, 42016, 58560, 0]],
        '謠' : [12, 13, 0, -2, 12, [0, 61920, 1568, 61600, 704, 63232, 480, 1152, 63456, 36992, 38048, 63456, 0]],
        '謳' : [12, 13, 0, -2, 12, [0, 61408, 2048, 60352, 2624, 60352, 2048, 3808, 60064, 44768, 43008, 61408, 0]],
        '鞫' : [12, 13, 0, -2, 12, [512, 20992, 64480, 21536, 30624, 8224, 64416, 43040, 64416, 8864, 64416, 8288, 0]],
        '謦' : [12, 13, 0, -2, 12, [4096, 65472, 4672, 64608, 21888, 32352, 33792, 65504, 0, 32704, 8320, 16256, 0]],
        '謫' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57920, 576, 61408, 2336, 4064, 59680, 43936, 43680, 60320, 0]],
        '謾' : [12, 13, 0, -2, 12, [0, 59328, 1088, 59328, 1088, 61408, 2720, 4064, 58432, 42048, 41856, 60512, 0]],
        '謨' : [12, 13, 0, -2, 12, [576, 57920, 4064, 57920, 2016, 58400, 2016, 1056, 61408, 41216, 41536, 60448, 0]],
        '譁' : [12, 13, 0, -2, 12, [576, 57920, 8160, 57920, 4064, 58656, 8160, 1312, 61408, 41216, 49120, 57600, 0]],
        '譌' : [12, 13, 0, -2, 12, [0, 61408, 2624, 61728, 3968, 59520, 4032, 2112, 61408, 43040, 46368, 58720, 0]],
        '譏' : [12, 13, 0, -2, 12, [256, 58688, 2464, 58688, 2720, 61408, 256, 4064, 59520, 44704, 43200, 62240, 0]],
        '譎' : [12, 13, 0, -2, 12, [0, 61376, 128, 65504, 1312, 59712, 4064, 2720, 60640, 43936, 43680, 60320, 0]],
        '證' : [12, 13, 0, -2, 12, [0, 61216, 320, 60064, 1088, 59360, 2048, 2016, 58400, 42976, 41536, 61408, 0]],
        '譖' : [12, 13, 0, -2, 12, [1152, 58496, 8160, 62592, 8160, 59712, 4704, 4032, 59456, 44992, 43072, 61376, 0]],
        '譛' : [12, 13, 0, -2, 12, [1152, 58496, 8160, 58496, 8160, 59712, 4640, 4032, 59456, 44992, 43072, 61376, 0]],
        '譚' : [12, 13, 0, -2, 12, [0, 61408, 640, 61408, 2720, 60064, 4064, 1088, 59328, 42048, 49120, 57600, 0]],
        '譫' : [12, 13, 0, -2, 12, [0, 59328, 2176, 65504, 2368, 61024, 2176, 3040, 59392, 44000, 45600, 58336, 0]],
        '譟' : [12, 13, 0, -2, 12, [0, 59328, 1088, 61408, 2720, 60064, 3808, 256, 61408, 41728, 42304, 63776, 0]],
        '譬' : [12, 13, 0, -2, 12, [128, 61568, 38880, 62016, 36832, 61568, 38880, 61568, 1024, 65504, 8320, 16256, 0]],
        '譯' : [12, 13, 0, -2, 12, [0, 61408, 2720, 60064, 4064, 57600, 4064, 1088, 65504, 41216, 45024, 57600, 0]],
        '譴' : [12, 13, 0, -2, 12, [128, 61568, 3040, 58016, 4768, 61408, 576, 7136, 59936, 44000, 44032, 62432, 0]],
        '譽' : [12, 13, 0, -2, 12, [0, 61152, 34848, 61152, 33312, 65504, 17472, 49056, 0, 65504, 8320, 16256, 0]],
        '讀' : [12, 13, 0, -2, 12, [256, 57600, 4064, 57600, 4064, 60064, 4064, 1088, 59328, 42048, 42944, 63584, 0]],
        '讌' : [12, 13, 0, -2, 12, [2304, 59648, 8160, 59648, 3904, 57440, 3904, 2368, 61280, 40960, 46400, 58656, 0]],
        '讎' : [12, 13, 0, -2, 12, [16512, 20640, 24224, 61920, 44864, 61920, 44864, 41280, 65504, 43328, 43328, 65504, 0]],
        '讒' : [12, 13, 0, -2, 12, [0, 61376, 4224, 61408, 2176, 61152, 2176, 8160, 59680, 45024, 41600, 60640, 0]],
        '讓' : [12, 13, 0, -2, 12, [256, 57600, 8160, 60064, 3808, 57984, 8160, 640, 61408, 41632, 48192, 59168, 0]],
        '讖' : [12, 13, 0, -2, 12, [2688, 60064, 5536, 59520, 8160, 60032, 7072, 2720, 64320, 43584, 43680, 65312, 0]],
        '讙' : [12, 13, 0, -2, 12, [1152, 58496, 8160, 60064, 3808, 58496, 4064, 6400, 61408, 43264, 43264, 61408, 0]],
        '讚' : [12, 13, 0, -2, 12, [576, 59968, 4064, 57920, 4064, 58688, 4064, 1088, 59328, 42048, 42944, 63584, 0]],
        '谺' : [12, 13, 0, -2, 12, [0, 21472, 35136, 320, 12608, 19424, 33856, 192, 63808, 36416, 34880, 63680, 0]],
        '豁' : [12, 13, 0, -2, 12, [8192, 8512, 64032, 34816, 8192, 63936, 8736, 64512, 9184, 64032, 35360, 64480, 0]],
        '谿' : [12, 13, 0, -2, 12, [0, 64832, 43552, 21504, 40960, 18880, 64032, 9216, 64480, 8736, 21024, 35808, 0]],
        '豈' : [12, 13, 0, -2, 12, [1024, 17472, 32704, 0, 65504, 0, 16256, 8320, 16256, 4352, 4352, 65504, 0]],
        '豌' : [12, 13, 0, -2, 12, [256, 61696, 8160, 61472, 40672, 37536, 37536, 64160, 18144, 21120, 21664, 63712, 0]],
        '豎' : [12, 13, 0, -2, 12, [0, 64480, 40992, 63776, 35136, 63872, 41536, 65504, 8320, 16256, 4352, 65504, 0]],
        '豐' : [12, 13, 0, -2, 12, [4352, 21824, 32704, 21824, 32704, 21824, 65504, 8320, 16256, 4352, 4352, 65504, 0]],
        '豕' : [12, 13, 0, -2, 12, [0, 65504, 1024, 3072, 29216, 1600, 6784, 58112, 1664, 6720, 57888, 3072, 0]],
        '豢' : [12, 13, 0, -2, 12, [1024, 9344, 17472, 65504, 4352, 65504, 17472, 39584, 26368, 6784, 57952, 3072, 0]],
        '豬' : [12, 13, 0, -2, 12, [256, 63744, 10208, 24864, 4416, 16352, 53824, 6112, 14880, 54240, 4640, 25568, 0]],
        '豸' : [12, 13, 0, -2, 12, [0, 32640, 9216, 4832, 3840, 61696, 1792, 30848, 896, 3200, 61568, 768, 0]],
        '豺' : [12, 13, 0, -2, 12, [32, 14368, 53280, 22496, 14368, 53280, 14432, 51360, 14624, 52768, 2080, 12512, 0]],
        '貂' : [12, 13, 0, -2, 12, [0, 16352, 53536, 22816, 12832, 54464, 12288, 55264, 13344, 54304, 5152, 26592, 0]],
        '貉' : [12, 13, 0, -2, 12, [512, 14848, 54240, 21536, 14912, 53632, 12864, 54304, 16352, 54304, 5152, 26592, 0]],
        '貅' : [12, 13, 0, -2, 12, [128, 14976, 53888, 21632, 14304, 56448, 13440, 54656, 14016, 54432, 5248, 25728, 0]],
        '貊' : [12, 13, 0, -2, 12, [0, 16352, 53376, 22784, 14304, 54304, 13344, 55264, 13344, 54304, 5152, 26592, 0]],
        '貍' : [12, 13, 0, -2, 12, [0, 16352, 54432, 22496, 15520, 54432, 14304, 53376, 14304, 53376, 4224, 26592, 0]],
        '貎' : [12, 13, 0, -2, 12, [256, 15968, 54304, 21536, 16224, 54304, 13344, 55264, 12928, 53888, 5280, 25824, 0]],
        '貔' : [12, 13, 0, -2, 12, [128, 14592, 55264, 21664, 15648, 54944, 13344, 55264, 13440, 55008, 5248, 28512, 0]],
        '豼' : [12, 13, 0, -2, 12, [128, 15488, 54400, 21664, 16320, 54400, 13440, 54400, 13440, 54432, 6048, 27872, 0]],
        '貘' : [12, 13, 0, -2, 12, [576, 12864, 57312, 21056, 32736, 54304, 14304, 54304, 16352, 53504, 4672, 27680, 0]],
        '戝' : [12, 13, 0, -2, 12, [256, 63776, 35104, 63744, 36832, 63744, 35136, 35136, 63616, 128, 21312, 35872, 0]],
        '貭' : [12, 13, 0, -2, 12, [0, 32736, 16640, 32736, 16640, 24544, 20512, 24544, 20512, 40928, 34880, 12320, 0]],
        '貪' : [12, 13, 0, -2, 12, [0, 3584, 12672, 57184, 256, 512, 16256, 8320, 16256, 8320, 16256, 49248, 0]],
        '貽' : [12, 13, 0, -2, 12, [256, 61696, 37408, 61984, 37856, 61440, 36864, 37856, 61984, 544, 21024, 35808, 0]],
        '貲' : [12, 13, 0, -2, 12, [512, 8704, 48096, 41472, 49120, 57472, 16256, 8320, 16256, 8320, 16256, 49248, 0]],
        '貳' : [12, 13, 0, -2, 12, [128, 160, 65504, 128, 32384, 128, 65408, 16960, 32320, 16960, 32288, 49952, 0]],
        '貮' : [12, 13, 0, -2, 12, [128, 32416, 160, 65504, 128, 32384, 17024, 32320, 16960, 32320, 9248, 49952, 0]],
        '貶' : [12, 13, 0, -2, 12, [0, 61664, 40704, 61696, 37120, 63456, 36896, 36928, 62592, 1280, 18944, 37344, 0]],
        '賈' : [12, 13, 0, -2, 12, [0, 65504, 2560, 32704, 19008, 19008, 32704, 8320, 16256, 8320, 16256, 49248, 0]],
        '賁' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 4352, 65504, 4352, 16256, 8320, 16256, 8320, 16256, 49248, 0]],
        '賤' : [12, 13, 0, -2, 12, [512, 61984, 40928, 62016, 37280, 65120, 38016, 40928, 62528, 640, 21280, 36064, 0]],
        '賣' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 1024, 32704, 19008, 32704, 8320, 16256, 8320, 16256, 49248, 0]],
        '賚' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 9344, 28352, 38176, 16256, 57568, 16256, 8320, 16256, 49248, 0]],
        '賽' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 32704, 4352, 65504, 8320, 32704, 41120, 16256, 49248, 0]],
        '賺' : [12, 13, 0, -2, 12, [576, 59008, 48096, 57984, 45024, 58016, 49120, 41632, 61408, 640, 22208, 35488, 0]],
        '賻' : [12, 13, 0, -2, 12, [256, 61728, 40928, 61696, 38880, 62752, 38880, 38176, 61504, 4064, 20544, 35264, 0]],
        '贄' : [12, 13, 0, -2, 12, [8448, 8448, 65472, 18752, 64320, 8864, 65440, 8320, 16256, 8320, 16256, 49248, 0]],
        '贅' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 9280, 64832, 18560, 49120, 8320, 16256, 8320, 16256, 49248, 0]],
        '贊' : [12, 13, 0, -2, 12, [8320, 41600, 64480, 8320, 65504, 20800, 49056, 8320, 16256, 8320, 16256, 49248, 0]],
        '贇' : [12, 13, 0, -2, 12, [8320, 9376, 65504, 37184, 21952, 9504, 32736, 41088, 16256, 8320, 16256, 49248, 0]],
        '贏' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 16384, 16320, 8448, 32704, 18752, 32576, 18880, 32576, 51552, 0]],
        '贍' : [12, 13, 0, -2, 12, [0, 59328, 43136, 65504, 43328, 61024, 43136, 45024, 59392, 4064, 21536, 38880, 0]],
        '贐' : [12, 13, 0, -2, 12, [256, 57600, 45024, 57632, 49120, 57632, 45024, 43584, 61728, 4064, 19104, 40928, 0]],
        '齎' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2560, 62944, 21824, 49056, 10560, 12096, 10560, 20288, 20672, 0]],
        '贓' : [12, 13, 0, -2, 12, [128, 57504, 45024, 59456, 49088, 59968, 44896, 47424, 61248, 2720, 24480, 37152, 0]],
        '賍' : [12, 13, 0, -2, 12, [256, 61696, 38880, 62464, 38016, 62592, 38880, 38016, 62592, 128, 20608, 36832, 0]],
        '贔' : [12, 13, 0, -2, 12, [0, 32704, 16448, 32704, 16448, 32704, 8320, 51808, 31680, 19008, 31680, 33824, 0]],
        '贖' : [12, 13, 0, -2, 12, [256, 57600, 45024, 57600, 45024, 60064, 45024, 42048, 59328, 1088, 22464, 34912, 0]],
        '赧' : [12, 13, 0, -2, 12, [8192, 9184, 64032, 9184, 8704, 65504, 21024, 21152, 55968, 22080, 37056, 45856, 0]],
        '赭' : [12, 13, 0, -2, 12, [8448, 8448, 63456, 8480, 8512, 65504, 20800, 21472, 54816, 23520, 37408, 46048, 0]],
        '赱' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 1024, 65504, 4096, 2176, 8448, 9728, 22528, 34784, 0]],
        '赳' : [12, 13, 0, -2, 12, [8192, 8256, 64064, 8768, 8768, 65088, 8800, 15296, 42560, 41024, 61440, 36832, 0]],
        '趁' : [12, 13, 0, -2, 12, [8192, 8640, 64032, 9216, 8384, 64256, 8224, 14528, 41728, 40960, 61440, 36832, 0]],
        '趙' : [12, 13, 0, -2, 12, [128, 8320, 64192, 9376, 8320, 64480, 8736, 15328, 41504, 41568, 61440, 36832, 0]],
        '跂' : [12, 13, 0, -2, 12, [256, 61696, 40928, 37120, 61696, 10176, 8256, 48192, 42048, 41600, 41856, 64608, 0]],
        '趾' : [12, 13, 0, -2, 12, [128, 63616, 34944, 34944, 64128, 8928, 8832, 47744, 41600, 41600, 41600, 65504, 0]],
        '趺' : [12, 13, 0, -2, 12, [256, 61696, 37120, 40928, 61696, 8448, 12256, 45312, 41216, 41600, 42048, 63520, 0]],
        '跏' : [12, 13, 0, -2, 12, [512, 62176, 37536, 40864, 62112, 8864, 9376, 46240, 43168, 43168, 41376, 63200, 0]],
        '跚' : [12, 13, 0, -2, 12, [0, 64480, 35488, 35488, 64160, 8864, 10208, 47776, 41632, 41632, 41632, 64160, 0]],
        '跖' : [12, 13, 0, -2, 12, [0, 65504, 34944, 34944, 63616, 8448, 9184, 42528, 47648, 41504, 41504, 64480, 0]],
        '跌' : [12, 13, 0, -2, 12, [128, 62592, 38016, 38880, 62592, 8320, 10208, 47232, 41088, 41280, 41504, 64544, 0]],
        '跛' : [12, 13, 0, -2, 12, [128, 61568, 38880, 38048, 62592, 10208, 9248, 46368, 43328, 43200, 41248, 63008, 0]],
        '跋' : [12, 13, 0, -2, 12, [256, 63776, 36832, 35072, 63776, 8864, 8864, 47776, 42048, 42048, 41120, 64288, 0]],
        '跪' : [12, 13, 0, -2, 12, [0, 62400, 37952, 39040, 63456, 9216, 9696, 46368, 42336, 42240, 43296, 63968, 0]],
        '跫' : [12, 13, 0, -2, 12, [0, 64448, 8768, 15168, 50272, 16256, 8320, 16256, 1024, 10176, 15360, 50144, 0]],
        '跟' : [12, 13, 0, -2, 12, [0, 63456, 37920, 38880, 62496, 9248, 10208, 46336, 42272, 42144, 42048, 65312, 0]],
        '跣' : [12, 13, 0, -2, 12, [128, 62592, 38016, 38880, 62592, 8320, 10208, 47744, 41600, 41600, 42144, 63712, 0]],
        '跼' : [12, 13, 0, -2, 12, [0, 63456, 37920, 38880, 62464, 10208, 9248, 47008, 42144, 42912, 42016, 63680, 0]],
        '踈' : [12, 13, 0, -2, 12, [128, 61568, 40928, 36992, 63456, 9376, 9376, 47072, 41344, 41664, 42144, 63616, 0]],
        '踉' : [12, 13, 0, -2, 12, [256, 61696, 38880, 37920, 63456, 9248, 10208, 46336, 42272, 42144, 42048, 65312, 0]],
        '跿' : [12, 13, 0, -2, 12, [256, 63744, 36832, 35072, 63744, 10208, 8448, 48384, 42464, 42240, 42752, 63712, 0]],
        '踝' : [12, 13, 0, -2, 12, [0, 63456, 38176, 38880, 62752, 9504, 10208, 45312, 45024, 41728, 42304, 63776, 0]],
        '踞' : [12, 13, 0, -2, 12, [0, 63456, 37920, 38880, 62592, 10208, 9344, 46208, 44000, 43552, 41504, 64480, 0]],
        '踐' : [12, 13, 0, -2, 12, [512, 61984, 40928, 37440, 61856, 11872, 9344, 49120, 42048, 41600, 41760, 64736, 0]],
        '踟' : [12, 13, 0, -2, 12, [1024, 62464, 38880, 39584, 62112, 8864, 12192, 41632, 47776, 41632, 42464, 63616, 0]],
        '蹂' : [12, 13, 0, -2, 12, [0, 63424, 36992, 37120, 65504, 8992, 11584, 41216, 49120, 41728, 42304, 63776, 0]],
        '踵' : [12, 13, 0, -2, 12, [0, 63456, 37120, 40928, 62752, 10208, 9504, 47072, 41216, 42976, 41216, 65504, 0]],
        '踰' : [12, 13, 0, -2, 12, [0, 63360, 37440, 37920, 64480, 8192, 11936, 47776, 44864, 43840, 44704, 64160, 0]],
        '踴' : [12, 13, 0, -2, 12, [0, 63424, 36992, 38880, 62752, 10208, 9504, 45312, 45024, 41248, 41504, 64704, 0]],
        '蹊' : [12, 13, 0, -2, 12, [0, 63456, 38208, 39200, 62080, 9504, 12256, 45312, 45024, 41600, 42048, 63520, 0]],
        '蹇' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 32704, 4352, 65504, 20800, 40736, 9664, 15360, 50144, 0]],
        '蹉' : [12, 13, 0, -2, 12, [1088, 62592, 40928, 37120, 63456, 8448, 12256, 45568, 42976, 43136, 41088, 63456, 0]],
        '蹌' : [12, 13, 0, -2, 12, [0, 62400, 37920, 40928, 62496, 10208, 9248, 47072, 43008, 44000, 41504, 64480, 0]],
        '蹐' : [12, 13, 0, -2, 12, [0, 63136, 36992, 38560, 61760, 8736, 12256, 42016, 47072, 42016, 42976, 62496, 0]],
        '蹈' : [12, 13, 0, -2, 12, [0, 63456, 38208, 39200, 61440, 8448, 9824, 46112, 42848, 42016, 42016, 63456, 0]],
        '蹙' : [12, 13, 0, -2, 12, [256, 288, 32736, 18720, 32064, 19072, 32672, 37088, 40832, 9408, 15360, 50144, 0]],
        '蹤' : [12, 13, 0, -2, 12, [1088, 62528, 37952, 39584, 61728, 9344, 10368, 47840, 43648, 43648, 43904, 64608, 0]],
        '蹠' : [12, 13, 0, -2, 12, [128, 61568, 38880, 38208, 62784, 10208, 9536, 46400, 43456, 43008, 41632, 62624, 0]],
        '踪' : [12, 13, 0, -2, 12, [256, 61696, 40928, 36896, 63424, 8192, 8192, 49120, 41216, 42304, 43296, 61696, 0]],
        '蹣' : [12, 13, 0, -2, 12, [1088, 62528, 40928, 37952, 63424, 8448, 12256, 47392, 44448, 43872, 44448, 63840, 0]],
        '蹕' : [12, 13, 0, -2, 12, [0, 63456, 38048, 38880, 62624, 12256, 8864, 49120, 41632, 41632, 45024, 61568, 0]],
        '蹶' : [12, 13, 0, -2, 12, [0, 65504, 34944, 36320, 64160, 12064, 10816, 43584, 46912, 41632, 42144, 62752, 0]],
        '蹲' : [12, 13, 0, -2, 12, [0, 62912, 38944, 38880, 62080, 12256, 10912, 44256, 43040, 49120, 42048, 62144, 0]],
        '蹼' : [12, 13, 0, -2, 12, [640, 62112, 40928, 37440, 62016, 12256, 8448, 47072, 41216, 45024, 41536, 64544, 0]],
        '躁' : [12, 13, 0, -2, 12, [0, 63424, 37952, 40928, 64160, 10912, 12000, 45312, 45024, 41728, 42304, 63776, 0]],
        '躇' : [12, 13, 0, -2, 12, [576, 62016, 40928, 37152, 63456, 8512, 10208, 45600, 42976, 43552, 41504, 64480, 0]],
        '躅' : [12, 13, 0, -2, 12, [0, 65504, 37536, 40928, 61952, 12256, 8736, 44960, 47776, 44960, 41632, 65376, 0]],
        '躄' : [12, 13, 0, -2, 12, [128, 61568, 38880, 62016, 36832, 61568, 40928, 61568, 8064, 9408, 15360, 50144, 0]],
        '躋' : [12, 13, 0, -2, 12, [512, 61952, 40928, 38208, 63296, 11680, 10304, 44992, 47168, 44992, 43072, 61504, 0]],
        '躊' : [12, 13, 0, -2, 12, [256, 61696, 38880, 37120, 65504, 8224, 10176, 45312, 45024, 43552, 43680, 65120, 0]],
        '躓' : [12, 13, 0, -2, 12, [0, 65248, 34944, 36576, 64160, 10912, 10176, 46144, 42944, 42048, 42944, 63584, 0]],
        '躑' : [12, 13, 0, -2, 12, [0, 64224, 37280, 40864, 62624, 11200, 8352, 49056, 42144, 49056, 43712, 61824, 0]],
        '躔' : [12, 13, 0, -2, 12, [128, 63616, 36832, 35488, 64480, 10912, 12256, 47424, 44704, 44000, 43136, 63456, 0]],
        '躙' : [12, 13, 0, -2, 12, [0, 65248, 35488, 36576, 64160, 12000, 10528, 48096, 44704, 44000, 43680, 64480, 0]],
        '躪' : [12, 13, 0, -2, 12, [576, 64064, 36832, 35488, 65248, 10912, 12000, 47392, 44000, 44704, 43680, 64480, 0]],
        '躡' : [12, 13, 0, -2, 12, [0, 65504, 37952, 38848, 62528, 12256, 10912, 44768, 47776, 44768, 43936, 61984, 0]],
        '躬' : [12, 13, 0, -2, 12, [8192, 17376, 63520, 34848, 64480, 35328, 64000, 35808, 64544, 10272, 18464, 39360, 0]],
        '躰' : [12, 13, 0, -2, 12, [8448, 16640, 61696, 40928, 61696, 37760, 62784, 39200, 61696, 14304, 20736, 45312, 0]],
        '軆' : [12, 13, 0, -2, 12, [8832, 17024, 65504, 37536, 65504, 37536, 65504, 37920, 63456, 12864, 21056, 49120, 0]],
        '躱' : [12, 13, 0, -2, 12, [8192, 18368, 63808, 35168, 64032, 35936, 63616, 36832, 63872, 10944, 19616, 39040, 0]],
        '躾' : [12, 13, 0, -2, 12, [9280, 17536, 65504, 36992, 63456, 36992, 65504, 36992, 63456, 12416, 20800, 46624, 0]],
        '軅' : [12, 13, 0, -2, 12, [8192, 24544, 62080, 37536, 62944, 38720, 64992, 38208, 62944, 13632, 21824, 46560, 0]],
        '軈' : [12, 13, 0, -2, 12, [8448, 16640, 65504, 38208, 62944, 40768, 62784, 38368, 61440, 13632, 21536, 45952, 0]],
        '軋' : [12, 13, 0, -2, 12, [4096, 4224, 65152, 4224, 65152, 37504, 65152, 37504, 65152, 4256, 65184, 4320, 0]],
        '軛' : [12, 13, 0, -2, 12, [0, 10208, 64512, 9216, 64992, 42272, 64800, 42336, 64768, 9472, 63776, 10720, 0]],
        '軣' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 9344, 16256, 9344, 65504, 17472, 9344, 0, 12672, 49248, 0]],
        '軼' : [12, 13, 0, -2, 12, [128, 8832, 64128, 9184, 64128, 43136, 65504, 43136, 63616, 8512, 64032, 9248, 0]],
        '軻' : [12, 13, 0, -2, 12, [0, 10208, 63520, 8224, 64416, 43680, 64160, 43680, 64416, 8224, 63520, 8416, 0]],
        '軫' : [12, 13, 0, -2, 12, [0, 9088, 64576, 8224, 63680, 43776, 63488, 43200, 64256, 8224, 63680, 9984, 0]],
        '軾' : [12, 13, 0, -2, 12, [128, 8352, 65504, 8320, 63616, 44928, 64128, 43584, 64064, 8768, 65440, 8224, 0]],
        '輊' : [12, 13, 0, -2, 12, [0, 10208, 63616, 8448, 64032, 45024, 63616, 43136, 64480, 8320, 63616, 10208, 0]],
        '輅' : [12, 13, 0, -2, 12, [256, 8448, 63968, 8736, 64832, 43136, 63808, 43552, 65504, 8736, 64032, 9184, 0]],
        '輕' : [12, 13, 0, -2, 12, [0, 10208, 64160, 8864, 64832, 43680, 64160, 43008, 64480, 8320, 63616, 10208, 0]],
        '輒' : [12, 13, 0, -2, 12, [0, 12256, 64128, 10880, 65152, 43648, 64128, 44672, 64128, 10912, 65184, 8928, 0]],
        '輙' : [12, 13, 0, -2, 12, [0, 12256, 64000, 11232, 65056, 43680, 64160, 44704, 64064, 10816, 65184, 8992, 0]],
        '輓' : [12, 13, 0, -2, 12, [0, 9152, 64576, 10368, 65504, 43296, 63776, 45024, 64128, 8832, 64672, 9440, 0]],
        '輜' : [12, 13, 0, -2, 12, [0, 8864, 64160, 9536, 64160, 43680, 63488, 45024, 63776, 12256, 63776, 12256, 0]],
        '輟' : [12, 13, 0, -2, 12, [0, 12000, 61984, 10912, 62528, 43680, 61440, 44768, 61984, 10912, 62528, 10912, 0]],
        '輛' : [12, 13, 0, -2, 12, [0, 12256, 63744, 8448, 65504, 43296, 64928, 44448, 64352, 10528, 63776, 10592, 0]],
        '輌' : [12, 13, 0, -2, 12, [0, 12256, 64000, 8704, 65504, 43552, 64160, 43680, 64160, 12192, 63520, 10336, 0]],
        '輦' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8320, 65504, 20800, 65504, 9344, 16256, 9344, 65504, 1024, 0]],
        '輳' : [12, 13, 0, -2, 12, [128, 8320, 65504, 8448, 65504, 43584, 65504, 42112, 64480, 8320, 63808, 9760, 0]],
        '輻' : [12, 13, 0, -2, 12, [0, 10208, 63488, 9152, 64064, 43968, 63488, 45024, 63776, 12256, 63776, 12256, 0]],
        '輹' : [12, 13, 0, -2, 12, [256, 8448, 64480, 9760, 64480, 43552, 64480, 43520, 65504, 8736, 63936, 9760, 0]],
        '轅' : [12, 13, 0, -2, 12, [128, 8320, 64480, 8320, 65504, 43552, 64032, 44000, 63808, 9792, 64032, 9120, 0]],
        '轂' : [12, 13, 0, -2, 12, [0, 9152, 64064, 8768, 65088, 9312, 63488, 45024, 64032, 8768, 63872, 9824, 0]],
        '輾' : [12, 13, 0, -2, 12, [0, 10208, 64544, 10208, 64832, 42976, 64832, 42304, 65504, 8864, 64064, 10016, 0]],
        '轌' : [12, 13, 0, -2, 12, [0, 12256, 61696, 12256, 63776, 43872, 63488, 44000, 63520, 10208, 63520, 9184, 0]],
        '轉' : [12, 13, 0, -2, 12, [256, 8448, 65504, 10528, 65504, 43296, 65504, 43328, 65504, 8256, 62528, 8896, 0]],
        '轆' : [12, 13, 0, -2, 12, [256, 8448, 65504, 10880, 65504, 43680, 65504, 42112, 63200, 9344, 62880, 11872, 0]],
        '轎' : [12, 13, 0, -2, 12, [0, 12256, 63744, 10208, 64128, 44736, 64416, 43008, 65504, 10912, 64416, 10336, 0]],
        '轗' : [12, 13, 0, -2, 12, [128, 8352, 65504, 9344, 65440, 42304, 65344, 43168, 63744, 9280, 64544, 9088, 0]],
        '轜' : [12, 13, 0, -2, 12, [0, 12256, 61696, 12256, 63776, 43872, 63488, 45024, 61952, 12256, 64160, 10912, 0]],
        '轢' : [12, 13, 0, -2, 12, [256, 9536, 64416, 9920, 64416, 44736, 64416, 43264, 65504, 8960, 62784, 10528, 0]],
        '轣' : [12, 13, 0, -2, 12, [0, 12256, 64064, 12256, 64064, 44736, 64352, 43584, 63616, 10976, 64128, 10208, 0]],
        '轤' : [12, 13, 0, -2, 12, [960, 8704, 65504, 10528, 64448, 43264, 65504, 42304, 62784, 12256, 64160, 12256, 0]],
        '辜' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 8320, 16256, 1024, 65504, 4352, 65504, 1024, 32704, 1024, 0]],
        '辟' : [12, 13, 0, -2, 12, [256, 61696, 40928, 37440, 62016, 33408, 36832, 61696, 37120, 40928, 37120, 61696, 0]],
        '辣' : [12, 13, 0, -2, 12, [128, 8320, 65504, 20608, 22496, 21664, 64672, 10208, 14720, 58048, 19616, 16512, 0]],
        '辭' : [12, 13, 0, -2, 12, [128, 64640, 44000, 38176, 31008, 4416, 65504, 42112, 46208, 53216, 46208, 52352, 0]],
        '辯' : [12, 13, 0, -2, 12, [8320, 11904, 61920, 24384, 20800, 24384, 61920, 8320, 11904, 64480, 19072, 20096, 0]],
        '辷' : [12, 13, 0, -2, 12, [0, 16384, 8192, 4064, 32768, 16384, 0, 57344, 8192, 8192, 20480, 36832, 0]],
        '迚' : [12, 13, 0, -2, 12, [256, 16640, 12256, 2336, 35104, 18720, 4064, 57600, 8448, 8448, 20480, 36832, 0]],
        '迥' : [12, 13, 0, -2, 12, [0, 20448, 10272, 2080, 35744, 19104, 2720, 60320, 10336, 8192, 20480, 36832, 0]],
        '迢' : [12, 13, 0, -2, 12, [0, 20448, 8480, 544, 33984, 16384, 2016, 58400, 9248, 10208, 20480, 36832, 0]],
        '迪' : [12, 13, 0, -2, 12, [256, 16640, 12256, 2336, 35104, 20448, 2336, 59680, 12256, 8192, 20480, 36832, 0]],
        '迯' : [12, 13, 0, -2, 12, [128, 18560, 11904, 2688, 35520, 22176, 640, 57984, 9344, 10368, 20480, 36832, 0]],
        '邇' : [12, 13, 0, -2, 12, [0, 20448, 9536, 2336, 40928, 19296, 3488, 60256, 11680, 11104, 20480, 36832, 0]],
        '迴' : [12, 13, 0, -2, 12, [0, 20448, 10272, 2976, 35488, 19104, 2976, 59424, 12256, 8192, 20480, 36832, 0]],
        '逅' : [12, 13, 0, -2, 12, [0, 16576, 12032, 2048, 36832, 18432, 4032, 58432, 9280, 10176, 20480, 36832, 0]],
        '迹' : [12, 13, 0, -2, 12, [256, 16640, 12256, 640, 33408, 19136, 4768, 57984, 9344, 9600, 20480, 36832, 0]],
        '迺' : [12, 13, 0, -2, 12, [0, 20448, 8832, 640, 36832, 19104, 2720, 60640, 10272, 12256, 20480, 36832, 0]],
        '逑' : [12, 13, 0, -2, 12, [256, 16672, 12256, 256, 35104, 17728, 256, 58688, 10528, 8960, 20480, 36832, 0]],
        '逕' : [12, 13, 0, -2, 12, [0, 20448, 8864, 1344, 33440, 16384, 2016, 57472, 8320, 10208, 20480, 36832, 0]],
        '逡' : [12, 13, 0, -2, 12, [512, 16960, 9280, 4064, 33408, 19552, 960, 60480, 8576, 11872, 20480, 36832, 0]],
        '逍' : [12, 13, 0, -2, 12, [256, 16640, 9536, 2336, 34752, 17472, 1984, 58432, 10176, 9280, 20480, 36832, 0]],
        '逞' : [12, 13, 0, -2, 12, [0, 18368, 9280, 1088, 36832, 16640, 1984, 57600, 8448, 12256, 20480, 36832, 0]],
        '逖' : [12, 13, 0, -2, 12, [128, 19072, 9344, 1184, 35488, 21120, 1728, 60224, 8736, 11296, 20480, 36832, 0]],
        '逋' : [12, 13, 0, -2, 12, [256, 16672, 12256, 256, 36832, 18720, 4064, 59680, 12256, 10528, 20480, 36832, 0]],
        '逧' : [12, 13, 0, -2, 12, [0, 17472, 10272, 896, 33856, 18464, 1984, 58432, 9280, 10176, 20480, 36832, 0]],
        '逶' : [12, 13, 0, -2, 12, [0, 20448, 8448, 4064, 34112, 18720, 4064, 58432, 9088, 11360, 20480, 36832, 0]],
        '逵' : [12, 13, 0, -2, 12, [256, 16640, 12256, 256, 36832, 17024, 672, 60896, 8448, 12256, 20480, 36832, 0]],
        '逹' : [12, 13, 0, -2, 12, [256, 16640, 12256, 256, 36832, 16960, 4064, 57600, 12256, 8448, 20480, 36832, 0]],
        '迸' : [12, 13, 0, -2, 12, [1088, 16960, 8320, 4064, 33344, 16960, 4064, 57920, 9280, 9280, 20480, 36832, 0]],
        '遏' : [12, 13, 0, -2, 12, [0, 18368, 9280, 1984, 33856, 18400, 6688, 59680, 12064, 8384, 20480, 36832, 0]],
        '遐' : [12, 13, 0, -2, 12, [0, 20192, 10784, 2784, 36352, 18912, 3744, 59552, 11840, 10656, 20480, 36832, 0]],
        '遑' : [12, 13, 0, -2, 12, [128, 16640, 10208, 1056, 34784, 17440, 4064, 57472, 10208, 8320, 20608, 36832, 0]],
        '遒' : [12, 13, 0, -2, 12, [0, 17856, 10272, 4064, 33408, 20448, 2720, 60640, 10272, 12256, 20480, 36832, 0]],
        '逎' : [12, 13, 0, -2, 12, [0, 20448, 8832, 4064, 35488, 19680, 2080, 61408, 10272, 12256, 20480, 36832, 0]],
        '遉' : [12, 13, 0, -2, 12, [256, 16832, 8448, 2016, 33824, 18400, 1056, 59360, 8768, 11296, 20480, 36832, 0]],
        '逾' : [12, 13, 0, -2, 12, [0, 20352, 9280, 2080, 40928, 19104, 3744, 60224, 11936, 10912, 20480, 36832, 0]],
        '遖' : [12, 13, 0, -2, 12, [256, 16640, 12256, 256, 36832, 19104, 4064, 59680, 12256, 10528, 20480, 36832, 0]],
        '遘' : [12, 13, 0, -2, 12, [576, 16960, 12256, 576, 40928, 18720, 4064, 59680, 12256, 10272, 20480, 36832, 0]],
        '遞' : [12, 13, 0, -2, 12, [0, 24544, 12544, 6112, 38176, 21952, 5248, 60032, 8864, 11488, 20480, 36832, 0]],
        '遨' : [12, 13, 0, -2, 12, [1152, 17536, 16096, 1312, 40608, 17568, 8000, 59968, 10912, 14112, 20480, 36832, 0]],
        '遯' : [12, 13, 0, -2, 12, [0, 40928, 21760, 5280, 40384, 22144, 5312, 56736, 22176, 21888, 40960, 40928, 0]],
        '遶' : [12, 13, 0, -2, 12, [256, 16640, 12256, 1088, 40672, 17472, 4064, 57984, 8864, 11488, 20480, 36832, 0]],
        '隨' : [12, 13, 0, -2, 12, [256, 61696, 45024, 41472, 55264, 43136, 42976, 47648, 44000, 51744, 37888, 41952, 0]],
        '遲' : [12, 13, 0, -2, 12, [0, 24544, 12320, 8160, 37120, 23904, 4352, 59328, 9472, 12256, 20736, 36832, 0]],
        '邂' : [12, 13, 0, -2, 12, [0, 36832, 21152, 16032, 38208, 24384, 5600, 56896, 21472, 22080, 40960, 40928, 0]],
        '遽' : [12, 13, 0, -2, 12, [448, 16640, 12256, 2336, 36800, 18688, 5088, 60576, 8896, 11680, 20480, 36832, 0]],
        '邁' : [12, 13, 0, -2, 12, [576, 16960, 12256, 1344, 34752, 17728, 4064, 59680, 11168, 10336, 20480, 36832, 0]],
        '邀' : [12, 13, 0, -2, 12, [1152, 18560, 16096, 4896, 40608, 18592, 8000, 59968, 10912, 14112, 20480, 36832, 0]],
        '邊' : [12, 13, 0, -2, 12, [256, 16896, 12224, 2112, 36800, 18496, 8160, 57888, 12256, 8768, 21632, 36832, 0]],
        '邉' : [12, 13, 0, -2, 12, [256, 16896, 12224, 2112, 36800, 18496, 8160, 58528, 10304, 14240, 21632, 36832, 0]],
        '邏' : [12, 13, 0, -2, 12, [0, 40928, 21664, 8160, 35136, 21984, 2880, 54752, 23872, 18912, 40960, 40928, 0]],
        '邨' : [12, 13, 0, -2, 12, [8192, 8672, 11552, 61728, 8512, 42368, 42304, 64800, 8480, 8480, 11712, 28928, 0]],
        '邯' : [12, 13, 0, -2, 12, [2048, 35296, 35104, 65312, 35136, 35200, 35136, 63776, 35104, 35104, 35264, 63744, 0]],
        '邱' : [12, 13, 0, -2, 12, [0, 3552, 61728, 33056, 33088, 64896, 35136, 35104, 35104, 35104, 40384, 57600, 0]],
        '邵' : [12, 13, 0, -2, 12, [0, 64992, 9504, 9504, 17728, 39296, 320, 64800, 34080, 34080, 34240, 64768, 0]],
        '郢' : [12, 13, 0, -2, 12, [0, 63968, 35104, 35104, 63808, 384, 64832, 8480, 63776, 8480, 11712, 61696, 0]],
        '郤' : [12, 13, 0, -2, 12, [0, 18912, 34080, 288, 12608, 18816, 34112, 288, 64800, 34080, 34240, 64768, 0]],
        '扈' : [12, 13, 0, -2, 12, [0, 65504, 0, 32736, 16416, 32736, 18560, 32736, 41504, 49120, 40960, 16352, 0]],
        '郛' : [12, 13, 0, -2, 12, [0, 32224, 16672, 38176, 38208, 384, 32064, 2336, 4384, 65312, 4544, 12544, 0]],
        '鄂' : [12, 13, 0, -2, 12, [0, 64992, 42272, 64800, 320, 64896, 320, 64800, 16672, 64800, 34240, 14592, 0]],
        '鄒' : [12, 13, 0, -2, 12, [16384, 16864, 32032, 42272, 64832, 9600, 18752, 32032, 42272, 64800, 17856, 39168, 0]],
        '鄙' : [12, 13, 0, -2, 12, [0, 64992, 34080, 64800, 4416, 65408, 320, 64800, 42272, 48416, 34240, 64768, 0]],
        '鄲' : [12, 13, 0, -2, 12, [0, 61408, 43296, 43296, 65344, 38272, 64832, 38176, 64800, 4384, 65472, 4352, 0]],
        '鄰' : [12, 13, 0, -2, 12, [4096, 21984, 22816, 65312, 21824, 37760, 30016, 24352, 54560, 16160, 9664, 50432, 0]],
        '酊' : [12, 13, 0, -2, 12, [0, 64992, 8256, 8256, 64576, 42048, 42048, 56384, 33856, 64576, 33856, 64704, 0]],
        '酖' : [12, 13, 0, -2, 12, [256, 63744, 10208, 9504, 64800, 43296, 43392, 55936, 35456, 60064, 36000, 63712, 0]],
        '酘' : [12, 13, 0, -2, 12, [0, 64448, 8768, 8768, 64064, 44128, 43008, 57312, 35360, 59968, 35200, 65120, 0]],
        '酣' : [12, 13, 0, -2, 12, [0, 64032, 8736, 10208, 64032, 43552, 43552, 56288, 35360, 64032, 35360, 64480, 0]],
        '酥' : [12, 13, 0, -2, 12, [0, 63584, 10112, 8320, 63616, 45024, 43136, 55680, 35520, 60576, 34944, 63616, 0]],
        '酩' : [12, 13, 0, -2, 12, [256, 63744, 8672, 8736, 64032, 44352, 43136, 56288, 36384, 64032, 35360, 64480, 0]],
        '酳' : [12, 13, 0, -2, 12, [128, 64768, 8736, 8512, 63616, 43296, 45024, 55840, 35808, 59936, 35808, 64032, 0]],
        '酲' : [12, 13, 0, -2, 12, [0, 64480, 8736, 8736, 64480, 43008, 45024, 55424, 35808, 59520, 34944, 65504, 0]],
        '醋' : [12, 13, 0, -2, 12, [576, 64064, 10208, 8768, 64064, 45024, 43008, 56288, 35360, 64480, 35360, 64480, 0]],
        '醉' : [12, 13, 0, -2, 12, [128, 63616, 10208, 8768, 64064, 44320, 43296, 55424, 36832, 63616, 34944, 63616, 0]],
        '醂' : [12, 13, 0, -2, 12, [576, 64064, 8768, 10208, 64064, 43584, 44736, 56160, 35392, 64064, 35392, 64064, 0]],
        '醢' : [12, 13, 0, -2, 12, [256, 63744, 10208, 8448, 64480, 44576, 44000, 55296, 35808, 60064, 35488, 65504, 0]],
        '醫' : [12, 13, 0, -2, 12, [0, 64448, 37440, 64480, 38464, 43392, 41536, 65504, 19008, 29632, 16448, 32704, 0]],
        '醯' : [12, 13, 0, -2, 12, [256, 61696, 20448, 16640, 62016, 40928, 54592, 46432, 38912, 63456, 37536, 65504, 0]],
        '醪' : [12, 13, 0, -2, 12, [0, 65520, 9552, 9136, 64848, 43280, 43456, 56864, 35024, 60160, 34912, 64384, 0]],
        '醵' : [12, 13, 0, -2, 12, [256, 63936, 8448, 12256, 63776, 44992, 43264, 55776, 36512, 63936, 36512, 63872, 0]],
        '醴' : [12, 13, 0, -2, 12, [640, 64128, 12256, 10912, 65504, 43680, 45024, 55840, 35808, 59712, 35136, 65504, 0]],
        '醺' : [12, 13, 0, -2, 12, [0, 63456, 8448, 12256, 64928, 43296, 45024, 55552, 36832, 59392, 35488, 64160, 0]],
        '釀' : [12, 13, 0, -2, 12, [256, 61696, 24544, 19104, 65248, 37504, 57312, 45696, 40928, 62112, 40000, 63264, 0]],
        '釁' : [12, 13, 0, -2, 12, [0, 61152, 35360, 61152, 35360, 65504, 35360, 29632, 16448, 65504, 34976, 13184, 0]],
        '釉' : [12, 13, 0, -2, 12, [128, 6272, 57472, 12256, 46240, 9376, 64672, 10208, 13472, 27808, 42144, 10208, 0]],
        '釋' : [12, 13, 0, -2, 12, [0, 8160, 60064, 12256, 45312, 10208, 64064, 8768, 12256, 28928, 45024, 8448, 0]],
        '釐' : [12, 13, 0, -2, 12, [512, 64000, 9184, 64576, 8832, 32736, 53824, 24512, 21056, 40896, 33280, 16352, 0]],
        '釖' : [12, 13, 0, -2, 12, [0, 14304, 18720, 33056, 63776, 8480, 64032, 8736, 46112, 42016, 8256, 63872, 0]],
        '釟' : [12, 13, 0, -2, 12, [0, 13248, 18496, 32832, 64064, 8768, 64064, 8768, 45632, 42016, 9248, 63520, 0]],
        '釡' : [12, 13, 0, -2, 12, [0, 12672, 49248, 3584, 12672, 49248, 16256, 1024, 32704, 9344, 5376, 65504, 0]],
        '釛' : [12, 13, 0, -2, 12, [256, 12544, 18688, 34784, 63776, 8480, 64032, 8736, 46112, 42016, 8256, 63872, 0]],
        '釼' : [12, 13, 0, -2, 12, [0, 14304, 18720, 33056, 64288, 8480, 64160, 8800, 46112, 42016, 8256, 63872, 0]],
        '釵' : [12, 13, 0, -2, 12, [0, 14304, 18464, 33312, 63776, 8224, 64064, 8512, 45184, 41344, 9792, 63520, 0]],
        '釶' : [12, 13, 0, -2, 12, [256, 12544, 17760, 34208, 63264, 11552, 62752, 9568, 46336, 42016, 9248, 63456, 0]],
        '鈞' : [12, 13, 0, -2, 12, [512, 12800, 19424, 33824, 62496, 9120, 63520, 8224, 45472, 42528, 8256, 63872, 0]],
        '釿' : [12, 13, 0, -2, 12, [0, 12384, 19328, 33280, 64000, 9184, 64064, 8768, 43584, 43584, 9280, 63552, 0]],
        '鈔' : [12, 13, 0, -2, 12, [256, 12544, 16640, 34112, 63776, 8448, 63744, 8960, 43040, 43072, 8576, 65024, 0]],
        '鈬' : [12, 13, 0, -2, 12, [0, 13280, 18976, 33312, 64032, 9184, 64128, 8832, 43584, 43584, 9248, 63520, 0]],
        '鈕' : [12, 13, 0, -2, 12, [0, 14304, 16672, 33056, 63776, 8480, 65504, 8736, 43552, 43552, 8736, 65504, 0]],
        '鈑' : [12, 13, 0, -2, 12, [0, 13280, 18944, 33280, 64480, 8736, 64160, 8864, 43584, 43584, 8352, 64288, 0]],
        '鉞' : [12, 13, 0, -2, 12, [128, 12448, 18400, 33920, 62592, 9376, 64672, 9280, 46400, 42560, 9376, 63776, 0]],
        '鉗' : [12, 13, 0, -2, 12, [512, 12832, 18976, 34784, 64032, 8736, 64032, 9184, 43552, 43552, 8736, 64480, 0]],
        '鉅' : [12, 13, 0, -2, 12, [0, 13280, 18944, 33280, 64480, 8736, 64032, 9184, 43520, 43520, 8704, 64480, 0]],
        '鉉' : [12, 13, 0, -2, 12, [256, 12544, 20448, 32896, 61696, 8736, 62528, 8832, 43264, 43552, 9248, 63456, 0]],
        '鉤' : [12, 13, 0, -2, 12, [512, 12800, 19424, 33824, 62496, 9120, 64160, 8864, 43936, 43040, 8224, 63680, 0]],
        '鉈' : [12, 13, 0, -2, 12, [256, 12544, 20448, 34848, 64032, 8704, 64096, 9088, 43520, 43552, 8736, 64480, 0]],
        '銕' : [12, 13, 0, -2, 12, [256, 12544, 20448, 33024, 63456, 8480, 65504, 9472, 47072, 41248, 8832, 64608, 0]],
        '鈿' : [12, 13, 0, -2, 12, [0, 14304, 17696, 34080, 62752, 9504, 63456, 9504, 46368, 42272, 9504, 65504, 0]],
        '鉋' : [12, 13, 0, -2, 12, [512, 12800, 19424, 33824, 64416, 8864, 64160, 9120, 43616, 43520, 8736, 64480, 0]],
        '鉐' : [12, 13, 0, -2, 12, [0, 14304, 18560, 32896, 63616, 8448, 64480, 9760, 43552, 43552, 8736, 64480, 0]],
        '銜' : [12, 13, 0, -2, 12, [0, 9952, 18688, 36864, 7936, 9440, 24352, 50208, 21792, 21792, 17440, 24416, 0]],
        '銖' : [12, 13, 0, -2, 12, [128, 13440, 17536, 34784, 63616, 8320, 65504, 8320, 43392, 43712, 9376, 63616, 0]],
        '銓' : [12, 13, 0, -2, 12, [0, 14208, 16704, 33312, 62464, 10208, 63616, 8320, 44000, 43136, 8320, 65504, 0]],
        '銛' : [12, 13, 0, -2, 12, [0, 12384, 19328, 32896, 63616, 10208, 63616, 8320, 44000, 43552, 8736, 64480, 0]],
        '鉚' : [12, 13, 0, -2, 12, [0, 13280, 19616, 34976, 64160, 10912, 64160, 10912, 44704, 41696, 9344, 62592, 0]],
        '鋏' : [12, 13, 0, -2, 12, [256, 12544, 20448, 33024, 62784, 9536, 64416, 8480, 43264, 43648, 9280, 63520, 0]],
        '銹' : [12, 13, 0, -2, 12, [0, 14304, 16640, 36832, 62208, 11584, 61728, 10112, 45792, 41504, 9248, 63680, 0]],
        '銷' : [12, 13, 0, -2, 12, [256, 12544, 17728, 35104, 61696, 10208, 62496, 10208, 46112, 42976, 9248, 64608, 0]],
        '鋩' : [12, 13, 0, -2, 12, [576, 12864, 20448, 33344, 63744, 8448, 65504, 8704, 43520, 43520, 8704, 63968, 0]],
        '錏' : [12, 13, 0, -2, 12, [0, 14304, 19072, 33408, 63200, 9248, 64544, 9952, 45696, 41600, 8832, 65504, 0]],
        '鋺' : [12, 13, 0, -2, 12, [256, 12544, 20448, 34848, 65248, 10912, 64160, 10912, 46816, 41600, 9376, 63712, 0]],
        '鍄' : [12, 13, 0, -2, 12, [256, 12544, 20448, 32768, 63424, 9280, 64576, 10176, 45312, 42304, 10528, 62208, 0]],
        '錮' : [12, 13, 0, -2, 12, [0, 16352, 18720, 35104, 65504, 10528, 64416, 10912, 43680, 43936, 10272, 65504, 0]],
        '錙' : [12, 13, 0, -2, 12, [0, 12960, 19104, 34112, 62784, 8864, 64160, 10208, 46240, 42976, 9376, 65504, 0]],
        '錢' : [12, 13, 0, -2, 12, [512, 12832, 20448, 33344, 61856, 9824, 64640, 10208, 46144, 41600, 8992, 64736, 0]],
        '錚' : [12, 13, 0, -2, 12, [0, 14304, 17728, 35104, 63488, 10208, 61600, 12256, 45216, 42976, 8320, 64384, 0]],
        '錣' : [12, 13, 0, -2, 12, [0, 12000, 16928, 35488, 62528, 10912, 61440, 12000, 41504, 43680, 9280, 64160, 0]],
        '錺' : [12, 13, 0, -2, 12, [576, 12864, 20448, 33344, 61696, 12256, 61952, 9184, 46112, 42016, 8256, 63872, 0]],
        '錵' : [12, 13, 0, -2, 12, [576, 12864, 20448, 33344, 61440, 8832, 64160, 9920, 43648, 41600, 8864, 64224, 0]],
        '錻' : [12, 13, 0, -2, 12, [128, 12448, 20128, 32896, 65504, 8768, 64064, 9152, 43584, 43584, 11168, 64544, 0]],
        '鍜' : [12, 13, 0, -2, 12, [0, 16352, 18976, 35360, 65248, 10240, 63968, 11808, 43168, 44704, 10432, 64288, 0]],
        '鍠' : [12, 13, 0, -2, 12, [128, 12544, 18400, 33824, 63456, 9248, 65504, 8448, 47072, 41216, 8448, 65504, 0]],
        '鍼' : [12, 13, 0, -2, 12, [128, 12448, 18400, 33920, 63360, 9376, 65440, 9536, 46912, 42048, 10400, 64288, 0]],
        '鍮' : [12, 13, 0, -2, 12, [0, 12160, 16960, 33824, 64480, 8192, 65184, 10912, 44864, 43840, 11936, 64160, 0]],
        '鍖' : [12, 13, 0, -2, 12, [576, 12864, 20448, 33344, 62400, 8768, 64064, 10208, 46400, 42592, 9216, 63456, 0]],
        '鎰' : [12, 13, 0, -2, 12, [0, 13760, 17472, 34848, 64416, 8192, 62912, 9280, 43040, 45024, 10912, 65504, 0]],
        '鎬' : [12, 13, 0, -2, 12, [256, 12544, 20448, 33856, 63424, 8192, 65504, 10272, 43936, 43680, 11168, 63584, 0]],
        '鎭' : [12, 13, 0, -2, 12, [1024, 13408, 18304, 33824, 62432, 8736, 64480, 10784, 45024, 40960, 8768, 64544, 0]],
        '鎔' : [12, 13, 0, -2, 12, [256, 12544, 20448, 35488, 62528, 10656, 62016, 9248, 45024, 46112, 9248, 63456, 0]],
        '鎹' : [12, 13, 0, -2, 12, [0, 12992, 19008, 33824, 63456, 8320, 63456, 8320, 47424, 43552, 11264, 62432, 0]],
        '鏖' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 41216, 48608, 41216, 65504, 42112, 32704, 42144, 5376, 65504, 0]],
        '鏗' : [12, 13, 0, -2, 12, [0, 16352, 18976, 36768, 63808, 12096, 64160, 12192, 45184, 42976, 8320, 65504, 0]],
        '鏨' : [12, 13, 0, -2, 12, [4096, 4576, 65280, 21984, 32032, 21792, 65312, 9376, 32704, 42144, 5376, 65504, 0]],
        '鏥' : [12, 13, 0, -2, 12, [256, 12544, 20448, 34848, 65504, 9344, 60384, 14880, 44000, 43552, 10784, 64480, 0]],
        '鏘' : [12, 13, 0, -2, 12, [0, 13024, 19232, 35488, 64064, 12192, 61984, 12256, 43808, 43680, 10784, 64096, 0]],
        '鏃' : [12, 13, 0, -2, 12, [256, 13568, 17888, 40704, 62944, 9536, 63040, 11232, 43584, 43584, 8864, 64800, 0]],
        '鏝' : [12, 13, 0, -2, 12, [0, 14272, 17472, 34752, 62528, 12256, 64160, 12256, 42048, 45632, 9088, 64608, 0]],
        '鏐' : [12, 13, 0, -2, 12, [0, 12256, 19104, 34656, 60064, 8736, 62336, 11328, 41376, 46592, 8384, 63232, 0]],
        '鏈' : [12, 13, 0, -2, 12, [128, 12416, 20448, 33440, 64480, 8864, 64480, 10368, 45024, 43136, 13312, 58336, 0]],
        '鏤' : [12, 13, 0, -2, 12, [128, 12416, 20448, 33952, 65504, 9376, 63456, 9376, 45024, 42048, 9088, 64608, 0]],
        '鐚' : [12, 13, 0, -2, 12, [0, 12256, 17024, 36576, 63520, 12000, 62080, 12256, 41216, 46144, 9248, 62336, 0]],
        '鐔' : [12, 13, 0, -2, 12, [0, 12256, 17024, 36832, 64160, 10912, 65504, 9280, 47040, 42048, 12256, 61696, 0]],
        '鐓' : [12, 13, 0, -2, 12, [1152, 9344, 24544, 32928, 61216, 10528, 65440, 8864, 42048, 48960, 9376, 60704, 0]],
        '鐃' : [12, 13, 0, -2, 12, [256, 12544, 20448, 33024, 65504, 9280, 65248, 9280, 45024, 41600, 8864, 64736, 0]],
        '鐇' : [12, 13, 0, -2, 12, [0, 14304, 19616, 33472, 63456, 8832, 64672, 10208, 46240, 42976, 9376, 63456, 0]],
        '鐐' : [12, 13, 0, -2, 12, [256, 12544, 20448, 35392, 63456, 9248, 65504, 9248, 47072, 41216, 10560, 62240, 0]],
        '鐶' : [12, 13, 0, -2, 12, [0, 12256, 19104, 36832, 61440, 12256, 62528, 10176, 45408, 41856, 11584, 61728, 0]],
        '鐫' : [12, 13, 0, -2, 12, [576, 12864, 18400, 35968, 63456, 9344, 65504, 8192, 44768, 43680, 11168, 63584, 0]],
        '鐵' : [12, 13, 0, -2, 12, [1152, 13472, 20128, 33920, 65504, 10880, 60064, 16288, 42048, 44608, 9376, 65312, 0]],
        '鐡' : [12, 13, 0, -2, 12, [1152, 13472, 20384, 33920, 65504, 8352, 65440, 10560, 44864, 41536, 10912, 65312, 0]],
        '鐺' : [12, 13, 0, -2, 12, [256, 13632, 20448, 35488, 64160, 9088, 61440, 12256, 43296, 45024, 10528, 65504, 0]],
        '鑁' : [12, 13, 0, -2, 12, [256, 11616, 19104, 36192, 64160, 12256, 62080, 11360, 41920, 44096, 9088, 64608, 0]],
        '鑒' : [12, 13, 0, -2, 12, [512, 64000, 41952, 64512, 35808, 62112, 40928, 9344, 32704, 42144, 5376, 65504, 0]],
        '鑄' : [12, 13, 0, -2, 12, [256, 12544, 20448, 33024, 61408, 8224, 63424, 8448, 45024, 43552, 10912, 65120, 0]],
        '鑛' : [12, 13, 0, -2, 12, [128, 12416, 20448, 35136, 65504, 10560, 65504, 9376, 47072, 42144, 10208, 63520, 0]],
        '鑠' : [12, 13, 0, -2, 12, [256, 12864, 20384, 38080, 61344, 9408, 63392, 8448, 45024, 41728, 9536, 63776, 0]],
        '鑢' : [12, 13, 0, -2, 12, [896, 12800, 20448, 35104, 64448, 10496, 61408, 13632, 42944, 45312, 9280, 63392, 0]],
        '鑞' : [12, 13, 0, -2, 12, [2336, 12864, 18720, 36832, 60064, 10528, 65504, 10560, 44480, 43296, 11680, 64288, 0]],
        '鑪' : [12, 13, 0, -2, 12, [896, 12800, 20448, 35104, 64448, 10496, 65504, 9536, 42304, 45024, 10912, 65504, 0]],
        '鈩' : [12, 13, 0, -2, 12, [0, 13280, 18432, 32768, 64480, 8736, 64032, 9184, 43520, 43520, 8704, 64512, 0]],
        '鑰' : [12, 13, 0, -2, 12, [0, 13184, 17472, 34848, 65504, 10912, 65504, 8192, 45024, 43680, 12256, 64160, 0]],
        '鑵' : [12, 13, 0, -2, 12, [1152, 13440, 20448, 35488, 65248, 9344, 61408, 14592, 45024, 43264, 10496, 65504, 0]],
        '鑷' : [12, 13, 0, -2, 12, [0, 12256, 17472, 34752, 62528, 12256, 64160, 12000, 43680, 44768, 11168, 61984, 0]],
        '鑽' : [12, 13, 0, -2, 12, [576, 12864, 20448, 35392, 61408, 9408, 63808, 10208, 46112, 42976, 9248, 63456, 0]],
        '鑚' : [12, 13, 0, -2, 12, [576, 12864, 20448, 33344, 61408, 9536, 63648, 10208, 46112, 42976, 9248, 63456, 0]],
        '鑼' : [12, 13, 0, -2, 12, [0, 12256, 19104, 36832, 62784, 11072, 62944, 10560, 45024, 42304, 12096, 62944, 0]],
        '鑾' : [12, 13, 0, -2, 12, [1024, 20032, 41120, 20032, 43680, 19008, 57184, 9344, 32704, 42144, 5376, 65504, 0]],
        '钁' : [12, 13, 0, -2, 12, [0, 12000, 19104, 35488, 61408, 14592, 61408, 10496, 49120, 42048, 9088, 64608, 0]],
        '鑿' : [12, 13, 0, -2, 12, [0, 21952, 65344, 10560, 65120, 21888, 32608, 9344, 32704, 42144, 5376, 65504, 0]],
        '閂' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 32800, 32800, 49056, 32800, 32800, 32864, 0]],
        '閇' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 32800, 49056, 33824, 34336, 34080, 33888, 0]],
        '閊' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 32800, 33824, 42144, 42144, 49056, 32864, 0]],
        '閔' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 33824, 49056, 35360, 35360, 33824, 39776, 0]],
        '閖' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 33824, 48544, 34336, 36128, 46240, 35936, 0]],
        '閘' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 42144, 49056, 42144, 49056, 33824, 33888, 0]],
        '閙' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 33824, 49056, 33824, 40736, 38176, 38240, 0]],
        '閠' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 32800, 49056, 33824, 40736, 33952, 49056, 0]],
        '閨' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 33824, 49056, 33824, 40736, 33824, 49056, 0]],
        '閧' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 35360, 40736, 35360, 49056, 35360, 37216, 0]],
        '閭' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 37152, 40736, 34848, 49056, 41120, 49056, 0]],
        '閼' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 37536, 64608, 37152, 48288, 50720, 39328, 0]],
        '閻' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 37152, 48032, 57504, 48032, 41120, 49056, 0]],
        '閹' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 42144, 65504, 42144, 49056, 33824, 33696, 0]],
        '閾' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 33312, 47776, 43680, 47392, 34464, 47200, 0]],
        '闊' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 41504, 53152, 41504, 36768, 43168, 53216, 0]],
        '濶' : [12, 13, 0, -2, 12, [0, 7904, 37536, 24288, 4768, 40928, 20768, 6048, 20768, 22432, 38048, 38880, 0]],
        '闃' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 37152, 40736, 33952, 49056, 35360, 45472, 0]],
        '闍' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 35104, 49056, 37024, 49056, 53408, 40864, 0]],
        '闌' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 33824, 49056, 42144, 49056, 38176, 42144, 0]],
        '闕' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 37408, 64416, 42272, 43296, 64160, 42080, 0]],
        '闔' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 33824, 49056, 35104, 49056, 38176, 49056, 0]],
        '闖' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 42016, 49056, 42016, 49056, 32928, 43680, 0]],
        '關' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 64480, 37152, 43680, 38176, 43680, 48032, 37472, 0]],
        '闡' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 42144, 49056, 38176, 38176, 49056, 33888, 0]],
        '闥' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 41504, 36768, 58656, 44960, 45600, 53216, 0]],
        '闢' : [12, 13, 0, -2, 12, [0, 64480, 35360, 64480, 35360, 65504, 43680, 47776, 42912, 47392, 44960, 47456, 0]],
        '阡' : [12, 13, 0, -2, 12, [0, 61664, 40704, 41216, 49408, 41216, 40928, 37120, 37120, 57600, 33024, 33024, 0]],
        '阨' : [12, 13, 0, -2, 12, [0, 61408, 43008, 43008, 52192, 43552, 43552, 43744, 43520, 51744, 37408, 37856, 0]],
        '阮' : [12, 13, 0, -2, 12, [0, 61408, 40960, 40960, 49152, 49120, 42112, 42112, 42112, 50336, 34976, 45280, 0]],
        '阯' : [12, 13, 0, -2, 12, [256, 57600, 41216, 41216, 51456, 43488, 43264, 43264, 51456, 35072, 35072, 49120, 0]],
        '陂' : [12, 13, 0, -2, 12, [256, 57600, 45024, 43296, 51456, 44992, 43072, 43584, 53568, 36992, 33600, 35872, 0]],
        '陌' : [12, 13, 0, -2, 12, [0, 63456, 36992, 41216, 51168, 42016, 37920, 38880, 37920, 58400, 33824, 34784, 0]],
        '陏' : [12, 13, 0, -2, 12, [256, 57600, 45024, 41472, 49664, 42976, 42016, 45024, 46112, 51168, 33824, 33888, 0]],
        '陋' : [12, 13, 0, -2, 12, [0, 59360, 41088, 41088, 52192, 43680, 43680, 43872, 43552, 51808, 34816, 36832, 0]],
        '陷' : [12, 13, 0, -2, 12, [0, 62432, 37920, 43072, 49408, 42592, 37920, 37920, 38752, 58400, 33824, 34784, 0]],
        '陜' : [12, 13, 0, -2, 12, [256, 57600, 49120, 41216, 50496, 42304, 43936, 43296, 41216, 49792, 33856, 38944, 0]],
        '陞' : [12, 13, 0, -2, 12, [0, 61248, 41536, 45024, 49728, 42048, 43328, 41216, 45024, 49408, 33024, 36832, 0]],
        '陝' : [12, 13, 0, -2, 12, [256, 57600, 49120, 41216, 52672, 42304, 42304, 43936, 53536, 33408, 33856, 38944, 0]],
        '陟' : [12, 13, 0, -2, 12, [256, 58624, 42432, 42240, 57312, 41216, 42240, 43296, 41760, 49216, 33152, 36352, 0]],
        '陦' : [12, 13, 0, -2, 12, [512, 57856, 49120, 41472, 53216, 41472, 49120, 42048, 57312, 34880, 37952, 33472, 0]],
        '陲' : [12, 13, 0, -2, 12, [0, 61408, 41216, 45024, 50464, 49120, 42272, 42272, 45024, 49408, 33024, 36832, 0]],
        '陬' : [12, 13, 0, -2, 12, [0, 65504, 43520, 45024, 51744, 43680, 44704, 43680, 43584, 52800, 39584, 33568, 0]],
        '隍' : [12, 13, 0, -2, 12, [256, 57856, 45024, 43040, 53216, 43040, 45024, 41216, 45024, 49408, 33024, 36832, 0]],
        '隘' : [12, 13, 0, -2, 12, [0, 58816, 42048, 43040, 52128, 40960, 42432, 42048, 43040, 53216, 35488, 40928, 0]],
        '隕' : [12, 13, 0, -2, 12, [0, 61408, 43040, 45024, 49152, 45024, 43040, 45024, 43040, 53216, 33856, 38944, 0]],
        '隗' : [12, 13, 0, -2, 12, [256, 57856, 45024, 43296, 53216, 43296, 43296, 45024, 50496, 34112, 35104, 45536, 0]],
        '險' : [12, 13, 0, -2, 12, [0, 59328, 43040, 47072, 49152, 44768, 43680, 43680, 44768, 50240, 35488, 37152, 0]],
        '隧' : [12, 13, 0, -2, 12, [0, 62144, 43584, 42016, 55264, 43264, 42656, 47552, 44704, 51616, 37888, 41952, 0]],
        '隱' : [12, 13, 0, -2, 12, [0, 61408, 43584, 45344, 57312, 40992, 45024, 40992, 45024, 50432, 37952, 41888, 0]],
        '隲' : [12, 13, 0, -2, 12, [256, 58688, 43296, 41792, 49280, 45024, 43264, 45024, 43264, 53216, 32800, 38240, 0]],
        '隰' : [12, 13, 0, -2, 12, [0, 61408, 43040, 45024, 51232, 45024, 42304, 43680, 45024, 49152, 38208, 41632, 0]],
        '隴' : [12, 13, 0, -2, 12, [1152, 58496, 48864, 43648, 51936, 48928, 43744, 44672, 43744, 52864, 35488, 35552, 0]],
        '隶' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1088, 65504, 1088, 32736, 17472, 9856, 5376, 9344, 52320, 0]],
        '隸' : [12, 13, 0, -2, 12, [8448, 8448, 65504, 24864, 45024, 8480, 63456, 288, 64320, 9600, 43328, 45856, 0]],
        '隹' : [12, 13, 0, -2, 12, [4096, 4352, 4352, 16352, 8704, 32704, 41472, 8704, 16320, 8704, 8704, 16352, 0]],
        '雎' : [12, 13, 0, -2, 12, [512, 576, 62016, 38880, 62592, 40928, 38016, 62592, 38880, 38016, 48256, 51168, 0]],
        '雋' : [12, 13, 0, -2, 12, [4352, 4352, 16320, 25088, 49088, 8704, 16320, 0, 64480, 35360, 36384, 32864, 0]],
        '雉' : [12, 13, 0, -2, 12, [512, 33344, 62016, 42976, 42112, 12256, 62592, 9344, 22496, 21632, 35968, 34784, 0]],
        '雍' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 8768, 18400, 35968, 59360, 17536, 38880, 62592, 9344, 51168, 0]],
        '襍' : [12, 13, 0, -2, 12, [1024, 17536, 20448, 63744, 12256, 10496, 20448, 49408, 28640, 17152, 17728, 22816, 0]],
        '雜' : [12, 13, 0, -2, 12, [256, 8512, 64832, 21472, 22080, 44000, 8768, 64064, 9184, 29248, 43584, 9184, 0]],
        '霍' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 8704, 32704, 50176, 32704, 17408, 32736, 0]],
        '雕' : [12, 13, 0, -2, 12, [128, 64640, 42112, 42464, 62784, 42976, 62784, 42304, 62944, 38208, 62784, 36320, 0]],
        '雹' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 9216, 16320, 25152, 48832, 8224, 16352, 0]],
        '霄' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 16256, 24768, 49056, 8320, 16256, 8320, 0]],
        '霆' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 1024, 61408, 8448, 20448, 57600, 8448, 57312, 0]],
        '霈' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 38048, 16640, 16352, 33024, 24544, 4384, 20768, 37216, 0]],
        '霓' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 2048, 29632, 16448, 32704, 4640, 58336, 0]],
        '霎' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 5248, 32704, 4352, 65504, 4224, 7936, 24768, 0]],
        '霑' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 33952, 8704, 37824, 16896, 8160, 20512, 40928, 0]],
        '霏' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42144, 2560, 64480, 2560, 31680, 2560, 62432, 4608, 0]],
        '霖' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 8320, 64480, 8576, 29376, 42144, 8320, 0]],
        '霙' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 38048, 4352, 65504, 9344, 65504, 4352, 57568, 0]],
        '霤' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 5248, 64480, 33056, 51808, 32704, 17472, 32704, 0]],
        '霪' : [12, 13, 0, -2, 12, [0, 32736, 1024, 65504, 38048, 17408, 4064, 35392, 20768, 4064, 16640, 40928, 0]],
        '霰' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42144, 20992, 64480, 21568, 64832, 35136, 63872, 36448, 0]],
        '霹' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 62592, 38880, 62016, 36832, 61568, 38880, 61568, 0]],
        '霽' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 5248, 65504, 21824, 46496, 16448, 32704, 49216, 0]],
        '霾' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 38048, 53248, 10208, 54432, 13472, 55264, 4224, 26592, 0]],
        '靄' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 33824, 63456, 1088, 63424, 1088, 63456, 39968, 63328, 0]],
        '靆' : [12, 13, 0, -2, 12, [128, 63616, 9184, 64672, 44000, 44192, 992, 64672, 1728, 64928, 17920, 63968, 0]],
        '靈' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42272, 5248, 32704, 19008, 65504, 9344, 21824, 65504, 0]],
        '靂' : [12, 13, 0, -2, 12, [0, 32704, 1024, 65504, 42144, 32704, 18560, 24000, 27296, 21440, 37376, 65504, 0]],
        '靉' : [12, 13, 0, -2, 12, [0, 65504, 8864, 63808, 45024, 43680, 512, 63968, 1600, 63808, 18816, 63072, 0]],
        '靜' : [12, 13, 0, -2, 12, [8192, 10208, 64832, 9504, 63488, 9184, 63648, 36832, 63648, 35808, 63616, 35200, 0]],
        '靠' : [12, 13, 0, -2, 12, [1024, 9216, 16320, 17408, 65504, 8320, 65504, 2560, 31680, 2560, 62432, 4608, 0]],
        '靤' : [12, 13, 0, -2, 12, [512, 64000, 17376, 17440, 65440, 43680, 60064, 43936, 60000, 43520, 43552, 63968, 0]],
        '靦' : [12, 13, 0, -2, 12, [0, 64480, 16928, 17376, 64032, 44000, 59936, 44000, 59712, 43328, 43584, 64608, 0]],
        '靨' : [12, 13, 0, -2, 12, [0, 65504, 38048, 48608, 42112, 48448, 9504, 65504, 19008, 20032, 19008, 32704, 0]],
        '勒' : [12, 13, 0, -2, 12, [256, 20736, 63744, 22496, 28960, 8480, 63776, 43296, 64032, 8736, 63552, 8576, 0]],
        '靫' : [12, 13, 0, -2, 12, [0, 22496, 63520, 20768, 28832, 8224, 63520, 43584, 63808, 8320, 63808, 9760, 0]],
        '靱' : [12, 13, 0, -2, 12, [0, 22496, 63776, 20768, 29984, 8992, 63776, 43424, 64096, 8736, 63552, 8576, 0]],
        '靹' : [12, 13, 0, -2, 12, [768, 20736, 63744, 22496, 29984, 9504, 64800, 42656, 64608, 9248, 64544, 9312, 0]],
        '鞅' : [12, 13, 0, -2, 12, [128, 18560, 65504, 19104, 31392, 8864, 64160, 45024, 63616, 8320, 63808, 9760, 0]],
        '靼' : [12, 13, 0, -2, 12, [0, 21472, 64032, 21024, 29664, 8736, 64032, 43552, 64480, 8192, 63488, 10208, 0]],
        '鞁' : [12, 13, 0, -2, 12, [128, 20608, 65504, 21664, 29824, 10208, 64544, 42272, 64832, 8384, 63776, 9760, 0]],
        '靺' : [12, 13, 0, -2, 12, [256, 20736, 65504, 20736, 28928, 10208, 63744, 43264, 64384, 9536, 63776, 8448, 0]],
        '鞆' : [12, 13, 0, -2, 12, [0, 22496, 63744, 20736, 30688, 9504, 64800, 42656, 64608, 9248, 64544, 9312, 0]],
        '鞋' : [12, 13, 0, -2, 12, [128, 20608, 64480, 20608, 28800, 10208, 63616, 43136, 64480, 8320, 63616, 10208, 0]],
        '鞏' : [12, 13, 0, -2, 12, [0, 64448, 8768, 15168, 50272, 4352, 65504, 4352, 16256, 9344, 65504, 1024, 0]],
        '鞐' : [12, 13, 0, -2, 12, [256, 20736, 63936, 20736, 28928, 10208, 63488, 45024, 63744, 8576, 63808, 8448, 0]],
        '鞜' : [12, 13, 0, -2, 12, [128, 20608, 65184, 21184, 29856, 10656, 64512, 42976, 64544, 10208, 64544, 10208, 0]],
        '鞨' : [12, 13, 0, -2, 12, [0, 22464, 64576, 22464, 29760, 10176, 62464, 45024, 64032, 10528, 65312, 8384, 0]],
        '鞦' : [12, 13, 0, -2, 12, [128, 21376, 64640, 21664, 29856, 12192, 62592, 42176, 64832, 9536, 65056, 9248, 0]],
        '鞣' : [12, 13, 0, -2, 12, [0, 22464, 63616, 20736, 32736, 9504, 63808, 43264, 65504, 8960, 62784, 10528, 0]],
        '鞳' : [12, 13, 0, -2, 12, [576, 21056, 65504, 21056, 29056, 8768, 64544, 43968, 63488, 10208, 64544, 10208, 0]],
        '鞴' : [12, 13, 0, -2, 12, [576, 21056, 65504, 21056, 30688, 9216, 63456, 44320, 63456, 9504, 65504, 9504, 0]],
        '韃' : [12, 13, 0, -2, 12, [64, 21568, 64480, 20544, 30176, 8864, 63968, 44608, 64480, 8768, 64256, 9440, 0]],
        '韆' : [12, 13, 0, -2, 12, [0, 21472, 64640, 21472, 30368, 9184, 63808, 44768, 64128, 8928, 64256, 9440, 0]],
        '韈' : [12, 13, 0, -2, 12, [576, 21056, 65504, 21056, 30688, 8864, 65504, 43328, 65504, 10528, 64704, 11040, 0]],
        '韋' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 2112, 65504, 8320, 16256, 1024, 32704, 17408, 65504, 1024, 0]],
        '韜' : [12, 13, 0, -2, 12, [8192, 10208, 62784, 21792, 63488, 37120, 63072, 9248, 63328, 42016, 64544, 10208, 0]],
        '韭' : [12, 13, 0, -2, 12, [2560, 2560, 64480, 2560, 2560, 31680, 2560, 2560, 31680, 2560, 2560, 65504, 0]],
        '齏' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2560, 62944, 21824, 49056, 10816, 15168, 10816, 32704, 16448, 0]],
        '韲' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2560, 62944, 21824, 38176, 31680, 2560, 31680, 2560, 65504, 0]],
        '竟' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 4352, 4352, 65504, 8320, 16256, 8320, 16256, 4640, 58336, 0]],
        '韶' : [12, 13, 0, -2, 12, [8192, 10208, 63776, 20768, 21024, 64704, 0, 64480, 35360, 64032, 35360, 64480, 0]],
        '韵' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 21536, 21536, 64416, 32, 63520, 35232, 65056, 34880, 63872, 0]],
        '頏' : [12, 13, 0, -2, 12, [8192, 10208, 8448, 65504, 1056, 30688, 21536, 22496, 21536, 24544, 33344, 35872, 0]],
        '頌' : [12, 13, 0, -2, 12, [0, 30688, 4352, 22496, 19488, 34784, 42016, 10208, 17440, 22496, 37440, 60448, 0]],
        '頸' : [12, 13, 0, -2, 12, [0, 65504, 20736, 42976, 21536, 22496, 1056, 65504, 9248, 10208, 12864, 52256, 0]],
        '頤' : [12, 13, 0, -2, 12, [0, 65504, 41216, 42976, 46112, 38880, 37920, 47072, 42016, 42976, 62016, 3104, 0]],
        '頡' : [12, 13, 0, -2, 12, [16384, 20448, 61696, 18400, 17440, 63456, 1056, 2016, 62496, 38880, 37440, 64544, 0]],
        '頷' : [12, 13, 0, -2, 12, [0, 28640, 37120, 59360, 1056, 63456, 5152, 10208, 62496, 38880, 37440, 64544, 0]],
        '頽' : [12, 13, 0, -2, 12, [0, 63456, 8448, 65504, 9248, 30688, 44064, 10208, 29728, 22496, 37440, 39968, 0]],
        '顆' : [12, 13, 0, -2, 12, [0, 64480, 43136, 64480, 43552, 44000, 64032, 9184, 65056, 9184, 26944, 42528, 0]],
        '顏' : [12, 13, 0, -2, 12, [8192, 10208, 63616, 21472, 43552, 64480, 37408, 58336, 39456, 58336, 6464, 58912, 0]],
        '顋' : [12, 13, 0, -2, 12, [0, 65504, 43136, 44000, 64032, 44000, 43552, 64480, 544, 11232, 41280, 46624, 0]],
        '顫' : [12, 13, 0, -2, 12, [4096, 5088, 64640, 44000, 47648, 33760, 65056, 19424, 31264, 19424, 18752, 65056, 0]],
        '顯' : [12, 13, 0, -2, 12, [0, 64480, 34944, 64480, 35360, 64480, 21024, 44000, 64032, 992, 21824, 43552, 0]],
        '顰' : [12, 13, 0, -2, 12, [8192, 47072, 41216, 65504, 9248, 61408, 12864, 50208, 16256, 10368, 65504, 1024, 0]],
        '顱' : [12, 13, 0, -2, 12, [14336, 9184, 64640, 38880, 64032, 37856, 65056, 44000, 43552, 32736, 21568, 65056, 0]],
        '顴' : [12, 13, 0, -2, 12, [20480, 22496, 63744, 42976, 64544, 22496, 64544, 42976, 64544, 42976, 41536, 64544, 0]],
        '顳' : [12, 13, 0, -2, 12, [0, 65504, 18560, 31712, 18976, 65504, 43552, 64480, 43552, 64480, 43328, 11808, 0]],
        '颪' : [12, 13, 0, -2, 12, [0, 65504, 1024, 1984, 1024, 32704, 17472, 24384, 21824, 24352, 33952, 49056, 0]],
        '颯' : [12, 13, 0, -2, 12, [0, 20448, 18464, 65504, 10784, 44960, 43680, 43680, 44960, 10784, 62112, 6048, 0]],
        '颱' : [12, 13, 0, -2, 12, [0, 64640, 33920, 64800, 42464, 64512, 42464, 42272, 64800, 42464, 10752, 63968, 0]],
        '颶' : [12, 13, 0, -2, 12, [0, 64992, 34080, 64992, 42272, 64800, 42976, 41984, 64832, 42272, 10752, 63968, 0]],
        '飄' : [12, 13, 0, -2, 12, [0, 65504, 10272, 65504, 43552, 65440, 2720, 64160, 4000, 64032, 21152, 47008, 0]],
        '飃' : [12, 13, 0, -2, 12, [0, 65504, 35136, 65504, 43680, 65504, 43008, 45024, 63488, 42976, 10560, 64288, 0]],
        '飆' : [12, 13, 0, -2, 12, [10240, 10208, 64544, 10208, 21792, 36832, 21792, 21792, 65504, 21760, 43296, 44000, 0]],
        '飩' : [12, 13, 0, -2, 12, [256, 28928, 36832, 61696, 288, 62752, 38176, 63456, 33024, 63744, 33056, 63968, 0]],
        '飫' : [12, 13, 0, -2, 12, [0, 28864, 36608, 61696, 256, 61696, 40928, 61696, 33024, 62080, 33856, 63520, 0]],
        '餃' : [12, 13, 0, -2, 12, [256, 28928, 36832, 61440, 576, 62496, 36864, 62016, 33088, 61568, 33088, 63008, 0]],
        '餉' : [12, 13, 0, -2, 12, [256, 28928, 35328, 63456, 1056, 62496, 38816, 62624, 34720, 62496, 33824, 62560, 0]],
        '餒' : [12, 13, 0, -2, 12, [0, 30688, 33792, 63808, 2336, 61440, 37120, 65504, 33344, 63040, 33664, 64608, 0]],
        '餔' : [12, 13, 0, -2, 12, [256, 28960, 36832, 61696, 2016, 62752, 38880, 62752, 34784, 62752, 34080, 62816, 0]],
        '餘' : [12, 13, 0, -2, 12, [0, 29568, 33856, 63520, 2016, 61696, 37120, 65504, 33024, 62784, 35104, 62208, 0]],
        '餡' : [12, 13, 0, -2, 12, [0, 29664, 33824, 63552, 256, 63072, 37920, 62496, 34656, 62496, 33824, 63456, 0]],
        '餝' : [12, 13, 0, -2, 12, [576, 29248, 36832, 62016, 256, 65504, 35072, 63968, 33312, 64032, 32832, 63872, 0]],
        '餞' : [12, 13, 0, -2, 12, [512, 29216, 36832, 62016, 416, 63072, 38016, 65504, 33856, 62080, 33568, 64736, 0]],
        '餤' : [12, 13, 0, -2, 12, [128, 29344, 35520, 61568, 320, 64032, 34816, 64160, 33472, 63616, 33088, 64032, 0]],
        '餠' : [12, 13, 0, -2, 12, [576, 29248, 33920, 61408, 576, 62016, 37696, 65248, 33344, 62016, 33856, 62528, 0]],
        '餬' : [12, 13, 0, -2, 12, [1024, 30176, 34080, 65312, 1504, 58656, 44832, 59872, 35104, 63776, 36640, 62048, 0]],
        '餮' : [12, 13, 0, -2, 12, [0, 64960, 18976, 43136, 4384, 61376, 8320, 32704, 41120, 16256, 8768, 63968, 0]],
        '餽' : [12, 13, 0, -2, 12, [256, 29184, 34784, 62752, 2016, 62752, 38176, 63456, 33088, 62272, 34080, 63968, 0]],
        '餾' : [12, 13, 0, -2, 12, [0, 28128, 34976, 60064, 2848, 64864, 36864, 63456, 34080, 63456, 34080, 63456, 0]],
        '饂' : [12, 13, 0, -2, 12, [0, 30656, 34112, 63168, 1088, 63424, 36864, 65504, 35488, 64160, 35488, 65504, 0]],
        '饉' : [12, 13, 0, -2, 12, [576, 29248, 36832, 62016, 2016, 62624, 38048, 63456, 32896, 63456, 32896, 63456, 0]],
        '饅' : [12, 13, 0, -2, 12, [0, 30656, 33856, 63424, 1088, 65504, 35488, 65504, 33856, 62016, 33664, 64608, 0]],
        '饐' : [12, 13, 0, -2, 12, [256, 28928, 36832, 61696, 4064, 63520, 38848, 62528, 34752, 62080, 33408, 65504, 0]],
        '饋' : [12, 13, 0, -2, 12, [256, 28928, 36832, 63776, 4064, 61696, 40928, 62528, 34752, 62528, 34752, 63584, 0]],
        '饑' : [12, 13, 0, -2, 12, [256, 30016, 35232, 62784, 2720, 61408, 41216, 61408, 34944, 61088, 35008, 62240, 0]],
        '饒' : [12, 13, 0, -2, 12, [256, 28928, 36832, 61696, 4064, 58432, 44768, 58432, 36832, 62080, 33952, 63712, 0]],
        '饌' : [12, 13, 0, -2, 12, [0, 28384, 35488, 61152, 2176, 61152, 41536, 61408, 33344, 61408, 33344, 60448, 0]],
        '饕' : [12, 13, 0, -2, 12, [448, 30976, 20448, 63776, 19072, 49120, 8320, 32704, 41120, 16256, 8768, 63968, 0]],
        '馗' : [12, 13, 0, -2, 12, [1088, 17536, 20448, 16640, 63456, 21536, 22496, 21536, 21536, 22496, 36864, 36832, 0]],
        '馘' : [12, 13, 0, -2, 12, [128, 41120, 45024, 61568, 16544, 63392, 38208, 63296, 36928, 65184, 37152, 63008, 0]],
        '馥' : [12, 13, 0, -2, 12, [256, 63744, 9184, 65056, 9184, 27168, 42976, 64000, 36832, 64032, 35264, 65056, 0]],
        '馭' : [12, 13, 0, -2, 12, [0, 64480, 40992, 63520, 41504, 64032, 41504, 63808, 2368, 43136, 2368, 13856, 0]],
        '馮' : [12, 13, 0, -2, 12, [0, 8160, 37376, 24512, 4608, 8128, 4608, 8160, 16416, 21792, 38176, 32960, 0]],
        '馼' : [12, 13, 0, -2, 12, [128, 63616, 41088, 65504, 41504, 64032, 41504, 63808, 2368, 43136, 2368, 13856, 0]],
        '駟' : [12, 13, 0, -2, 12, [0, 65504, 43680, 64160, 43680, 64160, 43680, 64736, 2080, 43040, 2080, 14304, 0]],
        '駛' : [12, 13, 0, -2, 12, [128, 63616, 42976, 62624, 42144, 62624, 42976, 63616, 2688, 43264, 2688, 13408, 0]],
        '駝' : [12, 13, 0, -2, 12, [256, 63744, 42976, 62496, 42016, 61952, 41568, 64384, 2560, 43520, 2592, 13280, 0]],
        '駘' : [12, 13, 0, -2, 12, [256, 63744, 41504, 64032, 41952, 63488, 40960, 64480, 2592, 43552, 2592, 13280, 0]],
        '駑' : [12, 13, 0, -2, 12, [8192, 9184, 64544, 18752, 12672, 51808, 32704, 17408, 32736, 32, 21792, 38240, 0]],
        '駭' : [12, 13, 0, -2, 12, [128, 63616, 42976, 63744, 41504, 63808, 41088, 63776, 2624, 43136, 2368, 13856, 0]],
        '駮' : [12, 13, 0, -2, 12, [128, 63616, 42976, 61440, 41536, 62496, 40960, 64064, 2368, 43136, 2368, 13856, 0]],
        '駱' : [12, 13, 0, -2, 12, [256, 63744, 41440, 64032, 42304, 63616, 41280, 64032, 4064, 43552, 2592, 13280, 0]],
        '駲' : [12, 13, 0, -2, 12, [0, 64160, 41632, 64160, 41632, 65504, 41632, 64160, 2720, 43680, 3232, 13344, 0]],
        '駻' : [12, 13, 0, -2, 12, [0, 64480, 41504, 64480, 41504, 64032, 41952, 63488, 3040, 43136, 3040, 12416, 0]],
        '駸' : [12, 13, 0, -2, 12, [0, 63424, 41024, 63456, 41024, 65504, 43040, 63424, 4160, 53824, 4992, 27744, 0]],
        '騁' : [12, 13, 0, -2, 12, [256, 61696, 42976, 62752, 42976, 62752, 45024, 61952, 5088, 54304, 5152, 24768, 0]],
        '騏' : [12, 13, 0, -2, 12, [576, 64064, 42976, 64064, 41920, 64064, 41920, 64064, 4064, 43008, 2624, 13344, 0]],
        '騅' : [12, 13, 0, -2, 12, [256, 63808, 41280, 64480, 42560, 64480, 41536, 64064, 3040, 43584, 2624, 13280, 0]],
        '駢' : [12, 13, 0, -2, 12, [1088, 64064, 41088, 63456, 41536, 62016, 41536, 65504, 4672, 53824, 5184, 25664, 0]],
        '騙' : [12, 13, 0, -2, 12, [0, 64480, 40960, 64480, 41504, 64480, 41472, 64480, 3744, 44000, 2720, 12960, 0]],
        '騫' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 37152, 32704, 4352, 65504, 9344, 32736, 32832, 10816, 19136, 0]],
        '騷' : [12, 13, 0, -2, 12, [0, 63456, 41632, 62784, 41344, 63072, 41216, 63456, 5408, 55232, 4384, 28640, 0]],
        '驅' : [12, 13, 0, -2, 12, [0, 64480, 41472, 64224, 41632, 64224, 41472, 64480, 2720, 44000, 2560, 13280, 0]],
        '驂' : [12, 13, 0, -2, 12, [256, 62016, 45024, 62592, 43296, 65504, 41600, 62560, 6528, 54816, 4288, 26368, 0]],
        '驀' : [12, 13, 0, -2, 12, [4352, 4352, 65504, 8320, 16256, 8320, 65504, 9344, 32736, 32832, 10816, 19136, 0]],
        '驃' : [12, 13, 0, -2, 12, [0, 65504, 41600, 65504, 43680, 65504, 40960, 63456, 4096, 57312, 5440, 27424, 0]],
        '騾' : [12, 13, 0, -2, 12, [0, 63456, 42272, 63456, 42272, 63456, 41536, 62592, 8160, 53504, 5440, 26912, 0]],
        '驕' : [12, 13, 0, -2, 12, [0, 63456, 41216, 63456, 41600, 63168, 43936, 63488, 4064, 43680, 2976, 12320, 0]],
        '驍' : [12, 13, 0, -2, 12, [256, 61696, 45024, 61696, 45024, 62528, 44768, 62528, 8160, 53888, 5280, 26848, 0]],
        '驛' : [12, 13, 0, -2, 12, [0, 65504, 43680, 64160, 45024, 61696, 42976, 62016, 8160, 53504, 6112, 24832, 0]],
        '驗' : [12, 13, 0, -2, 12, [0, 63424, 43040, 63456, 40960, 63456, 42144, 62624, 6112, 53824, 5280, 26912, 0]],
        '驟' : [12, 13, 0, -2, 12, [0, 65504, 43552, 61088, 43680, 65216, 43808, 61888, 5888, 55712, 5440, 26912, 0]],
        '驢' : [12, 13, 0, -2, 12, [448, 63744, 42976, 62624, 42944, 62592, 42976, 64832, 2368, 44000, 2720, 14304, 0]],
        '驥' : [12, 13, 0, -2, 12, [640, 62112, 44736, 62080, 45024, 62752, 42976, 62752, 6112, 53824, 8160, 25632, 0]],
        '驤' : [12, 13, 0, -2, 12, [256, 61696, 45024, 64160, 44768, 62080, 45024, 62080, 8160, 53920, 7232, 26400, 0]],
        '驩' : [12, 13, 0, -2, 12, [576, 62016, 45024, 64160, 44768, 62016, 42976, 64640, 6112, 54400, 5248, 26592, 0]],
        '驫' : [12, 13, 0, -2, 12, [0, 32704, 17408, 32704, 17408, 65504, 41600, 64480, 41600, 64480, 2080, 43680, 0]],
        '驪' : [12, 13, 0, -2, 12, [0, 65248, 40960, 65248, 43680, 61696, 45024, 64160, 4064, 43136, 2208, 14048, 0]],
        '骭' : [12, 13, 0, -2, 12, [0, 31712, 18560, 22656, 18560, 64640, 34784, 30848, 18560, 30848, 18560, 22656, 0]],
        '骰' : [12, 13, 0, -2, 12, [0, 31680, 19008, 23104, 19008, 64608, 33792, 31712, 18976, 31296, 18816, 20064, 0]],
        '骼' : [12, 13, 0, -2, 12, [256, 30976, 18912, 23072, 18752, 64640, 34112, 31264, 20448, 31264, 18976, 23520, 0]],
        '髀' : [12, 13, 0, -2, 12, [128, 28928, 22496, 29984, 22496, 64800, 34080, 30688, 21120, 29824, 24544, 20608, 0]],
        '髏' : [12, 13, 0, -2, 12, [256, 28928, 22496, 29984, 22496, 64800, 34784, 29984, 24544, 29760, 21376, 23648, 0]],
        '髑' : [12, 13, 0, -2, 12, [0, 30688, 21792, 30688, 21504, 65504, 35360, 30624, 22176, 30624, 21152, 22368, 0]],
        '髓' : [12, 13, 0, -2, 12, [128, 28800, 23520, 29952, 21472, 64064, 36320, 28960, 24032, 29984, 22016, 23008, 0]],
        '體' : [12, 13, 0, -2, 12, [640, 29312, 24544, 29344, 24544, 62112, 36832, 29760, 22464, 29312, 21120, 24544, 0]],
        '髞' : [12, 13, 0, -2, 12, [16384, 17344, 62016, 45024, 60064, 2720, 65248, 34944, 61408, 43392, 60096, 40096, 0]],
        '髟' : [12, 13, 0, -2, 12, [0, 64608, 33152, 63488, 32768, 63584, 33152, 64512, 36864, 34816, 37984, 57728, 0]],
        '髢' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31840, 16768, 65120, 18880, 58944, 15424, 58560, 8224, 16352, 0]],
        '髣' : [12, 13, 0, -2, 12, [0, 31840, 16768, 65024, 18528, 62848, 1024, 65504, 2048, 4032, 4160, 25472, 0]],
        '髦' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31840, 16768, 65024, 18528, 18304, 30816, 3968, 63520, 4064, 0]],
        '髯' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31840, 16768, 65120, 18816, 17408, 32704, 17472, 65504, 16448, 0]],
        '髫' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31744, 16480, 64896, 18432, 65504, 4128, 32736, 8256, 16320, 0]],
        '髮' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31840, 16768, 65120, 18816, 17472, 65504, 10304, 51072, 14432, 0]],
        '髴' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31744, 16480, 65472, 4672, 65472, 37376, 65504, 4640, 25280, 0]],
        '髱' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31744, 16480, 64896, 18432, 32736, 41248, 48992, 8192, 16352, 0]],
        '髷' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31744, 16480, 64896, 18688, 32736, 18720, 32736, 18720, 32736, 0]],
        '髻' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31744, 16480, 64896, 17408, 65504, 1024, 32704, 8320, 16256, 0]],
        '鬆' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31840, 16768, 65024, 9664, 62528, 10528, 26912, 41536, 12256, 0]],
        '鬘' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31744, 16480, 65408, 8320, 65504, 37152, 64480, 3584, 61920, 0]],
        '鬚' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31840, 17280, 64640, 2016, 13344, 51168, 1056, 14304, 52256, 0]],
        '鬟' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31744, 16480, 32704, 19008, 65504, 4224, 8064, 57920, 14816, 0]],
        '鬢' : [12, 13, 0, -2, 12, [0, 31840, 16768, 64608, 18816, 17408, 65504, 42144, 16256, 57472, 16256, 49248, 0]],
        '鬣' : [12, 13, 0, -2, 12, [0, 31840, 16768, 31744, 16480, 65472, 19008, 32704, 17472, 30528, 17440, 61344, 0]],
        '鬥' : [12, 13, 0, -2, 12, [0, 48032, 37152, 48032, 37152, 37152, 48032, 32800, 32800, 32800, 32800, 32864, 0]],
        '鬧' : [12, 13, 0, -2, 12, [0, 48032, 37152, 48032, 37152, 48032, 33824, 49056, 33824, 49056, 42144, 42400, 0]],
        '鬨' : [12, 13, 0, -2, 12, [0, 48032, 37152, 48032, 37152, 48032, 32800, 35360, 49056, 35360, 49056, 37152, 0]],
        '鬩' : [12, 13, 0, -2, 12, [0, 48032, 37152, 48032, 37152, 48032, 41120, 48032, 41120, 49056, 35360, 45984, 0]],
        '鬪' : [12, 13, 0, -2, 12, [0, 48032, 37152, 48032, 37152, 48032, 32928, 49120, 43168, 47776, 37024, 47520, 0]],
        '鬮' : [12, 13, 0, -2, 12, [0, 48032, 37152, 48032, 37152, 40864, 41504, 65440, 43680, 49056, 35360, 36768, 0]],
        '鬯' : [12, 13, 0, -2, 12, [1024, 9344, 37152, 35360, 42144, 35360, 65504, 16384, 16832, 32256, 16448, 16320, 0]],
        '鬲' : [12, 13, 0, -2, 12, [0, 65504, 0, 16256, 8320, 16256, 0, 65504, 37152, 57824, 40736, 33888, 0]],
        '魄' : [12, 13, 0, -2, 12, [256, 16896, 20448, 59680, 45024, 43296, 59680, 45024, 42304, 42304, 59680, 2528, 0]],
        '魃' : [12, 13, 0, -2, 12, [8448, 16672, 65504, 43264, 63904, 43680, 43584, 64064, 21664, 22304, 36864, 36832, 0]],
        '魏' : [12, 13, 0, -2, 12, [128, 63744, 10208, 64672, 10208, 29856, 44192, 10208, 63808, 21312, 25888, 39136, 0]],
        '魍' : [12, 13, 0, -2, 12, [8192, 18400, 64672, 42656, 64800, 42976, 42272, 65504, 21792, 21920, 36864, 36832, 0]],
        '魎' : [12, 13, 0, -2, 12, [8192, 20448, 63744, 45024, 63776, 44448, 43872, 63776, 21792, 23904, 36864, 36832, 0]],
        '魑' : [12, 13, 0, -2, 12, [8448, 16640, 65504, 43648, 63776, 45024, 43264, 65504, 21792, 21920, 36864, 36832, 0]],
        '魘' : [12, 13, 0, -2, 12, [0, 65504, 38048, 49120, 42112, 48448, 42272, 32704, 17472, 32704, 4640, 58336, 0]],
        '魴' : [12, 13, 0, -2, 12, [256, 28928, 38880, 8448, 63744, 43488, 63776, 43552, 64032, 32, 43072, 43392, 0]],
        '鮓' : [12, 13, 0, -2, 12, [1024, 29696, 38880, 9472, 63744, 43488, 63744, 43264, 63968, 256, 43264, 43264, 0]],
        '鮃' : [12, 13, 0, -2, 12, [0, 30688, 36992, 8320, 64192, 44192, 63616, 43136, 65504, 128, 43136, 43136, 0]],
        '鮑' : [12, 13, 0, -2, 12, [512, 29184, 37856, 9248, 65440, 43680, 64160, 43936, 64096, 512, 43552, 44000, 0]],
        '鮖' : [12, 13, 0, -2, 12, [0, 30688, 36992, 8320, 63616, 43264, 64480, 44576, 64032, 544, 43552, 44000, 0]],
        '鮗' : [12, 13, 0, -2, 12, [512, 29184, 37856, 9248, 64064, 43392, 65120, 43264, 63680, 0, 43776, 43232, 0]],
        '鮟' : [12, 13, 0, -2, 12, [256, 28928, 38880, 9248, 64800, 43264, 65504, 43584, 64064, 640, 43392, 44640, 0]],
        '鮠' : [12, 13, 0, -2, 12, [0, 29632, 37952, 8320, 65504, 43008, 64448, 43584, 64192, 2560, 43552, 44000, 0]],
        '鮨' : [12, 13, 0, -2, 12, [0, 29280, 37760, 8704, 64032, 43488, 63488, 44000, 64032, 992, 43552, 44000, 0]],
        '鮴' : [12, 13, 0, -2, 12, [128, 29824, 38016, 10368, 65504, 43136, 63616, 43392, 64192, 3232, 43136, 43136, 0]],
        '鯀' : [12, 13, 0, -2, 12, [0, 30688, 37120, 8704, 64576, 43648, 63744, 43552, 65504, 256, 42304, 43296, 0]],
        '鯊' : [12, 13, 0, -2, 12, [256, 17728, 10528, 33536, 16480, 8064, 8448, 65472, 17472, 32704, 21056, 35104, 0]],
        '鮹' : [12, 13, 0, -2, 12, [128, 28800, 37568, 9376, 63616, 44000, 64032, 44000, 64032, 992, 43552, 43616, 0]],
        '鯆' : [12, 13, 0, -2, 12, [256, 28960, 40928, 8448, 65504, 43296, 65504, 43296, 65504, 2336, 43296, 43360, 0]],
        '鯏' : [12, 13, 0, -2, 12, [0, 29088, 40480, 8864, 65440, 43680, 64160, 44704, 64416, 672, 43552, 43616, 0]],
        '鯑' : [12, 13, 0, -2, 12, [0, 30304, 37248, 9792, 63776, 45024, 64128, 45024, 64160, 672, 43744, 43136, 0]],
        '鯒' : [12, 13, 0, -2, 12, [0, 30656, 36928, 8320, 65504, 43296, 65504, 43296, 65504, 2336, 43296, 43360, 0]],
        '鯣' : [12, 13, 0, -2, 12, [0, 30688, 37920, 10208, 64544, 42976, 61952, 45024, 62112, 1184, 43296, 43712, 0]],
        '鯢' : [12, 13, 0, -2, 12, [128, 28928, 38496, 9248, 65376, 42016, 64544, 42976, 64832, 320, 43584, 43616, 0]],
        '鯤' : [12, 13, 0, -2, 12, [0, 30688, 37920, 10208, 64544, 42976, 64640, 42112, 65504, 1152, 42144, 44768, 0]],
        '鯔' : [12, 13, 0, -2, 12, [0, 29344, 37536, 9536, 64832, 43680, 64160, 45024, 63776, 4064, 43296, 45024, 0]],
        '鯡' : [12, 13, 0, -2, 12, [64, 28992, 38752, 8512, 63808, 44896, 63808, 43328, 65344, 352, 43584, 43584, 0]],
        '鰺' : [12, 13, 0, -2, 12, [256, 29248, 40928, 9344, 63776, 45024, 64128, 44128, 63872, 1568, 43200, 43776, 0]],
        '鯲' : [12, 13, 0, -2, 12, [1024, 29888, 40736, 9216, 64640, 42816, 64800, 42240, 63744, 2432, 41280, 42528, 0]],
        '鯱' : [12, 13, 0, -2, 12, [448, 28928, 40928, 10784, 65472, 43520, 64480, 43008, 64384, 2688, 41632, 44256, 0]],
        '鯰' : [12, 13, 0, -2, 12, [0, 29120, 37408, 9216, 64480, 43008, 64480, 43040, 63808, 128, 43552, 43936, 0]],
        '鰕' : [12, 13, 0, -2, 12, [0, 28384, 43552, 18976, 65248, 43008, 63968, 44576, 63648, 3744, 43200, 43808, 0]],
        '鰔' : [12, 13, 0, -2, 12, [128, 28832, 38880, 9344, 65408, 42144, 65440, 42304, 65344, 1088, 43168, 43808, 0]],
        '鰉' : [12, 13, 0, -2, 12, [128, 28928, 37856, 8736, 64480, 43552, 64032, 45024, 63616, 992, 43136, 45024, 0]],
        '鰓' : [12, 13, 0, -2, 12, [0, 30688, 38048, 10208, 64672, 42144, 65504, 42240, 64576, 544, 43520, 43456, 0]],
        '鰌' : [12, 13, 0, -2, 12, [0, 30144, 38944, 10208, 64128, 45024, 64160, 44256, 63520, 4064, 43040, 45024, 0]],
        '鰆' : [12, 13, 0, -2, 12, [512, 29184, 40928, 8704, 65504, 42112, 65504, 43072, 65504, 2112, 43072, 44992, 0]],
        '鰈' : [12, 13, 0, -2, 12, [128, 29344, 38880, 8864, 64224, 43520, 64480, 43136, 65504, 384, 43712, 44192, 0]],
        '鰒' : [12, 13, 0, -2, 12, [256, 28928, 37856, 9760, 64480, 43552, 65504, 43520, 65504, 544, 43456, 44576, 0]],
        '鰊' : [12, 13, 0, -2, 12, [256, 28928, 40928, 8448, 65504, 43296, 64864, 43296, 65504, 768, 42304, 43296, 0]],
        '鰄' : [12, 13, 0, -2, 12, [128, 28832, 38880, 9344, 65408, 42144, 65440, 42304, 64832, 1600, 43680, 43296, 0]],
        '鰮' : [12, 13, 0, -2, 12, [0, 29664, 37536, 9056, 64032, 44000, 63488, 45024, 64160, 2720, 43680, 45024, 0]],
        '鰛' : [12, 13, 0, -2, 12, [0, 29664, 37408, 9184, 64032, 44000, 63488, 45024, 64160, 2720, 43680, 45024, 0]],
        '鰥' : [12, 13, 0, -2, 12, [0, 28640, 43680, 19104, 65504, 43264, 64320, 44320, 63744, 832, 42272, 43264, 0]],
        '鰤' : [12, 13, 0, -2, 12, [512, 30176, 40576, 10880, 64480, 44704, 63648, 44704, 64160, 2720, 44704, 43136, 0]],
        '鰡' : [12, 13, 0, -2, 12, [0, 32224, 37024, 4768, 64800, 43360, 63488, 45024, 63776, 4064, 43296, 45024, 0]],
        '鰰' : [12, 13, 0, -2, 12, [128, 32384, 37344, 8352, 65184, 42464, 62624, 44704, 62944, 1152, 42112, 42112, 0]],
        '鱇' : [12, 13, 0, -2, 12, [128, 28800, 38880, 9376, 65504, 42144, 65504, 42112, 64928, 2752, 44192, 43392, 0]],
        '鰲' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 9280, 64832, 18816, 40544, 8704, 32704, 17472, 32704, 35104, 0]],
        '鱆' : [12, 13, 0, -2, 12, [128, 28800, 38880, 8512, 65504, 43552, 64480, 43552, 64480, 128, 45024, 43136, 0]],
        '鰾' : [12, 13, 0, -2, 12, [0, 32736, 37504, 12256, 64160, 45024, 61440, 45024, 61440, 4064, 42304, 43808, 0]],
        '鱚' : [12, 13, 0, -2, 12, [128, 28800, 38880, 8320, 64480, 43552, 64480, 43328, 65504, 544, 43552, 44000, 0]],
        '鱠' : [12, 13, 0, -2, 12, [0, 29632, 37920, 11232, 63488, 45024, 64160, 45024, 64032, 992, 43552, 44000, 0]],
        '鱧' : [12, 13, 0, -2, 12, [640, 29312, 40928, 10912, 65504, 43680, 65504, 42048, 63424, 640, 43648, 45024, 0]],
        '鱶' : [12, 13, 0, -2, 12, [1088, 29824, 40928, 8448, 65504, 42048, 65504, 42048, 63424, 1376, 42112, 44640, 0]],
        '鱸' : [12, 13, 0, -2, 12, [448, 28928, 38880, 9376, 65472, 42112, 65504, 42304, 64832, 992, 43680, 45024, 0]],
        '鳧' : [12, 13, 0, -2, 12, [1024, 2048, 32704, 16448, 32704, 16384, 32736, 19104, 37472, 7936, 4384, 57824, 0]],
        '鳬' : [12, 13, 0, -2, 12, [1024, 2048, 32704, 16448, 32704, 16448, 32736, 32, 8032, 4352, 4384, 57824, 0]],
        '鳰' : [12, 13, 0, -2, 12, [256, 512, 59328, 9280, 10176, 9280, 10208, 21504, 22496, 20512, 35488, 35552, 0]],
        '鴉' : [12, 13, 0, -2, 12, [256, 64000, 22464, 21568, 22464, 62528, 6112, 13312, 22496, 36896, 5408, 13664, 0]],
        '鴈' : [12, 13, 0, -2, 12, [0, 65504, 37376, 40896, 43072, 44992, 59456, 45024, 43008, 45024, 40992, 46432, 0]],
        '鳫' : [12, 13, 0, -2, 12, [0, 65504, 33792, 49088, 41024, 49088, 41024, 49120, 40960, 49120, 32800, 43712, 0]],
        '鴃' : [12, 13, 0, -2, 12, [8448, 16640, 65504, 35104, 63776, 35104, 65504, 33024, 63744, 2688, 43584, 44064, 0]],
        '鴆' : [12, 13, 0, -2, 12, [8448, 8704, 10176, 64576, 42944, 42048, 10208, 21504, 22496, 22560, 38176, 34144, 0]],
        '鴪' : [12, 13, 0, -2, 12, [8448, 8704, 10176, 64576, 34752, 46144, 6112, 21504, 22496, 20512, 35488, 35552, 0]],
        '鴦' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 17472, 65504, 9344, 51296, 16256, 8320, 16352, 16416, 38240, 0]],
        '鶯' : [12, 13, 0, -2, 12, [4352, 21792, 38208, 10880, 17472, 65504, 41120, 49056, 8320, 16352, 16416, 38240, 0]],
        '鴣' : [12, 13, 0, -2, 12, [16640, 16896, 18368, 62528, 18368, 17472, 18400, 62464, 38880, 36896, 38176, 62816, 0]],
        '鴟' : [12, 13, 0, -2, 12, [256, 61952, 42944, 42048, 42944, 62528, 42976, 41984, 38880, 53280, 1312, 62816, 0]],
        '鵄' : [12, 13, 0, -2, 12, [256, 61952, 10176, 17472, 38848, 62528, 18400, 17408, 63456, 16416, 17696, 62816, 0]],
        '鴕' : [12, 13, 0, -2, 12, [8448, 16640, 65504, 34848, 63520, 35328, 64096, 33664, 64000, 2560, 43552, 48096, 0]],
        '鴒' : [12, 13, 0, -2, 12, [256, 25088, 38848, 1088, 63424, 1088, 63456, 21504, 22496, 24608, 19104, 19168, 0]],
        '鵁' : [12, 13, 0, -2, 12, [8448, 8704, 65472, 1088, 22464, 35904, 2016, 1024, 22496, 8224, 21792, 34144, 0]],
        '鴿' : [12, 13, 0, -2, 12, [256, 25088, 38848, 1088, 1984, 62528, 2016, 1024, 63456, 36896, 38176, 62816, 0]],
        '鴾' : [12, 13, 0, -2, 12, [256, 16896, 18368, 37952, 63424, 9280, 42976, 62464, 10208, 61472, 10912, 10976, 0]],
        '鵆' : [12, 13, 0, -2, 12, [1024, 10464, 24320, 37120, 7936, 12768, 24352, 53280, 24352, 16672, 21792, 21856, 0]],
        '鵈' : [12, 13, 0, -2, 12, [256, 64000, 38848, 62528, 38848, 37952, 63456, 37888, 38880, 61472, 38176, 5472, 0]],
        '鵝' : [12, 13, 0, -2, 12, [4224, 62720, 22496, 21024, 65504, 21024, 21472, 30208, 52192, 18464, 21792, 58720, 0]],
        '鵞' : [12, 13, 0, -2, 12, [256, 63776, 9184, 64800, 8512, 14720, 57920, 16288, 8320, 16352, 16416, 38240, 0]],
        '鵤' : [12, 13, 0, -2, 12, [128, 28928, 37856, 8736, 64480, 43552, 64480, 43520, 64480, 34848, 35488, 39584, 0]],
        '鵑' : [12, 13, 0, -2, 12, [256, 61952, 38848, 62528, 1984, 62528, 38880, 62464, 38880, 61472, 38176, 46432, 0]],
        '鵐' : [12, 13, 0, -2, 12, [256, 512, 63424, 17472, 22464, 21568, 22496, 60416, 18400, 16416, 64160, 2784, 0]],
        '鵙' : [12, 13, 0, -2, 12, [256, 61952, 38848, 62528, 38848, 62528, 38880, 37888, 63456, 32, 21792, 38240, 0]],
        '鵲' : [12, 13, 0, -2, 12, [20736, 20992, 65472, 21568, 22464, 64576, 2016, 62464, 38880, 61472, 38176, 62816, 0]],
        '鶉' : [12, 13, 0, -2, 12, [8448, 8704, 65472, 1088, 63424, 37952, 63456, 5120, 10208, 63520, 10912, 27360, 0]],
        '鶇' : [12, 13, 0, -2, 12, [8320, 8448, 64480, 8736, 64480, 43552, 64480, 43520, 64480, 8224, 27296, 43744, 0]],
        '鶫' : [12, 13, 0, -2, 12, [8320, 8448, 64480, 8736, 64480, 43552, 56288, 43520, 64480, 8224, 27296, 43744, 0]],
        '鵯' : [12, 13, 0, -2, 12, [8320, 16640, 64480, 43552, 64480, 43552, 44000, 64000, 21472, 36896, 64800, 5472, 0]],
        '鵺' : [12, 13, 0, -2, 12, [8320, 8448, 65504, 21024, 24544, 21024, 54240, 27136, 19424, 17440, 19104, 29344, 0]],
        '鶚' : [12, 13, 0, -2, 12, [128, 63744, 44000, 64032, 992, 64032, 992, 65024, 17376, 63520, 36128, 13664, 0]],
        '鶤' : [12, 13, 0, -2, 12, [128, 64768, 34784, 64032, 44000, 64032, 44000, 43520, 64480, 8224, 64800, 9568, 0]],
        '鶩' : [12, 13, 0, -2, 12, [512, 64000, 5088, 9280, 64832, 20864, 37472, 16256, 8320, 16352, 16416, 38240, 0]],
        '鶲' : [12, 13, 0, -2, 12, [128, 22784, 19424, 42528, 19424, 64032, 992, 65024, 42976, 26656, 46368, 28000, 0]],
        '鷄' : [12, 13, 0, -2, 12, [256, 61952, 44992, 9280, 22464, 42048, 65504, 9216, 65504, 8224, 21792, 35168, 0]],
        '鷁' : [12, 13, 0, -2, 12, [256, 29184, 6080, 35904, 63424, 1088, 30688, 5120, 36832, 63520, 43680, 65184, 0]],
        '鶻' : [12, 13, 0, -2, 12, [256, 61952, 38848, 46144, 38848, 64576, 34784, 62464, 38880, 61472, 38176, 46432, 0]],
        '鶸' : [12, 13, 0, -2, 12, [128, 55552, 19424, 18976, 56288, 37408, 56288, 18944, 56288, 26656, 19104, 55968, 0]],
        '鶺' : [12, 13, 0, -2, 12, [256, 55808, 18368, 56384, 18368, 42048, 65504, 37888, 63456, 36896, 62752, 38240, 0]],
        '鷆' : [12, 13, 0, -2, 12, [16512, 22784, 25568, 18976, 31712, 43552, 48096, 43520, 65504, 32, 21152, 35488, 0]],
        '鷏' : [12, 13, 0, -2, 12, [8320, 8448, 65504, 8736, 31712, 18976, 31712, 18944, 65504, 32, 21152, 35488, 0]],
        '鷂' : [12, 13, 0, -2, 12, [128, 15616, 51168, 10784, 46048, 49696, 31712, 41472, 64480, 8224, 43680, 64224, 0]],
        '鷙' : [12, 13, 0, -2, 12, [8448, 8448, 65472, 8512, 64832, 19040, 65440, 8320, 65504, 32, 21792, 38240, 0]],
        '鷓' : [12, 13, 0, -2, 12, [8320, 8448, 64480, 33312, 44000, 65056, 44000, 43520, 48096, 32, 21792, 38240, 0]],
        '鷸' : [12, 13, 0, -2, 12, [128, 63744, 5088, 64032, 21472, 37408, 64480, 43520, 39904, 59424, 43680, 60064, 0]],
        '鷦' : [12, 13, 0, -2, 12, [20736, 20992, 65472, 42048, 63424, 42048, 63456, 41984, 65504, 32, 21792, 38240, 0]],
        '鷭' : [12, 13, 0, -2, 12, [128, 63744, 44000, 8736, 65504, 27168, 42976, 64000, 44000, 63520, 43680, 64160, 0]],
        '鷯' : [12, 13, 0, -2, 12, [8448, 8704, 65472, 21568, 63424, 40000, 63456, 37888, 63456, 8224, 43680, 43744, 0]],
        '鷽' : [12, 13, 0, -2, 12, [2560, 25792, 19008, 25792, 19008, 65504, 34848, 49056, 8320, 16352, 16416, 38240, 0]],
        '鸚' : [12, 13, 0, -2, 12, [128, 63744, 44000, 64032, 44000, 64032, 21472, 43520, 65504, 20512, 25888, 40288, 0]],
        '鸛' : [12, 13, 0, -2, 12, [20608, 20736, 65504, 43552, 64480, 21024, 64480, 41472, 64480, 40992, 43680, 64224, 0]],
        '鸞' : [12, 13, 0, -2, 12, [1024, 24384, 41120, 20032, 41120, 61152, 19008, 16256, 8320, 16352, 16416, 38240, 0]],
        '鹵' : [12, 13, 0, -2, 12, [1024, 2016, 1024, 32736, 17440, 21152, 18720, 26208, 18720, 21664, 16928, 32736, 0]],
        '鹹' : [12, 13, 0, -2, 12, [16512, 28832, 20448, 63616, 36736, 55456, 43936, 55968, 43968, 55360, 34976, 64288, 0]],
        '鹽' : [12, 13, 0, -2, 12, [512, 64000, 41952, 64640, 35808, 64160, 41824, 41632, 65504, 19008, 19008, 65504, 0]],
        '麁' : [12, 13, 0, -2, 12, [0, 16256, 16640, 65504, 18688, 32736, 18720, 32736, 20736, 40416, 37120, 15584, 0]],
        '麈' : [12, 13, 0, -2, 12, [512, 32736, 18688, 32736, 20736, 32224, 16896, 32736, 33280, 40896, 512, 32736, 0]],
        '麋' : [12, 13, 0, -2, 12, [512, 512, 32736, 18688, 32736, 20736, 32224, 21056, 19072, 49120, 37440, 8736, 0]],
        '麌' : [12, 13, 0, -2, 12, [512, 32736, 18688, 32736, 20736, 32224, 18560, 28608, 41024, 49120, 2304, 28896, 0]],
        '麒' : [12, 13, 0, -2, 12, [8768, 8768, 65504, 41536, 64448, 43584, 64448, 41536, 65504, 40960, 43328, 64032, 0]],
        '麕' : [12, 13, 0, -2, 12, [512, 32736, 18688, 32736, 37120, 64992, 17440, 32736, 19744, 29856, 17440, 32736, 0]],
        '麑' : [12, 13, 0, -2, 12, [1024, 65504, 37120, 65504, 37120, 48096, 8320, 15232, 8320, 16256, 2336, 61920, 0]],
        '麝' : [12, 13, 0, -2, 12, [512, 32736, 18688, 32736, 37120, 64992, 18464, 31712, 18464, 64032, 18720, 39008, 0]],
        '麥' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 9344, 13696, 52832, 13696, 50272, 8064, 24704, 3840, 61664, 0]],
        '麩' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 43136, 28800, 44000, 41088, 14464, 51520, 10784, 12288, 53216, 0]],
        '麸' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8320, 63616, 10208, 63616, 20608, 20800, 42528, 12288, 53216, 0]],
        '麪' : [12, 13, 0, -2, 12, [8192, 10208, 63616, 43648, 29408, 43680, 41632, 15328, 51744, 10432, 12288, 53216, 0]],
        '麭' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 44064, 30624, 43680, 41888, 14944, 51712, 10720, 12288, 53216, 0]],
        '靡' : [12, 13, 0, -2, 12, [512, 512, 32736, 20736, 32736, 21824, 20832, 17664, 32224, 34048, 47584, 2304, 0]],
        '黌' : [12, 13, 0, -2, 12, [2560, 25792, 19008, 17472, 65504, 37152, 32704, 4352, 65504, 9344, 16256, 49248, 0]],
        '黎' : [12, 13, 0, -2, 12, [512, 64000, 9184, 64160, 9376, 28960, 44608, 13696, 50272, 11904, 13696, 52320, 0]],
        '黏' : [12, 13, 0, -2, 12, [256, 63744, 8448, 64992, 12544, 26880, 41216, 22496, 44064, 29728, 44064, 26592, 0]],
        '黐' : [12, 13, 0, -2, 12, [256, 61696, 12256, 62080, 10528, 31392, 45024, 20736, 45024, 31264, 43936, 26720, 0]],
        '黔' : [12, 13, 0, -2, 12, [0, 63936, 43552, 56320, 35808, 43008, 63488, 9184, 63520, 32, 43072, 43392, 0]],
        '黜' : [12, 13, 0, -2, 12, [128, 63616, 43680, 55968, 35488, 44000, 63616, 8320, 64160, 672, 43680, 44000, 0]],
        '點' : [12, 13, 0, -2, 12, [256, 63744, 43264, 55776, 35072, 43264, 63744, 9184, 64032, 544, 43552, 44000, 0]],
        '黝' : [12, 13, 0, -2, 12, [128, 62592, 42112, 55776, 35488, 43680, 62624, 9376, 62752, 2336, 43552, 44736, 0]],
        '黠' : [12, 13, 0, -2, 12, [128, 63616, 45024, 55424, 34944, 44000, 63488, 8192, 64480, 544, 43552, 44000, 0]],
        '黥' : [12, 13, 0, -2, 12, [128, 63616, 45024, 55296, 35808, 43552, 64032, 9184, 63616, 672, 42144, 43392, 0]],
        '黨' : [12, 13, 0, -2, 12, [1024, 9344, 65504, 37152, 32704, 21824, 17472, 32704, 1024, 65504, 21056, 35104, 0]],
        '黯' : [12, 13, 0, -2, 12, [128, 63616, 45024, 55872, 35392, 45024, 63488, 9184, 64032, 992, 43552, 44000, 0]],
        '黴' : [12, 13, 0, -2, 12, [1152, 5248, 21728, 40864, 32, 16288, 21664, 57248, 17472, 24384, 16544, 21792, 0]],
        '黶' : [12, 13, 0, -2, 12, [0, 65504, 38048, 49120, 42112, 48448, 42272, 32704, 17472, 65504, 21056, 35104, 0]],
        '黷' : [12, 13, 0, -2, 12, [256, 63744, 45024, 55552, 36832, 43680, 65504, 9280, 65472, 1088, 44992, 43104, 0]],
        '黹' : [12, 13, 0, -2, 12, [2560, 19008, 10880, 65504, 9344, 17472, 65504, 17472, 21824, 21824, 25920, 17600, 0]],
        '黻' : [12, 13, 0, -2, 12, [20736, 55584, 21472, 64768, 10496, 42656, 64160, 43680, 64064, 44096, 43168, 43808, 0]],
        '黼' : [12, 13, 0, -2, 12, [20608, 55456, 21472, 64640, 11232, 42656, 64480, 43680, 64480, 43680, 43680, 43680, 0]],
        '黽' : [12, 13, 0, -2, 12, [0, 32704, 19008, 31680, 2560, 64480, 35360, 64480, 35360, 64480, 2048, 4064, 0]],
        '鼇' : [12, 13, 0, -2, 12, [8704, 8704, 64480, 8768, 64832, 18560, 65504, 19008, 64448, 35392, 64064, 4064, 0]],
        '鼈' : [12, 13, 0, -2, 12, [8704, 43520, 65504, 43584, 64832, 43136, 32736, 19008, 64448, 35392, 64064, 4064, 0]],
        '皷' : [12, 13, 0, -2, 12, [8320, 8320, 65504, 8864, 64128, 992, 64032, 35488, 64672, 21568, 20640, 64288, 0]],
        '鼕' : [12, 13, 0, -2, 12, [8320, 8320, 64480, 8320, 65504, 19008, 31104, 12128, 61568, 7936, 57568, 3584, 0]],
        '鼡' : [12, 13, 0, -2, 12, [18496, 9344, 0, 32704, 17472, 32704, 17472, 17472, 32704, 17472, 33824, 33824, 0]],
        '鼬' : [12, 13, 0, -2, 12, [128, 56448, 34784, 56480, 33952, 65504, 42144, 62624, 42976, 62464, 42016, 46048, 0]],
        '鼾' : [12, 13, 0, -2, 12, [4096, 9184, 31872, 17536, 31872, 17536, 65504, 37504, 65152, 10368, 10368, 51328, 0]],
        '齊' : [12, 13, 0, -2, 12, [1024, 1024, 65504, 2560, 62944, 21824, 42016, 16320, 8256, 16320, 16448, 16448, 0]],
        '齒' : [12, 13, 0, -2, 12, [1024, 10176, 9216, 65504, 4352, 43680, 32800, 65504, 37152, 43680, 32800, 65504, 0]],
        '齔' : [12, 13, 0, -2, 12, [8704, 47616, 41472, 65120, 43904, 55808, 35328, 64000, 43520, 55840, 35360, 64480, 0]],
        '齣' : [12, 13, 0, -2, 12, [8704, 47616, 41952, 64544, 43040, 56224, 35488, 64160, 43936, 55328, 34880, 63872, 0]],
        '齟' : [12, 13, 0, -2, 12, [8192, 48096, 41504, 65056, 44000, 55840, 35360, 64480, 43552, 55840, 35360, 65504, 0]],
        '齠' : [12, 13, 0, -2, 12, [8192, 48096, 41248, 64800, 43552, 56000, 34816, 64480, 43552, 55840, 35360, 64480, 0]],
        '齡' : [12, 13, 0, -2, 12, [8192, 47552, 41504, 64512, 44000, 55296, 34816, 64480, 43296, 55584, 35168, 63744, 0]],
        '齦' : [12, 13, 0, -2, 12, [8192, 48096, 41504, 65504, 43552, 55840, 35808, 64128, 43680, 55968, 35392, 64288, 0]],
        '齧' : [12, 13, 0, -2, 12, [8192, 10208, 63648, 9504, 63424, 9216, 65504, 21824, 32704, 19008, 21824, 32704, 0]],
        '齬' : [12, 13, 0, -2, 12, [8192, 48096, 41088, 65504, 43296, 55584, 35808, 63488, 44000, 55840, 35360, 64480, 0]],
        '齪' : [12, 13, 0, -2, 12, [8192, 48096, 41504, 65056, 44000, 55424, 34944, 64224, 43648, 55936, 35456, 65504, 0]],
        '齷' : [12, 13, 0, -2, 12, [8192, 49120, 43040, 65504, 43008, 57312, 35392, 64480, 43136, 56288, 34944, 65504, 0]],
        '齲' : [12, 13, 0, -2, 12, [8192, 48096, 41088, 65504, 43680, 56288, 34944, 64480, 43680, 56032, 35360, 64096, 0]],
        '齶' : [12, 13, 0, -2, 12, [8192, 48096, 41632, 65504, 43008, 56288, 34816, 65504, 43264, 56288, 35360, 63680, 0]],
        '龕' : [12, 13, 0, -2, 12, [0, 16256, 16448, 49056, 8320, 65504, 18944, 64480, 34848, 64480, 35328, 39904, 0]],
        '龜' : [12, 13, 0, -2, 12, [0, 16128, 16896, 65472, 19008, 64448, 10752, 64480, 10912, 65504, 10752, 61408, 0]],
        '龠' : [12, 13, 0, -2, 12, [0, 16256, 16448, 65504, 43680, 61152, 0, 65504, 37152, 65504, 37152, 37216, 0]],
        '堯' : [12, 13, 0, -2, 12, [1024, 1024, 32704, 1024, 65504, 8320, 64480, 8320, 65504, 4608, 4640, 58336, 0]],
        '槇' : [12, 13, 0, -2, 12, [1024, 9312, 10112, 62496, 9184, 10816, 27584, 27200, 45024, 40960, 8768, 11296, 0]],
        '遙' : [12, 13, 0, -2, 12, [0, 17376, 11296, 320, 34176, 24064, 992, 59648, 12256, 8448, 20768, 36832, 0]],
        '瑤' : [12, 13, 0, -2, 12, [0, 480, 63008, 8352, 8896, 12032, 61920, 9344, 10208, 8320, 13472, 51168, 0]],
        '凜' : [12, 13, 0, -2, 12, [256, 256, 40928, 19104, 2976, 2080, 4064, 256, 24544, 17152, 34112, 39200, 0]],
        '熙' : [12, 13, 0, -2, 12, [0, 64480, 37408, 47648, 44000, 43520, 47616, 37408, 64480, 0, 21056, 35104, 0]],
    }

    @staticmethod
    def hex_to_bin(hex_, BBw:int):
        if BBw > 8:
            return np.array([char == "1" for char in bin(int(hex_, 16))[2:].zfill(16)])[0:BBw]
        else:
            return np.array([char == "1" for char in bin(int(hex_, 16))[2:].zfill(8)])[0:BBw]

    @staticmethod
    def int_to_bin(int_, BBw:int):
        if BBw > 8:
            return np.array([char == "1" for char in bin(int_)[2:].zfill(16)])[0:BBw]
        else:
            return np.array([char == "1" for char in bin(int_)[2:].zfill(8)])[0:BBw]

    @staticmethod
    def glyph_to_array(glyph, BBw:int):
        return np.stack([Text2D.hex_to_bin(line, BBw) for line in glyph.split()])

    @staticmethod
    def glyph_compress(glyph):
        return [int(line, 16) for line in glyph.split()]

    @staticmethod
    def compress_to_array(compress, BBw:int):
        return np.stack([Text2D.int_to_bin(line, BBw) for line in compress])

    @staticmethod
    def save_griphs():
        JAPANESE = True

        if JAPANESE:
            f = open('mplus_j12r.txt', 'r', encoding="utf-8")
        else:
            f = open('mplus_h12r.txt', 'r', encoding="utf-8")
        datalist = f.readlines()
        i = 0
        charnum = 0
        state = -1
        while i < len(datalist):
            line = datalist[i]
            if len(line) > 5 and line[0:5] == "CHARS":
                state = 0
            elif len(line) > 9 and line[0:9] == "STARTCHAR":
                if state == 0:
                    letter = line[12:16]
                    if JAPANESE:
                        b = b'\033$B' + bytes.fromhex(letter)
                        if charnum == 292:
                            char = 'ゔ'
                        elif charnum == 525:
                            char = '①'
                        elif charnum == 526:
                            char = '②'
                        elif charnum == 527:
                            char = '③'
                        elif charnum == 528:
                            char = '④'
                        elif charnum == 529:
                            char = '⑤'
                        elif charnum == 530:
                            char = '⑥'
                        elif charnum == 531:
                            char = '⑦'
                        elif charnum == 532:
                            char = '⑧'
                        elif charnum == 533:
                            char = '⑨'
                        elif charnum == 534:
                            char = '⑩'
                        elif charnum == 535:
                            char = '⑪'
                        elif charnum == 536:
                            char = '⑫'
                        elif charnum == 537:
                            char = '⑬'
                        elif charnum == 538:
                            char = '⑭'
                        elif charnum == 539:
                            char = '⑮'
                        elif charnum == 540:
                            char = '⑯'
                        elif charnum == 541:
                            char = '⑰'
                        elif charnum == 542:
                            char = '⑱'
                        elif charnum == 543:
                            char = '⑲'
                        elif charnum == 544:
                            char = '⑳'
                        elif charnum == 545:
                            char = 'Ⅰ'
                        elif charnum == 546:
                            char = 'Ⅱ'
                        elif charnum == 547:
                            char = 'Ⅲ'
                        elif charnum == 548:
                            char = 'Ⅳ'
                        elif charnum == 549:
                            char = 'Ⅴ'
                        elif charnum == 550:
                            char = 'Ⅵ'
                        elif charnum == 551:
                            char = 'Ⅶ'
                        elif charnum == 552:
                            char = 'Ⅷ'
                        elif charnum == 553:
                            char = 'Ⅸ'
                        elif charnum == 554:
                            char = 'Ⅹ'
                        elif charnum == 555:
                            char = '㍉'
                        elif charnum == 556:
                            char = '㌔'
                        elif charnum == 557:
                            char = '㌢'
                        elif charnum == 558:
                            char = '㍍'
                        elif charnum == 559:
                            char = '㌘'
                        elif charnum == 560:
                            char = '㌧'
                        elif charnum == 561:
                            char = '㌃'
                        elif charnum == 562:
                            char = '㌶'
                        elif charnum == 563:
                            char = '㍑'
                        elif charnum == 564:
                            char = '㍗'
                        elif charnum == 565:
                            char = '㌍'
                        elif charnum == 566:
                            char = '㌦'
                        elif charnum == 567:
                            char = '㌣'
                        elif charnum == 568:
                            char = '㌫'
                        elif charnum == 569:
                            char = '㍊'
                        elif charnum == 570:
                            char = '㌻'
                        elif charnum == 571:
                            char = '㎜'
                        elif charnum == 572:
                            char = '㎝'
                        elif charnum == 573:
                            char = '㎞'
                        elif charnum == 574:
                            char = '㎎'
                        elif charnum == 575:
                            char = '㎏'
                        elif charnum == 576:
                            char = '㏄'
                        elif charnum == 577:
                            char = '㎡'
                        elif charnum == 578:
                            char = '㍻'
                        elif charnum == 579:
                            char = '〝'
                        elif charnum == 580:
                            char = '〟'
                        elif charnum == 581:
                            char = '№'
                        elif charnum == 582:
                            char = '㏍'
                        elif charnum == 583:
                            char = '℡'
                        elif charnum == 584:
                            char = '㊤'
                        elif charnum == 585:
                            char = '㊥'
                        elif charnum == 586:
                            char = '㊦'
                        elif charnum == 587:
                            char = '㊧'
                        elif charnum == 588:
                            char = '㊨'
                        elif charnum == 589:
                            char = '㈱'
                        elif charnum == 590:
                            char = '㈲'
                        elif charnum == 591:
                            char = '㈹'
                        elif charnum == 592:
                            char = '㍾'
                        elif charnum == 593:
                            char = '㍽'
                        elif charnum == 594:
                            char = '㍼'
                        elif charnum == 595:
                            char = '≒'
                        elif charnum == 596:
                            char = '≡'
                        elif charnum == 597:
                            char = '∫'
                        elif charnum == 598:
                            char = '∮'
                        elif charnum == 599:
                            char = '∑'
                        elif charnum == 600:
                            char = '√'
                        elif charnum == 601:
                            char = '⊥'
                        elif charnum == 602:
                            char = '∠'
                        elif charnum == 603:
                            char = '∟'
                        elif charnum == 604:
                            char = '⊿'
                        elif charnum == 605:
                            char = '∵'
                        elif charnum == 606:
                            char = '∩'
                        elif charnum == 607:
                            char = '∪'
                        else:
                            char = b.decode('iso2022_jp')
                    else:
                        char = chr(int(letter, 16))
                    state = 1
                else:
                    print("ERROR: STARTCHAR with state: {}".format(state))
                    break
            elif len(line) > 6 and line[0:6] == "DWIDTH":
                if state == 1:
                    Dwx = int(line.split()[1])
                    state = 2
                else:
                    print("ERROR: DWIDTH with state: {}".format(state))
                    break
            elif len(line) > 3 and line[0:3] == "BBX":
                if state == 2:
                    tmp = line.split()
                    BBw, BBh, BBxoff, BByoff = int(tmp[1]), int(tmp[2]), int(tmp[3]), int(tmp[4])
                    state = 3
                else:
                    print("ERROR: BBX with state: {}".format(state))
                    break
            elif len(line) > 5 and line[0:6] == "BITMAP":
                if state == 3:
                    bitmap = ""
                    state = 4
                else:
                    print("ERROR: BITMAP with state: {}".format(state))
                    break
            elif len(line) > 6 and line[0:7] == "ENDCHAR":
                if state == 4:
                    charnum += 1
                    # char : [BBw, BBh, BBxoff, BByoff, Dwx, BITMAP]
                    if JAPANESE:
                        with open("fontj.txt", mode='a') as f:
                            f.write('\n')
                            f.write("'{}' : {},".format(char, [BBw, BBh, BBxoff, BByoff, Dwx, Text2D.glyph_compress(bitmap)]))
                    else:
                        with open("fonth.txt", mode='a') as f:
                            f.write('\n')
                            f.write("'{}' : {},".format(char, [BBw, BBh, BBxoff, BByoff, Dwx, Text2D.glyph_compress(bitmap)]))
                    state = 0
                else:
                    print("ERROR: ENDCHAR with state: {}".format(state))
                    break
            else:
                if state == 4:
                    bitmap += line
                elif state == -1:
                    print(line, end='')
            i += 1

    @classmethod
    def text(cls, layer:Layer, x0:int, y0:int, txt, c, size=2, text_align='l'):
        # char : [BBw, BBh, BBxoff, BByoff, Dwx, BITMAP]
        zoom = round(size * Settings.SSAA_RATIO)
        offset = 0
        if text_align == 'c':
            offset = sum([cls.font_dic[char][4] for char in txt])
            offset = -round(zoom * offset / 2)
        elif text_align == 'r':
            offset = sum([cls.font_dic[char][4] for char in txt])
            offset = -zoom * offset
        xnow = x0 * Settings.SSAA_RATIO + offset
        ynow = y0 * Settings.SSAA_RATIO

        for charid in range(len(txt)):
            char = cls.font_dic[txt[charid]]
            BBw = char[0]
            BBh = char[1]
            BBxoff = char[2]
            BByoff = char[3]
            Dwx = char[4]
            BITMAP = char[5]

            if len(BITMAP) > 0:
                chararray = Text2D.compress_to_array(BITMAP, BBw).T.repeat(zoom, axis=0).repeat(zoom, axis=1)[:, :, np.newaxis]
                tmparray = np.zeros((BBw * zoom, BBh * zoom, 4), dtype='u1')
                zerofilter = np.full((BBw * zoom, BBh * zoom, 1), False)
                filter0 = np.concatenate([chararray, zerofilter, zerofilter, zerofilter], axis=2)
                filter1 = np.concatenate([zerofilter, chararray, zerofilter, zerofilter], axis=2)
                filter2 = np.concatenate([zerofilter, zerofilter, chararray, zerofilter], axis=2)
                filter3 = np.concatenate([zerofilter, zerofilter, zerofilter, chararray], axis=2)
                tmparray[filter0] = c[0]
                tmparray[filter1] = c[1]
                tmparray[filter2] = c[2]
                tmparray[filter3] = c[3]
                l = xnow + zoom * BBxoff
                r = xnow + zoom * BBxoff + zoom * BBw
                t = ynow - zoom * BByoff - zoom * BBh
                b = ynow - zoom * BByoff
                if 0 <= r and l < layer.W and 0 <= b and t < layer.H:
                    if l < 0:
                        tmparray = tmparray[-l:, :, :]
                        l = 0
                    if r > layer.W:
                        tmparray = tmparray[0:zoom * BBw - r + layer.W, :, :]
                        r = layer.W
                    if t < 0:
                        tmparray = tmparray[:, -t:, :]
                        t = 0
                    if b > layer.H:
                        tmparray = tmparray[:, 0:zoom * BBw - b + layer.H, :]
                        b = layer.H

                    x = np.arange(l, r)
                    y = np.arange(t, b)
                    xx, yy = np.meshgrid(x, y)
                    xx = xx.reshape(-1)
                    yy = yy.reshape(-1)
                    layer.img[xx, yy] = Color.over(tmparray[xx - l, yy - t], layer.img[xx, yy])

            xnow += zoom * Dwx


# Color

In [ ]:
class Color:
    """Color class as numpy array of size (4)(r, g, b, a) used in a layer"""
    # https://ja.wikipedia.org/wiki/ウェブカラー
    # https://ja.wikipedia.org/wiki/アルファチャンネル
    # https://ja.wikipedia.org/wiki/色

    clear_color = np.array([0, 0, 0, 0], dtype='u1')

    white       = np.array([255,    255,    255,    255], dtype='u1')
    lightgray   = np.array([192,    192,    192,    255], dtype='u1')
    gray        = np.array([128,    128,    128,    255], dtype='u1')
    darkgray    = np.array([64,     64,     64,     255], dtype='u1')
    black       = np.array([0,      0,      0,      255], dtype='u1')

    red         = np.array([255,    0,      0,      255], dtype='u1')
    maroon      = np.array([128,    0,      0,      255], dtype='u1')
    yellow      = np.array([255,    255,    0,      255], dtype='u1')
    olive       = np.array([128,    128,    0,      255], dtype='u1')
    lime        = np.array([0,      255,    0,      255], dtype='u1')
    green       = np.array([0,      128,    0,      255], dtype='u1')
    cyan        = np.array([0,      255,    255,    255], dtype='u1')
    teal        = np.array([0,      128,    128,    255], dtype='u1')
    blue        = np.array([0,      0,      255,    255], dtype='u1')
    navy        = np.array([0,      0,      128,    255], dtype='u1')
    magenta     = np.array([255,    0,      255,    255], dtype='u1')
    fuchsia     = np.array([255,    0,      255,    255], dtype='u1')
    purple      = np.array([128,    0,      128,    255], dtype='u1')

    @staticmethod
    def RGB_to_color(r: int, g: int, b: int, a: int=255):
        """Return color from red, green, blue, and alpha, all 0 to 255"""
        return np.clip(np.array([r, g, b, a], dtype='u1'), 0, 255)

    @staticmethod
    def RGB_hex_to_color(text: str, a: int=255):
        """Return color from rgb hex text and alpha (0~255) eg. A040AB, 150"""
        num = int(text, 16)
        r = num // 65536
        g = (num - r * 65536) // 256
        b = num - r * 65536 - g * 256
        return np.clip(np.array([r, g, b, a], dtype='u1'), 0, 255)

    @staticmethod
    def HSV_to_color(h: int, s: int, v: int, a: int=255):
        """Return color from hue (0~360), saturation (0~255), value (0~255), and alpha (0~255)"""
        # https://www.peko-step.com/tool/hsvrgb.html
        h %= 360
        s = np.clip(s, 0, 255)
        v = np.clip(v, 0, 255)
        maxvalue = v
        minvalue = maxvalue - maxvalue * s / 255
        if 0 <= h < 60:
            r = maxvalue
            g = h / 60 * (maxvalue - minvalue) + minvalue
            b = minvalue
        elif 60 <= h < 120:
            r = (120 - h) / 60 * (maxvalue - minvalue) + minvalue
            g = maxvalue
            b = minvalue
        elif 120 <= h < 180:
            r = minvalue
            g = maxvalue
            b = (h - 120) / 60 * (maxvalue - minvalue) + minvalue
        elif 180 <= h < 240:
            r = minvalue
            g = (h - 240) / 60 * (maxvalue - minvalue) + minvalue
            b = maxvalue
        elif 240 <= h < 300:
            r = (h - 240) / 60 * (maxvalue - minvalue) + minvalue
            g = minvalue
            b = maxvalue
        else:
            r = maxvalue
            g = minvalue
            b = (360 - h) / 60 * (maxvalue - minvalue) + minvalue
        r = round(r)
        g = round(g)
        b = round(b)

        return np.clip(np.array([r, g, b, a], dtype='u1'), 0, 255)

    @staticmethod
    def mix(src_color, src_f, dst_color, dst_f):
        """Mix two colors with alpha. Use Canvas.mix for mixing layers. Mainly for internal use. Source is on top of destination."""
        src_a = src_color[:, 3] / 255
        dst_a = dst_color[:, 3] / 255
        out_a = src_a * src_f + dst_a * dst_f
        outafilter = out_a > 0
        out_rgb = np.zeros((src_color.shape[0], 3), dtype='u1')
        out_rgb[outafilter] = np.clip(np.round((src_color[outafilter, 0:3] * np.tile(src_a[outafilter].reshape(-1, 1), (1, 3)) * np.tile(src_f[outafilter].reshape(-1, 1), (1, 3)) + dst_color[outafilter, 0:3] * np.tile(dst_a[outafilter].reshape(-1, 1), (1, 3)) * np.tile(dst_f[outafilter].reshape(-1, 1), (1, 3))) / np.tile(out_a[outafilter].reshape(-1, 1), (1, 3))), 0, 255)
        return np.concatenate([out_rgb, np.clip(np.round(out_a * 255), 0, 255).reshape(-1, 1)], axis=1).astype('u1').copy()

    @staticmethod
    def clear(src_color, dst_color):
        src_f = np.zeros(dst_color.shape[0])
        dst_f = np.zeros(dst_color.shape[0])
        return Color.mix(src_color, src_f, dst_color, dst_f)

    @staticmethod
    def source(src_color, dst_color):
        src_f = np.ones(dst_color.shape[0])
        dst_f = np.zeros(dst_color.shape[0])
        return Color.mix(src_color, src_f, dst_color, dst_f)

    @staticmethod
    def destination(src_color, dst_color):
        src_f = np.zeros(dst_color.shape[0])
        dst_f = np.ones(dst_color.shape[0])
        return Color.mix(src_color, src_f, dst_color, dst_f)

    @staticmethod
    def over(src_color, dst_color):
        src_f = np.ones(dst_color.shape[0])
        dst_f = np.ones(dst_color.shape[0]) - src_color[:, 3] / 255
        return Color.mix(src_color, src_f, dst_color, dst_f)

    @staticmethod
    def in_(src_color, dst_color):
        src_f = dst_color[:, 3] / 255
        dst_f = np.zeros(dst_color.shape[0])
        return Color.mix(src_color, src_f, dst_color, dst_f)

    @staticmethod
    def out(src_color, dst_color):
        src_f = np.ones(dst_color.shape[0]) - dst_color[:, 3] / 255
        dst_f = np.zeros(dst_color.shape[0])
        return Color.mix(src_color, src_f, dst_color, dst_f)

    @staticmethod
    def atop(src_color, dst_color):
        src_f = dst_color[:, 3] / 255
        dst_f = np.ones(dst_color.shape[0]) - src_color[:, 3] / 255
        return Color.mix(src_color, src_f, dst_color, dst_f)

    @staticmethod
    def xor(src_color, dst_color):
        src_f = np.ones(dst_color.shape[0]) - dst_color[:, 3] / 255
        dst_f = np.ones(dst_color.shape[0]) - src_color[:, 3] / 255
        return Color.mix(src_color, src_f, dst_color, dst_f)

    @staticmethod
    def add(src_color, dst_color):
        src_f = np.ones(dst_color.shape[0])
        dst_f = np.ones(dst_color.shape[0])
        return Color.mix(src_color, src_f, dst_color, dst_f)

    @staticmethod
    def weak_add(src_color, dst_color):
        src_f = np.ones(dst_color.shape[0]) * 0.7
        dst_f = np.ones(dst_color.shape[0]) * 0.7
        return Color.mix(src_color, src_f, dst_color, dst_f)

    @staticmethod
    def simple_add(src_color, dst_color):
        return np.clip(src_color + dst_color, 0, 255).astype('u1').copy()


# Canvas

In [ ]:
class Canvas:
    """Canvas class for drawing"""

    def __init__(self, W: int, H: int, name="Default Canvas"):
        self.W = W
        self.H = H
        self.name = name
        self.clear_layers()

    def output_mixed_layers(self):
        final_layer_img = self.bg_layer.img.copy()
        x = np.arange(self.W)
        y = np.arange(self.H)
        xx, yy = np.meshgrid(x, y)
        xx = xx.reshape(-1)
        yy = yy.reshape(-1)
        final_layer_img[xx, yy] = Color.over(self.front_layer.img[xx, yy], Color.over(self.main_layer.img[xx, yy], self.bg_layer.img[xx, yy]))
        return final_layer_img

    def clear_layers(self):
        self.bg_layer = Layer(self.W, self.H, Settings.FOCAL, Settings.C_U, Settings.C_V, "background", opaque=True, layer_type=0)
        self.main_layer = Layer(self.W, self.H, Settings.FOCAL, Settings.C_U, Settings.C_V, "main", opaque=False, layer_type=1)
        self.front_layer = Layer(self.W, self.H, Settings.FOCAL, Settings.C_U, Settings.C_V, "front", opaque=False, layer_type=0)


# AntiAliasing

In [ ]:
class AntiAliasing:
    """Anti Aliasing"""

    @staticmethod
    def SSAA_grid(img, K:int=2):
        """Supersampling Anti-Aliasing with grid, scale K"""
        # https://en.wikipedia.org/wiki/Supersampling
        # https://stackoverflow.com/questions/42463172/how-to-perform-max-mean-pooling-on-a-2d-array-using-numpy
        M, N, C = img.shape
        assert(M % K == 0)
        assert(N % K == 0)
        return img.reshape(M // K, K, N // K, K, C).mean(axis=(1, 3)).astype('u1')


# ShaderGeometry

In [ ]:
class ShaderGeometry:
    """Geometry Shader, for clipping and tessellation(not implemented)"""
    # https://www.gabrielgambetta.com/computer-graphics-from-scratch/clipping.html
    # https://www.gabrielgambetta.com/computer-graphics-from-scratch/hidden-surface-removal.html
    # https://en.wikipedia.org/wiki/Clipping_(computer_graphics)
    # https://en.wikipedia.org/wiki/Sutherland%E2%80%93Hodgman_algorithm
    # https://en.wikipedia.org/wiki/Geometry_pipelines
    # http://www.opengl-tutorial.org/intermediate-tutorials/tutorial-16-shadow-mapping/

    @staticmethod
    def frontface_culling(v1, v2, v3, tn, n1=None, n2=None, n3=None, c=None, wf=None):
        if wf is None:
            front_filter = np.matmul(np.array([0, 0, 1]), tn) >= -Settings.EPSILON
        else:
            front_filter = np.logical_or(np.matmul(np.array([0, 0, 1]), tn) >= -Settings.EPSILON, wf)
        if c is not None:
            return v1[:, front_filter], v2[:, front_filter], v3[:, front_filter], tn[:, front_filter], n1[:, front_filter], n2[:, front_filter], n3[:, front_filter], c[:, front_filter], wf[front_filter]
        else:
            return v1[:, front_filter], v2[:, front_filter], v3[:, front_filter]

    @staticmethod
    def backface_culling(v1, v2, v3, tn, n1=None, n2=None, n3=None, c=None, wf=None):
        if wf is None:
            back_filter = np.matmul(np.array([0, 0, 1]), tn) <= Settings.EPSILON
        else:
            back_filter = np.logical_or(np.matmul(np.array([0, 0, 1]), tn) <= Settings.EPSILON, wf)
        if c is not None:
            return v1[:, back_filter], v2[:, back_filter], v3[:, back_filter], tn[:, back_filter], n1[:, back_filter], n2[:, back_filter], n3[:, back_filter], c[:, back_filter], wf[back_filter]
        else:
            return v1[:, back_filter], v2[:, back_filter], v3[:, back_filter]

    @staticmethod
    def depth_clip(v1, v2, v3, nearclip, farclip, tn=None, n1=None, n2=None, n3=None, c=None, wf=None):
        depth_filter = np.logical_or(np.logical_or(np.logical_and(nearclip <= v1[2, :], v1[2, :] <= farclip), np.logical_and(nearclip <= v2[2, :], v2[2, :] <= farclip)), np.logical_and(nearclip <= v3[2, :], v3[2, :] <= farclip))
        v1 = v1[:, depth_filter]
        v2 = v2[:, depth_filter]
        v3 = v3[:, depth_filter]
        if n1 is not None:
            n1 = n1[:, depth_filter]
            n2 = n2[:, depth_filter]
            n3 = n3[:, depth_filter]
        if c is not None:
            tn = tn[:, depth_filter]
            c = c[:, depth_filter]
            wf = wf[depth_filter]

        M, N = v1.shape
        for i in range(N):
            if v2[2, i] < v1[2, i]:
                v1[:, i], v2[:, i] = v2[:, i].copy(), v1[:, i].copy()
                if n1 is not None:
                    n1[:, i], n2[:, i] = n2[:, i].copy(), n1[:, i].copy()
            if v3[2, i] < v1[2, i]:
                v1[:, i], v3[:, i] = v3[:, i].copy(), v1[:, i].copy()
                if n1 is not None:
                    n1[:, i], n3[:, i] = n3[:, i].copy(), n1[:, i].copy()
            if v3[2, i] < v2[2, i]:
                v2[:, i], v3[:, i] = v3[:, i].copy(), v2[:, i].copy()
                if n1 is not None:
                    n2[:, i], n3[:, i] = n3[:, i].copy(), n2[:, i].copy()

            if v1[2, i] < nearclip and v2[2, i] < nearclip and nearclip <= v3[2, i]:
                v1[:, i] = (v3[:, i] + (v1[:, i] - v3[:, i]) * (nearclip - v3[2, i]) / (v1[2, i] - v3[2, i])).copy()
                v2[:, i] = (v3[:, i] + (v2[:, i] - v3[:, i]) * (nearclip - v3[2, i]) / (v2[2, i] - v3[2, i])).copy()
                if n1 is not None:
                    n1[:, i] = (n3[:, i] + (n1[:, i] - n3[:, i]) * (nearclip - v3[2, i]) / (v1[2, i] - v3[2, i])).copy()
                    n2[:, i] = (n3[:, i] + (n2[:, i] - n3[:, i]) * (nearclip - v3[2, i]) / (v2[2, i] - v3[2, i])).copy()
            elif v1[2, i] < nearclip and nearclip <= v2[2, i] and nearclip <= v3[2, i]:
                tmpv1 = v2[:, i] + (v1[:, i] - v2[:, i]) * (nearclip - v2[2, i]) / (v1[2, i] - v2[2, i])
                tmpv2 = v3[:, i] + (v1[:, i] - v3[:, i]) * (nearclip - v3[2, i]) / (v1[2, i] - v3[2, i])
                v1[:, i] = tmpv1.copy()
                v1 = np.concatenate([v1, tmpv1.copy().reshape(-1, 1)], axis=1)
                v2 = np.concatenate([v2, tmpv2.copy().reshape(-1, 1)], axis=1)
                v3 = np.concatenate([v3, v3[:, i].copy().reshape(-1, 1)], axis=1)
                if n1 is not None:
                    tmpn1 = n2[:, i] + (n1[:, i] - n2[:, i]) * (nearclip - v2[2, i]) / (v1[2, i] - v2[2, i])
                    tmpn2 = n3[:, i] + (n1[:, i] - n3[:, i]) * (nearclip - v3[2, i]) / (v1[2, i] - v3[2, i])
                    n1[:, i] = tmpn1.copy()
                    n1 = np.concatenate([n1, tmpn1.copy().reshape(-1, 1)], axis=1)
                    n2 = np.concatenate([n2, tmpn2.copy().reshape(-1, 1)], axis=1)
                    n3 = np.concatenate([n3, n3[:, i].copy().reshape(-1, 1)], axis=1)
                if c is not None:
                    tn = np.concatenate([tn, tn[:, i].copy().reshape(-1, 1)], axis=1)
                    c = np.concatenate([c, c[:, i].copy().reshape(3, 1, 11)], axis=1)
                    wf = np.concatenate([wf, np.array([wf[i].copy()])],axis=0)

        M, N = v1.shape
        for i in range(N):
            if v2[2, i] < v1[2, i]:
                v1[:, i], v2[:, i] = v2[:, i].copy(), v1[:, i].copy()
                if n1 is not None:
                    n1[:, i], n2[:, i] = n2[:, i].copy(), n1[:, i].copy()
            if v3[2, i] < v1[2, i]:
                v1[:, i], v3[:, i] = v3[:, i].copy(), v1[:, i].copy()
                if n1 is not None:
                    n1[:, i], n3[:, i] = n3[:, i].copy(), n1[:, i].copy()
            if v3[2, i] < v2[2, i]:
                v2[:, i], v3[:, i] = v3[:, i].copy(), v2[:, i].copy()
                if n1 is not None:
                    n2[:, i], n3[:, i] = n3[:, i].copy(), n2[:, i].copy()

            if v1[2, i] <= farclip and v2[2, i] <= farclip and farclip < v3[2, i]:
                tmpv1 = v1[:, i] + (v3[:, i] - v1[:, i]) * (farclip - v1[2, i]) / (v3[2, i] - v1[2, i])
                tmpv2 = v2[:, i] + (v3[:, i] - v2[:, i]) * (farclip - v2[2, i]) / (v3[2, i] - v2[2, i])
                v3[:, i] = tmpv1.copy()
                v1 = np.concatenate([v1, tmpv1.copy().reshape(-1, 1)], axis=1)
                v2 = np.concatenate([v2, tmpv2.copy().reshape(-1, 1)], axis=1)
                v3 = np.concatenate([v3, v2[:, i].copy().reshape(-1, 1)], axis=1)
                if n1 is not None:
                    tmpn1 = n1[:, i] + (n3[:, i] - n1[:, i]) * (farclip - v1[2, i]) / (v3[2, i] - v1[2, i])
                    tmpn2 = n2[:, i] + (n3[:, i] - n2[:, i]) * (farclip - v2[2, i]) / (v3[2, i] - v2[2, i])
                    n3[:, i] = tmpn1.copy()
                    n1 = np.concatenate([n1, tmpn1.copy().reshape(-1, 1)], axis=1)
                    n2 = np.concatenate([n2, tmpn2.copy().reshape(-1, 1)], axis=1)
                    n3 = np.concatenate([n3, n2[:, i].copy().reshape(-1, 1)], axis=1)
                if c is not None:
                    tn = np.concatenate([tn, tn[:, i].copy().reshape(-1, 1)], axis=1)
                    c = np.concatenate([c, c[:, i].copy().reshape(3, 1, 11)], axis=1)
                    wf = np.concatenate([wf, np.array([wf[i].copy()])],axis=0)
            elif v1[2, i] <= farclip and farclip < v2[2, i] and farclip < v3[2, i]:
                v2[:, i] = (v1[:, i] + (v2[:, i] - v1[:, i]) * (farclip - v1[2, i]) / (v2[2, i] - v1[2, i])).copy()
                v3[:, i] = (v1[:, i] + (v3[:, i] - v1[:, i]) * (farclip - v1[2, i]) / (v3[2, i] - v1[2, i])).copy()
                if n1 is not None:
                    n2[:, i] = (n1[:, i] + (n2[:, i] - n1[:, i]) * (farclip - v1[2, i]) / (v2[2, i] - v1[2, i])).copy()
                    n3[:, i] = (n1[:, i] + (n3[:, i] - n1[:, i]) * (farclip - v1[2, i]) / (v3[2, i] - v1[2, i])).copy()

        if n1 is not None:
            return v1, v2, v3, tn, n1, n2, n3, c, wf
        else:
            return v1, v2, v3

    @staticmethod
    def side_clip(v1, z1, v2, z2, v3, z3, c_u, c_v, n1=None, n2=None, n3=None, c=None, wf=None):
        topclip = 0
        bottomclip = c_u * 2
        top_down_filter = np.logical_or(np.logical_or(np.logical_and(topclip <= v1[0, :], v1[0, :] <= bottomclip), np.logical_and(topclip <= v2[0, :], v2[0, :] <= bottomclip)), np.logical_and(topclip <= v3[0, :], v3[0, :] <= bottomclip))
        v1 = v1[:, top_down_filter]
        v2 = v2[:, top_down_filter]
        v3 = v3[:, top_down_filter]
        z1 = z1[top_down_filter]
        z2 = z2[top_down_filter]
        z3 = z3[top_down_filter]
        if c is not None:
            n1 = n1[:, top_down_filter]
            n2 = n2[:, top_down_filter]
            n3 = n3[:, top_down_filter]
            c = c[:, top_down_filter]
            wf = wf[top_down_filter]

        leftclip = 0
        rightclip = c_v * 2
        left_right_filter = np.logical_or(np.logical_or(np.logical_and(leftclip <= v1[1, :], v1[1, :] <= rightclip), np.logical_and(leftclip <= v2[1, :], v2[1, :] <= rightclip)), np.logical_and(leftclip <= v3[1, :], v3[1, :] <= rightclip))
        v1 = v1[:, left_right_filter]
        v2 = v2[:, left_right_filter]
        v3 = v3[:, left_right_filter]
        z1 = z1[left_right_filter]
        z2 = z2[left_right_filter]
        z3 = z3[left_right_filter]
        if c is not None:
            n1 = n1[:, left_right_filter]
            n2 = n2[:, left_right_filter]
            n3 = n3[:, left_right_filter]
            c = c[:, left_right_filter]
            wf = wf[left_right_filter]

        M, N = v1.shape
        for i in range(N):
            if v2[0, i] < v1[0, i]:
                v1[:, i], v2[:, i] = v2[:, i].copy(), v1[:, i].copy()
                if n1 is not None:
                    n1[:, i], n2[:, i] = n2[:, i].copy(), n1[:, i].copy()
                z1[i], z2[i] = z2[i].copy(), z1[i].copy()
            if v3[0, i] < v1[0, i]:
                v1[:, i], v3[:, i] = v3[:, i].copy(), v1[:, i].copy()
                if n1 is not None:
                    n1[:, i], n3[:, i] = n3[:, i].copy(), n1[:, i].copy()
                z1[i], z3[i] = z3[i].copy(), z1[i].copy()
            if v3[0, i] < v2[0, i]:
                v2[:, i], v3[:, i] = v3[:, i].copy(), v2[:, i].copy()
                if n1 is not None:
                    n2[:, i], n3[:, i] = n3[:, i].copy(), n2[:, i].copy()
                z2[i], z3[i] = z3[i].copy(), z2[i].copy()

            if v1[0, i] < topclip and v2[0, i] < topclip and topclip <= v3[0, i]:
                v1[:, i] = (v3[:, i] + (v1[:, i] - v3[:, i]) * (topclip - v3[0, i]) / (v1[0, i] - v3[0, i])).copy()
                z1[i] = (z3[i] + (z1[i] - z3[i]) * (topclip - v3[0, i]) / (v1[0, i] - v3[0, i])).copy()
                v2[:, i] = (v3[:, i] + (v2[:, i] - v3[:, i]) * (topclip - v3[0, i]) / (v2[0, i] - v3[0, i])).copy()
                z2[i] = (z3[i] + (z2[i] - z3[i]) * (topclip - v3[0, i]) / (v2[0, i] - v3[0, i])).copy()
                if n1 is not None:
                    n1[:, i] = (n3[:, i] + (n1[:, i] - n3[:, i]) * (topclip - v3[0, i]) / (v1[0, i] - v3[0, i])).copy()
                    n2[:, i] = (n3[:, i] + (n2[:, i] - n3[:, i]) * (topclip - v3[0, i]) / (v2[0, i] - v3[0, i])).copy()
            elif v1[0, i] < topclip and topclip <= v2[0, i] and topclip <= v3[0, i]:
                tmpv1 = v2[:, i] + (v1[:, i] - v2[:, i]) * (topclip - v2[0, i]) / (v1[0, i] - v2[0, i])
                tmpv2 = v3[:, i] + (v1[:, i] - v3[:, i]) * (topclip - v3[0, i]) / (v1[0, i] - v3[0, i])
                tmpz1 = z2[i] + (z1[i] - z2[i]) * (topclip - v2[0, i]) / (v1[0, i] - v2[0, i])
                tmpz2 = z3[i] + (z1[i] - z3[i]) * (topclip - v3[0, i]) / (v1[0, i] - v3[0, i])
                v1[:, i] = tmpv1.copy()
                z1[i] = tmpz1.copy()
                v1 = np.concatenate([v1, tmpv1.copy().reshape(-1, 1)], axis=1)
                z1 = np.concatenate([z1, tmpz1.copy().reshape(1)])
                v2 = np.concatenate([v2, tmpv2.copy().reshape(-1, 1)], axis=1)
                z2 = np.concatenate([z2, tmpz2.copy().reshape(1)])
                v3 = np.concatenate([v3, v3[:, i].copy().reshape(-1, 1)], axis=1)
                z3 = np.concatenate([z3, z3[i].copy().reshape(1)])
                if c is not None:
                    tmpn1 = n2[:, i] + (n1[:, i] - n2[:, i]) * (topclip - v2[0, i]) / (v1[0, i] - v2[0, i])
                    tmpn2 = n3[:, i] + (n1[:, i] - n3[:, i]) * (topclip - v3[0, i]) / (v1[0, i] - v3[0, i])
                    n1[:, i] = tmpn1.copy()
                    n1 = np.concatenate([n1, tmpn1.copy().reshape(-1, 1)], axis=1)
                    n2 = np.concatenate([n2, tmpn2.copy().reshape(-1, 1)], axis=1)
                    n3 = np.concatenate([n3, n3[:, i].copy().reshape(-1, 1)], axis=1)
                    c = np.concatenate([c, c[:, i].copy().reshape(3, 1, 11)], axis=1)
                    wf = np.concatenate([wf, np.array([wf[i]])], axis=0)

        M, N = v1.shape
        for i in range(N):
            if v2[0, i] < v1[0, i]:
                v1[:, i], v2[:, i] = v2[:, i].copy(), v1[:, i].copy()
                if n1 is not None:
                    n1[:, i], n2[:, i] = n2[:, i].copy(), n1[:, i].copy()
                z1[i], z2[i] = z2[i].copy(), z1[i].copy()
            if v3[0, i] < v1[0, i]:
                v1[:, i], v3[:, i] = v3[:, i].copy(), v1[:, i].copy()
                if n1 is not None:
                    n1[:, i], n3[:, i] = n3[:, i].copy(), n1[:, i].copy()
                z1[i], z3[i] = z3[i].copy(), z1[i].copy()
            if v3[0, i] < v2[0, i]:
                v2[:, i], v3[:, i] = v3[:, i].copy(), v2[:, i].copy()
                if n1 is not None:
                    n2[:, i], n3[:, i] = n3[:, i].copy(), n2[:, i].copy()
                z2[i], z3[i] = z3[i].copy(), z2[i].copy()

            if v1[0, i] <= bottomclip and v2[0, i] <= bottomclip and bottomclip < v3[0, i]:
                tmpv1 = v1[:, i] + (v3[:, i] - v1[:, i]) * (bottomclip - v1[0, i]) / (v3[0, i] - v1[0, i])
                tmpv2 = v2[:, i] + (v3[:, i] - v2[:, i]) * (bottomclip - v2[0, i]) / (v3[0, i] - v2[0, i])
                tmpz1 = z1[i] + (z3[i] - z1[i]) * (bottomclip - v1[0, i]) / (v3[0, i] - v1[0, i])
                tmpz2 = z2[i] + (z3[i] - z2[i]) * (bottomclip - v2[0, i]) / (v3[0, i] - v2[0, i])
                v3[:, i] = tmpv1.copy()
                z3[i] = tmpz1.copy()
                v1 = np.concatenate([v1, tmpv1.copy().reshape(-1, 1)], axis=1)
                z1 = np.concatenate([z1, tmpz1.copy().reshape(1)])
                v2 = np.concatenate([v2, tmpv2.copy().reshape(-1, 1)], axis=1)
                z2 = np.concatenate([z2, tmpz2.copy().reshape(1)])
                v3 = np.concatenate([v3, v2[:, i].copy().reshape(-1, 1)], axis=1)
                z3 = np.concatenate([z3, z2[i].copy().reshape(1)])
                if c is not None:
                    tmpn1 = n1[:, i] + (n3[:, i] - n1[:, i]) * (bottomclip - v1[0, i]) / (v3[0, i] - v1[0, i])
                    tmpn2 = n2[:, i] + (n3[:, i] - n2[:, i]) * (bottomclip - v2[0, i]) / (v3[0, i] - v2[0, i])
                    n3[:, i] = tmpn1.copy()
                    n1 = np.concatenate([n1, tmpn1.copy().reshape(-1, 1)], axis=1)
                    n2 = np.concatenate([n2, tmpn2.copy().reshape(-1, 1)], axis=1)
                    n3 = np.concatenate([n3, n2[:, i].copy().reshape(-1, 1)], axis=1)
                    c = np.concatenate([c, c[:, i].copy().reshape(3, 1, 11)], axis=1)
                    wf = np.concatenate([wf, np.array([wf[i]])], axis=0)
            elif v1[0, i] <= bottomclip and bottomclip < v2[0, i] and bottomclip < v3[0, i]:
                v2[:, i] = (v1[:, i] + (v2[:, i] - v1[:, i]) * (bottomclip - v1[0, i]) / (v2[0, i] - v1[0, i])).copy()
                z2[i] = (z1[i] + (z2[i] - z1[i]) * (bottomclip - v1[0, i]) / (v2[0, i] - v1[0, i])).copy()
                v3[:, i] = (v1[:, i] + (v3[:, i] - v1[:, i]) * (bottomclip - v1[0, i]) / (v3[0, i] - v1[0, i])).copy()
                z3[i] = (z1[i] + (z2[i] - z1[i]) * (bottomclip - v1[0, i]) / (v3[0, i] - v1[0, i])).copy()
                if n1 is not None:
                    n2[:, i] = (n1[:, i] + (n2[:, i] - n1[:, i]) * (bottomclip - v1[0, i]) / (v2[0, i] - v1[0, i])).copy()
                    n3[:, i] = (n1[:, i] + (n3[:, i] - n1[:, i]) * (bottomclip - v1[0, i]) / (v3[0, i] - v1[0, i])).copy()

        M, N = v1.shape
        for i in range(N):
            if v2[1, i] < v1[1, i]:
                v1[:, i], v2[:, i] = v2[:, i].copy(), v1[:, i].copy()
                if n1 is not None:
                    n1[:, i], n2[:, i] = n2[:, i].copy(), n1[:, i].copy()
                z1[i], z2[i] = z2[i].copy(), z1[i].copy()
            if v3[1, i] < v1[1, i]:
                v1[:, i], v3[:, i] = v3[:, i].copy(), v1[:, i].copy()
                if n1 is not None:
                    n1[:, i], n3[:, i] = n3[:, i].copy(), n1[:, i].copy()
                z1[i], z3[i] = z3[i].copy(), z1[i].copy()
            if v3[1, i] < v2[1, i]:
                v2[:, i], v3[:, i] = v3[:, i].copy(), v2[:, i].copy()
                if n1 is not None:
                    n2[:, i], n3[:, i] = n3[:, i].copy(), n2[:, i].copy()
                z2[i], z3[i] = z3[i].copy(), z2[i].copy()

            if v1[1, i] < leftclip and v2[1, i] < leftclip and leftclip <= v3[1, i]:
                v1[:, i] = (v3[:, i] + (v1[:, i] - v3[:, i]) * (leftclip - v3[1, i]) / (v1[1, i] - v3[1, i])).copy()
                z1[i] = (z3[i] + (z1[i] - z3[i]) * (leftclip - v3[1, i]) / (v1[1, i] - v3[1, i])).copy()
                v2[:, i] = (v3[:, i] + (v2[:, i] - v3[:, i]) * (leftclip - v3[1, i]) / (v2[1, i] - v3[1, i])).copy()
                z2[i] = (z3[i] + (z2[i] - z3[i]) * (leftclip - v3[1, i]) / (v2[1, i] - v3[1, i])).copy()
                if n1 is not None:
                    n1[:, i] = (n3[:, i] + (n1[:, i] - n3[:, i]) * (leftclip - v3[1, i]) / (v1[1, i] - v3[1, i])).copy()
                    n2[:, i] = (n3[:, i] + (n2[:, i] - n3[:, i]) * (leftclip - v3[1, i]) / (v2[1, i] - v3[1, i])).copy()
            elif v1[1, i] < leftclip and leftclip <= v2[1, i] and leftclip <= v3[1, i]:
                tmpv1 = v2[:, i] + (v1[:, i] - v2[:, i]) * (leftclip - v2[1, i]) / (v1[1, i] - v2[1, i])
                tmpv2 = v3[:, i] + (v1[:, i] - v3[:, i]) * (leftclip - v3[1, i]) / (v1[1, i] - v3[1, i])
                tmpz1 = z2[i] + (z1[i] - z2[i]) * (leftclip - v2[1, i]) / (v1[1, i] - v2[1, i])
                tmpz2 = z3[i] + (z1[i] - z3[i]) * (leftclip - v3[1, i]) / (v1[1, i] - v3[1, i])
                v1[:, i] = tmpv1.copy()
                z1[i] = tmpz1.copy()
                v1 = np.concatenate([v1, tmpv1.copy().reshape(-1, 1)], axis=1)
                z1 = np.concatenate([z1, tmpz1.copy().reshape(1)])
                v2 = np.concatenate([v2, tmpv2.copy().reshape(-1, 1)], axis=1)
                z2 = np.concatenate([z2, tmpz2.copy().reshape(1)])
                v3 = np.concatenate([v3, v3[:, i].copy().reshape(-1, 1)], axis=1)
                z3 = np.concatenate([z3, z3[i].copy().reshape(1)])
                if c is not None:
                    tmpn1 = n2[:, i] + (n1[:, i] - n2[:, i]) * (leftclip - v2[1, i]) / (v1[1, i] - v2[1, i])
                    tmpn2 = n3[:, i] + (n1[:, i] - n3[:, i]) * (leftclip - v3[1, i]) / (v1[1, i] - v3[1, i])
                    n1[:, i] = tmpn1.copy()
                    n1 = np.concatenate([n1, tmpn1.copy().reshape(-1, 1)], axis=1)
                    n2 = np.concatenate([n2, tmpn2.copy().reshape(-1, 1)], axis=1)
                    n3 = np.concatenate([n3, n3[:, i].copy().reshape(-1, 1)], axis=1)
                    c = np.concatenate([c, c[:, i].copy().reshape(3, 1, 11)], axis=1)
                    wf = np.concatenate([wf, np.array([wf[i]])], axis=0)

        M, N = v1.shape
        for i in range(N):
            if v2[1, i] < v1[1, i]:
                v1[:, i], v2[:, i] = v2[:, i].copy(), v1[:, i].copy()
                if n1 is not None:
                    n1[:, i], n2[:, i] = n2[:, i].copy(), n1[:, i].copy()
                z1[i], z2[i] = z2[i].copy(), z1[i].copy()
            if v3[1, i] < v1[1, i]:
                v1[:, i], v3[:, i] = v3[:, i].copy(), v1[:, i].copy()
                if n1 is not None:
                    n1[:, i], n3[:, i] = n3[:, i].copy(), n1[:, i].copy()
                z1[i], z3[i] = z3[i].copy(), z1[i].copy()
            if v3[1, i] < v2[1, i]:
                v2[:, i], v3[:, i] = v3[:, i].copy(), v2[:, i].copy()
                if n1 is not None:
                    n2[:, i], n3[:, i] = n3[:, i].copy(), n2[:, i].copy()
                z2[i], z3[i] = z3[i].copy(), z2[i].copy()

            if v1[1, i] <= rightclip and v2[1, i] <= rightclip and rightclip < v3[1, i]:
                tmpv1 = v1[:, i] + (v3[:, i] - v1[:, i]) * (rightclip - v1[1, i]) / (v3[1, i] - v1[1, i])
                tmpv2 = v2[:, i] + (v3[:, i] - v2[:, i]) * (rightclip - v2[1, i]) / (v3[1, i] - v2[1, i])
                tmpz1 = z1[i] + (z3[i] - z1[i]) * (rightclip - v1[1, i]) / (v3[1, i] - v1[1, i])
                tmpz2 = z2[i] + (z3[i] - z2[i]) * (rightclip - v2[1, i]) / (v3[1, i] - v2[1, i])
                v3[:, i] = tmpv1.copy()
                z3[i] = tmpz1.copy()
                v1 = np.concatenate([v1, tmpv1.copy().reshape(-1, 1)], axis=1)
                z1 = np.concatenate([z1, tmpz1.copy().reshape(1)])
                v2 = np.concatenate([v2, tmpv2.copy().reshape(-1, 1)], axis=1)
                z2 = np.concatenate([z2, tmpz2.copy().reshape(1)])
                v3 = np.concatenate([v3, v2[:, i].copy().reshape(-1, 1)], axis=1)
                z3 = np.concatenate([z3, z2[i].copy().reshape(1)])
                if c is not None:
                    tmpn1 = n1[:, i] + (n3[:, i] - n1[:, i]) * (rightclip - v1[1, i]) / (v3[1, i] - v1[1, i])
                    tmpn2 = n2[:, i] + (n3[:, i] - n2[:, i]) * (rightclip - v2[1, i]) / (v3[1, i] - v2[1, i])
                    n3[:, i] = tmpn1.copy()
                    n1 = np.concatenate([n1, tmpn1.copy().reshape(-1, 1)], axis=1)
                    n2 = np.concatenate([n2, tmpn2.copy().reshape(-1, 1)], axis=1)
                    n3 = np.concatenate([n3, n2[:, i].copy().reshape(-1, 1)], axis=1)
                    c = np.concatenate([c, c[:, i].copy().reshape(3, 1, 11)], axis=1)
                    wf = np.concatenate([wf, np.array([wf[i]])], axis=0)
            elif v1[1, i] <= rightclip and rightclip < v2[1, i] and rightclip < v3[1, i]:
                v2[:, i] = (v1[:, i] + (v2[:, i] - v1[:, i]) * (rightclip - v1[1, i]) / (v2[1, i] - v1[1, i])).copy()
                z2[i] = (z1[i] + (z2[i] - z1[i]) * (rightclip - v1[1, i]) / (v2[1, i] - v1[1, i])).copy()
                v3[:, i] = (v1[:, i] + (v3[:, i] - v1[:, i]) * (rightclip - v1[1, i]) / (v3[1, i] - v1[1, i])).copy()
                z3[i] = (z1[i] + (z2[i] - z1[i]) * (rightclip - v1[1, i]) / (v3[1, i] - v1[1, i])).copy()
                if n1 is not None:
                    n2[:, i] = (n1[:, i] + (n2[:, i] - n1[:, i]) * (rightclip - v1[1, i]) / (v2[1, i] - v1[1, i])).copy()
                    n3[:, i] = (n1[:, i] + (n3[:, i] - n1[:, i]) * (rightclip - v1[1, i]) / (v3[1, i] - v1[1, i])).copy()

        if n1 is not None:
            return v1, z1, v2, z2, v3, z3, n1, n2, n3, c, wf
        else:
            return v1, z1, v2, z2, v3, z3


# ShaderVertex

ワールド座標系$(X_W\ Y_W\ Z_W)^T$からカメラ座標系$(X\ Y\ Z)^T$への変換(回転変換$\boldsymbol{R}$、並進変換$\boldsymbol{t}$)(直交座標系ver)
$$
    \begin{pmatrix}
    X\\
    Y\\
    Z
    \end{pmatrix}
    =\boldsymbol{R} \left(\begin{pmatrix}
    X_W\\
    Y_W\\
    Z_W
    \end{pmatrix}
    \boldsymbol{t}\right)
$$

ワールド座標系$(X_W\ Y_W\ Z_W)^T$からカメラ座標系$(X\ Y\ Z)^T$への変換(斉次座標系ver)($\sim$は0ではない定数倍の違いを無視した同値関係。)

$$
    \begin{pmatrix} X\\ Y\\ Z\\ 1 \end{pmatrix} \sim \boldsymbol{M} \begin{pmatrix} X_W\\ Y_W\\ Z_W\\ 1 \end{pmatrix} \\ M = \begin{pmatrix} \boldsymbol{R} & \boldsymbol{t}\\ \boldsymbol{0}^T & 1 \end{pmatrix}
$$
(上の二つの定式化が同値であることは行列を計算すればわかる。)

斉次座標系をここでは使う。

ここで、画面の前方にある三角形のみを考えるためにクリッピングを行う。クリッピング面$Z=depth$に対して三角形の頂点がどこにあるかで場合分けする。

三角形の頂点が3つとも面の前方にある時、全て表示。
三角形の頂点が2つ面の前方にあり1つ後方にある時、前方の四角形を切り出し二つの三角形に分割して表示。
三角形の頂点が1つ面の前方にあり2つ後方にある時、前方にある三角形を切り出して表示。
三角形の頂点が3つとも面の後方にある時、表示しない。とする。

カメラ座標系$(X\ Y\ Z)^T$から正規化画像座標系$(x\ y)^T$へ変換する。(ここで、画面内は-1から1に正規化しないといけないが面倒なのでしない。)
$$
    \begin{pmatrix}
    x\\
    y\\
    1
    \end{pmatrix}
    \sim\begin{pmatrix}
    1 & 0 & 0 & 0\\
    0 & 1 & 0 & 0\\
    0 & 0 & 1 & 0
    \end{pmatrix}
    \begin{pmatrix}
    X\\
    Y\\
    Z\\
    1
    \end{pmatrix}
$$

これは奥行きから、画面中央からどれだけ離れてるかを比率で求めている。つまり、

$$
    x = \frac{X}{Z} \\
    y = \frac{Y}{Z}
$$
である。

正規化画像座標系$(x\ y)^T$から画像座標系$(u\ v)^T$へ変換する。ここで、$f$は焦点距離、$c_u$は画面横幅の半分、$c_v$は画面縦幅の半分である。($(2,2)$の$-f$は画像表示の際の都合によりマイナス。本来はこれも$f$になるはず。)
$$
    \begin{pmatrix}
    u\\
    v\\
    1
    \end{pmatrix}
    \sim\boldsymbol{A}
    \begin{pmatrix}
    x\\
    y\\
    1
    \end{pmatrix}\\
    \boldsymbol{A}=\begin{pmatrix}
    f & 0 & c_u\\
    0 & -f & c_v\\
    0 & 0 & 1
    \end{pmatrix}
$$


In [ ]:
class ShaderVertex:
    """Vertex Shader, for transforming 3D into 2D"""
    #left-handed system!!!
    # https://daily-tech.hatenablog.com/entry/2018/01/22/071346
    # https://daily-tech.hatenablog.com/entry/2018/01/23/064926
    # https://en.wikipedia.org/wiki/Camera_matrix
    # https://en.wikipedia.org/wiki/Vertex_pipeline

    @staticmethod
    def transform_world_to_camera(R, T, v=None, n=None):
        M = np.concatenate([np.concatenate([R, T], 1), np.array([[0, 0, 0, 1]])], 0)
        if v is not None and n is None:
            vdash = np.concatenate([v.T, np.ones((1, v.shape[0]))], 0)
            return np.matmul(M, vdash)[0:3, :]
        elif v is None and n is not None:
            return np.matmul(R, n.T)
        elif v is not None and n is not None:
            vdash = np.concatenate([v.T, np.ones((1, v.shape[0]))], 0)
            return np.matmul(M, vdash)[0:3, :], np.matmul(R, n.T)
        else:
            return None

    @staticmethod
    def transform_camera_to_canvas(f, c_u, c_v, v, ortho=False):
        z_buffer = 1 / v[2, :].copy()
        if ortho:
            v[2, :] = 1
            f /= 100

        A = np.array([[f, 0, c_u],[0, -f, c_v],[0, 0, 1]])
        new_v = np.matmul(A, v)
        scalerow = new_v[2, :].copy()

        return new_v[0:2, :] / np.stack([scalerow, scalerow]), z_buffer


# Raytracing

In [ ]:
class Raytracing:
    """Ray Tracing class"""

    @staticmethod
    def transform_camera_to_world(R, T, v):
        M = np.concatenate([np.concatenate([R, T], 1), np.array([[0, 0, 0, 1]])], 0)
        vdash = np.concatenate([v, np.ones((1, v.shape[1]))], axis=0)
        res = np.matmul(M, vdash)
        return res[0:3, :] / np.tile(res[3].reshape(1, -1), (3, 1))

    @staticmethod
    def transform_canvas_to_world(f, c_u, c_v, R, T, v, z_buffer):
        """Transform points on perspective v to normalized ray vectors (X_W, Y_W, Z_W)"""
        # https://www.gabrielgambetta.com/computer-graphics-from-scratch/basic-ray-tracing.html
        vdash = np.concatenate([v, np.ones((1, v.shape[1]))], axis=0)
        Ainv = np.array([[1/f, 0, -c_u/f],[0, -1/f, c_v/f],[0, 0, 1]])
        camera_points = np.matmul(Ainv, vdash) / np.tile(z_buffer.reshape(1, -1), (3, 1))
        return Raytracing.transform_camera_to_world(R, T, camera_points)


# GraphicsPipeline

In [ ]:
class GraphicsPipeline:
    """Graphics Pipeline"""
    # https://en.wikipedia.org/wiki/Graphics_pipeline
    # https://ja.wikipedia.org/wiki/グラフィックスパイプライン
    # https://en.wikipedia.org/wiki/Shading
    # https://en.wikipedia.org/wiki/Phong_shading
    # https://en.wikipedia.org/wiki/Rendering_(computer_graphics)

    @staticmethod
    def create_shadow_map(lightpos, lightrot, ortho, v1, v2, v3, tn, shadow_map_size=2048):
        R = lightrot.matrix
        T = -np.matmul(lightrot.matrix, lightpos.reshape(3, 1))

        c_uv = shadow_map_size // 2
        focal = c_uv

        shadow_map = Layer(shadow_map_size, shadow_map_size, focal, c_uv, c_uv, "shadow map", layer_type=2)
        shadow_map.save_pos_rot(lightpos, lightrot)

        if v1.size == 0:
            return shadow_map
        else:
            v1 = ShaderVertex.transform_world_to_camera(R, T, v1)
            v2 = ShaderVertex.transform_world_to_camera(R, T, v2)
            v3 = ShaderVertex.transform_world_to_camera(R, T, v3)
            tn = ShaderVertex.transform_world_to_camera(R, T, n=tn)

            v1, v2, v3 = ShaderGeometry.frontface_culling(v1, v2, v3, tn)
            v1, v2, v3 = ShaderGeometry.depth_clip(v1, v2, v3, Settings.LIGHT_NEARCLIP, Settings.LIGHT_FARCLIP)

            v1, z1 = ShaderVertex.transform_camera_to_canvas(focal, c_uv, c_uv, v1, ortho)
            v2, z2 = ShaderVertex.transform_camera_to_canvas(focal, c_uv, c_uv, v2, ortho)
            v3, z3 = ShaderVertex.transform_camera_to_canvas(focal, c_uv, c_uv, v3, ortho)

            v1, z1, v2, z2, v3, z3 = ShaderGeometry.side_clip(v1, z1, v2, z2, v3, z3, c_uv, c_uv)

            v1 = v1.astype('i8')
            v2 = v2.astype('i8')
            v3 = v3.astype('i8')

            for i in range(z1.size):
                Triangle2D.filled_triangle(shadow_map, v1[0, i], v1[1, i], v2[0, i], v2[1, i], v3[0, i], v3[1, i], z0=z1[i], z1=z2[i], z2=z3[i])

            return shadow_map

    @staticmethod
    def main_pipeline(canvas, campos, camrot, ortho, lights, shadows, v1, n1, v2, n2, v3, n3, tn, c, wf):
        R = camrot.matrix
        T = -np.matmul(camrot.matrix, campos.reshape(3, 1))

        canvas.main_layer.save_pos_rot(campos, camrot)

        c = c.reshape((3, -1, 11))

        v1 = ShaderVertex.transform_world_to_camera(R, T, v1)
        v2 = ShaderVertex.transform_world_to_camera(R, T, v2)
        v3 = ShaderVertex.transform_world_to_camera(R, T, v3)
        tn = ShaderVertex.transform_world_to_camera(R, T, n=tn)
        n1 = n1.T
        n2 = n2.T
        n3 = n3.T

        v1, v2, v3, tn, n1, n2, n3, c, wf = ShaderGeometry.backface_culling(v1, v2, v3, tn, n1, n2, n3, c, wf)
        v1, v2, v3, tn, n1, n2, n3, c, wf = ShaderGeometry.depth_clip(v1, v2, v3, Settings.CAMERA_NEARCLIP, Settings.CAMERA_FARCLIP, tn, n1, n2, n3, c, wf)

        v1, z1 = ShaderVertex.transform_camera_to_canvas(Settings.FOCAL, Settings.C_U, Settings.C_V, v1, ortho)
        v2, z2 = ShaderVertex.transform_camera_to_canvas(Settings.FOCAL, Settings.C_U, Settings.C_V, v2, ortho)
        v3, z3 = ShaderVertex.transform_camera_to_canvas(Settings.FOCAL, Settings.C_U, Settings.C_V, v3, ortho)

        v1, z1, v2, z2, v3, z3, n1, n2, n3, c, wf = ShaderGeometry.side_clip(v1, z1, v2, z2, v3, z3, Settings.C_U, Settings.C_V, n1, n2, n3, c, wf)

        v1 = v1.astype('i8')
        v2 = v2.astype('i8')
        v3 = v3.astype('i8')

        for i in range(z1.size):
            if wf[i]:
                Triangle2D.triangle(canvas.main_layer, v1[0, i], v1[1, i], v2[0, i], v2[1, i], v3[0, i], v3[1, i], c[0, i], c[1, i], c[2, i], z1[i], z2[i], z3[i], n1[:, i], n2[:, i], n3[:, i], lights, shadows)
            else:
                Triangle2D.filled_triangle(canvas.main_layer, v1[0, i], v1[1, i], v2[0, i], v2[1, i], v3[0, i], v3[1, i], c[0, i], c[1, i], c[2, i], z1[i], z2[i], z3[i], n1[:, i], n2[:, i], n3[:, i], lights, shadows)

    @staticmethod
    def run(canvas, camera, world):
        v1 = np.array([[0, 0, 0]])
        n1 = np.array([[0, 0, 0]])
        v2 = np.array([[0, 0, 0]])
        n2 = np.array([[0, 0, 0]])
        v3 = np.array([[0, 0, 0]])
        n3 = np.array([[0, 0, 0]])
        tn = np.array([[0, 0, 0]])
        tm = Material.clear_material.reshape(1, -1)
        wf = np.array([False])
        sh = np.array([False])
        for world_object in world.objects:
            v1 = np.concatenate([v1, np.matmul(world_object.rot.matrix, world_object.v[:, :, 0].T).T + world_object.pos], axis=0)
            n1 = np.concatenate([n1, np.matmul(world_object.rot.matrix, world_object.vn[:, :, 0].T).T], axis=0)
            v2 = np.concatenate([v2, np.matmul(world_object.rot.matrix, world_object.v[:, :, 1].T).T + world_object.pos], axis=0)
            n2 = np.concatenate([n2, np.matmul(world_object.rot.matrix, world_object.vn[:, :, 1].T).T], axis=0)
            v3 = np.concatenate([v3, np.matmul(world_object.rot.matrix, world_object.v[:, :, 2].T).T + world_object.pos], axis=0)
            n3 = np.concatenate([n3, np.matmul(world_object.rot.matrix, world_object.vn[:, :, 2].T).T], axis=0)
            tn = np.concatenate([tn, np.matmul(world_object.rot.matrix, world_object.tn.reshape(-1, 3).T).T], axis=0)
            tm = np.concatenate([tm, world_object.tm], axis=0)
            wf = np.concatenate([wf, np.tile(np.array([world_object.wire_frame]), (world_object.v.shape[0]))], axis=0)
            sh = np.concatenate([sh, np.tile(np.array([world_object.enable_shadow]), (world_object.v.shape[0]))], axis=0)
        v1 = v1[1:, :]
        n1 = n1[1:, :]
        v2 = v2[1:, :]
        n2 = n2[1:, :]
        v3 = v3[1:, :]
        n3 = n3[1:, :]
        tn = tn[1:, :]
        tm = tm[1:, :]
        wf = wf[1:]
        sh = sh[1:]

        time_ini = time.perf_counter()

        shadows = []
        for light_id in range(len(world.lights)):
            light = world.lights[light_id]
            if light.light_type == 0:
                shadows.append("")
            else:
                shadows.append(GraphicsPipeline.create_shadow_map(light.pos, light.rot, light.light_type==2, v1[np.invert(wf) & sh, :], v2[np.invert(wf) & sh, :], v3[np.invert(wf) & sh, :], tn[np.invert(wf) & sh, :]))

        c = np.tile(tm, (3, 1, 1))

        time_sha = time.perf_counter()

        GraphicsPipeline.main_pipeline(canvas, camera.pos, camera.rot, camera.ortho, world.lights, shadows, v1, n1, v2, n2, v3, n3, tn, c, wf)

        time_end = time.perf_counter()

        return shadows, time_ini, time_sha, time_end


# Material

In [ ]:
class Material:
    """Material class for World Objects"""
    # https://yttm-work.jp/model_render/model_render_0001.html
    # https://en.wikipedia.org/wiki/Wavefront_.obj_file
    # http://www.it.hiof.no/~borres/j3d/explain/light/p-materials.html
    # https://www.scratchapixel.com/lessons/3d-basic-rendering/phong-shader-BRDF

    # in order of Ka(RGB ambient, 0~1), Kd(RGB diffuse, 0~1), Ks(RGB specular, 0~1), Ns(specular exponent, 1~1000?), d(alpha, d=1 is opaque, d=0 is transparent)
    ####################         Ka R       G       B    Kd R       G       B    Ks R       G       B       Ns      d
    clear_material      = np.array([0,      0,      0,      0,      0,      0,      0,      0,      0,      1,      0], dtype='f8')

    matte_white         = np.array([0.5,    0.5,    0.5,    1,      1,      1,      0,      0,      0,      1,      1], dtype='f8')
    matte_lightgray     = np.array([0.375,  0.375,  0.375,  0.75,   0.75,   0.75,   0,      0,      0,      1,      1], dtype='f8')
    matte_gray          = np.array([0.25,   0.25,   0.25,   0.5,    0.5,    0.5,    0,      0,      0,      1,      1], dtype='f8')
    matte_darkgray      = np.array([0.125,  0.125,  0.125,  0.25,   0.25,   0.25,   0,      0,      0,      1,      1], dtype='f8')
    matte_black         = np.array([0,      0,      0,      0,      0,      0,      0,      0,      0,      1,      1], dtype='f8')

    matte_red           = np.array([0.5,    0,      0,      1,      0,      0,      0,      0,      0,      1,      1], dtype='f8')
    matte_maroon        = np.array([0.25,   0,      0,      0.5,    0,      0,      0,      0,      0,      1,      1], dtype='f8')
    matte_yellow        = np.array([0.5,    0.5,    0,      1,      1,      0,      0,      0,      0,      1,      1], dtype='f8')
    matte_olive         = np.array([0.25,   0.25,   0,      0.5,    0.5,    0,      0,      0,      0,      1,      1], dtype='f8')
    matte_lime          = np.array([0,      0.5,    0,      0,      1,      0,      0,      0,      0,      1,      1], dtype='f8')
    matte_green         = np.array([0,      0.25,   0,      0,      0.5,    0,      0,      0,      0,      1,      1], dtype='f8')
    matte_cyan          = np.array([0,      0.5,    0.5,    0,      1,      1,      0,      0,      0,      1,      1], dtype='f8')
    matte_teal          = np.array([0,      0.25,   0.25,   0,      0.5,    0.5,    0,      0,      0,      1,      1], dtype='f8')
    matte_blue          = np.array([0,      0,      0.5,    0,      0,      1,      0,      0,      0,      1,      1], dtype='f8')
    matte_navy          = np.array([0,      0,      0.25,   0,      0,      0.5,    0,      0,      0,      1,      1], dtype='f8')
    matte_magenta       = np.array([0.5,    0,      0.5,    1,      0,      1,      0,      0,      0,      1,      1], dtype='f8')
    matte_fuchsia       = np.array([0.5,    0,      0.5,    1,      0,      1,      0,      0,      0,      1,      1], dtype='f8')
    matte_purple        = np.array([0.25,   0,      0.25,   0.5,    0,      0.5,    0,      0,      0,      1,      1], dtype='f8')

    ####################         Ka R       G       B    Kd R       G       B    Ks R       G       B       Ns      d
    brass               = np.array([0.33,   0.22,   0.27,   0.78,   0.57,   0.11,   0.99,   0.94,   0.81,   28,     1], dtype='f8')
    bronze              = np.array([0.21,   0.13,   0.05,   0.71,   0.43,   0.18,   0.39,   0.27,   0.17,   26,     1], dtype='f8')
    polished_bronze     = np.array([0.25,   0.15,   0.06,   0.40,   0.24,   0.10,   0.77,   0.46,   0.20,   77,     1], dtype='f8')
    chrome              = np.array([0.25,   0.25,   0.25,   0.40,   0.40,   0.40,   0.78,   0.78,   0.78,   77,     1], dtype='f8')
    copper              = np.array([0.19,   0.07,   0.02,   0.70,   0.27,   0.08,   0.26,   0.14,   0.09,   13,     1], dtype='f8')
    polished_copper     = np.array([0.23,   0.09,   0.03,   0.55,   0.21,   0.07,   0.58,   0.22,   0.70,   51,     1], dtype='f8')
    gold                = np.array([0.25,   0.20,   0.07,   0.75,   0.61,   0.23,   0.63,   0.56,   0.37,   51,     1], dtype='f8')
    polished_gold       = np.array([0.25,   0.22,   0.06,   0.35,   0.31,   0.09,   0.80,   0.72,   0.21,   83,     1], dtype='f8')
    tin                 = np.array([0.11,   0.06,   0.11,   0.43,   0.47,   0.54,   0.33,   0.33,   0.52,   10,     1], dtype='f8')
    silver              = np.array([0.19,   0.19,   0.19,   0.51,   0.51,   0.51,   0.51,   0.51,   0.51,   51,     1], dtype='f8')
    polished_silver     = np.array([0.23,   0.23,   0.23,   0.28,   0.28,   0.28,   0.77,   0.77,   0.77,   90,     1], dtype='f8')
    emerald             = np.array([0.02,   0.17,   0.02,   0.08,   0.61,   0.08,   0.63,   0.73,   0.63,   77,     1], dtype='f8')
    jade                = np.array([0.14,   0.22,   0.16,   0.54,   0.89,   0.63,   0.32,   0.32,   0.32,   13,     1], dtype='f8')
    obsidian            = np.array([0.05,   0.05,   0.06,   0.18,   0.17,   0.23,   0.33,   0.33,   0.35,   38,     1], dtype='f8')
    perl                = np.array([0.25,   0.21,   0.21,   1.00,   0.83,   0.83,   0.30,   0.30,   0.30,   11,     1], dtype='f8')
    ruby                = np.array([0.17,   0.01,   0.01,   0.61,   0.04,   0.04,   0.73,   0.63,   0.63,   77,     1], dtype='f8')
    turquoise           = np.array([0.10,   0.19,   0.17,   0.01,   0.01,   0.01,   0.50,   0.50,   0.50,   32,     1], dtype='f8')

    ####################         Ka R       G       B    Kd R       G       B    Ks R       G       B       Ns      d
    plastic_white       = np.array([0.10,   0.10,   0.10,   0.50,   0.50,   0.50,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_lightgray   = np.array([0.08,   0.08,   0.08,   0.38,   0.38,   0.38,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_gray        = np.array([0.05,   0.05,   0.05,   0.25,   0.25,   0.25,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_darkgray    = np.array([0.03,   0.03,   0.03,   0.13,   0.13,   0.13,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_black       = np.array([0.01,   0.01,   0.01,   0.01,   0.01,   0.01,   0.50,   0.50,   0.50,   32,     1], dtype='f8')

    plastic_red         = np.array([0.10,   0.01,   0.01,   0.50,   0.01,   0.01,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_maroon      = np.array([0.05,   0.01,   0.01,   0.25,   0.01,   0.01,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_yellow      = np.array([0.10,   0.10,   0.01,   0.50,   0.50,   0.01,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_olive       = np.array([0.05,   0.05,   0.01,   0.25,   0.25,   0.01,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_lime        = np.array([0.01,   0.10,   0.01,   0.01,   0.50,   0.01,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_green       = np.array([0.01,   0.05,   0.01,   0.01,   0.25,   0.01,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_cyan        = np.array([0.01,   0.10,   0.10,   0.01,   0.50,   0.50,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_teal        = np.array([0.01,   0.05,   0.05,   0.01,   0.25,   0.25,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_blue        = np.array([0.01,   0.01,   0.10,   0.01,   0.01,   0.50,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_navy        = np.array([0.01,   0.01,   0.05,   0.01,   0.01,   0.25,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_magenta     = np.array([0.10,   0.01,   0.10,   0.50,   0.01,   0.50,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_fuchsia     = np.array([0.10,   0.01,   0.10,   0.50,   0.01,   0.50,   0.50,   0.50,   0.50,   32,     1], dtype='f8')
    plastic_purple      = np.array([0.05,   0.01,   0.05,   0.25,   0.01,   0.25,   0.50,   0.50,   0.50,   32,     1], dtype='f8')

    ####################         Ka R       G       B    Kd R       G       B    Ks R       G       B       Ns      d
    rubber_white        = np.array([0.05,   0.05,   0.05,   0.50,   0.50,   0.50,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_lightgray    = np.array([0.04,   0.04,   0.04,   0.38,   0.38,   0.38,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_gray         = np.array([0.03,   0.03,   0.03,   0.25,   0.25,   0.25,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_darkgray     = np.array([0.02,   0.02,   0.02,   0.13,   0.13,   0.13,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_black        = np.array([0.01,   0.01,   0.01,   0.01,   0.01,   0.01,   0.10,   0.10,   0.10,   10,     1], dtype='f8')

    rubber_red          = np.array([0.05,   0.01,   0.01,   0.50,   0.30,   0.30,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_maroon       = np.array([0.03,   0.01,   0.01,   0.40,   0.30,   0.30,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_yellow       = np.array([0.05,   0.05,   0.01,   0.50,   0.50,   0.30,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_olive        = np.array([0.03,   0.03,   0.01,   0.40,   0.40,   0.30,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_lime         = np.array([0.01,   0.05,   0.01,   0.30,   0.50,   0.30,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_green        = np.array([0.01,   0.03,   0.01,   0.30,   0.40,   0.30,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_cyan         = np.array([0.01,   0.05,   0.05,   0.30,   0.50,   0.50,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_teal         = np.array([0.01,   0.03,   0.03,   0.30,   0.40,   0.40,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_blue         = np.array([0.01,   0.01,   0.05,   0.30,   0.30,   0.50,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_navy         = np.array([0.01,   0.01,   0.03,   0.30,   0.30,   0.40,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_magenta      = np.array([0.05,   0.01,   0.05,   0.50,   0.30,   0.50,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_fuchsia      = np.array([0.05,   0.01,   0.05,   0.50,   0.30,   0.50,   0.10,   0.10,   0.10,   10,     1], dtype='f8')
    rubber_purple       = np.array([0.03,   0.01,   0.03,   0.40,   0.30,   0.40,   0.10,   0.10,   0.10,   10,     1], dtype='f8')

    @staticmethod
    def color_to_matte_material(color):
        return np.concatenate([color[0:3].copy() / 255 / 2, color[0:3].copy() / 255, np.zeros(3), np.array([1]), np.array([color[3].copy() / 255])], axis=0).astype("f8")

    @staticmethod
    def color_to_plastic_material(color):
        return np.concatenate([np.zeros(3), color[0:3].copy() / 255 / 2 + 0.01, color[0:3].copy() / 255 / 5 + 0.5, np.array([32]), np.array([color[3].copy() / 255])], axis=0).astype("f8")

    @staticmethod
    def color_to_rubber_material(color):
        return np.concatenate([color[0:3].copy() / 255 / 20, color[0:3].copy() / 255 / 2 + 0.01, color[0:3].copy() / 255 * 0.7 + 0.01, np.array([10]), np.array([color[3].copy() / 255])], axis=0).astype("f8")


# WorldObject

In [ ]:
class WorldObject:
    """Object in World"""
    # https://yttm-work.jp/model_render/model_render_0001.html
    # https://en.wikipedia.org/wiki/Wavefront_.obj_file

    def __init__(self, pos=np.array([0, 0, 0]), rot=Quaternion.euler_to_quaternion(0, 0, 0), wire_frame=False, enable_shadow=True):
        self.pos = pos
        self.rot = rot
        self.wire_frame = wire_frame
        self.enable_shadow = enable_shadow
        self.vertices = np.array([[]])
        self.arrv = [] # vertices
        self.arrvn = [] # vertex normals
        self.arrtn = [] # face normals
        self.arrtm = [] # face materials

    def triangle_mesh(self, a:int, b:int, c:int, material, normal=None):
        # counter clockwise!
        trinormal = np.cross(self.vertices[c].T - self.vertices[a].T, self.vertices[b].T - self.vertices[a].T).reshape(-1, 1)
        if np.sum(trinormal ** 2) == 0:
            trinormal = np.array([1, 0, 0]).reshape(-1, 1)
        else:
            trinormal = trinormal / np.sqrt(np.sum(trinormal ** 2))
        if normal is None:
            normal = np.concatenate([trinormal.copy(), trinormal.copy(), trinormal.copy()], axis=1)
        self.arrv.append(np.concatenate([self.vertices[a].reshape(-1, 1), self.vertices[b].reshape(-1, 1), self.vertices[c].reshape(-1, 1)], axis=1))
        self.arrvn.append(normal)
        self.arrtn.append(trinormal.reshape(-1))
        self.arrtm.append(material)

    @property
    def v(self):
        return np.array(self.arrv)

    @property
    def vn(self):
        return np.array(self.arrvn)

    @property
    def tn(self):
        return np.array(self.arrtn)

    @property
    def tm(self):
        return np.array(self.arrtm)

    # wire frame objects
    @staticmethod
    def line3D(pos1=np.array([0, 0, 0]), pos2=np.array([0, 0, 0]), material=Material.plastic_lightgray, enable_shadow=True):
        obj = WorldObject(pos, rot, True, enable_shadow)
        obj.vertices = np.concatenate([pos1.reshape(1, -1), pos2.reshape(1, -1)], axis=0)
        obj.triangle_mesh(0, 1, 1, material)
        return obj

    # flat objects
    @staticmethod
    def triangle3D(pos1=np.array([0, 0, 0]), pos2=np.array([0, 0, 0]), pos3=np.array([0, 0, 0]), material=Material.plastic_lightgray, wire_frame=False, enable_shadow=True):
        obj = WorldObject(pos, rot, wire_frame, enable_shadow)
        obj.vertices = np.concatenate([pos1.reshape(1, -1), pos2.reshape(1, -1), pos3.reshape(1, -1)], axis=0)
        obj.triangle_mesh(0, 1, 2, material)
        obj.triangle_mesh(0, 2, 1, material)
        return obj

    @staticmethod
    def rectangle3D(pos=np.array([0, 0, 0]), size=np.array([5, 5, 5]), rot=Quaternion.euler_to_quaternion(0, 0, 0), material=Material.plastic_lightgray, wire_frame=False, enable_shadow=True):
        obj = WorldObject(pos, rot, wire_frame, enable_shadow)
        grid = np.array([[-1, 0, -1], [1, 0, -1], [-1, 0, 1], [1, 0, 1]])
        obj.vertices = np.matmul(grid, np.array([[size[0], 0, 0], [0, size[1], 0], [0, 0, size[2]]]))
        obj.triangle_mesh(2, 0, 3, material)
        obj.triangle_mesh(2, 3, 0, material)
        obj.triangle_mesh(0, 1, 3, material)
        obj.triangle_mesh(0, 3, 1, material)
        return obj

    @staticmethod
    def regular_polygon3D(pos=np.array([0, 0, 0]), size=np.array([10, 10, 10]), rot=Quaternion.euler_to_quaternion(0, 0, 0), material=Material.plastic_lightgray, div=10, wire_frame=False, enable_shadow=True):
        obj = WorldObject(pos, rot, wire_frame, enable_shadow)
        a = 2 * np.pi * np.arange(div).reshape(-1, 1) / div
        grid_outer = np.concatenate([np.cos(a), np.zeros((div, 1)), np.sin(a)], axis=1)
        grid = np.concatenate([np.array([[0, 0, 0]]), grid_outer], axis=0)
        obj.vertices = np.matmul(grid, np.array([[size[0], 0, 0], [0, size[1], 0], [0, 0, size[2]]]))

        for i in range(1, div):
            if wire_frame:
                obj.triangle_mesh(i, i, i + 1, material)
            else:
                obj.triangle_mesh(0, i, i + 1, material)
        if wire_frame:
            obj.triangle_mesh(div, div, 1, material)
        else:
            obj.triangle_mesh(0, div, 1, material)
        return obj

    # volumetric objects
    @staticmethod
    def cube3D(pos=np.array([0, 0, 0]), size=np.array([5, 5, 5]), rot=Quaternion.euler_to_quaternion(0, 0, 0), material=np.tile(Material.plastic_lightgray.reshape(1, -1), (6, 1)), wire_frame=False, enable_shadow=True):
        # https://en.wikipedia.org/wiki/Cube

        obj = WorldObject(pos, rot, wire_frame, enable_shadow)
        grid = np.array([[-1, 1, -1], [1, 1, -1], [-1, 1, 1], [1, 1, 1], [-1, -1, -1], [1, -1, -1], [-1, -1, 1], [1, -1, 1]])
        obj.vertices = np.matmul(grid, np.array([[size[0], 0, 0], [0, size[1], 0], [0, 0, size[2]]]))
        obj.triangle_mesh(0, 4, 1, material[0])
        obj.triangle_mesh(4, 5, 1, material[0])
        obj.triangle_mesh(1, 5, 3, material[1])
        obj.triangle_mesh(5, 7, 3, material[1])
        obj.triangle_mesh(3, 7, 2, material[2])
        obj.triangle_mesh(7, 6, 2, material[2])
        obj.triangle_mesh(2, 6, 0, material[3])
        obj.triangle_mesh(6, 4, 0, material[3])
        obj.triangle_mesh(2, 0, 3, material[4])
        obj.triangle_mesh(0, 1, 3, material[4])
        obj.triangle_mesh(4, 6, 5, material[5])
        obj.triangle_mesh(6, 7, 5, material[5])
        return obj

    @staticmethod
    def tetrahedron3D(pos=np.array([0, 0, 0]), size=np.array([10, 10, 10]), rot=Quaternion.euler_to_quaternion(0, 0, 0), material=np.tile(Material.plastic_lightgray.reshape(1, -1), (4, 1)), wire_frame=False, enable_shadow=True):
        # https://en.wikipedia.org/wiki/Tetrahedron

        obj = WorldObject(pos, rot, wire_frame, enable_shadow)
        a = np.sqrt(3)
        b = np.sqrt(6)
        grid = np.array([[0, b / 3, 0], [0, 0, a / 3], [0.5, 0, -a / 6], [-0.5, 0, -a / 6]])
        obj.vertices = np.matmul(grid, np.array([[size[0], 0, 0], [0, size[1], 0], [0, 0, size[2]]]))
        obj.triangle_mesh(0, 2, 1, material[0])
        obj.triangle_mesh(0, 3, 2, material[1])
        obj.triangle_mesh(0, 1, 3, material[2])
        obj.triangle_mesh(1, 2, 3, material[3])
        return obj

    @staticmethod
    def octahedron3D(pos=np.array([0, 0, 0]), size=np.array([5, 5, 5]), rot=Quaternion.euler_to_quaternion(0, 0, 0), material=np.tile(Material.plastic_lightgray.reshape(1, -1), (8, 1)), wire_frame=False, enable_shadow=True):
        # https://en.wikipedia.org/wiki/Octahedron

        obj = WorldObject(pos, rot, wire_frame, enable_shadow)
        grid = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1], [-1, 0, 0], [0, 0, -1], [0, -1, 0]])
        obj.vertices = np.matmul(grid, np.array([[size[0], 0, 0], [0, size[1], 0], [0, 0, size[2]]]))
        obj.triangle_mesh(0, 1, 2, material[0])
        obj.triangle_mesh(0, 2, 3, material[1])
        obj.triangle_mesh(0, 3, 4, material[2])
        obj.triangle_mesh(0, 4, 1, material[3])
        obj.triangle_mesh(1, 5, 2, material[4])
        obj.triangle_mesh(2, 5, 3, material[5])
        obj.triangle_mesh(3, 5, 4, material[6])
        obj.triangle_mesh(4, 5, 1, material[7])
        return obj

    @staticmethod
    def icosahedron3D(pos=np.array([0, 0, 0]), size=np.array([10, 10, 10]), rot=Quaternion.euler_to_quaternion(0, 0, 0), material=np.tile(Material.plastic_lightgray.reshape(1, -1), (20, 1)), wire_frame=False, enable_shadow=True):
        # https://en.wikipedia.org/wiki/Regular_icosahedron

        obj = WorldObject(pos, rot, wire_frame, enable_shadow)
        s = 1 / np.sqrt(5)
        c = 2 / np.sqrt(5)
        a = np.pi * 2 / 5
        b = np.pi / 5
        grid = np.array([[0, 1, 0], [c * np.cos(a), s, c * np.sin(a)], [c * np.cos(a * 2), s, c * np.sin(a * 2)], [c * np.cos(a * 3), s, c * np.sin(a * 3)], [c * np.cos(a * 4), s, c * np.sin(a * 4)], [c, s, 0], [c * np.cos(b), -s, c * np.sin(b)], [c * np.cos(b + a), -s, c * np.sin(b + a)], [c * np.cos(b + a * 2), -s, c * np.sin(b + a * 2)], [c * np.cos(b + a * 3), -s, c * np.sin(b + a * 3)], [c * np.cos(b + a * 4), -s, c * np.sin(b + a * 4)], [0, -1, 0]])
        obj.vertices = np.matmul(grid, np.array([[size[0], 0, 0], [0, size[1], 0], [0, 0, size[2]]]))
        obj.triangle_mesh(0, 1, 2, material[0])
        obj.triangle_mesh(0, 2, 3, material[1])
        obj.triangle_mesh(0, 3, 4, material[2])
        obj.triangle_mesh(0, 4, 5, material[3])
        obj.triangle_mesh(0, 5, 1, material[4])
        obj.triangle_mesh(1, 6, 7, material[5])
        obj.triangle_mesh(1, 7, 2, material[6])
        obj.triangle_mesh(2, 7, 8, material[7])
        obj.triangle_mesh(2, 8, 3, material[8])
        obj.triangle_mesh(3, 8, 9, material[9])
        obj.triangle_mesh(3, 9, 4, material[10])
        obj.triangle_mesh(4, 9, 10, material[11])
        obj.triangle_mesh(4, 10, 5, material[12])
        obj.triangle_mesh(5, 10, 6, material[13])
        obj.triangle_mesh(5, 6, 1, material[14])
        obj.triangle_mesh(6, 11, 7, material[15])
        obj.triangle_mesh(7, 11, 8, material[16])
        obj.triangle_mesh(8, 11, 9, material[17])
        obj.triangle_mesh(9, 11, 10, material[18])
        obj.triangle_mesh(10, 11, 6, material[19])
        return obj

    @staticmethod
    def prism3D(pos=np.array([0, 0, 0]), size=np.array([10, 10, 10]), rot=Quaternion.euler_to_quaternion(0, 0, 0), material=np.tile(Material.plastic_lightgray.reshape(1, -1), (3, 1)), div=10, wire_frame=False, enable_shadow=True):
        # https://en.wikipedia.org/wiki/Prism_(geometry)
        # https://en.wikipedia.org/wiki/Cylinder

        obj = WorldObject(pos, rot, wire_frame, enable_shadow)
        a = 2 * np.pi * np.arange(div).reshape(-1, 1) / div
        gridtop = np.concatenate([np.cos(a), np.ones((div, 1)), np.sin(a)], axis=1)
        gridbottom = gridtop.copy()
        gridbottom[:, 1] = 0
        grid = np.concatenate([np.array([[0, 1, 0]]), gridtop, np.array([[0, 0, 0]]), gridbottom], axis=0)
        obj.vertices = np.matmul(grid, np.array([[size[0], 0, 0], [0, size[1], 0], [0, 0, size[2]]]))

        for i in range(1, div):
            obj.triangle_mesh(0, i, i + 1, material[0])
        obj.triangle_mesh(0, div, 1, material[0])
        for i in range(div + 2, 2 * div + 1):
            obj.triangle_mesh(div + 1, i + 1, i, material[1])
        obj.triangle_mesh(div + 1, div + 2, 2 * div + 1, material[1])
        for i in range(1, div):
            obj.triangle_mesh(i, div + i + 1, i + 1, material[2])
            obj.triangle_mesh(div + i + 1, div + i + 2, i + 1, material[2])
        obj.triangle_mesh(div, 2 * div + 1, 1, material[2])
        obj.triangle_mesh(2 * div + 1, div + 2, 1, material[2])
        return obj

    @staticmethod
    def pyramid3D(pos=np.array([0, 0, 0]), size=np.array([10, 10, 10]), rot=Quaternion.euler_to_quaternion(0, 0, 0), material=np.tile(Material.plastic_lightgray.reshape(1, -1), (3, 1)), div=10, wire_frame=False, enable_shadow=True):
        # https://en.wikipedia.org/wiki/Pyramid_(geometry)
        # https://en.wikipedia.org/wiki/Cone

        obj = WorldObject(pos, rot, wire_frame, enable_shadow)
        a = 2 * np.pi * np.arange(div).reshape(-1, 1) / div
        gridbottom = np.concatenate([np.cos(a), np.zeros((div, 1)), np.sin(a)], axis=1)
        grid = np.concatenate([np.array([[0, 0, 0]]), np.array([[0, 1, 0]]), gridbottom], axis=0)
        obj.vertices = np.matmul(grid, np.array([[size[0], 0, 0], [0, size[1], 0], [0, 0, size[2]]]))

        for i in range(2, div + 1):
            obj.triangle_mesh(0, i + 1, i, material[0])
        obj.triangle_mesh(0, 2, div + 1, material[0])
        for i in range(2, div + 1):
            obj.triangle_mesh(1, i, i + 1, material[1])
        obj.triangle_mesh(1, div + 1, 2, material[1])
        return obj

    @staticmethod
    def sphere3D(pos=np.array([0, 0, 0]), size=np.array([10, 10, 10]), rot=Quaternion.euler_to_quaternion(0, 0, 0), material=np.tile(Material.plastic_lightgray.reshape(1, -1), (80, 1)), div=1, smooth=True, wire_frame=False, enable_shadow=True):
        # https://en.wikipedia.org/wiki/Sphere

        obj = WorldObject.icosahedron3D(pos, size, rot, wire_frame=wire_frame, enable_shadow=enable_shadow)
        for i in range(div):
            if smooth:
                for trinum in range(len(obj.arrvn)):
                    obj.arrvn[trinum][:, 0] = obj.arrv[trinum][:, 0] / np.sqrt(np.sum(obj.arrv[trinum][:, 0] ** 2))
                    obj.arrvn[trinum][:, 1] = obj.arrv[trinum][:, 1] / np.sqrt(np.sum(obj.arrv[trinum][:, 1] ** 2))
                    obj.arrvn[trinum][:, 2] = obj.arrv[trinum][:, 2] / np.sqrt(np.sum(obj.arrv[trinum][:, 2] ** 2))

            WorldObject.tessellate_double(obj)
            for trinum in range(len(obj.arrvn)):
                obj.arrv[trinum][:, 0] = np.matmul(obj.arrv[trinum][:, 0] / np.sqrt(np.sum(obj.arrv[trinum][:, 0] ** 2)), np.array([[size[0], 0, 0], [0, size[1], 0], [0, 0, size[2]]]))
                obj.arrv[trinum][:, 1] = np.matmul(obj.arrv[trinum][:, 1] / np.sqrt(np.sum(obj.arrv[trinum][:, 1] ** 2)), np.array([[size[0], 0, 0], [0, size[1], 0], [0, 0, size[2]]]))
                obj.arrv[trinum][:, 2] = np.matmul(obj.arrv[trinum][:, 2] / np.sqrt(np.sum(obj.arrv[trinum][:, 2] ** 2)), np.array([[size[0], 0, 0], [0, size[1], 0], [0, 0, size[2]]]))

        findiv = round(20 * np.exp2(2 * div))
        assert(material.shape[0] >= findiv)
        obj.arrtm = material[0:20 * findiv]
        return obj

    @staticmethod
    def tessellate_double(obj, material_copy=False):
        new_arrv = []
        new_arrvn = []
        new_arrtn = []
        if material_copy:
            new_arrtm = []

        for trinum in range(len(obj.arrv)):
            A = obj.arrv[trinum][:, 0].copy().reshape(-1, 1)
            B = obj.arrv[trinum][:, 1].copy().reshape(-1, 1)
            C = obj.arrv[trinum][:, 2].copy().reshape(-1, 1)

            new_arrv.append(np.concatenate([A, (A + B) / 2, (A + C) / 2], axis=1))
            new_arrv.append(np.concatenate([(A + B) / 2, (B + C) / 2, (A + C) / 2], axis=1))
            new_arrv.append(np.concatenate([(A + C) / 2, (B + C) / 2, C], axis=1))
            new_arrv.append(np.concatenate([(A + B) / 2, B, (B + C) / 2], axis=1))

            An = obj.arrvn[trinum][:, 0].copy().reshape(-1, 1)
            Bn = obj.arrvn[trinum][:, 1].copy().reshape(-1, 1)
            Cn = obj.arrvn[trinum][:, 2].copy().reshape(-1, 1)

            new_arrvn.append(np.concatenate([An, (An + Bn) / 2, (An + Cn) / 2], axis=1))
            new_arrvn.append(np.concatenate([(An + Bn) / 2, (Bn + Cn) / 2, (An + Cn) / 2], axis=1))
            new_arrvn.append(np.concatenate([(An + Cn) / 2, (Bn + Cn) / 2, Cn], axis=1))
            new_arrvn.append(np.concatenate([(An + Bn) / 2, Bn, (Bn + Cn) / 2], axis=1))

            new_arrtn.append(((An * 2 + Bn / 2 + Cn / 2) / 3).reshape(-1))
            new_arrtn.append(((An + Bn + Cn) / 3).reshape(-1))
            new_arrtn.append(((An / 2 + Bn / 2 + Cn * 2) / 3).reshape(-1))
            new_arrtn.append(((An / 2 + Bn * 2 + Cn / 2) / 3).reshape(-1))

            if material_copy:
                new_arrtm.append(obj.arrtm[trinum].copy())
                new_arrtm.append(obj.arrtm[trinum].copy())
                new_arrtm.append(obj.arrtm[trinum].copy())
                new_arrtm.append(obj.arrtm[trinum].copy())

        obj.arrv = new_arrv
        obj.arrvn = new_arrvn
        obj.arrtn = new_arrtn
        if material_copy:
            obj.arrtm = new_arrtm


# Light

In [ ]:
class Light:
    """Light in World"""
    # https://www.gabrielgambetta.com/computer-graphics-from-scratch/shading.html
    # https://en.wikipedia.org/wiki/Phong_reflection_model
    # https://ja.wikipedia.org/wiki/Phongの反射モデル
    # http://www.opengl-tutorial.org/intermediate-tutorials/tutorial-16-shadow-mapping/
    # https://en.wikipedia.org/wiki/Computer_graphics_lighting

    def __init__(self, pos=np.array([0, 0, 0]), rot=Quaternion.euler_to_quaternion(0, 0, 0), light_type=1, intensity=0.1, intensity2=0.4, color=Color.white):
        #type = 0: ambient
        #type = 1: point
        #type = 2: directional
        self.pos = pos
        self.rot = rot
        self.light_type = light_type
        self.intensity = intensity
        self.intensity2 = intensity2
        self.color = color

        self.R = self.rot.matrix
        self.Rinv = self.rot.inv.matrix
        self.T = -np.matmul(self.rot.matrix, self.pos.reshape(3, 1))
        self.Tinv = self.pos.reshape(3, 1)

    def set_pos(self, pos):
        self.pos = pos
        self.T = -np.matmul(self.rot.matrix, self.pos.reshape(3, 1))
        self.Tinv = self.pos.reshape(3, 1)

    def set_rot(self, rot):
        self.rot = rot
        self.R = self.rot.matrix
        self.Rinv = self.rot.inv.matrix
        self.T = -np.matmul(self.rot.matrix, self.pos.reshape(3, 1))

    def calc(self, P, N, campos, dist=None, z_buffer=None, c=None, wire_frame=False):
        if self.light_type == 0:
            return np.clip(np.concatenate([np.tile(self.color[0:3].reshape(1, -1), (c.shape[0], 1)) * self.intensity * c[:, 0:3], c[:, 10].reshape(-1, 1) * 255], axis=1), 0, 255).astype('u1')
        elif self.light_type == 1 or self.light_type == 2:
            if self.light_type == 1:
                L = np.tile(self.pos.reshape(3, 1), (1, P.shape[1])) - P # column vector
            else:
                L = -np.matmul(self.Rinv, np.tile(np.array([0, 0, 1]).reshape(3, 1), (1, P.shape[1]))) # column vector
            L = L / np.sqrt(np.tile(np.sum(L ** 2, axis=0).reshape(1, -1), (3, 1))) # column vector
            if wire_frame:
                N = L.T # row vector
            else:
                N = N / np.sqrt(np.tile(np.sum(N ** 2, axis=1).reshape(-1, 1), (1, 3))) # row vector

            l_n = np.diag(np.matmul(N, L))
            lnfilter = l_n > 0
            l_n = l_n[lnfilter]
            if l_n.size > 0:
                dist = dist[lnfilter]
                z_buffer = z_buffer[lnfilter]
                distfilter = dist >= z_buffer - Settings.EPSILON * np.sqrt(1 / l_n / l_n - 1)
                dist = dist[distfilter]
                if dist.size > 0:
                    l_n = l_n[distfilter]
                    lnfilter[lnfilter] = distfilter
                    P = P[:, lnfilter]
                    N = N[lnfilter]
                    L = L[:, lnfilter]
                    c = c[lnfilter]

                    V = np.tile(campos.reshape(3, 1), (1, P.shape[1])) - P # column vector
                    R = 2 * np.tile(l_n.reshape(-1, 1), (1, 3)) * N - L.T # row vector
                    R = R / np.sqrt(np.tile(np.sum(R ** 2, axis=1).reshape(-1, 1), (1, 3)))
                    V = V / np.sqrt(np.tile(np.sum(V ** 2, axis=0).reshape(1, -1), (3, 1)))
                    r_v = np.diag(np.matmul(R, V))
                    
                    ret = np.zeros((lnfilter.size, 4))
                    ret[lnfilter] = np.clip(np.concatenate([np.tile(self.color[0:3].reshape(1, -1), (c.shape[0], 1)) * (self.intensity * c[:, 3:6] * np.tile(l_n.reshape(-1, 1), (1, 3)) + self.intensity2 * c[:, 6:9] * np.tile(np.power(r_v, c[:, 9]).reshape(-1, 1), (1, 3))), c[:, 10].reshape(-1, 1) * 255], axis=1), 0, 255)
                    return ret
                else:
                    return np.zeros((c.shape[0], 4))
            else:
                return np.zeros((c.shape[0], 4))
        else:
            return np.zeros((c.shape[0], 4))


# Camera

In [ ]:
class Camera:
    """Camera in 3D world"""

    def __init__(self, pos=np.array([0, 0, 0]), rot=Quaternion.euler_to_quaternion(0, 0, 0), ortho=False):
        self.pos = pos
        self.rot = rot
        self.ortho = ortho


# World

In [ ]:
class World:
    """3D world for placing polygons and lights"""

    def __init__(self, name="Default World"):
        self.name = name
        self.objects = []
        self.lights = []


# Main

In [ ]:
# https://www.ngdc.noaa.gov/mgg/image/images/etopo2icosahedron.pdf
# https://solarviews.com/cap/ico/icoearth.htm
# https://ja.wikipedia.org/wiki/軌道要素
# https://ja.wikipedia.org/wiki/太陽
# https://ja.wikipedia.org/wiki/地球
# https://ja.wikipedia.org/wiki/月
# https://ja.wikipedia.org/wiki/ケプラーの方程式
# https://en.wikipedia.org/wiki/Kepler's_equation
# http://fnorio.com/0158Kepler_equation/Kepler_equation.html

with warnings.catch_warnings():
    warnings.simplefilter("error")

    print("Initializing...")

    world = World()
    canvas = Canvas(Settings.WIDTH, Settings.HEIGHT)
    cam = Camera(pos=np.array([0, 0, -30,]))

    imgs = []
    shadows_list = []
    fig, ax = plt.subplots()

    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
    ax.axis('off')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)


    # Materials
    M = Material()
    wh = M.plastic_white
    lg = M.plastic_lightgray
    ye = M.rubber_yellow
    ol = M.rubber_olive
    gr = M.rubber_green
    cy = M.plastic_cyan
    bl = M.plastic_blue
    bw = M.color_to_rubber_material(Color.RGB_to_color(153, 76, 0))

    sun_material = np.tile(np.array([1000,    1000,    0,      0,      0,      0,      0,      0,      0,      1,      1], dtype='f8').reshape(1, -1), (80, 1))
    earth_material = np.stack([
    # 0 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        cy, cy, cy, cy, wh, wh, wh, bl, wh, wh, wh, wh, wh, wh, wh, wh,     wh, wh, bl, wh, bl, wh, bl, wh, bl, bl, wh, bl, bl, wh, wh, bl,     wh, wh, wh, bl, bl, bl, wh, bl, wh, wh, bl, bl, bl, bl, bl, bl,     bl, bl, wh, bl, wh, wh, wh, wh, wh, bl, wh, bl, wh, wh, wh, wh,
        wh, wh, wh, gr, gr, gr, bl, gr, bl, bl, wh, bl, gr, gr, gr, gr,     gr, gr, gr, gr, gr, gr, gr, wh, wh, wh, wh, wh, gr, gr, gr, bl,     bl, wh, bl, wh, wh, bl, bl, bl, bl, bl, bl, bl, gr, bl, bl, bl,     bl, gr, gr, gr, gr, gr, gr, bl, gr, gr, gr, bl, gr, gr, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, gr, bl, gr,     gr, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        gr, gr, gr, gr, gr, gr, bl, gr, gr, gr, bl, gr, gr, gr, gr, gr,     ol, ol, gr, gr, gr, gr, gr, gr, gr, gr, bl, gr, gr, gr, gr, gr,     gr, gr, gr, gr, gr, ol, gr, gr, gr, bl, bl, bl, gr, gr, gr, gr,     ol, ol, ol, ol, ol, gr, gr, gr, gr, gr, gr, gr, ol, ol, ol, ol,
    # 1 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        cy, cy, cy, cy, wh, bl, cy, bl, wh, wh, bl, bl, wh, wh, bl, wh,     wh, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, wh, gr, wh, wh, bl, gr, bl, gr, gr, gr, bl, wh, wh, gr, wh,     wh, wh, bl, wh, bl, wh, wh, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, gr, bl, bl, gr, bl, bl, bl, bl, bl,     bl, bl, bl, bl, gr, bl, gr, bl, gr, bl, bl, bl, gr, gr, gr, wh,     bl, bl, gr, gr, gr, bl, gr, gr, bl, gr, gr, gr, gr, gr, gr, gr,     bl, gr, bl, ol, gr, ol, gr, bl, gr, gr, gr, bl, bl, ol, gr, ol,
        gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr,     bl, bl, gr, gr, bl, gr, gr, bl, gr, gr, bl, gr, bl, bl, bl, bl,     bl, ol, ol, gr, bl, bl, gr, bl, bl, bl, bl, bl, bl, ye, ye, bl,     gr, bl, bl, bl, gr, ol, bl, ye, bl, ye, ye, ye, ol, ye, ye, ye,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, gr, bl, bl, bl, bl, bl, gr, ol, ye, gr, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
    # 2 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        cy, cy, cy, cy, bl, bl, bl, bl, wh, bl, wh, bl, bl, bl, wh, bl,     bl, wh, bl, bl, wh, wh, wh, wh, bl, wh, wh, wh, wh, gr, wh, gr,     wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, gr, wh, wh, gr, gr, gr,     bl, bl, bl, bl, bl, gr, wh, gr, wh, gr, gr, gr, bl, gr, gr, gr,
        gr, wh, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr,     ol, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, ye, gr, gr, gr,     gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, wh, wh, gr,     ye, gr, ye, gr, wh, gr, gr, wh, gr, gr, gr, ye, gr, wh, wh, gr,
        gr, gr, gr, wh, gr, gr, gr, ye, gr, gr, ye, ye, ye, ye, ye, ye,     gr, bw, bw, ye, ye, ye, bw, ye, bw, bw, ye, ye, wh, wh, ye, ol,     bw, ye, ye, ye, ye, ol, bw, ol, gr, gr, gr, gr, ye, wh, ol, ol,     gr, ye, ye, ye, wh, wh, wh, bw, wh, ol, bw, bw, wh, wh, wh, ol,
        gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, bl, gr, gr, gr, bl,     gr, wh, gr, gr, gr, bl, bl, bl, bl, ye, ye, ye, ol, ye, ye, bw,     bw, ol, bw, bw, ol, gr, gr, ye, wh, ol, ol, ye, ye, ye, bw, ye,     bl, bw, gr, ol, ye, ye, gr, ye, gr, ol, ol, ye, ye, ye, ye, ye,
    # 3 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        cy, cy, cy, cy, cy, cy, cy, wh, cy, wh, wh, wh, cy, wh, wh, bl,     bl, bl, bl, wh, wh, wh, bl, wh, bl, bl, wh, wh, wh, wh, wh, wh,     wh, bl, bl, wh, wh, wh, wh, wh, wh, wh, bl, wh, wh, wh, bl, wh,     wh, wh, wh, wh, wh, wh, wh, gr, wh, gr, wh, gr, wh, gr, wh, gr,
        wh, wh, wh, gr, ol, ol, wh, wh, gr, wh, wh, bl, ol, gr, wh, gr,     ol, wh, wh, bl, gr, gr, gr, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, wh, wh, bl, bl, bl, bl, bl, bl, bl, bl, bl,     gr, bl, bl, bl, gr, bl, gr, bl, bl, bl, bl, bl, gr, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        wh, wh, wh, gr, gr, ol, gr, ol, gr, bw, ol, gr, gr, ye, ye, ye,     ye, ol, ol, gr, bl, bl, ol, gr, bw, gr, gr, gr, gr, bl, gr, bl,     gr, bl, bl, bl, bl, gr, gr, bl, gr, bl, bl, bl, bl, gr, bl, bl,     ye, ye, ol, ye, gr, gr, gr, gr, gr, gr, bl, gr, gr, ol, gr, gr,
    # 4 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        cy, cy, cy, cy, cy, cy, cy, wh, cy, bl, wh, cy, cy, wh, wh, wh,     bl, bl, bl, wh, wh, wh, bl, wh, bl, bl, wh, bl, wh, wh, wh, wh,     wh, bl, wh, wh, wh, wh, wh, wh, wh, wh, wh, gr, wh, gr, wh, wh,     bl, bl, wh, bl, wh, bl, wh, bl, wh, gr, wh, wh, bl, bl, bl, bl,
        bl, bl, bl, bl, gr, gr, bl, bl, gr, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, wh, bl, bl, wh, wh, gr, gr, gr, gr, gr, bl, bl, gr, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        gr, gr, gr, gr, bw, bw, gr, gr, ol, ol, gr, gr, wh, bl, gr, bl,     bl, bl, gr, bl, bl, gr, gr, gr, gr, gr, gr, bw, bl, bl, gr, bl,     gr, bw, bw, bw, bw, bw, bw, bw, gr, bw, ol, bw, gr, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
    # 5 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        gr, gr, gr, bl, bl, bl, bl, bl, gr, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
    # 6 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        bw, bw, gr, gr, bw, gr, gr, bl, gr, bl, bl, bl, ol, gr, bl, gr,     gr, gr, bl, gr, gr, gr, bl, gr, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, gr, bl, bl, bl, gr, bl, bl, bl, bl, bl, gr, bl, gr, bl,     bl, bl, bl, bl, bl, bl, bl, bl, gr, gr, gr, gr, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, gr, bl, bl, gr, gr, gr, gr, gr,     gr, gr, gr, gr, gr, gr, bl, gr, bl, bl, bl, bl, gr, bl, bl, bl,     bl, gr, gr, bl, gr, bl, bl, bl, bl, bl, bl, bl, bl, gr, bl, bl,     gr, gr, gr, ol, gr, gr, gr, gr, bw, bw, bl, gr, gr, gr, gr, gr,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, gr, bl, gr,
        bl, bl, bl, bl, bl, gr, bl, gr, bw, gr, gr, gr, bl, bl, gr, bl,     bl, bw, bw, bw, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr,     gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr,     bl, bl, bw, bl, bl, bl, bw, bw, bw, bw, gr, bw, bl, bl, bw, bl,
    # 7 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, gr,
        gr, gr, gr, gr, gr, gr, gr, gr, bl, bl, bl, gr, gr, gr, gr, gr,     gr, gr, gr, bl, bl, gr, gr, bl, gr, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     gr, gr, bl, gr, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr,     gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr,     gr, gr, gr, gr, gr, bl, bl, bl, bl, bl, bl, bl, gr, gr, bl, gr,     gr, gr, gr, bw, bw, gr, gr, gr, gr, gr, gr, gr, bw, bw, gr, bw,
    # 8 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, ye, bl, bl, bl, ye, ye, ye, ye, ye,     bl, bl, bl, ye, ye, ye, ye, ye, bw, ye, ye, ye, ye, ye, ye, ye,     bl, bl, bl, bl, bl, bl, bl, bl, ye, bw, bw, gr, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, gr, gr, gr, gr, gr, bw, gr, bl, bl, gr, bl,     bl, bl, gr, bl, gr, gr, gr, ye, gr, bw, bw, ye, bw, ye, ye, ye,     ye, ye, ye, ye, ye, ye, ye, ye, ye, ye, ye, ye, ye, ye, ye, ye,     bl, bl, bl, bl, bl, gr, gr, gr, gr, bw, ye, bw, gr, gr, gr, gr,
        ye, ye, ye, bw, bw, bw, ye, bw, ye, ye, ye, ye, ye, ye, ye, bw,     ye, ye, ye, ye, ye, bw, ye, bw, ye, ye, bw, ye, ye, ye, ye, ye,     ye, ye, ye, ye, ye, ye, ye, ye, ye, ol, ye, ye, ye, ye, ye, ye,     ye, bl, ye, ol, ol, bw, ye, bw, ye, ye, ye, bw, gr, gr, gr, gr,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, gr, gr, gr, gr, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
    # 9 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        bl, bl, bl, ol, ol, bw, bl, bw, bl, bl, bl, bw, ol, ol, ol, ye,     ye, ol, bw, bw, bw, gr, bw, gr, bw, bw, bl, bw, gr, gr, gr, bw,     bw, bw, ye, gr, ye, ye, ol, gr, bl, bl, bl, bl, gr, gr, ol, bw,     ye, bw, bw, bw, gr, gr, gr, gr, bw, gr, gr, gr, gr, gr, gr, gr,
        gr, gr, gr, gr, gr, gr, gr, bl, gr, gr, bl, gr, gr, gr, gr, gr,     gr, gr, gr, bl, gr, gr, gr, gr, gr, gr, gr, gr, gr, bl, bl, bl,     gr, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     gr, gr, gr, ol, gr, bl, gr, bl, gr, bl, bl, bl, gr, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     gr, gr, bl, gr, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     gr, gr, gr, bl, bl, gr, gr, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, gr, bl, bl, gr, bl,     bl, bw, gr, bw, bw, ol, gr, ol, gr, gr, gr, ol, ye, bw, ol, bw,     gr, gr, ol, ol, ol, ol, gr, ol, gr, gr, bl, gr, ol, ol, gr, bw,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
    #10 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        ye, ye, ye, bw, bw, ye, ye, ye, ye, bl, ol, ye, bw, bw, ye, bw,     bw, ye, ye, ye, ye, ye, ye, ye, ye, bl, ol, bw, bl, bl, ye, bl,     bw, bw, bw, bl, bl, bl, bw, bl, bw, bw, ol, ol, bl, bl, bl, bl,     bw, bl, bl, ye, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, gr, gr, gr, ol,     bl, bl, bl, bl, bl, bl, bw, bw, ye, ye, ye, ol, gr, gr, bw, ol,     bl, bl, bl, bl, bl, gr, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        ye, ye, ol, ol, ol, bw, ol, bw, wh, wh, wh, bw, bw, gr, bw, ol,     bl, bl, gr, bl, bl, bl, gr, gr, gr, ol, wh, gr, bl, gr, gr, bl,     wh, gr, wh, gr, gr, gr, wh, ol, ol, ol, ol, ol, gr, gr, gr, gr,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
    #11 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        gr, gr, bw, gr, gr, bl, bw, bl, bl, gr, bl, bl, gr, ol, gr, ol,     ol, ol, gr, bl, gr, gr, gr, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, gr, bl, gr, bl, bl, gr, bl, bl, gr, gr, gr,     gr, gr, bl, bl, bl, bl, gr, gr, gr, gr, bl, gr, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, gr, gr, bl, gr, bl, bl, bl, bl,     bl, bl, bl, bl, bl, gr, gr, gr, gr, bl, bl, bl, bl, bl, gr, bl,     gr, gr, gr, bl, bl, bl, bl, bl, bl, gr, bl, gr, gr, gr, gr, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, gr, gr, bl, bl, bl, bl, gr, bl, gr, bl, bl, bl,     bl, bl, bl, ye, ye, ol, gr, ye, gr, ye, gr, ye, bw, bw, ol, gr,     gr, ol, ol, ol, ye, ye, ol, ye, ye, ye, ye, ye, ye, gr, ye, gr,     bl, bl, ye, bl, ol, ye, gr, ol, ye, gr, ol, gr, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
    #12 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        gr, ol, gr, ol, ol, ol, ol, bl, gr, bl, bl, bl, bl, bl, bl, bl,     bl, gr, bl, bl, bl, bl, bl, bl, gr, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, gr, gr, bl, gr, bl, gr, bl, bl, bl, bl, bl, gr, bl, bl, bl,
        gr, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, gr, gr, bl, gr, bl, bl, bl, gr, bl, gr, bl, bl, bl, gr,     bl, bl, bl, gr, bl, bl, gr, gr, gr, gr, bl, gr, bl, bl, gr, bl,     bl, bl, bl, bl, gr, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, gr, ol, ye, gr, gr, gr, bl, bl, bl, bl, ye, ol, gr, ol,
    #13 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, gr, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, gr, gr, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, gr, bl, bl, gr, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, gr, gr, bl, gr, bl, bl, bl, bl, gr, ol, ol, ol,
    #14 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, gr, bl, bl, bl, bl, bl, bl, gr, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, gr, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
    #15 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, wh, wh, wh, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, wh, wh, bl, wh,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh,
    #16 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        bw, bw, gr, bw, bw, gr, gr, gr, gr, gr, gr, gr, bw, bw, gr, bw,     bw, bl, gr, bl, bl, bl, gr, bl, gr, bl, gr, bl, bl, bl, bl, bl,     gr, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     wh, wh, bl, wh, wh, bl, bl, gr, bl, bl, bl, bl, wh, wh, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, wh, wh, bl, bl, bl, bl, bl, bl, wh, wh, bl, wh,     wh, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, wh,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, wh,     wh, wh, bl, wh, bl, bl, bl, wh, bl, wh, wh, wh, wh, wh, wh, wh,
    #17 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bw, bw, gr, bl, bl, bl, gr, bl, gr, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, wh,     wh, wh, bl, wh, wh, bl, bl, wh, bl, bl, bl, bl, wh, wh, wh, wh,     bl, bl, bl, bl, wh, bl, bl, bl, bl, bl, bl, bl, wh, wh, wh, wh,     wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh,
    #18 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, gr, gr, bl, gr, gr, gr, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, ol, bl, bl, bl, bl, bl,     ol, bl, gr, bl, bl, gr, gr, gr, bw, bw, ol, bw, bl, bl, gr, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, wh, wh, wh,     wh, wh, wh, wh, wh, wh, wh, wh, bl, bl, bl, bl, wh, wh, wh, wh,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, wh, wh, wh, wh,     wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh, wh,
    #19 0               4               8               12                  16              20              24              28                  32              36              40              44                  48              52              56              60
        ol, gr, gr, gr, gr, gr, gr, bl, bl, bl, bl, bl, gr, bl, bl, gr,     gr, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, gr, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, gr, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, gr, bl, bl, bl, bl,     gr, gr, bl, bl, bl, bl, gr, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,
        bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, wh, wh, bl, wh,     wh, wh, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl, bl,     wh, wh, bl, wh, wh, bl, bl, bl, bl, wh, bl, bl, wh, wh, bl, wh,
        ])
    sun_material = np.tile(np.array([1000,    1000,    0,      0,      0,      0,      0,      0,      0,      1,      1], dtype='f8').reshape(1, -1), (80, 1))
    moon_material = np.tile(np.array([0.10,   0.10,   0.14,   0.38,   0.38,   0.38,   0.65,   0.65,   0.65,   10,     1], dtype='f8').reshape(1, -1), (80, 1))

    # Objects
    # size
    #  units           km
    #     10     =      6378    (Earth equator radius)
    #      1     =       637.8
    #      9.966 =      6356    (Earth polar radius)
    #   1091     =    696000    (Sun equator radius)
    #      5.450 =      3475.8  (Moon equator radius)
    #      5.443 =      3471.3  (Moon polar radius)
    # 230636     = 147100000    (Earth periapsis radius) → 23063.6
    # 238476     = 152100000    (Earth apoapsis radius) → 23847.6
    #  569.6     =    363304    (Moon periapsis radius) → 56.96
    #  635.8     =    405495    (Moon apoapsis radius) → 63.58

    # ccentricity
    # Earth         0.0167
    # Moon          0.0549

    Sun = WorldObject.sphere3D(pos=np.array([0, 0, 23847.6]), size=np.array([1091, 1091, 1091]), material=sun_material, enable_shadow=False)
    Earth_base = WorldObject.sphere3D(pos=np.array([0, 0, 0]), size=np.array([10, 9.966, 10]), rot=Quaternion.euler_to_quaternion(0, 0, np.pi * 23.4 / 180), material=earth_material, div=4)
    Moon = WorldObject.sphere3D(pos=np.array([0, 0, -60]), size=np.array([5.450, 5.443, 5.450]), material=moon_material)

    # Lights
    ambientlight = Light(light_type=0, intensity=1)
    directionallight = Light(pos=np.array([0, 0, 23847.6]), rot=Quaternion.euler_to_quaternion(0, 0, 0), light_type=2, intensity=3, intensity2=3)
    #pointlight1 = Light(pos=np.array([0, 0, 23847.6]), rot=Quaternion.euler_to_quaternion(0, 0, 0), light_type=1, intensity=3, intensity2=3)
    #pointlight2 = Light(pos=np.array([0, 0, 23847.6]), rot=Quaternion.euler_to_quaternion(0, np.pi / 2, 0), light_type=1, intensity=3, intensity2=3)
    #pointlight3 = Light(pos=np.array([0, 0, 23847.6]), rot=Quaternion.euler_to_quaternion(0, np.pi, 0), light_type=1, intensity=3, intensity2=3)
    #pointlight4 = Light(pos=np.array([0, 0, 23847.6]), rot=Quaternion.euler_to_quaternion(0, -np.pi / 2, 0), light_type=1, intensity=3, intensity2=3)
    #pointlight5 = Light(pos=np.array([0, 0, 23847.6]), rot=Quaternion.euler_to_quaternion(0, 0, np.pi / 2), light_type=1, intensity=3, intensity2=3)
    #pointlight6 = Light(pos=np.array([0, 0, 23847.6]), rot=Quaternion.euler_to_quaternion(0, 0, -np.pi / 2), light_type=1, intensity=3, intensity2=3)



    max_time = 64 # 64


    # instantiation
    def instantiate(obj):
        if isinstance(obj, WorldObject):
            world.objects.append(obj)
        elif isinstance(obj, Light):
            world.lights.append(obj)

    # Lights
    instantiate(ambientlight)
    instantiate(directionallight)
    #instantiate(pointlight1)
    #instantiate(pointlight2)
    #instantiate(pointlight3)
    #instantiate(pointlight4)
    #instantiate(pointlight5)
    #instantiate(pointlight6)

    # Objects
    instantiate(Sun)
    instantiate(Earth_base)
    instantiate(Moon)


    moon_c = np.cos(np.pi * 5.1 / 180)
    moon_s = np.sin(np.pi * 5.1 / 180)

    def reverse_kepler(nt, e):
        return nt + (np.sin(nt)) * e + (np.sin(2 * nt) / 2) * (e ** 2) + (-np.sin(nt) / 8 + np.sin(3 * nt) * 3 / 8) * (e ** 3) + (-np.sin(2 * nt) / 6 + np.sin(4 * nt) / 3) * (e ** 4)


    bg_W = np.round(Settings.WIDTH * 2 * np.pi / np.arctan(16 / 9)).astype('i8')
    bg_H = Settings.HEIGHT

    star_bg = Layer(bg_W, bg_H, Settings.FOCAL, Settings.C_U, Settings.C_V, "star background", opaque=True, layer_type=0)

    star_num = 10000
    radT = np.random.rand(star_num) * 2 * np.pi
    rangeS = np.random.rand(star_num) * np.pi
    radS = (np.random.rand(star_num) * 2 - 1) * rangeS

    star_v = 1000000 * np.stack([np.sin(radS) * np.cos(radT), np.cos(radS), np.sin(radS) * np.sin(radT)], axis=1)

    main_render_enabled = False


    print("Initializing finished")


    def main_loop(sec):
        global main_render_enabled
        # -3.0 ~ -2.5 : black screen
        # -2.5 ~ -2.0 : title appears
        # -2.0 ~  0.0 : title hold
        #  0.0        : render start
        #  0.0 ~  0.5 : title / black screen disappears
        #second = sec - 3
        second = sec - 3

        sun_u = reverse_kepler(2 * np.pi * second / 60, 0.0167)
        sun_pos = np.array([-23063.6 * np.sin(sun_u), 0, 23847.6 * (np.cos(sun_u) - 0.0167)])
        Sun.pos = sun_pos
        sunrotdir = -sun_pos / np.sqrt(np.sum(sun_pos ** 2))

        if sunrotdir[0] > 0:
            sun_rot = 2 * np.pi - np.arccos(sunrotdir[2])
        elif sunrotdir[0] == 0:
            if sunrotdir[2] > 0:
                sun_rot = 0
            else:
                sun_rot = np.pi
        else:
            sun_rot = np.arccos(sunrotdir[2])
        directionallight.set_pos(sun_pos / 200)
        directionallight.set_rot(Quaternion.euler_to_quaternion(0, sun_rot, 0))

        Earth_base.rot = Quaternion.euler_to_quaternion(0, -2 * np.pi * second / 2, np.pi * 23.4 / 180)

        moon_u = reverse_kepler(2 * np.pi * second / 6, 0.0549)
        moon_mainc = 63.58 * (np.cos(moon_u) - 0.0549)
        Moon.pos = np.array([-56.96 * np.sin(moon_u), moon_mainc * moon_s, moon_mainc * moon_c])


        # numbers computed in order of pitch, yaw, roll, written in roll, yaw, pitch
        if second <= 10:
            #cam.pos = directionallight.pos
            #cam.rot = directionallight.rot
            pass
        elif second <= 40:
            camdeg = -np.pi / 6 * np.sin(2 * np.pi * (second - 10) / 30)
            if second <= 12:
                camdeg *= (second - 10) / 2
            elif 38 < second:
                camdeg *= (40 - second) / 2
            cam.rot = Quaternion.euler_to_quaternion(0, camdeg, 0)
            cam.pos = np.array([30 * np.sin(camdeg), 0, -30 * np.cos(camdeg)])
            #cam.pos = directionallight.pos
            #cam.rot = directionallight.rot
        elif second <= 50:
            t = (-np.cos(np.pi * (second - 40) / 10) + 1) / 2
            cam.rot = Quaternion.euler_to_quaternion(-np.pi / 12 * t, 0, 0)
            cam.pos = np.array([0, 0, -30,]) * (1 - t) + np.array([0, 30, -111.96]) * t
        else:
            cam.rot = Quaternion.euler_to_quaternion(-np.pi / 12, 0, 0)
            cam.pos = np.array([0, 30, -111.96])

        
        R = cam.rot.matrix
        T = -np.matmul(cam.rot.matrix, cam.pos.reshape(3, 1))
        new_star_v = ShaderVertex.transform_world_to_camera(R, T, star_v)
        star_depth_filter = 10 <= new_star_v[2, :]
        new_star_v, _ = ShaderVertex.transform_camera_to_canvas(Settings.FOCAL, Settings.C_U, Settings.C_V, new_star_v[:, star_depth_filter], False)
        star_side_filter = np.logical_and(np.logical_and(0 <= new_star_v[0], new_star_v[0] <= Settings.WIDTH), np.logical_and(0 <= new_star_v[1], new_star_v[1] <= Settings.HEIGHT))
        new_star_v = new_star_v[:, star_side_filter].astype('i8')
        canvas.bg_layer.img = np.tile(np.array([0, 0, 0, 255], dtype='u1').reshape(1, 1, 4), (Settings.WIDTH, Settings.HEIGHT, 1))
        canvas.bg_layer.img[new_star_v[0], new_star_v[1], :] = 255


        if second <= -2.5:
            canvas.front_layer.img = np.tile(np.array([0, 0, 0, 255], dtype='u1').reshape(1, 1, 4), (canvas.W, canvas.H, 1))
        elif second <= -2:
            canvas.front_layer.img = np.tile(np.array([0, 0, 0, 255], dtype='u1').reshape(1, 1, 4), (canvas.W, canvas.H, 1))
            Text2D.text(canvas.front_layer, 320, 120, "地球", Color.RGB_to_color(255, 255, 255, 255 * (2.5 + second) * 2), size=4, text_align='c')
            Text2D.text(canvas.front_layer, 320, 160, "(と太陽と月)", Color.RGB_to_color(255, 255, 255, 255 * (2.5 + second) * 2), size=2, text_align='c')
            Text2D.text(canvas.front_layer, 320, 260, "Земля", Color.RGB_to_color(255, 255, 255, 255 * (2.5 + second) * 2), size=4, text_align='c')
            Text2D.text(canvas.front_layer, 320, 300, "(и Солнце, и Луна)", Color.RGB_to_color(255, 255, 255, 255 * (2.5 + second) * 2), size=2, text_align='c')
        elif second <= 0:
            canvas.front_layer.img = np.tile(np.array([0, 0, 0, 255], dtype='u1').reshape(1, 1, 4), (canvas.W, canvas.H, 1))
            Text2D.text(canvas.front_layer, 320, 120, "地球", Color.RGB_to_color(255, 255, 255, 255), size=4, text_align='c')
            Text2D.text(canvas.front_layer, 320, 160, "(と太陽と月)", Color.RGB_to_color(255, 255, 255, 255), size=2, text_align='c')
            Text2D.text(canvas.front_layer, 320, 260, "Земля", Color.RGB_to_color(255, 255, 255, 255), size=4, text_align='c')
            Text2D.text(canvas.front_layer, 320, 300, "(и Солнце, и Луна)", Color.RGB_to_color(255, 255, 255, 255), size=2, text_align='c')
        elif second <= 0.5:
            main_render_enabled = True
            canvas.front_layer.img = np.tile(np.array([0, 0, 0, 255], dtype='u1').reshape(1, 1, 4), (canvas.W, canvas.H, 1))
            Text2D.text(canvas.front_layer, 320, 120, "地球", Color.RGB_to_color(255, 255, 255, 255), size=4, text_align='c')
            Text2D.text(canvas.front_layer, 320, 160, "(と太陽と月)", Color.RGB_to_color(255, 255, 255, 255), size=2, text_align='c')
            Text2D.text(canvas.front_layer, 320, 260, "Земля", Color.RGB_to_color(255, 255, 255, 255), size=4, text_align='c')
            Text2D.text(canvas.front_layer, 320, 300, "(и Солнце, и Луна)", Color.RGB_to_color(255, 255, 255, 255), size=2, text_align='c')
            canvas.front_layer.img[:, :, 3] = round(255 - second * 510)
        elif second <= 60:
            canvas.front_layer.img = np.zeros((canvas.W, canvas.H, 4), dtype='u1')
        else:
            canvas.front_layer.img = np.tile(np.array([0, 0, 0, 255], dtype='u1').reshape(1, 1, 4), (canvas.W, canvas.H, 1))
            canvas.front_layer.img[:, :, 3] = round((second - 60) * 255)

        return

    print("Rendering...")

    max_frame = round(max_time * Settings.FPS)

    for i in range(max_frame):
        print("Frame {:5d} / {:5d} ... ".format(i + 1, max_frame), end='')
        time_sta = time.perf_counter()
        canvas.clear_layers()

        main_loop(i / Settings.FPS)

        if main_render_enabled:
            shadows, time_ini, time_sha, time_end = GraphicsPipeline.run(canvas, cam, world)
            shadows_list.append(shadows)
        else:
            time_ini = time.perf_counter()
            time_sha = time.perf_counter()
            time_end = time.perf_counter()

        res = AntiAliasing.SSAA_grid(canvas.output_mixed_layers(), Settings.SSAA_RATIO).transpose(1, 0, 2)

        print("init: {:12.9f}, shad: {:12.9f}, main: {:12.9f}".format(time_ini - time_sta, time_sha - time_ini, time_end - time_sha))

        im = ax.imshow(res, animated=True, aspect='auto')
        if i == 0:
            ax.imshow(res)
        imgs.append([im])

    print("Rendering finished")


# Animation

In [ ]:
# https://stackoverflow.com/questions/25333732/matplotlib-animation-not-working-in-ipython-notebook-blank-plot/30845792
# https://ossyaritoori.hatenablog.com/entry/2018/12/01/matplotlib%E3%81%A7%E3%82%A2%E3%83%8B%E3%83%A1%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3%E3%82%92%E4%BD%9C%E6%88%90%EF%BC%8C%E4%BF%9D%E5%AD%98

ani = animation.ArtistAnimation(fig, imgs, interval=1000 / Settings.FPS, repeat=True, blit=True)
ani


In [ ]:
ani.save("test2.gif", writer='imagemagick')
ani.save('test.mp4', writer="ffmpeg")


In [ ]:
#plt.imshow(shadows_list[29][1].z_buffer.T)